# Hccb Warehouse Location


In [1]:
import pandas as pd
from pulp import *
import time
import pandas as pd
import plotly as py

In [2]:
scenario_name = 'Warehouses location' 


### optimal number of warehouse we need to obtaion

In [3]:
number_of_whs = 8 # that should be placed in East Region

### Percentage of demands met in different band zone

In [4]:
distance_band_1 = 50 
distance_band_2 = 100
distance_band_3 = 150
distance_band_4 = 200


### Potential warehouse that can be selected and what is their status

In [5]:
#This is where we can change the status of a warehouse
# meaning we can force it to be open or force it closed.
#(0,1 to let the solver pick, 1,1 to force it to be used, or 0,0 to force it closed.)


wh_status = pd.read_excel('wareh.xlsx', engine = 'openpyxl')
wh_status = wh_status.to_dict();
wh_status

{1: {0: 'Bihar', 1: 0, 2: 1},
 2: {0: 'Bihar', 1: 0, 2: 1},
 3: {0: 'North East', 1: 0, 2: 1},
 4: {0: 'Odisha', 1: 0, 2: 1},
 5: {0: 'Bihar', 1: 0, 2: 1},
 6: {0: 'Bihar', 1: 0, 2: 1},
 7: {0: 'Bihar', 1: 0, 2: 1},
 8: {0: 'North East', 1: 0, 2: 1},
 9: {0: 'Siliguri', 1: 0, 2: 1},
 10: {0: 'Bihar', 1: 0, 2: 1},
 11: {0: 'Bihar', 1: 0, 2: 1},
 12: {0: 'Odisha', 1: 0, 2: 1},
 13: {0: 'Jharkhand', 1: 0, 2: 1},
 14: {0: 'Siliguri', 1: 0, 2: 1},
 15: {0: 'Odisha', 1: 0, 2: 1},
 16: {0: 'Odisha', 1: 0, 2: 1},
 17: {0: 'Odisha', 1: 0, 2: 1},
 18: {0: 'Bihar', 1: 0, 2: 1},
 19: {0: 'Odisha', 1: 0, 2: 1},
 20: {0: 'Bihar', 1: 0, 2: 1}}

In [6]:
# name of supplier area , demand, lat, long 
supplier = {
    1:('unnao',100000,26.5393,80.4878),
    2:('Dankuni',200000,22.6809,88.2970),
    3:('Siliguri',230000,26.7271,88.3953),
    4:('Patna',100000,25.5941,85.1376),
    5:('Khurda',250000,20.1863,85.6223),
}

demand_supp = {
    1:100000,
    2:200000,
    3:230000,
    4:200000,
    5:100000,
}

# distance between supplier to potential warehouse (w,s)
dist_supply = {
    (1,1) : 462,	(1,2) : 448,	(1,3) : 381,	(1,4) : 30,	(1,5) : 542,
(2,1) : 481,	(2,2) : 459,	(2,3) : 340,	(2,4) : 32,	(2,5) : 586,
(3,1) : 1121,	(3,2) : 300,	(3,3) : 419,	(3,4) : 639,	(3,5) : 658,
(4,1) : 880,	(4,2) : 415,	(4,3) : 779,	(4,4) : 612,	(4,5) : 59,
(5,1) : 527,	(5,2) : 445,	(5,3) : 288,	(5,4) : 75,	(5,5) : 606,
(6,1) : 529,	(6,2) : 409,	(6,3) : 306,	(6,4) : 47,	(6,5) : 557,
(7,1) : 473,	(7,2) : 437,	(7,3) : 373,	(7,4) : 25,	(7,5) : 536,
(8,1) : 1365,	(8,2) : 590,	(8,3) : 585,	(8,4) : 883,	(8,5) : 945,
(9,1) : 788,	(9,2) : 394,	(9,3) : 26,	(9,4) : 328,	(9,5) : 700,
(10,1) : 471,	(10,2) : 440,	(10,3) : 375,	(10,4) : 26,	(10,5) : 537,
(11,1) : 480,	(11,2) : 460,	(11,3) : 340,	(11,4) : 33,	(11,5) : 587,
(12,1) : 676,	(12,2) : 559,	(12,3) : 821,	(12,4) : 551,	(12,5) : 243,
(13,1) : 552,	(13,2) : 340,	(13,3) : 446,	(13,4) : 184,	(13,5) : 371,
(14,1) : 784,	(14,2) : 392,	(14,3) : 31,	(14,4) : 324,	(14,5) : 696,
(15,1) : 758,	(15,2) : 317,	(15,3) : 637,	(15,4) : 456,	(15,5) : 98,
(16,1) : 865,	(16,2) : 525,	(16,3) : 868,	(16,4) : 663,	(16,5) : 152,
(17,1) : 842,	(17,2) : 376,	(17,3) : 731,	(17,4) : 563,	(17,5) : 9,
(18,1) : 536,	(18,2) : 426,	(18,3) : 285,	(18,4) : 69,	(18,5) : 589,
(19,1) : 628,	(19,2) : 522,	(19,3) : 764,	(19,4) : 488,	(19,5) : 244,
(20,1) : 468,	(20,2) : 441,	(20,3) : 379,	(20,4) : 29,	(20,5) : 536,

}

### Generate the map for warehouse and custumers

In [7]:

input_map = True
output_map = True


## Our Linear Programming

In [8]:
#This is code to start running the model. Linear programming
# to find the time for code
start_time=time.time()

def optimal_location(number_of_whs, warehouses, customers, customer_demands, distance, distance_band, scenario_name,supplier,dist_supply,demand_supp):
    
    hccb_min = LpProblem("Als", LpMinimize)


    # Y(i,j) is assign_vars.
    assign_vars = LpVariable.dicts("Asssignment",[(w, c) for w in warehouses for c in customers],0,1,LpInteger)
      
    Yij_supp = LpVariable.dicts("Supply",[(w,s) for w in warehouses for s in supplier],0,1,LpInteger)

    # Another dicionary called 'facility_vars' is created to contain 
    #the decision variables Xi - the decision of a facility at city i to be opened or not
    
    facility_vars = LpVariable.dicts("Open", [w for w in warehouses],0,1,LpInteger)
    # The objective function is added to 'prob' first
    
  
    # 1.
    total_weighted_demand_objective = lpSum([customer_demands[c]*distance[w,c]*assign_vars[w,c]
                       for w in warehouses for c in customers ])
    total_weighted_demand_objective += lpSum([demand_supp[s]* dist_supply[w,s]* Yij_supp[w,s] 
                    for w in warehouses for s in supplier])
    
    #
    # Every customer must be served
    """
    A Pulp LP constraint: LpConstraint(e, sense, name, rhs)
    Parameters:	
    e – an instance of LpExpression
    sense – one of LpConstraintEQ, LpConstraintGE, LpConstraintLE
    name – identifying string
    rhs – numerical value of constraint right hand side
     """
    #2
    for customer in customers:
        hccb_min += LpConstraint(e = lpSum([assign_vars[w, customer] for w in warehouses]), 
                                    sense=LpConstraintEQ, 
                                    name=str(customer)+"_Served", 
                                    rhs=1)
    #3
    # every supplier is fully used
    for supplys in supplier:
        hccb_min += LpConstraint(e= lpSum([Yij_supp[w, supplys] for w in warehouses]),
                                    sense = LpConstraintGE,
                                    name = str(supplys)+"Done",
                                    rhs = 1)
    #4
    # Ensure that P and exactly P facilities are opened.  We use the variable name 'number_of_whs' for P
    hccb_min += LpConstraint(e = lpSum([facility_vars[w] for w in warehouses]), 
                                    sense=LpConstraintEQ, 
                                    name="FacilityCount", 
                                    rhs=number_of_whs)
    
    # Use the user's status to fix some warehouses in the solution
    #5
    #constrint to link supplier to customers
    for ss in supplier:
        for w in warehouses:
            hccb_min+= LpConstraint(e = Yij_supp[w,ss] -facility_vars[w] ,
                                   sense = LpConstraintLE,
                                   name = str(ss)+str(w)+"complete",
                                   rhs = 0)
    for w in warehouses:
        hccb_min += LpConstraint(e = lpSum([Yij_supp[w,ss] for ss in supplier]) - facility_vars[w],
                                    sense = LpConstraintGE,
                                    name = str(w)+"serve each",
                                    rhs = 0)
        
    for facility in warehouses:
        w = facility 
        hccb_min += LpConstraint(e = facility_vars[facility] , 
                                   sense=LpConstraintGE, 
                                    name=str(facility) + "_" + "Lower Bound",
                                    rhs=wh_status[w][1])
    for facility in warehouses:
        w = facility
        hccb_min += LpConstraint(e = facility_vars[facility] , 
                                   sense=LpConstraintLE, 
                                    name=str(facility) + "_" + "Upper Bound",
                                    rhs=wh_status[w][2])

    #9
    for facility in warehouses:
        bigM = 10000
        hccb_min += LpConstraint(e = lpSum(assign_vars[facility, customer] for customer in customers)-bigM*facility_vars[facility] , 
                                    sense=LpConstraintLE, 
                                    name=str(facility) + "_Route-BigM Version",
                                    rhs=0)        
    

    # Setting problem objective
    hccb_min.setObjective(total_weighted_demand_objective)

    # The problem is solved using PuLP's choice of Solver
    _solver = pulp.PULP_CBC_CMD(keepFiles=False,fracGap=0.00,maxSeconds=3000, msg=True)
    hccb_min.solve(solver=_solver)

    # The status of the solution is printed to the screen
    #print ("Status:", LpStatus[als_problem.status])
    file.write('\nstatus:'+ LpStatus[hccb_min.status])
    print("Optimization Status",LpStatus[hccb_min.status] ) #print in Jupyter Notebook
    if LpStatus[hccb_min.status] == "Infeasible" :
        print("********* ERROR: Model not feasible, don't use results. One thing to check: see that the number of available warehouses is more than the fixed status.")
    #print objective
    total_supply = sum(demand_supp.values())
    total_demand = sum(customer_demands.values())
    total_demand_to_warehouse = {w: sum(customer_demands[c]*assign_vars[w,c].varValue for c in customers)
                                    for w in warehouses
                                    if facility_vars[w].varValue>0}
    
    #print(total_demand_to_warehouse)
    #print("Total Demand",total_demand)
    file.write("\nTotal Demand:"+ str(total_demand))
    file.write("\nTotal Supply:"+ str(total_supply))
    #print("Objective: ", value(als_problem.objective))
    file.write("\nObjective: "+ str(value(hccb_min.objective)))
    wgt_avg_dist = value(hccb_min.objective)/(total_demand+total_supply)
    
    file.write("\nWeighted Average Distance: {:.2f} km" .format(wgt_avg_dist))
    print("Weighted Average Distance: {:.2f} km" .format(wgt_avg_dist))
    
    end_time = time.time()
    time_diff = end_time - start_time
    #print("Run Time in seconds {:.1f}" .format(time_diff))
    file.write("\nRun Time in seconds {:.1f}" .format(time_diff))
    print("Run Time in seconds {:.1f}" .format(time_diff))
 
    
    #preparing data to write in excel sheets
    opened_warehouses = []
      
    for w in facility_vars.keys():
        if(facility_vars[w].varValue > 0):
            #print(warehouses[w])
            wh = {
                'Warehouse Key': w,
                'Warehouse City':warehouses[w][1],
                'State':warehouses[w][2],
                'ZipCode':warehouses[w][3],
                'Lat':warehouses[w][4],
                'Long':warehouses[w][5],
                'Total Demand to Warehouse':total_demand_to_warehouse[w]
            }
            opened_warehouses.append(wh)
            
            
    #converting the list to dataframe        
    df_wh = pd.DataFrame.from_records(opened_warehouses)
    
    df_wh = df_wh[['Warehouse Key', 'Warehouse City', 'State', 'Total Demand to Warehouse','Lat','Long']]
    list_warehouses_open = list(df_wh['Warehouse Key'])
    
    # writing detailed files
    writer = pd.ExcelWriter(scenario_name+'_depand for n no of warehouse.xlsx')
    df_wh.to_excel(writer,'Opened Warehouses',index=False)
    
#     print("Customers Assigned to warehouses")
    customers_assignment = []   
    for (w,c) in assign_vars.keys():
        if assign_vars[(w,c)].varValue > 0:
            cust = {
                'Warehouse':str(warehouses[w][0]+','+warehouses[w][1]),
                'Customer':str(customers[c][0]+','+customers[c][1]),
                'Customer Demand': customer_demands[c],
                'Distance': distance[w,c],
                'Warehouse Latitude' : warehouses[w][4],
                'Warehouse Longitude' : warehouses[w][5],
                'Customers Latitude' : customers[c][4],
                'Customers Longitude': customers[c][5]
            }
            customers_assignment.append(cust)
                  
    df_cu = pd.DataFrame.from_records(customers_assignment)
    df_cu_copy = df_cu.copy()
    df_cu = df_cu[['Warehouse','Warehouse Latitude', 'Warehouse Longitude' ,'Customer','Customers Latitude', 'Customers Longitude','Distance', 'Customer Demand']]
    df_cu.to_excel(writer,'Customer Distribution',index=False)
    
    supplier_assignment = []
    for (w,s) in Yij_supp.keys():
        if Yij_supp[(w,s)].varValue > 0:
            suppl = {
                'Supplier' : str(supplier[s][0]),
                'Warehouse' : str(warehouses[w][0]+','+warehouses[w][1]),
                'Supplier supply' : demand_supp[s],
                'Warehouse Latitude' : warehouses[w][4],
                'Warehouse Longitude' : warehouses[w][5],
                'Supplier Latitude' : supplier[s][2],
                'Supplier Longitude' : supplier[s][3]
            }
            supplier_assignment.append(suppl)
    df_su = pd.DataFrame.from_records(supplier_assignment)
    df_su_copy = df_su.copy()
    df_su = df_su[['Warehouse','Warehouse Latitude','Warehouse Longitude','Supplier','Supplier Latitude','Supplier Longitude','Supplier supply']]
    
    df_su.to_excel(writer,'Supplier Distribution',index = False)
    writer.close()

    #writing percent demand within each distance bands
    total_demand = sum(df_cu['Customer Demand'])
    percent_demand_distance_band_1 = sum(df_cu[df_cu['Distance']<distance_band_1]['Customer Demand'])*100/total_demand
    percent_demand_distance_band_2 = sum(df_cu[df_cu['Distance']<distance_band_2]['Customer Demand'])*100/total_demand
    percent_demand_distance_band_3 = sum(df_cu[df_cu['Distance']<distance_band_3]['Customer Demand'])*100/total_demand
    percent_demand_distance_band_4 = sum(df_cu[df_cu['Distance']<distance_band_4]['Customer Demand'])*100/total_demand
    file.write("\nPercent Demand served within {} kms : {:.1f}" .format(distance_band[0], percent_demand_distance_band_1))
    file.write("\nPercent Demand served within {} kms : {:.1f}" .format(distance_band[1], percent_demand_distance_band_2))
    file.write("\nPercent Demand served within {} kms : {:.1f}" .format(distance_band[2], percent_demand_distance_band_3))
    file.write("\nPercent Demand served within {} kms : {:.1f}" .format(distance_band[3], percent_demand_distance_band_4))
   
    return df_cu_copy,list_warehouses_open,df_su_copy
         
            


## Lat Long (location) for Potantial warehouse

In [9]:
def get_data():
    # Warehouses:: 
    # id:(name, city, state, pincode, lat, long)
    
    warehouses= pd.read_excel('warehou.xlsx', engine = 'openpyxl')
    warehouses = warehouses.to_dict()
    warehouses

# Customers/distributors details in form of their location
# customers:: 
    # id:(name, city, state, pincode, lat, long)
    customers={
        1:('Bihar' ,'Bihar' , 'India',' ', 25.444 , 84.8836) ,
2:('Bihar' ,'Bihar' , 'India',' ', 25.9111 , 85.1663) ,
3:('North East' ,'North East' , 'India',' ', 23.6251 , 91.0846) ,
4:('Odisha' ,'Odisha' , 'India',' ', 19.6112 , 85.6271) ,
5:('Bihar' ,'Bihar' , 'India',' ', 26.1311 , 85.6334) ,
6:('Bihar' ,'Bihar' , 'India',' ', 25.6513 , 85.598) ,
7:('Bihar' ,'Bihar' , 'India',' ', 25.404 , 84.9873) ,
8:('North East' ,'North East' , 'India',' ', 24.6498 , 93.7381) ,
9:('Siliguri' ,'Siliguri' , 'India',' ', 26.546 , 88.2129) ,
10:('Bihar' ,'Bihar' , 'India',' ', 25.4115 , 84.9619) ,
11:('Bihar' ,'Bihar' , 'India',' ', 25.9192 , 85.1571) ,
12:('Odisha' ,'Odisha' , 'India',' ', 20.523 , 83.2623) ,
13:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.7897 , 85.1545) ,
14:('Siliguri' ,'Siliguri' , 'India',' ', 26.519 , 88.1734) ,
15:('Odisha' ,'Odisha' , 'India',' ', 21.1465 , 85.5955) ,
16:('Odisha' ,'Odisha' , 'India',' ', 19.1162 , 84.5897) ,
17:('Odisha' ,'Odisha' , 'India',' ', 20.1024 , 85.658) ,
18:('Bihar' ,'Bihar' , 'India',' ', 25.9559 , 85.7097) ,
19:('Odisha' ,'Odisha' , 'India',' ', 21.1288 , 83.4242) ,
20:('Bihar' ,'Bihar' , 'India',' ', 25.3981 , 84.9282) ,
21:('Jharkhand' ,'Jharkhand' , 'India',' ', 22.5567 , 86.025) ,
22:('Bihar' ,'Bihar' , 'India',' ', 25.6678 , 86.3926) ,
23:('Bihar' ,'Bihar' , 'India',' ', 25.9012 , 85.7076) ,
24:('Odisha' ,'Odisha' , 'India',' ', 20.0786 , 85.6704) ,
25:('North East' ,'North East' , 'India',' ', 26.1536 , 92.4796) ,
26:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.1615 , 85.119) ,
27:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.7336 , 85.1785) ,
28:('Bihar' ,'Bihar' , 'India',' ', 25.3925 , 84.9501) ,
29:('Odisha' ,'Odisha' , 'India',' ', 21.0904 , 86.4839) ,
30:('Bihar' ,'Bihar' , 'India',' ', 25.4144 , 84.889) ,
31:('Bihar' ,'Bihar' , 'India',' ', 25.6838 , 85.4913) ,
32:('Bihar' ,'Bihar' , 'India',' ', 24.7063 , 85.3392) ,
33:('Bihar' ,'Bihar' , 'India',' ', 24.5449 , 84.165) ,
34:('Bihar' ,'Bihar' , 'India',' ', 25.5536 , 85.8314) ,
35:('Bihar' ,'Bihar' , 'India',' ', 26.6018 , 84.305) ,
36:('Siliguri' ,'Siliguri' , 'India',' ', 27.1234 , 88.4113) ,
37:('Siliguri' ,'Siliguri' , 'India',' ', 26.124 , 89.2447) ,
38:('Bihar' ,'Bihar' , 'India',' ', 25.9177 , 85.3365) ,
39:('Bihar' ,'Bihar' , 'India',' ', 25.9175 , 85.201) ,
40:('North East' ,'North East' , 'India',' ', 26.431 , 92.5878) ,
41:('Odisha' ,'Odisha' , 'India',' ', 20.744 , 85.934) ,
42:('Bihar' ,'Bihar' , 'India',' ', 26.2738 , 85.5053) ,
43:('Odisha' ,'Odisha' , 'India',' ', 19.6664 , 82.9598) ,
44:('Bihar' ,'Bihar' , 'India',' ', 25.6773 , 84.7103) ,
45:('Siliguri' ,'Siliguri' , 'India',' ', 23.4698 , 86.8358) ,
46:('Bihar' ,'Bihar' , 'India',' ', 26.7761 , 84.6504) ,
47:('Siliguri' ,'Siliguri' , 'India',' ', 26.5687 , 88.2469) ,
48:('North East' ,'North East' , 'India',' ', 26.901 , 93.4935) ,
49:('Bihar' ,'Bihar' , 'India',' ', 26.1604 , 85.8687) ,
50:('Odisha' ,'Odisha' , 'India',' ', 21.6794 , 83.8152) ,
51:('Bihar' ,'Bihar' , 'India',' ', 24.7054 , 83.9903) ,
52:('North East' ,'North East' , 'India',' ', 25.9323 , 91.6168) ,
53:('Bihar' ,'Bihar' , 'India',' ', 25.9491 , 85.2017) ,
54:('North East' ,'North East' , 'India',' ', 26.558 , 94.0192) ,
55:('Odisha' ,'Odisha' , 'India',' ', 20.62 , 84.9023) ,
56:('Odisha' ,'Odisha' , 'India',' ', 20.7063 , 84.9796) ,
57:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.1877 , 85.1755) ,
58:('Bihar' ,'Bihar' , 'India',' ', 24.752 , 83.783) ,
59:('North East' ,'North East' , 'India',' ', 25.9068 , 91.5105) ,
60:('Bihar' ,'Bihar' , 'India',' ', 25.3334 , 87.3697) ,
61:('Odisha' ,'Odisha' , 'India',' ', 19.9976 , 83.7159) ,
62:('Bihar' ,'Bihar' , 'India',' ', 25.9016 , 86.4001) ,
63:('Bihar' ,'Bihar' , 'India',' ', 25.368 , 84.4734) ,
64:('Bihar' ,'Bihar' , 'India',' ', 26.6014 , 84.3078) ,
65:('North East' ,'North East' , 'India',' ', 27.2813 , 95.1621) ,
66:('Odisha' ,'Odisha' , 'India',' ', 21.5532 , 87.1605) ,
67:('Odisha' ,'Odisha' , 'India',' ', 20.711 , 84.9586) ,
68:('Bihar' ,'Bihar' , 'India',' ', 26.0379 , 86.0752) ,
69:('Bihar' ,'Bihar' , 'India',' ', 25.581 , 87.2803) ,
70:('Bihar' ,'Bihar' , 'India',' ', 25.8219 , 85.2835) ,
71:('Bihar' ,'Bihar' , 'India',' ', 25.0395 , 86.7413) ,
72:('Siliguri' ,'Siliguri' , 'India',' ', 27.0351 , 88.2861) ,
73:('Bihar' ,'Bihar' , 'India',' ', 26.3796 , 85.9337) ,
74:('North East' ,'North East' , 'India',' ', 25.7088 , 93.5265) ,
75:('Odisha' ,'Odisha' , 'India',' ', 19.6878 , 85.6139) ,
76:('Odisha' ,'Odisha' , 'India',' ', 20.0568 , 85.5918) ,
77:('Odisha' ,'Odisha' , 'India',' ', 20.984 , 85.915) ,
78:('Odisha' ,'Odisha' , 'India',' ', 21.2965 , 86.7326) ,
79:('Odisha' ,'Odisha' , 'India',' ', 20.2197 , 85.1439) ,
80:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.2061 , 85.1942) ,
81:('Odisha' ,'Odisha' , 'India',' ', 19.9078 , 83.8509) ,
82:('Bihar' ,'Bihar' , 'India',' ', 26.249 , 86.0676) ,
83:('Odisha' ,'Odisha' , 'India',' ', 20.0771 , 85.557) ,
84:('Odisha' ,'Odisha' , 'India',' ', 20.6415 , 85.0257) ,
85:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.8414 , 83.8909) ,
86:('Bihar' ,'Bihar' , 'India',' ', 25.681 , 86.3976) ,
87:('Odisha' ,'Odisha' , 'India',' ', 21.1661 , 83.549) ,
88:('Bihar' ,'Bihar' , 'India',' ', 25.1785 , 86.9108) ,
89:('Bihar' ,'Bihar' , 'India',' ', 25.4985 , 85.0071) ,
90:('North East' ,'North East' , 'India',' ', 25.9707 , 91.5318) ,
91:('Odisha' ,'Odisha' , 'India',' ', 20.0968 , 85.6653) ,
92:('Bihar' ,'Bihar' , 'India',' ', 25.4017 , 84.103) ,
93:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.4516 , 85.9328) ,
94:('Odisha' ,'Odisha' , 'India',' ', 20.2784 , 85.9895) ,
95:('Odisha' ,'Odisha' , 'India',' ', 19.3097 , 84.689) ,
96:('Bihar' ,'Bihar' , 'India',' ', 25.5928 , 87.278) ,
97:('Siliguri' ,'Siliguri' , 'India',' ', 26.842 , 88.064) ,
98:('Bihar' ,'Bihar' , 'India',' ', 25.0235 , 86.772) ,
99:('Odisha' ,'Odisha' , 'India',' ', 20.5895 , 85.5229) ,
100:('Odisha' ,'Odisha' , 'India',' ', 20.2102 , 85.6264) ,
101:('Odisha' ,'Odisha' , 'India',' ', 22.035 , 84.61) ,
102:('Odisha' ,'Odisha' , 'India',' ', 21.6064 , 87.0367) ,
103:('Odisha' ,'Odisha' , 'India',' ', 20.876 , 84.8949) ,
104:('Bihar' ,'Bihar' , 'India',' ', 26.1014 , 87.0606) ,
105:('Siliguri' ,'Siliguri' , 'India',' ', 23.1521 , 86.1887) ,
106:('Bihar' ,'Bihar' , 'India',' ', 24.5651 , 84.8005) ,
107:('Odisha' ,'Odisha' , 'India',' ', 20.2718 , 85.6701) ,
108:('Odisha' ,'Odisha' , 'India',' ', 20.1064 , 86.4213) ,
109:('Odisha' ,'Odisha' , 'India',' ', 21.7203 , 86.5395) ,
110:('Odisha' ,'Odisha' , 'India',' ', 20.5968 , 85.3485) ,
111:('Odisha' ,'Odisha' , 'India',' ', 20.0689 , 85.9581) ,
112:('Bihar' ,'Bihar' , 'India',' ', 25.1165 , 85.0807) ,
113:('Bihar' ,'Bihar' , 'India',' ', 25.6883 , 85.4386) ,
114:('Odisha' ,'Odisha' , 'India',' ', 19.7461 , 85.619) ,
115:('Odisha' ,'Odisha' , 'India',' ', 19.9581 , 85.5098) ,
116:('Odisha' ,'Odisha' , 'India',' ', 20.7556 , 85.0197) ,
117:('Odisha' ,'Odisha' , 'India',' ', 18.9712 , 83.2115) ,
118:('Siliguri' ,'Siliguri' , 'India',' ', 24.9749 , 87.9798) ,
119:('Siliguri' ,'Siliguri' , 'India',' ', 26.9634 , 88.092) ,
120:('Odisha' ,'Odisha' , 'India',' ', 20.0511 , 85.6303) ,
121:('North East' ,'North East' , 'India',' ', 23.4718 , 91.0772) ,
122:('Bihar' ,'Bihar' , 'India',' ', 25.3813 , 84.6687) ,
123:('Bihar' ,'Bihar' , 'India',' ', 26.1125 , 86.6467) ,
124:('Odisha' ,'Odisha' , 'India',' ', 18.164 , 81.6974) ,
125:('Odisha' ,'Odisha' , 'India',' ', 20.6616 , 84.9103) ,
126:('Odisha' ,'Odisha' , 'India',' ', 21.2939 , 86.761) ,
127:('North East' ,'North East' , 'India',' ', 23.3354 , 91.2875) ,
128:('Odisha' ,'Odisha' , 'India',' ', 20.789 , 85.786) ,
129:('Jharkhand' ,'Jharkhand' , 'India',' ', 24.2936 , 86.5206) ,
130:('Odisha' ,'Odisha' , 'India',' ', 20.0378 , 85.6255) ,
131:('Bihar' ,'Bihar' , 'India',' ', 25.038 , 84.4697) ,
132:('Odisha' ,'Odisha' , 'India',' ', 20.0711 , 85.5948) ,
133:('Bihar' ,'Bihar' , 'India',' ', 24.7705 , 84.774) ,
134:('Bihar' ,'Bihar' , 'India',' ', 25.2071 , 85.9366) ,
135:('Bihar' ,'Bihar' , 'India',' ', 25.7205 , 86.5932) ,
136:('Bihar' ,'Bihar' , 'India',' ', 26.3909 , 85.286) ,
137:('North East' ,'North East' , 'India',' ', 25.4754 , 93.9041) ,
138:('Bihar' ,'Bihar' , 'India',' ', 24.9871 , 85.3152) ,
139:('North East' ,'North East' , 'India',' ', 23.5529 , 92.5193) ,
140:('Bihar' ,'Bihar' , 'India',' ', 25.9451 , 85.7031) ,
141:('Bihar' ,'Bihar' , 'India',' ', 25.3792 , 84.5896) ,
142:('Bihar' ,'Bihar' , 'India',' ', 26.4485 , 84.7274) ,
143:('Bihar' ,'Bihar' , 'India',' ', 25.4931 , 85.0263) ,
144:('Odisha' ,'Odisha' , 'India',' ', 21.287 , 86.222) ,
145:('Bihar' ,'Bihar' , 'India',' ', 25.3755 , 85.0536) ,
146:('Jharkhand' ,'Jharkhand' , 'India',' ', 22.5914 , 85.965) ,
147:('Bihar' ,'Bihar' , 'India',' ', 25.3952 , 84.977) ,
148:('Bihar' ,'Bihar' , 'India',' ', 25.261 , 85.796) ,
149:('Odisha' ,'Odisha' , 'India',' ', 22.0265 , 84.661) ,
150:('Odisha' ,'Odisha' , 'India',' ', 20.0879 , 82.9547) ,
151:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.4222 , 85.3125) ,
152:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.2209 , 85.0061) ,
153:('Odisha' ,'Odisha' , 'India',' ', 19.0385 , 83.621) ,
154:('Bihar' ,'Bihar' , 'India',' ', 24.3705 , 84.5884) ,
155:('Bihar' ,'Bihar' , 'India',' ', 25.3996 , 86.0639) ,
156:('Odisha' ,'Odisha' , 'India',' ', 20.417 , 86.011) ,
157:('Jharkhand' ,'Jharkhand' , 'India',' ', 22.6218 , 85.9792) ,
158:('Bihar' ,'Bihar' , 'India',' ', 25.0429 , 86.5232) ,
159:('Bihar' ,'Bihar' , 'India',' ', 26.4397 , 85.2213) ,
160:('Bihar' ,'Bihar' , 'India',' ', 25.8324 , 85.611) ,
161:('Odisha' ,'Odisha' , 'India',' ', 20.7279 , 85.1513) ,
162:('Odisha' ,'Odisha' , 'India',' ', 22.0314 , 84.6265) ,
163:('Odisha' ,'Odisha' , 'India',' ', 20.402 , 86.511) ,
164:('Odisha' ,'Odisha' , 'India',' ', 20.6375 , 84.1222) ,
165:('Bihar' ,'Bihar' , 'India',' ', 25.3598 , 87.356) ,
166:('Odisha' ,'Odisha' , 'India',' ', 19.9454 , 85.5414) ,
167:('Odisha' ,'Odisha' , 'India',' ', 20.7144 , 84.5397) ,
168:('Odisha' ,'Odisha' , 'India',' ', 21.282 , 83.7565) ,
169:('Bihar' ,'Bihar' , 'India',' ', 24.9827 , 85.8965) ,
170:('Siliguri' ,'Siliguri' , 'India',' ', 25.6705 , 87.6462) ,
171:('Odisha' ,'Odisha' , 'India',' ', 20.1348 , 85.6854) ,
172:('Bihar' ,'Bihar' , 'India',' ', 25.6956 , 84.4963) ,
173:('Odisha' ,'Odisha' , 'India',' ', 20.2746 , 84.0354) ,
174:('Bihar' ,'Bihar' , 'India',' ', 25.3615 , 85.9298) ,
175:('Bihar' ,'Bihar' , 'India',' ', 25.9914 , 85.5337) ,
176:('Bihar' ,'Bihar' , 'India',' ', 25.8945 , 85.206) ,
177:('Bihar' ,'Bihar' , 'India',' ', 25.0193 , 84.7896) ,
178:('Bihar' ,'Bihar' , 'India',' ', 24.8778 , 84.0086) ,
179:('Bihar' ,'Bihar' , 'India',' ', 25.8763 , 85.9321) ,
180:('Odisha' ,'Odisha' , 'India',' ', 19.6763 , 82.5913) ,
181:('Bihar' ,'Bihar' , 'India',' ', 25.4531 , 87.7158) ,
182:('Bihar' ,'Bihar' , 'India',' ', 25.1764 , 86.2761) ,
183:('Bihar' ,'Bihar' , 'India',' ', 26.2984 , 86.1962) ,
184:('Bihar' ,'Bihar' , 'India',' ', 24.6723 , 85.3461) ,
185:('North East' ,'North East' , 'India',' ', 25.8183 , 89.7885) ,
186:('Bihar' ,'Bihar' , 'India',' ', 26.249 , 86.0676) ,
187:('Bihar' ,'Bihar' , 'India',' ', 25.7289 , 85.0942) ,
188:('Bihar' ,'Bihar' , 'India',' ', 26.0132 , 85.8796) ,
189:('Odisha' ,'Odisha' , 'India',' ', 19.4675 , 82.0044) ,
190:('Odisha' ,'Odisha' , 'India',' ', 19.9497 , 86.0322) ,
191:('Bihar' ,'Bihar' , 'India',' ', 26.1358 , 84.9152) ,
192:('Bihar' ,'Bihar' , 'India',' ', 25.3063 , 86.2797) ,
193:('Odisha' ,'Odisha' , 'India',' ', 20.2755 , 85.6873) ,
194:('Odisha' ,'Odisha' , 'India',' ', 20.2474 , 85.6951) ,
195:('Bihar' ,'Bihar' , 'India',' ', 25.5846 , 84.5276) ,
196:('Bihar' ,'Bihar' , 'India',' ', 25.9333 , 85.1846) ,
197:('Bihar' ,'Bihar' , 'India',' ', 25.2651 , 84.2149) ,
198:('Bihar' ,'Bihar' , 'India',' ', 24.9423 , 85.6357) ,
199:('Odisha' ,'Odisha' , 'India',' ', 20.3029 , 85.7258) ,
200:('Bihar' ,'Bihar' , 'India',' ', 24.6965 , 84.1744) ,
201:('Bihar' ,'Bihar' , 'India',' ', 25.9017 , 84.5699) ,
202:('Bihar' ,'Bihar' , 'India',' ', 24.7421 , 84.6405) ,
203:('Odisha' ,'Odisha' , 'India',' ', 19.8586 , 85.1418) ,
204:('Bihar' ,'Bihar' , 'India',' ', 24.9969 , 85.32) ,
205:('Odisha' ,'Odisha' , 'India',' ', 20.3216 , 85.3283) ,
206:('Jharkhand' ,'Jharkhand' , 'India',' ', 24.2341 , 85.3258) ,
207:('Bihar' ,'Bihar' , 'India',' ', 24.8312 , 85.2175) ,
208:('Siliguri' ,'Siliguri' , 'India',' ', 25.9316 , 89.2612) ,
209:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.9806 , 86.119) ,
210:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.3737 , 85.4495) ,
211:('Odisha' ,'Odisha' , 'India',' ', 20.719 , 86.033) ,
212:('Odisha' ,'Odisha' , 'India',' ', 19.8791 , 84.8932) ,
213:('Odisha' ,'Odisha' , 'India',' ', 19.9907 , 83.0216) ,
214:('Bihar' ,'Bihar' , 'India',' ', 24.6823 , 86.0325) ,
215:('Bihar' ,'Bihar' , 'India',' ', 25.8349 , 84.9469) ,
216:('Odisha' ,'Odisha' , 'India',' ', 21.2915 , 83.7682) ,
217:('Bihar' ,'Bihar' , 'India',' ', 25.421 , 84.8452) ,
218:('Bihar' ,'Bihar' , 'India',' ', 26.0588 , 84.7394) ,
219:('Siliguri' ,'Siliguri' , 'India',' ', 24.8009 , 87.952) ,
220:('Bihar' ,'Bihar' , 'India',' ', 25.4054 , 84.9885) ,
221:('Bihar' ,'Bihar' , 'India',' ', 25.8906 , 87.0338) ,
222:('Odisha' ,'Odisha' , 'India',' ', 21.6184 , 83.7262) ,
223:('Odisha' ,'Odisha' , 'India',' ', 21.8275 , 85.2216) ,
224:('Odisha' ,'Odisha' , 'India',' ', 20.4835 , 85.0973) ,
225:('Odisha' ,'Odisha' , 'India',' ', 19.1196 , 84.5876) ,
226:('Odisha' ,'Odisha' , 'India',' ', 20.0029 , 85.6666) ,
227:('Odisha' ,'Odisha' , 'India',' ', 20.409 , 85.865) ,
228:('Odisha' ,'Odisha' , 'India',' ', 21.627 , 85.3698) ,
229:('Bihar' ,'Bihar' , 'India',' ', 24.5992 , 84.844) ,
230:('Odisha' ,'Odisha' , 'India',' ', 22.0565 , 85.9714) ,
231:('Odisha' ,'Odisha' , 'India',' ', 19.4175 , 84.4647) ,
232:('Bihar' ,'Bihar' , 'India',' ', 26.0085 , 85.0938) ,
233:('Odisha' ,'Odisha' , 'India',' ', 20.499 , 85.934) ,
234:('Odisha' ,'Odisha' , 'India',' ', 19.9742 , 85.4167) ,
235:('Bihar' ,'Bihar' , 'India',' ', 25.9012 , 85.7076) ,
236:('Odisha' ,'Odisha' , 'India',' ', 19.8304 , 85.7943) ,
237:('Odisha' ,'Odisha' , 'India',' ', 18.5877 , 83.8863) ,
238:('Jharkhand' ,'Jharkhand' , 'India',' ', 22.7352 , 84.8958) ,
239:('Odisha' ,'Odisha' , 'India',' ', 20.3092 , 85.4379) ,
240:('Bihar' ,'Bihar' , 'India',' ', 26.9099 , 84.2646) ,
241:('Odisha' ,'Odisha' , 'India',' ', 20.271 , 85.7067) ,
242:('Odisha' ,'Odisha' , 'India',' ', 20.069 , 85.611) ,
243:('Bihar' ,'Bihar' , 'India',' ', 25.4124 , 85.2858) ,
244:('Bihar' ,'Bihar' , 'India',' ', 24.5047 , 84.7946) ,
245:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.0774 , 85.1066) ,
246:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.1561 , 85.1075) ,
247:('Siliguri' ,'Siliguri' , 'India',' ', 24.8067 , 87.9352) ,
248:('Odisha' ,'Odisha' , 'India',' ', 21.448 , 87.0744) ,
249:('Odisha' ,'Odisha' , 'India',' ', 20.754 , 85.913) ,
250:('Bihar' ,'Bihar' , 'India',' ', 25.043 , 83.8674) ,
251:('Bihar' ,'Bihar' , 'India',' ', 25.9324689 , 87.2528067) ,
252:('Siliguri' ,'Siliguri' , 'India',' ', 23.0305 , 86.879) ,
253:('Odisha' ,'Odisha' , 'India',' ', 20.2694 , 82.7183) ,
254:('Siliguri' ,'Siliguri' , 'India',' ', 26.4405 , 88.3039) ,
255:('Bihar' ,'Bihar' , 'India',' ', 25.6039 , 85.1866) ,
256:('Odisha' ,'Odisha' , 'India',' ', 20.897 , 86.325) ,
257:('Odisha' ,'Odisha' , 'India',' ', 20.2515 , 85.6857) ,
258:('Odisha' ,'Odisha' , 'India',' ', 21.9965 , 84.389) ,
259:('Bihar' ,'Bihar' , 'India',' ', 25.306 , 85.1027) ,
260:('Bihar' ,'Bihar' , 'India',' ', 25.3703 , 83.781) ,
261:('Bihar' ,'Bihar' , 'India',' ', 25.6654 , 84.9745) ,
262:('Odisha' ,'Odisha' , 'India',' ', 19.4032 , 84.4655) ,
263:('Bihar' ,'Bihar' , 'India',' ', 25.0063 , 84.7909) ,
264:('Odisha' ,'Odisha' , 'India',' ', 17.8737 , 81.5499) ,
265:('Bihar' ,'Bihar' , 'India',' ', 25.513 , 85.3329) ,
266:('Odisha' ,'Odisha' , 'India',' ', 20.2892 , 85.6396) ,
267:('Bihar' ,'Bihar' , 'India',' ', 25.5857 , 87.3265) ,
268:('Jharkhand' ,'Jharkhand' , 'India',' ', 24.6311 , 87.0107) ,
269:('Bihar' ,'Bihar' , 'India',' ', 26.2016 , 85.9567) ,
270:('Siliguri' ,'Siliguri' , 'India',' ', 26.8672 , 88.2707) ,
271:('Bihar' ,'Bihar' , 'India',' ', 25.3692 , 85.4614) ,
272:('Bihar' ,'Bihar' , 'India',' ', 25.7217 , 85.6005) ,
273:('Bihar' ,'Bihar' , 'India',' ', 25.2739 , 85.5054) ,
274:('Bihar' ,'Bihar' , 'India',' ', 24.8148 , 85.4366) ,
275:('Bihar' ,'Bihar' , 'India',' ', 25.8851 , 87.7389) ,
276:('Odisha' ,'Odisha' , 'India',' ', 20.8405 , 83.352) ,
277:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.469 , 85.8202) ,
278:('Odisha' ,'Odisha' , 'India',' ', 20.388 , 86.367) ,
279:('Odisha' ,'Odisha' , 'India',' ', 19.7131 , 84.374) ,
280:('Odisha' ,'Odisha' , 'India',' ', 20.789 , 86.436) ,
281:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.985 , 86.1076) ,
282:('Odisha' ,'Odisha' , 'India',' ', 19.9196 , 85.874) ,
283:('Siliguri' ,'Siliguri' , 'India',' ', 25.6739 , 87.6457) ,
284:('Odisha' ,'Odisha' , 'India',' ', 20.9248 , 86.5448) ,
285:('North East' ,'North East' , 'India',' ', 24.6051 , 93.7541) ,
286:('Bihar' ,'Bihar' , 'India',' ', 25.0533 , 86.8045) ,
287:('Bihar' ,'Bihar' , 'India',' ', 25.3706 , 84.4898) ,
288:('Odisha' ,'Odisha' , 'India',' ', 21.3348 , 84.5199) ,
289:('Bihar' ,'Bihar' , 'India',' ', 25.3794 , 85.7011) ,
290:('Bihar' ,'Bihar' , 'India',' ', 24.5528 , 86.5042) ,
291:('Odisha' ,'Odisha' , 'India',' ', 21.2951 , 83.7809) ,
292:('Odisha' ,'Odisha' , 'India',' ', 20.6059 , 84.8114) ,
293:('Odisha' ,'Odisha' , 'India',' ', 21.014 , 86.092) ,
294:('Odisha' ,'Odisha' , 'India',' ', 21.606 , 87.037) ,
295:('Bihar' ,'Bihar' , 'India',' ', 25.0471 , 86.7795) ,
296:('North East' ,'North East' , 'India',' ', 26.7836 , 94.4284) ,
297:('Odisha' ,'Odisha' , 'India',' ', 19.0749 , 84.6963) ,
298:('Odisha' ,'Odisha' , 'India',' ', 20.313 , 86.166) ,
299:('Bihar' ,'Bihar' , 'India',' ', 25.0632 , 87.0275) ,
300:('North East' ,'North East' , 'India',' ', 25.9731 , 91.5716) ,
301:('Bihar' ,'Bihar' , 'India',' ', 25.8911 , 85.1903) ,
302:('Odisha' ,'Odisha' , 'India',' ', 20.848 , 86.333) ,
303:('North East' ,'North East' , 'India',' ', 25.987 , 91.5837) ,
304:('Odisha' ,'Odisha' , 'India',' ', 20.8349 , 84.7014) ,
305:('North East' ,'North East' , 'India',' ', 24.6303 , 92.5502) ,
306:('Odisha' ,'Odisha' , 'India',' ', 20.7998 , 82.8713) ,
307:('Siliguri' ,'Siliguri' , 'India',' ', 23.0273 , 86.8737) ,
308:('North East' ,'North East' , 'India',' ', 27.0334 , 93.9023) ,
309:('Odisha' ,'Odisha' , 'India',' ', 18.8788 , 83.6087) ,
310:('Odisha' ,'Odisha' , 'India',' ', 20.654 , 86.136) ,
311:('Jharkhand' ,'Jharkhand' , 'India',' ', 24.4386 , 87.6403) ,
312:('North East' ,'North East' , 'India',' ', 26.5402 , 92.9392) ,
313:('Odisha' ,'Odisha' , 'India',' ', 20.1075 , 85.6191) ,
314:('Odisha' ,'Odisha' , 'India',' ', 20.7326 , 85.0015) ,
315:('Odisha' ,'Odisha' , 'India',' ', 21.1211 , 86.6211) ,
316:('Siliguri' ,'Siliguri' , 'India',' ', 26.6687 , 88.5399) ,
317:('Odisha' ,'Odisha' , 'India',' ', 20.2479 , 84.879) ,
318:('Odisha' ,'Odisha' , 'India',' ', 19.8167 , 84.782) ,
319:('Bihar' ,'Bihar' , 'India',' ', 26.7503 , 84.3425) ,
320:('Jharkhand' ,'Jharkhand' , 'India',' ', 22.3225 , 86.2541) ,
321:('Bihar' ,'Bihar' , 'India',' ', 26.06 , 87.6226) ,
322:('Bihar' ,'Bihar' , 'India',' ', 25.0185 , 85.5323) ,
323:('Odisha' ,'Odisha' , 'India',' ', 20.0032 , 85.2527) ,
324:('Bihar' ,'Bihar' , 'India',' ', 25.4937 , 85.0092) ,
325:('Odisha' ,'Odisha' , 'India',' ', 21.4428 , 85.4079) ,
326:('Odisha' ,'Odisha' , 'India',' ', 18.6545 , 82.3809) ,
327:('Odisha' ,'Odisha' , 'India',' ', 19.0342 , 82.3481) ,
328:('Bihar' ,'Bihar' , 'India',' ', 25.4417 , 85.1357) ,
329:('Bihar' ,'Bihar' , 'India',' ', 24.4476 , 85.2919) ,
330:('Bihar' ,'Bihar' , 'India',' ', 26.6488 , 84.8056) ,
331:('North East' ,'North East' , 'India',' ', 27.2715 , 94.7101) ,
332:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.9588 , 83.602) ,
333:('Siliguri' ,'Siliguri' , 'India',' ', 23.5149 , 86.8646) ,
334:('North East' ,'North East' , 'India',' ', 25.9042 , 91.3061) ,
335:('Siliguri' ,'Siliguri' , 'India',' ', 25.2558 , 87.7105) ,
336:('Bihar' ,'Bihar' , 'India',' ', 25.93 , 87.25) ,
337:('Odisha' ,'Odisha' , 'India',' ', 20.233 , 86.284) ,
338:('Bihar' ,'Bihar' , 'India',' ', 26.345 , 84.4624) ,
339:('Bihar' ,'Bihar' , 'India',' ', 25.3 , 84.7615) ,
340:('Bihar' ,'Bihar' , 'India',' ', 24.9156 , 86.3506) ,
341:('Odisha' ,'Odisha' , 'India',' ', 18.5379 , 82.6398) ,
342:('Bihar' ,'Bihar' , 'India',' ', 25.3813 , 84.8964) ,
343:('Odisha' ,'Odisha' , 'India',' ', 21.4834 , 86.8698) ,
344:('Bihar' ,'Bihar' , 'India',' ', 25.3966 , 85.0269) ,
345:('Odisha' ,'Odisha' , 'India',' ', 20.489 , 86.308) ,
346:('North East' ,'North East' , 'India',' ', 26.7826098 , 94.4424521) ,
347:('Bihar' ,'Bihar' , 'India',' ', 26.2009 , 85.3067) ,
348:('Bihar' ,'Bihar' , 'India',' ', 25.4171 , 84.928) ,
349:('Odisha' ,'Odisha' , 'India',' ', 18.6145 , 82.5066) ,
350:('Bihar' ,'Bihar' , 'India',' ', 25.4835 , 86.7514) ,
351:('Odisha' ,'Odisha' , 'India',' ', 21.6275 , 83.6632) ,
352:('Bihar' ,'Bihar' , 'India',' ', 26.1046 , 86.3723) ,
353:('Bihar' ,'Bihar' , 'India',' ', 24.5849 , 84.7838) ,
354:('Odisha' ,'Odisha' , 'India',' ', 21.1893 , 85.6919) ,
355:('Odisha' ,'Odisha' , 'India',' ', 20.871 , 86.299) ,
356:('Bihar' ,'Bihar' , 'India',' ', 26.5398 , 85.0741) ,
357:('Odisha' ,'Odisha' , 'India',' ', 21.3255 , 86.6706) ,
358:('Odisha' ,'Odisha' , 'India',' ', 20.316 , 86.192) ,
359:('Odisha' ,'Odisha' , 'India',' ', 20.105 , 86.179) ,
360:('Odisha' ,'Odisha' , 'India',' ', 19.9788 , 85.4241) ,
361:('Bihar' ,'Bihar' , 'India',' ', 25.1441 , 84.0123) ,
362:('Bihar' ,'Bihar' , 'India',' ', 26.0636 , 85.8879) ,
363:('Bihar' ,'Bihar' , 'India',' ', 26.4727 , 84.9571) ,
364:('Bihar' ,'Bihar' , 'India',' ', 25.7632 , 84.6535) ,
365:('Bihar' ,'Bihar' , 'India',' ', 26.3142 , 86.8103) ,
366:('North East' ,'North East' , 'India',' ', 23.6325 , 91.1691) ,
367:('North East' ,'North East' , 'India',' ', 25.7069 , 93.5238) ,
368:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.1561 , 85.1415) ,
369:('Odisha' ,'Odisha' , 'India',' ', 21.2709 , 86.7023) ,
370:('Jharkhand' ,'Jharkhand' , 'India',' ', 24.2812 , 86.4928) ,
371:('Bihar' ,'Bihar' , 'India',' ', 24.8836 , 86.9603) ,
372:('Bihar' ,'Bihar' , 'India',' ', 25.8686 , 85.9387) ,
373:('Odisha' ,'Odisha' , 'India',' ', 19.9924 , 85.7594) ,
374:('Bihar' ,'Bihar' , 'India',' ', 24.7675 , 85.8373) ,
375:('Odisha' ,'Odisha' , 'India',' ', 21.4523 , 86.7101) ,
376:('North East' ,'North East' , 'India',' ', 27.8579 , 95.1127) ,
377:('Jharkhand' ,'Jharkhand' , 'India',' ', 24.0586 , 87.0557) ,
378:('Bihar' ,'Bihar' , 'India',' ', 25.1532 , 84.8303) ,
379:('Odisha' ,'Odisha' , 'India',' ', 19.4085 , 84.8682) ,
380:('Bihar' ,'Bihar' , 'India',' ', 25.9288 , 86.6859) ,
381:('Siliguri' ,'Siliguri' , 'India',' ', 23.2264 , 87.083) ,
382:('Odisha' ,'Odisha' , 'India',' ', 20.369 , 86.371) ,
383:('Odisha' ,'Odisha' , 'India',' ', 20.663 , 85.874) ,
384:('Odisha' ,'Odisha' , 'India',' ', 21.7374 , 86.524) ,
385:('Bihar' ,'Bihar' , 'India',' ', 25.6915 , 86.9942) ,
386:('Jharkhand' ,'Jharkhand' , 'India',' ', 24.0054 , 84.6723) ,
387:('Odisha' ,'Odisha' , 'India',' ', 20.6949 , 82.3115) ,
388:('Bihar' ,'Bihar' , 'India',' ', 24.6844 , 86.725) ,
389:('Bihar' ,'Bihar' , 'India',' ', 24.6055 , 84.8125) ,
390:('Odisha' ,'Odisha' , 'India',' ', 19.1012 , 84.5965) ,
391:('Bihar' ,'Bihar' , 'India',' ', 25.5803 , 84.557) ,
392:('Jharkhand' ,'Jharkhand' , 'India',' ', 24.2981 , 86.4883) ,
393:('Odisha' ,'Odisha' , 'India',' ', 21.5738 , 85.7795) ,
394:('Odisha' ,'Odisha' , 'India',' ', 20.9841 , 83.3864) ,
395:('Bihar' ,'Bihar' , 'India',' ', 25.3689 , 84.9248) ,
396:('Bihar' ,'Bihar' , 'India',' ', 25.2135 , 85.8942) ,
397:('Siliguri' ,'Siliguri' , 'India',' ', 23.5161 , 86.6786) ,
398:('Bihar' ,'Bihar' , 'India',' ', 25.3935 , 84.9964) ,
399:('North East' ,'North East' , 'India',' ', 26.5008 , 92.2768) ,
400:('Bihar' ,'Bihar' , 'India',' ', 25.0399 , 86.8436) ,
401:('Siliguri' ,'Siliguri' , 'India',' ', 26.4395 , 87.9734) ,
402:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.6506 , 86.257) ,
403:('Bihar' ,'Bihar' , 'India',' ', 25.3578 , 83.946) ,
404:('Odisha' ,'Odisha' , 'India',' ', 20.681 , 85.633) ,
405:('Bihar' ,'Bihar' , 'India',' ', 25.4951 , 84.9744) ,
406:('Odisha' ,'Odisha' , 'India',' ', 19.8099 , 85.9289) ,
407:('Odisha' ,'Odisha' , 'India',' ', 20.6142 , 86.4535) ,
408:('Bihar' ,'Bihar' , 'India',' ', 24.6377 , 85.2155) ,
409:('Odisha' ,'Odisha' , 'India',' ', 20.62 , 86.001) ,
410:('Odisha' ,'Odisha' , 'India',' ', 20.36 , 85.5387) ,
411:('Jharkhand' ,'Jharkhand' , 'India',' ', 24.1328 , 83.618) ,
412:('Bihar' ,'Bihar' , 'India',' ', 25.1243 , 86.2932) ,
413:('Bihar' ,'Bihar' , 'India',' ', 26.2131 , 87.239) ,
414:('Bihar' ,'Bihar' , 'India',' ', 24.8523 , 86.7144) ,
415:('Odisha' ,'Odisha' , 'India',' ', 19.8867 , 83.1299) ,
416:('Bihar' ,'Bihar' , 'India',' ', 26.295 , 84.7842) ,
417:('Odisha' ,'Odisha' , 'India',' ', 21.3165 , 86.9055) ,
418:('Bihar' ,'Bihar' , 'India',' ', 25.2206 , 86.1123) ,
419:('North East' ,'North East' , 'India',' ', 26.1264 , 90.8064) ,
420:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.4324 , 85.976) ,
421:('North East' ,'North East' , 'India',' ', 23.5311 , 92.5181) ,
422:('Siliguri' ,'Siliguri' , 'India',' ', 26.0618 , 87.9847) ,
423:('Siliguri' ,'Siliguri' , 'India',' ', 25.0536 , 88.58) ,
424:('Bihar' ,'Bihar' , 'India',' ', 26.6792 , 84.583) ,
425:('Bihar' ,'Bihar' , 'India',' ', 25.7668 , 84.3379) ,
426:('Bihar' ,'Bihar' , 'India',' ', 25.6957 , 86.7945) ,
427:('Bihar' ,'Bihar' , 'India',' ', 26.6377 , 85.3736) ,
428:('Bihar' ,'Bihar' , 'India',' ', 26.1378 , 86.9049) ,
429:('Odisha' ,'Odisha' , 'India',' ', 20.1375 , 86.063) ,
430:('Bihar' ,'Bihar' , 'India',' ', 25.2582 , 85.3282) ,
431:('Bihar' ,'Bihar' , 'India',' ', 24.4557 , 84.383) ,
432:('Odisha' ,'Odisha' , 'India',' ', 19.9171 , 85.6438) ,
433:('Bihar' ,'Bihar' , 'India',' ', 25.3758 , 84.881) ,
434:('Odisha' ,'Odisha' , 'India',' ', 19.2176 , 84.301) ,
435:('Odisha' ,'Odisha' , 'India',' ', 20.1037 , 82.5496) ,
436:('Bihar' ,'Bihar' , 'India',' ', 26.2507 , 86.069) ,
437:('Odisha' ,'Odisha' , 'India',' ', 20.6194 , 86.144) ,
438:('North East' ,'North East' , 'India',' ', 25.3749 , 91.6726) ,
439:('Odisha' ,'Odisha' , 'India',' ', 21.2141 , 83.8305) ,
440:('Siliguri' ,'Siliguri' , 'India',' ', 24.8141 , 87.9568) ,
441:('Bihar' ,'Bihar' , 'India',' ', 25.2167 , 85.9366) ,
442:('Odisha' ,'Odisha' , 'India',' ', 20.1578 , 85.6991) ,
443:('Bihar' ,'Bihar' , 'India',' ', 25.4216 , 87.1988) ,
444:('North East' ,'North East' , 'India',' ', 26.7247 , 94.5423) ,
445:('Siliguri' ,'Siliguri' , 'India',' ', 26.3283 , 88.5214) ,
446:('Bihar' ,'Bihar' , 'India',' ', 24.9223 , 85.2562) ,
447:('Jharkhand' ,'Jharkhand' , 'India',' ', 24.0746 , 87.0333) ,
448:('Odisha' ,'Odisha' , 'India',' ', 20.567 , 85.946) ,
449:('Odisha' ,'Odisha' , 'India',' ', 20.1262 , 84.6536) ,
450:('Bihar' ,'Bihar' , 'India',' ', 26.464 , 84.7109) ,
451:('Bihar' ,'Bihar' , 'India',' ', 25.1246 , 84.6084) ,
452:('Bihar' ,'Bihar' , 'India',' ', 24.9414 , 85.0017) ,
453:('Bihar' ,'Bihar' , 'India',' ', 25.3906 , 85.0215) ,
454:('Odisha' ,'Odisha' , 'India',' ', 20.5459 , 85.5803) ,
455:('North East' ,'North East' , 'India',' ', 24.6611 , 92.1692) ,
456:('Odisha' ,'Odisha' , 'India',' ', 20.2635 , 85.6682) ,
457:('Bihar' ,'Bihar' , 'India',' ', 25.0426 , 86.7687) ,
458:('Odisha' ,'Odisha' , 'India',' ', 20.5401 , 86.5055) ,
459:('Bihar' ,'Bihar' , 'India',' ', 25.8686 , 85.9387) ,
460:('Bihar' ,'Bihar' , 'India',' ', 25.3937 , 85.0335) ,
461:('Bihar' ,'Bihar' , 'India',' ', 24.3298 , 84.7064) ,
462:('Bihar' ,'Bihar' , 'India',' ', 25.4141 , 84.9342) ,
463:('Jharkhand' ,'Jharkhand' , 'India',' ', 24.3358 , 83.8011) ,
464:('Bihar' ,'Bihar' , 'India',' ', 25.9072 , 87.7453) ,
465:('Siliguri' ,'Siliguri' , 'India',' ', 26.2853 , 89.326) ,
466:('Odisha' ,'Odisha' , 'India',' ', 19.7239 , 85.194) ,
467:('Odisha' ,'Odisha' , 'India',' ', 21.4192 , 86.9908) ,
468:('Odisha' ,'Odisha' , 'India',' ', 21.8538 , 84.0854) ,
469:('Bihar' ,'Bihar' , 'India',' ', 25.7527 , 86.595) ,
470:('Siliguri' ,'Siliguri' , 'India',' ', 25.4249 , 87.9314) ,
471:('North East' ,'North East' , 'India',' ', 26.3012 , 90.7673) ,
472:('Bihar' ,'Bihar' , 'India',' ', 25.6597 , 87.3415) ,
473:('Odisha' ,'Odisha' , 'India',' ', 20.189 , 86.207) ,
474:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.8299 , 83.8796) ,
475:('Jharkhand' ,'Jharkhand' , 'India',' ', 24.1903 , 86.8592) ,
476:('Bihar' ,'Bihar' , 'India',' ', 25.048 , 86.0245) ,
477:('Odisha' ,'Odisha' , 'India',' ', 20.5977 , 84.7722) ,
478:('Bihar' ,'Bihar' , 'India',' ', 25.3651 , 86.9687) ,
479:('Bihar' ,'Bihar' , 'India',' ', 25.2527 , 87.0527) ,
480:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.1758 , 85.1067) ,
481:('Siliguri' ,'Siliguri' , 'India',' ', 26.6658 , 88.4707) ,
482:('Bihar' ,'Bihar' , 'India',' ', 25.2647 , 86.4394) ,
483:('Siliguri' ,'Siliguri' , 'India',' ', 25.4352 , 88.1245) ,
484:('Siliguri' ,'Siliguri' , 'India',' ', 25.0193 , 87.98) ,
485:('Bihar' ,'Bihar' , 'India',' ', 25.2615 , 84.2186) ,
486:('North East' ,'North East' , 'India',' ', 24.1461 , 93.4992) ,
487:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.2297 , 84.4841) ,
488:('Odisha' ,'Odisha' , 'India',' ', 21.2938 , 83.675) ,
489:('Bihar' ,'Bihar' , 'India',' ', 24.7553 , 86.0367) ,
490:('Bihar' ,'Bihar' , 'India',' ', 26.2255 , 87.9314) ,
491:('Bihar' ,'Bihar' , 'India',' ', 25.9264 , 87.2683) ,
492:('Odisha' ,'Odisha' , 'India',' ', 20.0588 , 85.0092) ,
493:('Odisha' ,'Odisha' , 'India',' ', 19.5579 , 84.2346) ,
494:('North East' ,'North East' , 'India',' ', 27.2897 , 95.1573) ,
495:('Odisha' ,'Odisha' , 'India',' ', 21.264 , 86.559) ,
496:('Bihar' ,'Bihar' , 'India',' ', 24.8836 , 86.465) ,
497:('Bihar' ,'Bihar' , 'India',' ', 25.6846 , 84.8277) ,
498:('Odisha' ,'Odisha' , 'India',' ', 20.3551 , 85.7996) ,
499:('North East' ,'North East' , 'India',' ', 25.7159 , 92.7557) ,
500:('Jharkhand' ,'Jharkhand' , 'India',' ', 25.0414 , 87.4453) ,
501:('Jharkhand' ,'Jharkhand' , 'India',' ', 22.4994 , 85.7364) ,
502:('Bihar' ,'Bihar' , 'India',' ', 25.4687 , 85.2051) ,
503:('Bihar' ,'Bihar' , 'India',' ', 25.3913 , 84.9391) ,
504:('Odisha' ,'Odisha' , 'India',' ', 20.6747 , 83.9284) ,
505:('Odisha' ,'Odisha' , 'India',' ', 19.1145 , 84.6076) ,
506:('Bihar' ,'Bihar' , 'India',' ', 25.5859 , 84.5514) ,
507:('Siliguri' ,'Siliguri' , 'India',' ', 25.2058 , 88.3161) ,
508:('Siliguri' ,'Siliguri' , 'India',' ', 25.0008 , 87.729) ,
509:('Bihar' ,'Bihar' , 'India',' ', 25.4553 , 84.7781) ,
510:('North East' ,'North East' , 'India',' ', 26.2784 , 90.357) ,
511:('Siliguri' ,'Siliguri' , 'India',' ', 23.1633 , 85.7785) ,
512:('Bihar' ,'Bihar' , 'India',' ', 25.8814 , 85.0245) ,
513:('Bihar' ,'Bihar' , 'India',' ', 25.6581 , 84.3661) ,
514:('Odisha' ,'Odisha' , 'India',' ', 19.3878 , 84.5457) ,
515:('Odisha' ,'Odisha' , 'India',' ', 19.9354 , 84.95) ,
516:('Bihar' ,'Bihar' , 'India',' ', 25.4646 , 85.6267) ,
517:('North East' ,'North East' , 'India',' ', 25.3782 , 91.7053) ,
518:('North East' ,'North East' , 'India',' ', 23.2975 , 91.1214) ,
519:('Odisha' ,'Odisha' , 'India',' ', 20.0401 , 84.1486) ,
520:('Odisha' ,'Odisha' , 'India',' ', 20.5168 , 85.3585) ,
521:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.9841 , 83.4987) ,
522:('Bihar' ,'Bihar' , 'India',' ', 24.6652 , 84.0218) ,
523:('Bihar' ,'Bihar' , 'India',' ', 25.2093 , 84.359) ,
524:('North East' ,'North East' , 'India',' ', 24.5958 , 92.6056) ,
525:('Jharkhand' ,'Jharkhand' , 'India',' ', 22.359 , 85.6133) ,
526:('North East' ,'North East' , 'India',' ', 26.257 , 92.2812) ,
527:('Bihar' ,'Bihar' , 'India',' ', 26.214 , 84.8557) ,
528:('North East' ,'North East' , 'India',' ', 23.7362 , 91.6356) ,
529:('Odisha' ,'Odisha' , 'India',' ', 20.994 , 86.419) ,
530:('Odisha' ,'Odisha' , 'India',' ', 21.3754 , 86.3675) ,
531:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.6168 , 86.0956) ,
532:('Jharkhand' ,'Jharkhand' , 'India',' ', 24.653 , 87.5785) ,
533:('Jharkhand' ,'Jharkhand' , 'India',' ', 24.1699 , 83.9882) ,
534:('Siliguri' ,'Siliguri' , 'India',' ', 26.485 , 88.0009) ,
535:('North East' ,'North East' , 'India',' ', 27.34 , 93.6171) ,
536:('Odisha' ,'Odisha' , 'India',' ', 21.9636 , 86.3496) ,
537:('Bihar' ,'Bihar' , 'India',' ', 25.3216 , 84.4398) ,
538:('Bihar' ,'Bihar' , 'India',' ', 24.9441 , 86.4874) ,
539:('Bihar' ,'Bihar' , 'India',' ', 25.3232 , 86.8344) ,
540:('Bihar' ,'Bihar' , 'India',' ', 25.6574 , 87.5184) ,
541:('Siliguri' ,'Siliguri' , 'India',' ', 26.3319 , 88.517) ,
542:('Siliguri' ,'Siliguri' , 'India',' ', 26.8206 , 88.2353) ,
543:('Odisha' ,'Odisha' , 'India',' ', 18.9664 , 84.5142) ,
544:('Siliguri' ,'Siliguri' , 'India',' ', 25.9021 , 87.798) ,
545:('Bihar' ,'Bihar' , 'India',' ', 25.5789 , 85.072) ,
546:('North East' ,'North East' , 'India',' ', 26.2399 , 91.4234) ,
547:('North East' ,'North East' , 'India',' ', 25.839 , 90.1038) ,
548:('Bihar' ,'Bihar' , 'India',' ', 25.0205 , 84.0615) ,
549:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.4538 , 84.7882) ,
550:('Bihar' ,'Bihar' , 'India',' ', 24.587 , 84.8022) ,
551:('Siliguri' ,'Siliguri' , 'India',' ', 25.878 , 87.7236) ,
552:('Odisha' ,'Odisha' , 'India',' ', 21.3152 , 86.9052) ,
553:('Bihar' ,'Bihar' , 'India',' ', 25.4094 , 84.9364) ,
554:('Odisha' ,'Odisha' , 'India',' ', 19.6114 , 84.5932) ,
555:('North East' ,'North East' , 'India',' ', 25.9025 , 91.3915) ,
556:('Jharkhand' ,'Jharkhand' , 'India',' ', 22.5665 , 85.9974) ,
557:('Siliguri' ,'Siliguri' , 'India',' ', 22.8665 , 87.1114) ,
558:('Odisha' ,'Odisha' , 'India',' ', 20.13 , 85.7259) ,
559:('Bihar' ,'Bihar' , 'India',' ', 26.6173 , 84.9305) ,
560:('Siliguri' ,'Siliguri' , 'India',' ', 23.4529 , 86.8894) ,
561:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.1827 , 85.1064) ,
562:('Bihar' ,'Bihar' , 'India',' ', 25.0741 , 85.8222) ,
563:('Siliguri' ,'Siliguri' , 'India',' ', 22.9616 , 86.7901) ,
564:('Jharkhand' ,'Jharkhand' , 'India',' ', 24.313 , 86.4484) ,
565:('Bihar' ,'Bihar' , 'India',' ', 26.4512 , 85.5196) ,
566:('Jharkhand' ,'Jharkhand' , 'India',' ', 22.3593 , 85.608) ,
567:('Bihar' ,'Bihar' , 'India',' ', 24.7442 , 83.4991) ,
568:('North East' ,'North East' , 'India',' ', 24.672 , 92.1679) ,
569:('Odisha' ,'Odisha' , 'India',' ', 22.0288 , 84.7231) ,
570:('Bihar' ,'Bihar' , 'India',' ', 24.7135 , 86.4263) ,
571:('Odisha' ,'Odisha' , 'India',' ', 20.711 , 86.236) ,
572:('North East' ,'North East' , 'India',' ', 26.203 , 90.0625) ,
573:('Bihar' ,'Bihar' , 'India',' ', 25.0912 , 87.2225) ,
574:('Bihar' ,'Bihar' , 'India',' ', 24.6012 , 85.0439) ,
575:('North East' ,'North East' , 'India',' ', 26.0324 , 90.0344) ,
576:('Odisha' ,'Odisha' , 'India',' ', 20.5 , 86.092) ,
577:('North East' ,'North East' , 'India',' ', 26.9782 , 94.7073) ,
578:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.7889 , 85.8359) ,
579:('Odisha' ,'Odisha' , 'India',' ', 21.016 , 85.922) ,
580:('Odisha' ,'Odisha' , 'India',' ', 17.5406 , 83.1084) ,
581:('Bihar' ,'Bihar' , 'India',' ', 25.0026 , 85.3264) ,
582:('North East' ,'North East' , 'India',' ', 25.9078 , 91.6523) ,
583:('Siliguri' ,'Siliguri' , 'India',' ', 25.0179 , 87.9943) ,
584:('Bihar' ,'Bihar' , 'India',' ', 26.3566 , 84.8638) ,
585:('North East' ,'North East' , 'India',' ', 26.9135 , 86.4463) ,
586:('North East' ,'North East' , 'India',' ', 25.7118 , 93.5331) ,
587:('North East' ,'North East' , 'India',' ', 26.6402 , 92.6057) ,
588:('Odisha' ,'Odisha' , 'India',' ', 19.1084 , 84.622) ,
589:('Odisha' ,'Odisha' , 'India',' ', 21.0826 , 86.2649) ,
590:('Odisha' ,'Odisha' , 'India',' ', 20.1101 , 85.6509) ,
591:('Odisha' ,'Odisha' , 'India',' ', 18.4194 , 82.3918) ,
592:('Odisha' ,'Odisha' , 'India',' ', 19.9346 , 85.3456) ,
593:('Odisha' ,'Odisha' , 'India',' ', 20.2156 , 85.7999) ,
594:('Odisha' ,'Odisha' , 'India',' ', 20.0042 , 86.1997) ,
595:('Bihar' ,'Bihar' , 'India',' ', 24.6149 , 84.4349) ,
596:('Siliguri' ,'Siliguri' , 'India',' ', 26.3642 , 88.6135) ,
597:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.4522 , 85.1609) ,
598:('Bihar' ,'Bihar' , 'India',' ', 25.4118 , 84.8754) ,
599:('Odisha' ,'Odisha' , 'India',' ', 20.243 , 83.158) ,
600:('Odisha' ,'Odisha' , 'India',' ', 20.0884 , 85.1774) ,
601:('Odisha' ,'Odisha' , 'India',' ', 19.9755 , 85.0775) ,
602:('Jharkhand' ,'Jharkhand' , 'India',' ', 22.6451 , 86.0338) ,
603:('Odisha' ,'Odisha' , 'India',' ', 20.4294 , 85.1015) ,
604:('Bihar' ,'Bihar' , 'India',' ', 24.8411 , 83.4133) ,
605:('Odisha' ,'Odisha' , 'India',' ', 19.3299 , 84.5096) ,
606:('Odisha' ,'Odisha' , 'India',' ', 20.832 , 85.553) ,
607:('Bihar' ,'Bihar' , 'India',' ', 26.9523 , 84.132) ,
608:('Odisha' ,'Odisha' , 'India',' ', 19.8974 , 82.431) ,
609:('Jharkhand' ,'Jharkhand' , 'India',' ', 22.4811 , 85.411) ,
610:('North East' ,'North East' , 'India',' ', 25.9948 , 93.6631) ,
611:('Bihar' ,'Bihar' , 'India',' ', 25.3815 , 83.781) ,
612:('North East' ,'North East' , 'India',' ', 27.0910662 , 95.4694606) ,
613:('Bihar' ,'Bihar' , 'India',' ', 25.9851 , 87.5762) ,
614:('Odisha' ,'Odisha' , 'India',' ', 20.76 , 86.165) ,
615:('Jharkhand' ,'Jharkhand' , 'India',' ', 22.3942 , 86.268) ,
616:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.6506 , 86.257) ,
617:('Bihar' ,'Bihar' , 'India',' ', 26.6488 , 84.8055) ,
618:('North East' ,'North East' , 'India',' ', 25.857 , 91.2029) ,
619:('North East' ,'North East' , 'India',' ', 25.9291 , 92.0039) ,
620:('Bihar' ,'Bihar' , 'India',' ', 24.6095 , 83.882) ,
621:('Odisha' ,'Odisha' , 'India',' ', 20.8043 , 85.3893) ,
622:('Odisha' ,'Odisha' , 'India',' ', 19.6607 , 82.7411) ,
623:('Odisha' ,'Odisha' , 'India',' ', 20.49 , 86.45) ,
624:('Bihar' ,'Bihar' , 'India',' ', 25.0256 , 83.7027) ,
625:('Siliguri' ,'Siliguri' , 'India',' ', 26.4915 , 88.2609) ,
626:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.6288 , 86.2867) ,
627:('Bihar' ,'Bihar' , 'India',' ', 26.6248 , 84.2863) ,
628:('Odisha' ,'Odisha' , 'India',' ', 20.419 , 86.28) ,
629:('Odisha' ,'Odisha' , 'India',' ', 20.2221 , 86.056) ,
630:('Odisha' ,'Odisha' , 'India',' ', 19.0497 , 83.2328) ,
631:('Siliguri' ,'Siliguri' , 'India',' ', 26.5136 , 88.9411) ,
632:('Odisha' ,'Odisha' , 'India',' ', 19.8375 , 85.5593) ,
633:('Bihar' ,'Bihar' , 'India',' ', 26.3465 , 85.433) ,
634:('Siliguri' ,'Siliguri' , 'India',' ', 24.7331 , 87.8354) ,
635:('Bihar' ,'Bihar' , 'India',' ', 26.391 , 85.2914) ,
636:('Bihar' ,'Bihar' , 'India',' ', 25.0099 , 87.173) ,
637:('Bihar' ,'Bihar' , 'India',' ', 25.9112 , 85.1843) ,
638:('North East' ,'North East' , 'India',' ', 24.1749 , 91.9582) ,
639:('Siliguri' ,'Siliguri' , 'India',' ', 26.3261 , 88.9956) ,
640:('Bihar' ,'Bihar' , 'India',' ', 25.7581 , 85.2354) ,
641:('Siliguri' ,'Siliguri' , 'India',' ', 26.1139 , 89.4674) ,
642:('Bihar' ,'Bihar' , 'India',' ', 25.2675 , 85.5009) ,
643:('Siliguri' ,'Siliguri' , 'India',' ', 23.4101 , 86.9219) ,
644:('Odisha' ,'Odisha' , 'India',' ', 19.6047 , 85.6083) ,
645:('Odisha' ,'Odisha' , 'India',' ', 20.582 , 86.272) ,
646:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.22 , 85.0265) ,
647:('Siliguri' ,'Siliguri' , 'India',' ', 24.661 , 87.8176) ,
648:('Odisha' ,'Odisha' , 'India',' ', 19.4393 , 84.7957) ,
649:('North East' ,'North East' , 'India',' ', 25.8683 , 92.778) ,
650:('North East' ,'North East' , 'India',' ', 26.9107 , 93.6237) ,
651:('Siliguri' ,'Siliguri' , 'India',' ', 23.0269 , 86.8847) ,
652:('North East' ,'North East' , 'India',' ', 27.9392 , 95.6417) ,
653:('Bihar' ,'Bihar' , 'India',' ', 26.3909 , 85.3009) ,
654:('North East' ,'North East' , 'India',' ', 26.4603 , 94.1172) ,
655:('Odisha' ,'Odisha' , 'India',' ', 18.993 , 84.415) ,
656:('Odisha' ,'Odisha' , 'India',' ', 19.7712 , 86.0664) ,
657:('Jharkhand' ,'Jharkhand' , 'India',' ', 24.0663 , 86.4455) ,
658:('Bihar' ,'Bihar' , 'India',' ', 25.0458 , 86.7988) ,
659:('Jharkhand' ,'Jharkhand' , 'India',' ', 25.0038 , 87.1687) ,
660:('Odisha' ,'Odisha' , 'India',' ', 19.8463 , 85.6271) ,
661:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.1885 , 85.0957) ,
662:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.231 , 84.4768) ,
663:('Jharkhand' ,'Jharkhand' , 'India',' ', 22.6492 , 85.9953) ,
664:('Bihar' ,'Bihar' , 'India',' ', 24.7549 , 83.8245) ,
665:('Siliguri' ,'Siliguri' , 'India',' ', 24.9167 , 87.8025) ,
666:('North East' ,'North East' , 'India',' ', 26.2557 , 92.28) ,
667:('Bihar' ,'Bihar' , 'India',' ', 25.3343 , 84.9756) ,
668:('Odisha' ,'Odisha' , 'India',' ', 20.525 , 86.128) ,
669:('Siliguri' ,'Siliguri' , 'India',' ', 25.5631 , 87.7606) ,
670:('Odisha' ,'Odisha' , 'India',' ', 18.9913 , 84.052) ,
671:('Jharkhand' ,'Jharkhand' , 'India',' ', 24.8363 , 87.1142) ,
672:('Odisha' ,'Odisha' , 'India',' ', 19.1753 , 84.7822) ,
673:('Odisha' ,'Odisha' , 'India',' ', 18.7638 , 82.2014) ,
674:('Odisha' ,'Odisha' , 'India',' ', 20.289 , 83.284) ,
675:('Odisha' ,'Odisha' , 'India',' ', 21.5283 , 86.5447) ,
676:('North East' ,'North East' , 'India',' ', 24.1189 , 91.7963) ,
677:('North East' ,'North East' , 'India',' ', 27.3716 , 95.3554) ,
678:('Jharkhand' ,'Jharkhand' , 'India',' ', 22.5662 , 85.9753) ,
679:('North East' ,'North East' , 'India',' ', 23.1105 , 91.3632) ,
680:('Siliguri' ,'Siliguri' , 'India',' ', 25.1779 , 88.0644) ,
681:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.5423 , 85.9339) ,
682:('Bihar' ,'Bihar' , 'India',' ', 24.3582 , 86.1964) ,
683:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.1146 , 85.1881) ,
684:('Odisha' ,'Odisha' , 'India',' ', 21.5497 , 84.1641) ,
685:('Siliguri' ,'Siliguri' , 'India',' ', 26.1425 , 89.0188) ,
686:('Odisha' ,'Odisha' , 'India',' ', 19.1725 , 84.68) ,
687:('Odisha' ,'Odisha' , 'India',' ', 19.8492 , 85.2649) ,
688:('Odisha' ,'Odisha' , 'India',' ', 20.1614 , 84.996) ,
689:('Odisha' ,'Odisha' , 'India',' ', 20.669 , 86.564) ,
690:('North East' ,'North East' , 'India',' ', 25.5127 , 89.698) ,
691:('North East' ,'North East' , 'India',' ', 23.3236 , 91.4589) ,
692:('Jharkhand' ,'Jharkhand' , 'India',' ', 22.0765 , 86.5237) ,
693:('Odisha' ,'Odisha' , 'India',' ', 20.3149 , 85.626) ,
694:('Bihar' ,'Bihar' , 'India',' ', 24.6096 , 86.8214) ,
695:('Odisha' ,'Odisha' , 'India',' ', 20.6075 , 83.8924) ,
696:('Bihar' ,'Bihar' , 'India',' ', 24.8495 , 83.4153) ,
697:('Odisha' ,'Odisha' , 'India',' ', 19.7947 , 85.2761) ,
698:('North East' ,'North East' , 'India',' ', 25.9868 , 91.469) ,
699:('North East' ,'North East' , 'India',' ', 26.2489 , 91.2442) ,
700:('Odisha' ,'Odisha' , 'India',' ', 20.5887 , 86.7038) ,
701:('Bihar' ,'Bihar' , 'India',' ', 24.5694 , 86.183) ,
702:('Jharkhand' ,'Jharkhand' , 'India',' ', 22.3884 , 84.2922) ,
703:('North East' ,'North East' , 'India',' ', 23.8573 , 91.4092) ,
704:('Odisha' ,'Odisha' , 'India',' ', 20.3269 , 85.7213) ,
705:('Bihar' ,'Bihar' , 'India',' ', 26.4392 , 84.7037) ,
706:('North East' ,'North East' , 'India',' ', 25.2559 , 96.4916) ,
707:('Jharkhand' ,'Jharkhand' , 'India',' ', 24.2081 , 85.7813) ,
708:('North East' ,'North East' , 'India',' ', 26.5426 , 91.8968) ,
709:('Siliguri' ,'Siliguri' , 'India',' ', 26.118 , 88.0908) ,
710:('Jharkhand' ,'Jharkhand' , 'India',' ', 22.4161 , 85.9581) ,
711:('Odisha' ,'Odisha' , 'India',' ', 20.5097 , 83.3082) ,
712:('Bihar' ,'Bihar' , 'India',' ', 25.2298 , 85.9311) ,
713:('Odisha' ,'Odisha' , 'India',' ', 19.4111 , 82.5086) ,
714:('Odisha' ,'Odisha' , 'India',' ', 19.8029 , 85.9895) ,
715:('Odisha' ,'Odisha' , 'India',' ', 20.1718 , 85.6393) ,
716:('Siliguri' ,'Siliguri' , 'India',' ', 25.1326 , 88.5624) ,
717:('Siliguri' ,'Siliguri' , 'India',' ', 22.8783 , 87.1395) ,
718:('Odisha' ,'Odisha' , 'India',' ', 19.0512 , 84.5681) ,
719:('North East' ,'North East' , 'India',' ', 27.4671 , 95.6575) ,
720:('Bihar' ,'Bihar' , 'India',' ', 26.3123 , 85.0897) ,
721:('Odisha' ,'Odisha' , 'India',' ', 19.417 , 82.6841) ,
722:('Odisha' ,'Odisha' , 'India',' ', 21.2303 , 84.9842) ,
723:('Siliguri' ,'Siliguri' , 'India',' ', 25.4083 , 87.9225) ,
724:('Bihar' ,'Bihar' , 'India',' ', 25.1769 , 85.3401) ,
725:('Siliguri' ,'Siliguri' , 'India',' ', 25.9481 , 87.8343) ,
726:('Bihar' ,'Bihar' , 'India',' ', 24.972 , 83.421) ,
727:('Jharkhand' ,'Jharkhand' , 'India',' ', 24.1039 , 85.2226) ,
728:('Siliguri' ,'Siliguri' , 'India',' ', 22.9472 , 87.3295) ,
729:('North East' ,'North East' , 'India',' ', 26.0517 , 91.3313) ,
730:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.5622641 , 85.8020693) ,
731:('Siliguri' ,'Siliguri' , 'India',' ', 24.9288 , 88.1695) ,
732:('Bihar' ,'Bihar' , 'India',' ', 26.8111 , 84.0327) ,
733:('Jharkhand' ,'Jharkhand' , 'India',' ', 24.5034 , 87.5122) ,
734:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.6019 , 86.4487) ,
735:('Bihar' ,'Bihar' , 'India',' ', 24.4352 , 83.7635) ,
736:('North East' ,'North East' , 'India',' ', 24.1793 , 91.9642) ,
737:('Bihar' ,'Bihar' , 'India',' ', 24.8969 , 83.9179) ,
738:('Odisha' ,'Odisha' , 'India',' ', 20.4738 , 83.3228) ,
739:('Siliguri' ,'Siliguri' , 'India',' ', 26.676 , 88.7767) ,
740:('Bihar' ,'Bihar' , 'India',' ', 25.2553 , 83.1945) ,
741:('Odisha' ,'Odisha' , 'India',' ', 20.5036 , 82.9467) ,
742:('Odisha' ,'Odisha' , 'India',' ', 21.0647 , 84.9958) ,
743:('Odisha' ,'Odisha' , 'India',' ', 19.9941 , 85.3896) ,
744:('Bihar' ,'Bihar' , 'India',' ', 26.3176 , 84.6439) ,
745:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.5956531 , 86.2303859) ,
746:('Odisha' ,'Odisha' , 'India',' ', 19.9399 , 85.6393) ,
747:('North East' ,'North East' , 'India',' ', 26.5143 , 93.7699) ,
748:('Jharkhand' ,'Jharkhand' , 'India',' ', 24.1846 , 85.0563) ,
749:('Siliguri' ,'Siliguri' , 'India',' ', 26.2695 , 89.5237) ,
750:('Odisha' ,'Odisha' , 'India',' ', 18.9808 , 82.8954) ,
751:('Siliguri' ,'Siliguri' , 'India',' ', 23.3344 , 86.4687) ,
752:('Odisha' ,'Odisha' , 'India',' ', 18.851 , 82.3511) ,
753:('Bihar' ,'Bihar' , 'India',' ', 26.125 , 87.7566) ,
754:('Odisha' ,'Odisha' , 'India',' ', 19.243 , 84.0625) ,
755:('Odisha' ,'Odisha' , 'India',' ', 20.836 , 85.921) ,
756:('Bihar' ,'Bihar' , 'India',' ', 26.4055 , 84.6317) ,
757:('North East' ,'North East' , 'India',' ', 25.9686 , 90.425) ,
758:('North East' ,'North East' , 'India',' ', 27.0109 , 92.2678) ,
759:('North East' ,'North East' , 'India',' ', 26.6907 , 93.4168) ,
760:('Odisha' ,'Odisha' , 'India',' ', 19.526 , 84.9925) ,
761:('Odisha' ,'Odisha' , 'India',' ', 22.2086 , 84.5325) ,
762:('Odisha' ,'Odisha' , 'India',' ', 20.1441 , 82.7119) ,
763:('Bihar' ,'Bihar' , 'India',' ', 26.551 , 85.2255) ,
764:('Odisha' ,'Odisha' , 'India',' ', 20.0148 , 85.8827) ,
765:('Siliguri' ,'Siliguri' , 'India',' ', 23.6082 , 86.6892) ,
766:('North East' ,'North East' , 'India',' ', 26.385 , 92.784) ,
767:('North East' ,'North East' , 'India',' ', 27.7407 , 95.9728) ,
768:('Odisha' ,'Odisha' , 'India',' ', 19.6323 , 83.2575) ,
769:('Siliguri' ,'Siliguri' , 'India',' ', 26.2378 , 88.06) ,
770:('Bihar' ,'Bihar' , 'India',' ', 25.5694 , 84.8341) ,
771:('North East' ,'North East' , 'India',' ', 25.8581 , 92.9938) ,
772:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.6238 , 86.2148) ,
773:('North East' ,'North East' , 'India',' ', 27.1658 , 95.4313) ,
774:('Siliguri' ,'Siliguri' , 'India',' ', 26.2078 , 89.1829) ,
775:('Odisha' ,'Odisha' , 'India',' ', 20.4484 , 83.5436) ,
776:('Odisha' ,'Odisha' , 'India',' ', 21.7068 , 85.045) ,
777:('Odisha' ,'Odisha' , 'India',' ', 18.6045 , 83.8712) ,
778:('North East' ,'North East' , 'India',' ', 25.9472 , 91.538) ,
779:('North East' ,'North East' , 'India',' ', 24.67 , 92.3579) ,
780:('North East' ,'North East' , 'India',' ', 26.7888 , 95.3052) ,
781:('Odisha' ,'Odisha' , 'India',' ', 19.7874 , 85.8162) ,
782:('Odisha' ,'Odisha' , 'India',' ', 20.1616 , 84.7829) ,
783:('North East' ,'North East' , 'India',' ', 24.485 , 92.3575) ,
784:('Odisha' ,'Odisha' , 'India',' ', 19.7502 , 85.9848) ,
785:('North East' ,'North East' , 'India',' ', 24.2958 , 93.566) ,
786:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.6292 , 84.5901) ,
787:('Odisha' ,'Odisha' , 'India',' ', 19.1383 , 84.731) ,
788:('Odisha' ,'Odisha' , 'India',' ', 19.077 , 82.0396) ,
789:('North East' ,'North East' , 'India',' ', 27.4747 , 95.364) ,
790:('North East' ,'North East' , 'India',' ', 25.0709 , 93.8244) ,
791:('Odisha' ,'Odisha' , 'India',' ', 18.959 , 84.3716) ,
792:('North East' ,'North East' , 'India',' ', 26.2476 , 91.8292) ,
793:('Bihar' ,'Bihar' , 'India',' ', 25.8283 , 87.5069) ,
794:('Odisha' ,'Odisha' , 'India',' ', 19.3196 , 84.892) ,
795:('Odisha' ,'Odisha' , 'India',' ', 19.4584 , 82.6022) ,
796:('Odisha' ,'Odisha' , 'India',' ', 20.1156 , 85.6485) ,
797:('Odisha' ,'Odisha' , 'India',' ', 21.1005 , 83.1974) ,
798:('Odisha' ,'Odisha' , 'India',' ', 19.5437 , 85.3678) ,
799:('North East' ,'North East' , 'India',' ', 25.3363 , 89.66) ,
800:('Siliguri' ,'Siliguri' , 'India',' ', 26.1821 , 88.1131) ,
801:('Siliguri' ,'Siliguri' , 'India',' ', 23.4482 , 87.0498) ,
802:('Odisha' ,'Odisha' , 'India',' ', 20.51 , 86.025) ,
803:('Odisha' ,'Odisha' , 'India',' ', 21.5668 , 83.8107) ,
804:('Siliguri' ,'Siliguri' , 'India',' ', 26.5713 , 88.2429) ,
805:('Odisha' ,'Odisha' , 'India',' ', 20.668 , 85.916) ,
806:('Odisha' ,'Odisha' , 'India',' ', 19.6739 , 84.4355) ,
807:('Jharkhand' ,'Jharkhand' , 'India',' ', 24.2824 , 86.5056) ,
808:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.9614 , 85.433) ,
809:('Bihar' ,'Bihar' , 'India',' ', 25.1632 , 84.9332) ,
810:('Bihar' ,'Bihar' , 'India',' ', 26.4626 , 84.7238) ,
811:('Siliguri' ,'Siliguri' , 'India',' ', 26.3868 , 88.8043) ,
812:('Jharkhand' ,'Jharkhand' , 'India',' ', 24.2642 , 85.3981) ,
813:('Odisha' ,'Odisha' , 'India',' ', 20.0888 , 85.0917) ,
814:('North East' ,'North East' , 'India',' ', 23.0511 , 91.251) ,
815:('Odisha' ,'Odisha' , 'India',' ', 20.668 , 85.916) ,
816:('Bihar' ,'Bihar' , 'India',' ', 25.608 , 87.4153) ,
817:('Odisha' ,'Odisha' , 'India',' ', 20.5334 , 83.4747) ,
818:('North East' ,'North East' , 'India',' ', 26.4429 , 93.3943) ,
819:('North East' ,'North East' , 'India',' ', 22.6836 , 92.5474) ,
820:('Bihar' ,'Bihar' , 'India',' ', 25.5647 , 84.8914) ,
821:('Siliguri' ,'Siliguri' , 'India',' ', 23.4839 , 86.657) ,
822:('Odisha' ,'Odisha' , 'India',' ', 19.2424 , 82.6269) ,
823:('Odisha' ,'Odisha' , 'India',' ', 19.1231 , 84.5876) ,
824:('Bihar' ,'Bihar' , 'India',' ', 25.0888 , 86.3759) ,
825:('North East' ,'North East' , 'India',' ', 25.6449 , 93.238) ,
826:('Odisha' ,'Odisha' , 'India',' ', 19.0385 , 83.621) ,
827:('Odisha' ,'Odisha' , 'India',' ', 20.0255 , 83.9282) ,
828:('Jharkhand' ,'Jharkhand' , 'India',' ', 22.7588 , 85.7656) ,
829:('Odisha' ,'Odisha' , 'India',' ', 18.6446 , 83.9819) ,
830:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.547 , 84.2958) ,
831:('North East' ,'North East' , 'India',' ', 26.9253 , 94.5489) ,
832:('Jharkhand' ,'Jharkhand' , 'India',' ', 24.7947 , 87.5433) ,
833:('Bihar' ,'Bihar' , 'India',' ', 26.5408 , 85.0741) ,
834:('Odisha' ,'Odisha' , 'India',' ', 20.1243 , 85.2719) ,
835:('North East' ,'North East' , 'India',' ', 27.1391 , 95.1176) ,
836:('North East' ,'North East' , 'India',' ', 27.9763 , 94.6002) ,
837:('North East' ,'North East' , 'India',' ', 26.8143 , 93.7164) ,
838:('Odisha' ,'Odisha' , 'India',' ', 19.8244 , 83.6528) ,
839:('Odisha' ,'Odisha' , 'India',' ', 19.606 , 85.6365) ,
840:('Odisha' ,'Odisha' , 'India',' ', 19.3082 , 83.3076) ,
841:('North East' ,'North East' , 'India',' ', 27.3946 , 91.6715) ,
842:('Odisha' ,'Odisha' , 'India',' ', 19.7681 , 84.5759) ,
843:('Siliguri' ,'Siliguri' , 'India',' ', 27.3012 , 88.3349) ,
844:('North East' ,'North East' , 'India',' ', 26.9105 , 93.514) ,
845:('Odisha' ,'Odisha' , 'India',' ', 19.8379 , 85.6801) ,
846:('Odisha' ,'Odisha' , 'India',' ', 19.6225 , 84.5968) ,
847:('North East' ,'North East' , 'India',' ', 25.9576 , 91.6384) ,
848:('North East' ,'North East' , 'India',' ', 27.7043 , 93.1501) ,
849:('Bihar' ,'Bihar' , 'India',' ', 25.2718 , 85.182) ,
850:('Odisha' ,'Odisha' , 'India',' ', 20.2347 , 85.6819) ,
851:('Odisha' ,'Odisha' , 'India',' ', 20.6483 , 83.684) ,
852:('North East' ,'North East' , 'India',' ', 26.6137 , 94.3215) ,
853:('North East' ,'North East' , 'India',' ', 22.8118 , 91.5204) ,
854:('North East' ,'North East' , 'India',' ', 25.5536 , 92.9721) ,
855:('North East' ,'North East' , 'India',' ', 23.7636 , 91.1606) ,
856:('North East' ,'North East' , 'India',' ', 23.6446 , 91.445) ,
857:('Siliguri' ,'Siliguri' , 'India',' ', 26.5072 , 88.2224) ,
858:('Jharkhand' ,'Jharkhand' , 'India',' ', 22.8335 , 84.3625) ,
859:('Siliguri' ,'Siliguri' , 'India',' ', 23.4525 , 86.6133) ,
860:('North East' ,'North East' , 'India',' ', 25.4241 , 91.6831) ,
861:('North East' ,'North East' , 'India',' ', 26.6317 , 94.219) ,
862:('North East' ,'North East' , 'India',' ', 26.1617 , 91.4605) ,
863:('Bihar' ,'Bihar' , 'India',' ', 24.9306 , 84.4604) ,
864:('North East' ,'North East' , 'India',' ', 23.7974 , 91.7928) ,
865:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.5833 , 86.2476) ,
866:('Odisha' ,'Odisha' , 'India',' ', 20.2794 , 85.6366) ,
867:('Odisha' ,'Odisha' , 'India',' ', 18.9005 , 83.848) ,
868:('North East' ,'North East' , 'India',' ', 25.3131 , 90.0169) ,
869:('North East' ,'North East' , 'India',' ', 25.3582 , 91.7122) ,
870:('North East' ,'North East' , 'India',' ', 25.375 , 91.6765) ,
871:('North East' ,'North East' , 'India',' ', 25.44 , 88.01) ,
872:('North East' ,'North East' , 'India',' ', 26.0774 , 91.061) ,
873:('Odisha' ,'Odisha' , 'India',' ', 19.659 , 85.185) ,
874:('North East' ,'North East' , 'India',' ', 27.7862879 , 94.0205209) ,
875:('North East' ,'North East' , 'India',' ', 26.1249 , 94.32) ,
876:('Siliguri' ,'Siliguri' , 'India',' ', 26.6937 , 87.9887) ,
877:('Odisha' ,'Odisha' , 'India',' ', 22.0338 , 84.6299) ,
878:('North East' ,'North East' , 'India',' ', 26.9017 , 93.4267) ,
879:('North East' ,'North East' , 'India',' ', 25.7273 , 93.5336) ,
880:('Odisha' ,'Odisha' , 'India',' ', 18.678 , 82.969) ,
881:('Odisha' ,'Odisha' , 'India',' ', 21.7257 , 86.5344) ,
882:('Odisha' ,'Odisha' , 'India',' ', 20.1392 , 85.6275) ,
883:('Bihar' ,'Bihar' , 'India',' ', 25.4737 , 85.635) ,
884:('Odisha' ,'Odisha' , 'India',' ', 18.9422 , 82.1199) ,
885:('Odisha' ,'Odisha' , 'India',' ', 19.3583 , 85.0433) ,
886:('North East' ,'North East' , 'India',' ', 24.3529 , 92.4074) ,
887:('Odisha' ,'Odisha' , 'India',' ', 20.1122 , 86.41) ,
888:('North East' ,'North East' , 'India',' ', 27.1576 , 92.8389) ,
889:('Jharkhand' ,'Jharkhand' , 'India',' ', 24.2376 , 84.6132) ,
890:('North East' ,'North East' , 'India',' ', 25.3144 , 90.014) ,
891:('North East' ,'North East' , 'India',' ', 26.3012 , 90.7683) ,
892:('Odisha' ,'Odisha' , 'India',' ', 22.0059 , 84.6337) ,
893:('Odisha' ,'Odisha' , 'India',' ', 18.5539 , 82.2335) ,
894:('Siliguri' ,'Siliguri' , 'India',' ', 23.2264 , 87.083) ,
895:('North East' ,'North East' , 'India',' ', 27.3946 , 94.5238) ,
896:('Odisha' ,'Odisha' , 'India',' ', 20.3067 , 82.4718) ,
897:('Odisha' ,'Odisha' , 'India',' ', 21.0561 , 84.3381) ,
898:('Odisha' ,'Odisha' , 'India',' ', 19.1404 , 82.3142) ,
899:('North East' ,'North East' , 'India',' ', 25.7119 , 91.6779) ,
900:('Bihar' ,'Bihar' , 'India',' ', 26.8239 , 84.4845) ,
901:('Odisha' ,'Odisha' , 'India',' ', 20.0458 , 85.5633) ,
902:('North East' ,'North East' , 'India',' ', 26.746 , 93.9653) ,
903:('North East' ,'North East' , 'India',' ', 26.3076 , 93.7695) ,
904:('North East' ,'North East' , 'India',' ', 26.2766 , 91.0696) ,
905:('North East' ,'North East' , 'India',' ', 26.2361 , 89.7773) ,
906:('North East' ,'North East' , 'India',' ', 24.0209 , 92.4811) ,
907:('Siliguri' ,'Siliguri' , 'India',' ', 26.5271012 , 88.1952861) ,
908:('Siliguri' ,'Siliguri' , 'India',' ', 26.6787 , 88.078) ,
909:('North East' ,'North East' , 'India',' ', 27.2865 , 96.0036) ,
910:('Bihar' ,'Bihar' , 'India',' ', 25.3789 , 84.8742) ,
911:('Odisha' ,'Odisha' , 'India',' ', 22.0332 , 84.6248) ,
912:('North East' ,'North East' , 'India',' ', 24.3414 , 92.6332) ,
913:('Siliguri' ,'Siliguri' , 'India',' ', 26.7979 , 87.967) ,
914:('Odisha' ,'Odisha' , 'India',' ', 20.742 , 85.921) ,
915:('North East' ,'North East' , 'India',' ', 27.6215058 , 95.0358035) ,
916:('North East' ,'North East' , 'India',' ', 27.0477 , 93.2913) ,
917:('North East' ,'North East' , 'India',' ', 25.5375 , 94.2388) ,
918:('Bihar' ,'Bihar' , 'India',' ', 25.004 , 85.3073) ,
919:('North East' ,'North East' , 'India',' ', 26.3104 , 90.9766) ,
920:('North East' ,'North East' , 'India',' ', 25.8967 , 94.0547) ,
921:('Bihar' ,'Bihar' , 'India',' ', 25.4931 , 85.0263) ,
922:('Odisha' ,'Odisha' , 'India',' ', 21.41 , 86.9911) ,
923:('North East' ,'North East' , 'India',' ', 23.5107 , 92.52) ,
924:('North East' ,'North East' , 'India',' ', 24.4113 , 92.1218) ,
925:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.0687 , 85.1964) ,
926:('North East' ,'North East' , 'India',' ', 24.4323 , 93.8086) ,
927:('North East' ,'North East' , 'India',' ', 23.277 , 93.1206) ,
928:('North East' ,'North East' , 'India',' ', 24.8847 , 94.16) ,
929:('North East' ,'North East' , 'India',' ', 27.2945 , 94.9905) ,
930:('Odisha' ,'Odisha' , 'India',' ', 19.9483 , 85.4012) ,
931:('Jharkhand' ,'Jharkhand' , 'India',' ', 24.2676805 , 85.3933645) ,
932:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.1969 , 85.1788) ,
933:('Odisha' ,'Odisha' , 'India',' ', 19.3358 , 84.89) ,
934:('North East' ,'North East' , 'India',' ', 24.9673 , 92.8129) ,
935:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.565 , 85.7976) ,
936:('North East' ,'North East' , 'India',' ', 28.4266 , 94.8191) ,
937:('Odisha' ,'Odisha' , 'India',' ', 20.0602 , 86.0011) ,
938:('Odisha' ,'Odisha' , 'India',' ', 20.2421 , 85.7064) ,
939:('Odisha' ,'Odisha' , 'India',' ', 20.6531 , 86.1277) ,
940:('Siliguri' ,'Siliguri' , 'India',' ', 26.487 , 89.2223) ,
941:('Bihar' ,'Bihar' , 'India',' ', 24.7453 , 85.501) ,
942:('Siliguri' ,'Siliguri' , 'India',' ', 23.617 , 86.781) ,
943:('Odisha' ,'Odisha' , 'India',' ', 19.4224 , 84.7355) ,
944:('Odisha' ,'Odisha' , 'India',' ', 21.2998 , 86.7158) ,
945:('Odisha' ,'Odisha' , 'India',' ', 20.6183 , 84.9052) ,
946:('Odisha' ,'Odisha' , 'India',' ', 21.912 , 85.1921) ,
947:('North East' ,'North East' , 'India',' ', 26.8868 , 93.4119) ,
948:('Odisha' ,'Odisha' , 'India',' ', 18.8988 , 84.436) ,
949:('Odisha' ,'Odisha' , 'India',' ', 19.6045 , 85.627) ,
950:('North East' ,'North East' , 'India',' ', 26.3076 , 93.77) ,
951:('Siliguri' ,'Siliguri' , 'India',' ', 26.5713 , 88.2429) ,
952:('North East' ,'North East' , 'India',' ', 28.5883 , 95.6907) ,
953:('Odisha' ,'Odisha' , 'India',' ', 19.9231332 , 84.9038426) ,
954:('Bihar' ,'Bihar' , 'India',' ', 25.366 , 84.449) ,
955:('Odisha' ,'Odisha' , 'India',' ', 21.2788 , 83.7594) ,
956:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.461 , 85.3139) ,
957:('Odisha' ,'Odisha' , 'India',' ', 20.653 , 86.128) ,
958:('Odisha' ,'Odisha' , 'India',' ', 19.8342 , 85.3205) ,
959:('Odisha' ,'Odisha' , 'India',' ', 20.0053 , 85.3303) ,
960:('Jharkhand' ,'Jharkhand' , 'India',' ', 24.2407 , 85.3242) ,
961:('Odisha' ,'Odisha' , 'India',' ', 18.9632 , 83.2174) ,
962:('North East' ,'North East' , 'India',' ', 26.9017 , 93.4267) ,
963:('Odisha' ,'Odisha' , 'India',' ', 21.2939 , 83.6752) ,
964:('Bihar' ,'Bihar' , 'India',' ', 25.3334 , 87.3697) ,
965:('Odisha' ,'Odisha' , 'India',' ', 18.0544 , 81.8582) ,
966:('Odisha' ,'Odisha' , 'India',' ', 19.0488 , 83.2372) ,
967:('Bihar' ,'Bihar' , 'India',' ', 25.017 , 85.4291) ,
968:('Odisha' ,'Odisha' , 'India',' ', 20.8742 , 84.1532) ,
969:('Odisha' ,'Odisha' , 'India',' ', 20.452 , 85.4044) ,
970:('Jharkhand' ,'Jharkhand' , 'India',' ', 22.9628 , 85.3874) ,
971:('North East' ,'North East' , 'India',' ', 25.3784 , 91.6742) ,
972:('Odisha' ,'Odisha' , 'India',' ', 19.6841 , 81.8778) ,
973:('Odisha' ,'Odisha' , 'India',' ', 18.164 , 81.6974) ,
974:('Odisha' ,'Odisha' , 'India',' ', 19.4111 , 82.5086) ,
975:('Odisha' ,'Odisha' , 'India',' ', 20.0022439 , 85.2517403) ,
976:('North East' ,'North East' , 'India',' ', 26.1160819 , 91.3983959) ,
977:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.201255 , 85.1379901) ,
978:('Siliguri' ,'Siliguri' , 'India',' ', 26.4892 , 88.2149) ,
979:('Odisha' ,'Odisha' , 'India',' ', 20.0960587 , 85.6245398) ,
980:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.201255 , 85.1379901) ,
981:('Odisha' ,'Odisha' , 'India',' ', 19.9231332 , 84.9038426) ,
982:('North East' ,'North East' , 'India',' ', 22.6670691 , 92.5655358) ,
983:('Bihar' ,'Bihar' , 'India',' ', 26.6014 , 84.3078) ,
984:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.4469641 , 84.7921549) ,
985:('Odisha' ,'Odisha' , 'India',' ', 21.2933578 , 86.7134794) ,
986:('Odisha' ,'Odisha' , 'India',' ', 19.9231332 , 84.9038426) ,
987:('Bihar' ,'Bihar' , 'India',' ', 25.6915 , 86.9942) ,
988:('Bihar' ,'Bihar' , 'India',' ', 25.1533 , 84.8304) ,
989:('North East' ,'North East' , 'India',' ', 25.9584678 , 90.4282947) ,
990:('Siliguri' ,'Siliguri' , 'India',' ', 23.2406 , 86.4744) ,
991:('Bihar' ,'Bihar' , 'India',' ', 25.9396 , 87.255) ,
992:('Odisha' ,'Odisha' , 'India',' ', 20.7619 , 86.1828) ,
993:('Jharkhand' ,'Jharkhand' , 'India',' ', 22.8414 , 84.3402) ,
994:('Siliguri' ,'Siliguri' , 'India',' ', 26.9342 , 88.0823) ,
995:('Siliguri' ,'Siliguri' , 'India',' ', 27.0316 , 88.2951) ,
996:('Odisha' ,'Odisha' , 'India',' ', 19.1559 , 84.7759) ,
997:('Odisha' ,'Odisha' , 'India',' ', 19.6134554 , 85.6312359) ,
998:('North East' ,'North East' , 'India',' ', 25.3201 , 91.0682) ,
999:('Odisha' ,'Odisha' , 'India',' ', 18.0654 , 81.873) ,
1000:('Bihar' ,'Bihar' , 'India',' ', 24.6149 , 84.4349) ,
1001:('Jharkhand' ,'Jharkhand' , 'India',' ', 22.6375 , 85.9925) ,
1002:('North East' ,'North East' , 'India',' ', 25.9445169 , 91.5362365) ,
1003:('North East' ,'North East' , 'India',' ', 25.1681768 , 91.5538817) ,
1004:('Odisha' ,'Odisha' , 'India',' ', 21.428933 , 85.3816847) ,
1005:('North East' ,'North East' , 'India',' ', 25.9744 , 91.6324) ,
1006:('Odisha' ,'Odisha' , 'India',' ', 21.9009 , 85.1805) ,
1007:('Jharkhand' ,'Jharkhand' , 'India',' ', 23.0775 , 85.1066) ,
1008:('Odisha' ,'Odisha' , 'India',' ', 20.2759 , 85.6436) ,
1009:('Siliguri' ,'Siliguri' , 'India',' ', 26.3868 , 88.8043) ,
1010:('Odisha' ,'Odisha' , 'India',' ', 19.0531 , 84.6947) ,
1011:('Odisha' ,'Odisha' , 'India',' ', 21.2373 , 85.1255) ,
1012:('North East' ,'North East' , 'India',' ', 26.9086309 , 93.4983979) ,
1013:('Odisha' ,'Odisha' , 'India',' ', 21.7599 , 86.5549) ,
1014:('Odisha' ,'Odisha' , 'India',' ', 19.7246059 , 85.1966448) ,
1015:('Odisha' ,'Odisha' , 'India',' ', 19.2246059 , 85.1966448) ,
1016:('Odisha' ,'Odisha' , 'India',' ', 19.46059 , 85.1966448) ,

    }
    
   # Demands of Customer/distributor  
    

     #  demand::
    # id: demand_value
    customer_demands ={
        
1: 23033,
2: 37771,
3: 1,
4: 132117,
5: 12909,
6: 14282,
7: 45776,
8: 43130,
9: 7149,
10: 44219,
11: 62395,
12: 12818,
13: 35500,
14: 80141,
15: 8778,
16: 86318,
17: 1189,
18: 30855,
19: 11340,
20: 1,
21: 91356,
22: 73239,
23: 27005,
24: 133157,
25: 32699,
26: 68828,
27: 30026,
28: 44074,
29: 28943,
30: 50792,
31: 13139,
32: 13180,
33: 42985,
34: 76310,
35: 48123,
36: 82056,
37: 30354,
38: 80651,
39: 65776,
40: 28436,
41: 29873,
42: 20802,
43: 68239,
44: 56811,
45: 18813,
46: 111953,
47: 66528,
48: 51925,
49: 58303,
50: 60532,
51: 25574,
52: 36118,
53: 28957,
54: 81884,
55: 103623,
56: 48814,
57: 9912,
58: 70021,
59: 90884,
60: 105486,
61: 16317,
62: 47897,
63: 62612,
64: 17099,
65: 19392,
66: 62257,
67: 38246,
68: 58757,
69: 29072,
70: 72381,
71: 16827,
72: 81032,
73: 39138,
74: 64636,
75: 76356,
76: 79808,
77: 54107,
78: 37234,
79: 55915,
80: 238087,
81: 26206,
82: 65945,
83: 46701,
84: 21470,
85: 76421,
86: 23034,
87: 89601,
88: 41089,
89: 36161,
90: 90534,
91: 49628,
92: 30235,
93: 31853,
94: 29913,
95: 16546,
96: 54617,
97: 32379,
98: 72530,
99: 1,
100: 48089,
101: 100049,
102: 63198,
103: 48985,
104: 12277,
105: 27918,
106: 24196,
107: 38691,
108: 116393,
109: 68933,
110: 22941,
111: 43184,
112: 59335,
113: 31078,
114: 18977,
115: 32067,
116: 75044,
117: 43746,
118: 17251,
119: 22811,
120: 69476,
121: 38772,
122: 39678,
123: 27775,
124: 38639,
125: 6012,
126: 34758,
127: 27145,
128: 48018,
129: 57145,
130: 19965,
131: 119822,
132: 2574,
133: 170324,
134: 32937,
135: 32581,
136: 48627,
137: 63091,
138: 78103,
139: 34256,
140: 40363,
141: 54182,
142: 49420,
143: 53516,
144: 46106,
145: 20053,
146: 4547,
147: 15385,
148: 61157,
149: 49706,
150: 95388,
151: 70223,
152: 35975,
153: 66667,
154: 51014,
155: 47693,
156: 36594,
157: 2382,
158: 24054,
159: 86374,
160: 120602,
161: 58280,
162: 91778,
163: 79118,
164: 27446,
165: 62194,
166: 71695,
167: 71498,
168: 115684,
169: 31124,
170: 49105,
171: 30392,
172: 72614,
173: 28758,
174: 40861,
175: 30935,
176: 38204,
177: 83562,
178: 54110,
179: 39156,
180: 109912,
181: 35862,
182: 106298,
183: 84993,
184: 78496,
185: 71991,
186: 59155,
187: 30116,
188: 33320,
189: 10629,
190: 46137,
191: 5887,
192: 2467,
193: 105179,
194: 26526,
195: 74794,
196: 100075,
197: 54494,
198: 48254,
199: 22190,
200: 61950,
201: 43627,
202: 81634,
203: 65957,
204: 74945,
205: 89752,
206: 7912,
207: 44090,
208: 36870,
209: 47754,
210: 52982,
211: 58575,
212: 33486,
213: 55079,
214: 23140,
215: 36489,
216: 50579,
217: 71975,
218: 15614,
219: 62554,
220: 53554,
221: 43351,
222: 14814,
223: 38281,
224: 28104,
225: 14651,
226: 68771,
227: 57237,
228: 60288,
229: 82096,
230: 100179,
231: 5147,
232: 35892,
233: 26614,
234: 37086,
235: 52160,
236: 67750,
237: 36543,
238: 34650,
239: 39465,
240: 62934,
241: 39121,
242: 14868,
243: 43105,
244: 24749,
245: 31227,
246: 13978,
247: 38931,
248: 54262,
249: 1,
250: 128184,
251: 72576,
252: 23670,
253: 31387,
254: 27867,
255: 27178,
256: 19301,
257: 31326,
258: 51121,
259: 119154,
260: 27463,
261: 68124,
262: 73773,
263: 76598,
264: 34129,
265: 18533,
266: 61132,
267: 57946,
268: 24780,
269: 130015,
270: 27671,
271: 79682,
272: 22019,
273: 50511,
274: 24238,
275: 32273,
276: 95999,
277: 38186,
278: 39280,
279: 175138,
280: 47631,
281: 1340,
282: 51155,
283: 96732,
284: 64390,
285: 5939,
286: 21253,
287: 38442,
288: 47287,
289: 32982,
290: 56562,
291: 1,
292: 197896,
293: 56296,
294: 73559,
295: 51236,
296: 69607,
297: 69450,
298: 112451,
299: 13141,
300: 1,
301: 29424,
302: 82321,
303: 95890,
304: 25098,
305: 20413,
306: 36905,
307: 20349,
308: 52358,
309: 10577,
310: 75106,
311: 42893,
312: 30131,
313: 25232,
314: 76865,
315: 64686,
316: 77023,
317: 24505,
318: 57959,
319: 17741,
320: 36633,
321: 62814,
322: 62145,
323: 149983,
324: 53356,
325: 77253,
326: 46755,
327: 55840,
328: 30948,
329: 45965,
330: 53265,
331: 69032,
332: 1,
333: 34869,
334: 43118,
335: 36528,
336: 86681,
337: 64588,
338: 31346,
339: 54820,
340: 2274,
341: 23953,
342: 81871,
343: 13111,
344: 110301,
345: 1,
346: 1,
347: 60732,
348: 1,
349: 116484,
350: 40996,
351: 67576,
352: 14011,
353: 15672,
354: 44142,
355: 1,
356: 105229,
357: 101611,
358: 53158,
359: 47322,
360: 32473,
361: 130610,
362: 23419,
363: 37264,
364: 38301,
365: 71561,
366: 46274,
367: 50835,
368: 7746,
369: 30672,
370: 23982,
371: 110156,
372: 36382,
373: 42551,
374: 90421,
375: 14369,
376: 73675,
377: 17685,
378: 78750,
379: 45114,
380: 46334,
381: 39641,
382: 102779,
383: 14867,
384: 77580,
385: 49321,
386: 157709,
387: 43282,
388: 63654,
389: 58874,
390: 28391,
391: 55068,
392: 1,
393: 22742,
394: 77008,
395: 1,
396: 27754,
397: 22588,
398: 463044,
399: 58161,
400: 51128,
401: 57981,
402: 12013,
403: 90676,
404: 28909,
405: 175109,
406: 30318,
407: 17852,
408: 26609,
409: 13710,
410: 9706,
411: 14948,
412: 37464,
413: 15912,
414: 5900,
415: 20510,
416: 5685,
417: 9487,
418: 36249,
419: 7544,
420: 61820,
421: 35188,
422: 1,
423: 18897,
424: 76388,
425: 29147,
426: 29058,
427: 33689,
428: 41774,
429: 24726,
430: 51745,
431: 113304,
432: 5952,
433: 1,
434: 42143,
435: 58105,
436: 979,
437: 24636,
438: 11362,
439: 2445,
440: 116058,
441: 20877,
442: 22282,
443: 23834,
444: 74575,
445: 5529,
446: 40586,
447: 61140,
448: 31674,
449: 39783,
450: 54980,
451: 15235,
452: 257659,
453: 67546,
454: 61350,
455: 55990,
456: 57843,
457: 35966,
458: 80797,
459: 44614,
460: 34933,
461: 34215,
462: 7011,
463: 26652,
464: 27556,
465: 51614,
466: 22058,
467: 49321,
468: 39994,
469: 36375,
470: 52359,
471: 45563,
472: 34741,
473: 26262,
474: 126923,
475: 18788,
476: 1,
477: 40843,
478: 79612,
479: 25844,
480: 151632,
481: 74614,
482: 52806,
483: 72869,
484: 52814,
485: 64861,
486: 45851,
487: 24523,
488: 85671,
489: 38985,
490: 65950,
491: 43488,
492: 72216,
493: 2193,
494: 54487,
495: 39121,
496: 23777,
497: 26420,
498: 67285,
499: 91960,
500: 120638,
501: 8884,
502: 87304,
503: 43309,
504: 48693,
505: 46172,
506: 112235,
507: 4424,
508: 50675,
509: 33490,
510: 30432,
511: 97833,
512: 33994,
513: 30252,
514: 43762,
515: 83304,
516: 47945,
517: 13203,
518: 35668,
519: 179302,
520: 53462,
521: 135471,
522: 17329,
523: 32863,
524: 1,
525: 69205,
526: 58577,
527: 17113,
528: 83480,
529: 1,
530: 11797,
531: 39201,
532: 59694,
533: 123381,
534: 41234,
535: 31799,
536: 82140,
537: 29121,
538: 97603,
539: 37703,
540: 62262,
541: 5069,
542: 34847,
543: 12651,
544: 43181,
545: 57008,
546: 70443,
547: 10008,
548: 46515,
549: 48940,
550: 35849,
551: 95392,
552: 52296,
553: 8729,
554: 13332,
555: 188203,
556: 20601,
557: 30831,
558: 13271,
559: 37509,
560: 100008,
561: 66785,
562: 5324,
563: 89783,
564: 44768,
565: 15696,
566: 56434,
567: 57067,
568: 32739,
569: 1,
570: 36656,
571: 47499,
572: 213311,
573: 30142,
574: 102744,
575: 62953,
576: 65008,
577: 29154,
578: 123775,
579: 54401,
580: 78732,
581: 19764,
582: 120604,
583: 17000,
584: 32379,
585: 36385,
586: 75345,
587: 78151,
588: 30168,
589: 50759,
590: 44037,
591: 77763,
592: 57898,
593: 123917,
594: 5753,
595: 157168,
596: 56684,
597: 9387,
598: 47220,
599: 68655,
600: 30827,
601: 82960,
602: 25624,
603: 137243,
604: 79890,
605: 91200,
606: 62866,
607: 23449,
608: 27899,
609: 1,
610: 23847,
611: 60998,
612: 63339,
613: 51200,
614: 23795,
615: 102045,
616: 29972,
617: 32247,
618: 19658,
619: 50364,
620: 40453,
621: 113797,
622: 64588,
623: 23867,
624: 102692,
625: 99894,
626: 1,
627: 57836,
628: 46014,
629: 21174,
630: 38527,
631: 1,
632: 81664,
633: 21322,
634: 4460,
635: 22996,
636: 2695,
637: 39698,
638: 1,
639: 60007,
640: 15474,
641: 3592,
642: 76947,
643: 76165,
644: 41483,
645: 1,
646: 18574,
647: 46141,
648: 13270,
649: 66111,
650: 39956,
651: 6933,
652: 12814,
653: 17053,
654: 17419,
655: 45736,
656: 23075,
657: 67279,
658: 53047,
659: 72756,
660: 43250,
661: 650,
662: 65341,
663: 29550,
664: 1,
665: 58838,
666: 18947,
667: 30737,
668: 38553,
669: 42073,
670: 18422,
671: 50668,
672: 41788,
673: 27691,
674: 4003,
675: 41690,
676: 105109,
677: 100030,
678: 58206,
679: 29501,
680: 41583,
681: 34860,
682: 42989,
683: 14763,
684: 41593,
685: 60401,
686: 21915,
687: 99628,
688: 54093,
689: 61315,
690: 23250,
691: 1,
692: 64232,
693: 41905,
694: 54217,
695: 43451,
696: 86893,
697: 76160,
698: 13520,
699: 76374,
700: 27461,
701: 17399,
702: 55969,
703: 86350,
704: 49676,
705: 67458,
706: 27953,
707: 3722,
708: 113679,
709: 49636,
710: 57076,
711: 44174,
712: 115950,
713: 17129,
714: 24866,
715: 72369,
716: 43632,
717: 34003,
718: 45181,
719: 2071,
720: 39245,
721: 93995,
722: 20607,
723: 72015,
724: 39804,
725: 77954,
726: 79649,
727: 31373,
728: 9474,
729: 70594,
730: 20384,
731: 9243,
732: 3963,
733: 15327,
734: 40230,
735: 44829,
736: 40253,
737: 42679,
738: 6018,
739: 137306,
740: 36851,
741: 95347,
742: 33846,
743: 34632,
744: 16718,
745: 34674,
746: 51247,
747: 1,
748: 32514,
749: 51692,
750: 35893,
751: 65668,
752: 6514,
753: 1,
754: 40059,
755: 27715,
756: 1,
757: 79799,
758: 13659,
759: 36608,
760: 50979,
761: 55342,
762: 71174,
763: 47402,
764: 48159,
765: 88281,
766: 42493,
767: 44949,
768: 127411,
769: 22239,
770: 43608,
771: 15756,
772: 47574,
773: 188665,
774: 25243,
775: 46670,
776: 29327,
777: 30726,
778: 65910,
779: 66779,
780: 61437,
781: 11238,
782: 37084,
783: 49282,
784: 71103,
785: 33950,
786: 34314,
787: 1250,
788: 31957,
789: 46635,
790: 1,
791: 186988,
792: 44587,
793: 50433,
794: 8988,
795: 39762,
796: 103791,
797: 3876,
798: 11570,
799: 37327,
800: 6474,
801: 22893,
802: 34340,
803: 65186,
804: 58487,
805: 61910,
806: 55979,
807: 183901,
808: 104982,
809: 31886,
810: 69785,
811: 54273,
812: 90524,
813: 21082,
814: 94440,
815: 22570,
816: 78522,
817: 24532,
818: 1,
819: 56371,
820: 47338,
821: 63910,
822: 80587,
823: 83997,
824: 2809,
825: 60955,
826: 35015,
827: 1,
828: 129429,
829: 18037,
830: 67512,
831: 91380,
832: 63179,
833: 44646,
834: 111610,
835: 54385,
836: 41159,
837: 79085,
838: 26650,
839: 56599,
840: 50176,
841: 1,
842: 89378,
843: 21307,
844: 5055,
845: 68836,
846: 42082,
847: 54473,
848: 28212,
849: 99404,
850: 51202,
851: 239538,
852: 57628,
853: 32258,
854: 4148,
855: 70358,
856: 51538,
857: 40836,
858: 39567,
859: 47605,
860: 59335,
861: 11429,
862: 202487,
863: 85082,
864: 1,
865: 18143,
866: 69823,
867: 69071,
868: 52603,
869: 67891,
870: 23967,
871: 98572,
872: 62821,
873: 56374,
874: 16699,
875: 90060,
876: 14054,
877: 41572,
878: 50101,
879: 19111,
880: 49117,
881: 49142,
882: 61975,
883: 78554,
884: 44800,
885: 47135,
886: 129052,
887: 48997,
888: 27756,
889: 41676,
890: 62357,
891: 59838,
892: 37658,
893: 39256,
894: 41844,
895: 90588,
896: 37292,
897: 1,
898: 52867,
899: 24563,
900: 41915,
901: 19699,
902: 109579,
903: 122090,
904: 60436,
905: 41449,
906: 2093,
907: 1,
908: 14101,
909: 33134,
910: 1,
911: 44773,
912: 55017,
913: 26952,
914: 53317,
915: 3681,
916: 31727,
917: 58312,
918: 38846,
919: 62330,
920: 33773,
921: 14700,
922: 26237,
923: 20118,
924: 49243,
925: 61372,
926: 46,
927: 53855,
928: 183749,
929: 140063,
930: 44900,
931: 71388,
932: 15272,
933: 99444,
934: 115472,
935: 114951,
936: 88283,
937: 92145,
938: 31891,
939: 3913,
940: 47326,
941: 20079,
942: 42382,
943: 52751,
944: 148018,
945: 81288,
946: 70138,
947: 65510,
948: 33734,
949: 26476,
950: 66744,
951: 56615,
952: 57038,
953: 22363,
954: 724,
955: 21783,
956: 20839,
957: 69917,
958: 1,
959: 118256,
960: 64701,
961: 20062,
962: 58462,
963: 277170,
964: 31959,
965: 101090,
966: 38833,
967: 40562,
968: 60402,
969: 78800,
970: 52795,
971: 18531,
972: 94936,
973: 57068,
974: 7879,
975: 11709,
976: 82616,
977: 15611,
978: 35882,
979: 30936,
980: 26259,
981: 21707,
982: 43723,
983: 44361,
984: 62178,
985: 16544,
986: 157766,
987: 32229,
988: 83050,
989: 80590,
990: 21313,
991: 48916,
992: 108117,
993: 1,
994: 42730,
995: 1,
996: 61291,
997: 63083,
998: 31380,
999: 81283,
1000: 57637,
1001: 116611,
1002: 45851,
1003: 46512,
1004: 155113,
1005: 106781,
1006: 82544,
1007: 44021,
1008: 43870,
1009: 15495,
1010: 193904,
1011: 34649,
1012: 40765,
1013: 32881,
1014: 82742,
1015: 33220,
1016: 17133,


    }
    
    # Distance between each warehouse to each customer
    # (from, to): distance
    distance = {
  (1,1) : 0,	(2,1) : 56,	(3,1) : 659,	(4,1) : 600,	(5,1) : 104,	(6,1) : 76,	(7,1) : 11,	(8,1) : 907,	(9,1) : 358,	(10,1) : 9,	(11,1) : 56,	(12,1) : 528,	(13,1) : 171,	(14,1) : 353,	(15,1) : 444,	(16,1) : 646,	(17,1) : 551,	(18,1) : 99,	(19,1) : 465,	(20,1) : 7,	(21,1) : 317,	(22,1) : 156,	(23,1) : 96,	(24,1) : 553,	(25,1) : 778,	(26,1) : 234,	(27,1) : 177,	(28,1) : 9,	(29,1) : 473,	(30,1) : 3,	(31,1) : 67,	(32,1) : 88,	(33,1) : 117,	(34,1) : 97,	(35,1) : 132,	(36,1) : 399,	(37,1) : 450,	(38,1) : 67,	(39,1) : 58,	(40,1) : 792,
(1,2) : 56,	(2,2) : 0,	(3,2) : 647,	(4,2) : 644,	(5,2) : 53,	(6,2) : 51,	(7,2) : 55,	(8,2) : 884,	(9,2) : 317,	(10,2) : 55,	(11,2) : 1,	(12,2) : 583,	(13,2) : 216,	(14,2) : 313,	(15,2) : 488,	(16,2) : 696,	(17,2) : 595,	(18,2) : 56,	(19,2) : 519,	(20,2) : 58,	(21,2) : 353,	(22,2) : 128,	(23,2) : 55,	(24,2) : 597,	(25,2) : 746,	(26,2) : 281,	(27,2) : 222,	(28,2) : 57,	(29,2) : 510,	(30,2) : 58,	(31,2) : 40,	(32,2) : 124,	(33,2) : 173,	(34,2) : 77,	(35,2) : 113,	(36,2) : 353,	(37,2) : 417,	(38,2) : 17,	(39,2) : 4,	(40,2) : 759,
(1,3) : 659,	(2,3) : 647,	(3,3) : 0,	(4,3) : 691,	(5,3) : 612,	(6,3) : 597,	(7,3) : 648,	(8,3) : 290,	(9,3) : 418,	(10,3) : 651,	(11,3) : 648,	(12,3) : 858,	(13,3) : 605,	(14,3) : 419,	(15,3) : 614,	(16,3) : 806,	(17,3) : 660,	(18,3) : 598,	(19,3) : 822,	(20,3) : 653,	(21,3) : 527,	(22,3) : 522,	(23,3) : 596,	(24,3) : 660,	(25,3) : 295,	(26,3) : 610,	(27,3) : 603,	(28,3) : 651,	(29,3) : 536,	(30,3) : 658,	(31,3) : 608,	(32,3) : 596,	(33,3) : 712,	(34,3) : 571,	(35,3) : 755,	(36,3) : 449,	(37,3) : 317,	(38,3) : 631,	(39,3) : 644,	(40,3) : 325,
(1,4) : 600,	(2,4) : 644,	(3,4) : 691,	(4,4) : 0,	(5,4) : 665,	(6,4) : 616,	(7,4) : 594,	(8,4) : 974,	(9,4) : 755,	(10,4) : 596,	(11,4) : 645,	(12,4) : 259,	(13,4) : 429,	(14,4) : 751,	(15,4) : 157,	(16,4) : 117,	(17,4) : 50,	(18,4) : 647,	(19,4) : 273,	(20,4) : 595,	(21,4) : 303,	(22,4) : 623,	(23,4) : 642,	(24,4) : 48,	(25,4) : 966,	(26,4) : 366,	(27,4) : 423,	(28,4) : 594,	(29,4) : 174,	(30,4) : 597,	(31,4) : 620,	(32,4) : 521,	(33,4) : 525,	(34,4) : 606,	(35,4) : 726,	(36,4) : 817,	(37,4) : 760,	(38,4) : 644,	(39,4) : 645,	(40,4) : 994,
(1,5) : 104,	(2,5) : 53,	(3,5) : 612,	(4,5) : 665,	(5,5) : 0,	(6,5) : 49,	(7,5) : 99,	(8,5) : 840,	(9,5) : 266,	(10,5) : 100,	(11,5) : 53,	(12,5) : 621,	(13,5) : 244,	(14,5) : 262,	(15,5) : 508,	(16,5) : 723,	(17,5) : 615,	(18,5) : 19,	(19,5) : 558,	(20,5) : 104,	(21,5) : 367,	(22,5) : 91,	(23,5) : 25,	(24,5) : 617,	(25,5) : 698,	(26,5) : 307,	(27,5) : 249,	(28,5) : 103,	(29,5) : 521,	(30,5) : 105,	(31,5) : 48,	(32,5) : 148,	(33,5) : 220,	(34,5) : 62,	(35,5) : 144,	(36,5) : 301,	(37,5) : 368,	(38,5) : 37,	(39,5) : 49,	(40,5) : 710,
(1,6) : 76,	(2,6) : 51,	(3,6) : 597,	(4,6) : 616,	(5,6) : 49,	(6,6) : 0,	(7,6) : 67,	(8,6) : 837,	(9,6) : 282,	(10,6) : 69,	(11,6) : 53,	(12,6) : 575,	(13,6) : 195,	(14,6) : 277,	(15,6) : 459,	(16,6) : 674,	(17,6) : 566,	(18,6) : 33,	(19,6) : 512,	(20,6) : 73,	(21,6) : 319,	(22,6) : 81,	(23,6) : 28,	(24,6) : 568,	(25,6) : 704,	(26,6) : 259,	(27,6) : 200,	(28,6) : 71,	(29,6) : 474,	(30,6) : 76,	(31,6) : 11,	(32,6) : 100,	(33,6) : 185,	(34,6) : 26,	(35,6) : 164,	(36,6) : 324,	(37,6) : 375,	(38,6) : 38,	(39,6) : 49,	(40,6) : 717,
(1,7) : 11,	(2,7) : 55,	(3,7) : 648,	(4,7) : 594,	(5,7) : 99,	(6,7) : 67,	(7,7) : 0,	(8,7) : 896,	(9,7) : 349,	(10,7) : 3,	(11,7) : 55,	(12,7) : 528,	(13,7) : 166,	(14,7) : 344,	(15,7) : 439,	(16,7) : 643,	(17,7) : 545,	(18,7) : 93,	(19,7) : 464,	(20,7) : 6,	(21,7) : 309,	(22,7) : 146,	(23,7) : 89,	(24,7) : 548,	(25,7) : 768,	(26,7) : 229,	(27,7) : 171,	(28,7) : 4,	(29,7) : 466,	(30,7) : 10,	(31,7) : 59,	(32,7) : 80,	(33,7) : 121,	(34,7) : 87,	(35,7) : 141,	(36,7) : 391,	(37,7) : 440,	(38,7) : 63,	(39,7) : 57,	(40,7) : 782,
(1,8) : 907,	(2,8) : 884,	(3,8) : 290,	(4,8) : 974,	(5,8) : 840,	(6,8) : 837,	(7,8) : 896,	(8,8) : 0,	(9,8) : 596,	(10,8) : 899,	(11,8) : 885,	(12,8) : 1148,	(13,8) : 880,	(14,8) : 599,	(15,8) : 904,	(16,8) : 1091,	(17,8) : 946,	(18,8) : 830,	(19,8) : 1112,	(20,8) : 902,	(21,8) : 815,	(22,8) : 756,	(23,8) : 829,	(24,8) : 946,	(25,8) : 200,	(26,8) : 892,	(27,8) : 878,	(28,8) : 900,	(29,8) : 824,	(30,8) : 906,	(31,8) : 848,	(32,8) : 857,	(33,8) : 977,	(34,8) : 812,	(35,8) : 983,	(36,8) : 599,	(37,8) : 482,	(38,8) : 867,	(39,8) : 880,	(40,8) : 216,
(1,9) : 358,	(2,9) : 317,	(3,9) : 418,	(4,9) : 755,	(5,9) : 266,	(6,9) : 282,	(7,9) : 349,	(8,9) : 596,	(9,9) : 0,	(10,9) : 351,	(11,9) : 318,	(12,9) : 795,	(13,9) : 420,	(14,9) : 5,	(15,9) : 612,	(16,9) : 843,	(17,9) : 707,	(18,9) : 262,	(19,9) : 737,	(20,9) : 355,	(21,9) : 464,	(22,9) : 206,	(23,9) : 264,	(24,9) : 709,	(25,9) : 437,	(26,9) : 468,	(27,9) : 422,	(28,9) : 353,	(29,9) : 584,	(30,9) : 358,	(31,9) : 291,	(32,9) : 348,	(33,9) : 461,	(34,9) : 263,	(35,9) : 399,	(36,9) : 62,	(37,9) : 114,	(38,9) : 300,	(39,9) : 314,	(40,9) : 446,
(1,10) : 9,	(2,10) : 55,	(3,10) : 651,	(4,10) : 596,	(5,10) : 100,	(6,10) : 69,	(7,10) : 3,	(8,10) : 899,	(9,10) : 351,	(10,10) : 0,	(11,10) : 55,	(12,10) : 528,	(13,10) : 167,	(14,10) : 347,	(15,10) : 440,	(16,10) : 643,	(17,10) : 546,	(18,10) : 94,	(19,10) : 464,	(20,10) : 4,	(21,10) : 311,	(22,10) : 148,	(23,10) : 91,	(24,10) : 549,	(25,10) : 771,	(26,10) : 230,	(27,10) : 173,	(28,10) : 2,	(29,10) : 467,	(30,10) : 7,	(31,10) : 61,	(32,10) : 82,	(33,10) : 120,	(34,10) : 90,	(35,10) : 139,	(36,10) : 393,	(37,10) : 443,	(38,10) : 64,	(39,10) : 57,	(40,10) : 785,
(1,11) : 56,	(2,11) : 1,	(3,11) : 648,	(4,11) : 645,	(5,11) : 53,	(6,11) : 53,	(7,11) : 55,	(8,11) : 885,	(9,11) : 318,	(10,11) : 55,	(11,11) : 0,	(12,11) : 583,	(13,11) : 217,	(14,11) : 314,	(15,11) : 489,	(16,11) : 696,	(17,11) : 596,	(18,11) : 56,	(19,11) : 520,	(20,11) : 58,	(21,11) : 354,	(22,11) : 129,	(23,11) : 56,	(24,11) : 598,	(25,11) : 747,	(26,11) : 281,	(27,11) : 223,	(28,11) : 58,	(29,11) : 511,	(30,11) : 58,	(31,11) : 42,	(32,11) : 125,	(33,11) : 173,	(34,11) : 78,	(35,11) : 111,	(36,11) : 354,	(37,11) : 417,	(38,11) : 18,	(39,11) : 4,	(40,11) : 760,
(1,12) : 528,	(2,12) : 583,	(3,12) : 858,	(4,12) : 259,	(5,12) : 621,	(6,12) : 575,	(7,12) : 528,	(8,12) : 1148,	(9,12) : 795,	(10,12) : 528,	(11,12) : 583,	(12,12) : 0,	(13,12) : 385,	(14,12) : 791,	(15,12) : 246,	(16,12) : 197,	(17,12) : 248,	(18,12) : 608,	(19,12) : 64,	(20,12) : 525,	(21,12) : 350,	(22,12) : 614,	(23,12) : 603,	(24,12) : 250,	(25,12) : 1102,	(26,12) : 329,	(27,12) : 381,	(28,12) : 526,	(29,12) : 334,	(30,12) : 526,	(31,12) : 573,	(32,12) : 476,	(33,12) : 420,	(34,12) : 576,	(35,12) : 629,	(36,12) : 854,	(37,12) : 836,	(38,12) : 590,	(39,12) : 585,	(40,12) : 1126,
(1,13) : 171,	(2,13) : 216,	(3,13) : 605,	(4,13) : 429,	(5,13) : 244,	(6,13) : 195,	(7,13) : 166,	(8,13) : 880,	(9,13) : 420,	(10,13) : 167,	(11,13) : 217,	(12,13) : 385,	(13,13) : 0,	(14,13) : 415,	(15,13) : 273,	(16,13) : 480,	(17,13) : 380,	(18,13) : 228,	(19,13) : 324,	(20,13) : 166,	(21,13) : 154,	(22,13) : 229,	(23,13) : 223,	(24,13) : 382,	(25,13) : 785,	(26,13) : 64,	(27,13) : 6,	(28,13) : 165,	(29,13) : 307,	(30,13) : 168,	(31,13) : 196,	(32,13) : 95,	(33,13) : 127,	(34,13) : 193,	(35,13) : 300,	(36,13) : 475,	(37,13) : 480,	(38,13) : 218,	(39,13) : 217,	(40,13) : 805,
(1,14) : 353,	(2,14) : 313,	(3,14) : 419,	(4,14) : 751,	(5,14) : 262,	(6,14) : 277,	(7,14) : 344,	(8,14) : 599,	(9,14) : 5,	(10,14) : 347,	(11,14) : 314,	(12,14) : 791,	(13,14) : 415,	(14,14) : 0,	(15,14) : 608,	(16,14) : 839,	(17,14) : 703,	(18,14) : 258,	(19,14) : 733,	(20,14) : 350,	(21,14) : 460,	(22,14) : 201,	(23,14) : 259,	(24,14) : 705,	(25,14) : 441,	(26,14) : 463,	(27,14) : 417,	(28,14) : 348,	(29,14) : 580,	(30,14) : 353,	(31,14) : 287,	(32,14) : 343,	(33,14) : 456,	(34,14) : 258,	(35,14) : 395,	(36,14) : 66,	(37,14) : 116,	(38,14) : 296,	(39,14) : 309,	(40,14) : 450,
(1,15) : 444,	(2,15) : 488,	(3,15) : 614,	(4,15) : 157,	(5,15) : 508,	(6,15) : 459,	(7,15) : 439,	(8,15) : 904,	(9,15) : 612,	(10,15) : 440,	(11,15) : 489,	(12,15) : 246,	(13,15) : 273,	(14,15) : 608,	(15,15) : 0,	(16,15) : 231,	(17,15) : 107,	(18,15) : 491,	(19,15) : 221,	(20,15) : 439,	(21,15) : 150,	(22,15) : 468,	(23,15) : 485,	(24,15) : 109,	(25,15) : 868,	(26,15) : 211,	(27,15) : 267,	(28,15) : 438,	(29,15) : 91,	(30,15) : 441,	(31,15) : 463,	(32,15) : 364,	(33,15) : 376,	(34,15) : 450,	(35,15) : 572,	(36,15) : 674,	(37,15) : 630,	(38,15) : 487,	(39,15) : 488,	(40,15) : 894,
(1,16) : 646,	(2,16) : 696,	(3,16) : 806,	(4,16) : 117,	(5,16) : 723,	(6,16) : 674,	(7,16) : 643,	(8,16) : 1091,	(9,16) : 843,	(10,16) : 643,	(11,16) : 696,	(12,16) : 197,	(13,16) : 480,	(14,16) : 839,	(15,16) : 231,	(16,16) : 0,	(17,16) : 148,	(18,16) : 707,	(19,16) : 237,	(20,16) : 642,	(21,16) : 380,	(22,16) : 693,	(23,16) : 701,	(24,16) : 148,	(25,16) : 1078,	(26,16) : 416,	(27,16) : 475,	(28,16) : 641,	(29,16) : 279,	(30,16) : 643,	(31,16) : 676,	(32,16) : 575,	(33,16) : 555,	(34,16) : 669,	(35,16) : 764,	(36,16) : 905,	(37,16) : 858,	(38,16) : 698,	(39,16) : 697,	(40,16) : 1106,
(1,17) : 551,	(2,17) : 595,	(3,17) : 660,	(4,17) : 50,	(5,17) : 615,	(6,17) : 566,	(7,17) : 545,	(8,17) : 946,	(9,17) : 707,	(10,17) : 546,	(11,17) : 596,	(12,17) : 248,	(13,17) : 380,	(14,17) : 703,	(15,17) : 107,	(16,17) : 148,	(17,17) : 0,	(18,17) : 597,	(19,17) : 251,	(20,17) : 545,	(21,17) : 253,	(22,17) : 573,	(23,17) : 591,	(24,17) : 3,	(25,17) : 930,	(26,17) : 317,	(27,17) : 374,	(28,17) : 544,	(29,17) : 131,	(30,17) : 547,	(31,17) : 570,	(32,17) : 471,	(33,17) : 478,	(34,17) : 556,	(35,17) : 677,	(36,17) : 769,	(37,17) : 715,	(38,17) : 594,	(39,17) : 595,	(40,17) : 957,
(1,18) : 99,	(2,18) : 56,	(3,18) : 598,	(4,18) : 647,	(5,18) : 19,	(6,18) : 33,	(7,18) : 93,	(8,18) : 830,	(9,18) : 262,	(10,18) : 94,	(11,18) : 56,	(12,18) : 608,	(13,18) : 228,	(14,18) : 258,	(15,18) : 491,	(16,18) : 707,	(17,18) : 597,	(18,18) : 0,	(19,18) : 545,	(20,18) : 98,	(21,18) : 348,	(22,18) : 76,	(23,18) : 6,	(24,18) : 599,	(25,18) : 691,	(26,18) : 291,	(27,18) : 233,	(28,18) : 96,	(29,18) : 503,	(30,18) : 100,	(31,18) : 36,	(32,18) : 133,	(33,18) : 213,	(34,18) : 43,	(35,18) : 158,	(36,18) : 300,	(37,18) : 361,	(38,18) : 38,	(39,18) : 52,	(40,18) : 703,
(1,19) : 465,	(2,19) : 519,	(3,19) : 822,	(4,19) : 273,	(5,19) : 558,	(6,19) : 512,	(7,19) : 464,	(8,19) : 1112,	(9,19) : 737,	(10,19) : 464,	(11,19) : 520,	(12,19) : 64,	(13,19) : 324,	(14,19) : 733,	(15,19) : 221,	(16,19) : 237,	(17,19) : 251,	(18,19) : 545,	(19,19) : 0,	(20,19) : 462,	(21,19) : 303,	(22,19) : 553,	(23,19) : 540,	(24,19) : 253,	(25,19) : 1056,	(26,19) : 270,	(27,19) : 320,	(28,19) : 462,	(29,19) : 312,	(30,19) : 462,	(31,19) : 510,	(32,19) : 414,	(33,19) : 357,	(34,19) : 514,	(35,19) : 565,	(36,19) : 795,	(37,19) : 782,	(38,19) : 526,	(39,19) : 521,	(40,19) : 1080,
(1,20) : 7,	(2,20) : 58,	(3,20) : 653,	(4,20) : 595,	(5,20) : 104,	(6,20) : 73,	(7,20) : 6,	(8,20) : 902,	(9,20) : 355,	(10,20) : 4,	(11,20) : 58,	(12,20) : 525,	(13,20) : 166,	(14,20) : 350,	(15,20) : 439,	(16,20) : 642,	(17,20) : 545,	(18,20) : 98,	(19,20) : 462,	(20,20) : 0,	(21,20) : 311,	(22,20) : 152,	(23,20) : 95,	(24,20) : 548,	(25,20) : 774,	(26,20) : 229,	(27,20) : 172,	(28,20) : 2,	(29,20) : 467,	(30,20) : 4,	(31,20) : 64,	(32,20) : 82,	(33,20) : 117,	(34,20) : 93,	(35,20) : 138,	(36,20) : 396,	(37,20) : 446,	(38,20) : 67,	(39,20) : 60,	(40,20) : 788,
(1,21) : 317,	(2,21) : 353,	(3,21) : 527,	(4,21) : 303,	(5,21) : 367,	(6,21) : 319,	(7,21) : 309,	(8,21) : 815,	(9,21) : 464,	(10,21) : 311,	(11,21) : 354,	(12,21) : 350,	(13,21) : 154,	(14,21) : 460,	(15,21) : 150,	(16,21) : 380,	(17,21) : 253,	(18,21) : 348,	(19,21) : 303,	(20,21) : 311,	(21,21) : 0,	(22,21) : 320,	(23,21) : 343,	(24,21) : 255,	(25,21) : 754,	(26,21) : 111,	(27,21) : 148,	(28,21) : 309,	(29,21) : 157,	(30,21) : 314,	(31,21) : 324,	(32,21) : 230,	(33,21) : 278,	(34,21) : 306,	(35,21) : 448,	(36,21) : 526,	(37,21) : 490,	(38,21) : 350,	(39,21) : 353,	(40,21) : 777,
(1,22) : 156,	(2,22) : 128,	(3,22) : 522,	(4,22) : 623,	(5,22) : 91,	(6,22) : 81,	(7,22) : 146,	(8,22) : 756,	(9,22) : 206,	(10,22) : 148,	(11,22) : 129,	(12,22) : 614,	(13,22) : 229,	(14,22) : 201,	(15,22) : 468,	(16,22) : 693,	(17,22) : 573,	(18,22) : 76,	(19,22) : 553,	(20,22) : 152,	(21,22) : 320,	(22,22) : 0,	(23,22) : 74,	(24,22) : 575,	(25,22) : 623,	(26,22) : 287,	(27,22) : 233,	(28,22) : 150,	(29,22) : 467,	(30,22) : 156,	(31,22) : 92,	(32,22) : 145,	(33,22) : 254,	(34,22) : 58,	(35,22) : 233,	(36,22) : 254,	(37,22) : 295,	(38,22) : 111,	(39,22) : 124,	(40,22) : 637,
(1,23) : 96,	(2,23) : 55,	(3,23) : 596,	(4,23) : 642,	(5,23) : 25,	(6,23) : 28,	(7,23) : 89,	(8,23) : 829,	(9,23) : 264,	(10,23) : 91,	(11,23) : 56,	(12,23) : 603,	(13,23) : 223,	(14,23) : 259,	(15,23) : 485,	(16,23) : 701,	(17,23) : 591,	(18,23) : 6,	(19,23) : 540,	(20,23) : 95,	(21,23) : 343,	(22,23) : 74,	(23,23) : 0,	(24,23) : 594,	(25,23) : 691,	(26,23) : 286,	(27,23) : 228,	(28,23) : 93,	(29,23) : 497,	(30,23) : 97,	(31,23) : 31,	(32,23) : 128,	(33,23) : 210,	(34,23) : 38,	(35,23) : 160,	(36,23) : 303,	(37,23) : 361,	(38,23) : 38,	(39,23) : 52,	(40,23) : 704,
(1,24) : 553,	(2,24) : 597,	(3,24) : 660,	(4,24) : 48,	(5,24) : 617,	(6,24) : 568,	(7,24) : 548,	(8,24) : 946,	(9,24) : 709,	(10,24) : 549,	(11,24) : 598,	(12,24) : 250,	(13,24) : 382,	(14,24) : 705,	(15,24) : 109,	(16,24) : 148,	(17,24) : 3,	(18,24) : 599,	(19,24) : 253,	(20,24) : 548,	(21,24) : 255,	(22,24) : 575,	(23,24) : 594,	(24,24) : 0,	(25,24) : 931,	(26,24) : 319,	(27,24) : 376,	(28,24) : 547,	(29,24) : 132,	(30,24) : 550,	(31,24) : 572,	(32,24) : 473,	(33,24) : 481,	(34,24) : 559,	(35,24) : 680,	(36,24) : 771,	(37,24) : 716,	(38,24) : 597,	(39,24) : 597,	(40,24) : 958,
(1,25) : 778,	(2,25) : 746,	(3,25) : 295,	(4,25) : 966,	(5,25) : 698,	(6,25) : 704,	(7,25) : 768,	(8,25) : 200,	(9,25) : 437,	(10,25) : 771,	(11,25) : 747,	(12,25) : 1102,	(13,25) : 785,	(14,25) : 441,	(15,25) : 868,	(16,25) : 1078,	(17,25) : 930,	(18,25) : 691,	(19,25) : 1056,	(20,25) : 774,	(21,25) : 754,	(22,25) : 623,	(23,25) : 691,	(24,25) : 931,	(25,25) : 0,	(26,25) : 810,	(27,25) : 785,	(28,25) : 772,	(29,25) : 800,	(30,25) : 778,	(31,25) : 714,	(32,25) : 743,	(33,25) : 864,	(34,25) : 681,	(35,25) : 835,	(36,25) : 427,	(37,25) : 330,	(38,25) : 729,	(39,25) : 743,	(40,25) : 30,
(1,26) : 234,	(2,26) : 281,	(3,26) : 610,	(4,26) : 366,	(5,26) : 307,	(6,26) : 259,	(7,26) : 229,	(8,26) : 892,	(9,26) : 468,	(10,26) : 230,	(11,26) : 281,	(12,26) : 329,	(13,26) : 64,	(14,26) : 463,	(15,26) : 211,	(16,26) : 416,	(17,26) : 317,	(18,26) : 291,	(19,26) : 270,	(20,26) : 229,	(21,26) : 111,	(22,26) : 287,	(23,26) : 286,	(24,26) : 319,	(25,26) : 810,	(26,26) : 0,	(27,26) : 59,	(28,26) : 228,	(29,26) : 253,	(30,26) : 231,	(31,26) : 260,	(32,26) : 159,	(33,26) : 171,	(34,26) : 255,	(35,26) : 361,	(36,26) : 525,	(37,26) : 518,	(38,26) : 282,	(39,26) : 281,	(40,26) : 832,
(1,27) : 177,	(2,27) : 222,	(3,27) : 603,	(4,27) : 423,	(5,27) : 249,	(6,27) : 200,	(7,27) : 171,	(8,27) : 878,	(9,27) : 422,	(10,27) : 173,	(11,27) : 223,	(12,27) : 381,	(13,27) : 6,	(14,27) : 417,	(15,27) : 267,	(16,27) : 475,	(17,27) : 374,	(18,27) : 233,	(19,27) : 320,	(20,27) : 172,	(21,27) : 148,	(22,27) : 233,	(23,27) : 228,	(24,27) : 376,	(25,27) : 785,	(26,27) : 59,	(27,27) : 0,	(28,27) : 171,	(29,27) : 301,	(30,27) : 174,	(31,27) : 201,	(32,27) : 101,	(33,27) : 132,	(34,27) : 197,	(35,27) : 306,	(36,27) : 478,	(37,27) : 481,	(38,27) : 223,	(39,27) : 223,	(40,27) : 804,
(1,28) : 9,	(2,28) : 57,	(3,28) : 651,	(4,28) : 594,	(5,28) : 103,	(6,28) : 71,	(7,28) : 4,	(8,28) : 900,	(9,28) : 353,	(10,28) : 2,	(11,28) : 58,	(12,28) : 526,	(13,28) : 165,	(14,28) : 348,	(15,28) : 438,	(16,28) : 641,	(17,28) : 544,	(18,28) : 96,	(19,28) : 462,	(20,28) : 2,	(21,28) : 309,	(22,28) : 150,	(23,28) : 93,	(24,28) : 547,	(25,28) : 772,	(26,28) : 228,	(27,28) : 171,	(28,28) : 0,	(29,28) : 466,	(30,28) : 7,	(31,28) : 63,	(32,28) : 80,	(33,28) : 118,	(34,28) : 91,	(35,28) : 140,	(36,28) : 395,	(37,28) : 444,	(38,28) : 67,	(39,28) : 59,	(40,28) : 786,
(1,29) : 473,	(2,29) : 510,	(3,29) : 536,	(4,29) : 174,	(5,29) : 521,	(6,29) : 474,	(7,29) : 466,	(8,29) : 824,	(9,29) : 584,	(10,29) : 467,	(11,29) : 511,	(12,29) : 334,	(13,29) : 307,	(14,29) : 580,	(15,29) : 91,	(16,29) : 279,	(17,29) : 131,	(18,29) : 503,	(19,29) : 312,	(20,29) : 467,	(21,29) : 157,	(22,29) : 467,	(23,29) : 497,	(24,29) : 132,	(25,29) : 800,	(26,29) : 253,	(27,29) : 301,	(28,29) : 466,	(29,29) : 0,	(30,29) : 470,	(31,29) : 479,	(32,29) : 387,	(33,29) : 424,	(34,29) : 460,	(35,29) : 605,	(36,29) : 646,	(37,29) : 586,	(38,29) : 506,	(39,29) : 509,	(40,29) : 827,
(1,30) : 3,	(2,30) : 58,	(3,30) : 658,	(4,30) : 597,	(5,30) : 105,	(6,30) : 76,	(7,30) : 10,	(8,30) : 906,	(9,30) : 358,	(10,30) : 7,	(11,30) : 58,	(12,30) : 526,	(13,30) : 168,	(14,30) : 353,	(15,30) : 441,	(16,30) : 643,	(17,30) : 547,	(18,30) : 100,	(19,30) : 462,	(20,30) : 4,	(21,30) : 314,	(22,30) : 156,	(23,30) : 97,	(24,30) : 550,	(25,30) : 778,	(26,30) : 231,	(27,30) : 174,	(28,30) : 7,	(29,30) : 470,	(30,30) : 0,	(31,30) : 67,	(32,30) : 86,	(33,30) : 115,	(34,30) : 97,	(35,30) : 135,	(36,30) : 399,	(37,30) : 450,	(38,30) : 69,	(39,30) : 60,	(40,30) : 792,
(1,31) : 67,	(2,31) : 40,	(3,31) : 608,	(4,31) : 620,	(5,31) : 48,	(6,31) : 11,	(7,31) : 59,	(8,31) : 848,	(9,31) : 291,	(10,31) : 61,	(11,31) : 42,	(12,31) : 573,	(13,31) : 196,	(14,31) : 287,	(15,31) : 463,	(16,31) : 676,	(17,31) : 570,	(18,31) : 36,	(19,31) : 510,	(20,31) : 64,	(21,31) : 324,	(22,31) : 92,	(23,31) : 31,	(24,31) : 572,	(25,31) : 714,	(26,31) : 260,	(27,31) : 201,	(28,31) : 63,	(29,31) : 479,	(30,31) : 67,	(31,31) : 0,	(32,31) : 101,	(33,31) : 178,	(34,31) : 37,	(35,31) : 153,	(36,31) : 332,	(37,31) : 385,	(38,31) : 29,	(39,31) : 38,	(40,31) : 728,
(1,32) : 88,	(2,32) : 124,	(3,32) : 596,	(4,32) : 521,	(5,32) : 148,	(6,32) : 100,	(7,32) : 80,	(8,32) : 857,	(9,32) : 348,	(10,32) : 82,	(11,32) : 125,	(12,32) : 476,	(13,32) : 95,	(14,32) : 343,	(15,32) : 364,	(16,32) : 575,	(17,32) : 471,	(18,32) : 133,	(19,32) : 414,	(20,32) : 82,	(21,32) : 230,	(22,32) : 145,	(23,32) : 128,	(24,32) : 473,	(25,32) : 743,	(26,32) : 159,	(27,32) : 101,	(28,32) : 80,	(29,32) : 387,	(30,32) : 86,	(31,32) : 101,	(32,32) : 0,	(33,32) : 121,	(34,32) : 100,	(35,32) : 220,	(36,32) : 399,	(37,32) : 424,	(38,32) : 124,	(39,32) : 124,	(40,32) : 760,
(1,33) : 117,	(2,33) : 173,	(3,33) : 712,	(4,33) : 525,	(5,33) : 220,	(6,33) : 185,	(7,33) : 121,	(8,33) : 977,	(9,33) : 461,	(10,33) : 120,	(11,33) : 173,	(12,33) : 420,	(13,33) : 127,	(14,33) : 456,	(15,33) : 376,	(16,33) : 555,	(17,33) : 478,	(18,33) : 213,	(19,33) : 357,	(20,33) : 117,	(21,33) : 278,	(22,33) : 254,	(23,33) : 210,	(24,33) : 481,	(25,33) : 864,	(26,33) : 171,	(27,33) : 132,	(28,33) : 118,	(29,33) : 424,	(30,33) : 115,	(31,33) : 178,	(32,33) : 121,	(33,33) : 0,	(34,33) : 199,	(35,33) : 210,	(36,33) : 507,	(37,33) : 543,	(38,33) : 184,	(39,33) : 175,	(40,33) : 880,
(1,34) : 97,	(2,34) : 77,	(3,34) : 571,	(4,34) : 606,	(5,34) : 62,	(6,34) : 26,	(7,34) : 87,	(8,34) : 812,	(9,34) : 263,	(10,34) : 90,	(11,34) : 78,	(12,34) : 576,	(13,34) : 193,	(14,34) : 258,	(15,34) : 450,	(16,34) : 669,	(17,34) : 556,	(18,34) : 43,	(19,34) : 514,	(20,34) : 93,	(21,34) : 306,	(22,34) : 58,	(23,34) : 38,	(24,34) : 559,	(25,34) : 681,	(26,34) : 255,	(27,34) : 197,	(28,34) : 91,	(29,34) : 460,	(30,34) : 97,	(31,34) : 37,	(32,34) : 100,	(33,34) : 199,	(34,34) : 0,	(35,34) : 189,	(36,34) : 308,	(37,34) : 353,	(38,34) : 63,	(39,34) : 74,	(40,34) : 695,
(1,35) : 132,	(2,35) : 113,	(3,35) : 755,	(4,35) : 726,	(5,35) : 144,	(6,35) : 164,	(7,35) : 141,	(8,35) : 983,	(9,35) : 399,	(10,35) : 139,	(11,35) : 111,	(12,35) : 629,	(13,35) : 300,	(14,35) : 395,	(15,35) : 572,	(16,35) : 764,	(17,35) : 677,	(18,35) : 158,	(19,35) : 565,	(20,35) : 138,	(21,35) : 448,	(22,35) : 233,	(23,35) : 160,	(24,35) : 680,	(25,35) : 835,	(26,35) : 361,	(27,35) : 306,	(28,35) : 140,	(29,35) : 605,	(30,35) : 135,	(31,35) : 153,	(32,35) : 220,	(33,35) : 210,	(34,35) : 189,	(35,35) : 0,	(36,35) : 422,	(37,35) : 506,	(38,35) : 126,	(39,35) : 115,	(40,35) : 845,
(1,36) : 399,	(2,36) : 353,	(3,36) : 449,	(4,36) : 817,	(5,36) : 301,	(6,36) : 324,	(7,36) : 391,	(8,36) : 599,	(9,36) : 62,	(10,36) : 393,	(11,36) : 354,	(12,36) : 854,	(13,36) : 475,	(14,36) : 66,	(15,36) : 674,	(16,36) : 905,	(17,36) : 769,	(18,36) : 300,	(19,36) : 795,	(20,36) : 396,	(21,36) : 526,	(22,36) : 254,	(23,36) : 303,	(24,36) : 771,	(25,36) : 427,	(26,36) : 525,	(27,36) : 478,	(28,36) : 395,	(29,36) : 646,	(30,36) : 399,	(31,36) : 332,	(32,36) : 399,	(33,36) : 507,	(34,36) : 308,	(35,36) : 422,	(36,36) : 0,	(37,36) : 133,	(38,36) : 337,	(39,36) : 350,	(40,36) : 432,
(1,37) : 450,	(2,37) : 417,	(3,37) : 317,	(4,37) : 760,	(5,37) : 368,	(6,37) : 375,	(7,37) : 440,	(8,37) : 482,	(9,37) : 114,	(10,37) : 443,	(11,37) : 417,	(12,37) : 836,	(13,37) : 480,	(14,37) : 116,	(15,37) : 630,	(16,37) : 858,	(17,37) : 715,	(18,37) : 361,	(19,37) : 782,	(20,37) : 446,	(21,37) : 490,	(22,37) : 295,	(23,37) : 361,	(24,37) : 716,	(25,37) : 330,	(26,37) : 518,	(27,37) : 481,	(28,37) : 444,	(29,37) : 586,	(30,37) : 450,	(31,37) : 385,	(32,37) : 424,	(33,37) : 543,	(34,37) : 353,	(35,37) : 506,	(36,37) : 133,	(37,37) : 0,	(38,37) : 399,	(39,37) : 413,	(40,37) : 342,
(1,38) : 67,	(2,38) : 17,	(3,38) : 631,	(4,38) : 644,	(5,38) : 37,	(6,38) : 38,	(7,38) : 63,	(8,38) : 867,	(9,38) : 300,	(10,38) : 64,	(11,38) : 18,	(12,38) : 590,	(13,38) : 218,	(14,38) : 296,	(15,38) : 487,	(16,38) : 698,	(17,38) : 594,	(18,38) : 38,	(19,38) : 526,	(20,38) : 67,	(21,38) : 350,	(22,38) : 111,	(23,38) : 38,	(24,38) : 597,	(25,38) : 729,	(26,38) : 282,	(27,38) : 223,	(28,38) : 67,	(29,38) : 506,	(30,38) : 69,	(31,38) : 29,	(32,38) : 124,	(33,38) : 184,	(34,38) : 63,	(35,38) : 126,	(36,38) : 337,	(37,38) : 399,	(38,38) : 0,	(39,38) : 14,	(40,38) : 741,
(1,39) : 58,	(2,39) : 4,	(3,39) : 644,	(4,39) : 645,	(5,39) : 49,	(6,39) : 49,	(7,39) : 57,	(8,39) : 880,	(9,39) : 314,	(10,39) : 57,	(11,39) : 4,	(12,39) : 585,	(13,39) : 217,	(14,39) : 309,	(15,39) : 488,	(16,39) : 697,	(17,39) : 595,	(18,39) : 52,	(19,39) : 521,	(20,39) : 60,	(21,39) : 353,	(22,39) : 124,	(23,39) : 52,	(24,39) : 597,	(25,39) : 743,	(26,39) : 281,	(27,39) : 223,	(28,39) : 59,	(29,39) : 509,	(30,39) : 60,	(31,39) : 38,	(32,39) : 124,	(33,39) : 175,	(34,39) : 74,	(35,39) : 115,	(36,39) : 350,	(37,39) : 413,	(38,39) : 14,	(39,39) : 0,	(40,39) : 755,
(1,40) : 792,	(2,40) : 759,	(3,40) : 325,	(4,40) : 994,	(5,40) : 710,	(6,40) : 717,	(7,40) : 782,	(8,40) : 216,	(9,40) : 446,	(10,40) : 785,	(11,40) : 760,	(12,40) : 1126,	(13,40) : 805,	(14,40) : 450,	(15,40) : 894,	(16,40) : 1106,	(17,40) : 957,	(18,40) : 703,	(19,40) : 1080,	(20,40) : 788,	(21,40) : 777,	(22,40) : 637,	(23,40) : 704,	(24,40) : 958,	(25,40) : 30,	(26,40) : 832,	(27,40) : 804,	(28,40) : 786,	(29,40) : 827,	(30,40) : 792,	(31,40) : 728,	(32,40) : 760,	(33,40) : 880,	(34,40) : 695,	(35,40) : 845,	(36,40) : 432,	(37,40) : 342,	(38,40) : 741,	(39,40) : 755,	(40,40) : 0,
(1,41) : 491,	(2,41) : 533,	(3,41) : 602,	(4,41) : 120,	(5,41) : 550,	(6,41) : 502,	(7,41) : 485,	(8,41) : 890,	(9,41) : 636,	(10,41) : 486,	(11,41) : 534,	(12,41) : 273,	(13,41) : 321,	(14,41) : 632,	(15,41) : 54,	(16,41) : 215,	(17,41) : 71,	(18,41) : 532,	(19,41) : 259,	(20,41) : 486,	(21,41) : 185,	(22,41) : 504,	(23,41) : 527,	(24,41) : 73,	(25,41) : 866,	(26,41) : 260,	(27,41) : 315,	(28,41) : 485,	(29,41) : 66,	(30,41) : 488,	(31,41) : 506,	(32,41) : 409,	(33,41) : 428,	(34,41) : 491,	(35,41) : 620,	(36,41) : 698,	(37,41) : 644,	(38,41) : 531,	(39,41) : 533,	(40,41) : 893,
(1,42) : 106,	(2,42) : 51,	(3,42) : 630,	(4,42) : 680,	(5,42) : 20,	(6,42) : 64,	(7,42) : 103,	(8,42) : 856,	(9,42) : 278,	(10,42) : 104,	(11,42) : 51,	(12,42) : 630,	(13,42) : 256,	(14,42) : 273,	(15,42) : 523,	(16,42) : 736,	(17,42) : 630,	(18,42) : 39,	(19,42) : 566,	(20,42) : 107,	(21,42) : 383,	(22,42) : 110,	(23,42) : 43,	(24,42) : 632,	(25,42) : 711,	(26,42) : 320,	(27,42) : 261,	(28,42) : 106,	(29,42) : 538,	(30,42) : 108,	(31,42) : 60,	(32,42) : 161,	(33,42) : 223,	(34,42) : 81,	(35,42) : 127,	(36,42) : 309,	(37,42) : 382,	(38,42) : 40,	(39,42) : 48,	(40,42) : 723,
(1,43) : 621,	(2,43) : 676,	(3,43) : 922,	(4,43) : 272,	(5,43) : 714,	(6,43) : 667,	(7,43) : 621,	(8,43) : 1211,	(9,43) : 883,	(10,43) : 621,	(11,43) : 676,	(12,43) : 93,	(13,43) : 476,	(14,43) : 878,	(15,43) : 308,	(16,43) : 175,	(17,43) : 279,	(18,43) : 700,	(19,43) : 157,	(20,43) : 618,	(21,43) : 430,	(22,43) : 705,	(23,43) : 695,	(24,43) : 280,	(25,43) : 1175,	(26,43) : 419,	(27,43) : 473,	(28,43) : 618,	(29,43) : 388,	(30,43) : 618,	(31,43) : 666,	(32,43) : 568,	(33,43) : 513,	(34,43) : 668,	(35,43) : 721,	(36,43) : 942,	(37,43) : 919,	(38,43) : 682,	(39,43) : 677,	(40,43) : 1200,
(1,44) : 30,	(2,44) : 52,	(3,44) : 683,	(4,44) : 626,	(5,44) : 105,	(6,44) : 91,	(7,44) : 40,	(8,44) : 927,	(9,44) : 368,	(10,44) : 37,	(11,44) : 52,	(12,44) : 546,	(13,44) : 198,	(14,44) : 364,	(15,44) : 471,	(16,44) : 669,	(17,44) : 577,	(18,44) : 106,	(19,44) : 482,	(20,44) : 36,	(21,44) : 345,	(22,44) : 172,	(23,44) : 104,	(24,44) : 579,	(25,44) : 794,	(26,44) : 260,	(27,44) : 204,	(28,44) : 38,	(29,44) : 502,	(30,44) : 32,	(31,44) : 80,	(32,44) : 118,	(33,44) : 128,	(34,44) : 115,	(35,44) : 103,	(36,44) : 405,	(37,44) : 465,	(38,44) : 68,	(39,44) : 56,	(40,44) : 807,
(1,45) : 283,	(2,45) : 302,	(3,45) : 434,	(4,45) : 412,	(5,45) : 298,	(6,45) : 256,	(7,45) : 273,	(8,45) : 714,	(9,45) : 344,	(10,45) : 275,	(11,45) : 303,	(12,45) : 472,	(13,45) : 175,	(14,45) : 340,	(15,45) : 269,	(16,45) : 500,	(17,45) : 364,	(18,45) : 278,	(19,45) : 422,	(20,45) : 277,	(21,45) : 125,	(22,45) : 229,	(23,45) : 273,	(24,45) : 366,	(25,45) : 637,	(26,45) : 178,	(27,45) : 171,	(28,45) : 275,	(29,45) : 245,	(30,45) : 281,	(31,45) : 264,	(32,45) : 198,	(33,45) : 294,	(34,45) : 236,	(35,45) : 411,	(36,45) : 406,	(37,45) : 366,	(38,45) : 293,	(39,45) : 300,	(40,45) : 660,
(1,46) : 138,	(2,46) : 103,	(3,46) : 731,	(4,46) : 738,	(5,46) : 120,	(6,46) : 150,	(7,46) : 144,	(8,46) : 952,	(9,46) : 364,	(10,46) : 143,	(11,46) : 102,	(12,46) : 653,	(13,46) : 309,	(14,46) : 360,	(15,46) : 582,	(16,46) : 781,	(17,46) : 688,	(18,46) : 137,	(19,46) : 589,	(20,46) : 143,	(21,46) : 453,	(22,46) : 211,	(23,46) : 140,	(24,46) : 691,	(25,46) : 801,	(26,46) : 372,	(27,46) : 315,	(28,46) : 144,	(29,46) : 609,	(30,46) : 141,	(31,46) : 141,	(32,46) : 223,	(33,46) : 233,	(34,46) : 173,	(35,46) : 39,	(36,46) : 385,	(37,46) : 473,	(38,46) : 112,	(39,46) : 104,	(40,46) : 810,
(1,47) : 362,	(2,47) : 321,	(3,47) : 417,	(4,47) : 758,	(5,47) : 270,	(6,47) : 286,	(7,47) : 353,	(8,47) : 593,	(9,47) : 4,	(10,47) : 355,	(11,47) : 322,	(12,47) : 799,	(13,47) : 424,	(14,47) : 9,	(15,47) : 616,	(16,47) : 847,	(17,47) : 710,	(18,47) : 266,	(19,47) : 742,	(20,47) : 359,	(21,47) : 468,	(22,47) : 210,	(23,47) : 268,	(24,47) : 712,	(25,47) : 434,	(26,47) : 472,	(27,47) : 426,	(28,47) : 357,	(29,47) : 587,	(30,47) : 362,	(31,47) : 295,	(32,47) : 352,	(33,47) : 465,	(34,47) : 267,	(35,47) : 402,	(36,47) : 59,	(37,47) : 111,	(38,47) : 304,	(39,47) : 318,	(40,47) : 443,
(1,48) : 891,	(2,48) : 855,	(3,48) : 415,	(4,48) : 1094,	(5,48) : 806,	(6,48) : 815,	(7,48) : 881,	(8,48) : 231,	(9,48) : 540,	(10,48) : 883,	(11,48) : 856,	(12,48) : 1230,	(13,48) : 908,	(14,48) : 544,	(15,48) : 997,	(16,48) : 1206,	(17,48) : 1058,	(18,48) : 800,	(19,48) : 1184,	(20,48) : 887,	(21,48) : 881,	(22,48) : 735,	(23,48) : 801,	(24,48) : 1059,	(25,48) : 128,	(26,48) : 935,	(27,48) : 908,	(28,48) : 885,	(29,48) : 929,	(30,48) : 891,	(31,48) : 826,	(32,48) : 861,	(33,48) : 981,	(34,48) : 794,	(35,48) : 938,	(36,48) : 519,	(37,48) : 441,	(38,48) : 838,	(39,48) : 852,	(40,48) : 104,
(1,49) : 124,	(2,49) : 76,	(3,49) : 592,	(4,49) : 668,	(5,49) : 24,	(6,49) : 59,	(7,49) : 118,	(8,49) : 817,	(9,49) : 242,	(10,49) : 120,	(11,49) : 77,	(12,49) : 633,	(13,49) : 253,	(14,49) : 238,	(15,49) : 512,	(16,49) : 730,	(17,49) : 618,	(18,49) : 26,	(19,49) : 571,	(20,49) : 123,	(21,49) : 368,	(22,49) : 73,	(23,49) : 31,	(24,49) : 621,	(25,49) : 674,	(26,49) : 315,	(27,49) : 257,	(28,49) : 122,	(29,49) : 521,	(30,49) : 126,	(31,49) : 62,	(32,49) : 158,	(33,49) : 239,	(34,49) : 62,	(35,49) : 166,	(36,49) : 277,	(37,49) : 344,	(38,49) : 60,	(39,49) : 72,	(40,49) : 686,
(1,50) : 399,	(2,50) : 453,	(3,50) : 768,	(4,50) : 280,	(5,50) : 490,	(6,50) : 444,	(7,50) : 398,	(8,50) : 1057,	(9,50) : 669,	(10,50) : 398,	(11,50) : 454,	(12,50) : 131,	(13,50) : 255,	(14,50) : 664,	(15,50) : 190,	(16,50) : 273,	(17,50) : 247,	(18,50) : 477,	(19,50) : 69,	(20,50) : 396,	(21,50) : 243,	(22,50) : 484,	(23,50) : 472,	(24,50) : 250,	(25,50) : 995,	(26,50) : 201,	(27,50) : 251,	(28,50) : 396,	(29,50) : 279,	(30,50) : 396,	(31,50) : 443,	(32,50) : 346,	(33,50) : 294,	(34,50) : 445,	(35,50) : 505,	(36,50) : 727,	(37,50) : 716,	(38,50) : 459,	(39,50) : 455,	(40,50) : 1018,
(1,51) : 118,	(2,51) : 172,	(3,51) : 732,	(4,51) : 546,	(5,51) : 222,	(6,51) : 190,	(7,51) : 124,	(8,51) : 994,	(9,51) : 470,	(10,51) : 123,	(11,51) : 172,	(12,51) : 433,	(13,51) : 151,	(14,51) : 465,	(15,51) : 398,	(16,51) : 573,	(17,51) : 499,	(18,51) : 217,	(19,51) : 369,	(20,51) : 119,	(21,51) : 302,	(22,51) : 264,	(23,51) : 213,	(24,51) : 502,	(25,51) : 878,	(26,51) : 195,	(27,51) : 157,	(28,51) : 120,	(29,51) : 448,	(30,51) : 117,	(31,51) : 183,	(32,51) : 138,	(33,51) : 24,	(34,51) : 207,	(35,51) : 196,	(36,51) : 514,	(37,51) : 555,	(38,51) : 185,	(39,51) : 175,	(40,51) : 894,
(1,52) : 689,	(2,52) : 658,	(3,52) : 242,	(4,52) : 888,	(5,52) : 611,	(6,52) : 615,	(7,52) : 678,	(8,52) : 253,	(9,52) : 353,	(10,52) : 681,	(11,52) : 659,	(12,52) : 1015,	(13,52) : 694,	(14,52) : 356,	(15,52) : 785,	(16,52) : 999,	(17,52) : 850,	(18,52) : 603,	(19,52) : 969,	(20,52) : 684,	(21,52) : 666,	(22,52) : 534,	(23,52) : 603,	(24,52) : 851,	(25,52) : 91,	(26,52) : 721,	(27,52) : 694,	(28,52) : 682,	(29,52) : 720,	(30,52) : 688,	(31,52) : 625,	(32,52) : 652,	(33,52) : 773,	(34,52) : 591,	(35,52) : 749,	(36,52) : 349,	(37,52) : 243,	(38,52) : 641,	(39,52) : 654,	(40,52) : 111,
(1,53) : 61,	(2,53) : 5,	(3,53) : 645,	(4,53) : 648,	(5,53) : 48,	(6,53) : 51,	(7,53) : 60,	(8,53) : 881,	(9,53) : 313,	(10,53) : 60,	(11,53) : 5,	(12,53) : 588,	(13,53) : 220,	(14,53) : 309,	(15,53) : 492,	(16,53) : 700,	(17,53) : 598,	(18,53) : 52,	(19,53) : 524,	(20,53) : 63,	(21,53) : 356,	(22,53) : 125,	(23,53) : 52,	(24,53) : 601,	(25,53) : 743,	(26,53) : 284,	(27,53) : 226,	(28,53) : 62,	(29,53) : 513,	(30,53) : 63,	(31,53) : 40,	(32,53) : 128,	(33,53) : 178,	(34,53) : 76,	(35,53) : 113,	(36,53) : 349,	(37,53) : 413,	(38,53) : 14,	(39,53) : 3,	(40,53) : 755,
(1,54) : 939,	(2,54) : 905,	(3,54) : 423,	(4,54) : 1111,	(5,54) : 856,	(6,54) : 864,	(7,54) : 929,	(8,54) : 197,	(9,54) : 592,	(10,54) : 931,	(11,54) : 906,	(12,54) : 1258,	(13,54) : 947,	(14,54) : 596,	(15,54) : 1021,	(16,54) : 1225,	(17,54) : 1077,	(18,54) : 850,	(19,54) : 1214,	(20,54) : 935,	(21,54) : 912,	(22,54) : 783,	(23,54) : 850,	(24,54) : 1078,	(25,54) : 162,	(26,54) : 972,	(27,54) : 947,	(28,54) : 933,	(29,54) : 950,	(30,54) : 939,	(31,54) : 874,	(32,54) : 905,	(33,54) : 1026,	(34,54) : 841,	(35,54) : 991,	(36,54) : 575,	(37,54) : 489,	(38,54) : 888,	(39,54) : 902,	(40,54) : 147,
(1,55) : 492,	(2,55) : 540,	(3,55) : 701,	(4,55) : 127,	(5,55) : 567,	(6,55) : 518,	(7,55) : 488,	(8,55) : 991,	(9,55) : 692,	(10,55) : 489,	(11,55) : 541,	(12,55) : 168,	(13,55) : 324,	(14,55) : 688,	(15,55) : 89,	(16,55) : 157,	(17,55) : 93,	(18,55) : 550,	(19,55) : 159,	(20,55) : 487,	(21,55) : 228,	(22,55) : 537,	(23,55) : 545,	(24,55) : 96,	(25,55) : 957,	(26,55) : 260,	(27,55) : 319,	(28,55) : 487,	(29,55) : 168,	(30,55) : 489,	(31,55) : 520,	(32,55) : 419,	(33,55) : 407,	(34,55) : 512,	(35,55) : 613,	(36,55) : 754,	(37,55) : 715,	(38,55) : 542,	(39,55) : 541,	(40,55) : 983,
(1,56) : 483,	(2,56) : 531,	(3,56) : 690,	(4,56) : 130,	(5,56) : 557,	(6,56) : 508,	(7,56) : 479,	(8,56) : 980,	(9,56) : 681,	(10,56) : 480,	(11,56) : 532,	(12,56) : 176,	(13,56) : 315,	(14,56) : 676,	(15,56) : 77,	(16,56) : 167,	(17,56) : 93,	(18,56) : 541,	(19,56) : 164,	(20,56) : 479,	(21,56) : 217,	(22,56) : 526,	(23,56) : 535,	(24,56) : 95,	(25,56) : 945,	(26,56) : 251,	(27,56) : 309,	(28,56) : 478,	(29,56) : 158,	(30,56) : 480,	(31,56) : 510,	(32,56) : 410,	(33,56) : 400,	(34,56) : 502,	(35,56) : 605,	(36,56) : 742,	(37,56) : 703,	(38,56) : 533,	(39,56) : 532,	(40,56) : 971,
(1,57) : 232,	(2,57) : 278,	(3,57) : 604,	(4,57) : 368,	(5,57) : 304,	(6,57) : 255,	(7,57) : 227,	(8,57) : 886,	(9,57) : 462,	(10,57) : 228,	(11,57) : 279,	(12,57) : 335,	(13,57) : 61,	(14,57) : 457,	(15,57) : 213,	(16,57) : 420,	(17,57) : 319,	(18,57) : 288,	(19,57) : 276,	(20,57) : 227,	(21,57) : 108,	(22,57) : 282,	(23,57) : 282,	(24,57) : 321,	(25,57) : 804,	(26,57) : 6,	(27,57) : 56,	(28,57) : 226,	(29,57) : 252,	(30,57) : 229,	(31,57) : 257,	(32,57) : 156,	(33,57) : 173,	(34,57) : 250,	(35,57) : 359,	(36,57) : 520,	(37,57) : 512,	(38,57) : 279,	(39,57) : 278,	(40,57) : 825,
(1,58) : 133,	(2,58) : 184,	(3,58) : 754,	(4,58) : 557,	(5,58) : 235,	(6,58) : 207,	(7,58) : 140,	(8,58) : 1015,	(9,58) : 487,	(10,58) : 138,	(11,58) : 184,	(12,58) : 435,	(13,58) : 171,	(14,58) : 483,	(15,58) : 412,	(16,58) : 581,	(17,58) : 511,	(18,58) : 232,	(19,58) : 371,	(20,58) : 134,	(21,58) : 320,	(22,58) : 282,	(23,58) : 229,	(24,58) : 514,	(25,58) : 898,	(26,58) : 212,	(27,58) : 176,	(28,58) : 136,	(29,58) : 464,	(30,58) : 131,	(31,58) : 198,	(32,58) : 159,	(33,58) : 44,	(34,58) : 224,	(35,58) : 196,	(36,58) : 530,	(37,58) : 574,	(38,58) : 198,	(39,58) : 187,	(40,58) : 914,
(1,59) : 678,	(2,59) : 647,	(3,59) : 237,	(4,59) : 879,	(5,59) : 600,	(6,59) : 604,	(7,59) : 667,	(8,59) : 261,	(9,59) : 343,	(10,59) : 670,	(11,59) : 648,	(12,59) : 1005,	(13,59) : 683,	(14,59) : 346,	(15,59) : 774,	(16,59) : 989,	(17,59) : 841,	(18,59) : 592,	(19,59) : 958,	(20,59) : 673,	(21,59) : 656,	(22,59) : 523,	(23,59) : 592,	(24,59) : 842,	(25,59) : 102,	(26,59) : 710,	(27,59) : 683,	(28,59) : 671,	(29,59) : 710,	(30,59) : 677,	(31,59) : 614,	(32,59) : 641,	(33,59) : 762,	(34,59) : 580,	(35,59) : 738,	(36,59) : 340,	(37,59) : 232,	(38,59) : 630,	(39,59) : 644,	(40,59) : 122,
(1,60) : 254,	(2,60) : 232,	(3,60) : 417,	(4,60) : 610,	(5,60) : 195,	(6,60) : 184,	(7,60) : 243,	(8,60) : 653,	(9,60) : 151,	(10,60) : 246,	(11,60) : 233,	(12,60) : 645,	(13,60) : 275,	(14,60) : 146,	(15,60) : 464,	(16,60) : 695,	(17,60) : 561,	(18,60) : 181,	(19,60) : 588,	(20,60) : 249,	(21,60) : 315,	(22,60) : 105,	(23,60) : 179,	(24,60) : 563,	(25,60) : 528,	(26,60) : 319,	(27,60) : 277,	(28,60) : 247,	(29,60) : 442,	(30,60) : 253,	(31,60) : 195,	(32,60) : 217,	(33,60) : 337,	(34,60) : 159,	(35,60) : 338,	(36,60) : 211,	(37,60) : 208,	(38,60) : 216,	(39,60) : 229,	(40,60) : 544,
(1,61) : 568,	(2,61) : 621,	(3,61) : 838,	(4,61) : 199,	(5,61) : 655,	(6,61) : 608,	(7,61) : 566,	(8,61) : 1127,	(9,61) : 810,	(10,61) : 567,	(11,61) : 622,	(12,61) : 71,	(13,61) : 414,	(14,61) : 806,	(15,61) : 225,	(16,61) : 127,	(17,61) : 198,	(18,61) : 641,	(19,61) : 119,	(20,61) : 565,	(21,61) : 352,	(22,61) : 640,	(23,61) : 636,	(24,61) : 200,	(25,61) : 1092,	(26,61) : 353,	(27,61) : 409,	(28,61) : 564,	(29,61) : 304,	(30,61) : 565,	(31,61) : 608,	(32,61) : 508,	(33,61) : 466,	(34,61) : 606,	(35,61) : 676,	(36,61) : 870,	(37,61) : 842,	(38,61) : 626,	(39,61) : 623,	(40,61) : 1118,
(1,62) : 162,	(2,62) : 126,	(3,62) : 531,	(4,62) : 646,	(5,62) : 82,	(6,62) : 86,	(7,62) : 153,	(8,62) : 759,	(9,62) : 196,	(10,62) : 155,	(11,62) : 127,	(12,62) : 635,	(13,62) : 250,	(14,62) : 192,	(15,62) : 492,	(16,62) : 716,	(17,62) : 596,	(18,62) : 71,	(19,62) : 574,	(20,62) : 159,	(21,62) : 343,	(22,62) : 24,	(23,62) : 71,	(24,62) : 599,	(25,62) : 621,	(26,62) : 309,	(27,62) : 254,	(28,62) : 157,	(29,62) : 491,	(30,62) : 162,	(31,62) : 95,	(32,62) : 163,	(33,62) : 267,	(34,62) : 68,	(35,62) : 225,	(36,62) : 240,	(37,62) : 291,	(38,62) : 108,	(39,62) : 122,	(40,62) : 633,
(1,63) : 43,	(2,63) : 90,	(3,63) : 697,	(4,63) : 599,	(5,63) : 142,	(6,63) : 118,	(7,63) : 53,	(8,63) : 948,	(9,63) : 400,	(10,63) : 50,	(11,63) : 90,	(12,63) : 509,	(13,63) : 175,	(14,63) : 395,	(15,63) : 446,	(16,63) : 638,	(17,63) : 551,	(18,63) : 140,	(19,63) : 445,	(20,63) : 46,	(21,63) : 328,	(22,63) : 198,	(23,63) : 137,	(24,63) : 553,	(25,63) : 821,	(26,63) : 234,	(27,63) : 182,	(28,63) : 49,	(29,63) : 482,	(30,63) : 43,	(31,63) : 109,	(32,63) : 111,	(33,63) : 90,	(34,63) : 140,	(35,63) : 127,	(36,63) : 440,	(37,63) : 493,	(38,63) : 104,	(39,63) : 93,	(40,63) : 835,
(1,64) : 132,	(2,64) : 112,	(3,64) : 755,	(4,64) : 726,	(5,64) : 143,	(6,64) : 163,	(7,64) : 140,	(8,64) : 982,	(9,64) : 398,	(10,64) : 138,	(11,64) : 111,	(12,64) : 629,	(13,64) : 300,	(14,64) : 394,	(15,64) : 572,	(16,64) : 764,	(17,64) : 677,	(18,64) : 157,	(19,64) : 565,	(20,64) : 138,	(21,64) : 448,	(22,64) : 233,	(23,64) : 160,	(24,64) : 680,	(25,64) : 835,	(26,64) : 360,	(27,64) : 306,	(28,64) : 140,	(29,64) : 604,	(30,64) : 135,	(31,64) : 153,	(32,64) : 220,	(33,64) : 210,	(34,64) : 189,	(35,64) : 0,	(36,64) : 422,	(37,64) : 506,	(38,64) : 126,	(39,64) : 115,	(40,64) : 845,
(1,65) : 1065,	(2,65) : 1029,	(3,65) : 559,	(4,65) : 1248,	(5,65) : 979,	(6,65) : 990,	(7,65) : 1055,	(8,65) : 305,	(9,65) : 713,	(10,65) : 1058,	(11,65) : 1030,	(12,65) : 1396,	(13,65) : 1081,	(14,65) : 717,	(15,65) : 1159,	(16,65) : 1363,	(17,65) : 1215,	(18,65) : 974,	(19,65) : 1352,	(20,65) : 1061,	(21,65) : 1049,	(22,65) : 910,	(23,65) : 975,	(24,65) : 1215,	(25,65) : 297,	(26,65) : 1107,	(27,65) : 1081,	(28,65) : 1059,	(29,65) : 1087,	(30,65) : 1065,	(31,65) : 1000,	(32,65) : 1036,	(33,65) : 1156,	(34,65) : 968,	(35,65) : 1110,	(36,65) : 689,	(37,65) : 615,	(38,65) : 1012,	(39,65) : 1026,	(40,65) : 277,
(1,66) : 460,	(2,66) : 489,	(3,66) : 453,	(4,66) : 252,	(5,66) : 492,	(6,66) : 447,	(7,66) : 451,	(8,66) : 742,	(9,66) : 520,	(10,66) : 453,	(11,66) : 490,	(12,66) : 411,	(13,66) : 306,	(14,66) : 517,	(15,66) : 165,	(16,66) : 361,	(17,66) : 213,	(18,66) : 473,	(19,66) : 384,	(20,66) : 453,	(21,66) : 155,	(22,66) : 427,	(23,66) : 468,	(24,66) : 214,	(25,66) : 717,	(26,66) : 265,	(27,66) : 301,	(28,66) : 452,	(29,66) : 84,	(30,66) : 457,	(31,66) : 454,	(32,66) : 371,	(33,66) : 432,	(34,66) : 430,	(35,66) : 592,	(36,66) : 582,	(37,66) : 512,	(38,66) : 482,	(39,66) : 488,	(40,66) : 744,
(1,67) : 483,	(2,67) : 531,	(3,67) : 692,	(4,67) : 131,	(5,67) : 557,	(6,67) : 508,	(7,67) : 479,	(8,67) : 982,	(9,67) : 681,	(10,67) : 479,	(11,67) : 532,	(12,67) : 174,	(13,67) : 315,	(14,67) : 677,	(15,67) : 79,	(16,67) : 167,	(17,67) : 95,	(18,67) : 540,	(19,67) : 162,	(20,67) : 478,	(21,67) : 217,	(22,67) : 526,	(23,67) : 535,	(24,67) : 97,	(25,67) : 947,	(26,67) : 250,	(27,67) : 309,	(28,67) : 478,	(29,67) : 160,	(30,67) : 480,	(31,67) : 510,	(32,67) : 409,	(33,67) : 399,	(34,67) : 502,	(35,67) : 605,	(36,67) : 743,	(37,67) : 704,	(38,67) : 532,	(39,67) : 532,	(40,67) : 973,
(1,68) : 136,	(2,68) : 94,	(3,68) : 567,	(4,68) : 657,	(5,68) : 46,	(6,68) : 63,	(7,68) : 128,	(8,68) : 794,	(9,68) : 224,	(10,68) : 130,	(11,68) : 94,	(12,68) : 631,	(13,68) : 248,	(14,68) : 220,	(15,68) : 501,	(16,68) : 722,	(17,68) : 607,	(18,68) : 38,	(19,68) : 569,	(20,68) : 134,	(21,68) : 355,	(22,68) : 50,	(23,68) : 40,	(24,68) : 609,	(25,68) : 653,	(26,68) : 309,	(27,68) : 252,	(28,68) : 132,	(29,68) : 506,	(30,68) : 137,	(31,68) : 70,	(32,68) : 155,	(33,68) : 247,	(34,68) : 55,	(35,68) : 190,	(36,68) : 263,	(37,68) : 323,	(38,68) : 76,	(39,68) : 90,	(40,68) : 665,
(1,69) : 245,	(2,69) : 218,	(3,69) : 436,	(4,69) : 632,	(5,69) : 177,	(6,69) : 172,	(7,69) : 235,	(8,69) : 666,	(9,69) : 137,	(10,69) : 237,	(11,69) : 219,	(12,69) : 659,	(13,69) : 284,	(14,69) : 132,	(15,69) : 484,	(16,69) : 714,	(17,69) : 583,	(18,69) : 165,	(19,69) : 601,	(20,69) : 241,	(21,69) : 334,	(22,69) : 91,	(23,69) : 164,	(24,69) : 585,	(25,69) : 534,	(26,69) : 331,	(27,69) : 285,	(28,69) : 238,	(29,69) : 465,	(30,69) : 245,	(31,69) : 183,	(32,69) : 217,	(33,69) : 335,	(34,69) : 148,	(35,69) : 321,	(36,69) : 195,	(37,69) : 208,	(38,69) : 201,	(39,69) : 215,	(40,69) : 548,
(1,70) : 56,	(2,70) : 15,	(3,70) : 633,	(4,70) : 634,	(5,70) : 48,	(6,70) : 36,	(7,70) : 52,	(8,70) : 871,	(9,70) : 308,	(10,70) : 53,	(11,70) : 16,	(12,70) : 578,	(13,70) : 208,	(14,70) : 303,	(15,70) : 478,	(16,70) : 688,	(17,70) : 585,	(18,70) : 46,	(19,70) : 515,	(20,70) : 56,	(21,70) : 342,	(22,70) : 114,	(23,70) : 44,	(24,70) : 587,	(25,70) : 735,	(26,70) : 272,	(27,70) : 213,	(28,70) : 55,	(29,70) : 498,	(30,70) : 58,	(31,70) : 25,	(32,70) : 114,	(33,70) : 173,	(34,70) : 62,	(35,70) : 128,	(36,70) : 346,	(37,70) : 405,	(38,70) : 11,	(39,70) : 13,	(40,70) : 748,
(1,71) : 194,	(2,71) : 184,	(3,71) : 466,	(4,71) : 565,	(5,71) : 159,	(6,71) : 132,	(7,71) : 183,	(8,71) : 715,	(9,71) : 215,	(10,71) : 185,	(11,71) : 185,	(12,71) : 582,	(13,71) : 206,	(14,71) : 210,	(15,71) : 414,	(16,71) : 643,	(17,71) : 516,	(18,71) : 141,	(19,71) : 523,	(20,71) : 189,	(21,71) : 264,	(22,71) : 73,	(23,71) : 137,	(24,71) : 518,	(25,71) : 596,	(26,71) : 253,	(27,71) : 208,	(28,71) : 186,	(29,71) : 404,	(30,71) : 193,	(31,71) : 143,	(32,71) : 147,	(33,71) : 268,	(34,71) : 107,	(35,71) : 295,	(36,71) : 272,	(37,71) : 278,	(38,71) : 169,	(39,71) : 181,	(40,71) : 613,
(1,72) : 383,	(2,72) : 338,	(3,72) : 450,	(4,72) : 804,	(5,72) : 286,	(6,72) : 308,	(7,72) : 375,	(8,72) : 607,	(9,72) : 50,	(10,72) : 377,	(11,72) : 339,	(12,72) : 839,	(13,72) : 460,	(14,72) : 54,	(15,72) : 660,	(16,72) : 891,	(17,72) : 756,	(18,72) : 285,	(19,72) : 780,	(20,72) : 381,	(21,72) : 512,	(22,72) : 238,	(23,72) : 287,	(24,72) : 758,	(25,72) : 437,	(26,72) : 510,	(27,72) : 462,	(28,72) : 379,	(29,72) : 634,	(30,72) : 384,	(31,72) : 317,	(32,72) : 383,	(33,72) : 491,	(34,72) : 292,	(35,72) : 408,	(36,72) : 16,	(37,72) : 135,	(38,72) : 322,	(39,72) : 335,	(40,72) : 443,
(1,73) : 143,	(2,73) : 92,	(3,73) : 596,	(4,73) : 691,	(5,73) : 40,	(6,73) : 82,	(7,73) : 139,	(8,73) : 815,	(9,73) : 233,	(10,73) : 140,	(11,73) : 92,	(12,73) : 657,	(13,73) : 276,	(14,73) : 229,	(15,73) : 535,	(16,73) : 753,	(17,73) : 641,	(18,73) : 49,	(19,73) : 594,	(20,73) : 143,	(21,73) : 390,	(22,73) : 86,	(23,73) : 54,	(24,73) : 643,	(25,73) : 668,	(26,73) : 339,	(27,73) : 281,	(28,73) : 142,	(29,73) : 542,	(30,73) : 145,	(31,73) : 84,	(32,73) : 181,	(33,73) : 260,	(34,73) : 85,	(35,73) : 168,	(36,73) : 264,	(37,73) : 339,	(38,73) : 77,	(39,73) : 88,	(40,73) : 679,
(1,74) : 882,	(2,74) : 853,	(3,74) : 327,	(4,74) : 1018,	(5,74) : 806,	(6,74) : 809,	(7,74) : 872,	(8,74) : 110,	(9,74) : 549,	(10,74) : 874,	(11,74) : 854,	(12,74) : 1173,	(13,74) : 876,	(14,74) : 552,	(15,74) : 933,	(16,74) : 1133,	(17,74) : 985,	(18,74) : 798,	(19,74) : 1131,	(20,74) : 878,	(21,74) : 830,	(22,74) : 728,	(23,74) : 798,	(24,74) : 986,	(25,74) : 116,	(26,74) : 896,	(27,74) : 875,	(28,74) : 875,	(29,74) : 859,	(30,74) : 882,	(31,74) : 820,	(32,74) : 841,	(33,74) : 962,	(34,74) : 785,	(35,74) : 945,	(36,74) : 541,	(37,74) : 439,	(38,74) : 836,	(39,74) : 849,	(40,74) : 121,
(1,75) : 592,	(2,75) : 636,	(3,75) : 688,	(4,75) : 8,	(5,75) : 657,	(6,75) : 608,	(7,75) : 587,	(8,75) : 971,	(9,75) : 748,	(10,75) : 588,	(11,75) : 637,	(12,75) : 255,	(13,75) : 421,	(14,75) : 744,	(15,75) : 149,	(16,75) : 120,	(17,75) : 43,	(18,75) : 639,	(19,75) : 267,	(20,75) : 587,	(21,75) : 296,	(22,75) : 615,	(23,75) : 634,	(24,75) : 40,	(25,75) : 962,	(26,75) : 358,	(27,75) : 415,	(28,75) : 586,	(29,75) : 168,	(30,75) : 589,	(31,75) : 612,	(32,75) : 513,	(33,75) : 517,	(34,75) : 599,	(35,75) : 718,	(36,75) : 810,	(37,75) : 754,	(38,75) : 636,	(39,75) : 637,	(40,75) : 989,
(1,76) : 554,	(2,76) : 599,	(3,76) : 668,	(4,76) : 46,	(5,76) : 620,	(6,76) : 571,	(7,76) : 549,	(8,76) : 954,	(9,76) : 714,	(10,76) : 550,	(11,76) : 600,	(12,76) : 242,	(13,76) : 383,	(14,76) : 710,	(15,76) : 111,	(16,76) : 140,	(17,76) : 8,	(18,76) : 602,	(19,76) : 247,	(20,76) : 549,	(21,76) : 259,	(22,76) : 578,	(23,76) : 596,	(24,76) : 8,	(25,76) : 938,	(26,76) : 320,	(27,76) : 377,	(28,76) : 548,	(29,76) : 139,	(30,76) : 551,	(31,76) : 574,	(32,76) : 475,	(33,76) : 480,	(34,76) : 561,	(35,76) : 680,	(36,76) : 776,	(37,76) : 722,	(38,76) : 598,	(39,76) : 599,	(40,76) : 965,
(1,77) : 467,	(2,77) : 508,	(3,77) : 592,	(4,77) : 143,	(5,77) : 526,	(6,77) : 477,	(7,77) : 461,	(8,77) : 881,	(9,77) : 614,	(10,77) : 462,	(11,77) : 509,	(12,77) : 275,	(13,77) : 297,	(14,77) : 610,	(15,77) : 37,	(16,77) : 234,	(17,77) : 94,	(18,77) : 508,	(19,77) : 254,	(20,77) : 461,	(21,77) : 161,	(22,77) : 480,	(23,77) : 502,	(24,77) : 96,	(25,77) : 852,	(26,77) : 236,	(27,77) : 290,	(28,77) : 460,	(29,77) : 59,	(30,77) : 464,	(31,77) : 481,	(32,77) : 384,	(33,77) : 405,	(34,77) : 466,	(35,77) : 596,	(36,77) : 676,	(37,77) : 625,	(38,77) : 507,	(39,77) : 508,	(40,77) : 879,
(1,78) : 463,	(2,78) : 497,	(3,78) : 503,	(4,78) : 206,	(5,78) : 506,	(6,78) : 459,	(7,78) : 455,	(8,78) : 792,	(9,78) : 556,	(10,78) : 457,	(11,78) : 498,	(12,78) : 363,	(13,78) : 301,	(14,78) : 553,	(15,78) : 117,	(16,78) : 312,	(17,78) : 164,	(18,78) : 487,	(19,78) : 338,	(20,78) : 457,	(21,78) : 147,	(22,78) : 447,	(23,78) : 481,	(24,78) : 165,	(25,78) : 768,	(26,78) : 252,	(27,78) : 295,	(28,78) : 456,	(29,78) : 33,	(30,78) : 460,	(31,78) : 465,	(32,78) : 376,	(33,78) : 422,	(34,78) : 444,	(35,78) : 595,	(36,78) : 619,	(37,78) : 555,	(38,78) : 492,	(39,78) : 497,	(40,78) : 794,
(1,79) : 534,	(2,79) : 581,	(3,79) : 698,	(4,79) : 79,	(5,79) : 605,	(6,79) : 556,	(7,79) : 529,	(8,79) : 986,	(9,79) : 717,	(10,79) : 530,	(11,79) : 581,	(12,79) : 194,	(13,79) : 364,	(14,79) : 713,	(15,79) : 105,	(16,79) : 126,	(17,79) : 54,	(18,79) : 588,	(19,79) : 198,	(20,79) : 529,	(21,79) : 255,	(22,79) : 570,	(23,79) : 582,	(24,79) : 56,	(25,79) : 962,	(26,79) : 300,	(27,79) : 358,	(28,79) : 528,	(29,79) : 163,	(30,79) : 530,	(31,79) : 558,	(32,79) : 458,	(33,79) : 452,	(34,79) : 549,	(35,79) : 657,	(36,79) : 779,	(37,79) : 733,	(38,79) : 582,	(39,79) : 581,	(40,79) : 989,
(1,80) : 230,	(2,80) : 276,	(3,80) : 602,	(4,80) : 369,	(5,80) : 302,	(6,80) : 253,	(7,80) : 225,	(8,80) : 884,	(9,80) : 459,	(10,80) : 226,	(11,80) : 277,	(12,80) : 337,	(13,80) : 60,	(14,80) : 454,	(15,80) : 214,	(16,80) : 422,	(17,80) : 320,	(18,80) : 285,	(19,80) : 278,	(20,80) : 225,	(21,80) : 108,	(22,80) : 279,	(23,80) : 280,	(24,80) : 323,	(25,80) : 802,	(26,80) : 9,	(27,80) : 54,	(28,80) : 224,	(29,80) : 253,	(30,80) : 227,	(31,80) : 255,	(32,80) : 154,	(33,80) : 172,	(34,80) : 248,	(35,80) : 358,	(36,80) : 517,	(37,80) : 509,	(38,80) : 277,	(39,80) : 277,	(40,80) : 823,
(1,81) : 574,	(2,81) : 627,	(3,81) : 830,	(4,81) : 184,	(5,81) : 660,	(6,81) : 612,	(7,81) : 572,	(8,81) : 1118,	(9,81) : 810,	(10,81) : 573,	(11,81) : 627,	(12,81) : 87,	(13,81) : 418,	(14,81) : 806,	(15,81) : 218,	(16,81) : 110,	(17,81) : 185,	(18,81) : 645,	(19,81) : 132,	(20,81) : 571,	(21,81) : 350,	(22,81) : 642,	(23,81) : 640,	(24,81) : 186,	(25,81) : 1087,	(26,81) : 356,	(27,81) : 413,	(28,81) : 571,	(29,81) : 294,	(30,81) : 572,	(31,81) : 612,	(32,81) : 512,	(33,81) : 474,	(34,81) : 610,	(35,81) : 684,	(36,81) : 871,	(37,81) : 839,	(38,81) : 631,	(39,81) : 628,	(40,81) : 1112,
(1,82) : 146,	(2,82) : 98,	(3,82) : 577,	(4,82) : 679,	(5,82) : 46,	(6,82) : 78,	(7,82) : 140,	(8,82) : 799,	(9,82) : 221,	(10,82) : 141,	(11,82) : 99,	(12,82) : 650,	(13,82) : 268,	(14,82) : 217,	(15,82) : 523,	(16,82) : 743,	(17,82) : 628,	(18,82) : 47,	(19,82) : 588,	(20,82) : 145,	(21,82) : 377,	(22,82) : 68,	(23,82) : 51,	(24,82) : 631,	(25,82) : 654,	(26,82) : 329,	(27,82) : 272,	(28,82) : 144,	(29,82) : 528,	(30,82) : 147,	(31,82) : 82,	(32,82) : 174,	(33,82) : 261,	(34,82) : 75,	(35,82) : 183,	(36,82) : 255,	(37,82) : 324,	(38,82) : 82,	(39,82) : 95,	(40,82) : 665,
(1,83) : 552,	(2,83) : 596,	(3,83) : 670,	(4,83) : 48,	(5,83) : 618,	(6,83) : 569,	(7,83) : 546,	(8,83) : 956,	(9,83) : 713,	(10,83) : 547,	(11,83) : 597,	(12,83) : 238,	(13,83) : 381,	(14,83) : 709,	(15,83) : 109,	(16,83) : 139,	(17,83) : 11,	(18,83) : 600,	(19,83) : 243,	(20,83) : 547,	(21,83) : 257,	(22,83) : 577,	(23,83) : 594,	(24,83) : 12,	(25,83) : 940,	(26,83) : 318,	(27,83) : 375,	(28,83) : 546,	(29,83) : 140,	(30,83) : 549,	(31,83) : 572,	(32,83) : 473,	(33,83) : 477,	(34,83) : 559,	(35,83) : 678,	(36,83) : 775,	(37,83) : 722,	(38,83) : 596,	(39,83) : 597,	(40,83) : 967,
(1,84) : 490,	(2,84) : 538,	(3,84) : 689,	(4,84) : 122,	(5,84) : 563,	(6,84) : 514,	(7,84) : 486,	(8,84) : 978,	(9,84) : 684,	(10,84) : 487,	(11,84) : 538,	(12,84) : 180,	(13,84) : 321,	(14,84) : 680,	(15,84) : 78,	(16,84) : 162,	(17,84) : 85,	(18,84) : 547,	(19,84) : 171,	(20,84) : 485,	(21,84) : 220,	(22,84) : 531,	(23,84) : 541,	(24,84) : 87,	(25,84) : 946,	(26,84) : 257,	(27,84) : 316,	(28,84) : 485,	(29,84) : 156,	(30,84) : 487,	(31,84) : 517,	(32,84) : 416,	(33,84) : 408,	(34,84) : 508,	(35,84) : 612,	(36,84) : 746,	(37,84) : 706,	(38,84) : 539,	(39,84) : 538,	(40,84) : 971,
(1,85) : 192,	(2,85) : 248,	(3,85) : 734,	(4,85) : 466,	(5,85) : 293,	(6,85) : 254,	(7,85) : 195,	(8,85) : 1008,	(9,85) : 520,	(10,85) : 194,	(11,85) : 248,	(12,85) : 344,	(13,85) : 129,	(14,85) : 515,	(15,85) : 325,	(16,85) : 487,	(17,85) : 422,	(18,85) : 284,	(19,85) : 281,	(20,85) : 191,	(21,85) : 254,	(22,85) : 316,	(23,85) : 280,	(24,85) : 425,	(25,85) : 907,	(26,85) : 143,	(27,85) : 132,	(28,85) : 192,	(29,85) : 386,	(30,85) : 190,	(31,85) : 249,	(32,85) : 172,	(33,85) : 77,	(34,85) : 264,	(35,85) : 285,	(36,85) : 570,	(37,85) : 594,	(38,85) : 258,	(39,85) : 250,	(40,85) : 926,
(1,86) : 156,	(2,86) : 128,	(3,86) : 522,	(4,86) : 624,	(5,86) : 90,	(6,86) : 82,	(7,86) : 147,	(8,86) : 756,	(9,86) : 205,	(10,86) : 149,	(11,86) : 129,	(12,86) : 616,	(13,86) : 231,	(14,86) : 200,	(15,86) : 470,	(16,86) : 695,	(17,86) : 574,	(18,86) : 76,	(19,86) : 555,	(20,86) : 153,	(21,86) : 321,	(22,86) : 1,	(23,86) : 74,	(24,86) : 576,	(25,86) : 622,	(26,86) : 288,	(27,86) : 234,	(28,86) : 151,	(29,86) : 468,	(30,86) : 156,	(31,86) : 92,	(32,86) : 147,	(33,86) : 256,	(34,86) : 59,	(35,86) : 233,	(36,86) : 253,	(37,86) : 294,	(38,86) : 111,	(39,86) : 124,	(40,86) : 636,
(1,87) : 457,	(2,87) : 511,	(3,87) : 809,	(4,87) : 265,	(5,87) : 549,	(6,87) : 503,	(7,87) : 456,	(8,87) : 1098,	(9,87) : 726,	(10,87) : 456,	(11,87) : 512,	(12,87) : 72,	(13,87) : 314,	(14,87) : 722,	(15,87) : 209,	(16,87) : 234,	(17,87) : 241,	(18,87) : 536,	(19,87) : 13,	(20,87) : 454,	(21,87) : 290,	(22,87) : 543,	(23,87) : 531,	(24,87) : 243,	(25,87) : 1043,	(26,87) : 259,	(27,87) : 310,	(28,87) : 454,	(29,87) : 299,	(30,87) : 454,	(31,87) : 502,	(32,87) : 405,	(33,87) : 350,	(34,87) : 504,	(35,87) : 560,	(36,87) : 784,	(37,87) : 770,	(38,87) : 518,	(39,87) : 513,	(40,87) : 1067,
(1,88) : 209,	(2,88) : 193,	(3,88) : 454,	(4,88) : 583,	(5,88) : 163,	(6,88) : 142,	(7,88) : 198,	(8,88) : 698,	(9,88) : 193,	(10,88) : 200,	(11,88) : 194,	(12,88) : 603,	(13,88) : 228,	(14,88) : 188,	(15,88) : 433,	(16,88) : 662,	(17,88) : 533,	(18,88) : 146,	(19,88) : 545,	(20,88) : 203,	(21,88) : 282,	(22,88) : 73,	(23,88) : 143,	(24,88) : 535,	(25,88) : 577,	(26,88) : 275,	(27,88) : 230,	(28,88) : 201,	(29,88) : 419,	(30,88) : 208,	(31,88) : 154,	(32,88) : 167,	(33,88) : 287,	(34,88) : 117,	(35,88) : 303,	(36,88) : 251,	(37,88) : 257,	(38,88) : 177,	(39,88) : 190,	(40,88) : 593,
(1,89) : 14,	(2,89) : 45,	(3,89) : 649,	(4,89) : 604,	(5,89) : 91,	(6,89) : 62,	(7,89) : 10,	(8,89) : 895,	(9,89) : 344,	(10,89) : 10,	(11,89) : 46,	(12,89) : 538,	(13,89) : 175,	(14,89) : 339,	(15,89) : 448,	(16,89) : 652,	(17,89) : 554,	(18,89) : 86,	(19,89) : 474,	(20,89) : 13,	(21,89) : 318,	(22,89) : 142,	(23,89) : 82,	(24,89) : 557,	(25,89) : 765,	(26,89) : 239,	(27,89) : 181,	(28,89) : 12,	(29,89) : 474,	(30,89) : 15,	(31,89) : 53,	(32,89) : 88,	(33,89) : 130,	(34,89) : 84,	(35,89) : 133,	(36,89) : 385,	(37,89) : 437,	(38,89) : 54,	(39,89) : 47,	(40,89) : 779,
(1,90) : 680,	(2,90) : 649,	(3,90) : 244,	(4,90) : 885,	(5,90) : 602,	(6,90) : 606,	(7,90) : 670,	(8,90) : 262,	(9,90) : 344,	(10,90) : 673,	(11,90) : 650,	(12,90) : 1010,	(13,90) : 687,	(14,90) : 347,	(15,90) : 780,	(16,90) : 995,	(17,90) : 847,	(18,90) : 594,	(19,90) : 963,	(20,90) : 676,	(21,90) : 661,	(22,90) : 525,	(23,90) : 594,	(24,90) : 848,	(25,90) : 98,	(26,90) : 714,	(27,90) : 687,	(28,90) : 674,	(29,90) : 716,	(30,90) : 680,	(31,90) : 617,	(32,90) : 645,	(33,90) : 765,	(34,90) : 583,	(35,90) : 740,	(36,90) : 339,	(37,90) : 234,	(38,90) : 632,	(39,90) : 646,	(40,90) : 117,
(1,91) : 551,	(2,91) : 595,	(3,91) : 660,	(4,91) : 50,	(5,91) : 616,	(6,91) : 567,	(7,91) : 546,	(8,91) : 945,	(9,91) : 707,	(10,91) : 547,	(11,91) : 596,	(12,91) : 249,	(13,91) : 380,	(14,91) : 703,	(15,91) : 107,	(16,91) : 148,	(17,91) : 1,	(18,91) : 598,	(19,91) : 252,	(20,91) : 546,	(21,91) : 254,	(22,91) : 573,	(23,91) : 592,	(24,91) : 2,	(25,91) : 930,	(26,91) : 318,	(27,91) : 374,	(28,91) : 545,	(29,91) : 131,	(30,91) : 548,	(31,91) : 570,	(32,91) : 471,	(33,91) : 479,	(34,91) : 557,	(35,91) : 678,	(36,91) : 769,	(37,91) : 715,	(38,91) : 595,	(39,91) : 596,	(40,91) : 957,
(1,92) : 80,	(2,92) : 120,	(3,92) : 735,	(4,92) : 611,	(5,92) : 173,	(6,92) : 155,	(7,92) : 90,	(8,92) : 986,	(9,92) : 435,	(10,92) : 88,	(11,92) : 120,	(12,92) : 505,	(13,92) : 196,	(14,92) : 431,	(15,92) : 460,	(16,92) : 643,	(17,92) : 563,	(18,92) : 173,	(19,92) : 441,	(20,92) : 84,	(21,92) : 350,	(22,92) : 235,	(23,92) : 171,	(24,92) : 566,	(25,92) : 858,	(26,92) : 251,	(27,92) : 202,	(28,92) : 86,	(29,92) : 502,	(30,92) : 80,	(31,92) : 145,	(32,92) : 145,	(33,92) : 88,	(34,92) : 177,	(35,92) : 124,	(36,92) : 473,	(37,92) : 530,	(38,92) : 136,	(39,92) : 124,	(40,92) : 872,
(1,93) : 230,	(2,93) : 263,	(3,93) : 526,	(4,93) : 393,	(5,93) : 275,	(6,93) : 227,	(7,93) : 221,	(8,93) : 805,	(9,93) : 392,	(10,93) : 223,	(11,93) : 264,	(12,93) : 404,	(13,93) : 87,	(14,93) : 387,	(15,93) : 238,	(16,93) : 463,	(17,93) : 343,	(18,93) : 256,	(19,93) : 349,	(20,93) : 223,	(21,93) : 92,	(22,93) : 231,	(23,93) : 251,	(24,93) : 345,	(25,93) : 722,	(26,93) : 88,	(27,93) : 82,	(28,93) : 222,	(29,93) : 247,	(30,93) : 227,	(31,93) : 232,	(32,93) : 142,	(33,93) : 212,	(34,93) : 215,	(35,93) : 362,	(36,93) : 452,	(37,93) : 434,	(38,93) : 259,	(39,93) : 262,	(40,93) : 744,
(1,94) : 539,	(2,94) : 581,	(3,94) : 622,	(4,94) : 77,	(5,94) : 598,	(6,94) : 549,	(7,94) : 533,	(8,94) : 907,	(9,94) : 678,	(10,94) : 534,	(11,94) : 582,	(12,94) : 279,	(13,94) : 368,	(14,94) : 674,	(15,94) : 97,	(16,94) : 186,	(17,94) : 38,	(18,94) : 580,	(19,94) : 276,	(20,94) : 533,	(21,94) : 232,	(22,94) : 551,	(23,94) : 574,	(24,94) : 38,	(25,94) : 893,	(26,94) : 307,	(27,94) : 362,	(28,94) : 532,	(29,94) : 97,	(30,94) : 536,	(31,94) : 554,	(32,94) : 456,	(33,94) : 473,	(34,94) : 538,	(35,94) : 667,	(36,94) : 741,	(37,94) : 682,	(38,94) : 579,	(39,94) : 581,	(40,94) : 920,
(1,95) : 626,	(2,95) : 675,	(3,95) : 787,	(4,95) : 101,	(5,95) : 702,	(6,95) : 653,	(7,95) : 622,	(8,95) : 1072,	(9,95) : 821,	(10,95) : 623,	(11,95) : 676,	(12,95) : 191,	(13,95) : 459,	(14,95) : 817,	(15,95) : 209,	(16,95) : 22,	(17,95) : 128,	(18,95) : 686,	(19,95) : 226,	(20,95) : 621,	(21,95) : 358,	(22,95) : 671,	(23,95) : 680,	(24,95) : 127,	(25,95) : 1058,	(26,95) : 395,	(27,95) : 454,	(28,95) : 621,	(29,95) : 258,	(30,95) : 623,	(31,95) : 655,	(32,95) : 554,	(33,95) : 537,	(34,95) : 647,	(35,95) : 745,	(36,95) : 883,	(37,95) : 836,	(38,95) : 677,	(39,95) : 676,	(40,95) : 1085,
(1,96) : 245,	(2,96) : 218,	(3,96) : 437,	(4,96) : 633,	(5,96) : 177,	(6,96) : 171,	(7,96) : 234,	(8,96) : 666,	(9,96) : 136,	(10,96) : 237,	(11,96) : 219,	(12,96) : 660,	(13,96) : 284,	(14,96) : 131,	(15,96) : 485,	(16,96) : 715,	(17,96) : 584,	(18,96) : 164,	(19,96) : 602,	(20,96) : 241,	(21,96) : 335,	(22,96) : 91,	(23,96) : 163,	(24,96) : 586,	(25,96) : 534,	(26,96) : 332,	(27,96) : 286,	(28,96) : 238,	(29,96) : 466,	(30,96) : 244,	(31,96) : 182,	(32,96) : 217,	(33,96) : 335,	(34,96) : 148,	(35,96) : 320,	(36,96) : 194,	(37,96) : 208,	(38,96) : 201,	(39,96) : 214,	(40,96) : 548,
(1,97) : 354,	(2,97) : 310,	(3,97) : 450,	(4,97) : 778,	(5,97) : 258,	(6,97) : 279,	(7,97) : 346,	(8,97) : 620,	(9,97) : 34,	(10,97) : 348,	(11,97) : 311,	(12,97) : 810,	(13,97) : 430,	(14,97) : 35,	(15,97) : 633,	(16,97) : 864,	(17,97) : 730,	(18,97) : 257,	(19,97) : 751,	(20,97) : 352,	(21,97) : 484,	(22,97) : 208,	(23,97) : 259,	(24,97) : 732,	(25,97) : 456,	(26,97) : 481,	(27,97) : 433,	(28,97) : 350,	(29,97) : 608,	(30,97) : 355,	(31,97) : 288,	(32,97) : 353,	(33,97) : 462,	(34,97) : 263,	(35,97) : 384,	(36,97) : 46,	(37,97) : 141,	(38,97) : 294,	(39,97) : 307,	(40,97) : 463,
(1,98) : 197,	(2,98) : 187,	(3,98) : 462,	(4,98) : 564,	(5,98) : 162,	(6,98) : 136,	(7,98) : 186,	(8,98) : 712,	(9,98) : 214,	(10,98) : 189,	(11,98) : 188,	(12,98) : 582,	(13,98) : 208,	(14,98) : 209,	(15,98) : 413,	(16,98) : 642,	(17,98) : 515,	(18,98) : 144,	(19,98) : 524,	(20,98) : 192,	(21,98) : 263,	(22,98) : 76,	(23,98) : 141,	(24,98) : 517,	(25,98) : 593,	(26,98) : 254,	(27,98) : 209,	(28,98) : 190,	(29,98) : 402,	(30,98) : 196,	(31,98) : 147,	(32,98) : 150,	(33,98) : 270,	(34,98) : 110,	(35,98) : 299,	(36,98) : 272,	(37,98) : 276,	(38,98) : 173,	(39,98) : 184,	(40,98) : 610,
(1,99) : 499,	(2,99) : 544,	(3,99) : 646,	(4,99) : 100,	(5,99) : 565,	(6,99) : 516,	(7,99) : 494,	(8,99) : 935,	(9,99) : 667,	(10,99) : 495,	(11,99) : 545,	(12,99) : 231,	(13,99) : 329,	(14,99) : 662,	(15,99) : 57,	(16,99) : 178,	(17,99) : 52,	(18,99) : 548,	(19,99) : 221,	(20,99) : 494,	(21,99) : 207,	(22,99) : 526,	(23,99) : 542,	(24,99) : 54,	(25,99) : 909,	(26,99) : 266,	(27,99) : 323,	(28,99) : 493,	(29,99) : 111,	(30,99) : 496,	(31,99) : 520,	(32,99) : 420,	(33,99) : 427,	(34,99) : 507,	(35,99) : 626,	(36,99) : 729,	(37,99) : 680,	(38,99) : 544,	(39,99) : 544,	(40,99) : 935,
(1,100) : 539,	(2,100) : 583,	(3,100) : 657,	(4,100) : 61,	(5,100) : 604,	(6,100) : 555,	(7,100) : 534,	(8,100) : 943,	(9,100) : 698,	(10,100) : 535,	(11,100) : 584,	(12,100) : 243,	(13,100) : 368,	(14,100) : 694,	(15,100) : 96,	(16,100) : 154,	(17,100) : 11,	(18,100) : 586,	(19,100) : 243,	(20,100) : 534,	(21,100) : 243,	(22,100) : 562,	(23,100) : 581,	(24,100) : 14,	(25,100) : 925,	(26,100) : 305,	(27,100) : 362,	(28,100) : 533,	(29,100) : 125,	(30,100) : 536,	(31,100) : 558,	(32,100) : 460,	(33,100) : 467,	(34,100) : 545,	(35,100) : 666,	(36,100) : 760,	(37,100) : 707,	(38,100) : 583,	(39,100) : 584,	(40,100) : 952,
(1,101) : 349,	(2,101) : 399,	(3,101) : 680,	(4,101) : 268,	(5,101) : 431,	(6,101) : 382,	(7,101) : 346,	(8,101) : 969,	(9,101) : 589,	(10,101) : 346,	(11,101) : 400,	(12,101) : 207,	(13,101) : 187,	(14,101) : 584,	(15,101) : 135,	(16,101) : 298,	(17,101) : 224,	(18,101) : 415,	(19,101) : 152,	(20,101) : 345,	(21,101) : 154,	(22,101) : 413,	(23,101) : 410,	(24,101) : 227,	(25,101) : 906,	(26,101) : 126,	(27,101) : 183,	(28,101) : 344,	(29,101) : 214,	(30,101) : 346,	(31,101) : 383,	(32,101) : 282,	(33,101) : 260,	(34,101) : 380,	(35,101) : 467,	(36,101) : 648,	(37,101) : 630,	(38,101) : 403,	(39,101) : 401,	(40,101) : 929,
(1,102) : 449,	(2,102) : 479,	(3,102) : 461,	(4,102) : 249,	(5,102) : 483,	(6,102) : 438,	(7,102) : 440,	(8,102) : 751,	(9,102) : 518,	(10,102) : 442,	(11,102) : 480,	(12,102) : 401,	(13,102) : 294,	(14,102) : 514,	(15,102) : 154,	(16,102) : 356,	(17,102) : 208,	(18,102) : 464,	(19,102) : 372,	(20,102) : 443,	(21,102) : 142,	(22,102) : 419,	(23,102) : 459,	(24,102) : 209,	(25,102) : 723,	(26,102) : 252,	(27,102) : 288,	(28,102) : 441,	(29,102) : 77,	(30,102) : 446,	(31,102) : 445,	(32,102) : 360,	(33,102) : 419,	(34,102) : 421,	(35,102) : 581,	(36,102) : 580,	(37,102) : 513,	(38,102) : 473,	(39,102) : 478,	(40,102) : 750,
(1,103) : 466,	(2,103) : 514,	(3,103) : 691,	(4,103) : 149,	(5,103) : 541,	(6,103) : 492,	(7,103) : 462,	(8,103) : 981,	(9,103) : 670,	(10,103) : 463,	(11,103) : 515,	(12,103) : 170,	(13,103) : 298,	(14,103) : 666,	(15,103) : 77,	(16,103) : 182,	(17,103) : 111,	(18,103) : 525,	(19,103) : 152,	(20,103) : 461,	(21,103) : 207,	(22,103) : 512,	(23,103) : 519,	(24,103) : 113,	(25,103) : 942,	(26,103) : 234,	(27,103) : 293,	(28,103) : 461,	(29,103) : 164,	(30,103) : 463,	(31,103) : 494,	(32,103) : 393,	(33,103) : 382,	(34,103) : 487,	(35,103) : 587,	(36,103) : 731,	(37,103) : 695,	(38,103) : 516,	(39,103) : 515,	(40,103) : 968,
(1,104) : 232,	(2,104) : 194,	(3,104) : 482,	(4,104) : 678,	(5,104) : 146,	(6,104) : 156,	(7,104) : 223,	(8,104) : 697,	(9,104) : 126,	(10,104) : 225,	(11,104) : 195,	(12,104) : 688,	(13,104) : 306,	(14,104) : 121,	(15,104) : 527,	(16,104) : 756,	(17,104) : 628,	(18,104) : 139,	(19,104) : 628,	(20,104) : 229,	(21,104) : 377,	(22,104) : 81,	(23,104) : 140,	(24,104) : 630,	(25,104) : 553,	(26,104) : 359,	(27,104) : 309,	(28,104) : 227,	(29,104) : 514,	(30,104) : 232,	(31,104) : 166,	(32,104) : 226,	(33,104) : 335,	(34,104) : 137,	(35,104) : 286,	(36,104) : 173,	(37,104) : 223,	(38,104) : 177,	(39,104) : 191,	(40,104) : 565,
(1,105) : 269,	(2,105) : 300,	(3,105) : 502,	(4,105) : 366,	(5,105) : 309,	(6,105) : 262,	(7,105) : 260,	(8,105) : 785,	(9,105) : 403,	(10,105) : 262,	(11,105) : 301,	(12,105) : 401,	(13,105) : 124,	(14,105) : 399,	(15,105) : 213,	(16,105) : 443,	(17,105) : 316,	(18,105) : 290,	(19,105) : 349,	(20,105) : 263,	(21,105) : 63,	(22,105) : 257,	(23,105) : 285,	(24,105) : 318,	(25,105) : 711,	(26,105) : 109,	(27,105) : 119,	(28,105) : 261,	(29,105) : 212,	(30,105) : 266,	(31,105) : 268,	(32,105) : 181,	(33,105) : 251,	(34,105) : 248,	(35,105) : 401,	(36,105) : 464,	(37,105) : 435,	(38,105) : 295,	(39,105) : 300,	(40,105) : 733,
(1,106) : 90,	(2,106) : 142,	(3,106) : 648,	(4,106) : 512,	(5,106) : 181,	(6,106) : 137,	(7,106) : 88,	(8,106) : 912,	(9,106) : 402,	(10,106) : 88,	(11,106) : 143,	(12,106) : 441,	(13,106) : 87,	(14,106) : 398,	(15,106) : 358,	(16,106) : 556,	(17,106) : 464,	(18,106) : 169,	(19,106) : 378,	(20,106) : 86,	(21,106) : 240,	(22,106) : 198,	(23,106) : 165,	(24,106) : 466,	(25,106) : 800,	(26,106) : 147,	(27,106) : 93,	(28,106) : 86,	(29,106) : 394,	(30,106) : 87,	(31,106) : 134,	(32,106) : 57,	(33,106) : 65,	(34,106) : 146,	(35,106) : 214,	(36,106) : 451,	(37,106) : 480,	(38,106) : 148,	(39,106) : 144,	(40,106) : 817,
(1,107) : 534,	(2,107) : 577,	(3,107) : 650,	(4,107) : 68,	(5,107) : 598,	(6,107) : 549,	(7,107) : 528,	(8,107) : 936,	(9,107) : 691,	(10,107) : 529,	(11,107) : 578,	(12,107) : 247,	(13,107) : 363,	(14,107) : 686,	(15,107) : 90,	(16,107) : 161,	(17,107) : 17,	(18,107) : 580,	(19,107) : 245,	(20,107) : 528,	(21,107) : 236,	(22,107) : 555,	(23,107) : 574,	(24,107) : 20,	(25,107) : 918,	(26,107) : 300,	(27,107) : 357,	(28,107) : 527,	(29,107) : 118,	(30,107) : 531,	(31,107) : 552,	(32,107) : 454,	(33,107) : 462,	(34,107) : 539,	(35,107) : 661,	(36,107) : 753,	(37,107) : 699,	(38,107) : 577,	(39,107) : 578,	(40,107) : 945,
(1,108) : 567,	(2,108) : 606,	(3,108) : 596,	(4,108) : 95,	(5,108) : 620,	(6,108) : 572,	(7,108) : 560,	(8,108) : 878,	(9,108) : 682,	(10,108) : 561,	(11,108) : 607,	(12,108) : 325,	(13,108) : 397,	(14,108) : 678,	(15,108) : 135,	(16,108) : 212,	(17,108) : 78,	(18,108) : 601,	(19,108) : 323,	(20,108) : 561,	(21,108) : 253,	(22,108) : 567,	(23,108) : 596,	(24,108) : 77,	(25,108) : 873,	(26,108) : 339,	(27,108) : 391,	(28,108) : 560,	(29,108) : 101,	(30,108) : 564,	(31,108) : 577,	(32,108) : 482,	(33,108) : 508,	(34,108) : 559,	(35,108) : 697,	(36,108) : 744,	(37,108) : 678,	(38,108) : 603,	(39,108) : 606,	(40,108) : 901,
(1,109) : 416,	(2,109) : 450,	(3,109) : 503,	(4,109) : 234,	(5,109) : 459,	(6,109) : 412,	(7,109) : 408,	(8,109) : 793,	(9,109) : 521,	(10,109) : 409,	(11,109) : 451,	(12,109) : 356,	(13,109) : 254,	(14,109) : 517,	(15,109) : 113,	(16,109) : 332,	(17,109) : 188,	(18,109) : 440,	(19,109) : 323,	(20,109) : 410,	(21,109) : 100,	(22,109) : 403,	(23,109) : 435,	(24,109) : 189,	(25,109) : 756,	(26,109) : 206,	(27,109) : 248,	(28,109) : 408,	(29,109) : 64,	(30,109) : 413,	(31,109) : 418,	(32,109) : 328,	(33,109) : 376,	(34,109) : 398,	(35,109) : 548,	(36,109) : 583,	(37,109) : 527,	(38,109) : 445,	(39,109) : 449,	(40,109) : 782,
(1,110) : 497,	(2,110) : 542,	(3,110) : 662,	(4,110) : 104,	(5,110) : 565,	(6,110) : 516,	(7,110) : 492,	(8,110) : 950,	(9,110) : 673,	(10,110) : 493,	(11,110) : 543,	(12,110) : 213,	(13,110) : 326,	(14,110) : 669,	(15,110) : 61,	(16,110) : 170,	(17,110) : 59,	(18,110) : 548,	(19,110) : 204,	(20,110) : 492,	(21,110) : 211,	(22,110) : 528,	(23,110) : 542,	(24,110) : 62,	(25,110) : 922,	(26,110) : 263,	(27,110) : 320,	(28,110) : 491,	(29,110) : 126,	(30,110) : 494,	(31,110) : 519,	(32,110) : 419,	(33,110) : 420,	(34,110) : 508,	(35,110) : 622,	(36,110) : 735,	(37,110) : 690,	(38,110) : 543,	(39,110) : 543,	(40,110) : 948,
(1,111) : 559,	(2,111) : 601,	(3,111) : 636,	(4,111) : 58,	(5,111) : 619,	(6,111) : 571,	(7,111) : 553,	(8,111) : 921,	(9,111) : 700,	(10,111) : 554,	(11,111) : 602,	(12,111) : 279,	(13,111) : 388,	(14,111) : 696,	(15,111) : 116,	(16,111) : 170,	(17,111) : 31,	(18,111) : 601,	(19,111) : 280,	(20,111) : 554,	(21,111) : 254,	(22,111) : 573,	(23,111) : 595,	(24,111) : 29,	(25,111) : 910,	(26,111) : 327,	(27,111) : 382,	(28,111) : 553,	(29,111) : 117,	(30,111) : 556,	(31,111) : 575,	(32,111) : 477,	(33,111) : 492,	(34,111) : 560,	(35,111) : 687,	(36,111) : 762,	(37,111) : 703,	(38,111) : 600,	(39,111) : 602,	(40,111) : 937,
(1,112) : 39,	(2,112) : 82,	(3,112) : 631,	(4,112) : 564,	(5,112) : 118,	(6,112) : 76,	(7,112) : 31,	(8,112) : 884,	(9,112) : 351,	(10,112) : 32,	(11,112) : 82,	(12,112) : 504,	(13,112) : 136,	(14,112) : 346,	(15,112) : 408,	(16,112) : 614,	(17,112) : 515,	(18,112) : 107,	(19,112) : 440,	(20,112) : 33,	(21,112) : 278,	(22,112) : 145,	(23,112) : 102,	(24,112) : 517,	(25,112) : 762,	(26,112) : 199,	(27,112) : 141,	(28,112) : 31,	(29,112) : 435,	(30,112) : 36,	(31,112) : 71,	(32,112) : 49,	(33,112) : 110,	(34,112) : 89,	(35,112) : 171,	(36,112) : 397,	(37,112) : 437,	(38,112) : 86,	(39,112) : 83,	(40,112) : 777,
(1,113) : 62,	(2,113) : 36,	(3,113) : 613,	(4,113) : 620,	(5,113) : 49,	(6,113) : 17,	(7,113) : 54,	(8,113) : 853,	(9,113) : 296,	(10,113) : 56,	(11,113) : 37,	(12,113) : 572,	(13,113) : 196,	(14,113) : 292,	(15,113) : 464,	(16,113) : 676,	(17,113) : 570,	(18,113) : 39,	(19,113) : 508,	(20,113) : 60,	(21,113) : 325,	(22,113) : 97,	(23,113) : 35,	(24,113) : 573,	(25,113) : 720,	(26,113) : 260,	(27,113) : 201,	(28,113) : 58,	(29,113) : 481,	(30,113) : 63,	(31,113) : 5,	(32,113) : 101,	(33,113) : 175,	(34,113) : 42,	(35,113) : 148,	(36,113) : 337,	(37,113) : 391,	(38,113) : 26,	(39,113) : 34,	(40,113) : 733,
(1,114) : 586,	(2,114) : 631,	(3,114) : 684,	(4,114) : 14,	(5,114) : 651,	(6,114) : 602,	(7,114) : 581,	(8,114) : 967,	(9,114) : 742,	(10,114) : 582,	(11,114) : 631,	(12,114) : 253,	(13,114) : 415,	(14,114) : 738,	(15,114) : 143,	(16,114) : 123,	(17,114) : 37,	(18,114) : 633,	(19,114) : 265,	(20,114) : 581,	(21,114) : 290,	(22,114) : 609,	(23,114) : 628,	(24,114) : 34,	(25,114) : 958,	(26,114) : 352,	(27,114) : 409,	(28,114) : 580,	(29,114) : 163,	(30,114) : 583,	(31,114) : 606,	(32,114) : 507,	(33,114) : 511,	(34,114) : 593,	(35,114) : 712,	(36,114) : 805,	(37,114) : 748,	(38,114) : 630,	(39,114) : 631,	(40,114) : 985,
(1,115) : 563,	(2,115) : 608,	(3,115) : 681,	(4,115) : 37,	(5,115) : 630,	(6,115) : 581,	(7,115) : 558,	(8,115) : 966,	(9,115) : 726,	(10,115) : 559,	(11,115) : 609,	(12,115) : 236,	(13,115) : 392,	(14,115) : 722,	(15,115) : 122,	(16,115) : 127,	(17,115) : 21,	(18,115) : 612,	(19,115) : 244,	(20,115) : 558,	(21,115) : 270,	(22,115) : 589,	(23,115) : 607,	(24,115) : 20,	(25,115) : 951,	(26,115) : 329,	(27,115) : 387,	(28,115) : 557,	(29,115) : 152,	(30,115) : 560,	(31,115) : 584,	(32,115) : 485,	(33,115) : 488,	(34,115) : 572,	(35,115) : 689,	(36,115) : 789,	(37,115) : 735,	(38,115) : 608,	(39,115) : 609,	(40,115) : 978,
(1,116) : 478,	(2,116) : 526,	(3,116) : 684,	(4,116) : 132,	(5,116) : 552,	(6,116) : 503,	(7,116) : 474,	(8,116) : 974,	(9,116) : 674,	(10,116) : 475,	(11,116) : 527,	(12,116) : 181,	(13,116) : 310,	(14,116) : 670,	(15,116) : 71,	(16,116) : 173,	(17,116) : 93,	(18,116) : 535,	(19,116) : 167,	(20,116) : 474,	(21,116) : 210,	(22,116) : 520,	(23,116) : 530,	(24,116) : 96,	(25,116) : 939,	(26,116) : 246,	(27,116) : 304,	(28,116) : 473,	(29,116) : 153,	(30,116) : 475,	(31,116) : 505,	(32,116) : 404,	(33,116) : 396,	(34,116) : 496,	(35,116) : 601,	(36,116) : 736,	(37,116) : 697,	(38,116) : 528,	(39,116) : 527,	(40,116) : 965,
(1,117) : 682,	(2,117) : 735,	(3,117) : 933,	(4,117) : 255,	(5,117) : 771,	(6,117) : 724,	(7,117) : 681,	(8,117) : 1220,	(9,117) : 926,	(10,117) : 681,	(11,117) : 736,	(12,117) : 158,	(13,117) : 530,	(14,117) : 921,	(15,117) : 329,	(16,117) : 141,	(17,117) : 275,	(18,117) : 757,	(19,117) : 221,	(20,117) : 679,	(21,117) : 465,	(22,117) : 756,	(23,117) : 751,	(24,117) : 275,	(25,117) : 1196,	(26,117) : 470,	(27,117) : 526,	(28,117) : 679,	(29,117) : 398,	(30,117) : 679,	(31,117) : 723,	(32,117) : 624,	(33,117) : 577,	(34,117) : 723,	(35,117) : 786,	(36,117) : 986,	(37,117) : 954,	(38,117) : 741,	(39,117) : 737,	(40,117) : 1222,
(1,118) : 319,	(2,118) : 302,	(3,118) : 345,	(4,118) : 597,	(5,118) : 267,	(6,118) : 253,	(7,118) : 308,	(8,118) : 588,	(9,118) : 162,	(10,118) : 311,	(11,118) : 304,	(12,118) : 662,	(13,118) : 313,	(14,118) : 159,	(15,118) : 460,	(16,118) : 690,	(17,118) : 551,	(18,118) : 252,	(19,118) : 608,	(20,118) : 314,	(21,118) : 317,	(22,118) : 177,	(23,118) : 250,	(24,118) : 552,	(25,118) : 474,	(26,118) : 345,	(27,118) : 313,	(28,118) : 312,	(29,118) : 425,	(30,118) : 318,	(31,118) : 264,	(32,118) : 271,	(33,118) : 392,	(34,118) : 227,	(35,118) : 410,	(36,118) : 224,	(37,118) : 174,	(38,118) : 286,	(39,118) : 299,	(40,118) : 493,
(1,119) : 362,	(2,119) : 317,	(3,119) : 457,	(4,119) : 791,	(5,119) : 265,	(6,119) : 287,	(7,119) : 354,	(8,119) : 622,	(9,119) : 44,	(10,119) : 356,	(11,119) : 318,	(12,119) : 821,	(13,119) : 441,	(14,119) : 46,	(15,119) : 646,	(16,119) : 877,	(17,119) : 743,	(18,119) : 264,	(19,119) : 762,	(20,119) : 360,	(21,119) : 496,	(22,119) : 218,	(23,119) : 266,	(24,119) : 744,	(25,119) : 455,	(26,119) : 492,	(27,119) : 444,	(28,119) : 358,	(29,119) : 621,	(30,119) : 363,	(31,119) : 296,	(32,119) : 363,	(33,119) : 470,	(34,119) : 272,	(35,119) : 388,	(36,119) : 36,	(37,119) : 145,	(38,119) : 301,	(39,119) : 314,	(40,119) : 462,
(1,120) : 555,	(2,120) : 600,	(3,120) : 665,	(4,120) : 45,	(5,120) : 620,	(6,120) : 571,	(7,120) : 550,	(8,120) : 951,	(9,120) : 713,	(10,120) : 551,	(11,120) : 600,	(12,120) : 246,	(13,120) : 384,	(14,120) : 709,	(15,120) : 112,	(16,120) : 143,	(17,120) : 6,	(18,120) : 602,	(19,120) : 250,	(20,120) : 550,	(21,120) : 259,	(22,120) : 578,	(23,120) : 597,	(24,120) : 5,	(25,120) : 936,	(26,120) : 322,	(27,120) : 378,	(28,120) : 549,	(29,120) : 137,	(30,120) : 552,	(31,120) : 575,	(32,120) : 476,	(33,120) : 482,	(34,120) : 562,	(35,120) : 682,	(36,120) : 775,	(37,120) : 721,	(38,120) : 599,	(39,120) : 600,	(40,120) : 963,
(1,121) : 663,	(2,121) : 652,	(3,121) : 16,	(4,121) : 681,	(5,121) : 618,	(6,121) : 601,	(7,121) : 652,	(8,121) : 297,	(9,121) : 429,	(10,121) : 654,	(11,121) : 653,	(12,121) : 852,	(13,121) : 605,	(14,121) : 429,	(15,121) : 607,	(16,121) : 797,	(17,121) : 651,	(18,121) : 603,	(19,121) : 816,	(20,121) : 657,	(21,121) : 524,	(22,121) : 528,	(23,121) : 601,	(24,121) : 651,	(25,121) : 309,	(26,121) : 609,	(27,121) : 602,	(28,121) : 655,	(29,121) : 528,	(30,121) : 662,	(31,121) : 613,	(32,121) : 599,	(33,121) : 713,	(34,121) : 576,	(35,121) : 761,	(36,121) : 461,	(37,121) : 329,	(38,121) : 636,	(39,121) : 649,	(40,121) : 339,
(1,122) : 23,	(2,122) : 74,	(3,122) : 678,	(4,122) : 597,	(5,122) : 125,	(6,122) : 99,	(7,122) : 33,	(8,122) : 928,	(9,122) : 381,	(10,122) : 30,	(11,122) : 74,	(12,122) : 516,	(13,122) : 170,	(14,122) : 376,	(15,122) : 442,	(16,122) : 639,	(17,122) : 548,	(18,122) : 121,	(19,122) : 452,	(20,122) : 27,	(21,122) : 320,	(22,122) : 178,	(23,122) : 118,	(24,122) : 550,	(25,122) : 801,	(26,122) : 231,	(27,122) : 176,	(28,122) : 29,	(29,122) : 475,	(30,122) : 23,	(31,122) : 89,	(32,122) : 97,	(33,122) : 100,	(34,122) : 120,	(35,122) : 130,	(36,122) : 421,	(37,122) : 473,	(38,122) : 87,	(39,122) : 77,	(40,122) : 815,
(1,123) : 192,	(2,123) : 152,	(3,123) : 519,	(4,123) : 671,	(5,123) : 103,	(6,123) : 117,	(7,123) : 184,	(8,123) : 739,	(9,123) : 166,	(10,123) : 186,	(11,123) : 153,	(12,123) : 666,	(13,123) : 282,	(14,123) : 161,	(15,123) : 518,	(16,123) : 744,	(17,123) : 621,	(18,123) : 97,	(19,123) : 605,	(20,123) : 190,	(21,123) : 368,	(22,123) : 52,	(23,123) : 98,	(24,123) : 623,	(25,123) : 595,	(26,123) : 339,	(27,123) : 285,	(28,123) : 188,	(29,123) : 513,	(30,123) : 193,	(31,123) : 126,	(32,123) : 196,	(33,123) : 299,	(34,123) : 101,	(35,123) : 244,	(36,123) : 207,	(37,123) : 265,	(38,123) : 135,	(39,123) : 149,	(40,123) : 607,
(1,124) : 811,	(2,124) : 866,	(3,124) : 1108,	(4,124) : 427,	(5,124) : 906,	(6,124) : 861,	(7,124) : 811,	(8,124) : 1395,	(9,124) : 1083,	(10,124) : 811,	(11,124) : 866,	(12,124) : 289,	(13,124) : 674,	(14,124) : 1078,	(15,124) : 501,	(16,124) : 311,	(17,124) : 450,	(18,124) : 894,	(19,124) : 350,	(20,124) : 808,	(21,124) : 629,	(22,124) : 903,	(23,124) : 889,	(24,124) : 450,	(25,124) : 1369,	(26,124) : 618,	(27,124) : 670,	(28,124) : 809,	(29,124) : 572,	(30,124) : 808,	(31,124) : 859,	(32,124) : 764,	(33,124) : 698,	(34,124) : 864,	(35,124) : 901,	(36,124) : 1142,	(37,124) : 1119,	(38,124) : 874,	(39,124) : 868,	(40,124) : 1395,
(1,125) : 488,	(2,125) : 536,	(3,125) : 699,	(4,125) : 130,	(5,125) : 563,	(6,125) : 514,	(7,125) : 484,	(8,125) : 988,	(9,125) : 688,	(10,125) : 485,	(11,125) : 537,	(12,125) : 169,	(13,125) : 320,	(14,125) : 684,	(15,125) : 86,	(16,125) : 161,	(17,125) : 95,	(18,125) : 546,	(19,125) : 159,	(20,125) : 483,	(21,125) : 224,	(22,125) : 533,	(23,125) : 541,	(24,125) : 98,	(25,125) : 954,	(26,125) : 256,	(27,125) : 315,	(28,125) : 483,	(29,125) : 166,	(30,125) : 485,	(31,125) : 516,	(32,125) : 415,	(33,125) : 403,	(34,125) : 508,	(35,125) : 609,	(36,125) : 750,	(37,125) : 711,	(38,125) : 538,	(39,125) : 537,	(40,125) : 980,
(1,126) : 465,	(2,126) : 498,	(3,126) : 501,	(4,126) : 207,	(5,126) : 507,	(6,126) : 460,	(7,126) : 457,	(8,126) : 790,	(9,126) : 556,	(10,126) : 458,	(11,126) : 499,	(12,126) : 365,	(13,126) : 303,	(14,126) : 552,	(15,126) : 120,	(16,126) : 314,	(17,126) : 166,	(18,126) : 487,	(19,126) : 341,	(20,126) : 458,	(21,126) : 149,	(22,126) : 448,	(23,126) : 482,	(24,126) : 167,	(25,126) : 765,	(26,126) : 254,	(27,126) : 297,	(28,126) : 457,	(29,126) : 35,	(30,126) : 462,	(31,126) : 466,	(32,126) : 377,	(33,126) : 424,	(34,126) : 445,	(35,126) : 597,	(36,126) : 618,	(37,126) : 554,	(38,126) : 494,	(39,126) : 498,	(40,126) : 792,
(1,127) : 688,	(2,127) : 677,	(3,127) : 36,	(4,127) : 691,	(5,127) : 643,	(6,127) : 627,	(7,127) : 676,	(8,127) : 284,	(9,127) : 453,	(10,127) : 679,	(11,127) : 679,	(12,127) : 867,	(13,127) : 627,	(14,127) : 454,	(15,127) : 622,	(16,127) : 807,	(17,127) : 662,	(18,127) : 629,	(19,127) : 833,	(20,127) : 682,	(21,127) : 543,	(22,127) : 553,	(23,127) : 626,	(24,127) : 662,	(25,127) : 312,	(26,127) : 629,	(27,127) : 624,	(28,127) : 680,	(29,127) : 541,	(30,127) : 686,	(31,127) : 638,	(32,127) : 623,	(33,127) : 737,	(34,127) : 601,	(35,127) : 786,	(36,127) : 485,	(37,127) : 353,	(38,127) : 662,	(39,127) : 674,	(40,127) : 342,
(1,128) : 484,	(2,128) : 526,	(3,128) : 613,	(4,128) : 121,	(5,128) : 545,	(6,128) : 496,	(7,128) : 478,	(8,128) : 902,	(9,128) : 637,	(10,128) : 479,	(11,128) : 527,	(12,128) : 259,	(13,128) : 313,	(14,128) : 633,	(15,128) : 41,	(16,128) : 210,	(17,128) : 71,	(18,128) : 527,	(19,128) : 243,	(20,128) : 478,	(21,128) : 182,	(22,128) : 501,	(23,128) : 522,	(24,128) : 73,	(25,128) : 875,	(26,128) : 251,	(27,128) : 307,	(28,128) : 477,	(29,128) : 78,	(30,128) : 481,	(31,128) : 500,	(32,128) : 402,	(33,128) : 417,	(34,128) : 486,	(35,128) : 612,	(36,128) : 699,	(37,128) : 649,	(38,128) : 525,	(39,128) : 526,	(40,128) : 901,
(1,129) : 204,	(2,129) : 215,	(3,129) : 470,	(4,129) : 486,	(5,129) : 208,	(6,129) : 167,	(7,129) : 193,	(8,129) : 737,	(9,129) : 287,	(10,129) : 196,	(11,129) : 216,	(12,129) : 508,	(13,129) : 149,	(14,129) : 283,	(15,129) : 335,	(16,129) : 564,	(17,129) : 436,	(18,129) : 189,	(19,129) : 452,	(20,129) : 198,	(21,129) : 184,	(22,129) : 141,	(23,129) : 184,	(24,129) : 439,	(25,129) : 637,	(26,129) : 184,	(27,129) : 148,	(28,129) : 196,	(29,129) : 327,	(30,129) : 202,	(31,129) : 176,	(32,129) : 128,	(33,129) : 242,	(34,129) : 146,	(35,129) : 326,	(36,129) : 347,	(37,129) : 335,	(38,129) : 205,	(39,129) : 213,	(40,129) : 656,
(1,130) : 557,	(2,130) : 601,	(3,130) : 666,	(4,130) : 44,	(5,130) : 622,	(6,130) : 573,	(7,130) : 551,	(8,130) : 952,	(9,130) : 714,	(10,130) : 552,	(11,130) : 602,	(12,130) : 246,	(13,130) : 386,	(14,130) : 710,	(15,130) : 113,	(16,130) : 141,	(17,130) : 7,	(18,130) : 604,	(19,130) : 251,	(20,130) : 551,	(21,130) : 260,	(22,130) : 580,	(23,130) : 598,	(24,130) : 6,	(25,130) : 937,	(26,130) : 323,	(27,130) : 380,	(28,130) : 551,	(29,130) : 139,	(30,130) : 554,	(31,130) : 576,	(32,130) : 477,	(33,130) : 483,	(34,130) : 563,	(35,130) : 683,	(36,130) : 777,	(37,130) : 722,	(38,130) : 600,	(39,130) : 601,	(40,130) : 964,
(1,131) : 59,	(2,131) : 114,	(3,131) : 690,	(4,131) : 566,	(5,131) : 163,	(6,131) : 131,	(7,131) : 65,	(8,131) : 946,	(9,131) : 412,	(10,131) : 63,	(11,131) : 114,	(12,131) : 477,	(13,131) : 145,	(14,131) : 407,	(15,131) : 413,	(16,131) : 604,	(17,131) : 518,	(18,131) : 157,	(19,131) : 413,	(20,131) : 59,	(21,131) : 299,	(22,131) : 206,	(23,131) : 154,	(24,131) : 520,	(25,131) : 825,	(26,131) : 203,	(27,131) : 151,	(28,131) : 61,	(29,131) : 452,	(30,131) : 57,	(31,131) : 123,	(32,131) : 95,	(33,131) : 59,	(34,131) : 149,	(35,131) : 160,	(36,131) : 455,	(37,131) : 499,	(38,131) : 126,	(39,131) : 117,	(40,131) : 840,
(1,132) : 553,	(2,132) : 597,	(3,132) : 667,	(4,132) : 47,	(5,132) : 618,	(6,132) : 569,	(7,132) : 547,	(8,132) : 953,	(9,132) : 712,	(10,132) : 549,	(11,132) : 598,	(12,132) : 242,	(13,132) : 382,	(14,132) : 708,	(15,132) : 110,	(16,132) : 141,	(17,132) : 7,	(18,132) : 600,	(19,132) : 246,	(20,132) : 548,	(21,132) : 257,	(22,132) : 577,	(23,132) : 595,	(24,132) : 8,	(25,132) : 937,	(26,132) : 319,	(27,132) : 376,	(28,132) : 547,	(29,132) : 138,	(30,132) : 550,	(31,132) : 573,	(32,132) : 474,	(33,132) : 479,	(34,132) : 560,	(35,132) : 679,	(36,132) : 775,	(37,132) : 721,	(38,132) : 597,	(39,132) : 598,	(40,132) : 964,
(1,133) : 70,	(2,133) : 123,	(3,133) : 654,	(4,133) : 533,	(5,133) : 164,	(6,133) : 123,	(7,133) : 68,	(8,133) : 914,	(9,133) : 395,	(10,133) : 68,	(11,133) : 124,	(12,133) : 460,	(13,133) : 107,	(14,133) : 390,	(15,133) : 379,	(16,133) : 577,	(17,133) : 485,	(18,133) : 154,	(19,133) : 396,	(20,133) : 66,	(21,133) : 259,	(22,133) : 189,	(23,133) : 150,	(24,133) : 487,	(25,133) : 799,	(26,133) : 168,	(27,133) : 114,	(28,133) : 66,	(29,133) : 414,	(30,133) : 67,	(31,133) : 118,	(32,133) : 58,	(33,133) : 66,	(34,133) : 134,	(35,133) : 193,	(36,133) : 442,	(37,133) : 476,	(38,133) : 130,	(39,133) : 125,	(40,133) : 815,
(1,134) : 110,	(2,134) : 106,	(3,134) : 549,	(4,134) : 572,	(5,134) : 99,	(6,134) : 57,	(7,134) : 99,	(8,134) : 798,	(9,134) : 269,	(10,134) : 102,	(11,134) : 108,	(12,134) : 550,	(13,134) : 165,	(14,134) : 264,	(15,134) : 416,	(16,134) : 636,	(17,134) : 521,	(18,134) : 80,	(19,134) : 489,	(20,134) : 105,	(21,134) : 270,	(22,134) : 66,	(23,134) : 75,	(24,134) : 524,	(25,134) : 674,	(26,134) : 225,	(27,134) : 169,	(28,134) : 102,	(29,134) : 424,	(30,134) : 109,	(31,134) : 67,	(32,134) : 80,	(33,134) : 193,	(34,134) : 37,	(35,134) : 219,	(36,134) : 319,	(37,134) : 350,	(38,134) : 95,	(39,134) : 104,	(40,134) : 690,
(1,135) : 177,	(2,135) : 147,	(3,135) : 506,	(4,135) : 631,	(5,135) : 106,	(6,135) : 102,	(7,135) : 167,	(8,135) : 737,	(9,135) : 185,	(10,135) : 169,	(11,135) : 148,	(12,135) : 630,	(13,135) : 246,	(14,135) : 181,	(15,135) : 478,	(16,135) : 704,	(17,135) : 581,	(18,135) : 93,	(19,135) : 569,	(20,135) : 173,	(21,135) : 328,	(22,135) : 21,	(23,135) : 92,	(24,135) : 583,	(25,135) : 602,	(26,135) : 301,	(27,135) : 249,	(28,135) : 171,	(29,135) : 472,	(30,135) : 177,	(31,135) : 112,	(32,135) : 165,	(33,135) : 275,	(34,135) : 80,	(35,135) : 250,	(36,135) : 234,	(37,135) : 274,	(38,135) : 130,	(39,135) : 143,	(40,135) : 616,
(1,136) : 105,	(2,136) : 50,	(3,136) : 655,	(4,136) : 692,	(5,136) : 44,	(6,136) : 82,	(7,136) : 105,	(8,136) : 880,	(9,136) : 299,	(10,136) : 105,	(11,136) : 50,	(12,136) : 633,	(13,136) : 266,	(14,136) : 295,	(15,136) : 536,	(16,136) : 745,	(17,136) : 643,	(18,136) : 62,	(19,136) : 569,	(20,136) : 108,	(21,136) : 398,	(22,136) : 135,	(23,136) : 66,	(24,136) : 645,	(25,136) : 734,	(26,136) : 330,	(27,136) : 271,	(28,136) : 107,	(29,136) : 554,	(30,136) : 108,	(31,136) : 75,	(32,136) : 172,	(33,136) : 220,	(34,136) : 102,	(35,136) : 102,	(36,136) : 327,	(37,136) : 405,	(38,136) : 49,	(39,136) : 49,	(40,136) : 745,
(1,137) : 920,	(2,137) : 892,	(3,137) : 344,	(4,137) : 1035,	(5,137) : 846,	(6,137) : 847,	(7,137) : 910,	(8,137) : 86,	(9,137) : 591,	(10,137) : 912,	(11,137) : 893,	(12,137) : 1197,	(13,137) : 909,	(14,137) : 594,	(15,137) : 956,	(16,137) : 1150,	(17,137) : 1004,	(18,137) : 837,	(19,137) : 1157,	(20,137) : 916,	(21,137) : 857,	(22,137) : 766,	(23,137) : 837,	(24,137) : 1004,	(25,137) : 161,	(26,137) : 927,	(27,137) : 908,	(28,137) : 913,	(29,137) : 879,	(30,137) : 920,	(31,137) : 858,	(32,137) : 877,	(33,137) : 998,	(34,137) : 823,	(35,137) : 986,	(36,137) : 585,	(37,137) : 480,	(38,137) : 875,	(39,137) : 889,	(40,137) : 166,
(1,138) : 64,	(2,138) : 95,	(3,138) : 605,	(4,138) : 549,	(5,138) : 121,	(6,138) : 74,	(7,138) : 54,	(8,138) : 860,	(9,138) : 336,	(10,138) : 56,	(11,138) : 96,	(12,138) : 501,	(13,138) : 123,	(14,138) : 331,	(15,138) : 393,	(16,138) : 603,	(17,138) : 499,	(18,138) : 107,	(19,138) : 438,	(20,138) : 58,	(21,138) : 258,	(22,138) : 130,	(23,138) : 101,	(24,138) : 502,	(25,138) : 740,	(26,138) : 187,	(27,138) : 129,	(28,138) : 56,	(29,138) : 415,	(30,138) : 62,	(31,138) : 73,	(32,138) : 29,	(33,138) : 126,	(34,138) : 78,	(35,138) : 194,	(36,138) : 384,	(37,138) : 417,	(38,138) : 95,	(39,138) : 96,	(40,138) : 756,
(1,139) : 802,	(2,139) : 788,	(3,139) : 147,	(4,139) : 810,	(5,139) : 750,	(6,139) : 738,	(7,139) : 791,	(8,139) : 167,	(9,139) : 535,	(10,139) : 794,	(11,139) : 789,	(12,139) : 994,	(13,139) : 752,	(14,139) : 537,	(15,139) : 748,	(16,139) : 927,	(17,139) : 783,	(18,139) : 737,	(19,139) : 960,	(20,139) : 797,	(21,139) : 670,	(22,139) : 661,	(23,139) : 735,	(24,139) : 783,	(25,139) : 265,	(26,139) : 756,	(27,139) : 749,	(28,139) : 795,	(29,139) : 665,	(30,139) : 801,	(31,139) : 749,	(32,139) : 742,	(33,139) : 858,	(34,139) : 712,	(35,139) : 894,	(36,139) : 555,	(37,139) : 425,	(38,139) : 771,	(39,139) : 784,	(40,139) : 294,
(1,140) : 98,	(2,140) : 55,	(3,140) : 598,	(4,140) : 646,	(5,140) : 20,	(6,140) : 32,	(7,140) : 92,	(8,140) : 830,	(9,140) : 263,	(10,140) : 93,	(11,140) : 56,	(12,140) : 607,	(13,140) : 227,	(14,140) : 259,	(15,140) : 490,	(16,140) : 706,	(17,140) : 596,	(18,140) : 1,	(19,140) : 543,	(20,140) : 97,	(21,140) : 347,	(22,140) : 76,	(23,140) : 5,	(24,140) : 598,	(25,140) : 692,	(26,140) : 290,	(27,140) : 232,	(28,140) : 95,	(29,140) : 502,	(30,140) : 99,	(31,140) : 34,	(32,140) : 132,	(33,140) : 212,	(34,140) : 42,	(35,140) : 158,	(36,140) : 301,	(37,140) : 362,	(38,140) : 37,	(39,140) : 51,	(40,140) : 704,
(1,141) : 31,	(2,141) : 80,	(3,141) : 686,	(4,141) : 598,	(5,141) : 131,	(6,141) : 107,	(7,141) : 41,	(8,141) : 936,	(9,141) : 388,	(10,141) : 38,	(11,141) : 80,	(12,141) : 514,	(13,141) : 172,	(14,141) : 384,	(15,141) : 444,	(16,141) : 639,	(17,141) : 549,	(18,141) : 129,	(19,141) : 450,	(20,141) : 35,	(21,141) : 323,	(22,141) : 186,	(23,141) : 126,	(24,141) : 552,	(25,141) : 809,	(26,141) : 233,	(27,141) : 178,	(28,141) : 37,	(29,141) : 478,	(30,141) : 31,	(31,141) : 97,	(32,141) : 103,	(33,141) : 95,	(34,141) : 128,	(35,141) : 128,	(36,141) : 428,	(37,141) : 481,	(38,141) : 94,	(39,141) : 83,	(40,141) : 823,
(1,142) : 104,	(2,142) : 71,	(3,142) : 710,	(4,142) : 703,	(5,142) : 98,	(6,142) : 120,	(7,142) : 110,	(8,142) : 937,	(9,142) : 356,	(10,142) : 108,	(11,142) : 70,	(12,142) : 623,	(13,142) : 275,	(14,142) : 352,	(15,142) : 548,	(16,142) : 748,	(17,142) : 654,	(18,142) : 112,	(19,142) : 559,	(20,142) : 109,	(21,142) : 418,	(22,142) : 188,	(23,142) : 115,	(24,142) : 657,	(25,142) : 791,	(26,142) : 338,	(27,142) : 281,	(28,142) : 110,	(29,142) : 575,	(30,142) : 107,	(31,142) : 110,	(32,142) : 188,	(33,142) : 202,	(34,142) : 145,	(35,142) : 46,	(36,142) : 382,	(37,142) : 462,	(38,142) : 82,	(39,142) : 73,	(40,142) : 802,
(1,143) : 15,	(2,143) : 45,	(3,143) : 647,	(4,143) : 603,	(5,143) : 90,	(6,143) : 61,	(7,143) : 10,	(8,143) : 893,	(9,143) : 342,	(10,143) : 11,	(11,143) : 45,	(12,143) : 538,	(13,143) : 174,	(14,143) : 338,	(15,143) : 447,	(16,143) : 652,	(17,143) : 554,	(18,143) : 84,	(19,143) : 474,	(20,143) : 14,	(21,143) : 316,	(22,143) : 140,	(23,143) : 81,	(24,143) : 556,	(25,143) : 763,	(26,143) : 238,	(27,143) : 180,	(28,143) : 13,	(29,143) : 473,	(30,143) : 16,	(31,143) : 51,	(32,143) : 86,	(33,143) : 131,	(34,143) : 82,	(35,143) : 135,	(36,143) : 383,	(37,143) : 435,	(38,143) : 54,	(39,143) : 47,	(40,143) : 777,
(1,144) : 445,	(2,144) : 484,	(3,144) : 550,	(4,144) : 181,	(5,144) : 498,	(6,144) : 450,	(7,144) : 438,	(8,144) : 840,	(9,144) : 574,	(10,144) : 440,	(11,144) : 485,	(12,144) : 312,	(13,144) : 278,	(14,144) : 570,	(15,144) : 65,	(16,144) : 277,	(17,144) : 134,	(18,144) : 479,	(19,144) : 286,	(20,144) : 440,	(21,144) : 131,	(22,144) : 447,	(23,144) : 474,	(24,144) : 135,	(25,144) : 809,	(26,144) : 222,	(27,144) : 271,	(28,144) : 438,	(29,144) : 33,	(30,144) : 442,	(31,144) : 455,	(32,144) : 360,	(33,144) : 393,	(34,144) : 437,	(35,144) : 576,	(36,144) : 636,	(37,144) : 582,	(38,144) : 481,	(39,144) : 484,	(40,144) : 835,
(1,145) : 19,	(2,145) : 56,	(3,145) : 641,	(4,145) : 591,	(5,145) : 97,	(6,145) : 62,	(7,145) : 7,	(8,145) : 889,	(9,145) : 344,	(10,145) : 10,	(11,145) : 56,	(12,145) : 528,	(13,145) : 162,	(14,145) : 339,	(15,145) : 435,	(16,145) : 640,	(17,145) : 541,	(18,145) : 89,	(19,145) : 464,	(20,145) : 13,	(21,145) : 304,	(22,145) : 140,	(23,145) : 86,	(24,145) : 544,	(25,145) : 762,	(26,145) : 226,	(27,145) : 168,	(28,145) : 11,	(29,145) : 461,	(30,145) : 17,	(31,145) : 55,	(32,145) : 74,	(33,145) : 124,	(34,145) : 81,	(35,145) : 147,	(36,145) : 386,	(37,145) : 434,	(38,145) : 62,	(39,145) : 57,	(40,145) : 776,
(1,146) : 311,	(2,146) : 348,	(3,146) : 533,	(4,146) : 306,	(5,146) : 363,	(6,146) : 314,	(7,146) : 304,	(8,146) : 820,	(9,146) : 464,	(10,146) : 305,	(11,146) : 349,	(12,146) : 347,	(13,146) : 148,	(14,146) : 460,	(15,146) : 152,	(16,146) : 381,	(17,146) : 256,	(18,146) : 344,	(19,146) : 299,	(20,146) : 305,	(21,146) : 7,	(22,146) : 317,	(23,146) : 339,	(24,146) : 258,	(25,146) : 757,	(26,146) : 104,	(27,146) : 141,	(28,146) : 304,	(29,146) : 162,	(30,146) : 308,	(31,146) : 319,	(32,146) : 225,	(33,146) : 271,	(34,146) : 302,	(35,146) : 443,	(36,146) : 525,	(37,146) : 492,	(38,146) : 345,	(39,146) : 348,	(40,146) : 781,
(1,147) : 11,	(2,147) : 56,	(3,147) : 649,	(4,147) : 594,	(5,147) : 101,	(6,147) : 69,	(7,147) : 1,	(8,147) : 897,	(9,147) : 350,	(10,147) : 2,	(11,147) : 57,	(12,147) : 527,	(13,147) : 165,	(14,147) : 346,	(15,147) : 438,	(16,147) : 642,	(17,147) : 544,	(18,147) : 94,	(19,147) : 463,	(20,147) : 5,	(21,147) : 309,	(22,147) : 147,	(23,147) : 91,	(24,147) : 547,	(25,147) : 769,	(26,147) : 228,	(27,147) : 171,	(28,147) : 3,	(29,147) : 465,	(30,147) : 9,	(31,147) : 60,	(32,147) : 79,	(33,147) : 120,	(34,147) : 89,	(35,147) : 141,	(36,147) : 392,	(37,147) : 442,	(38,147) : 65,	(39,147) : 58,	(40,147) : 783,
(1,148) : 95,	(2,148) : 92,	(3,148) : 565,	(4,148) : 577,	(5,148) : 90,	(6,148) : 45,	(7,148) : 84,	(8,148) : 812,	(9,148) : 279,	(10,148) : 86,	(11,148) : 94,	(12,148) : 548,	(13,148) : 164,	(14,148) : 274,	(15,148) : 420,	(16,148) : 639,	(17,148) : 526,	(18,148) : 71,	(19,148) : 486,	(20,148) : 90,	(21,148) : 277,	(22,148) : 74,	(23,148) : 66,	(24,148) : 529,	(25,148) : 688,	(26,148) : 225,	(27,148) : 168,	(28,148) : 87,	(29,148) : 431,	(30,148) : 94,	(31,148) : 53,	(32,148) : 73,	(33,148) : 182,	(34,148) : 30,	(35,148) : 205,	(36,148) : 327,	(37,148) : 363,	(38,148) : 82,	(39,148) : 90,	(40,148) : 703,
(1,149) : 349,	(2,149) : 400,	(3,149) : 675,	(4,149) : 265,	(5,149) : 430,	(6,149) : 382,	(7,149) : 346,	(8,149) : 964,	(9,149) : 586,	(10,149) : 347,	(11,149) : 400,	(12,149) : 209,	(13,149) : 187,	(14,149) : 582,	(15,149) : 131,	(16,149) : 297,	(17,149) : 221,	(18,149) : 415,	(19,149) : 156,	(20,149) : 345,	(21,149) : 149,	(22,149) : 411,	(23,149) : 409,	(24,149) : 224,	(25,149) : 902,	(26,149) : 125,	(27,149) : 182,	(28,149) : 345,	(29,149) : 209,	(30,149) : 346,	(31,149) : 383,	(32,149) : 282,	(33,149) : 262,	(34,149) : 379,	(35,149) : 468,	(36,149) : 645,	(37,149) : 627,	(38,149) : 403,	(39,149) : 401,	(40,149) : 925,
(1,150) : 581,	(2,150) : 635,	(3,150) : 904,	(4,150) : 277,	(5,150) : 674,	(6,150) : 628,	(7,150) : 581,	(8,150) : 1194,	(9,150) : 849,	(10,150) : 580,	(11,150) : 636,	(12,150) : 54,	(13,150) : 439,	(14,150) : 845,	(15,150) : 290,	(16,150) : 194,	(17,150) : 276,	(18,150) : 661,	(19,150) : 116,	(20,150) : 578,	(21,150) : 402,	(22,150) : 669,	(23,150) : 656,	(24,150) : 277,	(25,150) : 1152,	(26,150) : 383,	(27,150) : 436,	(28,150) : 578,	(29,150) : 374,	(30,150) : 578,	(31,150) : 627,	(32,150) : 530,	(33,150) : 471,	(34,150) : 630,	(35,150) : 679,	(36,150) : 908,	(37,150) : 889,	(38,150) : 642,	(39,150) : 637,	(40,150) : 1176,
(1,151) : 211,	(2,151) : 254,	(3,151) : 589,	(4,151) : 390,	(5,151) : 278,	(6,151) : 229,	(7,151) : 205,	(8,151) : 868,	(9,151) : 435,	(10,151) : 206,	(11,151) : 255,	(12,151) : 362,	(13,151) : 41,	(14,151) : 430,	(15,151) : 234,	(16,151) : 445,	(17,151) : 340,	(18,151) : 262,	(19,151) : 303,	(20,151) : 205,	(21,151) : 114,	(22,151) : 254,	(23,151) : 256,	(24,151) : 343,	(25,151) : 782,	(26,151) : 33,	(27,151) : 35,	(28,151) : 204,	(29,151) : 266,	(30,151) : 208,	(31,151) : 231,	(32,151) : 131,	(33,151) : 164,	(34,151) : 224,	(35,151) : 340,	(36,151) : 492,	(37,151) : 487,	(38,151) : 255,	(39,151) : 255,	(40,151) : 803,
(1,152) : 227,	(2,152) : 275,	(3,152) : 621,	(4,152) : 374,	(5,152) : 304,	(6,152) : 255,	(7,152) : 223,	(8,152) : 903,	(9,152) : 471,	(10,152) : 223,	(11,152) : 276,	(12,152) : 328,	(13,152) : 60,	(14,152) : 466,	(15,152) : 220,	(16,152) : 421,	(17,152) : 325,	(18,152) : 288,	(19,152) : 268,	(20,152) : 222,	(21,152) : 124,	(22,152) : 287,	(23,152) : 283,	(24,152) : 328,	(25,152) : 819,	(26,152) : 13,	(27,152) : 55,	(28,152) : 222,	(29,152) : 264,	(30,152) : 224,	(31,152) : 256,	(32,152) : 155,	(33,152) : 160,	(34,152) : 252,	(35,152) : 352,	(36,152) : 528,	(37,152) : 524,	(38,152) : 277,	(39,152) : 276,	(40,152) : 840,
(1,153) : 666,	(2,153) : 719,	(3,153) : 894,	(4,153) : 213,	(5,153) : 752,	(6,153) : 704,	(7,153) : 664,	(8,153) : 1180,	(9,153) : 898,	(10,153) : 664,	(11,153) : 719,	(12,153) : 156,	(13,153) : 509,	(14,153) : 893,	(15,153) : 295,	(16,153) : 99,	(17,153) : 234,	(18,153) : 737,	(19,153) : 214,	(20,153) : 662,	(21,153) : 435,	(22,153) : 733,	(23,153) : 732,	(24,153) : 234,	(25,153) : 1159,	(26,153) : 447,	(27,153) : 505,	(28,153) : 662,	(29,153) : 359,	(30,153) : 663,	(31,153) : 704,	(32,153) : 604,	(33,153) : 564,	(34,153) : 702,	(35,153) : 775,	(36,153) : 959,	(37,153) : 923,	(38,153) : 723,	(39,153) : 720,	(40,153) : 1185,
(1,154) : 114,	(2,154) : 168,	(3,154) : 667,	(4,154) : 497,	(5,154) : 209,	(6,154) : 166,	(7,154) : 113,	(8,154) : 934,	(9,154) : 431,	(10,154) : 113,	(11,154) : 168,	(12,154) : 415,	(13,154) : 83,	(14,154) : 426,	(15,154) : 345,	(16,154) : 536,	(17,154) : 449,	(18,154) : 198,	(19,154) : 351,	(20,154) : 110,	(21,154) : 236,	(22,154) : 227,	(23,154) : 193,	(24,154) : 451,	(25,154) : 825,	(26,154) : 135,	(27,154) : 89,	(28,154) : 111,	(29,154) : 386,	(30,154) : 111,	(31,154) : 163,	(32,154) : 84,	(33,154) : 47,	(34,154) : 175,	(35,154) : 229,	(36,154) : 481,	(37,154) : 508,	(38,154) : 175,	(39,154) : 170,	(40,154) : 843,
(1,155) : 120,	(2,155) : 105,	(3,155) : 543,	(4,155) : 592,	(5,155) : 87,	(6,155) : 54,	(7,155) : 110,	(8,155) : 786,	(9,155) : 248,	(10,155) : 112,	(11,155) : 107,	(12,155) : 574,	(13,155) : 189,	(14,155) : 244,	(15,155) : 436,	(16,155) : 658,	(17,155) : 542,	(18,155) : 67,	(19,155) : 512,	(20,155) : 116,	(21,155) : 290,	(22,155) : 43,	(23,155) : 63,	(24,155) : 544,	(25,155) : 659,	(26,155) : 248,	(27,155) : 192,	(28,155) : 114,	(29,155) : 442,	(30,155) : 120,	(31,155) : 65,	(32,155) : 102,	(33,155) : 212,	(34,155) : 28,	(35,155) : 217,	(36,155) : 297,	(37,155) : 333,	(38,155) : 91,	(39,155) : 103,	(40,155) : 674,
(1,156) : 525,	(2,156) : 567,	(3,156) : 612,	(4,156) : 91,	(5,156) : 584,	(6,156) : 536,	(7,156) : 519,	(8,156) : 899,	(9,156) : 664,	(10,156) : 521,	(11,156) : 568,	(12,156) : 281,	(13,156) : 355,	(14,156) : 660,	(15,156) : 86,	(16,156) : 197,	(17,156) : 48,	(18,156) : 566,	(19,156) : 274,	(20,156) : 520,	(21,156) : 218,	(22,156) : 537,	(23,156) : 560,	(24,156) : 49,	(25,156) : 882,	(26,156) : 294,	(27,156) : 349,	(28,156) : 519,	(29,156) : 84,	(30,156) : 522,	(31,156) : 540,	(32,156) : 443,	(33,156) : 461,	(34,156) : 524,	(35,156) : 654,	(36,156) : 727,	(37,156) : 669,	(38,156) : 565,	(39,156) : 567,	(40,156) : 909,
(1,157) : 309,	(2,157) : 346,	(3,157) : 531,	(4,157) : 309,	(5,157) : 360,	(6,157) : 311,	(7,157) : 301,	(8,157) : 818,	(9,157) : 461,	(10,157) : 303,	(11,157) : 347,	(12,157) : 350,	(13,157) : 146,	(14,157) : 456,	(15,157) : 155,	(16,157) : 385,	(17,157) : 259,	(18,157) : 341,	(19,157) : 302,	(20,157) : 303,	(21,157) : 8,	(22,157) : 314,	(23,157) : 336,	(24,157) : 261,	(25,157) : 755,	(26,157) : 104,	(27,157) : 140,	(28,157) : 301,	(29,157) : 164,	(30,157) : 306,	(31,157) : 316,	(32,157) : 222,	(33,157) : 270,	(34,157) : 299,	(35,157) : 440,	(36,157) : 522,	(37,157) : 488,	(38,157) : 343,	(39,157) : 345,	(40,157) : 778,
(1,158) : 172,	(2,158) : 164,	(3,158) : 487,	(4,158) : 562,	(5,158) : 143,	(6,158) : 113,	(7,158) : 161,	(8,158) : 737,	(9,158) : 231,	(10,158) : 164,	(11,158) : 166,	(12,158) : 568,	(13,158) : 189,	(14,158) : 226,	(15,158) : 409,	(16,158) : 636,	(17,158) : 512,	(18,158) : 125,	(19,158) : 509,	(20,158) : 167,	(21,158) : 259,	(22,158) : 65,	(23,158) : 121,	(24,158) : 514,	(25,158) : 618,	(26,158) : 239,	(27,158) : 191,	(28,158) : 164,	(29,158) : 403,	(30,158) : 171,	(31,158) : 124,	(32,158) : 126,	(33,158) : 246,	(34,158) : 88,	(35,158) : 277,	(36,158) : 287,	(37,158) : 299,	(38,158) : 150,	(39,158) : 162,	(40,158) : 635,
(1,159) : 107,	(2,159) : 54,	(3,159) : 663,	(4,159) : 698,	(5,159) : 53,	(6,159) : 89,	(7,159) : 108,	(8,159) : 888,	(9,159) : 305,	(10,159) : 108,	(11,159) : 53,	(12,159) : 636,	(13,159) : 270,	(14,159) : 301,	(15,159) : 541,	(16,159) : 750,	(17,159) : 648,	(18,159) : 70,	(19,159) : 572,	(20,159) : 110,	(21,159) : 404,	(22,159) : 143,	(23,159) : 74,	(24,159) : 650,	(25,159) : 741,	(26,159) : 335,	(27,159) : 276,	(28,159) : 110,	(29,159) : 561,	(30,159) : 110,	(31,159) : 82,	(32,159) : 177,	(33,159) : 221,	(34,159) : 110,	(35,159) : 95,	(36,159) : 333,	(37,159) : 412,	(38,159) : 55,	(39,159) : 53,	(40,159) : 751,
(1,160) : 84,	(2,160) : 46,	(3,160) : 602,	(4,160) : 635,	(5,160) : 31,	(6,160) : 19,	(7,160) : 77,	(8,160) : 838,	(9,160) : 275,	(10,160) : 79,	(11,160) : 47,	(12,160) : 592,	(13,160) : 213,	(14,160) : 271,	(15,160) : 478,	(16,160) : 693,	(17,160) : 584,	(18,160) : 16,	(19,160) : 529,	(20,160) : 83,	(21,160) : 337,	(22,160) : 81,	(23,160) : 12,	(24,160) : 587,	(25,160) : 701,	(26,160) : 277,	(27,160) : 219,	(28,160) : 81,	(29,160) : 492,	(30,160) : 85,	(31,160) : 19,	(32,160) : 118,	(33,160) : 197,	(34,160) : 36,	(35,160) : 155,	(36,160) : 315,	(37,160) : 372,	(38,160) : 29,	(39,160) : 43,	(40,160) : 714,
(1,161) : 482,	(2,161) : 529,	(3,161) : 673,	(4,161) : 124,	(5,161) : 553,	(6,161) : 504,	(7,161) : 477,	(8,161) : 963,	(9,161) : 671,	(10,161) : 478,	(11,161) : 530,	(12,161) : 194,	(13,161) : 312,	(14,161) : 666,	(15,161) : 62,	(16,161) : 174,	(17,161) : 82,	(18,161) : 536,	(19,161) : 181,	(20,161) : 477,	(21,161) : 207,	(22,161) : 520,	(23,161) : 531,	(24,161) : 85,	(25,161) : 930,	(26,161) : 248,	(27,161) : 307,	(28,161) : 476,	(29,161) : 141,	(30,161) : 479,	(31,161) : 507,	(32,161) : 406,	(33,161) : 402,	(34,161) : 497,	(35,161) : 605,	(36,161) : 732,	(37,161) : 691,	(38,161) : 530,	(39,161) : 529,	(40,161) : 956,
(1,162) : 349,	(2,162) : 400,	(3,162) : 678,	(4,162) : 267,	(5,162) : 431,	(6,162) : 382,	(7,162) : 346,	(8,162) : 967,	(9,162) : 588,	(10,162) : 346,	(11,162) : 400,	(12,162) : 207,	(13,162) : 187,	(14,162) : 583,	(15,162) : 134,	(16,162) : 297,	(17,162) : 223,	(18,162) : 415,	(19,162) : 153,	(20,162) : 345,	(21,162) : 152,	(22,162) : 412,	(23,162) : 410,	(24,162) : 226,	(25,162) : 905,	(26,162) : 126,	(27,162) : 183,	(28,162) : 344,	(29,162) : 212,	(30,162) : 346,	(31,162) : 383,	(32,162) : 282,	(33,162) : 261,	(34,162) : 380,	(35,162) : 467,	(36,162) : 647,	(37,162) : 629,	(38,162) : 403,	(39,162) : 401,	(40,162) : 928,
(1,163) : 540,	(2,163) : 578,	(3,163) : 571,	(4,163) : 121,	(5,163) : 591,	(6,163) : 543,	(7,163) : 533,	(8,163) : 855,	(9,163) : 650,	(10,163) : 535,	(11,163) : 579,	(12,163) : 332,	(13,163) : 372,	(14,163) : 647,	(15,163) : 120,	(16,163) : 236,	(17,163) : 92,	(18,163) : 572,	(19,163) : 323,	(20,163) : 535,	(21,163) : 225,	(22,163) : 537,	(23,163) : 567,	(24,163) : 92,	(25,163) : 845,	(26,163) : 315,	(27,163) : 366,	(28,163) : 533,	(29,163) : 70,	(30,163) : 537,	(31,163) : 549,	(32,163) : 455,	(33,163) : 486,	(34,163) : 530,	(35,163) : 671,	(36,163) : 712,	(37,163) : 647,	(38,163) : 575,	(39,163) : 578,	(40,163) : 873,
(1,164) : 496,	(2,164) : 548,	(3,164) : 773,	(4,164) : 186,	(5,164) : 581,	(6,164) : 533,	(7,164) : 494,	(8,164) : 1063,	(9,164) : 733,	(10,164) : 494,	(11,164) : 549,	(12,164) : 88,	(13,164) : 338,	(14,164) : 728,	(15,164) : 159,	(16,164) : 162,	(17,164) : 166,	(18,164) : 566,	(19,164) : 87,	(20,164) : 492,	(21,164) : 276,	(22,164) : 563,	(23,164) : 561,	(24,164) : 168,	(25,164) : 1021,	(26,164) : 277,	(27,164) : 334,	(28,164) : 492,	(29,164) : 245,	(30,164) : 493,	(31,164) : 533,	(32,164) : 433,	(33,164) : 399,	(34,164) : 531,	(35,164) : 609,	(36,164) : 793,	(37,164) : 766,	(38,164) : 553,	(39,164) : 550,	(40,164) : 1046,
(1,165) : 252,	(2,165) : 230,	(3,165) : 419,	(4,165) : 612,	(5,165) : 193,	(6,165) : 182,	(7,165) : 242,	(8,165) : 655,	(9,165) : 149,	(10,165) : 244,	(11,165) : 231,	(12,165) : 646,	(13,165) : 276,	(14,165) : 145,	(15,165) : 466,	(16,165) : 697,	(17,165) : 564,	(18,165) : 179,	(19,165) : 589,	(20,165) : 248,	(21,165) : 317,	(22,165) : 103,	(23,165) : 177,	(24,165) : 565,	(25,165) : 529,	(26,165) : 320,	(27,165) : 277,	(28,165) : 245,	(29,165) : 444,	(30,165) : 252,	(31,165) : 193,	(32,165) : 216,	(33,165) : 336,	(34,165) : 157,	(35,165) : 336,	(36,165) : 210,	(37,165) : 208,	(38,165) : 214,	(39,165) : 227,	(40,165) : 545,
(1,166) : 565,	(2,166) : 610,	(3,166) : 679,	(4,166) : 35,	(5,166) : 631,	(6,166) : 582,	(7,166) : 560,	(8,166) : 964,	(9,166) : 726,	(10,166) : 561,	(11,166) : 611,	(12,166) : 240,	(13,166) : 394,	(14,166) : 722,	(15,166) : 123,	(16,166) : 129,	(17,166) : 20,	(18,166) : 613,	(19,166) : 247,	(20,166) : 560,	(21,166) : 271,	(22,166) : 590,	(23,166) : 608,	(24,166) : 19,	(25,166) : 950,	(26,166) : 331,	(27,166) : 388,	(28,166) : 559,	(29,166) : 151,	(30,166) : 562,	(31,166) : 585,	(32,166) : 486,	(33,166) : 490,	(34,166) : 573,	(35,166) : 691,	(36,166) : 789,	(37,166) : 735,	(38,166) : 610,	(39,166) : 610,	(40,166) : 977,
(1,167) : 484,	(2,167) : 534,	(3,167) : 731,	(4,167) : 158,	(5,167) : 564,	(6,167) : 515,	(7,167) : 481,	(8,167) : 1020,	(9,167) : 703,	(10,167) : 481,	(11,167) : 535,	(12,167) : 132,	(13,167) : 320,	(14,167) : 699,	(15,167) : 116,	(16,167) : 163,	(17,167) : 130,	(18,167) : 548,	(19,167) : 121,	(20,167) : 479,	(21,167) : 241,	(22,167) : 539,	(23,167) : 542,	(24,167) : 132,	(25,167) : 982,	(26,167) : 257,	(27,167) : 315,	(28,167) : 479,	(29,167) : 202,	(30,167) : 481,	(31,167) : 516,	(32,167) : 415,	(33,167) : 393,	(34,167) : 511,	(35,167) : 601,	(36,167) : 764,	(37,167) : 731,	(38,167) : 537,	(39,167) : 535,	(40,167) : 1007,
(1,168) : 440,	(2,168) : 494,	(3,168) : 785,	(4,168) : 256,	(5,168) : 530,	(6,168) : 484,	(7,168) : 439,	(8,168) : 1075,	(9,168) : 703,	(10,168) : 439,	(11,168) : 494,	(12,168) : 92,	(13,168) : 293,	(14,168) : 699,	(15,168) : 188,	(16,168) : 237,	(17,168) : 228,	(18,168) : 517,	(19,168) : 37,	(20,168) : 437,	(21,168) : 265,	(22,168) : 522,	(23,168) : 511,	(24,168) : 231,	(25,168) : 1019,	(26,168) : 237,	(27,168) : 289,	(28,168) : 437,	(29,168) : 279,	(30,168) : 437,	(31,168) : 483,	(32,168) : 385,	(33,168) : 335,	(34,168) : 484,	(35,168) : 545,	(36,168) : 762,	(37,168) : 747,	(38,168) : 500,	(39,168) : 495,	(40,168) : 1043,
(1,169) : 114,	(2,169) : 120,	(3,169) : 547,	(4,169) : 549,	(5,169) : 120,	(6,169) : 75,	(7,169) : 102,	(8,169) : 801,	(9,169) : 285,	(10,169) : 105,	(11,169) : 122,	(12,169) : 528,	(13,169) : 143,	(14,169) : 280,	(15,169) : 392,	(16,169) : 613,	(17,169) : 498,	(18,169) : 101,	(19,169) : 467,	(20,169) : 107,	(21,169) : 248,	(22,169) : 86,	(23,169) : 96,	(24,169) : 501,	(25,169) : 682,	(26,169) : 202,	(27,169) : 147,	(28,169) : 105,	(29,169) : 402,	(30,169) : 112,	(31,169) : 83,	(32,169) : 63,	(33,169) : 182,	(34,169) : 59,	(35,169) : 232,	(36,169) : 337,	(37,169) : 361,	(38,169) : 111,	(39,169) : 119,	(40,169) : 698,
(1,170) : 283,	(2,170) : 254,	(3,170) : 408,	(4,170) : 651,	(5,170) : 211,	(6,170) : 209,	(7,170) : 273,	(8,170) : 630,	(9,170) : 106,	(10,170) : 275,	(11,170) : 255,	(12,170) : 690,	(13,170) : 318,	(14,170) : 102,	(15,170) : 507,	(16,170) : 738,	(17,170) : 603,	(18,170) : 200,	(19,170) : 633,	(20,170) : 279,	(21,170) : 358,	(22,170) : 128,	(23,170) : 199,	(24,170) : 605,	(25,170) : 495,	(26,170) : 363,	(27,170) : 320,	(28,170) : 276,	(29,170) : 482,	(30,170) : 282,	(31,170) : 220,	(32,170) : 255,	(33,170) : 373,	(34,170) : 185,	(35,170) : 354,	(36,170) : 167,	(37,170) : 169,	(38,170) : 237,	(39,170) : 251,	(40,170) : 510,
(1,171) : 548,	(2,171) : 592,	(3,171) : 656,	(4,171) : 54,	(5,171) : 612,	(6,171) : 563,	(7,171) : 542,	(8,171) : 942,	(9,171) : 703,	(10,171) : 543,	(11,171) : 592,	(12,171) : 250,	(13,171) : 377,	(14,171) : 699,	(15,171) : 104,	(16,171) : 153,	(17,171) : 4,	(18,171) : 594,	(19,171) : 252,	(20,171) : 542,	(21,171) : 249,	(22,171) : 569,	(23,171) : 588,	(24,171) : 6,	(25,171) : 926,	(26,171) : 314,	(27,171) : 371,	(28,171) : 542,	(29,171) : 127,	(30,171) : 545,	(31,171) : 566,	(32,171) : 468,	(33,171) : 476,	(34,171) : 553,	(35,171) : 674,	(36,171) : 765,	(37,171) : 711,	(38,171) : 591,	(39,171) : 592,	(40,171) : 953,
(1,172) : 47,	(2,172) : 72,	(3,172) : 704,	(4,172) : 631,	(5,172) : 124,	(6,172) : 112,	(7,172) : 58,	(8,172) : 949,	(9,172) : 389,	(10,172) : 56,	(11,172) : 71,	(12,172) : 542,	(13,172) : 206,	(14,172) : 384,	(15,172) : 477,	(16,172) : 671,	(17,172) : 583,	(18,172) : 127,	(19,172) : 478,	(20,172) : 53,	(21,172) : 356,	(22,172) : 193,	(23,172) : 125,	(24,172) : 585,	(25,172) : 816,	(26,172) : 266,	(27,172) : 212,	(28,172) : 56,	(29,172) : 512,	(30,172) : 49,	(31,172) : 101,	(32,172) : 133,	(33,172) : 122,	(34,172) : 137,	(35,172) : 94,	(36,172) : 425,	(37,172) : 486,	(38,172) : 89,	(39,172) : 75,	(40,172) : 829,
(1,173) : 534,	(2,173) : 586,	(3,173) : 796,	(4,173) : 176,	(5,173) : 619,	(6,173) : 571,	(7,173) : 532,	(8,173) : 1086,	(9,173) : 769,	(10,173) : 532,	(11,173) : 587,	(12,173) : 83,	(13,173) : 376,	(14,173) : 764,	(15,173) : 182,	(16,173) : 131,	(17,173) : 166,	(18,173) : 604,	(19,173) : 107,	(20,173) : 530,	(21,173) : 309,	(22,173) : 600,	(23,173) : 599,	(24,173) : 168,	(25,173) : 1049,	(26,173) : 315,	(27,173) : 372,	(28,173) : 530,	(29,173) : 263,	(30,173) : 531,	(31,173) : 571,	(32,173) : 471,	(33,173) : 436,	(34,173) : 569,	(35,173) : 646,	(36,173) : 829,	(37,173) : 799,	(38,173) : 591,	(39,173) : 588,	(40,173) : 1075,
(1,174) : 107,	(2,174) : 96,	(3,174) : 555,	(4,174) : 587,	(5,174) : 84,	(6,174) : 45,	(7,174) : 96,	(8,174) : 800,	(9,174) : 262,	(10,174) : 99,	(11,174) : 97,	(12,174) : 564,	(13,174) : 179,	(14,174) : 258,	(15,174) : 431,	(16,174) : 652,	(17,174) : 537,	(18,174) : 65,	(19,174) : 502,	(20,174) : 102,	(21,174) : 286,	(22,174) : 57,	(23,174) : 60,	(24,174) : 540,	(25,174) : 673,	(26,174) : 239,	(27,174) : 183,	(28,174) : 100,	(29,174) : 439,	(30,174) : 106,	(31,174) : 56,	(32,174) : 90,	(33,174) : 198,	(34,174) : 22,	(35,174) : 208,	(36,174) : 310,	(37,174) : 347,	(38,174) : 83,	(39,174) : 94,	(40,174) : 688,
(1,175) : 87,	(2,175) : 38,	(3,175) : 615,	(4,175) : 651,	(5,175) : 18,	(6,175) : 35,	(7,175) : 82,	(8,175) : 848,	(9,175) : 279,	(10,175) : 83,	(11,175) : 39,	(12,175) : 604,	(13,175) : 228,	(14,175) : 275,	(15,175) : 494,	(16,175) : 708,	(17,175) : 601,	(18,175) : 18,	(19,175) : 541,	(20,175) : 86,	(21,175) : 354,	(22,175) : 94,	(23,175) : 20,	(24,175) : 603,	(25,175) : 709,	(26,175) : 292,	(27,175) : 233,	(28,175) : 85,	(29,175) : 509,	(30,175) : 88,	(31,175) : 32,	(32,175) : 133,	(33,175) : 203,	(34,175) : 54,	(35,175) : 140,	(36,175) : 315,	(37,175) : 379,	(38,175) : 21,	(39,175) : 35,	(40,175) : 721,
(1,176) : 57,	(2,176) : 4,	(3,176) : 643,	(4,176) : 642,	(5,176) : 50,	(6,176) : 47,	(7,176) : 55,	(8,176) : 879,	(9,176) : 314,	(10,176) : 55,	(11,176) : 6,	(12,176) : 583,	(13,176) : 215,	(14,176) : 309,	(15,176) : 486,	(16,176) : 694,	(17,176) : 593,	(18,176) : 52,	(19,176) : 519,	(20,176) : 58,	(21,176) : 351,	(22,176) : 123,	(23,176) : 51,	(24,176) : 595,	(25,176) : 742,	(26,176) : 279,	(27,176) : 220,	(28,176) : 57,	(29,176) : 507,	(30,176) : 59,	(31,176) : 36,	(32,176) : 122,	(33,176) : 174,	(34,176) : 73,	(35,176) : 117,	(36,176) : 350,	(37,176) : 413,	(38,176) : 14,	(39,176) : 2,	(40,176) : 755,
(1,177) : 44,	(2,177) : 99,	(3,177) : 658,	(4,177) : 558,	(5,177) : 142,	(6,177) : 105,	(7,177) : 44,	(8,177) : 914,	(9,177) : 382,	(10,177) : 44,	(11,177) : 99,	(12,177) : 484,	(13,177) : 131,	(14,177) : 378,	(15,177) : 403,	(16,177) : 602,	(17,177) : 509,	(18,177) : 134,	(19,177) : 421,	(20,177) : 41,	(21,177) : 281,	(22,177) : 176,	(23,177) : 130,	(24,177) : 512,	(25,177) : 793,	(26,177) : 192,	(27,177) : 137,	(28,177) : 41,	(29,177) : 436,	(30,177) : 42,	(31,177) : 99,	(32,177) : 65,	(33,177) : 80,	(34,177) : 119,	(35,177) : 169,	(36,177) : 427,	(37,177) : 468,	(38,177) : 107,	(39,177) : 101,	(40,177) : 808,
(1,178) : 106,	(2,178) : 158,	(3,178) : 733,	(4,178) : 562,	(5,178) : 209,	(6,178) : 180,	(7,178) : 113,	(8,178) : 993,	(9,178) : 461,	(10,178) : 111,	(11,178) : 158,	(12,178) : 451,	(13,178) : 161,	(14,178) : 457,	(15,178) : 414,	(16,178) : 591,	(17,178) : 515,	(18,178) : 205,	(19,178) : 387,	(20,178) : 108,	(21,178) : 314,	(22,178) : 256,	(23,178) : 202,	(24,178) : 518,	(25,178) : 874,	(26,178) : 209,	(27,178) : 167,	(28,178) : 109,	(29,178) : 462,	(30,178) : 105,	(31,178) : 172,	(32,178) : 137,	(33,178) : 38,	(34,178) : 198,	(35,178) : 178,	(36,178) : 504,	(37,178) : 549,	(38,178) : 172,	(39,178) : 161,	(40,178) : 889,
(1,179) : 116,	(2,179) : 78,	(3,179) : 574,	(4,179) : 640,	(5,179) : 40,	(6,179) : 41,	(7,179) : 108,	(8,179) : 806,	(9,179) : 242,	(10,179) : 110,	(11,179) : 79,	(12,179) : 610,	(13,179) : 227,	(14,179) : 238,	(15,179) : 484,	(16,179) : 703,	(17,179) : 590,	(18,179) : 24,	(19,179) : 548,	(20,179) : 113,	(21,179) : 339,	(22,179) : 52,	(23,179) : 23,	(24,179) : 592,	(25,179) : 668,	(26,179) : 289,	(27,179) : 232,	(28,179) : 112,	(29,179) : 491,	(30,179) : 116,	(31,179) : 49,	(32,179) : 134,	(33,179) : 226,	(34,179) : 34,	(35,179) : 182,	(36,179) : 283,	(37,179) : 339,	(38,179) : 61,	(39,179) : 75,	(40,179) : 681,
(1,180) : 633,	(2,180) : 688,	(3,180) : 955,	(4,180) : 310,	(5,180) : 728,	(6,180) : 682,	(7,180) : 633,	(8,180) : 1245,	(9,180) : 905,	(10,180) : 633,	(11,180) : 688,	(12,180) : 110,	(13,180) : 494,	(14,180) : 901,	(15,180) : 341,	(16,180) : 212,	(17,180) : 316,	(18,180) : 715,	(19,180) : 171,	(20,180) : 630,	(21,180) : 457,	(22,180) : 724,	(23,180) : 710,	(24,180) : 317,	(25,180) : 1206,	(26,180) : 439,	(27,180) : 491,	(28,180) : 631,	(29,180) : 422,	(30,180) : 630,	(31,180) : 680,	(32,180) : 585,	(33,180) : 522,	(34,180) : 685,	(35,180) : 728,	(36,180) : 964,	(37,180) : 945,	(38,180) : 695,	(39,180) : 690,	(40,180) : 1231,
(1,181) : 289,	(2,181) : 264,	(3,181) : 391,	(4,181) : 633,	(5,181) : 223,	(6,181) : 217,	(7,181) : 278,	(8,181) : 620,	(9,181) : 122,	(10,181) : 281,	(11,181) : 265,	(12,181) : 678,	(13,181) : 312,	(14,181) : 118,	(15,181) : 490,	(16,181) : 721,	(17,181) : 585,	(18,181) : 211,	(19,181) : 621,	(20,181) : 284,	(21,181) : 342,	(22,181) : 137,	(23,181) : 210,	(24,181) : 587,	(25,181) : 491,	(26,181) : 353,	(27,181) : 313,	(28,181) : 282,	(29,181) : 462,	(30,181) : 288,	(31,181) : 228,	(32,181) : 254,	(33,181) : 374,	(34,181) : 192,	(35,181) : 367,	(36,181) : 185,	(37,181) : 170,	(38,181) : 247,	(39,181) : 261,	(40,181) : 507,
(1,182) : 145,	(2,182) : 136,	(3,182) : 515,	(4,182) : 571,	(5,182) : 117,	(6,182) : 84,	(7,182) : 133,	(8,182) : 763,	(9,182) : 242,	(10,182) : 136,	(11,182) : 137,	(12,182) : 565,	(13,182) : 182,	(14,182) : 237,	(15,182) : 417,	(16,182) : 642,	(17,182) : 521,	(18,182) : 98,	(19,182) : 505,	(20,182) : 139,	(21,182) : 268,	(22,182) : 52,	(23,182) : 94,	(24,182) : 524,	(25,182) : 641,	(26,182) : 237,	(27,182) : 185,	(28,182) : 137,	(29,182) : 417,	(30,182) : 144,	(31,182) : 95,	(32,182) : 107,	(33,182) : 225,	(34,182) : 59,	(35,182) : 248,	(36,182) : 295,	(37,182) : 318,	(38,182) : 122,	(39,182) : 133,	(40,182) : 656,
(1,183) : 160,	(2,183) : 112,	(3,183) : 568,	(4,183) : 685,	(5,183) : 60,	(6,183) : 90,	(7,183) : 153,	(8,183) : 787,	(9,183) : 207,	(10,183) : 155,	(11,183) : 113,	(12,183) : 661,	(13,183) : 277,	(14,183) : 203,	(15,183) : 529,	(16,183) : 751,	(17,183) : 634,	(18,183) : 61,	(19,183) : 598,	(20,183) : 159,	(21,183) : 382,	(22,183) : 67,	(23,183) : 64,	(24,183) : 637,	(25,183) : 641,	(26,183) : 338,	(27,183) : 281,	(28,183) : 157,	(29,183) : 532,	(30,183) : 161,	(31,183) : 95,	(32,183) : 184,	(33,183) : 274,	(34,183) : 85,	(35,183) : 195,	(36,183) : 241,	(37,183) : 311,	(38,183) : 96,	(39,183) : 109,	(40,183) : 652,
(1,184) : 92,	(2,184) : 128,	(3,184) : 595,	(4,184) : 517,	(5,184) : 152,	(6,184) : 103,	(7,184) : 83,	(8,184) : 856,	(9,184) : 349,	(10,184) : 85,	(11,184) : 129,	(12,184) : 474,	(13,184) : 92,	(14,184) : 344,	(15,184) : 361,	(16,184) : 572,	(17,184) : 467,	(18,184) : 136,	(19,184) : 411,	(20,184) : 85,	(21,184) : 227,	(22,184) : 147,	(23,184) : 131,	(24,184) : 470,	(25,184) : 743,	(26,184) : 156,	(27,184) : 97,	(28,184) : 84,	(29,184) : 383,	(30,184) : 89,	(31,184) : 104,	(32,184) : 4,	(33,184) : 121,	(34,184) : 103,	(35,184) : 224,	(36,184) : 400,	(37,184) : 424,	(38,184) : 127,	(39,184) : 128,	(40,184) : 760,
(1,185) : 502,	(2,185) : 472,	(3,185) : 260,	(4,185) : 762,	(5,185) : 425,	(6,185) : 428,	(7,185) : 492,	(8,185) : 420,	(9,185) : 177,	(10,185) : 494,	(11,185) : 473,	(12,185) : 857,	(13,185) : 516,	(14,185) : 180,	(15,185) : 640,	(16,185) : 865,	(17,185) : 719,	(18,185) : 416,	(19,185) : 806,	(20,185) : 498,	(21,185) : 508,	(22,185) : 347,	(23,185) : 416,	(24,185) : 721,	(25,185) : 277,	(26,185) : 548,	(27,185) : 516,	(28,185) : 495,	(29,185) : 588,	(30,185) : 501,	(31,185) : 439,	(32,185) : 468,	(33,185) : 588,	(34,185) : 405,	(35,185) : 565,	(36,185) : 194,	(37,185) : 64,	(38,185) : 454,	(39,185) : 468,	(40,185) : 292,
(1,186) : 146,	(2,186) : 98,	(3,186) : 577,	(4,186) : 679,	(5,186) : 46,	(6,186) : 78,	(7,186) : 140,	(8,186) : 799,	(9,186) : 221,	(10,186) : 141,	(11,186) : 99,	(12,186) : 650,	(13,186) : 268,	(14,186) : 217,	(15,186) : 523,	(16,186) : 743,	(17,186) : 628,	(18,186) : 47,	(19,186) : 588,	(20,186) : 145,	(21,186) : 377,	(22,186) : 68,	(23,186) : 51,	(24,186) : 631,	(25,186) : 654,	(26,186) : 329,	(27,186) : 272,	(28,186) : 144,	(29,186) : 528,	(30,186) : 147,	(31,186) : 82,	(32,186) : 174,	(33,186) : 261,	(34,186) : 75,	(35,186) : 183,	(36,186) : 255,	(37,186) : 324,	(38,186) : 82,	(39,186) : 95,	(40,186) : 665,
(1,187) : 36,	(2,187) : 20,	(3,187) : 648,	(4,187) : 626,	(5,187) : 69,	(6,187) : 52,	(7,187) : 35,	(8,187) : 889,	(9,187) : 329,	(10,187) : 35,	(11,187) : 20,	(12,187) : 563,	(13,187) : 198,	(14,187) : 324,	(15,187) : 470,	(16,187) : 676,	(17,187) : 577,	(18,187) : 67,	(19,187) : 499,	(20,187) : 38,	(21,187) : 337,	(22,187) : 133,	(23,187) : 65,	(24,187) : 579,	(25,187) : 755,	(26,187) : 262,	(27,187) : 204,	(28,187) : 37,	(29,187) : 494,	(30,187) : 38,	(31,187) : 41,	(32,187) : 107,	(33,187) : 154,	(34,187) : 77,	(35,187) : 120,	(36,187) : 367,	(37,187) : 425,	(38,187) : 31,	(39,187) : 22,	(40,187) : 768,
(1,188) : 117,	(2,188) : 73,	(3,188) : 584,	(4,188) : 654,	(5,188) : 28,	(6,188) : 47,	(7,188) : 110,	(8,188) : 814,	(9,188) : 244,	(10,188) : 112,	(11,188) : 74,	(12,188) : 620,	(13,188) : 239,	(14,188) : 240,	(15,188) : 497,	(16,188) : 716,	(17,188) : 603,	(18,188) : 18,	(19,188) : 558,	(20,188) : 116,	(21,188) : 353,	(22,188) : 63,	(23,188) : 21,	(24,188) : 606,	(25,188) : 673,	(26,188) : 301,	(27,188) : 243,	(28,188) : 114,	(29,188) : 506,	(30,188) : 118,	(31,188) : 52,	(32,188) : 144,	(33,188) : 230,	(34,188) : 47,	(35,188) : 171,	(36,188) : 282,	(37,188) : 343,	(38,188) : 56,	(39,188) : 70,	(40,188) : 686,
(1,189) : 677,	(2,189) : 732,	(3,189) : 1019,	(4,189) : 370,	(5,189) : 774,	(6,189) : 730,	(7,189) : 678,	(8,189) : 1308,	(9,189) : 960,	(10,189) : 677,	(11,189) : 732,	(12,189) : 167,	(13,189) : 546,	(14,189) : 956,	(15,189) : 404,	(16,189) : 266,	(17,189) : 378,	(18,189) : 762,	(19,189) : 223,	(20,189) : 674,	(21,189) : 517,	(22,189) : 775,	(23,189) : 757,	(24,189) : 379,	(25,189) : 1268,	(26,189) : 493,	(27,189) : 542,	(28,189) : 675,	(29,189) : 486,	(30,189) : 674,	(31,189) : 727,	(32,189) : 633,	(33,189) : 563,	(34,189) : 733,	(35,189) : 765,	(36,189) : 1018,	(37,189) : 1003,	(38,189) : 741,	(39,189) : 734,	(40,189) : 1292,
(1,190) : 573,	(2,190) : 614,	(3,190) : 637,	(4,190) : 54,	(5,190) : 632,	(6,190) : 583,	(7,190) : 566,	(8,190) : 921,	(9,190) : 709,	(10,190) : 568,	(11,190) : 615,	(12,190) : 289,	(13,190) : 402,	(14,190) : 705,	(15,190) : 130,	(16,190) : 170,	(17,190) : 41,	(18,190) : 614,	(19,190) : 292,	(20,190) : 567,	(21,190) : 266,	(22,190) : 584,	(23,190) : 608,	(24,190) : 39,	(25,190) : 913,	(26,190) : 341,	(27,190) : 396,	(28,190) : 566,	(29,190) : 125,	(30,190) : 569,	(31,190) : 587,	(32,190) : 490,	(33,190) : 506,	(34,190) : 572,	(35,190) : 701,	(36,190) : 771,	(37,190) : 710,	(38,190) : 613,	(39,190) : 615,	(40,190) : 940,
(1,191) : 71,	(2,191) : 34,	(3,191) : 679,	(4,191) : 669,	(5,191) : 73,	(6,191) : 85,	(7,191) : 75,	(8,191) : 913,	(9,191) : 339,	(10,191) : 74,	(11,191) : 33,	(12,191) : 597,	(13,191) : 241,	(14,191) : 335,	(15,191) : 514,	(16,191) : 717,	(17,191) : 620,	(18,191) : 83,	(19,191) : 533,	(20,191) : 75,	(21,191) : 382,	(22,191) : 158,	(23,191) : 84,	(24,191) : 623,	(25,191) : 772,	(26,191) : 304,	(27,191) : 246,	(28,191) : 76,	(29,191) : 539,	(30,191) : 74,	(31,191) : 75,	(32,191) : 152,	(33,191) : 179,	(34,191) : 111,	(35,191) : 78,	(36,191) : 371,	(37,191) : 442,	(38,191) : 48,	(39,191) : 37,	(40,191) : 783,
(1,192) : 143,	(2,192) : 129,	(3,192) : 519,	(4,192) : 585,	(5,192) : 107,	(6,192) : 78,	(7,192) : 132,	(8,192) : 764,	(9,192) : 234,	(10,192) : 135,	(11,192) : 130,	(12,192) : 577,	(13,192) : 193,	(14,192) : 229,	(15,192) : 430,	(16,192) : 654,	(17,192) : 535,	(18,192) : 88,	(19,192) : 516,	(20,192) : 138,	(21,192) : 282,	(22,192) : 39,	(23,192) : 84,	(24,192) : 537,	(25,192) : 638,	(26,192) : 249,	(27,192) : 196,	(28,192) : 136,	(29,192) : 431,	(30,192) : 142,	(31,192) : 89,	(32,192) : 114,	(33,192) : 229,	(34,192) : 52,	(35,192) : 241,	(36,192) : 286,	(37,192) : 314,	(38,192) : 115,	(39,192) : 126,	(40,192) : 654,
(1,193) : 534,	(2,193) : 577,	(3,193) : 648,	(4,193) : 68,	(5,193) : 597,	(6,193) : 548,	(7,193) : 528,	(8,193) : 935,	(9,193) : 690,	(10,193) : 529,	(11,193) : 578,	(12,193) : 249,	(13,193) : 363,	(14,193) : 685,	(15,193) : 89,	(16,193) : 163,	(17,193) : 18,	(18,193) : 579,	(19,193) : 247,	(20,193) : 528,	(21,193) : 235,	(22,193) : 555,	(23,193) : 574,	(24,193) : 20,	(25,193) : 916,	(26,193) : 300,	(27,193) : 357,	(28,193) : 527,	(29,193) : 116,	(30,193) : 530,	(31,193) : 552,	(32,193) : 453,	(33,193) : 462,	(34,193) : 539,	(35,193) : 661,	(36,193) : 752,	(37,193) : 698,	(38,193) : 577,	(39,193) : 578,	(40,193) : 943,
(1,194) : 536,	(2,194) : 580,	(3,194) : 649,	(4,194) : 65,	(5,194) : 600,	(6,194) : 551,	(7,194) : 531,	(8,194) : 935,	(9,194) : 692,	(10,194) : 532,	(11,194) : 581,	(12,194) : 250,	(13,194) : 365,	(14,194) : 688,	(15,194) : 92,	(16,194) : 161,	(17,194) : 15,	(18,194) : 582,	(19,194) : 248,	(20,194) : 531,	(21,194) : 238,	(22,194) : 557,	(23,194) : 577,	(24,194) : 17,	(25,194) : 918,	(26,194) : 303,	(27,194) : 359,	(28,194) : 530,	(29,194) : 118,	(30,194) : 533,	(31,194) : 555,	(32,194) : 456,	(33,194) : 465,	(34,194) : 541,	(35,194) : 663,	(36,194) : 754,	(37,194) : 700,	(38,194) : 580,	(39,194) : 581,	(40,194) : 945,
(1,195) : 39,	(2,195) : 73,	(3,195) : 698,	(4,195) : 620,	(5,195) : 126,	(6,195) : 109,	(7,195) : 50,	(8,195) : 944,	(9,195) : 388,	(10,195) : 48,	(11,195) : 73,	(12,195) : 532,	(13,195) : 194,	(14,195) : 384,	(15,195) : 466,	(16,195) : 660,	(17,195) : 571,	(18,195) : 126,	(19,195) : 468,	(20,195) : 45,	(21,195) : 345,	(22,195) : 190,	(23,195) : 125,	(24,195) : 574,	(25,195) : 813,	(26,195) : 254,	(27,195) : 200,	(28,195) : 47,	(29,195) : 500,	(30,195) : 41,	(31,195) : 99,	(32,195) : 122,	(33,195) : 112,	(34,195) : 133,	(35,195) : 106,	(36,195) : 426,	(37,195) : 484,	(38,195) : 89,	(39,195) : 77,	(40,195) : 827,
(1,196) : 59,	(2,196) : 3,	(3,196) : 646,	(4,196) : 646,	(5,196) : 50,	(6,196) : 51,	(7,196) : 58,	(8,196) : 882,	(9,196) : 315,	(10,196) : 58,	(11,196) : 3,	(12,196) : 586,	(13,196) : 219,	(14,196) : 311,	(15,196) : 490,	(16,196) : 698,	(17,196) : 597,	(18,196) : 54,	(19,196) : 522,	(20,196) : 61,	(21,196) : 355,	(22,196) : 126,	(23,196) : 53,	(24,196) : 599,	(25,196) : 744,	(26,196) : 283,	(27,196) : 224,	(28,196) : 60,	(29,196) : 511,	(30,196) : 61,	(31,196) : 40,	(32,196) : 126,	(33,196) : 176,	(34,196) : 77,	(35,196) : 113,	(36,196) : 351,	(37,196) : 415,	(38,196) : 16,	(39,196) : 2,	(40,196) : 757,
(1,197) : 71,	(2,197) : 117,	(3,197) : 720,	(4,197) : 594,	(5,197) : 170,	(6,197) : 146,	(7,197) : 80,	(8,197) : 973,	(9,197) : 428,	(10,197) : 78,	(11,197) : 117,	(12,197) : 493,	(13,197) : 178,	(14,197) : 424,	(15,197) : 443,	(16,197) : 628,	(17,197) : 547,	(18,197) : 168,	(19,197) : 430,	(20,197) : 74,	(21,197) : 332,	(22,197) : 226,	(23,197) : 166,	(24,197) : 549,	(25,197) : 848,	(26,197) : 234,	(27,197) : 185,	(28,197) : 76,	(29,197) : 485,	(30,197) : 70,	(31,197) : 137,	(32,197) : 128,	(33,197) : 74,	(34,197) : 167,	(35,197) : 137,	(36,197) : 468,	(37,197) : 520,	(38,197) : 132,	(39,197) : 121,	(40,197) : 862,
(1,198) : 92,	(2,198) : 110,	(3,198) : 572,	(4,198) : 544,	(5,198) : 121,	(6,198) : 72,	(7,198) : 81,	(8,198) : 827,	(9,198) : 310,	(10,198) : 84,	(11,198) : 111,	(12,198) : 512,	(13,198) : 127,	(14,198) : 305,	(15,198) : 387,	(16,198) : 604,	(17,198) : 494,	(18,198) : 104,	(19,198) : 450,	(20,198) : 86,	(21,198) : 247,	(22,198) : 107,	(23,198) : 98,	(24,198) : 496,	(25,198) : 709,	(26,198) : 189,	(27,198) : 132,	(28,198) : 84,	(29,198) : 402,	(30,198) : 90,	(31,198) : 77,	(32,198) : 39,	(33,198) : 155,	(34,198) : 65,	(35,198) : 217,	(36,198) : 360,	(37,198) : 387,	(38,198) : 104,	(39,198) : 109,	(40,198) : 725,
(1,199) : 531,	(2,199) : 575,	(3,199) : 643,	(4,199) : 71,	(5,199) : 595,	(6,199) : 546,	(7,199) : 526,	(8,199) : 930,	(9,199) : 685,	(10,199) : 527,	(11,199) : 576,	(12,199) : 252,	(13,199) : 360,	(14,199) : 681,	(15,199) : 87,	(16,199) : 168,	(17,199) : 22,	(18,199) : 577,	(19,199) : 249,	(20,199) : 526,	(21,199) : 232,	(22,199) : 551,	(23,199) : 571,	(24,199) : 24,	(25,199) : 911,	(26,199) : 298,	(27,199) : 354,	(28,199) : 525,	(29,199) : 111,	(30,199) : 528,	(31,199) : 549,	(32,199) : 451,	(33,199) : 461,	(34,199) : 536,	(35,199) : 659,	(36,199) : 748,	(37,199) : 694,	(38,199) : 574,	(39,199) : 575,	(40,199) : 938,
(1,200) : 105,	(2,200) : 160,	(3,200) : 713,	(4,200) : 539,	(5,200) : 209,	(6,200) : 175,	(7,200) : 110,	(8,200) : 976,	(9,200) : 453,	(10,200) : 108,	(11,200) : 160,	(12,200) : 436,	(13,200) : 136,	(14,200) : 448,	(15,200) : 390,	(16,200) : 571,	(17,200) : 492,	(18,200) : 203,	(19,200) : 372,	(20,200) : 105,	(21,200) : 289,	(22,200) : 247,	(23,200) : 199,	(24,200) : 495,	(25,200) : 860,	(26,200) : 184,	(27,200) : 142,	(28,200) : 106,	(29,200) : 437,	(30,200) : 103,	(31,200) : 168,	(32,200) : 119,	(33,200) : 15,	(34,200) : 190,	(35,200) : 195,	(36,200) : 498,	(37,200) : 537,	(38,200) : 172,	(39,200) : 163,	(40,200) : 876,
(1,201) : 57,	(2,201) : 61,	(3,201) : 704,	(4,201) : 651,	(5,201) : 111,	(6,201) : 108,	(7,201) : 66,	(8,201) : 944,	(9,201) : 377,	(10,201) : 64,	(11,201) : 60,	(12,201) : 565,	(13,201) : 224,	(14,201) : 373,	(15,201) : 496,	(16,201) : 692,	(17,201) : 602,	(18,201) : 116,	(19,201) : 501,	(20,201) : 63,	(21,201) : 372,	(22,201) : 187,	(23,201) : 116,	(24,201) : 604,	(25,201) : 807,	(26,201) : 285,	(27,201) : 230,	(28,201) : 65,	(29,201) : 528,	(30,201) : 59,	(31,201) : 97,	(32,201) : 145,	(33,201) : 144,	(34,201) : 133,	(35,201) : 76,	(36,201) : 411,	(37,201) : 477,	(38,201) : 78,	(39,201) : 64,	(40,201) : 820,
(1,202) : 76,	(2,202) : 131,	(3,202) : 667,	(4,202) : 533,	(5,202) : 174,	(6,202) : 135,	(7,202) : 76,	(8,202) : 928,	(9,202) : 408,	(10,202) : 76,	(11,202) : 131,	(12,202) : 453,	(13,202) : 110,	(14,202) : 403,	(15,202) : 379,	(16,202) : 574,	(17,202) : 484,	(18,202) : 165,	(19,202) : 389,	(20,202) : 73,	(21,202) : 264,	(22,202) : 202,	(23,202) : 161,	(24,202) : 487,	(25,202) : 812,	(26,202) : 168,	(27,202) : 117,	(28,202) : 73,	(29,202) : 417,	(30,202) : 73,	(31,202) : 129,	(32,202) : 71,	(33,202) : 53,	(34,202) : 147,	(35,202) : 193,	(36,202) : 455,	(37,202) : 490,	(38,202) : 139,	(39,202) : 133,	(40,202) : 829,
(1,203) : 570,	(2,203) : 617,	(3,203) : 718,	(4,203) : 56,	(5,203) : 642,	(6,203) : 593,	(7,203) : 566,	(8,203) : 1004,	(9,203) : 751,	(10,203) : 567,	(11,203) : 618,	(12,203) : 203,	(13,203) : 401,	(14,203) : 746,	(15,203) : 139,	(16,203) : 94,	(17,203) : 58,	(18,203) : 625,	(19,203) : 218,	(20,203) : 565,	(21,203) : 290,	(22,203) : 606,	(23,203) : 619,	(24,203) : 58,	(25,203) : 986,	(26,203) : 337,	(27,203) : 395,	(28,203) : 565,	(29,203) : 186,	(30,203) : 567,	(31,203) : 595,	(32,203) : 495,	(33,203) : 488,	(34,203) : 585,	(35,203) : 693,	(36,203) : 813,	(37,203) : 764,	(38,203) : 618,	(39,203) : 618,	(40,203) : 1013,
(1,204) : 64,	(2,204) : 95,	(3,204) : 604,	(4,204) : 550,	(5,204) : 120,	(6,204) : 73,	(7,204) : 54,	(8,204) : 859,	(9,204) : 335,	(10,204) : 56,	(11,204) : 96,	(12,204) : 502,	(13,204) : 124,	(14,204) : 330,	(15,204) : 394,	(16,204) : 604,	(17,204) : 500,	(18,204) : 106,	(19,204) : 439,	(20,204) : 57,	(21,204) : 259,	(22,204) : 129,	(23,204) : 100,	(24,204) : 503,	(25,204) : 740,	(26,204) : 188,	(27,204) : 130,	(28,204) : 55,	(29,204) : 416,	(30,204) : 61,	(31,204) : 72,	(32,204) : 30,	(33,204) : 127,	(34,204) : 77,	(35,204) : 194,	(36,204) : 383,	(37,204) : 417,	(38,204) : 94,	(39,204) : 95,	(40,204) : 756,
(1,205) : 524,	(2,205) : 570,	(3,205) : 677,	(4,205) : 79,	(5,205) : 593,	(6,205) : 544,	(7,205) : 520,	(8,205) : 965,	(9,205) : 700,	(10,205) : 521,	(11,205) : 571,	(12,205) : 212,	(13,205) : 354,	(14,205) : 696,	(15,205) : 88,	(16,205) : 144,	(17,205) : 40,	(18,205) : 576,	(19,205) : 211,	(20,205) : 519,	(21,205) : 239,	(22,205) : 556,	(23,205) : 570,	(24,205) : 43,	(25,205) : 941,	(26,205) : 290,	(27,205) : 348,	(28,205) : 519,	(29,205) : 142,	(30,205) : 521,	(31,205) : 547,	(32,205) : 447,	(33,205) : 447,	(34,205) : 536,	(35,205) : 649,	(36,205) : 762,	(37,205) : 714,	(38,205) : 571,	(39,205) : 571,	(40,205) : 968,
(1,206) : 131,	(2,206) : 172,	(3,206) : 591,	(4,206) : 473,	(5,206) : 196,	(6,206) : 147,	(7,206) : 124,	(8,206) : 859,	(9,206) : 377,	(10,206) : 126,	(11,206) : 173,	(12,206) : 433,	(13,206) : 49,	(14,206) : 372,	(15,206) : 316,	(16,206) : 527,	(17,206) : 423,	(18,206) : 180,	(19,206) : 371,	(20,206) : 125,	(21,206) : 185,	(22,206) : 182,	(23,206) : 174,	(24,206) : 425,	(25,206) : 755,	(26,206) : 111,	(27,206) : 53,	(28,206) : 124,	(29,206) : 342,	(30,206) : 128,	(31,206) : 149,	(32,206) : 48,	(33,206) : 123,	(34,206) : 144,	(35,206) : 263,	(36,206) : 431,	(37,206) : 444,	(38,206) : 172,	(39,206) : 172,	(40,206) : 774,
(1,207) : 71,	(2,207) : 110,	(3,207) : 611,	(4,207) : 534,	(5,207) : 139,	(6,207) : 92,	(7,207) : 63,	(8,207) : 869,	(9,207) : 352,	(10,207) : 65,	(11,207) : 111,	(12,207) : 483,	(13,207) : 106,	(14,207) : 347,	(15,207) : 378,	(16,207) : 586,	(17,207) : 484,	(18,207) : 125,	(19,207) : 420,	(20,207) : 65,	(21,207) : 246,	(22,207) : 147,	(23,207) : 120,	(24,207) : 487,	(25,207) : 753,	(26,207) : 171,	(27,207) : 112,	(28,207) : 63,	(29,207) : 403,	(30,207) : 68,	(31,207) : 91,	(32,207) : 18,	(33,207) : 111,	(34,207) : 97,	(35,207) : 203,	(36,207) : 401,	(37,207) : 431,	(38,207) : 111,	(39,207) : 111,	(40,207) : 769,
(1,208) : 449,	(2,208) : 418,	(3,208) : 300,	(4,208) : 744,	(5,208) : 371,	(6,208) : 375,	(7,208) : 439,	(8,208) : 475,	(9,208) : 124,	(10,208) : 442,	(11,208) : 419,	(12,208) : 824,	(13,208) : 472,	(14,208) : 126,	(15,208) : 615,	(16,208) : 843,	(17,208) : 699,	(18,208) : 362,	(19,208) : 771,	(20,208) : 445,	(21,208) : 477,	(22,208) : 294,	(23,208) : 362,	(24,208) : 700,	(25,208) : 329,	(26,208) : 508,	(27,208) : 473,	(28,208) : 443,	(29,208) : 569,	(30,208) : 449,	(31,208) : 385,	(32,208) : 419,	(33,208) : 539,	(34,208) : 352,	(35,208) : 510,	(36,208) : 149,	(37,208) : 20,	(38,208) : 400,	(39,208) : 414,	(40,208) : 343,
(1,209) : 195,	(2,209) : 220,	(3,209) : 508,	(4,209) : 448,	(5,209) : 225,	(6,209) : 179,	(7,209) : 185,	(8,209) : 780,	(9,209) : 338,	(10,209) : 188,	(11,209) : 221,	(12,209) : 457,	(13,209) : 100,	(14,209) : 333,	(15,209) : 294,	(16,209) : 520,	(17,209) : 398,	(18,209) : 206,	(19,209) : 400,	(20,209) : 189,	(21,209) : 146,	(22,209) : 174,	(23,209) : 200,	(24,209) : 401,	(25,209) : 686,	(26,209) : 132,	(27,209) : 99,	(28,209) : 187,	(29,209) : 297,	(30,209) : 193,	(31,209) : 185,	(32,209) : 109,	(33,209) : 207,	(34,209) : 163,	(35,209) : 325,	(36,209) : 397,	(37,209) : 387,	(38,209) : 213,	(39,209) : 219,	(40,209) : 706,
(1,210) : 219,	(2,210) : 260,	(3,210) : 575,	(4,210) : 384,	(5,210) : 282,	(6,210) : 233,	(7,210) : 212,	(8,210) : 855,	(9,210) : 429,	(10,210) : 214,	(11,210) : 261,	(12,210) : 366,	(13,210) : 52,	(14,210) : 424,	(15,210) : 228,	(16,210) : 443,	(17,210) : 334,	(18,210) : 265,	(19,210) : 308,	(20,210) : 213,	(21,210) : 102,	(22,210) : 253,	(23,210) : 259,	(24,210) : 337,	(25,210) : 771,	(26,210) : 40,	(27,210) : 46,	(28,210) : 212,	(29,210) : 256,	(30,210) : 216,	(31,210) : 236,	(32,210) : 136,	(33,210) : 177,	(34,210) : 226,	(35,210) : 349,	(36,210) : 487,	(37,210) : 478,	(38,210) : 260,	(39,210) : 261,	(40,210) : 792,
(1,211) : 496,	(2,211) : 537,	(3,211) : 594,	(4,211) : 120,	(5,211) : 554,	(6,211) : 505,	(7,211) : 490,	(8,211) : 882,	(9,211) : 635,	(10,211) : 491,	(11,211) : 538,	(12,211) : 283,	(13,211) : 326,	(14,211) : 631,	(15,211) : 62,	(16,211) : 220,	(17,211) : 74,	(18,211) : 535,	(19,211) : 269,	(20,211) : 490,	(21,211) : 187,	(22,211) : 506,	(23,211) : 530,	(24,211) : 75,	(25,211) : 860,	(26,211) : 266,	(27,211) : 320,	(28,211) : 489,	(29,211) : 60,	(30,211) : 493,	(31,211) : 509,	(32,211) : 413,	(33,211) : 434,	(34,211) : 494,	(35,211) : 625,	(36,211) : 697,	(37,211) : 641,	(38,211) : 535,	(39,211) : 537,	(40,211) : 887,
(1,212) : 568,	(2,212) : 616,	(3,212) : 738,	(4,212) : 80,	(5,212) : 642,	(6,212) : 593,	(7,212) : 564,	(8,212) : 1025,	(9,212) : 760,	(10,212) : 564,	(11,212) : 617,	(12,212) : 179,	(13,212) : 400,	(14,212) : 755,	(15,212) : 148,	(16,212) : 84,	(17,212) : 81,	(18,212) : 625,	(19,212) : 197,	(20,212) : 563,	(21,212) : 297,	(22,212) : 610,	(23,212) : 620,	(24,212) : 82,	(25,212) : 1004,	(26,212) : 336,	(27,212) : 394,	(28,212) : 562,	(29,212) : 204,	(30,212) : 565,	(31,212) : 595,	(32,212) : 494,	(33,212) : 482,	(34,212) : 587,	(35,212) : 688,	(36,212) : 821,	(37,212) : 776,	(38,212) : 618,	(39,212) : 617,	(40,212) : 1031,
(1,213) : 588,	(2,213) : 642,	(3,213) : 902,	(4,213) : 269,	(5,213) : 681,	(6,213) : 634,	(7,213) : 587,	(8,213) : 1192,	(9,213) : 853,	(10,213) : 587,	(11,213) : 643,	(12,213) : 60,	(13,213) : 444,	(14,213) : 848,	(15,213) : 288,	(16,213) : 183,	(17,213) : 269,	(18,213) : 667,	(19,213) : 123,	(20,213) : 585,	(21,213) : 403,	(22,213) : 673,	(23,213) : 662,	(24,213) : 270,	(25,213) : 1151,	(26,213) : 388,	(27,213) : 441,	(28,213) : 585,	(29,213) : 371,	(30,213) : 585,	(31,213) : 633,	(32,213) : 536,	(33,213) : 479,	(34,213) : 636,	(35,213) : 687,	(36,213) : 912,	(37,213) : 891,	(38,213) : 649,	(39,213) : 644,	(40,213) : 1176,
(1,214) : 141,	(2,214) : 153,	(3,214) : 526,	(4,214) : 519,	(5,214) : 153,	(6,214) : 108,	(7,214) : 130,	(8,214) : 786,	(9,214) : 293,	(10,214) : 132,	(11,214) : 155,	(12,214) : 510,	(13,214) : 128,	(14,214) : 288,	(15,214) : 363,	(16,214) : 587,	(17,214) : 469,	(18,214) : 134,	(19,214) : 450,	(20,214) : 134,	(21,214) : 217,	(22,214) : 107,	(23,214) : 129,	(24,214) : 471,	(25,214) : 675,	(26,214) : 181,	(27,214) : 130,	(28,214) : 132,	(29,214) : 369,	(30,214) : 138,	(31,214) : 116,	(32,214) : 71,	(33,214) : 191,	(34,214) : 91,	(35,214) : 263,	(36,214) : 348,	(37,214) : 359,	(38,214) : 145,	(39,214) : 152,	(40,214) : 692,
(1,215) : 40,	(2,215) : 24,	(3,215) : 665,	(4,215) : 639,	(5,215) : 76,	(6,215) : 69,	(7,215) : 44,	(8,215) : 905,	(9,215) : 341,	(10,215) : 43,	(11,215) : 23,	(12,215) : 568,	(13,215) : 210,	(14,215) : 336,	(15,215) : 483,	(16,215) : 686,	(17,215) : 589,	(18,215) : 79,	(19,215) : 505,	(20,215) : 45,	(21,215) : 352,	(22,215) : 148,	(23,215) : 78,	(24,215) : 592,	(25,215) : 769,	(26,215) : 273,	(27,215) : 216,	(28,215) : 45,	(29,215) : 509,	(30,215) : 43,	(31,215) : 58,	(32,215) : 122,	(33,215) : 154,	(34,215) : 95,	(35,215) : 102,	(36,215) : 377,	(37,215) : 439,	(38,215) : 41,	(39,215) : 27,	(40,215) : 782,
(1,216) : 439,	(2,216) : 492,	(3,216) : 783,	(4,216) : 256,	(5,216) : 529,	(6,216) : 482,	(7,216) : 438,	(8,216) : 1073,	(9,216) : 702,	(10,216) : 438,	(11,216) : 493,	(12,216) : 94,	(13,216) : 291,	(14,216) : 697,	(15,216) : 187,	(16,216) : 237,	(17,216) : 228,	(18,216) : 515,	(19,216) : 39,	(20,216) : 435,	(21,216) : 264,	(22,216) : 520,	(23,216) : 510,	(24,216) : 230,	(25,216) : 1018,	(26,216) : 235,	(27,216) : 288,	(28,216) : 435,	(29,216) : 278,	(30,216) : 436,	(31,216) : 481,	(32,216) : 383,	(33,216) : 334,	(34,216) : 483,	(35,216) : 544,	(36,216) : 760,	(37,216) : 745,	(38,216) : 498,	(39,216) : 494,	(40,216) : 1041,
(1,217) : 5,	(2,217) : 60,	(3,217) : 662,	(4,217) : 598,	(5,217) : 108,	(6,217) : 80,	(7,217) : 15,	(8,217) : 910,	(9,217) : 362,	(10,217) : 12,	(11,217) : 60,	(12,217) : 525,	(13,217) : 169,	(14,217) : 357,	(15,217) : 443,	(16,217) : 644,	(17,217) : 549,	(18,217) : 104,	(19,217) : 461,	(20,217) : 9,	(21,217) : 316,	(22,217) : 160,	(23,217) : 101,	(24,217) : 551,	(25,217) : 782,	(26,217) : 232,	(27,217) : 175,	(28,217) : 11,	(29,217) : 472,	(30,217) : 5,	(31,217) : 71,	(32,217) : 89,	(33,217) : 113,	(34,217) : 101,	(35,217) : 132,	(36,217) : 403,	(37,217) : 454,	(38,217) : 71,	(39,217) : 62,	(40,217) : 796,
(1,218) : 64,	(2,218) : 46,	(3,218) : 693,	(4,218) : 664,	(5,218) : 91,	(6,218) : 97,	(7,218) : 71,	(8,218) : 929,	(9,218) : 358,	(10,218) : 70,	(11,218) : 45,	(12,218) : 584,	(13,218) : 235,	(14,218) : 353,	(15,218) : 509,	(16,218) : 708,	(17,218) : 615,	(18,218) : 100,	(19,218) : 520,	(20,218) : 70,	(21,218) : 381,	(22,218) : 173,	(23,218) : 100,	(24,218) : 617,	(25,218) : 790,	(26,218) : 298,	(27,218) : 241,	(28,218) : 71,	(29,218) : 537,	(30,218) : 67,	(31,218) : 86,	(32,218) : 151,	(33,218) : 165,	(34,218) : 123,	(35,218) : 71,	(36,218) : 390,	(37,218) : 460,	(38,218) : 63,	(39,218) : 49,	(40,218) : 801,
(1,219) : 320,	(2,219) : 306,	(3,219) : 341,	(4,219) : 580,	(5,219) : 273,	(6,219) : 255,	(7,219) : 309,	(8,219) : 590,	(9,219) : 180,	(10,219) : 311,	(11,219) : 307,	(12,219) : 647,	(13,219) : 303,	(14,219) : 177,	(15,219) : 444,	(16,219) : 674,	(17,219) : 533,	(18,219) : 257,	(19,219) : 595,	(20,219) : 314,	(21,219) : 302,	(22,219) : 182,	(23,219) : 255,	(24,219) : 535,	(25,219) : 482,	(26,219) : 334,	(27,219) : 303,	(28,219) : 312,	(29,219) : 407,	(30,219) : 319,	(31,219) : 267,	(32,219) : 267,	(33,219) : 387,	(34,219) : 230,	(35,219) : 415,	(36,219) : 241,	(37,219) : 189,	(38,219) : 290,	(39,219) : 303,	(40,219) : 501,
(1,220) : 11,	(2,220) : 55,	(3,220) : 648,	(4,220) : 595,	(5,220) : 99,	(6,220) : 67,	(7,220) : 0,	(8,220) : 896,	(9,220) : 349,	(10,220) : 3,	(11,220) : 55,	(12,220) : 528,	(13,220) : 166,	(14,220) : 344,	(15,220) : 439,	(16,220) : 643,	(17,220) : 545,	(18,220) : 93,	(19,220) : 464,	(20,220) : 6,	(21,220) : 309,	(22,220) : 146,	(23,220) : 89,	(24,220) : 548,	(25,220) : 768,	(26,220) : 229,	(27,220) : 172,	(28,220) : 4,	(29,220) : 466,	(30,220) : 10,	(31,220) : 59,	(32,220) : 80,	(33,220) : 121,	(34,220) : 87,	(35,220) : 141,	(36,220) : 391,	(37,220) : 440,	(38,220) : 63,	(39,220) : 57,	(40,220) : 782,
(1,221) : 224,	(2,221) : 190,	(3,221) : 473,	(4,221) : 656,	(5,221) : 145,	(6,221) : 148,	(7,221) : 215,	(8,221) : 695,	(9,221) : 138,	(10,221) : 217,	(11,221) : 191,	(12,221) : 669,	(13,221) : 288,	(14,221) : 133,	(15,221) : 506,	(16,221) : 735,	(17,221) : 607,	(18,221) : 135,	(19,221) : 609,	(20,221) : 221,	(21,221) : 355,	(22,221) : 69,	(23,221) : 135,	(24,221) : 609,	(25,221) : 556,	(26,221) : 340,	(27,221) : 290,	(28,221) : 219,	(29,221) : 493,	(30,221) : 224,	(31,221) : 159,	(32,221) : 211,	(33,221) : 323,	(34,221) : 127,	(35,221) : 288,	(36,221) : 189,	(37,221) : 227,	(38,221) : 173,	(39,221) : 187,	(40,221) : 569,
(1,222) : 408,	(2,222) : 462,	(3,222) : 778,	(4,222) : 282,	(5,222) : 500,	(6,222) : 454,	(7,222) : 407,	(8,222) : 1067,	(9,222) : 680,	(10,222) : 407,	(11,222) : 462,	(12,222) : 121,	(13,222) : 265,	(14,222) : 675,	(15,222) : 197,	(16,222) : 270,	(17,222) : 250,	(18,222) : 486,	(19,222) : 59,	(20,222) : 405,	(21,222) : 253,	(22,222) : 495,	(23,222) : 481,	(24,222) : 253,	(25,222) : 1006,	(26,222) : 212,	(27,222) : 262,	(28,222) : 405,	(29,222) : 286,	(30,222) : 405,	(31,222) : 452,	(32,222) : 355,	(33,222) : 302,	(34,222) : 455,	(35,222) : 512,	(36,222) : 737,	(37,222) : 727,	(38,222) : 468,	(39,222) : 464,	(40,222) : 1029,
(1,223) : 370,	(2,223) : 417,	(3,223) : 626,	(4,223) : 230,	(5,223) : 441,	(6,223) : 392,	(7,223) : 366,	(8,223) : 915,	(9,223) : 570,	(10,223) : 367,	(11,223) : 417,	(12,223) : 240,	(13,223) : 200,	(14,223) : 565,	(15,223) : 79,	(16,223) : 284,	(17,223) : 182,	(18,223) : 424,	(19,223) : 197,	(20,223) : 365,	(21,223) : 111,	(22,223) : 410,	(23,223) : 418,	(24,223) : 184,	(25,223) : 862,	(26,223) : 136,	(27,223) : 194,	(28,223) : 365,	(29,223) : 149,	(30,223) : 367,	(31,223) : 394,	(32,223) : 294,	(33,223) : 297,	(34,223) : 385,	(35,223) : 496,	(36,223) : 631,	(37,223) : 600,	(38,223) : 417,	(39,223) : 417,	(40,223) : 886,
(1,224) : 506,	(2,224) : 554,	(3,224) : 690,	(4,224) : 104,	(5,224) : 579,	(6,224) : 530,	(7,224) : 502,	(8,224) : 978,	(9,224) : 695,	(10,224) : 503,	(11,224) : 554,	(12,224) : 187,	(13,224) : 337,	(14,224) : 691,	(15,224) : 85,	(16,224) : 149,	(17,224) : 69,	(18,224) : 562,	(19,224) : 183,	(20,224) : 502,	(21,224) : 232,	(22,224) : 545,	(23,224) : 556,	(24,224) : 72,	(25,224) : 949,	(26,224) : 273,	(27,224) : 332,	(28,224) : 501,	(29,224) : 154,	(30,224) : 503,	(31,224) : 532,	(32,224) : 431,	(33,224) : 425,	(34,224) : 523,	(35,224) : 629,	(36,224) : 757,	(37,224) : 714,	(38,224) : 555,	(39,224) : 554,	(40,224) : 976,
(1,225) : 646,	(2,225) : 695,	(3,225) : 806,	(4,225) : 117,	(5,225) : 723,	(6,225) : 674,	(7,225) : 642,	(8,225) : 1091,	(9,225) : 843,	(10,225) : 643,	(11,225) : 696,	(12,225) : 197,	(13,225) : 480,	(14,225) : 839,	(15,225) : 231,	(16,225) : 0,	(17,225) : 148,	(18,225) : 707,	(19,225) : 237,	(20,225) : 641,	(21,225) : 380,	(22,225) : 693,	(23,225) : 701,	(24,225) : 148,	(25,225) : 1078,	(26,225) : 416,	(27,225) : 474,	(28,225) : 641,	(29,225) : 279,	(30,225) : 643,	(31,225) : 676,	(32,225) : 575,	(33,225) : 555,	(34,225) : 668,	(35,225) : 764,	(36,225) : 905,	(37,225) : 858,	(38,225) : 698,	(39,225) : 696,	(40,225) : 1105,
(1,226) : 561,	(2,226) : 605,	(3,226) : 665,	(4,226) : 40,	(5,226) : 625,	(6,226) : 576,	(7,226) : 555,	(8,226) : 950,	(9,226) : 716,	(10,226) : 556,	(11,226) : 606,	(12,226) : 251,	(13,226) : 390,	(14,226) : 712,	(15,226) : 117,	(16,226) : 142,	(17,226) : 10,	(18,226) : 607,	(19,226) : 256,	(20,226) : 555,	(21,226) : 263,	(22,226) : 583,	(23,226) : 602,	(24,226) : 8,	(25,226) : 936,	(26,226) : 327,	(27,226) : 384,	(28,226) : 555,	(29,226) : 139,	(30,226) : 558,	(31,226) : 580,	(32,226) : 481,	(33,226) : 488,	(34,226) : 566,	(35,226) : 687,	(36,226) : 778,	(37,226) : 723,	(38,226) : 604,	(39,226) : 605,	(40,226) : 963,
(1,227) : 523,	(2,227) : 566,	(3,227) : 625,	(4,227) : 85,	(5,227) : 584,	(6,227) : 535,	(7,227) : 517,	(8,227) : 912,	(9,227) : 670,	(10,227) : 519,	(11,227) : 567,	(12,227) : 266,	(13,227) : 352,	(14,227) : 666,	(15,227) : 80,	(16,227) : 185,	(17,227) : 38,	(18,227) : 566,	(19,227) : 260,	(20,227) : 518,	(21,227) : 220,	(22,227) : 539,	(23,227) : 560,	(24,227) : 39,	(25,227) : 894,	(26,227) : 291,	(27,227) : 346,	(28,227) : 517,	(29,227) : 94,	(30,227) : 520,	(31,227) : 539,	(32,227) : 442,	(33,227) : 456,	(34,227) : 525,	(35,227) : 651,	(36,227) : 732,	(37,227) : 677,	(38,227) : 564,	(39,227) : 566,	(40,227) : 921,
(1,228) : 392,	(2,228) : 437,	(3,228) : 618,	(4,228) : 207,	(5,228) : 460,	(6,228) : 411,	(7,228) : 387,	(8,228) : 908,	(9,228) : 580,	(10,228) : 388,	(11,228) : 438,	(12,228) : 243,	(13,228) : 222,	(14,228) : 575,	(15,228) : 54,	(16,228) : 268,	(17,228) : 158,	(18,228) : 443,	(19,228) : 205,	(20,228) : 387,	(21,228) : 116,	(22,228) : 425,	(23,228) : 437,	(24,228) : 161,	(25,228) : 860,	(26,228) : 159,	(27,228) : 216,	(28,228) : 386,	(29,228) : 126,	(30,228) : 389,	(31,228) : 414,	(32,228) : 314,	(33,228) : 322,	(34,228) : 403,	(35,228) : 519,	(36,228) : 641,	(37,228) : 605,	(38,228) : 438,	(39,228) : 438,	(40,228) : 884,
(1,229) : 86,	(2,229) : 138,	(3,229) : 644,	(4,229) : 515,	(5,229) : 176,	(6,229) : 132,	(7,229) : 83,	(8,229) : 907,	(9,229) : 397,	(10,229) : 84,	(11,229) : 138,	(12,229) : 446,	(13,229) : 88,	(14,229) : 392,	(15,229) : 360,	(16,229) : 560,	(17,229) : 466,	(18,229) : 164,	(19,229) : 382,	(20,229) : 82,	(21,229) : 241,	(22,229) : 192,	(23,229) : 159,	(24,229) : 469,	(25,229) : 795,	(26,229) : 149,	(27,229) : 95,	(28,229) : 82,	(29,229) : 395,	(30,229) : 83,	(31,229) : 129,	(32,229) : 52,	(33,229) : 69,	(34,229) : 140,	(35,229) : 212,	(36,229) : 446,	(37,229) : 475,	(38,229) : 144,	(39,229) : 139,	(40,229) : 812,
(1,230) : 363,	(2,230) : 402,	(3,230) : 546,	(4,230) : 252,	(5,230) : 417,	(6,230) : 369,	(7,230) : 356,	(8,230) : 835,	(9,230) : 512,	(10,230) : 357,	(11,230) : 403,	(12,230) : 318,	(13,230) : 195,	(14,230) : 508,	(15,230) : 100,	(16,230) : 331,	(17,230) : 202,	(18,230) : 399,	(19,230) : 277,	(20,230) : 357,	(21,230) : 51,	(22,230) : 371,	(23,230) : 393,	(24,230) : 204,	(25,230) : 784,	(26,230) : 142,	(27,230) : 189,	(28,230) : 356,	(29,230) : 112,	(30,230) : 360,	(31,230) : 373,	(32,230) : 278,	(33,230) : 314,	(34,230) : 357,	(35,230) : 494,	(36,230) : 574,	(37,230) : 533,	(38,230) : 399,	(39,230) : 402,	(40,230) : 809,
(1,231) : 616,	(2,231) : 666,	(3,231) : 800,	(4,231) : 120,	(5,231) : 695,	(6,231) : 646,	(7,231) : 613,	(8,231) : 1086,	(9,231) : 821,	(10,231) : 613,	(11,231) : 667,	(12,231) : 167,	(13,231) : 451,	(14,231) : 817,	(15,231) : 211,	(16,231) : 33,	(17,231) : 140,	(18,231) : 679,	(19,231) : 204,	(20,231) : 612,	(21,231) : 358,	(22,231) : 667,	(23,231) : 673,	(24,231) : 140,	(25,231) : 1068,	(26,231) : 388,	(27,231) : 446,	(28,231) : 611,	(29,231) : 267,	(30,231) : 613,	(31,231) : 648,	(32,231) : 547,	(33,231) : 524,	(34,231) : 641,	(35,231) : 733,	(36,231) : 883,	(37,231) : 840,	(38,231) : 669,	(39,231) : 667,	(40,231) : 1095,
(1,232) : 61,	(2,232) : 12,	(3,232) : 658,	(4,232) : 655,	(5,232) : 56,	(6,232) : 63,	(7,232) : 63,	(8,232) : 893,	(9,232) : 323,	(10,232) : 62,	(11,232) : 11,	(12,232) : 590,	(13,232) : 226,	(14,232) : 318,	(15,232) : 499,	(16,232) : 705,	(17,232) : 605,	(18,232) : 63,	(19,232) : 526,	(20,232) : 65,	(21,232) : 365,	(22,232) : 137,	(23,232) : 64,	(24,232) : 608,	(25,232) : 753,	(26,232) : 290,	(27,232) : 232,	(28,232) : 65,	(29,232) : 521,	(30,232) : 64,	(31,232) : 52,	(32,232) : 135,	(33,232) : 177,	(34,232) : 88,	(35,232) : 101,	(36,232) : 357,	(37,232) : 424,	(38,232) : 26,	(39,232) : 14,	(40,232) : 766,
(1,233) : 516,	(2,233) : 558,	(3,233) : 615,	(4,233) : 96,	(5,233) : 575,	(6,233) : 527,	(7,233) : 510,	(8,233) : 902,	(9,233) : 659,	(10,233) : 511,	(11,233) : 559,	(12,233) : 273,	(13,233) : 345,	(14,233) : 655,	(15,233) : 75,	(16,233) : 197,	(17,233) : 49,	(18,233) : 557,	(19,233) : 264,	(20,233) : 510,	(21,233) : 210,	(22,233) : 529,	(23,233) : 552,	(24,233) : 51,	(25,233) : 882,	(26,233) : 284,	(27,233) : 339,	(28,233) : 509,	(29,233) : 82,	(30,233) : 513,	(31,233) : 531,	(32,233) : 433,	(33,233) : 450,	(34,233) : 516,	(35,233) : 644,	(36,233) : 721,	(37,233) : 666,	(38,233) : 556,	(39,233) : 558,	(40,233) : 909,
(1,234) : 561,	(2,234) : 606,	(3,234) : 688,	(4,234) : 43,	(5,234) : 628,	(6,234) : 579,	(7,234) : 556,	(8,234) : 974,	(9,234) : 728,	(10,234) : 557,	(11,234) : 607,	(12,234) : 227,	(13,234) : 390,	(14,234) : 724,	(15,234) : 121,	(16,234) : 122,	(17,234) : 28,	(18,234) : 611,	(19,234) : 235,	(20,234) : 555,	(21,234) : 271,	(22,234) : 589,	(23,234) : 605,	(24,234) : 28,	(25,234) : 957,	(26,234) : 327,	(27,234) : 384,	(28,234) : 555,	(29,234) : 158,	(30,234) : 558,	(31,234) : 582,	(32,234) : 483,	(33,234) : 483,	(34,234) : 571,	(35,234) : 685,	(36,234) : 791,	(37,234) : 739,	(38,234) : 606,	(39,234) : 607,	(40,234) : 984,
(1,235) : 96,	(2,235) : 55,	(3,235) : 596,	(4,235) : 642,	(5,235) : 25,	(6,235) : 28,	(7,235) : 89,	(8,235) : 829,	(9,235) : 264,	(10,235) : 91,	(11,235) : 56,	(12,235) : 603,	(13,235) : 223,	(14,235) : 259,	(15,235) : 485,	(16,235) : 701,	(17,235) : 591,	(18,235) : 6,	(19,235) : 540,	(20,235) : 95,	(21,235) : 343,	(22,235) : 74,	(23,235) : 0,	(24,235) : 594,	(25,235) : 691,	(26,235) : 286,	(27,235) : 228,	(28,235) : 93,	(29,235) : 497,	(30,235) : 97,	(31,235) : 31,	(32,235) : 128,	(33,235) : 210,	(34,235) : 38,	(35,235) : 160,	(36,235) : 303,	(37,235) : 361,	(38,235) : 38,	(39,235) : 52,	(40,235) : 704,
(1,236) : 580,	(2,236) : 624,	(3,236) : 664,	(4,236) : 28,	(5,236) : 643,	(6,236) : 594,	(7,236) : 574,	(8,236) : 948,	(9,236) : 728,	(10,236) : 576,	(11,236) : 624,	(12,236) : 268,	(13,236) : 409,	(14,236) : 724,	(15,236) : 136,	(16,236) : 143,	(17,236) : 31,	(18,236) : 625,	(19,236) : 276,	(20,236) : 575,	(21,236) : 279,	(22,236) : 599,	(23,236) : 619,	(24,236) : 28,	(25,236) : 939,	(26,236) : 347,	(27,236) : 403,	(28,236) : 574,	(29,236) : 147,	(30,236) : 577,	(31,236) : 598,	(32,236) : 500,	(33,236) : 509,	(34,236) : 584,	(35,236) : 707,	(36,236) : 790,	(37,236) : 732,	(38,236) : 623,	(39,236) : 624,	(40,236) : 966,
(1,237) : 707,	(2,237) : 758,	(3,237) : 896,	(4,237) : 206,	(5,237) : 790,	(6,237) : 741,	(7,237) : 704,	(8,237) : 1180,	(9,237) : 924,	(10,237) : 705,	(11,237) : 759,	(12,237) : 207,	(13,237) : 546,	(14,237) : 920,	(15,237) : 314,	(16,237) : 90,	(17,237) : 238,	(18,237) : 774,	(19,237) : 263,	(20,237) : 703,	(21,237) : 460,	(22,237) : 766,	(23,237) : 769,	(24,237) : 237,	(25,237) : 1168,	(26,237) : 483,	(27,237) : 541,	(28,237) : 703,	(29,237) : 368,	(30,237) : 704,	(31,237) : 742,	(32,237) : 641,	(33,237) : 608,	(34,237) : 738,	(35,237) : 819,	(36,237) : 985,	(37,237) : 943,	(38,237) : 762,	(39,237) : 760,	(40,237) : 1195,
(1,238) : 276,	(2,238) : 325,	(3,238) : 638,	(4,238) : 327,	(5,238) : 354,	(6,238) : 306,	(7,238) : 272,	(8,238) : 923,	(9,238) : 515,	(10,238) : 273,	(11,238) : 326,	(12,238) : 280,	(13,238) : 111,	(14,238) : 511,	(15,238) : 177,	(16,238) : 370,	(17,238) : 280,	(18,238) : 339,	(19,238) : 222,	(20,238) : 272,	(21,238) : 117,	(22,238) : 336,	(23,238) : 333,	(24,238) : 282,	(25,238) : 848,	(26,238) : 49,	(27,238) : 106,	(28,238) : 271,	(29,238) : 233,	(30,238) : 273,	(31,238) : 307,	(32,238) : 206,	(33,238) : 199,	(34,238) : 303,	(35,238) : 399,	(36,238) : 574,	(37,238) : 562,	(38,238) : 328,	(39,238) : 326,	(40,238) : 870,
(1,239) : 527,	(2,239) : 572,	(3,239) : 668,	(4,239) : 74,	(5,239) : 594,	(6,239) : 545,	(7,239) : 522,	(8,239) : 955,	(9,239) : 696,	(10,239) : 523,	(11,239) : 573,	(12,239) : 223,	(13,239) : 356,	(14,239) : 692,	(15,239) : 87,	(16,239) : 149,	(17,239) : 31,	(18,239) : 577,	(19,239) : 222,	(20,239) : 522,	(21,239) : 237,	(22,239) : 555,	(23,239) : 571,	(24,239) : 33,	(25,239) : 933,	(26,239) : 293,	(27,239) : 350,	(28,239) : 521,	(29,239) : 133,	(30,239) : 524,	(31,239) : 548,	(32,239) : 449,	(33,239) : 451,	(34,239) : 536,	(35,239) : 652,	(36,239) : 758,	(37,239) : 709,	(38,239) : 572,	(39,239) : 573,	(40,239) : 960,
(1,240) : 162,	(2,240) : 137,	(3,240) : 772,	(4,240) : 757,	(5,240) : 161,	(6,240) : 187,	(7,240) : 170,	(8,240) : 993,	(9,240) : 404,	(10,240) : 169,	(11,240) : 136,	(12,240) : 659,	(13,240) : 331,	(14,240) : 401,	(15,240) : 603,	(16,240) : 796,	(17,240) : 709,	(18,240) : 177,	(19,240) : 596,	(20,240) : 168,	(21,240) : 479,	(22,240) : 251,	(23,240) : 180,	(24,240) : 711,	(25,240) : 841,	(26,240) : 392,	(27,240) : 337,	(28,240) : 170,	(29,240) : 635,	(30,240) : 165,	(31,240) : 177,	(32,240) : 250,	(33,240) : 241,	(34,240) : 211,	(35,240) : 32,	(36,240) : 424,	(37,240) : 514,	(38,240) : 149,	(39,240) : 139,	(40,240) : 850,
(1,241) : 534,	(2,241) : 578,	(3,241) : 646,	(4,241) : 68,	(5,241) : 598,	(6,241) : 549,	(7,241) : 529,	(8,241) : 933,	(9,241) : 689,	(10,241) : 530,	(11,241) : 579,	(12,241) : 251,	(13,241) : 363,	(14,241) : 685,	(15,241) : 90,	(16,241) : 164,	(17,241) : 18,	(18,241) : 580,	(19,241) : 249,	(20,241) : 529,	(21,241) : 235,	(22,241) : 555,	(23,241) : 574,	(24,241) : 20,	(25,241) : 915,	(26,241) : 301,	(27,241) : 357,	(28,241) : 528,	(29,241) : 115,	(30,241) : 531,	(31,241) : 553,	(32,241) : 454,	(33,241) : 463,	(34,241) : 539,	(35,241) : 661,	(36,241) : 751,	(37,241) : 698,	(38,241) : 577,	(39,241) : 578,	(40,241) : 942,
(1,242) : 553,	(2,242) : 598,	(3,242) : 666,	(4,242) : 47,	(5,242) : 618,	(6,242) : 569,	(7,242) : 548,	(8,242) : 952,	(9,242) : 712,	(10,242) : 549,	(11,242) : 599,	(12,242) : 244,	(13,242) : 382,	(14,242) : 708,	(15,242) : 110,	(16,242) : 142,	(17,242) : 6,	(18,242) : 601,	(19,242) : 248,	(20,242) : 548,	(21,242) : 257,	(22,242) : 577,	(23,242) : 595,	(24,242) : 6,	(25,242) : 936,	(26,242) : 319,	(27,242) : 376,	(28,242) : 547,	(29,242) : 137,	(30,242) : 550,	(31,242) : 573,	(32,242) : 474,	(33,242) : 480,	(34,242) : 560,	(35,242) : 680,	(36,242) : 774,	(37,242) : 720,	(38,242) : 597,	(39,242) : 598,	(40,242) : 963,
(1,243) : 41,	(2,243) : 52,	(3,243) : 619,	(4,243) : 593,	(5,243) : 81,	(6,243) : 40,	(7,243) : 30,	(8,243) : 866,	(9,243) : 320,	(10,243) : 33,	(11,243) : 53,	(12,243) : 540,	(13,243) : 166,	(14,243) : 315,	(15,243) : 436,	(16,243) : 646,	(17,243) : 543,	(18,243) : 70,	(19,243) : 476,	(20,243) : 37,	(21,243) : 301,	(22,243) : 116,	(23,243) : 66,	(24,243) : 545,	(25,243) : 738,	(26,243) : 230,	(27,243) : 172,	(28,243) : 34,	(29,243) : 457,	(30,243) : 40,	(31,243) : 35,	(32,243) : 72,	(33,243) : 145,	(34,243) : 57,	(35,243) : 157,	(36,243) : 363,	(37,243) : 410,	(38,243) : 52,	(39,243) : 52,	(40,243) : 752,
(1,244) : 96,	(2,244) : 148,	(3,244) : 648,	(4,244) : 506,	(5,244) : 187,	(6,244) : 143,	(7,244) : 94,	(8,244) : 912,	(9,244) : 406,	(10,244) : 94,	(11,244) : 149,	(12,244) : 435,	(13,244) : 82,	(14,244) : 401,	(15,244) : 352,	(16,244) : 550,	(17,244) : 458,	(18,244) : 175,	(19,244) : 372,	(20,244) : 92,	(21,244) : 235,	(22,244) : 202,	(23,244) : 170,	(24,244) : 460,	(25,244) : 802,	(26,244) : 141,	(27,244) : 88,	(28,244) : 92,	(29,244) : 389,	(30,244) : 93,	(31,244) : 140,	(32,244) : 59,	(33,244) : 64,	(34,244) : 150,	(35,244) : 220,	(36,244) : 455,	(37,244) : 483,	(38,244) : 154,	(39,244) : 150,	(40,244) : 819,
(1,245) : 242,	(2,245) : 289,	(3,245) : 612,	(4,245) : 358,	(5,245) : 316,	(6,245) : 267,	(7,245) : 238,	(8,245) : 895,	(9,245) : 475,	(10,245) : 239,	(11,245) : 290,	(12,245) : 321,	(13,245) : 73,	(14,245) : 470,	(15,245) : 203,	(16,245) : 407,	(17,245) : 309,	(18,245) : 300,	(19,245) : 263,	(20,245) : 237,	(21,245) : 108,	(22,245) : 295,	(23,245) : 294,	(24,245) : 311,	(25,245) : 815,	(26,245) : 9,	(27,245) : 67,	(28,245) : 237,	(29,245) : 247,	(30,245) : 239,	(31,245) : 269,	(32,245) : 168,	(33,245) : 178,	(34,245) : 263,	(35,245) : 369,	(36,245) : 533,	(37,245) : 524,	(38,245) : 291,	(39,245) : 290,	(40,245) : 836,
(1,246) : 234,	(2,246) : 281,	(3,246) : 612,	(4,246) : 365,	(5,246) : 308,	(6,246) : 259,	(7,246) : 230,	(8,246) : 893,	(9,246) : 469,	(10,246) : 231,	(11,246) : 282,	(12,246) : 328,	(13,246) : 65,	(14,246) : 464,	(15,246) : 211,	(16,246) : 415,	(17,246) : 316,	(18,246) : 292,	(19,246) : 269,	(20,246) : 229,	(21,246) : 112,	(22,246) : 288,	(23,246) : 287,	(24,246) : 319,	(25,246) : 812,	(26,246) : 1,	(27,246) : 59,	(28,246) : 229,	(29,246) : 253,	(30,246) : 231,	(31,246) : 261,	(32,246) : 160,	(33,246) : 171,	(34,246) : 255,	(35,246) : 361,	(36,246) : 527,	(37,246) : 519,	(38,246) : 283,	(39,246) : 282,	(40,246) : 833,
(1,247) : 318,	(2,247) : 304,	(3,247) : 343,	(4,247) : 580,	(5,247) : 271,	(6,247) : 253,	(7,247) : 307,	(8,247) : 592,	(9,247) : 180,	(10,247) : 309,	(11,247) : 305,	(12,247) : 647,	(13,247) : 302,	(14,247) : 176,	(15,247) : 443,	(16,247) : 673,	(17,247) : 533,	(18,247) : 255,	(19,247) : 594,	(20,247) : 313,	(21,247) : 301,	(22,247) : 180,	(23,247) : 253,	(24,247) : 535,	(25,247) : 483,	(26,247) : 333,	(27,247) : 302,	(28,247) : 310,	(29,247) : 407,	(30,247) : 317,	(31,247) : 265,	(32,247) : 265,	(33,247) : 385,	(34,247) : 228,	(35,247) : 413,	(36,247) : 241,	(37,247) : 189,	(38,247) : 288,	(39,247) : 301,	(40,247) : 503,
(1,248) : 465,	(2,248) : 495,	(3,248) : 465,	(4,248) : 239,	(5,248) : 500,	(6,248) : 454,	(7,248) : 456,	(8,248) : 754,	(9,248) : 533,	(10,248) : 458,	(11,248) : 496,	(12,248) : 400,	(13,248) : 309,	(14,248) : 529,	(15,248) : 154,	(16,248) : 348,	(17,248) : 199,	(18,248) : 480,	(19,248) : 374,	(20,248) : 459,	(21,248) : 156,	(22,248) : 436,	(23,248) : 475,	(24,248) : 200,	(25,248) : 731,	(26,248) : 265,	(27,248) : 303,	(28,248) : 457,	(29,248) : 70,	(30,248) : 462,	(31,248) : 461,	(32,248) : 377,	(33,248) : 433,	(34,248) : 438,	(35,248) : 597,	(36,248) : 595,	(37,248) : 526,	(38,248) : 489,	(39,248) : 494,	(40,248) : 758,
(1,249) : 490,	(2,249) : 532,	(3,249) : 603,	(4,249) : 120,	(5,249) : 549,	(6,249) : 501,	(7,249) : 484,	(8,249) : 892,	(9,249) : 636,	(10,249) : 485,	(11,249) : 532,	(12,249) : 271,	(13,249) : 319,	(14,249) : 632,	(15,249) : 51,	(16,249) : 215,	(17,249) : 71,	(18,249) : 531,	(19,249) : 257,	(20,249) : 484,	(21,249) : 184,	(22,249) : 504,	(23,249) : 525,	(24,249) : 73,	(25,249) : 867,	(26,249) : 259,	(27,249) : 313,	(28,249) : 483,	(29,249) : 68,	(30,249) : 487,	(31,249) : 505,	(32,249) : 407,	(33,249) : 426,	(34,249) : 490,	(35,249) : 619,	(36,249) : 698,	(37,249) : 645,	(38,249) : 530,	(39,249) : 532,	(40,249) : 894,
(1,250) : 111,	(2,250) : 159,	(3,250) : 750,	(4,250) : 582,	(5,250) : 212,	(6,250) : 187,	(7,250) : 120,	(8,250) : 1008,	(9,250) : 469,	(10,250) : 118,	(11,250) : 159,	(12,250) : 465,	(13,250) : 183,	(14,250) : 464,	(15,250) : 435,	(16,250) : 609,	(17,250) : 536,	(18,250) : 210,	(19,250) : 402,	(20,250) : 114,	(21,250) : 336,	(22,250) : 265,	(23,250) : 207,	(24,250) : 539,	(25,250) : 886,	(26,250) : 230,	(27,250) : 189,	(28,250) : 116,	(29,250) : 483,	(30,250) : 111,	(31,250) : 178,	(32,250) : 154,	(33,250) : 59,	(34,250) : 207,	(35,250) : 165,	(36,250) : 510,	(37,250) : 559,	(38,250) : 174,	(39,250) : 163,	(40,250) : 901,
(1,251) : 247,	(2,251) : 213,	(3,251) : 456,	(4,251) : 666,	(5,251) : 166,	(6,251) : 171,	(7,251) : 237,	(8,251) : 674,	(9,251) : 116,	(10,251) : 240,	(11,251) : 214,	(12,251) : 686,	(13,251) : 306,	(14,251) : 111,	(15,251) : 517,	(16,251) : 746,	(17,251) : 617,	(18,251) : 157,	(19,251) : 627,	(20,251) : 243,	(21,251) : 366,	(22,251) : 92,	(23,251) : 158,	(24,251) : 619,	(25,251) : 534,	(26,251) : 357,	(27,251) : 308,	(28,251) : 241,	(29,251) : 500,	(30,251) : 247,	(31,251) : 181,	(32,251) : 232,	(33,251) : 345,	(34,251) : 150,	(35,251) : 308,	(36,251) : 169,	(37,251) : 204,	(38,251) : 195,	(39,251) : 209,	(40,251) : 547,
(1,252) : 319,	(2,252) : 342,	(3,252) : 433,	(4,252) : 371,	(5,252) : 341,	(6,252) : 298,	(7,252) : 310,	(8,252) : 719,	(9,252) : 384,	(10,252) : 312,	(11,252) : 343,	(12,252) : 449,	(13,252) : 192,	(14,252) : 380,	(15,252) : 233,	(16,252) : 463,	(17,252) : 324,	(18,252) : 321,	(19,252) : 402,	(20,252) : 313,	(21,252) : 100,	(22,252) : 274,	(23,252) : 316,	(24,252) : 325,	(25,252) : 654,	(26,252) : 180,	(27,252) : 188,	(28,252) : 311,	(29,252) : 202,	(30,252) : 317,	(31,252) : 305,	(32,252) : 232,	(33,252) : 317,	(34,252) : 279,	(35,252) : 449,	(36,252) : 446,	(37,252) : 397,	(38,252) : 334,	(39,252) : 341,	(40,252) : 678,
(1,253) : 572,	(2,253) : 627,	(3,253) : 919,	(4,253) : 304,	(5,253) : 668,	(6,253) : 623,	(7,253) : 573,	(8,253) : 1210,	(9,253) : 851,	(10,253) : 572,	(11,253) : 628,	(12,253) : 61,	(13,253) : 437,	(14,253) : 846,	(15,253) : 307,	(16,253) : 224,	(17,253) : 300,	(18,253) : 655,	(19,253) : 113,	(20,253) : 570,	(21,253) : 410,	(22,253) : 666,	(23,253) : 650,	(24,253) : 302,	(25,253) : 1163,	(26,253) : 383,	(27,253) : 433,	(28,253) : 570,	(29,253) : 393,	(30,253) : 570,	(31,253) : 620,	(32,253) : 526,	(33,253) : 460,	(34,253) : 626,	(35,253) : 666,	(36,253) : 909,	(37,253) : 894,	(38,253) : 635,	(39,253) : 629,	(40,253) : 1187,
(1,254) : 363,	(2,254) : 325,	(3,254) : 404,	(4,254) : 748,	(5,254) : 274,	(6,254) : 288,	(7,254) : 354,	(8,254) : 584,	(9,254) : 14,	(10,254) : 357,	(11,254) : 325,	(12,254) : 793,	(13,254) : 420,	(14,254) : 16,	(15,254) : 607,	(16,254) : 838,	(17,254) : 701,	(18,254) : 269,	(19,254) : 736,	(20,254) : 360,	(21,254) : 459,	(22,254) : 210,	(23,254) : 270,	(24,254) : 702,	(25,254) : 427,	(26,254) : 466,	(27,254) : 422,	(28,254) : 358,	(29,254) : 576,	(30,254) : 364,	(31,254) : 297,	(32,254) : 350,	(33,254) : 464,	(34,254) : 268,	(35,254) : 408,	(36,254) : 71,	(37,254) : 101,	(38,254) : 307,	(39,254) : 321,	(40,254) : 437,
(1,255) : 35,	(2,255) : 31,	(3,255) : 635,	(4,255) : 613,	(5,255) : 70,	(6,255) : 42,	(7,255) : 29,	(8,255) : 878,	(9,255) : 323,	(10,255) : 30,	(11,255) : 32,	(12,255) : 554,	(13,255) : 185,	(14,255) : 319,	(15,255) : 457,	(16,255) : 665,	(17,255) : 563,	(18,255) : 64,	(19,255) : 491,	(20,255) : 34,	(21,255) : 322,	(22,255) : 123,	(23,255) : 61,	(24,255) : 566,	(25,255) : 746,	(26,255) : 249,	(27,255) : 191,	(28,255) : 32,	(29,255) : 479,	(30,255) : 36,	(31,255) : 32,	(32,255) : 93,	(33,255) : 150,	(34,255) : 66,	(35,255) : 136,	(36,255) : 364,	(37,255) : 417,	(38,255) : 35,	(39,255) : 32,	(40,255) : 760,
(1,256) : 487,	(2,256) : 525,	(3,256) : 560,	(4,256) : 149,	(5,256) : 539,	(6,256) : 491,	(7,256) : 480,	(8,256) : 848,	(9,256) : 608,	(10,256) : 481,	(11,256) : 526,	(12,256) : 315,	(13,256) : 318,	(14,256) : 604,	(15,256) : 79,	(16,256) : 254,	(17,256) : 106,	(18,256) : 520,	(19,256) : 297,	(20,256) : 481,	(21,256) : 172,	(22,256) : 487,	(23,256) : 514,	(24,256) : 107,	(25,256) : 826,	(26,256) : 262,	(27,256) : 312,	(28,256) : 480,	(29,256) : 26,	(30,256) : 483,	(31,256) : 496,	(32,256) : 401,	(33,256) : 432,	(34,256) : 478,	(35,256) : 617,	(36,256) : 670,	(37,256) : 611,	(38,256) : 522,	(39,256) : 525,	(40,256) : 852,
(1,257) : 536,	(2,257) : 580,	(3,257) : 649,	(4,257) : 66,	(5,257) : 600,	(6,257) : 551,	(7,257) : 530,	(8,257) : 936,	(9,257) : 692,	(10,257) : 531,	(11,257) : 581,	(12,257) : 249,	(13,257) : 365,	(14,257) : 688,	(15,257) : 92,	(16,257) : 161,	(17,257) : 15,	(18,257) : 582,	(19,257) : 247,	(20,257) : 531,	(21,257) : 238,	(22,257) : 557,	(23,257) : 576,	(24,257) : 18,	(25,257) : 918,	(26,257) : 302,	(27,257) : 359,	(28,257) : 530,	(29,257) : 118,	(30,257) : 533,	(31,257) : 554,	(32,257) : 456,	(33,257) : 465,	(34,257) : 541,	(35,257) : 663,	(36,257) : 754,	(37,257) : 700,	(38,257) : 579,	(39,257) : 580,	(40,257) : 945,
(1,258) : 355,	(2,258) : 407,	(3,258) : 703,	(4,258) : 274,	(5,258) : 440,	(6,258) : 393,	(7,258) : 353,	(8,258) : 991,	(9,258) : 606,	(10,258) : 353,	(11,258) : 408,	(12,258) : 189,	(13,258) : 199,	(14,258) : 601,	(15,258) : 151,	(16,258) : 295,	(17,258) : 233,	(18,258) : 426,	(19,258) : 132,	(20,258) : 351,	(21,258) : 176,	(22,258) : 427,	(23,258) : 420,	(24,258) : 235,	(25,258) : 928,	(26,258) : 140,	(27,258) : 195,	(28,258) : 351,	(29,258) : 233,	(30,258) : 352,	(31,258) : 393,	(32,258) : 293,	(33,258) : 261,	(34,258) : 392,	(35,258) : 470,	(36,258) : 665,	(37,258) : 650,	(38,258) : 411,	(39,258) : 408,	(40,258) : 951,
(1,259) : 26,	(2,259) : 62,	(3,259) : 634,	(4,259) : 583,	(5,259) : 100,	(6,259) : 62,	(7,259) : 15,	(8,259) : 883,	(9,259) : 342,	(10,259) : 18,	(11,259) : 63,	(12,259) : 523,	(13,259) : 155,	(14,259) : 337,	(15,259) : 427,	(16,259) : 634,	(17,259) : 534,	(18,259) : 91,	(19,259) : 459,	(20,259) : 20,	(21,259) : 296,	(22,259) : 137,	(23,259) : 87,	(24,259) : 536,	(25,259) : 757,	(26,259) : 219,	(27,259) : 161,	(28,259) : 18,	(29,259) : 452,	(30,259) : 24,	(31,259) : 55,	(32,259) : 66,	(33,259) : 123,	(34,259) : 79,	(35,259) : 155,	(36,259) : 385,	(37,259) : 431,	(38,259) : 67,	(39,259) : 63,	(40,259) : 772,
(1,260) : 113,	(2,260) : 152,	(3,260) : 766,	(4,260) : 617,	(5,260) : 204,	(6,260) : 188,	(7,260) : 123,	(8,260) : 1018,	(9,260) : 468,	(10,260) : 121,	(11,260) : 151,	(12,260) : 497,	(13,260) : 214,	(14,260) : 463,	(15,260) : 469,	(16,260) : 643,	(17,260) : 570,	(18,260) : 206,	(19,260) : 434,	(20,260) : 117,	(21,260) : 367,	(22,260) : 268,	(23,260) : 204,	(24,260) : 573,	(25,260) : 891,	(26,260) : 263,	(27,260) : 220,	(28,260) : 119,	(29,260) : 516,	(30,260) : 113,	(31,260) : 177,	(32,260) : 173,	(33,260) : 93,	(34,260) : 210,	(35,260) : 137,	(36,260) : 505,	(37,260) : 563,	(38,260) : 168,	(39,260) : 155,	(40,260) : 905,
(1,261) : 24,	(2,261) : 32,	(3,261) : 657,	(4,261) : 621,	(5,261) : 82,	(6,261) : 64,	(7,261) : 27,	(8,261) : 900,	(9,261) : 342,	(10,261) : 26,	(11,261) : 32,	(12,261) : 553,	(13,261) : 192,	(14,261) : 338,	(15,261) : 465,	(16,261) : 669,	(17,261) : 572,	(18,261) : 81,	(19,261) : 489,	(20,261) : 28,	(21,261) : 335,	(22,261) : 145,	(23,261) : 79,	(24,261) : 574,	(25,261) : 767,	(26,261) : 256,	(27,261) : 198,	(28,261) : 28,	(29,261) : 491,	(30,261) : 27,	(31,261) : 53,	(32,261) : 105,	(33,261) : 141,	(34,261) : 88,	(35,261) : 117,	(36,261) : 381,	(37,261) : 438,	(38,261) : 45,	(39,261) : 35,	(40,261) : 780,
(1,262) : 618,	(2,262) : 668,	(3,262) : 801,	(4,262) : 120,	(5,262) : 697,	(6,262) : 648,	(7,262) : 614,	(8,262) : 1087,	(9,262) : 823,	(10,262) : 615,	(11,262) : 668,	(12,262) : 168,	(13,262) : 453,	(14,262) : 818,	(15,262) : 212,	(16,262) : 32,	(17,262) : 141,	(18,262) : 680,	(19,262) : 206,	(20,262) : 613,	(21,262) : 359,	(22,262) : 669,	(23,262) : 675,	(24,262) : 141,	(25,262) : 1069,	(26,262) : 389,	(27,262) : 448,	(28,262) : 613,	(29,262) : 268,	(30,262) : 615,	(31,262) : 649,	(32,262) : 548,	(33,262) : 525,	(34,262) : 643,	(35,262) : 734,	(36,262) : 884,	(37,262) : 841,	(38,262) : 670,	(39,262) : 669,	(40,262) : 1096,
(1,263) : 46,	(2,263) : 100,	(3,263) : 657,	(4,263) : 557,	(5,263) : 143,	(6,263) : 105,	(7,263) : 45,	(8,263) : 913,	(9,263) : 383,	(10,263) : 45,	(11,263) : 100,	(12,263) : 483,	(13,263) : 130,	(14,263) : 378,	(15,263) : 402,	(16,263) : 601,	(17,263) : 508,	(18,263) : 135,	(19,263) : 419,	(20,263) : 42,	(21,263) : 280,	(22,263) : 177,	(23,263) : 131,	(24,263) : 511,	(25,263) : 793,	(26,263) : 191,	(27,263) : 136,	(28,263) : 43,	(29,263) : 435,	(30,263) : 43,	(31,263) : 99,	(32,263) : 64,	(33,263) : 79,	(34,263) : 120,	(35,263) : 170,	(36,263) : 428,	(37,263) : 468,	(38,263) : 108,	(39,263) : 102,	(40,263) : 808,
(1,264) : 844,	(2,264) : 899,	(3,264) : 1136,	(4,264) : 452,	(5,264) : 940,	(6,264) : 894,	(7,264) : 844,	(8,264) : 1422,	(9,264) : 1116,	(10,264) : 844,	(11,264) : 899,	(12,264) : 322,	(13,264) : 707,	(14,264) : 1111,	(15,264) : 531,	(16,264) : 335,	(17,264) : 477,	(18,264) : 927,	(19,264) : 383,	(20,264) : 841,	(21,264) : 661,	(22,264) : 936,	(23,264) : 922,	(24,264) : 477,	(25,264) : 1399,	(26,264) : 651,	(27,264) : 703,	(28,264) : 842,	(29,264) : 601,	(30,264) : 841,	(31,264) : 892,	(32,264) : 797,	(33,264) : 731,	(34,264) : 897,	(35,264) : 934,	(36,264) : 1175,	(37,264) : 1151,	(38,264) : 907,	(39,264) : 901,	(40,264) : 1425,
(1,265) : 46,	(2,265) : 44,	(3,265) : 617,	(4,265) : 603,	(5,265) : 70,	(6,265) : 30,	(7,265) : 37,	(8,265) : 862,	(9,265) : 312,	(10,265) : 39,	(11,265) : 45,	(12,265) : 551,	(13,265) : 177,	(14,265) : 307,	(15,265) : 446,	(16,265) : 657,	(17,265) : 553,	(18,265) : 59,	(19,265) : 488,	(20,265) : 43,	(21,265) : 310,	(22,265) : 109,	(23,265) : 55,	(24,265) : 555,	(25,265) : 732,	(26,265) : 241,	(27,265) : 182,	(28,265) : 41,	(29,265) : 466,	(30,265) : 46,	(31,265) : 24,	(32,265) : 82,	(33,265) : 155,	(34,265) : 51,	(35,265) : 153,	(36,265) : 354,	(37,265) : 404,	(38,265) : 41,	(39,265) : 43,	(40,265) : 746,
(1,266) : 531,	(2,266) : 575,	(3,266) : 651,	(4,266) : 69,	(5,266) : 596,	(6,266) : 547,	(7,266) : 526,	(8,266) : 938,	(9,266) : 690,	(10,266) : 527,	(11,266) : 576,	(12,266) : 244,	(13,266) : 360,	(14,266) : 686,	(15,266) : 88,	(16,266) : 161,	(17,266) : 19,	(18,266) : 578,	(19,266) : 242,	(20,266) : 526,	(21,266) : 235,	(22,266) : 554,	(23,266) : 572,	(24,266) : 22,	(25,266) : 919,	(26,266) : 298,	(27,266) : 354,	(28,266) : 525,	(29,266) : 119,	(30,266) : 528,	(31,266) : 550,	(32,266) : 452,	(33,266) : 459,	(34,266) : 537,	(35,266) : 658,	(36,266) : 752,	(37,266) : 700,	(38,266) : 575,	(39,266) : 576,	(40,266) : 946,
(1,267) : 250,	(2,267) : 223,	(3,267) : 432,	(4,267) : 634,	(5,267) : 181,	(6,267) : 176,	(7,267) : 239,	(8,267) : 661,	(9,267) : 133,	(10,267) : 242,	(11,267) : 224,	(12,267) : 662,	(13,267) : 287,	(14,267) : 129,	(15,267) : 486,	(16,267) : 717,	(17,267) : 585,	(18,267) : 169,	(19,267) : 604,	(20,267) : 245,	(21,267) : 336,	(22,267) : 96,	(23,267) : 168,	(24,267) : 587,	(25,267) : 529,	(26,267) : 334,	(27,267) : 289,	(28,267) : 243,	(29,267) : 467,	(30,267) : 249,	(31,267) : 187,	(32,267) : 222,	(33,267) : 339,	(34,267) : 153,	(35,267) : 325,	(36,267) : 192,	(37,267) : 203,	(38,267) : 206,	(39,267) : 219,	(40,267) : 544,
(1,268) : 232,	(2,268) : 229,	(3,268) : 428,	(4,268) : 531,	(5,268) : 208,	(6,268) : 178,	(7,268) : 221,	(8,268) : 686,	(9,268) : 231,	(10,268) : 224,	(11,268) : 230,	(12,268) : 567,	(13,268) : 208,	(14,268) : 226,	(15,268) : 384,	(16,268) : 614,	(17,268) : 482,	(18,268) : 189,	(19,268) : 511,	(20,268) : 226,	(21,268) : 234,	(22,268) : 123,	(23,268) : 186,	(24,268) : 484,	(25,268) : 579,	(26,268) : 244,	(27,268) : 208,	(28,268) : 224,	(29,268) : 365,	(30,268) : 231,	(31,268) : 189,	(32,268) : 171,	(33,268) : 290,	(34,268) : 153,	(35,268) : 341,	(36,268) : 292,	(37,268) : 274,	(38,268) : 215,	(39,268) : 226,	(40,268) : 598,
(1,269) : 134,	(2,269) : 86,	(3,269) : 585,	(4,269) : 673,	(5,269) : 34,	(6,269) : 67,	(7,269) : 128,	(8,269) : 809,	(9,269) : 233,	(10,269) : 130,	(11,269) : 86,	(12,269) : 641,	(13,269) : 259,	(14,269) : 228,	(15,269) : 517,	(16,269) : 736,	(17,269) : 623,	(18,269) : 36,	(19,269) : 578,	(20,269) : 133,	(21,269) : 372,	(22,269) : 70,	(23,269) : 40,	(24,269) : 625,	(25,269) : 665,	(26,269) : 322,	(27,269) : 264,	(28,269) : 132,	(29,269) : 524,	(30,269) : 135,	(31,269) : 71,	(32,269) : 165,	(33,269) : 249,	(34,269) : 67,	(35,269) : 173,	(36,269) : 267,	(37,269) : 335,	(38,269) : 70,	(39,269) : 82,	(40,269) : 677,
(1,270) : 375,	(2,270) : 331,	(3,270) : 438,	(4,270) : 788,	(5,270) : 279,	(6,270) : 300,	(7,270) : 367,	(8,270) : 602,	(9,270) : 33,	(10,270) : 369,	(11,270) : 332,	(12,270) : 824,	(13,270) : 447,	(14,270) : 37,	(15,270) : 644,	(16,270) : 875,	(17,270) : 740,	(18,270) : 277,	(19,270) : 766,	(20,270) : 372,	(21,270) : 496,	(22,270) : 227,	(23,270) : 279,	(24,270) : 741,	(25,270) : 435,	(26,270) : 496,	(27,270) : 449,	(28,270) : 371,	(29,270) : 617,	(30,270) : 375,	(31,270) : 308,	(32,270) : 371,	(33,270) : 481,	(34,270) : 283,	(35,270) : 405,	(36,270) : 30,	(37,270) : 125,	(38,270) : 315,	(39,270) : 328,	(40,270) : 443,
(1,271) : 59,	(2,271) : 63,	(3,271) : 601,	(4,271) : 588,	(5,271) : 80,	(6,271) : 32,	(7,271) : 48,	(8,271) : 847,	(9,271) : 305,	(10,271) : 51,	(11,271) : 64,	(12,271) : 543,	(13,271) : 164,	(14,271) : 300,	(15,271) : 431,	(16,271) : 644,	(17,271) : 538,	(18,271) : 65,	(19,271) : 480,	(20,271) : 54,	(21,271) : 293,	(22,271) : 100,	(23,271) : 60,	(24,271) : 540,	(25,271) : 720,	(26,271) : 228,	(27,271) : 169,	(28,271) : 52,	(29,271) : 449,	(30,271) : 59,	(31,271) : 32,	(32,271) : 69,	(33,271) : 157,	(34,271) : 42,	(35,271) : 172,	(36,271) : 350,	(37,271) : 394,	(38,271) : 57,	(39,271) : 62,	(40,271) : 735,
(1,272) : 78,	(2,272) : 48,	(3,272) : 599,	(4,272) : 623,	(5,272) : 42,	(6,272) : 7,	(7,272) : 70,	(8,272) : 837,	(9,272) : 279,	(10,272) : 72,	(11,272) : 50,	(12,272) : 581,	(13,272) : 202,	(14,272) : 275,	(15,272) : 467,	(16,272) : 682,	(17,272) : 573,	(18,272) : 26,	(19,272) : 518,	(20,272) : 76,	(21,272) : 326,	(22,272) : 81,	(23,272) : 21,	(24,272) : 576,	(25,272) : 703,	(26,272) : 266,	(27,272) : 207,	(28,272) : 74,	(29,272) : 481,	(30,272) : 79,	(31,272) : 12,	(32,272) : 107,	(33,272) : 189,	(34,272) : 29,	(35,272) : 160,	(36,272) : 320,	(37,272) : 374,	(38,272) : 34,	(39,272) : 45,	(40,272) : 716,
(1,273) : 66,	(2,273) : 74,	(3,273) : 593,	(4,273) : 578,	(5,273) : 88,	(6,273) : 40,	(7,273) : 54,	(8,273) : 842,	(9,273) : 305,	(10,273) : 57,	(11,273) : 75,	(12,273) : 536,	(13,273) : 156,	(14,273) : 300,	(15,273) : 421,	(16,273) : 635,	(17,273) : 528,	(18,273) : 73,	(19,273) : 473,	(20,273) : 60,	(21,273) : 282,	(22,273) : 99,	(23,273) : 67,	(24,273) : 530,	(25,273) : 717,	(26,273) : 219,	(27,273) : 161,	(28,273) : 58,	(29,273) : 438,	(30,273) : 64,	(31,273) : 42,	(32,273) : 60,	(33,273) : 156,	(34,273) : 44,	(35,273) : 183,	(36,273) : 351,	(37,273) : 391,	(38,273) : 68,	(39,273) : 73,	(40,273) : 732,
(1,274) : 85,	(2,274) : 115,	(3,274) : 589,	(4,274) : 531,	(5,274) : 136,	(6,274) : 87,	(7,274) : 76,	(8,274) : 847,	(9,274) : 334,	(10,274) : 78,	(11,274) : 116,	(12,274) : 491,	(13,274) : 108,	(14,274) : 329,	(15,274) : 375,	(16,274) : 588,	(17,274) : 481,	(18,274) : 120,	(19,274) : 428,	(20,274) : 79,	(21,274) : 238,	(22,274) : 131,	(23,274) : 114,	(24,274) : 484,	(25,274) : 731,	(26,274) : 172,	(27,274) : 113,	(28,274) : 77,	(29,274) : 395,	(30,274) : 83,	(31,274) : 89,	(32,274) : 15,	(33,274) : 133,	(34,274) : 85,	(35,274) : 216,	(36,274) : 384,	(37,274) : 411,	(38,274) : 113,	(39,274) : 115,	(40,274) : 748,
(1,275) : 295,	(2,275) : 262,	(3,275) : 412,	(4,275) : 675,	(5,275) : 216,	(6,275) : 220,	(7,275) : 285,	(8,275) : 625,	(9,275) : 83,	(10,275) : 287,	(11,275) : 263,	(12,275) : 712,	(13,275) : 339,	(14,275) : 78,	(15,275) : 530,	(16,275) : 761,	(17,275) : 627,	(18,275) : 207,	(19,275) : 655,	(20,275) : 291,	(21,275) : 382,	(22,275) : 139,	(23,275) : 207,	(24,275) : 629,	(25,275) : 484,	(26,275) : 385,	(27,275) : 341,	(28,275) : 289,	(29,275) : 506,	(30,275) : 295,	(31,275) : 230,	(32,275) : 273,	(33,275) : 389,	(34,275) : 197,	(35,275) : 358,	(36,275) : 144,	(37,275) : 156,	(38,275) : 245,	(39,275) : 259,	(40,275) : 498,
(1,276) : 495,	(2,276) : 549,	(3,276) : 838,	(4,276) : 264,	(5,276) : 588,	(6,276) : 542,	(7,276) : 494,	(8,276) : 1128,	(9,276) : 765,	(10,276) : 494,	(11,276) : 550,	(12,276) : 34,	(13,276) : 353,	(14,276) : 760,	(15,276) : 231,	(16,276) : 216,	(17,276) : 247,	(18,276) : 575,	(19,276) : 30,	(20,276) : 492,	(21,276) : 324,	(22,276) : 582,	(23,276) : 569,	(24,276) : 249,	(25,276) : 1077,	(26,276) : 298,	(27,276) : 349,	(28,276) : 492,	(29,276) : 320,	(30,276) : 492,	(31,276) : 540,	(32,276) : 443,	(33,276) : 387,	(34,276) : 543,	(35,276) : 596,	(36,276) : 823,	(37,276) : 807,	(38,276) : 556,	(39,276) : 551,	(40,276) : 1101,
(1,277) : 223,	(2,277) : 258,	(3,277) : 537,	(4,277) : 394,	(5,277) : 272,	(6,277) : 224,	(7,277) : 215,	(8,277) : 817,	(9,277) : 398,	(10,277) : 217,	(11,277) : 259,	(12,277) : 398,	(13,277) : 75,	(14,277) : 393,	(15,277) : 238,	(16,277) : 461,	(17,277) : 344,	(18,277) : 254,	(19,277) : 342,	(20,277) : 217,	(21,277) : 95,	(22,277) : 232,	(23,277) : 248,	(24,277) : 346,	(25,277) : 732,	(26,277) : 78,	(27,277) : 71,	(28,277) : 215,	(29,277) : 252,	(30,277) : 220,	(31,277) : 228,	(32,277) : 135,	(33,277) : 201,	(34,277) : 213,	(35,277) : 355,	(36,277) : 457,	(37,277) : 442,	(38,277) : 255,	(39,277) : 258,	(40,277) : 754,
(1,278) : 537,	(2,278) : 577,	(3,278) : 584,	(4,278) : 109,	(5,278) : 591,	(6,278) : 543,	(7,278) : 531,	(8,278) : 868,	(9,278) : 656,	(10,278) : 532,	(11,278) : 578,	(12,278) : 317,	(13,278) : 368,	(14,278) : 652,	(15,278) : 110,	(16,278) : 223,	(17,278) : 78,	(18,278) : 572,	(19,278) : 310,	(20,278) : 532,	(21,278) : 224,	(22,278) : 539,	(23,278) : 566,	(24,278) : 78,	(25,278) : 857,	(26,278) : 310,	(27,278) : 362,	(28,278) : 531,	(29,278) : 73,	(30,278) : 534,	(31,278) : 548,	(32,278) : 453,	(33,278) : 480,	(34,278) : 530,	(35,278) : 668,	(36,278) : 718,	(37,278) : 655,	(38,278) : 574,	(39,278) : 576,	(40,278) : 885,
(1,279) : 587,	(2,279) : 637,	(3,279) : 792,	(4,279) : 128,	(5,279) : 667,	(6,279) : 618,	(7,279) : 584,	(8,279) : 1080,	(9,279) : 799,	(10,279) : 584,	(11,279) : 638,	(12,279) : 140,	(13,279) : 423,	(14,279) : 795,	(15,279) : 192,	(16,279) : 65,	(17,279) : 137,	(18,279) : 651,	(19,279) : 174,	(20,279) : 583,	(21,279) : 335,	(22,279) : 641,	(23,279) : 646,	(24,279) : 137,	(25,279) : 1056,	(26,279) : 360,	(27,279) : 418,	(28,279) : 582,	(29,279) : 257,	(30,279) : 584,	(31,279) : 620,	(32,279) : 519,	(33,279) : 493,	(34,279) : 614,	(35,279) : 703,	(36,279) : 861,	(37,279) : 821,	(38,279) : 640,	(39,279) : 638,	(40,279) : 1082,
(1,280) : 501,	(2,280) : 538,	(3,280) : 555,	(4,280) : 146,	(5,280) : 551,	(6,280) : 503,	(7,280) : 493,	(8,280) : 843,	(9,280) : 615,	(10,280) : 495,	(11,280) : 539,	(12,280) : 325,	(13,280) : 333,	(14,280) : 611,	(15,280) : 93,	(16,280) : 254,	(17,280) : 106,	(18,280) : 532,	(19,280) : 309,	(20,280) : 495,	(21,280) : 185,	(22,280) : 498,	(23,280) : 527,	(24,280) : 107,	(25,280) : 824,	(26,280) : 277,	(27,280) : 327,	(28,280) : 493,	(29,280) : 31,	(30,280) : 497,	(31,280) : 508,	(32,280) : 415,	(33,280) : 448,	(34,280) : 490,	(35,280) : 631,	(36,280) : 677,	(37,280) : 615,	(38,280) : 535,	(39,280) : 538,	(40,280) : 851,
(1,281) : 194,	(2,281) : 219,	(3,281) : 509,	(4,281) : 449,	(5,281) : 224,	(6,281) : 178,	(7,281) : 184,	(8,281) : 781,	(9,281) : 338,	(10,281) : 187,	(11,281) : 220,	(12,281) : 457,	(13,281) : 99,	(14,281) : 333,	(15,281) : 294,	(16,281) : 520,	(17,281) : 399,	(18,281) : 205,	(19,281) : 400,	(20,281) : 188,	(21,281) : 146,	(22,281) : 174,	(23,281) : 200,	(24,281) : 401,	(25,281) : 687,	(26,281) : 131,	(27,281) : 98,	(28,281) : 186,	(29,281) : 298,	(30,281) : 192,	(31,281) : 184,	(32,281) : 107,	(33,281) : 206,	(34,281) : 162,	(35,281) : 324,	(36,281) : 397,	(37,281) : 387,	(38,281) : 212,	(39,281) : 218,	(40,281) : 706,
(1,282) : 572,	(2,282) : 615,	(3,282) : 652,	(4,282) : 40,	(5,282) : 634,	(6,282) : 585,	(7,282) : 567,	(8,282) : 936,	(9,282) : 717,	(10,282) : 568,	(11,282) : 616,	(12,282) : 273,	(13,282) : 402,	(14,282) : 713,	(15,282) : 128,	(16,282) : 155,	(17,282) : 29,	(18,282) : 616,	(19,282) : 279,	(20,282) : 567,	(21,282) : 269,	(22,282) : 589,	(23,282) : 610,	(24,282) : 26,	(25,282) : 926,	(26,282) : 340,	(27,282) : 395,	(28,282) : 566,	(29,282) : 135,	(30,282) : 569,	(31,282) : 589,	(32,282) : 491,	(33,282) : 503,	(34,282) : 575,	(35,282) : 700,	(36,282) : 779,	(37,282) : 720,	(38,282) : 614,	(39,282) : 616,	(40,282) : 954,
(1,283) : 283,	(2,283) : 254,	(3,283) : 408,	(4,283) : 652,	(5,283) : 210,	(6,283) : 209,	(7,283) : 273,	(8,283) : 630,	(9,283) : 106,	(10,283) : 275,	(11,283) : 255,	(12,283) : 690,	(13,283) : 319,	(14,283) : 102,	(15,283) : 507,	(16,283) : 738,	(17,283) : 603,	(18,283) : 200,	(19,283) : 633,	(20,283) : 279,	(21,283) : 358,	(22,283) : 128,	(23,283) : 199,	(24,283) : 605,	(25,283) : 495,	(26,283) : 363,	(27,283) : 320,	(28,283) : 276,	(29,283) : 482,	(30,283) : 282,	(31,283) : 220,	(32,283) : 255,	(33,283) : 373,	(34,283) : 185,	(35,283) : 354,	(36,283) : 167,	(37,283) : 169,	(38,283) : 237,	(39,283) : 251,	(40,283) : 510,
(1,284) : 491,	(2,284) : 528,	(3,284) : 539,	(4,284) : 163,	(5,284) : 539,	(6,284) : 492,	(7,284) : 484,	(8,284) : 826,	(9,284) : 598,	(10,284) : 485,	(11,284) : 529,	(12,284) : 337,	(13,284) : 325,	(14,284) : 594,	(15,284) : 99,	(16,284) : 272,	(17,284) : 123,	(18,284) : 520,	(19,284) : 319,	(20,284) : 485,	(21,284) : 175,	(22,284) : 484,	(23,284) : 515,	(24,284) : 124,	(25,284) : 807,	(26,284) : 271,	(27,284) : 319,	(28,284) : 484,	(29,284) : 18,	(30,284) : 488,	(31,284) : 497,	(32,284) : 405,	(33,284) : 442,	(34,284) : 478,	(35,284) : 622,	(36,284) : 660,	(37,284) : 598,	(38,284) : 524,	(39,284) : 527,	(40,284) : 834,
(1,285) : 909,	(2,285) : 886,	(3,285) : 290,	(4,285) : 973,	(5,285) : 843,	(6,285) : 839,	(7,285) : 898,	(8,285) : 5,	(9,285) : 599,	(10,285) : 901,	(11,285) : 887,	(12,285) : 1148,	(13,285) : 881,	(14,285) : 602,	(15,285) : 904,	(16,285) : 1090,	(17,285) : 945,	(18,285) : 832,	(19,285) : 1112,	(20,285) : 904,	(21,285) : 816,	(22,285) : 759,	(23,285) : 831,	(24,285) : 945,	(25,285) : 205,	(26,285) : 893,	(27,285) : 879,	(28,285) : 902,	(29,285) : 824,	(30,285) : 908,	(31,285) : 850,	(32,285) : 858,	(33,285) : 978,	(34,285) : 814,	(35,285) : 985,	(36,285) : 602,	(37,285) : 485,	(38,285) : 869,	(39,285) : 883,	(40,285) : 221,
(1,286) : 200,	(2,286) : 189,	(3,286) : 460,	(4,286) : 568,	(5,286) : 162,	(6,286) : 137,	(7,286) : 189,	(8,286) : 708,	(9,286) : 209,	(10,286) : 191,	(11,286) : 190,	(12,286) : 587,	(13,286) : 212,	(14,286) : 205,	(15,286) : 417,	(16,286) : 646,	(17,286) : 518,	(18,286) : 145,	(19,286) : 528,	(20,286) : 195,	(21,286) : 267,	(22,286) : 75,	(23,286) : 141,	(24,286) : 520,	(25,286) : 590,	(26,286) : 258,	(27,286) : 214,	(28,286) : 192,	(29,286) : 406,	(30,286) : 199,	(31,286) : 149,	(32,286) : 154,	(33,286) : 274,	(34,286) : 112,	(35,286) : 300,	(36,286) : 267,	(37,286) : 272,	(38,286) : 174,	(39,286) : 186,	(40,286) : 606,
(1,287) : 41,	(2,287) : 88,	(3,287) : 696,	(4,287) : 599,	(5,287) : 140,	(6,287) : 117,	(7,287) : 51,	(8,287) : 946,	(9,287) : 398,	(10,287) : 48,	(11,287) : 88,	(12,287) : 510,	(13,287) : 175,	(14,287) : 394,	(15,287) : 445,	(16,287) : 638,	(17,287) : 550,	(18,287) : 138,	(19,287) : 446,	(20,287) : 45,	(21,287) : 327,	(22,287) : 196,	(23,287) : 135,	(24,287) : 553,	(25,287) : 819,	(26,287) : 234,	(27,287) : 181,	(28,287) : 47,	(29,287) : 482,	(30,287) : 41,	(31,287) : 107,	(32,287) : 110,	(33,287) : 91,	(34,287) : 138,	(35,287) : 127,	(36,287) : 438,	(37,287) : 491,	(38,287) : 103,	(39,287) : 92,	(40,287) : 833,
(1,288) : 421,	(2,288) : 471,	(3,288) : 709,	(4,288) : 209,	(5,288) : 502,	(6,288) : 454,	(7,288) : 418,	(8,288) : 999,	(9,288) : 651,	(10,288) : 418,	(11,288) : 472,	(12,288) : 153,	(13,288) : 259,	(14,288) : 647,	(15,288) : 111,	(16,288) : 226,	(17,288) : 171,	(18,288) : 487,	(19,288) : 114,	(20,288) : 417,	(21,288) : 198,	(22,288) : 481,	(23,288) : 481,	(24,288) : 174,	(25,288) : 949,	(26,288) : 196,	(27,288) : 254,	(28,288) : 416,	(29,288) : 202,	(30,288) : 418,	(31,288) : 455,	(32,288) : 354,	(33,288) : 329,	(34,288) : 451,	(35,288) : 538,	(36,288) : 711,	(37,288) : 686,	(38,288) : 475,	(39,288) : 473,	(40,288) : 973,
(1,289) : 84,	(2,289) : 77,	(3,289) : 578,	(4,289) : 588,	(5,289) : 77,	(6,289) : 30,	(7,289) : 73,	(8,289) : 823,	(9,289) : 282,	(10,289) : 75,	(11,289) : 78,	(12,289) : 554,	(13,289) : 171,	(14,289) : 278,	(15,289) : 432,	(16,289) : 649,	(17,289) : 538,	(18,289) : 59,	(19,289) : 492,	(20,289) : 79,	(21,289) : 290,	(22,289) : 76,	(23,289) : 53,	(24,289) : 541,	(25,289) : 696,	(26,289) : 234,	(27,289) : 176,	(28,289) : 77,	(29,289) : 445,	(30,289) : 83,	(31,289) : 38,	(32,289) : 78,	(33,289) : 178,	(34,289) : 22,	(35,289) : 189,	(36,289) : 329,	(37,289) : 369,	(38,289) : 66,	(39,289) : 75,	(40,289) : 711,
(1,290) : 189,	(2,290) : 194,	(3,290) : 477,	(4,290) : 512,	(5,290) : 184,	(6,290) : 145,	(7,290) : 177,	(8,290) : 738,	(9,290) : 268,	(10,290) : 180,	(11,290) : 196,	(12,290) : 528,	(13,290) : 158,	(14,290) : 263,	(15,290) : 360,	(16,290) : 588,	(17,290) : 462,	(18,290) : 164,	(19,290) : 470,	(20,290) : 182,	(21,290) : 209,	(22,290) : 114,	(23,290) : 160,	(24,290) : 464,	(25,290) : 631,	(26,290) : 200,	(27,290) : 159,	(28,290) : 180,	(29,290) : 353,	(30,290) : 187,	(31,290) : 155,	(32,290) : 120,	(33,290) : 239,	(34,290) : 123,	(35,290) : 307,	(36,290) : 326,	(37,290) : 322,	(38,290) : 183,	(39,290) : 192,	(40,290) : 649,
(1,291) : 438,	(2,291) : 492,	(3,291) : 782,	(4,291) : 255,	(5,291) : 528,	(6,291) : 481,	(7,291) : 437,	(8,291) : 1072,	(9,291) : 701,	(10,291) : 437,	(11,291) : 492,	(12,291) : 95,	(13,291) : 290,	(14,291) : 696,	(15,291) : 186,	(16,291) : 237,	(17,291) : 227,	(18,291) : 515,	(19,291) : 40,	(20,291) : 435,	(21,291) : 263,	(22,291) : 520,	(23,291) : 509,	(24,291) : 229,	(25,291) : 1016,	(26,291) : 234,	(27,291) : 287,	(28,291) : 435,	(29,291) : 276,	(30,291) : 435,	(31,291) : 480,	(32,291) : 383,	(33,291) : 334,	(34,291) : 482,	(35,291) : 544,	(36,291) : 759,	(37,291) : 744,	(38,291) : 497,	(39,291) : 493,	(40,291) : 1040,
(1,292) : 494,	(2,292) : 542,	(3,292) : 710,	(4,292) : 131,	(5,292) : 570,	(6,292) : 521,	(7,292) : 490,	(8,292) : 1000,	(9,292) : 698,	(10,292) : 490,	(11,292) : 543,	(12,292) : 158,	(13,292) : 327,	(14,292) : 694,	(15,292) : 97,	(16,292) : 154,	(17,292) : 100,	(18,292) : 553,	(19,292) : 151,	(20,292) : 489,	(21,292) : 234,	(22,292) : 541,	(23,292) : 548,	(24,292) : 103,	(25,292) : 965,	(26,292) : 263,	(27,292) : 321,	(28,292) : 488,	(29,292) : 178,	(30,292) : 491,	(31,292) : 523,	(32,292) : 422,	(33,292) : 407,	(34,292) : 515,	(35,292) : 614,	(36,292) : 759,	(37,292) : 722,	(38,292) : 544,	(39,292) : 543,	(40,292) : 991,
(1,293) : 468,	(2,293) : 508,	(3,293) : 575,	(4,293) : 151,	(5,293) : 524,	(6,293) : 476,	(7,293) : 462,	(8,293) : 864,	(9,293) : 604,	(10,293) : 463,	(11,293) : 509,	(12,293) : 293,	(13,293) : 299,	(14,293) : 600,	(15,293) : 52,	(16,293) : 247,	(17,293) : 103,	(18,293) : 506,	(19,293) : 272,	(20,293) : 463,	(21,293) : 158,	(22,293) : 476,	(23,293) : 500,	(24,293) : 105,	(25,293) : 836,	(26,293) : 240,	(27,293) : 293,	(28,293) : 462,	(29,293) : 41,	(30,293) : 465,	(31,293) : 480,	(32,293) : 384,	(33,293) : 410,	(34,293) : 464,	(35,293) : 598,	(36,293) : 667,	(37,293) : 612,	(38,293) : 506,	(39,293) : 508,	(40,293) : 863,
(1,294) : 449,	(2,294) : 479,	(3,294) : 461,	(4,294) : 249,	(5,294) : 483,	(6,294) : 438,	(7,294) : 440,	(8,294) : 751,	(9,294) : 518,	(10,294) : 442,	(11,294) : 480,	(12,294) : 401,	(13,294) : 294,	(14,294) : 514,	(15,294) : 154,	(16,294) : 356,	(17,294) : 208,	(18,294) : 464,	(19,294) : 372,	(20,294) : 443,	(21,294) : 142,	(22,294) : 419,	(23,294) : 459,	(24,294) : 209,	(25,294) : 723,	(26,294) : 252,	(27,294) : 288,	(28,294) : 441,	(29,294) : 77,	(30,294) : 446,	(31,294) : 445,	(32,294) : 361,	(33,294) : 419,	(34,294) : 421,	(35,294) : 581,	(36,294) : 580,	(37,294) : 513,	(38,294) : 473,	(39,294) : 478,	(40,294) : 750,
(1,295) : 198,	(2,295) : 187,	(3,295) : 462,	(4,295) : 567,	(5,295) : 161,	(6,295) : 135,	(7,295) : 186,	(8,295) : 711,	(9,295) : 212,	(10,295) : 189,	(11,295) : 188,	(12,295) : 585,	(13,295) : 210,	(14,295) : 207,	(15,295) : 416,	(16,295) : 645,	(17,295) : 517,	(18,295) : 143,	(19,295) : 526,	(20,295) : 192,	(21,295) : 265,	(22,295) : 75,	(23,295) : 140,	(24,295) : 519,	(25,295) : 592,	(26,295) : 256,	(27,295) : 211,	(28,295) : 190,	(29,295) : 405,	(30,295) : 196,	(31,295) : 147,	(32,295) : 151,	(33,295) : 272,	(34,295) : 110,	(35,295) : 298,	(36,295) : 269,	(37,295) : 274,	(38,295) : 172,	(39,295) : 184,	(40,295) : 609,
(1,296) : 983,	(2,296) : 949,	(3,296) : 469,	(4,296) : 1158,	(5,296) : 900,	(6,296) : 908,	(7,296) : 973,	(8,296) : 229,	(9,296) : 634,	(10,296) : 976,	(11,296) : 950,	(12,296) : 1306,	(13,296) : 994,	(14,296) : 639,	(15,296) : 1069,	(16,296) : 1272,	(17,296) : 1125,	(18,296) : 893,	(19,296) : 1262,	(20,296) : 979,	(21,296) : 959,	(22,296) : 828,	(23,296) : 894,	(24,296) : 1125,	(25,296) : 209,	(26,296) : 1019,	(27,296) : 993,	(28,296) : 977,	(29,296) : 997,	(30,296) : 983,	(31,296) : 918,	(32,296) : 951,	(33,296) : 1071,	(34,296) : 886,	(35,296) : 1033,	(36,296) : 615,	(37,296) : 533,	(38,296) : 932,	(39,296) : 945,	(40,296) : 191,
(1,297) : 650,	(2,297) : 699,	(3,297) : 800,	(4,297) : 110,	(5,297) : 726,	(6,297) : 677,	(7,297) : 646,	(8,297) : 1083,	(9,297) : 842,	(10,297) : 647,	(11,297) : 700,	(12,297) : 208,	(13,297) : 483,	(14,297) : 838,	(15,297) : 230,	(16,297) : 12,	(17,297) : 144,	(18,297) : 709,	(19,297) : 246,	(20,297) : 645,	(21,297) : 380,	(22,297) : 694,	(23,297) : 704,	(24,297) : 143,	(25,297) : 1073,	(26,297) : 419,	(27,297) : 478,	(28,297) : 645,	(29,297) : 275,	(30,297) : 647,	(31,297) : 679,	(32,297) : 578,	(33,297) : 561,	(34,297) : 671,	(35,297) : 769,	(36,297) : 904,	(37,297) : 856,	(38,297) : 701,	(39,297) : 700,	(40,297) : 1100,
(1,298) : 539,	(2,298) : 580,	(3,298) : 605,	(4,298) : 90,	(5,298) : 596,	(6,298) : 548,	(7,298) : 533,	(8,298) : 890,	(9,298) : 669,	(10,298) : 534,	(11,298) : 581,	(12,298) : 297,	(13,298) : 369,	(14,298) : 665,	(15,298) : 103,	(16,298) : 202,	(17,298) : 56,	(18,298) : 577,	(19,298) : 292,	(20,298) : 534,	(21,298) : 229,	(22,298) : 547,	(23,298) : 572,	(24,298) : 56,	(25,298) : 877,	(26,298) : 310,	(27,298) : 363,	(28,298) : 533,	(29,298) : 86,	(30,298) : 536,	(31,298) : 552,	(32,298) : 456,	(33,298) : 477,	(34,298) : 536,	(35,298) : 669,	(36,298) : 731,	(37,298) : 671,	(38,298) : 578,	(39,298) : 580,	(40,298) : 905,
(1,299) : 222,	(2,299) : 209,	(3,299) : 439,	(4,299) : 574,	(5,299) : 179,	(6,299) : 158,	(7,299) : 211,	(8,299) : 686,	(9,299) : 194,	(10,299) : 214,	(11,299) : 210,	(12,299) : 602,	(13,299) : 231,	(14,299) : 189,	(15,299) : 425,	(16,299) : 656,	(17,299) : 525,	(18,299) : 162,	(19,299) : 544,	(20,299) : 217,	(21,299) : 275,	(22,299) : 89,	(23,299) : 159,	(24,299) : 527,	(25,299) : 567,	(26,299) : 275,	(27,299) : 232,	(28,299) : 215,	(29,299) : 409,	(30,299) : 221,	(31,299) : 169,	(32,299) : 176,	(33,299) : 297,	(34,299) : 132,	(35,299) : 319,	(36,299) : 253,	(37,299) : 251,	(38,299) : 193,	(39,299) : 206,	(40,299) : 584,
(1,300) : 684,	(2,300) : 653,	(3,300) : 245,	(4,300) : 888,	(5,300) : 606,	(6,300) : 610,	(7,300) : 674,	(8,300) : 259,	(9,300) : 348,	(10,300) : 677,	(11,300) : 654,	(12,300) : 1014,	(13,300) : 691,	(14,300) : 351,	(15,300) : 784,	(16,300) : 998,	(17,300) : 850,	(18,300) : 598,	(19,300) : 967,	(20,300) : 680,	(21,300) : 664,	(22,300) : 529,	(23,300) : 598,	(24,300) : 851,	(25,300) : 94,	(26,300) : 718,	(27,300) : 691,	(28,300) : 678,	(29,300) : 719,	(30,300) : 684,	(31,300) : 621,	(32,300) : 649,	(33,300) : 769,	(34,300) : 587,	(35,300) : 744,	(36,300) : 343,	(37,300) : 238,	(38,300) : 636,	(39,300) : 650,	(40,300) : 114,
(1,301) : 55,	(2,301) : 3,	(3,301) : 644,	(4,301) : 642,	(5,301) : 51,	(6,301) : 48,	(7,301) : 54,	(8,301) : 881,	(9,301) : 315,	(10,301) : 54,	(11,301) : 4,	(12,301) : 582,	(13,301) : 214,	(14,301) : 311,	(15,301) : 486,	(16,301) : 694,	(17,301) : 592,	(18,301) : 53,	(19,301) : 518,	(20,301) : 57,	(21,301) : 351,	(22,301) : 125,	(23,301) : 53,	(24,301) : 595,	(25,301) : 744,	(26,301) : 279,	(27,301) : 220,	(28,301) : 56,	(29,301) : 507,	(30,301) : 58,	(31,301) : 37,	(32,301) : 122,	(33,301) : 173,	(34,301) : 74,	(35,301) : 116,	(36,301) : 352,	(37,301) : 414,	(38,301) : 15,	(39,301) : 3,	(40,301) : 757,
(1,302) : 492,	(2,302) : 530,	(3,302) : 561,	(4,302) : 145,	(5,302) : 544,	(6,302) : 496,	(7,302) : 485,	(8,302) : 849,	(9,302) : 612,	(10,302) : 486,	(11,302) : 531,	(12,302) : 315,	(13,302) : 323,	(14,302) : 608,	(15,302) : 81,	(16,302) : 251,	(17,302) : 103,	(18,302) : 525,	(19,302) : 298,	(20,302) : 486,	(21,302) : 177,	(22,302) : 492,	(23,302) : 519,	(24,302) : 104,	(25,302) : 828,	(26,302) : 266,	(27,302) : 317,	(28,302) : 485,	(29,302) : 29,	(30,302) : 489,	(31,302) : 501,	(32,302) : 406,	(33,302) : 437,	(34,302) : 483,	(35,302) : 622,	(36,302) : 674,	(37,302) : 615,	(38,302) : 527,	(39,302) : 530,	(40,302) : 855,
(1,303) : 686,	(2,303) : 655,	(3,303) : 246,	(4,303) : 890,	(5,303) : 607,	(6,303) : 612,	(7,303) : 675,	(8,303) : 259,	(9,303) : 349,	(10,303) : 678,	(11,303) : 656,	(12,303) : 1015,	(13,303) : 693,	(14,303) : 352,	(15,303) : 785,	(16,303) : 1000,	(17,303) : 852,	(18,303) : 599,	(19,303) : 969,	(20,303) : 682,	(21,303) : 666,	(22,303) : 530,	(23,303) : 599,	(24,303) : 853,	(25,303) : 93,	(26,303) : 720,	(27,303) : 693,	(28,303) : 679,	(29,303) : 721,	(30,303) : 685,	(31,303) : 622,	(32,303) : 650,	(33,303) : 771,	(34,303) : 588,	(35,303) : 745,	(36,303) : 344,	(37,303) : 239,	(38,303) : 637,	(39,303) : 651,	(40,303) : 112,
(1,304) : 470,	(2,304) : 520,	(3,304) : 711,	(4,304) : 157,	(5,304) : 549,	(6,304) : 500,	(7,304) : 467,	(8,304) : 1001,	(9,304) : 684,	(10,304) : 468,	(11,304) : 521,	(12,304) : 150,	(13,304) : 305,	(14,304) : 679,	(15,304) : 97,	(16,304) : 176,	(17,304) : 123,	(18,304) : 532,	(19,304) : 134,	(20,304) : 466,	(21,304) : 222,	(22,304) : 522,	(23,304) : 527,	(24,304) : 125,	(25,304) : 961,	(26,304) : 241,	(27,304) : 300,	(28,304) : 466,	(29,304) : 184,	(30,304) : 468,	(31,304) : 501,	(32,304) : 400,	(33,304) : 382,	(34,304) : 495,	(35,304) : 590,	(36,304) : 745,	(37,304) : 711,	(38,304) : 522,	(39,304) : 521,	(40,304) : 986,
(1,305) : 786,	(2,305) : 764,	(3,305) : 181,	(4,305) : 872,	(5,305) : 722,	(6,305) : 717,	(7,305) : 775,	(8,305) : 121,	(9,305) : 484,	(10,305) : 778,	(11,305) : 765,	(12,305) : 1036,	(13,305) : 759,	(14,305) : 486,	(15,305) : 793,	(16,305) : 988,	(17,305) : 841,	(18,305) : 711,	(19,305) : 997,	(20,305) : 781,	(21,305) : 698,	(22,305) : 637,	(23,305) : 710,	(24,305) : 841,	(25,305) : 156,	(26,305) : 773,	(27,305) : 757,	(28,305) : 779,	(29,305) : 716,	(30,305) : 786,	(31,305) : 728,	(32,305) : 736,	(33,305) : 855,	(34,305) : 692,	(35,305) : 865,	(36,305) : 493,	(37,305) : 370,	(38,305) : 747,	(39,305) : 761,	(40,305) : 184,
(1,306) : 516,	(2,306) : 571,	(3,306) : 886,	(4,306) : 306,	(5,306) : 612,	(6,306) : 568,	(7,306) : 517,	(8,306) : 1176,	(9,306) : 800,	(10,306) : 516,	(11,306) : 572,	(12,306) : 49,	(13,306) : 384,	(14,306) : 795,	(15,306) : 280,	(16,306) : 245,	(17,306) : 293,	(18,306) : 600,	(19,306) : 66,	(20,306) : 514,	(21,306) : 368,	(22,306) : 613,	(23,306) : 595,	(24,306) : 295,	(25,306) : 1122,	(26,306) : 333,	(27,306) : 381,	(28,306) : 514,	(29,306) : 370,	(30,306) : 514,	(31,306) : 565,	(32,306) : 471,	(33,306) : 404,	(34,306) : 571,	(35,306) : 610,	(36,306) : 858,	(37,306) : 847,	(38,306) : 579,	(39,306) : 574,	(40,306) : 1145,
(1,307) : 319,	(2,307) : 342,	(3,307) : 434,	(4,307) : 371,	(5,307) : 341,	(6,307) : 298,	(7,307) : 310,	(8,307) : 719,	(9,307) : 384,	(10,307) : 312,	(11,307) : 343,	(12,307) : 448,	(13,307) : 192,	(14,307) : 380,	(15,307) : 232,	(16,307) : 462,	(17,307) : 323,	(18,307) : 321,	(19,307) : 402,	(20,307) : 313,	(21,307) : 99,	(22,307) : 274,	(23,307) : 316,	(24,307) : 325,	(25,307) : 655,	(26,307) : 180,	(27,307) : 187,	(28,307) : 311,	(29,307) : 202,	(30,307) : 317,	(31,307) : 305,	(32,307) : 232,	(33,307) : 317,	(34,307) : 279,	(35,307) : 449,	(36,307) : 446,	(37,307) : 398,	(38,307) : 334,	(39,307) : 341,	(40,307) : 678,
(1,308) : 934,	(2,308) : 898,	(3,308) : 451,	(4,308) : 1134,	(5,308) : 848,	(6,308) : 859,	(7,308) : 924,	(8,308) : 244,	(9,308) : 582,	(10,308) : 927,	(11,308) : 899,	(12,308) : 1272,	(13,308) : 952,	(14,308) : 587,	(15,308) : 1038,	(16,308) : 1247,	(17,308) : 1099,	(18,308) : 843,	(19,308) : 1227,	(20,308) : 930,	(21,308) : 924,	(22,308) : 779,	(23,308) : 844,	(24,308) : 1099,	(25,308) : 171,	(26,308) : 979,	(27,308) : 951,	(28,308) : 928,	(29,308) : 970,	(30,308) : 934,	(31,308) : 869,	(32,308) : 905,	(33,308) : 1025,	(34,308) : 837,	(35,308) : 980,	(36,308) : 560,	(37,308) : 484,	(38,308) : 881,	(39,308) : 895,	(40,308) : 147,
(1,309) : 682,	(2,309) : 735,	(3,309) : 903,	(4,309) : 219,	(5,309) : 768,	(6,309) : 720,	(7,309) : 680,	(8,309) : 1189,	(9,309) : 912,	(10,309) : 680,	(11,309) : 735,	(12,309) : 171,	(13,309) : 525,	(14,309) : 908,	(15,309) : 308,	(16,309) : 103,	(17,309) : 243,	(18,309) : 753,	(19,309) : 230,	(20,309) : 678,	(21,309) : 449,	(22,309) : 748,	(23,309) : 748,	(24,309) : 243,	(25,309) : 1170,	(26,309) : 463,	(27,309) : 520,	(28,309) : 678,	(29,309) : 370,	(30,309) : 679,	(31,309) : 720,	(32,309) : 620,	(33,309) : 581,	(34,309) : 718,	(35,309) : 791,	(36,309) : 973,	(37,309) : 936,	(38,309) : 739,	(39,309) : 736,	(40,309) : 1197,
(1,310) : 505,	(2,310) : 545,	(3,310) : 589,	(4,310) : 118,	(5,310) : 561,	(6,310) : 513,	(7,310) : 498,	(8,310) : 876,	(9,310) : 637,	(10,310) : 500,	(11,310) : 546,	(12,310) : 293,	(13,310) : 335,	(14,310) : 633,	(15,310) : 75,	(16,310) : 222,	(17,310) : 74,	(18,310) : 543,	(19,310) : 281,	(20,310) : 499,	(21,310) : 194,	(22,310) : 512,	(23,310) : 537,	(24,310) : 75,	(25,310) : 856,	(26,310) : 276,	(27,310) : 329,	(28,310) : 498,	(29,310) : 57,	(30,310) : 502,	(31,310) : 517,	(32,310) : 421,	(33,310) : 445,	(34,310) : 501,	(35,310) : 635,	(36,310) : 700,	(37,310) : 642,	(38,310) : 543,	(39,310) : 545,	(40,310) : 883,
(1,311) : 299,	(2,311) : 294,	(3,311) : 361,	(4,311) : 533,	(5,311) : 268,	(6,311) : 242,	(7,311) : 288,	(8,311) : 622,	(9,311) : 223,	(10,311) : 291,	(11,311) : 295,	(12,311) : 599,	(13,311) : 262,	(14,311) : 219,	(15,311) : 395,	(16,311) : 626,	(17,311) : 486,	(18,311) : 250,	(19,311) : 547,	(20,311) : 293,	(21,311) : 253,	(22,311) : 179,	(23,311) : 247,	(24,311) : 488,	(25,311) : 524,	(26,311) : 288,	(27,311) : 261,	(28,311) : 291,	(29,311) : 361,	(30,311) : 298,	(31,311) : 253,	(32,311) : 236,	(33,311) : 355,	(34,311) : 217,	(35,311) : 405,	(36,311) : 285,	(37,311) : 237,	(38,311) : 279,	(39,311) : 291,	(40,311) : 544,
(1,312) : 829,	(2,312) : 795,	(3,312) : 352,	(4,312) : 1028,	(5,312) : 746,	(6,312) : 754,	(7,312) : 819,	(8,312) : 209,	(9,312) : 482,	(10,312) : 822,	(11,312) : 796,	(12,312) : 1162,	(13,312) : 842,	(14,312) : 486,	(15,312) : 929,	(16,312) : 1140,	(17,312) : 991,	(18,312) : 740,	(19,312) : 1117,	(20,312) : 825,	(21,312) : 814,	(22,312) : 674,	(23,312) : 740,	(24,312) : 992,	(25,312) : 61,	(26,312) : 869,	(27,312) : 842,	(28,312) : 823,	(29,312) : 862,	(30,312) : 829,	(31,312) : 765,	(32,312) : 797,	(33,312) : 918,	(34,312) : 732,	(35,312) : 881,	(36,312) : 466,	(37,312) : 379,	(38,312) : 778,	(39,312) : 792,	(40,312) : 38,
(1,313) : 549,	(2,313) : 594,	(3,313) : 663,	(4,313) : 51,	(5,313) : 614,	(6,313) : 565,	(7,313) : 544,	(8,313) : 949,	(9,313) : 708,	(10,313) : 545,	(11,313) : 595,	(12,313) : 244,	(13,313) : 379,	(14,313) : 704,	(15,313) : 106,	(16,313) : 146,	(17,313) : 4,	(18,313) : 597,	(19,313) : 247,	(20,313) : 544,	(21,313) : 253,	(22,313) : 573,	(23,313) : 591,	(24,313) : 6,	(25,313) : 933,	(26,313) : 316,	(27,313) : 373,	(28,313) : 543,	(29,313) : 134,	(30,313) : 546,	(31,313) : 569,	(32,313) : 470,	(33,313) : 476,	(34,313) : 556,	(35,313) : 676,	(36,313) : 770,	(37,313) : 716,	(38,313) : 593,	(39,313) : 594,	(40,313) : 960,
(1,314) : 481,	(2,314) : 528,	(3,314) : 687,	(4,314) : 131,	(5,314) : 554,	(6,314) : 505,	(7,314) : 476,	(8,314) : 977,	(9,314) : 677,	(10,314) : 477,	(11,314) : 529,	(12,314) : 179,	(13,314) : 312,	(14,314) : 673,	(15,314) : 74,	(16,314) : 170,	(17,314) : 93,	(18,314) : 538,	(19,314) : 166,	(20,314) : 476,	(21,314) : 213,	(22,314) : 523,	(23,314) : 532,	(24,314) : 95,	(25,314) : 942,	(26,314) : 248,	(27,314) : 307,	(28,314) : 475,	(29,314) : 156,	(30,314) : 478,	(31,314) : 507,	(32,314) : 407,	(33,314) : 398,	(34,314) : 499,	(35,314) : 603,	(36,314) : 739,	(37,314) : 700,	(38,314) : 530,	(39,314) : 529,	(40,314) : 968,
(1,315) : 475,	(2,315) : 511,	(3,315) : 522,	(4,315) : 184,	(5,315) : 521,	(6,315) : 474,	(7,315) : 468,	(8,315) : 810,	(9,315) : 577,	(10,315) : 469,	(11,315) : 512,	(12,315) : 348,	(13,315) : 311,	(14,315) : 573,	(15,315) : 105,	(16,315) : 291,	(17,315) : 143,	(18,315) : 502,	(19,315) : 326,	(20,315) : 469,	(21,315) : 159,	(22,315) : 464,	(23,315) : 496,	(24,315) : 144,	(25,315) : 788,	(26,315) : 258,	(27,315) : 304,	(28,315) : 468,	(29,315) : 14,	(30,315) : 472,	(31,315) : 479,	(32,315) : 388,	(33,315) : 430,	(34,315) : 459,	(35,315) : 607,	(36,315) : 639,	(37,315) : 576,	(38,315) : 506,	(39,315) : 510,	(40,315) : 815,
(1,316) : 393,	(2,316) : 353,	(3,316) : 405,	(4,316) : 779,	(5,316) : 301,	(6,316) : 318,	(7,316) : 385,	(8,316) : 569,	(9,316) : 36,	(10,316) : 387,	(11,316) : 353,	(12,316) : 826,	(13,316) : 453,	(14,316) : 40,	(15,316) : 638,	(16,316) : 869,	(17,316) : 731,	(18,316) : 298,	(19,316) : 769,	(20,316) : 391,	(21,316) : 492,	(22,316) : 242,	(23,316) : 299,	(24,316) : 733,	(25,316) : 405,	(26,316) : 500,	(27,316) : 455,	(28,316) : 389,	(29,316) : 606,	(30,316) : 394,	(31,316) : 327,	(32,316) : 383,	(33,316) : 496,	(34,316) : 299,	(35,316) : 432,	(36,316) : 48,	(37,316) : 91,	(38,316) : 336,	(39,316) : 349,	(40,316) : 414,
(1,317) : 530,	(2,317) : 578,	(3,317) : 721,	(4,317) : 100,	(5,317) : 605,	(6,317) : 556,	(7,317) : 526,	(8,317) : 1009,	(9,317) : 727,	(10,317) : 527,	(11,317) : 579,	(12,317) : 167,	(13,317) : 362,	(14,317) : 723,	(15,317) : 117,	(16,317) : 119,	(17,317) : 81,	(18,317) : 588,	(19,317) : 173,	(20,317) : 525,	(21,317) : 263,	(22,317) : 574,	(23,317) : 583,	(24,317) : 83,	(25,317) : 982,	(26,317) : 298,	(27,317) : 357,	(28,317) : 525,	(29,317) : 185,	(30,317) : 527,	(31,317) : 558,	(32,317) : 457,	(33,317) : 444,	(34,317) : 550,	(35,317) : 651,	(36,317) : 788,	(37,317) : 747,	(38,317) : 580,	(39,317) : 579,	(40,317) : 1008,
(1,318) : 574,	(2,318) : 623,	(3,318) : 751,	(4,318) : 89,	(5,318) : 650,	(6,318) : 601,	(7,318) : 570,	(8,318) : 1038,	(9,318) : 770,	(10,318) : 571,	(11,318) : 624,	(12,318) : 171,	(13,318) : 407,	(14,318) : 766,	(15,318) : 159,	(16,318) : 74,	(17,318) : 94,	(18,318) : 633,	(19,318) : 193,	(20,318) : 569,	(21,318) : 307,	(22,318) : 619,	(23,318) : 628,	(24,318) : 94,	(25,318) : 1017,	(26,318) : 343,	(27,318) : 402,	(28,318) : 569,	(29,318) : 217,	(30,318) : 571,	(31,318) : 603,	(32,318) : 502,	(33,318) : 486,	(34,318) : 595,	(35,318) : 694,	(36,318) : 832,	(37,318) : 788,	(38,318) : 625,	(39,318) : 624,	(40,318) : 1044,
(1,319) : 144,	(2,319) : 120,	(3,319) : 758,	(4,319) : 740,	(5,319) : 146,	(6,319) : 170,	(7,319) : 152,	(8,319) : 982,	(9,319) : 395,	(10,319) : 150,	(11,319) : 119,	(12,319) : 645,	(13,319) : 313,	(14,319) : 391,	(15,319) : 586,	(16,319) : 779,	(17,319) : 691,	(18,319) : 161,	(19,319) : 581,	(20,319) : 150,	(21,319) : 461,	(22,319) : 236,	(23,319) : 164,	(24,319) : 694,	(25,319) : 832,	(26,319) : 375,	(27,319) : 319,	(28,319) : 152,	(29,319) : 617,	(30,319) : 147,	(31,319) : 160,	(32,319) : 232,	(33,319) : 226,	(34,319) : 195,	(35,319) : 16,	(36,319) : 417,	(37,319) : 504,	(38,319) : 132,	(39,319) : 122,	(40,319) : 842,
(1,320) : 348,	(2,320) : 382,	(3,320) : 510,	(4,320) : 284,	(5,320) : 394,	(6,320) : 346,	(7,320) : 340,	(8,320) : 799,	(9,320) : 475,	(10,320) : 342,	(11,320) : 384,	(12,320) : 356,	(13,320) : 187,	(14,320) : 471,	(15,320) : 137,	(16,320) : 368,	(17,320) : 234,	(18,320) : 375,	(19,320) : 313,	(20,320) : 342,	(21,320) : 33,	(22,320) : 342,	(23,320) : 369,	(24,320) : 236,	(25,320) : 746,	(26,320) : 144,	(27,320) : 181,	(28,320) : 340,	(29,320) : 128,	(30,320) : 345,	(31,320) : 352,	(32,320) : 260,	(33,320) : 311,	(34,320) : 332,	(35,320) : 480,	(36,320) : 537,	(37,320) : 493,	(38,320) : 378,	(39,320) : 382,	(40,320) : 770,
(1,321) : 286,	(2,321) : 251,	(3,321) : 432,	(4,321) : 689,	(5,321) : 203,	(6,321) : 211,	(7,321) : 277,	(8,321) : 640,	(9,321) : 78,	(10,321) : 279,	(11,321) : 252,	(12,321) : 719,	(13,321) : 342,	(14,321) : 73,	(15,321) : 542,	(16,321) : 773,	(17,321) : 640,	(18,321) : 195,	(19,321) : 661,	(20,321) : 283,	(21,321) : 393,	(22,321) : 132,	(23,321) : 196,	(24,321) : 642,	(25,321) : 496,	(26,321) : 391,	(27,321) : 344,	(28,321) : 281,	(29,321) : 520,	(30,321) : 286,	(31,321) : 221,	(32,321) : 271,	(33,321) : 385,	(34,321) : 190,	(35,321) : 343,	(36,321) : 135,	(37,321) : 166,	(38,321) : 234,	(39,321) : 247,	(40,321) : 508,
(1,322) : 79,	(2,322) : 98,	(3,322) : 584,	(4,322) : 552,	(5,322) : 114,	(6,322) : 65,	(7,322) : 68,	(8,322) : 838,	(9,322) : 315,	(10,322) : 71,	(11,322) : 100,	(12,322) : 514,	(13,322) : 131,	(14,322) : 310,	(15,322) : 395,	(16,322) : 610,	(17,322) : 502,	(18,322) : 97,	(19,322) : 451,	(20,322) : 73,	(21,322) : 256,	(22,322) : 110,	(23,322) : 92,	(24,322) : 504,	(25,322) : 718,	(26,322) : 194,	(27,322) : 136,	(28,322) : 71,	(29,322) : 412,	(30,322) : 77,	(31,322) : 68,	(32,322) : 37,	(33,322) : 148,	(34,322) : 63,	(35,322) : 204,	(36,322) : 364,	(37,322) : 395,	(38,322) : 94,	(39,322) : 98,	(40,322) : 734,
(1,323) : 556,	(2,323) : 603,	(3,323) : 700,	(4,323) : 55,	(5,323) : 626,	(6,323) : 577,	(7,323) : 552,	(8,323) : 987,	(9,323) : 732,	(10,323) : 552,	(11,323) : 604,	(12,323) : 210,	(13,323) : 386,	(14,323) : 728,	(15,323) : 122,	(16,323) : 113,	(17,323) : 43,	(18,323) : 609,	(19,323) : 219,	(20,323) : 551,	(21,323) : 272,	(22,323) : 589,	(23,323) : 603,	(24,323) : 43,	(25,323) : 968,	(26,323) : 322,	(27,323) : 381,	(28,323) : 551,	(29,323) : 168,	(30,323) : 553,	(31,323) : 580,	(32,323) : 480,	(33,323) : 476,	(34,323) : 569,	(35,323) : 680,	(36,323) : 795,	(37,323) : 745,	(38,323) : 603,	(39,323) : 603,	(40,323) : 995,
(1,324) : 14,	(2,324) : 45,	(3,324) : 648,	(4,324) : 603,	(5,324) : 91,	(6,324) : 62,	(7,324) : 9,	(8,324) : 894,	(9,324) : 344,	(10,324) : 10,	(11,324) : 46,	(12,324) : 537,	(13,324) : 174,	(14,324) : 339,	(15,324) : 447,	(16,324) : 652,	(17,324) : 554,	(18,324) : 86,	(19,324) : 474,	(20,324) : 13,	(21,324) : 317,	(22,324) : 142,	(23,324) : 82,	(24,324) : 556,	(25,324) : 765,	(26,324) : 238,	(27,324) : 180,	(28,324) : 12,	(29,324) : 474,	(30,324) : 15,	(31,324) : 53,	(32,324) : 87,	(33,324) : 130,	(34,324) : 84,	(35,324) : 134,	(36,324) : 385,	(37,324) : 437,	(38,324) : 55,	(39,324) : 47,	(40,324) : 779,
(1,325) : 412,	(2,325) : 456,	(3,325) : 620,	(4,325) : 188,	(5,325) : 479,	(6,325) : 430,	(7,325) : 406,	(8,325) : 910,	(9,325) : 594,	(10,325) : 407,	(11,325) : 457,	(12,325) : 238,	(13,325) : 241,	(14,325) : 590,	(15,325) : 36,	(16,325) : 252,	(17,325) : 139,	(18,325) : 461,	(19,325) : 205,	(20,325) : 406,	(21,325) : 130,	(22,325) : 442,	(23,325) : 456,	(24,325) : 142,	(25,325) : 867,	(26,325) : 178,	(27,325) : 235,	(28,325) : 406,	(29,325) : 115,	(30,325) : 409,	(31,325) : 433,	(32,325) : 333,	(33,325) : 341,	(34,325) : 422,	(35,325) : 538,	(36,325) : 655,	(37,325) : 617,	(38,325) : 456,	(39,325) : 457,	(40,325) : 892,
(1,326) : 738,	(2,326) : 793,	(3,326) : 1022,	(4,326) : 345,	(5,326) : 832,	(6,326) : 785,	(7,326) : 738,	(8,326) : 1310,	(9,326) : 1001,	(10,326) : 738,	(11,326) : 793,	(12,326) : 211,	(13,326) : 595,	(14,326) : 996,	(15,326) : 415,	(16,326) : 230,	(17,326) : 365,	(18,326) : 818,	(19,326) : 274,	(20,326) : 735,	(21,326) : 545,	(22,326) : 824,	(23,326) : 813,	(24,326) : 366,	(25,326) : 1283,	(26,326) : 538,	(27,326) : 591,	(28,326) : 736,	(29,326) : 487,	(30,326) : 735,	(31,326) : 784,	(32,326) : 687,	(33,326) : 628,	(34,326) : 787,	(35,326) : 834,	(36,326) : 1060,	(37,326) : 1035,	(38,326) : 800,	(39,326) : 795,	(40,326) : 1309,
(1,327) : 703,	(2,327) : 758,	(3,327) : 1007,	(4,327) : 340,	(5,327) : 798,	(6,327) : 752,	(7,327) : 703,	(8,327) : 1295,	(9,327) : 972,	(10,327) : 703,	(11,327) : 758,	(12,327) : 178,	(13,327) : 563,	(14,327) : 967,	(15,327) : 395,	(16,327) : 229,	(17,327) : 355,	(18,327) : 785,	(19,327) : 240,	(20,327) : 700,	(21,327) : 519,	(22,327) : 792,	(23,327) : 780,	(24,327) : 355,	(25,327) : 1263,	(26,327) : 507,	(27,327) : 560,	(28,327) : 701,	(29,327) : 471,	(30,327) : 700,	(31,327) : 750,	(32,327) : 654,	(33,327) : 592,	(34,327) : 754,	(35,327) : 797,	(36,327) : 1031,	(37,327) : 1009,	(38,327) : 765,	(39,327) : 760,	(40,327) : 1288,
(1,328) : 26,	(2,328) : 48,	(3,328) : 634,	(4,328) : 597,	(5,328) : 87,	(6,328) : 52,	(7,328) : 16,	(8,328) : 881,	(9,328) : 333,	(10,328) : 18,	(11,328) : 49,	(12,328) : 537,	(13,328) : 169,	(14,328) : 329,	(15,328) : 441,	(16,328) : 648,	(17,328) : 547,	(18,328) : 79,	(19,328) : 473,	(20,328) : 22,	(21,328) : 308,	(22,328) : 130,	(23,328) : 75,	(24,328) : 550,	(25,328) : 753,	(26,328) : 233,	(27,328) : 174,	(28,328) : 20,	(29,328) : 465,	(30,328) : 25,	(31,328) : 44,	(32,328) : 78,	(33,328) : 135,	(34,328) : 72,	(35,328) : 146,	(36,328) : 376,	(37,328) : 425,	(38,328) : 53,	(39,328) : 49,	(40,328) : 767,
(1,329) : 110,	(2,329) : 150,	(3,329) : 597,	(4,329) : 494,	(5,329) : 175,	(6,329) : 127,	(7,329) : 102,	(8,329) : 862,	(9,329) : 367,	(10,329) : 104,	(11,329) : 151,	(12,329) : 451,	(13,329) : 69,	(14,329) : 362,	(15,329) : 338,	(16,329) : 548,	(17,329) : 445,	(18,329) : 160,	(19,329) : 388,	(20,329) : 104,	(21,329) : 207,	(22,329) : 168,	(23,329) : 154,	(24,329) : 447,	(25,329) : 754,	(26,329) : 132,	(27,329) : 74,	(28,329) : 102,	(29,329) : 363,	(30,329) : 107,	(31,329) : 128,	(32,329) : 27,	(33,329) : 115,	(34,329) : 126,	(35,329) : 242,	(36,329) : 419,	(37,329) : 438,	(38,329) : 150,	(39,329) : 150,	(40,329) : 771,
(1,330) : 123,	(2,330) : 84,	(3,330) : 711,	(4,330) : 723,	(5,330) : 100,	(6,330) : 130,	(7,330) : 128,	(8,330) : 934,	(9,330) : 348,	(10,330) : 127,	(11,330) : 83,	(12,330) : 644,	(13,330) : 294,	(14,330) : 344,	(15,330) : 567,	(16,330) : 769,	(17,330) : 673,	(18,330) : 116,	(19,330) : 580,	(20,330) : 128,	(21,330) : 436,	(22,330) : 190,	(23,330) : 119,	(24,330) : 676,	(25,330) : 784,	(26,330) : 357,	(27,330) : 300,	(28,330) : 129,	(29,330) : 592,	(30,330) : 126,	(31,330) : 121,	(32,330) : 205,	(33,330) : 224,	(34,330) : 153,	(35,330) : 51,	(36,330) : 371,	(37,330) : 456,	(38,330) : 92,	(39,330) : 85,	(40,330) : 794,
(1,331) : 1019,	(2,331) : 983,	(3,331) : 524,	(4,331) : 1212,	(5,331) : 933,	(6,331) : 944,	(7,331) : 1010,	(8,331) : 285,	(9,331) : 667,	(10,331) : 1012,	(11,331) : 984,	(12,331) : 1355,	(13,331) : 1037,	(14,331) : 671,	(15,331) : 1120,	(16,331) : 1326,	(17,331) : 1178,	(18,331) : 928,	(19,331) : 1311,	(20,331) : 1016,	(21,331) : 1008,	(22,331) : 864,	(23,331) : 929,	(24,331) : 1178,	(25,331) : 254,	(26,331) : 1064,	(27,331) : 1037,	(28,331) : 1014,	(29,331) : 1050,	(30,331) : 1020,	(31,331) : 954,	(32,331) : 991,	(33,331) : 1111,	(34,331) : 922,	(35,331) : 1064,	(36,331) : 643,	(37,331) : 570,	(38,331) : 966,	(39,331) : 980,	(40,331) : 233,
(1,332) : 200,	(2,332) : 255,	(3,332) : 764,	(4,332) : 489,	(5,332) : 303,	(6,332) : 267,	(7,332) : 204,	(8,332) : 1036,	(9,332) : 539,	(10,332) : 203,	(11,332) : 255,	(12,332) : 352,	(13,332) : 159,	(14,332) : 534,	(15,332) : 352,	(16,332) : 504,	(17,332) : 446,	(18,332) : 296,	(19,332) : 289,	(20,332) : 200,	(21,332) : 286,	(22,332) : 334,	(23,332) : 292,	(24,332) : 449,	(25,332) : 933,	(26,332) : 175,	(27,332) : 162,	(28,332) : 201,	(29,332) : 415,	(30,332) : 198,	(31,332) : 261,	(32,332) : 193,	(33,332) : 83,	(34,332) : 280,	(35,332) : 279,	(36,332) : 587,	(37,332) : 616,	(38,332) : 267,	(39,332) : 258,	(40,332) : 951,
(1,333) : 282,	(2,333) : 300,	(3,333) : 431,	(4,333) : 418,	(5,333) : 295,	(6,333) : 253,	(7,333) : 272,	(8,333) : 711,	(9,333) : 338,	(10,333) : 274,	(11,333) : 301,	(12,333) : 478,	(13,333) : 177,	(14,333) : 334,	(15,333) : 274,	(16,333) : 505,	(17,333) : 369,	(18,333) : 275,	(19,333) : 427,	(20,333) : 276,	(21,333) : 130,	(22,333) : 225,	(23,333) : 271,	(24,333) : 371,	(25,333) : 633,	(26,333) : 182,	(27,333) : 173,	(28,333) : 274,	(29,333) : 250,	(30,333) : 280,	(31,333) : 262,	(32,333) : 197,	(33,333) : 295,	(34,333) : 233,	(35,333) : 409,	(36,333) : 400,	(37,333) : 360,	(38,333) : 290,	(39,333) : 298,	(40,333) : 655,
(1,334) : 657,	(2,334) : 626,	(3,334) : 234,	(4,334) : 865,	(5,334) : 579,	(6,334) : 583,	(7,334) : 647,	(8,334) : 279,	(9,334) : 322,	(10,334) : 649,	(11,334) : 627,	(12,334) : 987,	(13,334) : 663,	(14,334) : 326,	(15,334) : 758,	(16,334) : 974,	(17,334) : 826,	(18,334) : 571,	(19,334) : 940,	(20,334) : 653,	(21,334) : 638,	(22,334) : 502,	(23,334) : 571,	(24,334) : 827,	(25,334) : 122,	(26,334) : 690,	(27,334) : 663,	(28,334) : 650,	(29,334) : 695,	(30,334) : 656,	(31,334) : 594,	(32,334) : 621,	(33,334) : 741,	(34,334) : 560,	(35,334) : 718,	(36,334) : 320,	(37,334) : 211,	(38,334) : 609,	(39,334) : 623,	(40,334) : 141,
(1,335) : 289,	(2,335) : 268,	(3,335) : 382,	(4,335) : 614,	(5,335) : 230,	(6,335) : 219,	(7,335) : 278,	(8,335) : 618,	(9,335) : 141,	(10,335) : 281,	(11,335) : 269,	(12,335) : 662,	(13,335) : 301,	(14,335) : 137,	(15,335) : 471,	(16,335) : 702,	(17,335) : 566,	(18,335) : 216,	(19,335) : 607,	(20,335) : 284,	(21,335) : 325,	(22,335) : 141,	(23,335) : 215,	(24,335) : 568,	(25,335) : 495,	(26,335) : 340,	(27,335) : 301,	(28,335) : 282,	(29,335) : 443,	(30,335) : 288,	(31,335) : 231,	(32,335) : 248,	(33,335) : 369,	(34,335) : 194,	(35,335) : 374,	(36,335) : 203,	(37,335) : 180,	(38,335) : 251,	(39,335) : 265,	(40,335) : 512,
(1,336) : 246,	(2,336) : 213,	(3,336) : 456,	(4,336) : 665,	(5,336) : 166,	(6,336) : 171,	(7,336) : 237,	(8,336) : 675,	(9,336) : 117,	(10,336) : 239,	(11,336) : 213,	(12,336) : 685,	(13,336) : 306,	(14,336) : 112,	(15,336) : 516,	(16,336) : 746,	(17,336) : 616,	(18,336) : 157,	(19,336) : 626,	(20,336) : 243,	(21,336) : 366,	(22,336) : 91,	(23,336) : 157,	(24,336) : 618,	(25,336) : 534,	(26,336) : 356,	(27,336) : 308,	(28,336) : 241,	(29,336) : 500,	(30,336) : 246,	(31,336) : 181,	(32,336) : 231,	(33,336) : 345,	(34,336) : 150,	(35,336) : 308,	(36,336) : 170,	(37,336) : 204,	(38,336) : 195,	(39,336) : 209,	(40,336) : 547,
(1,337) : 550,	(2,337) : 590,	(3,337) : 600,	(4,337) : 92,	(5,337) : 605,	(6,337) : 557,	(7,337) : 544,	(8,337) : 884,	(9,337) : 673,	(10,337) : 545,	(11,337) : 591,	(12,337) : 310,	(13,337) : 381,	(14,337) : 670,	(15,337) : 117,	(16,337) : 207,	(17,337) : 65,	(18,337) : 587,	(19,337) : 306,	(20,337) : 545,	(21,337) : 238,	(22,337) : 554,	(23,337) : 581,	(24,337) : 65,	(25,337) : 874,	(26,337) : 321,	(27,337) : 374,	(28,337) : 544,	(29,337) : 90,	(30,337) : 547,	(31,337) : 562,	(32,337) : 466,	(33,337) : 490,	(34,337) : 545,	(35,337) : 680,	(36,337) : 736,	(37,337) : 673,	(38,337) : 588,	(39,337) : 590,	(40,337) : 902,
(1,338) : 101,	(2,338) : 84,	(3,338) : 730,	(4,338) : 697,	(5,338) : 121,	(6,338) : 136,	(7,338) : 110,	(8,338) : 962,	(9,338) : 383,	(10,338) : 108,	(11,338) : 83,	(12,338) : 606,	(13,338) : 270,	(14,338) : 379,	(15,338) : 543,	(16,338) : 737,	(17,338) : 648,	(18,338) : 133,	(19,338) : 542,	(20,338) : 108,	(21,338) : 418,	(22,338) : 209,	(23,338) : 135,	(24,338) : 651,	(25,338) : 818,	(26,338) : 332,	(27,338) : 276,	(28,338) : 109,	(29,338) : 574,	(30,338) : 104,	(31,338) : 125,	(32,338) : 190,	(33,338) : 186,	(34,338) : 161,	(35,338) : 31,	(36,338) : 411,	(37,338) : 488,	(38,338) : 99,	(39,338) : 87,	(40,338) : 829,
(1,339) : 19,	(2,339) : 75,	(3,339) : 667,	(4,339) : 587,	(5,339) : 123,	(6,339) : 93,	(7,339) : 25,	(8,339) : 918,	(9,339) : 374,	(10,339) : 23,	(11,339) : 75,	(12,339) : 511,	(13,339) : 159,	(14,339) : 370,	(15,339) : 432,	(16,339) : 631,	(17,339) : 538,	(18,339) : 118,	(19,339) : 447,	(20,339) : 20,	(21,339) : 308,	(22,339) : 171,	(23,339) : 114,	(24,339) : 541,	(25,339) : 792,	(26,339) : 221,	(27,339) : 165,	(28,339) : 21,	(29,339) : 464,	(30,339) : 17,	(31,339) : 84,	(32,339) : 84,	(33,339) : 98,	(34,339) : 112,	(35,339) : 141,	(36,339) : 416,	(37,339) : 465,	(38,339) : 86,	(39,339) : 77,	(40,339) : 807,
(1,340) : 159,	(2,340) : 158,	(3,340) : 500,	(4,340) : 546,	(5,340) : 144,	(6,340) : 107,	(7,340) : 148,	(8,340) : 754,	(9,340) : 252,	(10,340) : 150,	(11,340) : 159,	(12,340) : 548,	(13,340) : 168,	(14,340) : 248,	(15,340) : 392,	(16,340) : 618,	(17,340) : 496,	(18,340) : 125,	(19,340) : 488,	(20,340) : 153,	(21,340) : 243,	(22,340) : 77,	(23,340) : 120,	(24,340) : 498,	(25,340) : 638,	(26,340) : 219,	(27,340) : 170,	(28,340) : 151,	(29,340) : 390,	(30,340) : 158,	(31,340) : 118,	(32,340) : 105,	(33,340) : 226,	(34,340) : 84,	(35,340) : 270,	(36,340) : 308,	(37,340) : 320,	(38,340) : 145,	(39,340) : 156,	(40,340) : 655,
(1,341) : 741,	(2,341) : 795,	(3,341) : 1006,	(4,341) : 324,	(5,341) : 833,	(6,341) : 786,	(7,341) : 740,	(8,341) : 1292,	(9,341) : 995,	(10,341) : 740,	(11,341) : 795,	(12,341) : 212,	(13,341) : 594,	(14,341) : 991,	(15,341) : 402,	(16,341) : 207,	(17,341) : 347,	(18,341) : 819,	(19,341) : 276,	(20,341) : 738,	(21,341) : 536,	(22,341) : 822,	(23,341) : 814,	(24,341) : 347,	(25,341) : 1269,	(26,341) : 535,	(27,341) : 590,	(28,341) : 738,	(29,341) : 471,	(30,341) : 738,	(31,341) : 785,	(32,341) : 687,	(33,341) : 632,	(34,341) : 786,	(35,341) : 840,	(36,341) : 1055,	(37,341) : 1026,	(38,341) : 801,	(39,341) : 797,	(40,341) : 1295,
(1,342) : 7,	(2,342) : 61,	(3,342) : 656,	(4,342) : 593,	(5,342) : 107,	(6,342) : 77,	(7,342) : 10,	(8,342) : 905,	(9,342) : 359,	(10,342) : 7,	(11,342) : 61,	(12,342) : 523,	(13,342) : 164,	(14,342) : 354,	(15,342) : 438,	(16,342) : 640,	(17,342) : 544,	(18,342) : 102,	(19,342) : 459,	(20,342) : 4,	(21,342) : 310,	(22,342) : 155,	(23,342) : 98,	(24,342) : 547,	(25,342) : 777,	(26,342) : 228,	(27,342) : 171,	(28,342) : 6,	(29,342) : 467,	(30,342) : 3,	(31,342) : 68,	(32,342) : 82,	(33,342) : 113,	(34,342) : 97,	(35,342) : 138,	(36,342) : 400,	(37,342) : 450,	(38,342) : 71,	(39,342) : 63,	(40,342) : 792,
(1,343) : 452,	(2,343) : 484,	(3,343) : 482,	(4,343) : 229,	(5,343) : 491,	(6,343) : 444,	(7,343) : 444,	(8,343) : 771,	(9,343) : 534,	(10,343) : 445,	(11,343) : 485,	(12,343) : 381,	(13,343) : 293,	(14,343) : 531,	(15,343) : 134,	(16,343) : 335,	(17,343) : 187,	(18,343) : 471,	(19,343) : 353,	(20,343) : 446,	(21,343) : 139,	(22,343) : 430,	(23,343) : 466,	(24,343) : 188,	(25,343) : 745,	(26,343) : 247,	(27,343) : 287,	(28,343) : 444,	(29,343) : 56,	(30,343) : 449,	(31,343) : 451,	(32,343) : 364,	(33,343) : 417,	(34,343) : 428,	(35,343) : 584,	(36,343) : 596,	(37,343) : 532,	(38,343) : 479,	(39,343) : 483,	(40,343) : 771,
(1,344) : 15,	(2,344) : 54,	(3,344) : 644,	(4,344) : 593,	(5,344) : 97,	(6,344) : 64,	(7,344) : 4,	(8,344) : 892,	(9,344) : 345,	(10,344) : 7,	(11,344) : 55,	(12,344) : 529,	(13,344) : 164,	(14,344) : 341,	(15,344) : 437,	(16,344) : 642,	(17,344) : 544,	(18,344) : 90,	(19,344) : 465,	(20,344) : 10,	(21,344) : 307,	(22,344) : 142,	(23,344) : 86,	(24,344) : 546,	(25,344) : 764,	(26,344) : 228,	(27,344) : 170,	(28,344) : 8,	(29,344) : 464,	(30,344) : 14,	(31,344) : 56,	(32,344) : 77,	(33,344) : 124,	(34,344) : 84,	(35,344) : 143,	(36,344) : 388,	(37,344) : 437,	(38,344) : 62,	(39,344) : 56,	(40,344) : 778,
(1,345) : 526,	(2,345) : 565,	(3,345) : 583,	(4,345) : 113,	(5,345) : 580,	(6,345) : 532,	(7,345) : 519,	(8,345) : 869,	(9,345) : 648,	(10,345) : 521,	(11,345) : 566,	(12,345) : 311,	(13,345) : 357,	(14,345) : 644,	(15,345) : 99,	(16,345) : 224,	(17,345) : 77,	(18,345) : 561,	(19,345) : 301,	(20,345) : 520,	(21,345) : 213,	(22,345) : 528,	(23,345) : 555,	(24,345) : 77,	(25,345) : 854,	(26,345) : 298,	(27,345) : 350,	(28,345) : 519,	(29,345) : 64,	(30,345) : 523,	(31,345) : 536,	(32,345) : 441,	(33,345) : 468,	(34,345) : 519,	(35,345) : 656,	(36,345) : 710,	(37,345) : 648,	(38,345) : 563,	(39,345) : 565,	(40,345) : 882,
(1,346) : 985,	(2,346) : 950,	(3,346) : 470,	(4,346) : 1159,	(5,346) : 901,	(6,346) : 909,	(7,346) : 975,	(8,346) : 229,	(9,346) : 636,	(10,346) : 977,	(11,346) : 951,	(12,346) : 1307,	(13,346) : 995,	(14,346) : 640,	(15,346) : 1070,	(16,346) : 1273,	(17,346) : 1126,	(18,346) : 895,	(19,346) : 1263,	(20,346) : 981,	(21,346) : 961,	(22,346) : 829,	(23,346) : 895,	(24,346) : 1126,	(25,346) : 210,	(26,346) : 1020,	(27,346) : 995,	(28,346) : 979,	(29,346) : 998,	(30,346) : 984,	(31,346) : 920,	(32,346) : 952,	(33,346) : 1073,	(34,346) : 887,	(35,346) : 1034,	(36,346) : 616,	(37,346) : 534,	(38,346) : 933,	(39,346) : 947,	(40,346) : 193,
(1,347) : 88,	(2,347) : 33,	(3,347) : 645,	(4,347) : 673,	(5,347) : 34,	(6,347) : 63,	(7,347) : 88,	(8,347) : 874,	(9,347) : 299,	(10,347) : 88,	(11,347) : 33,	(12,347) : 616,	(13,347) : 246,	(14,347) : 294,	(15,347) : 516,	(16,347) : 726,	(17,347) : 623,	(18,347) : 48,	(19,347) : 552,	(20,347) : 91,	(21,347) : 379,	(22,347) : 123,	(23,347) : 51,	(24,347) : 626,	(25,347) : 732,	(26,347) : 311,	(27,347) : 252,	(28,347) : 90,	(29,347) : 535,	(30,347) : 91,	(31,347) : 56,	(32,347) : 152,	(33,347) : 205,	(34,347) : 85,	(35,347) : 110,	(36,347) : 330,	(37,347) : 402,	(38,347) : 29,	(39,347) : 31,	(40,347) : 743,
(1,348) : 5,	(2,348) : 56,	(3,348) : 654,	(4,348) : 596,	(5,348) : 102,	(6,348) : 72,	(7,348) : 6,	(8,348) : 902,	(9,348) : 354,	(10,348) : 4,	(11,348) : 56,	(12,348) : 527,	(13,348) : 168,	(14,348) : 350,	(15,348) : 441,	(16,348) : 644,	(17,348) : 547,	(18,348) : 97,	(19,348) : 464,	(20,348) : 2,	(21,348) : 312,	(22,348) : 152,	(23,348) : 94,	(24,348) : 550,	(25,348) : 774,	(26,348) : 231,	(27,348) : 174,	(28,348) : 3,	(29,348) : 469,	(30,348) : 4,	(31,348) : 64,	(32,348) : 84,	(33,348) : 118,	(34,348) : 93,	(35,348) : 137,	(36,348) : 396,	(37,348) : 446,	(38,348) : 66,	(39,348) : 58,	(40,348) : 788,
(1,349) : 738,	(2,349) : 792,	(3,349) : 1013,	(4,349) : 334,	(5,349) : 830,	(6,349) : 784,	(7,349) : 737,	(8,349) : 1301,	(9,349) : 997,	(10,349) : 737,	(11,349) : 793,	(12,349) : 209,	(13,349) : 593,	(14,349) : 992,	(15,349) : 407,	(16,349) : 219,	(17,349) : 355,	(18,349) : 817,	(19,349) : 273,	(20,349) : 735,	(21,349) : 539,	(22,349) : 821,	(23,349) : 812,	(24,349) : 356,	(25,349) : 1275,	(26,349) : 535,	(27,349) : 589,	(28,349) : 735,	(29,349) : 478,	(30,349) : 735,	(31,349) : 783,	(32,349) : 685,	(33,349) : 628,	(34,349) : 785,	(35,349) : 835,	(36,349) : 1056,	(37,349) : 1029,	(38,349) : 799,	(39,349) : 794,	(40,349) : 1301,
(1,350) : 191,	(2,350) : 167,	(3,350) : 481,	(4,350) : 610,	(5,350) : 132,	(6,350) : 119,	(7,350) : 180,	(8,350) : 718,	(9,350) : 184,	(10,350) : 183,	(11,350) : 169,	(12,350) : 619,	(13,350) : 237,	(14,350) : 179,	(15,350) : 458,	(16,350) : 686,	(17,350) : 560,	(18,350) : 117,	(19,350) : 559,	(20,350) : 186,	(21,350) : 308,	(22,350) : 41,	(23,350) : 115,	(24,350) : 562,	(25,350) : 588,	(26,350) : 290,	(27,350) : 240,	(28,350) : 184,	(29,350) : 449,	(30,350) : 190,	(31,350) : 130,	(32,350) : 164,	(33,350) : 281,	(34,350) : 94,	(35,350) : 274,	(36,350) : 238,	(37,350) : 263,	(38,350) : 151,	(39,350) : 164,	(40,350) : 603,
(1,351) : 409,	(2,351) : 463,	(3,351) : 784,	(4,351) : 287,	(5,351) : 501,	(6,351) : 455,	(7,351) : 408,	(8,351) : 1073,	(9,351) : 683,	(10,351) : 408,	(11,351) : 464,	(12,351) : 120,	(13,351) : 268,	(14,351) : 679,	(15,351) : 203,	(16,351) : 273,	(17,351) : 256,	(18,351) : 488,	(19,351) : 56,	(20,351) : 406,	(21,351) : 259,	(22,351) : 497,	(23,351) : 483,	(24,351) : 259,	(25,351) : 1011,	(26,351) : 216,	(27,351) : 265,	(28,351) : 406,	(29,351) : 293,	(30,351) : 406,	(31,351) : 454,	(32,351) : 358,	(33,351) : 302,	(34,351) : 457,	(35,351) : 512,	(36,351) : 741,	(37,351) : 731,	(38,351) : 470,	(39,351) : 465,	(40,351) : 1034,
(1,352) : 166,	(2,352) : 125,	(3,352) : 543,	(4,352) : 667,	(5,352) : 75,	(6,352) : 92,	(7,352) : 158,	(8,352) : 766,	(9,352) : 193,	(10,352) : 160,	(11,352) : 125,	(12,352) : 652,	(13,352) : 267,	(14,352) : 189,	(15,352) : 512,	(16,352) : 736,	(17,352) : 617,	(18,352) : 69,	(19,352) : 590,	(20,352) : 164,	(21,352) : 364,	(22,352) : 45,	(23,352) : 71,	(24,352) : 619,	(25,352) : 623,	(26,352) : 326,	(27,352) : 271,	(28,352) : 162,	(29,352) : 512,	(30,352) : 167,	(31,352) : 100,	(32,352) : 177,	(33,352) : 276,	(34,352) : 79,	(35,352) : 217,	(36,352) : 232,	(37,352) : 293,	(38,352) : 107,	(39,352) : 121,	(40,352) : 635,
(1,353) : 88,	(2,353) : 141,	(3,353) : 650,	(4,353) : 515,	(5,353) : 180,	(6,353) : 137,	(7,353) : 86,	(8,353) : 913,	(9,353) : 403,	(10,353) : 86,	(11,353) : 141,	(12,353) : 442,	(13,353) : 89,	(14,353) : 398,	(15,353) : 360,	(16,353) : 558,	(17,353) : 466,	(18,353) : 169,	(19,353) : 379,	(20,353) : 84,	(21,353) : 243,	(22,353) : 198,	(23,353) : 164,	(24,353) : 468,	(25,353) : 801,	(26,353) : 149,	(27,353) : 96,	(28,353) : 84,	(29,353) : 396,	(30,353) : 85,	(31,353) : 133,	(32,353) : 58,	(33,353) : 63,	(34,353) : 146,	(35,353) : 211,	(36,353) : 452,	(37,353) : 481,	(38,353) : 147,	(39,353) : 142,	(40,353) : 818,
(1,354) : 442,	(2,354) : 485,	(3,354) : 604,	(4,354) : 161,	(5,354) : 504,	(6,354) : 455,	(7,354) : 436,	(8,354) : 893,	(9,354) : 604,	(10,354) : 437,	(11,354) : 486,	(12,354) : 257,	(13,354) : 271,	(14,354) : 600,	(15,354) : 11,	(16,354) : 239,	(17,354) : 111,	(18,354) : 486,	(19,354) : 231,	(20,354) : 436,	(21,354) : 144,	(22,354) : 462,	(23,354) : 481,	(24,354) : 113,	(25,354) : 858,	(26,354) : 209,	(27,354) : 265,	(28,354) : 435,	(29,354) : 81,	(30,354) : 439,	(31,354) : 459,	(32,354) : 361,	(33,354) : 376,	(34,354) : 445,	(35,354) : 570,	(36,354) : 666,	(37,354) : 620,	(38,354) : 484,	(39,354) : 485,	(40,354) : 884,
(1,355) : 488,	(2,355) : 527,	(3,355) : 563,	(4,355) : 146,	(5,355) : 541,	(6,355) : 493,	(7,355) : 481,	(8,355) : 851,	(9,355) : 611,	(10,355) : 483,	(11,355) : 528,	(12,355) : 312,	(13,355) : 320,	(14,355) : 607,	(15,355) : 77,	(16,355) : 250,	(17,355) : 102,	(18,355) : 522,	(19,355) : 294,	(20,355) : 482,	(21,355) : 174,	(22,355) : 489,	(23,355) : 517,	(24,355) : 103,	(25,355) : 829,	(26,355) : 263,	(27,355) : 314,	(28,355) : 481,	(29,355) : 29,	(30,355) : 485,	(31,355) : 498,	(32,355) : 403,	(33,355) : 433,	(34,355) : 480,	(35,355) : 619,	(36,355) : 673,	(37,355) : 614,	(38,355) : 524,	(39,355) : 527,	(40,355) : 856,
(1,356) : 113,	(2,356) : 65,	(3,356) : 681,	(4,356) : 709,	(5,356) : 71,	(6,356) : 105,	(7,356) : 116,	(8,356) : 905,	(9,356) : 320,	(10,356) : 116,	(11,356) : 64,	(12,356) : 641,	(13,356) : 281,	(14,356) : 316,	(15,356) : 553,	(16,356) : 759,	(17,356) : 659,	(18,356) : 88,	(19,356) : 577,	(20,356) : 117,	(21,356) : 418,	(22,356) : 161,	(23,356) : 92,	(24,356) : 662,	(25,356) : 756,	(26,356) : 345,	(27,356) : 286,	(28,356) : 118,	(29,356) : 574,	(30,356) : 116,	(31,356) : 97,	(32,356) : 189,	(33,356) : 224,	(34,356) : 127,	(35,356) : 79,	(36,356) : 346,	(37,356) : 428,	(38,356) : 69,	(39,356) : 65,	(40,356) : 766,
(1,357) : 458,	(2,357) : 492,	(3,357) : 508,	(4,357) : 205,	(5,357) : 501,	(6,357) : 455,	(7,357) : 450,	(8,357) : 797,	(9,357) : 555,	(10,357) : 452,	(11,357) : 493,	(12,357) : 357,	(13,357) : 295,	(14,357) : 551,	(15,357) : 111,	(16,357) : 310,	(17,357) : 162,	(18,357) : 482,	(19,357) : 332,	(20,357) : 452,	(21,357) : 142,	(22,357) : 444,	(23,357) : 477,	(24,357) : 163,	(25,357) : 770,	(26,357) : 245,	(27,357) : 289,	(28,357) : 450,	(29,357) : 31,	(30,357) : 455,	(31,357) : 461,	(32,357) : 371,	(33,357) : 416,	(34,357) : 440,	(35,357) : 590,	(36,357) : 617,	(37,357) : 555,	(38,357) : 488,	(39,357) : 492,	(40,357) : 797,
(1,358) : 540,	(2,358) : 580,	(3,358) : 602,	(4,358) : 92,	(5,358) : 596,	(6,358) : 548,	(7,358) : 533,	(8,358) : 888,	(9,358) : 668,	(10,358) : 535,	(11,358) : 581,	(12,358) : 300,	(13,358) : 370,	(14,358) : 664,	(15,358) : 104,	(16,358) : 204,	(17,358) : 59,	(18,358) : 577,	(19,358) : 294,	(20,358) : 534,	(21,358) : 229,	(22,358) : 546,	(23,358) : 572,	(24,358) : 58,	(25,358) : 875,	(26,358) : 310,	(27,358) : 364,	(28,358) : 533,	(29,358) : 84,	(30,358) : 537,	(31,358) : 552,	(32,358) : 456,	(33,358) : 478,	(34,358) : 535,	(35,358) : 669,	(36,358) : 730,	(37,358) : 669,	(38,358) : 578,	(39,358) : 580,	(40,358) : 903,
(1,359) : 560,	(2,359) : 601,	(3,359) : 616,	(4,359) : 76,	(5,359) : 617,	(6,359) : 569,	(7,359) : 554,	(8,359) : 900,	(9,359) : 689,	(10,359) : 555,	(11,359) : 602,	(12,359) : 301,	(13,359) : 390,	(14,359) : 685,	(15,359) : 122,	(16,359) : 191,	(17,359) : 53,	(18,359) : 599,	(19,359) : 300,	(20,359) : 555,	(21,359) : 251,	(22,359) : 568,	(23,359) : 593,	(24,359) : 52,	(25,359) : 891,	(26,359) : 330,	(27,359) : 384,	(28,359) : 554,	(29,359) : 105,	(30,359) : 557,	(31,359) : 573,	(32,359) : 477,	(33,359) : 497,	(34,359) : 557,	(35,359) : 690,	(36,359) : 751,	(37,359) : 689,	(38,359) : 599,	(39,359) : 601,	(40,359) : 919,
(1,360) : 560,	(2,360) : 606,	(3,360) : 687,	(4,360) : 43,	(5,360) : 628,	(6,360) : 579,	(7,360) : 555,	(8,360) : 973,	(9,360) : 728,	(10,360) : 556,	(11,360) : 607,	(12,360) : 227,	(13,360) : 390,	(14,360) : 724,	(15,360) : 120,	(16,360) : 122,	(17,360) : 27,	(18,360) : 610,	(19,360) : 235,	(20,360) : 555,	(21,360) : 270,	(22,360) : 589,	(23,360) : 605,	(24,360) : 27,	(25,360) : 956,	(26,360) : 326,	(27,360) : 384,	(28,360) : 554,	(29,360) : 157,	(30,360) : 557,	(31,360) : 582,	(32,360) : 482,	(33,360) : 483,	(34,360) : 570,	(35,360) : 685,	(36,360) : 790,	(37,360) : 738,	(38,360) : 606,	(39,360) : 606,	(40,360) : 983,
(1,361) : 94,	(2,361) : 141,	(3,361) : 738,	(4,361) : 588,	(5,361) : 194,	(6,361) : 170,	(7,361) : 103,	(8,361) : 993,	(9,361) : 452,	(10,361) : 101,	(11,361) : 141,	(12,361) : 478,	(13,361) : 181,	(14,361) : 447,	(15,361) : 439,	(16,361) : 618,	(17,361) : 541,	(18,361) : 192,	(19,361) : 414,	(20,361) : 97,	(21,361) : 334,	(22,361) : 249,	(23,361) : 189,	(24,361) : 544,	(25,361) : 870,	(26,361) : 232,	(27,361) : 187,	(28,361) : 99,	(29,361) : 484,	(30,361) : 94,	(31,361) : 161,	(32,361) : 143,	(33,361) : 63,	(34,361) : 190,	(35,361) : 152,	(36,361) : 492,	(37,361) : 543,	(38,361) : 156,	(39,361) : 145,	(40,361) : 884,
(1,362) : 120,	(2,362) : 75,	(3,362) : 586,	(4,362) : 659,	(5,362) : 27,	(6,362) : 51,	(7,362) : 114,	(8,362) : 814,	(9,362) : 242,	(10,362) : 116,	(11,362) : 76,	(12,362) : 625,	(13,362) : 244,	(14,362) : 238,	(15,362) : 502,	(16,362) : 721,	(17,362) : 608,	(18,362) : 21,	(19,362) : 563,	(20,362) : 119,	(21,362) : 358,	(22,362) : 65,	(23,362) : 25,	(24,362) : 611,	(25,362) : 672,	(26,362) : 306,	(27,362) : 248,	(28,362) : 118,	(29,362) : 511,	(30,362) : 122,	(31,362) : 56,	(32,362) : 149,	(33,362) : 234,	(34,362) : 52,	(35,362) : 171,	(36,362) : 279,	(37,362) : 342,	(38,362) : 58,	(39,362) : 72,	(40,362) : 684,
(1,363) : 105,	(2,363) : 61,	(3,363) : 689,	(4,363) : 703,	(5,363) : 77,	(6,363) : 106,	(7,363) : 109,	(8,363) : 915,	(9,363) : 332,	(10,363) : 108,	(11,363) : 60,	(12,363) : 631,	(13,363) : 274,	(14,363) : 328,	(15,363) : 547,	(16,363) : 751,	(17,363) : 654,	(18,363) : 93,	(19,363) : 567,	(20,363) : 110,	(21,363) : 414,	(22,363) : 168,	(23,363) : 96,	(24,363) : 656,	(25,363) : 768,	(26,363) : 338,	(27,363) : 280,	(28,363) : 110,	(29,363) : 571,	(30,363) : 108,	(31,363) : 97,	(32,363) : 184,	(33,363) : 213,	(34,363) : 129,	(35,363) : 68,	(36,363) : 359,	(37,363) : 439,	(38,363) : 69,	(39,363) : 62,	(40,363) : 778,
(1,364) : 40,	(2,364) : 54,	(3,364) : 691,	(4,364) : 635,	(5,364) : 107,	(6,364) : 97,	(7,364) : 50,	(8,364) : 934,	(9,364) : 372,	(10,364) : 48,	(11,364) : 54,	(12,364) : 553,	(13,364) : 208,	(14,364) : 367,	(15,364) : 481,	(16,364) : 678,	(17,364) : 586,	(18,364) : 110,	(19,364) : 489,	(20,364) : 47,	(21,364) : 356,	(22,364) : 178,	(23,364) : 108,	(24,364) : 589,	(25,364) : 799,	(26,364) : 270,	(27,364) : 214,	(28,364) : 48,	(29,364) : 512,	(30,364) : 43,	(31,364) : 86,	(32,364) : 129,	(33,364) : 134,	(34,364) : 122,	(35,364) : 93,	(36,364) : 408,	(37,364) : 470,	(38,364) : 71,	(39,364) : 58,	(40,364) : 812,
(1,365) : 216,	(2,365) : 173,	(3,365) : 515,	(4,365) : 694,	(5,365) : 121,	(6,365) : 141,	(7,365) : 208,	(8,365) : 727,	(9,365) : 145,	(10,365) : 210,	(11,365) : 173,	(12,365) : 693,	(13,365) : 308,	(14,365) : 141,	(15,365) : 541,	(16,365) : 768,	(17,365) : 644,	(18,365) : 118,	(19,365) : 632,	(20,365) : 214,	(21,365) : 392,	(22,365) : 79,	(23,365) : 120,	(24,365) : 647,	(25,365) : 579,	(26,365) : 365,	(27,365) : 311,	(28,365) : 212,	(29,365) : 534,	(30,365) : 216,	(31,365) : 149,	(32,365) : 222,	(33,365) : 325,	(34,365) : 126,	(35,365) : 257,	(36,365) : 183,	(37,365) : 249,	(38,365) : 156,	(39,365) : 169,	(40,365) : 589,
(1,366) : 667,	(2,366) : 655,	(3,366) : 9,	(4,366) : 698,	(5,366) : 619,	(6,366) : 604,	(7,366) : 656,	(8,366) : 282,	(9,366) : 423,	(10,366) : 659,	(11,366) : 656,	(12,366) : 867,	(13,366) : 614,	(14,366) : 424,	(15,366) : 622,	(16,366) : 814,	(17,366) : 668,	(18,366) : 605,	(19,366) : 830,	(20,366) : 662,	(21,366) : 536,	(22,366) : 530,	(23,366) : 603,	(24,366) : 668,	(25,366) : 290,	(26,366) : 619,	(27,366) : 611,	(28,366) : 659,	(29,366) : 544,	(30,366) : 666,	(31,366) : 616,	(32,366) : 605,	(33,366) : 720,	(34,366) : 579,	(35,366) : 763,	(36,366) : 454,	(37,366) : 321,	(38,366) : 639,	(39,366) : 652,	(40,366) : 320,
(1,367) : 882,	(2,367) : 853,	(3,367) : 327,	(4,367) : 1018,	(5,367) : 806,	(6,367) : 808,	(7,367) : 871,	(8,367) : 110,	(9,367) : 548,	(10,367) : 874,	(11,367) : 854,	(12,367) : 1173,	(13,367) : 876,	(14,367) : 552,	(15,367) : 933,	(16,367) : 1132,	(17,367) : 985,	(18,367) : 797,	(19,367) : 1131,	(20,367) : 877,	(21,367) : 830,	(22,367) : 727,	(23,367) : 797,	(24,367) : 986,	(25,367) : 116,	(26,367) : 896,	(27,367) : 875,	(28,367) : 875,	(29,367) : 859,	(30,367) : 881,	(31,367) : 819,	(32,367) : 841,	(33,367) : 962,	(34,367) : 785,	(35,367) : 945,	(36,367) : 541,	(37,367) : 439,	(38,367) : 835,	(39,367) : 849,	(40,367) : 121,
(1,368) : 235,	(2,368) : 281,	(3,368) : 608,	(4,368) : 365,	(5,368) : 308,	(6,368) : 259,	(7,368) : 230,	(8,368) : 890,	(9,368) : 467,	(10,368) : 231,	(11,368) : 282,	(12,368) : 330,	(13,368) : 65,	(14,368) : 462,	(15,368) : 210,	(16,368) : 416,	(17,368) : 316,	(18,368) : 291,	(19,368) : 271,	(20,368) : 230,	(21,368) : 109,	(22,368) : 286,	(23,368) : 286,	(24,368) : 319,	(25,368) : 809,	(26,368) : 2,	(27,368) : 59,	(28,368) : 229,	(29,368) : 251,	(30,368) : 232,	(31,368) : 260,	(32,368) : 159,	(33,368) : 173,	(34,368) : 254,	(35,368) : 362,	(36,368) : 524,	(37,368) : 517,	(38,368) : 282,	(39,368) : 282,	(40,368) : 830,
(1,369) : 464,	(2,369) : 499,	(3,369) : 507,	(4,369) : 202,	(5,369) : 508,	(6,369) : 461,	(7,369) : 456,	(8,369) : 796,	(9,369) : 560,	(10,369) : 458,	(11,369) : 500,	(12,369) : 359,	(13,369) : 302,	(14,369) : 556,	(15,369) : 114,	(16,369) : 308,	(17,369) : 160,	(18,369) : 488,	(19,369) : 335,	(20,369) : 458,	(21,369) : 148,	(22,369) : 450,	(23,369) : 483,	(24,369) : 161,	(25,369) : 772,	(26,369) : 252,	(27,369) : 295,	(28,369) : 457,	(29,369) : 29,	(30,369) : 461,	(31,369) : 467,	(32,369) : 377,	(33,369) : 422,	(34,369) : 446,	(35,369) : 596,	(36,369) : 622,	(37,369) : 559,	(38,369) : 494,	(39,369) : 498,	(40,369) : 798,
(1,370) : 203,	(2,370) : 214,	(3,370) : 473,	(4,370) : 484,	(5,370) : 208,	(6,370) : 167,	(7,370) : 192,	(8,370) : 740,	(9,370) : 290,	(10,370) : 194,	(11,370) : 216,	(12,370) : 505,	(13,370) : 145,	(14,370) : 285,	(15,370) : 333,	(16,370) : 561,	(17,370) : 435,	(18,370) : 189,	(19,370) : 449,	(20,370) : 196,	(21,370) : 182,	(22,370) : 142,	(23,370) : 184,	(24,370) : 437,	(25,370) : 640,	(26,370) : 181,	(27,370) : 145,	(28,370) : 194,	(29,370) : 325,	(30,370) : 200,	(31,370) : 176,	(32,370) : 125,	(33,370) : 239,	(34,370) : 146,	(35,370) : 325,	(36,370) : 350,	(37,370) : 338,	(38,370) : 204,	(39,370) : 213,	(40,370) : 659,
(1,371) : 219,	(2,371) : 211,	(3,371) : 440,	(4,371) : 555,	(5,371) : 186,	(6,371) : 159,	(7,371) : 208,	(8,371) : 692,	(9,371) : 212,	(10,371) : 211,	(11,371) : 212,	(12,371) : 583,	(13,371) : 215,	(14,371) : 208,	(15,371) : 406,	(16,371) : 636,	(17,371) : 505,	(18,371) : 168,	(19,371) : 526,	(20,371) : 214,	(21,371) : 256,	(22,371) : 99,	(23,371) : 165,	(24,371) : 507,	(25,371) : 578,	(26,371) : 257,	(27,371) : 216,	(28,371) : 212,	(29,371) : 390,	(30,371) : 218,	(31,371) : 171,	(32,371) : 166,	(33,371) : 287,	(34,371) : 134,	(35,371) : 323,	(36,371) : 272,	(37,371) : 265,	(38,371) : 196,	(39,371) : 208,	(40,371) : 595,
(1,372) : 116,	(2,372) : 79,	(3,372) : 573,	(4,372) : 639,	(5,372) : 41,	(6,372) : 41,	(7,372) : 108,	(8,372) : 805,	(9,372) : 242,	(10,372) : 110,	(11,372) : 80,	(12,372) : 610,	(13,372) : 227,	(14,372) : 237,	(15,372) : 483,	(16,372) : 702,	(17,372) : 589,	(18,372) : 25,	(19,372) : 547,	(20,372) : 114,	(21,372) : 338,	(22,372) : 51,	(23,372) : 24,	(24,372) : 591,	(25,372) : 668,	(26,372) : 289,	(27,372) : 231,	(28,372) : 112,	(29,372) : 491,	(30,372) : 117,	(31,372) : 49,	(32,372) : 133,	(33,372) : 226,	(34,372) : 34,	(35,372) : 183,	(36,372) : 283,	(37,372) : 338,	(38,372) : 62,	(39,372) : 75,	(40,372) : 681,
(1,373) : 563,	(2,373) : 607,	(3,373) : 658,	(4,373) : 41,	(5,373) : 626,	(6,373) : 577,	(7,373) : 558,	(8,373) : 942,	(9,373) : 714,	(10,373) : 559,	(11,373) : 608,	(12,373) : 260,	(13,373) : 392,	(14,373) : 710,	(15,373) : 119,	(16,373) : 149,	(17,373) : 15,	(18,373) : 608,	(19,373) : 265,	(20,373) : 558,	(21,373) : 263,	(22,373) : 582,	(23,373) : 603,	(24,373) : 13,	(25,373) : 930,	(26,373) : 330,	(27,373) : 386,	(28,373) : 557,	(29,373) : 134,	(30,373) : 560,	(31,373) : 581,	(32,373) : 483,	(33,373) : 492,	(34,373) : 567,	(35,373) : 690,	(36,373) : 776,	(37,373) : 719,	(38,373) : 606,	(39,373) : 607,	(40,373) : 957,
(1,374) : 119,	(2,374) : 135,	(3,374) : 548,	(4,374) : 526,	(5,374) : 141,	(6,374) : 93,	(7,374) : 108,	(8,374) : 806,	(9,374) : 303,	(10,374) : 111,	(11,374) : 136,	(12,374) : 506,	(13,374) : 122,	(14,374) : 298,	(15,374) : 370,	(16,374) : 590,	(17,374) : 476,	(18,374) : 122,	(19,374) : 445,	(20,374) : 113,	(21,374) : 226,	(22,374) : 108,	(23,374) : 116,	(24,374) : 479,	(25,374) : 692,	(26,374) : 179,	(27,374) : 125,	(28,374) : 111,	(29,374) : 381,	(30,374) : 117,	(31,374) : 100,	(32,374) : 51,	(33,374) : 172,	(34,374) : 80,	(35,374) : 244,	(36,374) : 356,	(37,374) : 374,	(38,374) : 128,	(39,374) : 134,	(40,374) : 709,
(1,375) : 448,	(2,375) : 481,	(3,375) : 498,	(4,375) : 218,	(5,375) : 490,	(6,375) : 443,	(7,375) : 440,	(8,375) : 788,	(9,375) : 542,	(10,375) : 441,	(11,375) : 482,	(12,375) : 364,	(13,375) : 286,	(14,375) : 538,	(15,375) : 118,	(16,375) : 322,	(17,375) : 175,	(18,375) : 471,	(19,375) : 337,	(20,375) : 442,	(21,375) : 133,	(22,375) : 431,	(23,375) : 465,	(24,375) : 176,	(25,375) : 759,	(26,375) : 238,	(27,375) : 280,	(28,375) : 440,	(29,375) : 44,	(30,375) : 445,	(31,375) : 449,	(32,375) : 360,	(33,375) : 409,	(34,375) : 428,	(35,375) : 580,	(36,375) : 604,	(37,375) : 542,	(38,375) : 477,	(39,375) : 481,	(40,375) : 786,
(1,376) : 1072,	(2,376) : 1034,	(3,376) : 596,	(4,376) : 1282,	(5,376) : 983,	(6,376) : 996,	(7,376) : 1063,	(8,376) : 356,	(9,376) : 716,	(10,376) : 1065,	(11,376) : 1035,	(12,376) : 1422,	(13,376) : 1097,	(14,376) : 721,	(15,376) : 1188,	(16,376) : 1395,	(17,376) : 1247,	(18,376) : 979,	(19,376) : 1376,	(20,376) : 1069,	(21,376) : 1073,	(22,376) : 917,	(23,376) : 980,	(24,376) : 1248,	(25,376) : 320,	(26,376) : 1126,	(27,376) : 1097,	(28,376) : 1067,	(29,376) : 1119,	(30,376) : 1072,	(31,376) : 1006,	(32,376) : 1047,	(33,376) : 1167,	(34,376) : 975,	(35,376) : 1110,	(36,376) : 688,	(37,376) : 624,	(38,376) : 1017,	(39,376) : 1030,	(40,376) : 296,
(1,377) : 263,	(2,377) : 270,	(3,377) : 413,	(4,377) : 476,	(5,377) : 256,	(6,377) : 220,	(7,377) : 252,	(8,377) : 684,	(9,377) : 280,	(10,377) : 254,	(11,377) : 271,	(12,377) : 529,	(13,377) : 196,	(14,377) : 276,	(15,377) : 332,	(16,377) : 563,	(17,377) : 428,	(18,377) : 237,	(19,377) : 476,	(20,377) : 256,	(21,377) : 186,	(22,377) : 178,	(23,377) : 233,	(24,377) : 430,	(25,377) : 593,	(26,377) : 218,	(27,377) : 194,	(28,377) : 254,	(29,377) : 308,	(30,377) : 261,	(31,377) : 230,	(32,377) : 187,	(33,377) : 299,	(34,377) : 197,	(35,377) : 382,	(36,377) : 342,	(37,377) : 307,	(38,377) : 258,	(39,377) : 268,	(40,377) : 614,
(1,378) : 30,	(2,378) : 85,	(3,378) : 657,	(4,378) : 571,	(5,378) : 129,	(6,378) : 93,	(7,378) : 30,	(8,378) : 910,	(9,378) : 373,	(10,378) : 30,	(11,378) : 85,	(12,378) : 499,	(13,378) : 143,	(14,378) : 368,	(15,378) : 416,	(16,378) : 616,	(17,378) : 522,	(18,378) : 121,	(19,378) : 435,	(20,378) : 27,	(21,378) : 292,	(22,378) : 168,	(23,378) : 118,	(24,378) : 525,	(25,378) : 787,	(26,378) : 205,	(27,378) : 149,	(28,378) : 27,	(29,378) : 447,	(30,378) : 27,	(31,378) : 86,	(32,378) : 69,	(33,378) : 92,	(34,378) : 110,	(35,378) : 157,	(36,378) : 417,	(37,378) : 461,	(38,378) : 94,	(39,378) : 87,	(40,378) : 802,
(1,379) : 616,	(2,379) : 664,	(3,379) : 766,	(4,379) : 80,	(5,379) : 690,	(6,379) : 641,	(7,379) : 612,	(8,379) : 1051,	(9,379) : 804,	(10,379) : 612,	(11,379) : 665,	(12,379) : 199,	(13,379) : 448,	(14,379) : 800,	(15,379) : 192,	(16,379) : 41,	(17,379) : 107,	(18,379) : 673,	(19,379) : 229,	(20,379) : 611,	(21,379) : 342,	(22,379) : 657,	(23,379) : 668,	(24,379) : 107,	(25,379) : 1037,	(26,379) : 384,	(27,379) : 442,	(28,379) : 610,	(29,379) : 238,	(30,379) : 613,	(31,379) : 643,	(32,379) : 543,	(33,379) : 529,	(34,379) : 634,	(35,379) : 736,	(36,379) : 866,	(37,379) : 818,	(38,379) : 666,	(39,379) : 665,	(40,379) : 1064,
(1,380) : 190,	(2,380) : 155,	(3,380) : 506,	(4,380) : 653,	(5,380) : 109,	(6,380) : 115,	(7,380) : 181,	(8,380) : 731,	(9,380) : 168,	(10,380) : 184,	(11,380) : 156,	(12,380) : 653,	(13,380) : 268,	(14,380) : 163,	(15,380) : 500,	(16,380) : 727,	(17,380) : 603,	(18,380) : 100,	(19,380) : 592,	(20,380) : 187,	(21,380) : 350,	(22,380) : 40,	(23,380) : 100,	(24,380) : 606,	(25,380) : 591,	(26,380) : 324,	(27,380) : 272,	(28,380) : 185,	(29,380) : 494,	(30,380) : 191,	(31,380) : 124,	(32,380) : 186,	(33,380) : 293,	(34,380) : 95,	(35,380) : 252,	(36,380) : 214,	(37,380) : 262,	(38,380) : 138,	(39,380) : 151,	(40,380) : 604,
(1,381) : 319,	(2,381) : 336,	(3,381) : 410,	(4,381) : 398,	(5,381) : 331,	(6,381) : 290,	(7,381) : 308,	(8,381) : 694,	(9,381) : 358,	(10,381) : 311,	(11,381) : 338,	(12,381) : 477,	(13,381) : 205,	(14,381) : 354,	(15,381) : 261,	(16,381) : 490,	(17,381) : 350,	(18,381) : 312,	(19,381) : 430,	(20,381) : 312,	(21,381) : 128,	(22,381) : 259,	(23,381) : 307,	(24,381) : 352,	(25,381) : 626,	(26,381) : 200,	(27,381) : 201,	(28,381) : 310,	(29,381) : 226,	(30,381) : 316,	(31,381) : 299,	(32,381) : 233,	(33,381) : 327,	(34,381) : 270,	(35,381) : 446,	(36,381) : 420,	(37,381) : 369,	(38,381) : 327,	(39,381) : 335,	(40,381) : 650,
(1,382) : 539,	(2,382) : 578,	(3,382) : 584,	(4,382) : 108,	(5,382) : 593,	(6,382) : 545,	(7,382) : 533,	(8,382) : 869,	(9,382) : 657,	(10,382) : 534,	(11,382) : 580,	(12,382) : 317,	(13,382) : 370,	(14,382) : 654,	(15,382) : 112,	(16,382) : 222,	(17,382) : 78,	(18,382) : 574,	(19,382) : 310,	(20,382) : 534,	(21,382) : 226,	(22,382) : 540,	(23,382) : 568,	(24,382) : 77,	(25,382) : 858,	(26,382) : 312,	(27,382) : 364,	(28,382) : 532,	(29,382) : 74,	(30,382) : 536,	(31,382) : 549,	(32,382) : 455,	(33,382) : 482,	(34,382) : 532,	(35,382) : 670,	(36,382) : 720,	(37,382) : 656,	(38,382) : 576,	(39,382) : 578,	(40,382) : 886,
(1,383) : 498,	(2,383) : 540,	(3,383) : 611,	(4,383) : 110,	(5,383) : 558,	(6,383) : 510,	(7,383) : 492,	(8,383) : 899,	(9,383) : 646,	(10,383) : 493,	(11,383) : 541,	(12,383) : 267,	(13,383) : 327,	(14,383) : 642,	(15,383) : 57,	(16,383) : 205,	(17,383) : 61,	(18,383) : 540,	(19,383) : 254,	(20,383) : 493,	(21,383) : 194,	(22,383) : 513,	(23,383) : 535,	(24,383) : 63,	(25,383) : 876,	(26,383) : 266,	(27,383) : 321,	(28,383) : 492,	(29,383) : 76,	(30,383) : 495,	(31,383) : 514,	(32,383) : 416,	(33,383) : 433,	(34,383) : 499,	(35,383) : 627,	(36,383) : 708,	(37,383) : 655,	(38,383) : 539,	(39,383) : 540,	(40,383) : 903,
(1,384) : 413,	(2,384) : 448,	(3,384) : 503,	(4,384) : 235,	(5,384) : 457,	(6,384) : 410,	(7,384) : 406,	(8,384) : 794,	(9,384) : 520,	(10,384) : 407,	(11,384) : 449,	(12,384) : 355,	(13,384) : 252,	(14,384) : 516,	(15,384) : 112,	(16,384) : 332,	(17,384) : 189,	(18,384) : 438,	(19,384) : 322,	(20,384) : 407,	(21,384) : 98,	(22,384) : 401,	(23,384) : 433,	(24,384) : 190,	(25,384) : 756,	(26,384) : 204,	(27,384) : 246,	(28,384) : 406,	(29,384) : 66,	(30,384) : 410,	(31,384) : 416,	(32,384) : 326,	(33,384) : 374,	(34,384) : 396,	(35,384) : 545,	(36,384) : 582,	(37,384) : 527,	(38,384) : 443,	(39,384) : 447,	(40,384) : 782,
(1,385) : 217,	(2,385) : 188,	(3,385) : 467,	(4,385) : 636,	(5,385) : 146,	(6,385) : 142,	(7,385) : 207,	(8,385) : 696,	(9,385) : 152,	(10,385) : 209,	(11,385) : 189,	(12,385) : 650,	(13,385) : 270,	(14,385) : 147,	(15,385) : 485,	(16,385) : 714,	(17,385) : 586,	(18,385) : 134,	(19,385) : 591,	(20,385) : 213,	(21,385) : 335,	(22,385) : 61,	(23,385) : 133,	(24,385) : 588,	(25,385) : 561,	(26,385) : 321,	(27,385) : 272,	(28,385) : 211,	(29,385) : 472,	(30,385) : 217,	(31,385) : 153,	(32,385) : 196,	(33,385) : 311,	(34,385) : 119,	(35,385) : 290,	(36,385) : 205,	(37,385) : 234,	(38,385) : 171,	(39,385) : 184,	(40,385) : 576,
(1,386) : 148,	(2,386) : 201,	(3,386) : 655,	(4,386) : 459,	(5,386) : 238,	(6,386) : 193,	(7,386) : 146,	(8,386) : 927,	(9,386) : 444,	(10,386) : 146,	(11,386) : 201,	(12,386) : 383,	(13,386) : 54,	(14,386) : 440,	(15,386) : 306,	(16,386) : 499,	(17,386) : 411,	(18,386) : 225,	(19,386) : 320,	(20,386) : 144,	(21,386) : 202,	(22,386) : 244,	(23,386) : 220,	(24,386) : 413,	(25,386) : 826,	(26,386) : 97,	(27,386) : 59,	(28,386) : 144,	(29,386) : 350,	(30,386) : 145,	(31,386) : 190,	(32,386) : 99,	(33,386) : 76,	(34,386) : 197,	(35,386) : 267,	(36,386) : 497,	(37,386) : 514,	(38,386) : 206,	(39,386) : 202,	(40,386) : 844,
(1,387) : 551,	(2,387) : 607,	(3,387) : 943,	(4,387) : 356,	(5,387) : 650,	(6,387) : 607,	(7,387) : 552,	(8,387) : 1233,	(9,387) : 848,	(10,387) : 552,	(11,387) : 607,	(12,387) : 99,	(13,387) : 429,	(14,387) : 843,	(15,387) : 338,	(16,387) : 283,	(17,387) : 347,	(18,387) : 639,	(19,387) : 122,	(20,387) : 549,	(21,387) : 424,	(22,387) : 656,	(23,387) : 634,	(24,387) : 348,	(25,387) : 1177,	(26,387) : 381,	(27,387) : 426,	(28,387) : 550,	(29,387) : 427,	(30,387) : 549,	(31,387) : 603,	(32,387) : 513,	(33,387) : 436,	(34,387) : 612,	(35,387) : 636,	(36,387) : 904,	(37,387) : 898,	(38,387) : 616,	(39,387) : 609,	(40,387) : 1200,
(1,388) : 203,	(2,388) : 202,	(3,388) : 458,	(4,388) : 529,	(5,388) : 185,	(6,388) : 151,	(7,388) : 192,	(8,388) : 715,	(9,388) : 243,	(10,388) : 195,	(11,388) : 204,	(12,388) : 552,	(13,388) : 184,	(14,388) : 238,	(15,388) : 379,	(16,388) : 608,	(17,388) : 480,	(18,388) : 166,	(19,388) : 495,	(20,388) : 197,	(21,388) : 228,	(22,388) : 106,	(23,388) : 162,	(24,388) : 482,	(25,388) : 606,	(26,388) : 226,	(27,388) : 185,	(28,388) : 195,	(29,388) : 367,	(30,388) : 202,	(31,388) : 162,	(32,388) : 141,	(33,388) : 262,	(34,388) : 127,	(35,388) : 315,	(36,388) : 302,	(37,388) : 296,	(38,388) : 189,	(39,388) : 200,	(40,388) : 624,
(1,389) : 86,	(2,389) : 138,	(3,389) : 648,	(4,389) : 516,	(5,389) : 177,	(6,389) : 133,	(7,389) : 83,	(8,389) : 910,	(9,389) : 399,	(10,389) : 84,	(11,389) : 139,	(12,389) : 445,	(13,389) : 90,	(14,389) : 394,	(15,389) : 362,	(16,389) : 560,	(17,389) : 467,	(18,389) : 165,	(19,389) : 382,	(20,389) : 82,	(21,389) : 243,	(22,389) : 194,	(23,389) : 161,	(24,389) : 470,	(25,389) : 798,	(26,389) : 151,	(27,389) : 96,	(28,389) : 81,	(29,389) : 397,	(30,389) : 83,	(31,389) : 130,	(32,389) : 55,	(33,389) : 66,	(34,389) : 142,	(35,389) : 210,	(36,389) : 448,	(37,389) : 478,	(38,389) : 144,	(39,389) : 140,	(40,389) : 815,
(1,390) : 648,	(2,390) : 697,	(3,390) : 807,	(4,390) : 117,	(5,390) : 725,	(6,390) : 676,	(7,390) : 644,	(8,390) : 1091,	(9,390) : 844,	(10,390) : 645,	(11,390) : 698,	(12,390) : 199,	(13,390) : 482,	(14,390) : 840,	(15,390) : 232,	(16,390) : 2,	(17,390) : 149,	(18,390) : 708,	(19,390) : 239,	(20,390) : 643,	(21,390) : 381,	(22,390) : 694,	(23,390) : 703,	(24,390) : 148,	(25,390) : 1079,	(26,390) : 418,	(27,390) : 476,	(28,390) : 643,	(29,390) : 280,	(30,390) : 645,	(31,390) : 678,	(32,390) : 577,	(33,390) : 557,	(34,390) : 670,	(35,390) : 766,	(36,390) : 906,	(37,390) : 859,	(38,390) : 699,	(39,390) : 698,	(40,390) : 1106,
(1,391) : 36,	(2,391) : 71,	(3,391) : 695,	(4,391) : 619,	(5,391) : 123,	(6,391) : 106,	(7,391) : 47,	(8,391) : 941,	(9,391) : 386,	(10,391) : 45,	(11,391) : 70,	(12,391) : 532,	(13,391) : 193,	(14,391) : 381,	(15,391) : 464,	(16,391) : 659,	(17,391) : 570,	(18,391) : 124,	(19,391) : 469,	(20,391) : 42,	(21,391) : 343,	(22,391) : 187,	(23,391) : 122,	(24,391) : 573,	(25,391) : 810,	(26,391) : 253,	(27,391) : 199,	(28,391) : 44,	(29,391) : 498,	(30,391) : 38,	(31,391) : 96,	(32,391) : 120,	(33,391) : 113,	(34,391) : 130,	(35,391) : 107,	(36,391) : 423,	(37,391) : 481,	(38,391) : 87,	(39,391) : 74,	(40,391) : 824,
(1,392) : 201,	(2,392) : 213,	(3,392) : 474,	(4,392) : 486,	(5,392) : 206,	(6,392) : 165,	(7,392) : 190,	(8,392) : 740,	(9,392) : 289,	(10,392) : 193,	(11,392) : 214,	(12,392) : 507,	(13,392) : 146,	(14,392) : 284,	(15,392) : 334,	(16,392) : 563,	(17,392) : 436,	(18,392) : 187,	(19,392) : 450,	(20,392) : 195,	(21,392) : 184,	(22,392) : 140,	(23,392) : 182,	(24,392) : 438,	(25,392) : 640,	(26,392) : 182,	(27,392) : 145,	(28,392) : 193,	(29,392) : 327,	(30,392) : 199,	(31,392) : 174,	(32,392) : 124,	(33,392) : 238,	(34,392) : 145,	(35,392) : 324,	(36,392) : 349,	(37,392) : 337,	(38,392) : 203,	(39,392) : 211,	(40,392) : 659,
(1,393) : 405,	(2,393) : 447,	(3,393) : 580,	(4,393) : 201,	(5,393) : 465,	(6,393) : 416,	(7,393) : 399,	(8,393) : 870,	(9,393) : 565,	(10,393) : 400,	(11,393) : 448,	(12,393) : 278,	(13,393) : 235,	(14,393) : 560,	(15,393) : 47,	(16,393) : 279,	(17,393) : 151,	(18,393) : 447,	(19,393) : 244,	(20,393) : 400,	(21,393) : 103,	(22,393) : 422,	(23,393) : 441,	(24,393) : 153,	(25,393) : 828,	(26,393) : 175,	(27,393) : 229,	(28,393) : 399,	(29,393) : 87,	(30,393) : 402,	(31,393) : 420,	(32,393) : 323,	(33,393) : 345,	(34,393) : 406,	(35,393) : 534,	(36,393) : 626,	(37,393) : 583,	(38,393) : 445,	(39,393) : 447,	(40,393) : 853,
(1,394) : 480,	(2,394) : 534,	(3,394) : 830,	(4,394) : 268,	(5,394) : 573,	(6,394) : 527,	(7,394) : 479,	(8,394) : 1120,	(9,394) : 751,	(10,394) : 479,	(11,394) : 535,	(12,394) : 49,	(13,394) : 338,	(14,394) : 746,	(15,394) : 226,	(16,394) : 227,	(17,394) : 249,	(18,394) : 560,	(19,394) : 15,	(20,394) : 477,	(21,394) : 313,	(22,394) : 568,	(23,394) : 555,	(24,394) : 251,	(25,394) : 1067,	(26,394) : 284,	(27,394) : 335,	(28,394) : 477,	(29,394) : 316,	(30,394) : 477,	(31,394) : 525,	(32,394) : 429,	(33,394) : 372,	(34,394) : 529,	(35,394) : 581,	(36,394) : 809,	(37,394) : 795,	(38,394) : 541,	(39,394) : 536,	(40,394) : 1091,
(1,395) : 9,	(2,395) : 61,	(3,395) : 653,	(4,395) : 592,	(5,395) : 106,	(6,395) : 74,	(7,395) : 7,	(8,395) : 902,	(9,395) : 356,	(10,395) : 6,	(11,395) : 61,	(12,395) : 523,	(13,395) : 163,	(14,395) : 352,	(15,395) : 436,	(16,395) : 639,	(17,395) : 542,	(18,395) : 100,	(19,395) : 459,	(20,395) : 3,	(21,395) : 308,	(22,395) : 153,	(23,395) : 97,	(24,395) : 545,	(25,395) : 775,	(26,395) : 226,	(27,395) : 169,	(28,395) : 4,	(29,395) : 464,	(30,395) : 6,	(31,395) : 66,	(32,395) : 80,	(33,395) : 114,	(34,395) : 94,	(35,395) : 141,	(36,395) : 398,	(37,395) : 447,	(38,395) : 70,	(39,395) : 63,	(40,395) : 789,
(1,396) : 106,	(2,396) : 103,	(3,396) : 554,	(4,396) : 572,	(5,396) : 97,	(6,396) : 54,	(7,396) : 95,	(8,396) : 802,	(9,396) : 273,	(10,396) : 97,	(11,396) : 104,	(12,396) : 549,	(13,396) : 164,	(14,396) : 268,	(15,396) : 416,	(16,396) : 636,	(17,396) : 522,	(18,396) : 78,	(19,396) : 487,	(20,396) : 100,	(21,396) : 271,	(22,396) : 69,	(23,396) : 73,	(24,396) : 524,	(25,396) : 679,	(26,396) : 224,	(27,396) : 168,	(28,396) : 98,	(29,396) : 425,	(30,396) : 105,	(31,396) : 63,	(32,396) : 77,	(33,396) : 189,	(34,396) : 35,	(35,396) : 215,	(36,396) : 322,	(37,396) : 354,	(38,396) : 92,	(39,396) : 101,	(40,396) : 694,
(1,397) : 269,	(2,397) : 289,	(3,397) : 450,	(4,397) : 412,	(5,397) : 287,	(6,397) : 244,	(7,397) : 259,	(8,397) : 729,	(9,397) : 346,	(10,397) : 261,	(11,397) : 290,	(12,397) : 463,	(13,397) : 158,	(14,397) : 342,	(15,397) : 266,	(16,397) : 497,	(17,397) : 363,	(18,397) : 268,	(19,397) : 412,	(20,397) : 262,	(21,397) : 118,	(22,397) : 221,	(23,397) : 263,	(24,397) : 365,	(25,397) : 650,	(26,397) : 163,	(27,397) : 155,	(28,397) : 260,	(29,397) : 248,	(30,397) : 266,	(31,397) : 252,	(32,397) : 183,	(33,397) : 277,	(34,397) : 225,	(35,397) : 397,	(36,397) : 408,	(37,397) : 373,	(38,397) : 281,	(39,397) : 288,	(40,397) : 672,
(1,398) : 13,	(2,398) : 56,	(3,398) : 647,	(4,398) : 593,	(5,398) : 99,	(6,398) : 67,	(7,398) : 1,	(8,398) : 895,	(9,398) : 349,	(10,398) : 4,	(11,398) : 56,	(12,398) : 527,	(13,398) : 164,	(14,398) : 344,	(15,398) : 437,	(16,398) : 642,	(17,398) : 544,	(18,398) : 93,	(19,398) : 464,	(20,398) : 7,	(21,398) : 308,	(22,398) : 145,	(23,398) : 89,	(24,398) : 546,	(25,398) : 767,	(26,398) : 228,	(27,398) : 170,	(28,398) : 5,	(29,398) : 464,	(30,398) : 11,	(31,398) : 59,	(32,398) : 78,	(33,398) : 121,	(34,398) : 87,	(35,398) : 142,	(36,398) : 390,	(37,398) : 440,	(38,398) : 64,	(39,398) : 57,	(40,398) : 782,
(1,399) : 762,	(2,399) : 728,	(3,399) : 318,	(4,399) : 977,	(5,399) : 679,	(6,399) : 687,	(7,399) : 752,	(8,399) : 241,	(9,399) : 415,	(10,399) : 754,	(11,399) : 729,	(12,399) : 1103,	(13,399) : 777,	(14,399) : 419,	(15,399) : 873,	(16,399) : 1087,	(17,399) : 939,	(18,399) : 672,	(19,399) : 1056,	(20,399) : 758,	(21,399) : 754,	(22,399) : 606,	(23,399) : 673,	(24,399) : 940,	(25,399) : 41,	(26,399) : 806,	(27,399) : 777,	(28,399) : 756,	(29,399) : 809,	(30,399) : 762,	(31,399) : 697,	(32,399) : 731,	(33,399) : 851,	(34,399) : 664,	(35,399) : 813,	(36,399) : 399,	(37,399) : 312,	(38,399) : 710,	(39,399) : 724,	(40,399) : 33,
(1,400) : 204,	(2,400) : 193,	(3,400) : 456,	(4,400) : 567,	(5,400) : 166,	(6,400) : 142,	(7,400) : 193,	(8,400) : 704,	(9,400) : 208,	(10,400) : 196,	(11,400) : 194,	(12,400) : 588,	(13,400) : 214,	(14,400) : 203,	(15,400) : 417,	(16,400) : 646,	(17,400) : 518,	(18,400) : 149,	(19,400) : 530,	(20,400) : 199,	(21,400) : 267,	(22,400) : 79,	(23,400) : 145,	(24,400) : 520,	(25,400) : 586,	(26,400) : 260,	(27,400) : 216,	(28,400) : 196,	(29,400) : 405,	(30,400) : 203,	(31,400) : 153,	(32,400) : 157,	(33,400) : 278,	(34,400) : 116,	(35,400) : 304,	(36,400) : 266,	(37,400) : 269,	(38,400) : 178,	(39,400) : 190,	(40,400) : 603,
(1,401) : 331,	(2,401) : 291,	(3,401) : 428,	(4,401) : 736,	(5,401) : 241,	(6,401) : 255,	(7,401) : 322,	(8,401) : 616,	(9,401) : 27,	(10,401) : 325,	(11,401) : 292,	(12,401) : 771,	(13,401) : 395,	(14,401) : 22,	(15,401) : 592,	(16,401) : 823,	(17,401) : 688,	(18,401) : 236,	(19,401) : 713,	(20,401) : 328,	(21,401) : 443,	(22,401) : 179,	(23,401) : 238,	(24,401) : 690,	(25,401) : 461,	(26,401) : 443,	(27,401) : 397,	(28,401) : 326,	(29,401) : 566,	(30,401) : 332,	(31,401) : 265,	(32,401) : 322,	(33,401) : 434,	(34,401) : 236,	(35,401) : 375,	(36,401) : 83,	(37,401) : 134,	(38,401) : 274,	(39,401) : 288,	(40,401) : 471,
(1,402) : 230,	(2,402) : 256,	(3,402) : 492,	(4,402) : 417,	(5,402) : 261,	(6,402) : 215,	(7,402) : 221,	(8,402) : 770,	(9,402) : 356,	(10,402) : 223,	(11,402) : 257,	(12,402) : 442,	(13,402) : 113,	(14,402) : 352,	(15,402) : 264,	(16,402) : 493,	(17,402) : 367,	(18,402) : 242,	(19,402) : 387,	(20,402) : 224,	(21,402) : 114,	(22,402) : 206,	(23,402) : 236,	(24,402) : 369,	(25,402) : 684,	(26,402) : 126,	(27,402) : 110,	(28,402) : 222,	(29,402) : 262,	(30,402) : 228,	(31,402) : 222,	(32,402) : 143,	(33,402) : 232,	(34,402) : 199,	(35,402) : 361,	(36,402) : 417,	(37,402) : 396,	(38,402) : 250,	(39,402) : 255,	(40,402) : 705,
(1,403) : 96,	(2,403) : 137,	(3,403) : 749,	(4,403) : 611,	(5,403) : 189,	(6,403) : 171,	(7,403) : 106,	(8,403) : 1001,	(9,403) : 452,	(10,403) : 104,	(11,403) : 136,	(12,403) : 498,	(13,403) : 202,	(14,403) : 447,	(15,403) : 461,	(16,403) : 640,	(17,403) : 564,	(18,403) : 190,	(19,403) : 435,	(20,403) : 100,	(21,403) : 356,	(22,403) : 252,	(23,403) : 188,	(24,403) : 566,	(25,403) : 874,	(26,403) : 254,	(27,403) : 208,	(28,403) : 102,	(29,403) : 506,	(30,403) : 96,	(31,403) : 161,	(32,403) : 157,	(33,403) : 86,	(34,403) : 193,	(35,403) : 132,	(36,403) : 490,	(37,403) : 546,	(38,403) : 153,	(39,403) : 140,	(40,403) : 888,
(1,404) : 492,	(2,404) : 536,	(3,404) : 632,	(4,404) : 109,	(5,404) : 556,	(6,404) : 507,	(7,404) : 486,	(8,404) : 921,	(9,404) : 654,	(10,404) : 487,	(11,404) : 536,	(12,404) : 242,	(13,404) : 321,	(14,404) : 649,	(15,404) : 48,	(16,404) : 192,	(17,404) : 59,	(18,404) : 538,	(19,404) : 230,	(20,404) : 486,	(21,404) : 195,	(22,404) : 515,	(23,404) : 533,	(24,404) : 62,	(25,404) : 894,	(26,404) : 258,	(27,404) : 315,	(28,404) : 486,	(29,404) : 96,	(30,404) : 489,	(31,404) : 510,	(32,404) : 412,	(33,404) : 422,	(34,404) : 497,	(35,404) : 619,	(36,404) : 716,	(37,404) : 666,	(38,404) : 535,	(39,404) : 536,	(40,404) : 920,
(1,405) : 11,	(2,405) : 47,	(3,405) : 652,	(4,405) : 604,	(5,405) : 93,	(6,405) : 66,	(7,405) : 9,	(8,405) : 898,	(9,405) : 347,	(10,405) : 9,	(11,405) : 47,	(12,405) : 536,	(13,405) : 175,	(14,405) : 343,	(15,405) : 448,	(16,405) : 652,	(17,405) : 554,	(18,405) : 89,	(19,405) : 473,	(20,405) : 11,	(21,405) : 318,	(22,405) : 146,	(23,405) : 85,	(24,405) : 557,	(25,405) : 768,	(26,405) : 238,	(27,405) : 181,	(28,405) : 11,	(29,405) : 475,	(30,405) : 12,	(31,405) : 56,	(32,405) : 89,	(33,405) : 127,	(34,405) : 88,	(35,405) : 132,	(36,405) : 388,	(37,405) : 440,	(38,405) : 57,	(39,405) : 49,	(40,405) : 782,
(1,406) : 584,	(2,406) : 627,	(3,406) : 654,	(4,406) : 37,	(5,406) : 645,	(6,406) : 597,	(7,406) : 579,	(8,406) : 937,	(9,406) : 726,	(10,406) : 580,	(11,406) : 628,	(12,406) : 282,	(13,406) : 414,	(14,406) : 722,	(15,406) : 141,	(16,406) : 154,	(17,406) : 41,	(18,406) : 627,	(19,406) : 289,	(20,406) : 579,	(21,406) : 280,	(22,406) : 599,	(23,406) : 622,	(24,406) : 38,	(25,406) : 930,	(26,406) : 352,	(27,406) : 407,	(28,406) : 578,	(29,406) : 142,	(30,406) : 581,	(31,406) : 601,	(32,406) : 503,	(33,406) : 515,	(34,406) : 586,	(35,406) : 712,	(36,406) : 788,	(37,406) : 727,	(38,406) : 626,	(39,406) : 627,	(40,406) : 958,
(1,407) : 518,	(2,407) : 556,	(3,407) : 563,	(4,407) : 133,	(5,407) : 569,	(6,407) : 521,	(7,407) : 511,	(8,407) : 849,	(9,407) : 631,	(10,407) : 512,	(11,407) : 557,	(12,407) : 326,	(13,407) : 350,	(14,407) : 627,	(15,407) : 103,	(16,407) : 244,	(17,407) : 96,	(18,407) : 550,	(19,407) : 313,	(20,407) : 512,	(21,407) : 203,	(22,407) : 516,	(23,407) : 545,	(24,407) : 97,	(25,407) : 835,	(26,407) : 293,	(27,407) : 344,	(28,407) : 511,	(29,407) : 49,	(30,407) : 515,	(31,407) : 526,	(32,407) : 433,	(33,407) : 464,	(34,407) : 508,	(35,407) : 649,	(36,407) : 693,	(37,407) : 630,	(38,407) : 553,	(39,407) : 556,	(40,407) : 862,
(1,408) : 89,	(2,408) : 130,	(3,408) : 607,	(4,408) : 514,	(5,408) : 158,	(6,408) : 111,	(7,408) : 82,	(8,408) : 869,	(9,408) : 362,	(10,408) : 83,	(11,408) : 131,	(12,408) : 465,	(13,408) : 87,	(14,408) : 358,	(15,408) : 358,	(16,408) : 567,	(17,408) : 465,	(18,408) : 144,	(19,408) : 402,	(20,408) : 83,	(21,408) : 228,	(22,408) : 160,	(23,408) : 138,	(24,408) : 467,	(25,408) : 757,	(26,408) : 151,	(27,408) : 92,	(28,408) : 82,	(29,408) : 384,	(30,408) : 86,	(31,408) : 110,	(32,408) : 14,	(33,408) : 108,	(34,408) : 113,	(35,408) : 221,	(36,408) : 413,	(37,408) : 438,	(38,408) : 131,	(39,408) : 131,	(40,408) : 774,
(1,409) : 505,	(2,409) : 546,	(3,409) : 602,	(4,409) : 110,	(5,409) : 563,	(6,409) : 515,	(7,409) : 499,	(8,409) : 890,	(9,409) : 645,	(10,409) : 500,	(11,409) : 547,	(12,409) : 280,	(13,409) : 335,	(14,409) : 641,	(15,409) : 68,	(16,409) : 210,	(17,409) : 63,	(18,409) : 545,	(19,409) : 268,	(20,409) : 499,	(21,409) : 198,	(22,409) : 516,	(23,409) : 540,	(24,409) : 65,	(25,409) : 869,	(26,409) : 274,	(27,409) : 328,	(28,409) : 498,	(29,409) : 69,	(30,409) : 502,	(31,409) : 519,	(32,409) : 422,	(33,409) : 442,	(34,409) : 504,	(35,409) : 634,	(36,409) : 707,	(37,409) : 652,	(38,409) : 545,	(39,409) : 546,	(40,409) : 896,
(1,410) : 523,	(2,410) : 567,	(3,410) : 656,	(4,410) : 77,	(5,410) : 589,	(6,410) : 540,	(7,410) : 518,	(8,410) : 944,	(9,410) : 687,	(10,410) : 519,	(11,410) : 568,	(12,410) : 233,	(13,410) : 352,	(14,410) : 683,	(15,410) : 80,	(16,410) : 160,	(17,410) : 29,	(18,410) : 571,	(19,410) : 229,	(20,410) : 518,	(21,410) : 229,	(22,410) : 548,	(23,410) : 565,	(24,410) : 32,	(25,410) : 922,	(26,410) : 289,	(27,410) : 346,	(28,410) : 517,	(29,410) : 122,	(30,410) : 520,	(31,410) : 543,	(32,410) : 444,	(33,410) : 449,	(34,410) : 531,	(35,410) : 649,	(36,410) : 750,	(37,410) : 699,	(38,410) : 567,	(39,410) : 568,	(40,410) : 949,
(1,411) : 186,	(2,411) : 241,	(3,411) : 763,	(4,411) : 505,	(5,411) : 289,	(6,411) : 255,	(7,411) : 191,	(8,411) : 1034,	(9,411) : 529,	(10,411) : 189,	(11,411) : 241,	(12,411) : 370,	(13,411) : 161,	(14,411) : 525,	(15,411) : 365,	(16,411) : 521,	(17,411) : 461,	(18,411) : 283,	(19,411) : 307,	(20,411) : 186,	(21,411) : 293,	(22,411) : 323,	(23,411) : 279,	(24,411) : 463,	(25,411) : 927,	(26,411) : 182,	(27,411) : 164,	(28,411) : 187,	(29,411) : 426,	(30,411) : 184,	(31,411) : 248,	(32,411) : 185,	(33,411) : 70,	(34,411) : 268,	(35,411) : 261,	(36,411) : 576,	(37,411) : 609,	(38,411) : 253,	(39,411) : 243,	(40,411) : 944,
(1,412) : 147,	(2,412) : 140,	(3,412) : 512,	(4,412) : 566,	(5,412) : 123,	(6,412) : 89,	(7,412) : 136,	(8,412) : 761,	(9,412) : 244,	(10,412) : 139,	(11,412) : 141,	(12,412) : 562,	(13,412) : 179,	(14,412) : 239,	(15,412) : 412,	(16,412) : 637,	(17,412) : 516,	(18,412) : 104,	(19,412) : 502,	(20,412) : 142,	(21,412) : 263,	(22,412) : 56,	(23,412) : 99,	(24,412) : 519,	(25,412) : 640,	(26,412) : 233,	(27,412) : 182,	(28,412) : 140,	(29,412) : 412,	(30,412) : 146,	(31,412) : 100,	(32,412) : 106,	(33,412) : 225,	(34,412) : 64,	(35,412) : 253,	(36,412) : 297,	(37,412) : 318,	(38,412) : 127,	(39,412) : 138,	(40,412) : 656,
(1,413) : 253,	(2,413) : 214,	(3,413) : 473,	(4,413) : 693,	(5,413) : 164,	(6,413) : 177,	(7,413) : 244,	(8,413) : 682,	(9,413) : 105,	(10,413) : 246,	(11,413) : 214,	(12,413) : 708,	(13,413) : 326,	(14,413) : 100,	(15,413) : 543,	(16,413) : 773,	(17,413) : 644,	(18,413) : 158,	(19,413) : 648,	(20,413) : 250,	(21,413) : 393,	(22,413) : 103,	(23,413) : 159,	(24,413) : 646,	(25,413) : 535,	(26,413) : 379,	(27,413) : 329,	(28,413) : 248,	(29,413) : 528,	(30,413) : 253,	(31,413) : 186,	(32,413) : 247,	(33,413) : 357,	(34,413) : 159,	(35,413) : 302,	(36,413) : 151,	(37,413) : 205,	(38,413) : 196,	(39,413) : 210,	(40,413) : 546,
(1,414) : 196,	(2,414) : 191,	(3,414) : 463,	(4,414) : 546,	(5,414) : 171,	(6,414) : 140,	(7,414) : 185,	(8,414) : 717,	(9,414) : 231,	(10,414) : 188,	(11,414) : 193,	(12,414) : 565,	(13,414) : 193,	(14,414) : 226,	(15,414) : 395,	(16,414) : 624,	(17,414) : 496,	(18,414) : 152,	(19,414) : 507,	(20,414) : 191,	(21,414) : 244,	(22,414) : 89,	(23,414) : 148,	(24,414) : 498,	(25,414) : 603,	(26,414) : 237,	(27,414) : 194,	(28,414) : 188,	(29,414) : 384,	(30,414) : 195,	(31,414) : 151,	(32,414) : 141,	(33,414) : 262,	(34,414) : 115,	(35,414) : 304,	(36,414) : 289,	(37,414) : 289,	(38,414) : 178,	(39,414) : 189,	(40,414) : 620,
(1,415) : 594,	(2,415) : 649,	(3,415) : 897,	(4,415) : 256,	(5,415) : 686,	(6,415) : 640,	(7,415) : 594,	(8,415) : 1186,	(9,415) : 855,	(10,415) : 594,	(11,415) : 649,	(12,415) : 66,	(13,415) : 448,	(14,415) : 850,	(15,415) : 282,	(16,415) : 168,	(17,415) : 259,	(18,415) : 673,	(19,415) : 130,	(20,415) : 591,	(21,415) : 402,	(22,415) : 677,	(23,415) : 667,	(24,415) : 260,	(25,415) : 1148,	(26,415) : 391,	(27,415) : 445,	(28,415) : 591,	(29,415) : 363,	(30,415) : 592,	(31,415) : 638,	(32,415) : 541,	(33,415) : 487,	(34,415) : 640,	(35,415) : 695,	(36,415) : 914,	(37,415) : 891,	(38,415) : 655,	(39,415) : 650,	(40,415) : 1173,
(1,416) : 87,	(2,416) : 55,	(3,416) : 698,	(4,416) : 687,	(5,416) : 88,	(6,416) : 106,	(7,416) : 93,	(8,416) : 929,	(9,416) : 351,	(10,416) : 92,	(11,416) : 54,	(12,416) : 609,	(13,416) : 258,	(14,416) : 346,	(15,416) : 532,	(16,416) : 733,	(17,416) : 638,	(18,416) : 101,	(19,416) : 545,	(20,416) : 93,	(21,416) : 402,	(22,416) : 176,	(23,416) : 102,	(24,416) : 640,	(25,416) : 785,	(26,416) : 321,	(27,416) : 264,	(28,416) : 94,	(29,416) : 558,	(30,416) : 90,	(31,416) : 95,	(32,416) : 172,	(33,416) : 189,	(34,416) : 131,	(35,416) : 58,	(36,416) : 379,	(37,416) : 455,	(38,416) : 68,	(39,416) : 57,	(40,416) : 796,
(1,417) : 469,	(2,417) : 501,	(3,417) : 487,	(4,417) : 217,	(5,417) : 508,	(6,417) : 462,	(7,417) : 461,	(8,417) : 775,	(9,417) : 550,	(10,417) : 462,	(11,417) : 502,	(12,417) : 380,	(13,417) : 309,	(14,417) : 546,	(15,417) : 135,	(16,417) : 326,	(17,417) : 178,	(18,417) : 489,	(19,417) : 356,	(20,417) : 463,	(21,417) : 155,	(22,417) : 447,	(23,417) : 483,	(24,417) : 178,	(25,417) : 753,	(26,417) : 262,	(27,417) : 303,	(28,417) : 461,	(29,417) : 49,	(30,417) : 466,	(31,417) : 468,	(32,417) : 381,	(33,417) : 432,	(34,417) : 446,	(35,417) : 601,	(36,417) : 612,	(37,417) : 545,	(38,417) : 496,	(39,417) : 500,	(40,417) : 780,
(1,418) : 127,	(2,418) : 119,	(3,418) : 533,	(4,418) : 574,	(5,418) : 105,	(6,418) : 68,	(7,418) : 116,	(8,418) : 780,	(9,418) : 253,	(10,418) : 119,	(11,418) : 121,	(12,418) : 560,	(13,418) : 176,	(14,418) : 248,	(15,418) : 419,	(16,418) : 642,	(17,418) : 524,	(18,418) : 86,	(19,418) : 499,	(20,418) : 122,	(21,418) : 272,	(22,418) : 54,	(23,418) : 81,	(24,418) : 526,	(25,418) : 656,	(26,418) : 233,	(27,418) : 179,	(28,418) : 120,	(29,418) : 423,	(30,418) : 126,	(31,418) : 79,	(32,418) : 95,	(33,418) : 210,	(34,418) : 44,	(35,418) : 232,	(36,418) : 304,	(37,418) : 333,	(38,418) : 106,	(39,418) : 117,	(40,418) : 672,
(1,419) : 608,	(2,419) : 576,	(3,419) : 257,	(4,419) : 849,	(5,419) : 528,	(6,419) : 533,	(7,419) : 598,	(8,419) : 335,	(9,419) : 268,	(10,419) : 601,	(11,419) : 577,	(12,419) : 959,	(13,419) : 624,	(14,419) : 272,	(15,419) : 735,	(16,419) : 956,	(17,419) : 808,	(18,419) : 520,	(19,419) : 909,	(20,419) : 604,	(21,419) : 609,	(22,419) : 453,	(23,419) : 521,	(24,419) : 809,	(25,419) : 171,	(26,419) : 654,	(27,419) : 624,	(28,419) : 602,	(29,419) : 677,	(30,419) : 608,	(31,419) : 544,	(32,419) : 576,	(33,419) : 696,	(34,419) : 511,	(35,419) : 665,	(36,419) : 265,	(37,419) : 159,	(38,419) : 558,	(39,419) : 572,	(40,419) : 184,
(1,420) : 233,	(2,420) : 266,	(3,420) : 521,	(4,420) : 391,	(5,420) : 277,	(6,420) : 230,	(7,420) : 225,	(8,420) : 801,	(9,420) : 391,	(10,420) : 227,	(11,420) : 267,	(12,420) : 406,	(13,420) : 91,	(14,420) : 386,	(15,420) : 236,	(16,420) : 462,	(17,420) : 341,	(18,420) : 259,	(19,420) : 351,	(20,420) : 227,	(21,420) : 89,	(22,420) : 232,	(23,420) : 253,	(24,420) : 344,	(25,420) : 719,	(26,420) : 92,	(27,420) : 87,	(28,420) : 226,	(29,420) : 244,	(30,420) : 231,	(31,420) : 235,	(32,420) : 145,	(33,420) : 217,	(34,420) : 217,	(35,420) : 365,	(36,420) : 451,	(37,420) : 432,	(38,420) : 262,	(39,420) : 266,	(40,420) : 741,
(1,421) : 803,	(2,421) : 788,	(3,421) : 147,	(4,421) : 809,	(5,421) : 751,	(6,421) : 738,	(7,421) : 792,	(8,421) : 169,	(9,421) : 536,	(10,421) : 794,	(11,421) : 789,	(12,421) : 993,	(13,421) : 752,	(14,421) : 538,	(15,421) : 747,	(16,421) : 926,	(17,421) : 782,	(18,421) : 737,	(19,421) : 959,	(20,421) : 797,	(21,421) : 670,	(22,421) : 662,	(23,421) : 736,	(24,421) : 782,	(25,421) : 268,	(26,421) : 756,	(27,421) : 749,	(28,421) : 795,	(29,421) : 664,	(30,421) : 802,	(31,421) : 750,	(32,421) : 742,	(33,421) : 858,	(34,421) : 713,	(35,421) : 894,	(36,421) : 557,	(37,421) : 426,	(38,421) : 772,	(39,421) : 785,	(40,421) : 296,
(1,422) : 323,	(2,422) : 288,	(3,422) : 402,	(4,422) : 701,	(5,422) : 240,	(6,422) : 247,	(7,422) : 313,	(8,422) : 604,	(9,422) : 55,	(10,422) : 315,	(11,422) : 289,	(12,422) : 742,	(13,422) : 370,	(14,422) : 50,	(15,422) : 557,	(16,422) : 789,	(17,422) : 653,	(18,422) : 232,	(19,422) : 685,	(20,422) : 319,	(21,422) : 410,	(22,422) : 167,	(23,422) : 233,	(24,422) : 654,	(25,422) : 459,	(26,422) : 416,	(27,422) : 372,	(28,422) : 317,	(29,422) : 530,	(30,422) : 323,	(31,422) : 257,	(32,422) : 303,	(33,422) : 419,	(34,422) : 226,	(35,422) : 379,	(36,422) : 117,	(37,422) : 129,	(38,422) : 271,	(39,422) : 284,	(40,422) : 471,
(1,423) : 379,	(2,423) : 359,	(3,423) : 294,	(4,423) : 632,	(5,423) : 320,	(6,423) : 310,	(7,423) : 368,	(8,423) : 528,	(9,423) : 157,	(10,423) : 371,	(11,423) : 360,	(12,423) : 713,	(13,423) : 372,	(14,423) : 155,	(15,423) : 501,	(16,423) : 730,	(17,423) : 586,	(18,423) : 307,	(19,423) : 661,	(20,423) : 374,	(21,423) : 364,	(22,423) : 232,	(23,423) : 305,	(24,423) : 588,	(25,423) : 413,	(26,423) : 402,	(27,423) : 372,	(28,423) : 372,	(29,423) : 457,	(30,423) : 378,	(31,423) : 322,	(32,423) : 332,	(33,423) : 453,	(34,423) : 285,	(35,423) : 464,	(36,423) : 212,	(37,423) : 129,	(38,423) : 342,	(39,423) : 356,	(40,423) : 432,
(1,424) : 130,	(2,424) : 98,	(3,424) : 733,	(4,424) : 729,	(5,424) : 121,	(6,424) : 147,	(7,424) : 136,	(8,424) : 956,	(9,424) : 370,	(10,424) : 135,	(11,424) : 97,	(12,424) : 642,	(13,424) : 300,	(14,424) : 367,	(15,424) : 574,	(16,424) : 771,	(17,424) : 680,	(18,424) : 137,	(19,424) : 578,	(20,424) : 135,	(21,424) : 445,	(22,424) : 211,	(23,424) : 139,	(24,424) : 682,	(25,424) : 807,	(26,424) : 363,	(27,424) : 307,	(28,424) : 136,	(29,424) : 602,	(30,424) : 133,	(31,424) : 137,	(32,424) : 216,	(33,424) : 222,	(34,424) : 171,	(35,424) : 29,	(36,424) : 393,	(37,424) : 479,	(38,424) : 109,	(39,424) : 100,	(40,424) : 817,
(1,425) : 65,	(2,425) : 86,	(3,425) : 722,	(4,425) : 641,	(5,425) : 137,	(6,425) : 129,	(7,425) : 76,	(8,425) : 966,	(9,425) : 403,	(10,425) : 73,	(11,425) : 85,	(12,425) : 546,	(13,425) : 218,	(14,425) : 399,	(15,425) : 488,	(16,425) : 679,	(17,425) : 593,	(18,425) : 141,	(19,425) : 482,	(20,425) : 71,	(21,425) : 370,	(22,425) : 210,	(23,425) : 140,	(24,425) : 596,	(25,425) : 831,	(26,425) : 277,	(27,425) : 224,	(28,425) : 73,	(29,425) : 525,	(30,425) : 67,	(31,425) : 118,	(32,425) : 149,	(33,425) : 126,	(34,425) : 154,	(35,425) : 85,	(36,425) : 438,	(37,425) : 502,	(38,425) : 103,	(39,425) : 89,	(40,425) : 844,
(1,426) : 197,	(2,426) : 168,	(3,426) : 486,	(4,426) : 632,	(5,426) : 126,	(6,426) : 122,	(7,426) : 187,	(8,426) : 716,	(9,426) : 169,	(10,426) : 189,	(11,426) : 169,	(12,426) : 639,	(13,426) : 256,	(14,426) : 164,	(15,426) : 480,	(16,426) : 708,	(17,426) : 582,	(18,426) : 114,	(19,426) : 579,	(20,426) : 193,	(21,426) : 330,	(22,426) : 41,	(23,426) : 113,	(24,426) : 584,	(25,426) : 582,	(26,426) : 310,	(27,426) : 259,	(28,426) : 191,	(29,426) : 471,	(30,426) : 196,	(31,426) : 133,	(32,426) : 179,	(33,426) : 293,	(34,426) : 99,	(35,426) : 270,	(36,426) : 220,	(37,426) : 254,	(38,426) : 150,	(39,426) : 164,	(40,426) : 596,
(1,427) : 132,	(2,427) : 77,	(3,427) : 659,	(4,427) : 717,	(5,427) : 58,	(6,427) : 103,	(7,427) : 132,	(8,427) : 877,	(9,427) : 290,	(10,427) : 132,	(11,427) : 77,	(12,427) : 660,	(13,427) : 291,	(14,427) : 286,	(15,427) : 561,	(16,427) : 771,	(17,427) : 667,	(18,427) : 78,	(19,427) : 596,	(20,427) : 134,	(21,427) : 422,	(22,427) : 143,	(23,427) : 82,	(24,427) : 670,	(25,427) : 726,	(26,427) : 356,	(27,427) : 297,	(28,427) : 134,	(29,427) : 577,	(30,427) : 134,	(31,427) : 98,	(32,427) : 197,	(33,427) : 247,	(34,427) : 120,	(35,427) : 109,	(36,427) : 314,	(37,427) : 398,	(38,427) : 74,	(39,427) : 76,	(40,427) : 736,
(1,428) : 218,	(2,428) : 179,	(3,428) : 497,	(4,428) : 678,	(5,428) : 130,	(6,428) : 142,	(7,428) : 209,	(8,428) : 713,	(9,428) : 140,	(10,428) : 212,	(11,428) : 180,	(12,428) : 683,	(13,428) : 299,	(14,428) : 135,	(15,428) : 526,	(16,428) : 754,	(17,428) : 629,	(18,428) : 123,	(19,428) : 622,	(20,428) : 215,	(21,428) : 376,	(22,428) : 71,	(23,428) : 124,	(24,428) : 631,	(25,428) : 569,	(26,428) : 354,	(27,428) : 302,	(28,428) : 213,	(29,428) : 517,	(30,428) : 218,	(31,428) : 151,	(32,428) : 216,	(33,428) : 323,	(34,428) : 125,	(35,428) : 269,	(36,428) : 184,	(37,428) : 239,	(38,428) : 162,	(39,428) : 175,	(40,428) : 580,
(1,429) : 554,	(2,429) : 596,	(3,429) : 624,	(4,429) : 70,	(5,429) : 613,	(6,429) : 564,	(7,429) : 548,	(8,429) : 908,	(9,429) : 689,	(10,429) : 550,	(11,429) : 597,	(12,429) : 288,	(13,429) : 384,	(14,429) : 686,	(15,429) : 113,	(16,429) : 183,	(17,429) : 41,	(18,429) : 595,	(19,429) : 288,	(20,429) : 549,	(21,429) : 247,	(22,429) : 565,	(23,429) : 589,	(24,429) : 40,	(25,429) : 897,	(26,429) : 323,	(27,429) : 378,	(28,429) : 548,	(29,429) : 106,	(30,429) : 551,	(31,429) : 569,	(32,429) : 472,	(33,429) : 489,	(34,429) : 553,	(35,429) : 683,	(36,429) : 752,	(37,429) : 692,	(38,429) : 594,	(39,429) : 596,	(40,429) : 925,
(1,430) : 49,	(2,430) : 69,	(3,430) : 610,	(4,430) : 577,	(5,430) : 94,	(6,430) : 49,	(7,430) : 38,	(8,430) : 860,	(9,430) : 322,	(10,430) : 41,	(11,430) : 70,	(12,430) : 527,	(13,430) : 151,	(14,430) : 317,	(15,430) : 420,	(16,430) : 631,	(17,430) : 527,	(18,430) : 81,	(19,430) : 464,	(20,430) : 43,	(21,430) : 285,	(22,430) : 116,	(23,430) : 76,	(24,430) : 529,	(25,430) : 735,	(26,430) : 215,	(27,430) : 156,	(28,430) : 41,	(29,430) : 441,	(30,430) : 48,	(31,430) : 46,	(32,430) : 56,	(33,430) : 139,	(34,430) : 60,	(35,430) : 172,	(36,430) : 368,	(37,430) : 409,	(38,430) : 67,	(39,430) : 68,	(40,430) : 750,
(1,431) : 113,	(2,431) : 169,	(3,431) : 689,	(4,431) : 510,	(5,431) : 213,	(6,431) : 174,	(7,431) : 115,	(8,431) : 954,	(9,431) : 445,	(10,431) : 114,	(11,431) : 169,	(12,431) : 417,	(13,431) : 104,	(14,431) : 440,	(15,431) : 359,	(16,431) : 545,	(17,431) : 463,	(18,431) : 204,	(19,431) : 353,	(20,431) : 111,	(21,431) : 256,	(22,431) : 239,	(23,431) : 200,	(24,431) : 465,	(25,431) : 844,	(26,431) : 152,	(27,431) : 110,	(28,431) : 112,	(29,431) : 405,	(30,431) : 111,	(31,431) : 169,	(32,431) : 101,	(33,431) : 24,	(34,431) : 185,	(35,431) : 219,	(36,431) : 493,	(37,431) : 524,	(38,431) : 178,	(39,431) : 171,	(40,431) : 861,
(1,432) : 569,	(2,432) : 613,	(3,432) : 672,	(4,432) : 31,	(5,432) : 634,	(6,432) : 585,	(7,432) : 564,	(8,432) : 956,	(9,432) : 725,	(10,432) : 565,	(11,432) : 614,	(12,432) : 251,	(13,432) : 398,	(14,432) : 721,	(15,432) : 125,	(16,432) : 135,	(17,432) : 19,	(18,432) : 616,	(19,432) : 258,	(20,432) : 564,	(21,432) : 272,	(22,432) : 592,	(23,432) : 610,	(24,432) : 17,	(25,432) : 944,	(26,432) : 335,	(27,432) : 392,	(28,432) : 563,	(29,432) : 147,	(30,432) : 566,	(31,432) : 588,	(32,432) : 489,	(33,432) : 496,	(34,432) : 575,	(35,432) : 695,	(36,432) : 787,	(37,432) : 732,	(38,432) : 613,	(39,432) : 614,	(40,432) : 971,
(1,433) : 7,	(2,433) : 62,	(3,433) : 657,	(4,433) : 593,	(5,433) : 109,	(6,433) : 78,	(7,433) : 11,	(8,433) : 906,	(9,433) : 360,	(10,433) : 9,	(11,433) : 62,	(12,433) : 522,	(13,433) : 164,	(14,433) : 355,	(15,433) : 438,	(16,433) : 639,	(17,433) : 544,	(18,433) : 103,	(19,433) : 458,	(20,433) : 5,	(21,433) : 310,	(22,433) : 157,	(23,433) : 100,	(24,433) : 546,	(25,433) : 779,	(26,433) : 227,	(27,433) : 170,	(28,433) : 7,	(29,433) : 467,	(30,433) : 4,	(31,433) : 70,	(32,433) : 83,	(33,433) : 112,	(34,433) : 99,	(35,433) : 138,	(36,433) : 402,	(37,433) : 452,	(38,433) : 72,	(39,433) : 64,	(40,433) : 793,
(1,434) : 638,	(2,434) : 688,	(3,434) : 825,	(4,434) : 141,	(5,434) : 718,	(6,434) : 669,	(7,434) : 635,	(8,434) : 1111,	(9,434) : 847,	(10,434) : 635,	(11,434) : 689,	(12,434) : 170,	(13,434) : 474,	(14,434) : 843,	(15,434) : 237,	(16,434) : 31,	(17,434) : 165,	(18,434) : 702,	(19,434) : 214,	(20,434) : 634,	(21,434) : 383,	(22,434) : 692,	(23,434) : 697,	(24,434) : 165,	(25,434) : 1094,	(26,434) : 411,	(27,434) : 469,	(28,434) : 633,	(29,434) : 293,	(30,434) : 635,	(31,434) : 671,	(32,434) : 570,	(33,434) : 544,	(34,434) : 665,	(35,434) : 753,	(36,434) : 909,	(37,434) : 866,	(38,434) : 692,	(39,434) : 690,	(40,434) : 1121,
(1,435) : 594,	(2,435) : 650,	(3,435) : 942,	(4,435) : 318,	(5,435) : 691,	(6,435) : 646,	(7,435) : 595,	(8,435) : 1232,	(9,435) : 875,	(10,435) : 595,	(11,435) : 650,	(12,435) : 84,	(13,435) : 460,	(14,435) : 870,	(15,435) : 328,	(16,435) : 231,	(17,435) : 317,	(18,435) : 678,	(19,435) : 137,	(20,435) : 592,	(21,435) : 434,	(22,435) : 690,	(23,435) : 673,	(24,435) : 318,	(25,435) : 1186,	(26,435) : 407,	(27,435) : 457,	(28,435) : 592,	(29,435) : 414,	(30,435) : 592,	(31,435) : 643,	(32,435) : 549,	(33,435) : 482,	(34,435) : 649,	(35,435) : 687,	(36,435) : 933,	(37,435) : 918,	(38,435) : 658,	(39,435) : 652,	(40,435) : 1210,
(1,436) : 146,	(2,436) : 98,	(3,436) : 577,	(4,436) : 679,	(5,436) : 46,	(6,436) : 78,	(7,436) : 140,	(8,436) : 799,	(9,436) : 221,	(10,436) : 142,	(11,436) : 99,	(12,436) : 651,	(13,436) : 268,	(14,436) : 216,	(15,436) : 523,	(16,436) : 743,	(17,436) : 629,	(18,436) : 47,	(19,436) : 588,	(20,436) : 145,	(21,436) : 377,	(22,436) : 68,	(23,436) : 51,	(24,436) : 631,	(25,436) : 654,	(26,436) : 330,	(27,436) : 272,	(28,436) : 144,	(29,436) : 528,	(30,436) : 148,	(31,436) : 83,	(32,436) : 174,	(33,436) : 261,	(34,436) : 75,	(35,436) : 183,	(36,436) : 255,	(37,436) : 324,	(38,436) : 82,	(39,436) : 95,	(40,436) : 665,
(1,437) : 509,	(2,437) : 549,	(3,437) : 590,	(4,437) : 116,	(5,437) : 565,	(6,437) : 516,	(7,437) : 502,	(8,437) : 877,	(9,437) : 640,	(10,437) : 503,	(11,437) : 550,	(12,437) : 294,	(13,437) : 339,	(14,437) : 636,	(15,437) : 78,	(16,437) : 221,	(17,437) : 72,	(18,437) : 546,	(19,437) : 282,	(20,437) : 503,	(21,437) : 198,	(22,437) : 516,	(23,437) : 541,	(24,437) : 73,	(25,437) : 858,	(26,437) : 280,	(27,437) : 333,	(28,437) : 502,	(29,437) : 59,	(30,437) : 506,	(31,437) : 521,	(32,437) : 425,	(33,437) : 448,	(34,437) : 504,	(35,437) : 638,	(36,437) : 703,	(37,437) : 644,	(38,437) : 547,	(39,437) : 549,	(40,437) : 885,
(1,438) : 693,	(2,438) : 666,	(3,438) : 188,	(4,438) : 852,	(5,438) : 621,	(6,438) : 620,	(7,438) : 682,	(8,438) : 223,	(9,438) : 373,	(10,438) : 685,	(11,438) : 667,	(12,438) : 990,	(13,438) : 684,	(14,438) : 376,	(15,438) : 755,	(16,438) : 964,	(17,438) : 816,	(18,438) : 611,	(19,438) : 946,	(20,438) : 688,	(21,438) : 644,	(22,438) : 539,	(23,438) : 611,	(24,438) : 816,	(25,438) : 114,	(26,438) : 706,	(27,438) : 683,	(28,438) : 686,	(29,438) : 686,	(30,438) : 692,	(31,438) : 631,	(32,438) : 650,	(33,438) : 770,	(34,438) : 596,	(35,438) : 762,	(36,438) : 377,	(37,438) : 259,	(38,438) : 649,	(39,438) : 662,	(40,438) : 143,
(1,439) : 445,	(2,439) : 498,	(3,439) : 780,	(4,439) : 246,	(5,439) : 534,	(6,439) : 487,	(7,439) : 443,	(8,439) : 1070,	(9,439) : 704,	(10,439) : 443,	(11,439) : 499,	(12,439) : 91,	(13,439) : 295,	(14,439) : 699,	(15,439) : 180,	(16,439) : 228,	(17,439) : 218,	(18,439) : 520,	(19,439) : 42,	(20,439) : 441,	(21,439) : 262,	(22,439) : 524,	(23,439) : 515,	(24,439) : 221,	(25,439) : 1016,	(26,439) : 238,	(27,439) : 291,	(28,439) : 441,	(29,439) : 271,	(30,439) : 442,	(31,439) : 486,	(32,439) : 388,	(33,439) : 341,	(34,439) : 487,	(35,439) : 552,	(36,439) : 763,	(37,439) : 746,	(38,439) : 504,	(39,439) : 500,	(40,439) : 1040,
(1,440) : 320,	(2,440) : 306,	(3,440) : 341,	(4,440) : 581,	(5,440) : 272,	(6,440) : 255,	(7,440) : 309,	(8,440) : 590,	(9,440) : 179,	(10,440) : 311,	(11,440) : 307,	(12,440) : 649,	(13,440) : 304,	(14,440) : 175,	(15,440) : 445,	(16,440) : 675,	(17,440) : 535,	(18,440) : 257,	(19,440) : 596,	(20,440) : 315,	(21,440) : 303,	(22,440) : 182,	(23,440) : 255,	(24,440) : 536,	(25,440) : 481,	(26,440) : 335,	(27,440) : 304,	(28,440) : 312,	(29,440) : 408,	(30,440) : 319,	(31,440) : 267,	(32,440) : 267,	(33,440) : 388,	(34,440) : 230,	(35,440) : 415,	(36,440) : 240,	(37,440) : 187,	(38,440) : 290,	(39,440) : 303,	(40,440) : 500,
(1,441) : 110,	(2,441) : 106,	(3,441) : 550,	(4,441) : 573,	(5,441) : 98,	(6,441) : 56,	(7,441) : 99,	(8,441) : 798,	(9,441) : 269,	(10,441) : 101,	(11,441) : 107,	(12,441) : 551,	(13,441) : 166,	(14,441) : 264,	(15,441) : 417,	(16,441) : 637,	(17,441) : 522,	(18,441) : 79,	(19,441) : 489,	(20,441) : 105,	(21,441) : 271,	(22,441) : 65,	(23,441) : 74,	(24,441) : 525,	(25,441) : 674,	(26,441) : 226,	(27,441) : 170,	(28,441) : 102,	(29,441) : 425,	(30,441) : 109,	(31,441) : 66,	(32,441) : 80,	(33,441) : 193,	(34,441) : 36,	(35,441) : 218,	(36,441) : 319,	(37,441) : 350,	(38,441) : 94,	(39,441) : 104,	(40,441) : 690,
(1,442) : 546,	(2,442) : 589,	(3,442) : 653,	(4,442) : 56,	(5,442) : 609,	(6,442) : 560,	(7,442) : 540,	(8,442) : 939,	(9,442) : 700,	(10,442) : 541,	(11,442) : 590,	(12,442) : 251,	(13,442) : 375,	(14,442) : 696,	(15,442) : 101,	(16,442) : 155,	(17,442) : 7,	(18,442) : 591,	(19,442) : 252,	(20,442) : 540,	(21,442) : 247,	(22,442) : 566,	(23,442) : 586,	(24,442) : 9,	(25,442) : 923,	(26,442) : 312,	(27,442) : 369,	(28,442) : 539,	(29,442) : 124,	(30,442) : 543,	(31,442) : 564,	(32,442) : 465,	(33,442) : 474,	(34,442) : 551,	(35,442) : 672,	(36,442) : 762,	(37,442) : 708,	(38,442) : 589,	(39,442) : 590,	(40,442) : 950,
(1,443) : 236,	(2,443) : 213,	(3,443) : 437,	(4,443) : 614,	(5,443) : 175,	(6,443) : 165,	(7,443) : 226,	(8,443) : 672,	(9,443) : 154,	(10,443) : 228,	(11,443) : 214,	(12,443) : 641,	(13,443) : 267,	(14,443) : 150,	(15,443) : 466,	(16,443) : 696,	(17,443) : 565,	(18,443) : 161,	(19,443) : 583,	(20,443) : 232,	(21,443) : 316,	(22,443) : 86,	(23,443) : 160,	(24,443) : 567,	(25,443) : 544,	(26,443) : 313,	(27,443) : 269,	(28,443) : 229,	(29,443) : 448,	(30,443) : 236,	(31,443) : 176,	(32,443) : 203,	(33,443) : 322,	(34,443) : 140,	(35,443) : 319,	(36,443) : 213,	(37,443) : 221,	(38,443) : 197,	(39,443) : 210,	(40,443) : 559,
(1,444) : 994,	(2,444) : 960,	(3,444) : 474,	(4,444) : 1163,	(5,444) : 911,	(6,444) : 919,	(7,444) : 984,	(8,444) : 227,	(9,444) : 646,	(10,444) : 986,	(11,444) : 961,	(12,444) : 1313,	(13,444) : 1003,	(14,444) : 650,	(15,444) : 1075,	(16,444) : 1278,	(17,444) : 1130,	(18,444) : 904,	(19,444) : 1270,	(20,444) : 990,	(21,444) : 967,	(22,444) : 838,	(23,444) : 905,	(24,444) : 1131,	(25,444) : 218,	(26,444) : 1028,	(27,444) : 1003,	(28,444) : 988,	(29,444) : 1003,	(30,444) : 994,	(31,444) : 929,	(32,444) : 961,	(33,444) : 1082,	(34,444) : 897,	(35,444) : 1044,	(36,444) : 627,	(37,444) : 544,	(38,444) : 943,	(39,444) : 956,	(40,444) : 202,
(1,445) : 382,	(2,445) : 345,	(3,445) : 380,	(4,445) : 746,	(5,445) : 295,	(6,445) : 306,	(7,445) : 373,	(8,445) : 559,	(9,445) : 39,	(10,445) : 375,	(11,445) : 346,	(12,445) : 799,	(13,445) : 430,	(14,445) : 40,	(15,445) : 607,	(16,445) : 838,	(17,445) : 699,	(18,445) : 289,	(19,445) : 743,	(20,445) : 379,	(21,445) : 461,	(22,445) : 227,	(23,445) : 290,	(24,445) : 701,	(25,445) : 404,	(26,445) : 474,	(27,445) : 432,	(28,445) : 377,	(29,445) : 573,	(30,445) : 382,	(31,445) : 316,	(32,445) : 364,	(33,445) : 480,	(34,445) : 286,	(35,445) : 431,	(36,445) : 82,	(37,445) : 77,	(38,445) : 328,	(39,445) : 341,	(40,445) : 415,
(1,446) : 65,	(2,446) : 101,	(3,446) : 609,	(4,446) : 543,	(5,446) : 129,	(6,446) : 82,	(7,446) : 56,	(8,446) : 866,	(9,446) : 344,	(10,446) : 58,	(11,446) : 102,	(12,446) : 493,	(13,446) : 116,	(14,446) : 339,	(15,446) : 387,	(16,446) : 596,	(17,446) : 493,	(18,446) : 115,	(19,446) : 430,	(20,446) : 59,	(21,446) : 254,	(22,446) : 139,	(23,446) : 110,	(24,446) : 496,	(25,446) : 747,	(26,446) : 180,	(27,446) : 122,	(28,446) : 57,	(29,446) : 410,	(30,446) : 63,	(31,446) : 81,	(32,446) : 24,	(33,446) : 118,	(34,446) : 87,	(35,446) : 197,	(36,446) : 392,	(37,446) : 425,	(38,446) : 102,	(39,446) : 102,	(40,446) : 763,
(1,447) : 260,	(2,447) : 267,	(3,447) : 416,	(4,447) : 477,	(5,447) : 254,	(6,447) : 217,	(7,447) : 249,	(8,447) : 686,	(9,447) : 279,	(10,447) : 251,	(11,447) : 268,	(12,447) : 528,	(13,447) : 194,	(14,447) : 275,	(15,447) : 333,	(16,447) : 564,	(17,447) : 429,	(18,447) : 235,	(19,447) : 475,	(20,447) : 254,	(21,447) : 186,	(22,447) : 175,	(23,447) : 230,	(24,447) : 431,	(25,447) : 595,	(26,447) : 216,	(27,447) : 192,	(28,447) : 251,	(29,447) : 310,	(30,447) : 258,	(31,447) : 227,	(32,447) : 184,	(33,447) : 296,	(34,447) : 194,	(35,447) : 379,	(36,447) : 341,	(37,447) : 308,	(38,447) : 256,	(39,447) : 265,	(40,447) : 615,
(1,448) : 509,	(2,448) : 551,	(3,448) : 610,	(4,448) : 103,	(5,448) : 568,	(6,448) : 520,	(7,448) : 503,	(8,448) : 897,	(9,448) : 652,	(10,448) : 504,	(11,448) : 552,	(12,448) : 274,	(13,448) : 338,	(14,448) : 648,	(15,448) : 69,	(16,448) : 203,	(17,448) : 56,	(18,448) : 550,	(19,448) : 264,	(20,448) : 504,	(21,448) : 203,	(22,448) : 522,	(23,448) : 545,	(24,448) : 57,	(25,448) : 877,	(26,448) : 278,	(27,448) : 332,	(28,448) : 503,	(29,448) : 77,	(30,448) : 506,	(31,448) : 524,	(32,448) : 427,	(33,448) : 445,	(34,448) : 509,	(35,448) : 638,	(36,448) : 714,	(37,448) : 659,	(38,448) : 549,	(39,448) : 551,	(40,448) : 904,
(1,449) : 543,	(2,449) : 592,	(3,449) : 747,	(4,449) : 112,	(5,449) : 621,	(6,449) : 572,	(7,449) : 539,	(8,449) : 1035,	(9,449) : 749,	(10,449) : 540,	(11,449) : 593,	(12,449) : 148,	(13,449) : 377,	(14,449) : 744,	(15,449) : 142,	(16,449) : 103,	(17,449) : 102,	(18,449) : 604,	(19,449) : 162,	(20,449) : 538,	(21,449) : 285,	(22,449) : 592,	(23,449) : 599,	(24,449) : 104,	(25,449) : 1008,	(26,449) : 313,	(27,449) : 372,	(28,449) : 538,	(29,449) : 211,	(30,449) : 540,	(31,449) : 573,	(32,449) : 472,	(33,449) : 453,	(34,449) : 566,	(35,449) : 661,	(36,449) : 810,	(37,449) : 770,	(38,449) : 595,	(39,449) : 593,	(40,449) : 1034,
(1,450) : 106,	(2,450) : 73,	(3,450) : 712,	(4,450) : 705,	(5,450) : 100,	(6,450) : 123,	(7,450) : 112,	(8,450) : 939,	(9,450) : 357,	(10,450) : 110,	(11,450) : 72,	(12,450) : 624,	(13,450) : 277,	(14,450) : 353,	(15,450) : 550,	(16,450) : 750,	(17,450) : 656,	(18,450) : 114,	(19,450) : 560,	(20,450) : 111,	(21,450) : 420,	(22,450) : 190,	(23,450) : 117,	(24,450) : 659,	(25,450) : 793,	(26,450) : 339,	(27,450) : 283,	(28,450) : 112,	(29,450) : 577,	(30,450) : 109,	(31,450) : 113,	(32,450) : 190,	(33,450) : 204,	(34,450) : 147,	(35,450) : 44,	(36,450) : 383,	(37,450) : 464,	(38,450) : 85,	(39,450) : 75,	(40,450) : 803,
(1,451) : 43,	(2,451) : 98,	(3,451) : 678,	(4,451) : 572,	(5,451) : 147,	(6,451) : 114,	(7,451) : 48,	(8,451) : 932,	(9,451) : 395,	(10,451) : 46,	(11,451) : 98,	(12,451) : 489,	(13,451) : 147,	(14,451) : 390,	(15,451) : 418,	(16,451) : 613,	(17,451) : 523,	(18,451) : 141,	(19,451) : 425,	(20,451) : 43,	(21,451) : 299,	(22,451) : 190,	(23,451) : 137,	(24,451) : 526,	(25,451) : 810,	(26,451) : 207,	(27,451) : 153,	(28,451) : 44,	(29,451) : 454,	(30,451) : 41,	(31,451) : 107,	(32,451) : 86,	(33,451) : 74,	(34,451) : 132,	(35,451) : 154,	(36,451) : 438,	(37,451) : 484,	(38,451) : 110,	(39,451) : 101,	(40,451) : 825,
(1,452) : 53,	(2,452) : 100,	(3,452) : 635,	(4,452) : 547,	(5,452) : 137,	(6,452) : 95,	(7,452) : 47,	(8,452) : 892,	(9,452) : 366,	(10,452) : 48,	(11,452) : 101,	(12,452) : 484,	(13,452) : 119,	(14,452) : 361,	(15,452) : 392,	(16,452) : 596,	(17,452) : 498,	(18,452) : 126,	(19,452) : 421,	(20,452) : 47,	(21,452) : 265,	(22,452) : 160,	(23,452) : 122,	(24,452) : 501,	(25,452) : 773,	(26,452) : 182,	(27,452) : 125,	(28,452) : 46,	(29,452) : 421,	(30,452) : 50,	(31,452) : 91,	(32,452) : 42,	(33,452) : 94,	(34,452) : 105,	(35,452) : 184,	(36,452) : 413,	(37,452) : 449,	(38,452) : 105,	(39,452) : 102,	(40,452) : 789,
(1,453) : 15,	(2,453) : 55,	(3,453) : 644,	(4,453) : 593,	(5,453) : 98,	(6,453) : 65,	(7,453) : 4,	(8,453) : 892,	(9,453) : 346,	(10,453) : 6,	(11,453) : 56,	(12,453) : 528,	(13,453) : 164,	(14,453) : 341,	(15,453) : 437,	(16,453) : 642,	(17,453) : 543,	(18,453) : 91,	(19,453) : 464,	(20,453) : 10,	(21,453) : 307,	(22,453) : 143,	(23,453) : 87,	(24,453) : 546,	(25,453) : 765,	(26,453) : 228,	(27,453) : 170,	(28,453) : 7,	(29,453) : 463,	(30,453) : 14,	(31,453) : 56,	(32,453) : 77,	(33,453) : 123,	(34,453) : 84,	(35,453) : 144,	(36,453) : 388,	(37,453) : 437,	(38,453) : 63,	(39,453) : 57,	(40,453) : 779,
(1,454) : 505,	(2,454) : 549,	(3,454) : 643,	(4,454) : 95,	(5,454) : 570,	(6,454) : 521,	(7,454) : 499,	(8,454) : 931,	(9,454) : 668,	(10,454) : 500,	(11,454) : 550,	(12,454) : 236,	(13,454) : 334,	(14,454) : 664,	(15,454) : 61,	(16,454) : 177,	(17,454) : 46,	(18,454) : 552,	(19,454) : 228,	(20,454) : 499,	(21,454) : 210,	(22,454) : 529,	(23,454) : 546,	(24,454) : 49,	(25,454) : 907,	(26,454) : 271,	(27,454) : 328,	(28,454) : 499,	(29,454) : 108,	(30,454) : 502,	(31,454) : 524,	(32,454) : 425,	(33,454) : 433,	(34,454) : 511,	(35,454) : 631,	(36,454) : 730,	(37,454) : 681,	(38,454) : 548,	(39,454) : 549,	(40,454) : 933,
(1,455) : 747,	(2,455) : 726,	(3,455) : 153,	(4,455) : 843,	(5,455) : 683,	(6,455) : 678,	(7,455) : 736,	(8,455) : 160,	(9,455) : 447,	(10,455) : 739,	(11,455) : 727,	(12,455) : 1002,	(13,455) : 721,	(14,455) : 449,	(15,455) : 760,	(16,455) : 958,	(17,455) : 811,	(18,455) : 672,	(19,455) : 962,	(20,455) : 742,	(21,455) : 662,	(22,455) : 598,	(23,455) : 671,	(24,455) : 811,	(25,455) : 155,	(26,455) : 735,	(27,455) : 719,	(28,455) : 740,	(29,455) : 685,	(30,455) : 747,	(31,455) : 689,	(32,455) : 697,	(33,455) : 817,	(34,455) : 653,	(35,455) : 826,	(36,455) : 458,	(37,455) : 334,	(38,455) : 709,	(39,455) : 722,	(40,455) : 186,
(1,456) : 534,	(2,456) : 578,	(3,456) : 650,	(4,456) : 67,	(5,456) : 599,	(6,456) : 550,	(7,456) : 529,	(8,456) : 937,	(9,456) : 691,	(10,456) : 530,	(11,456) : 579,	(12,456) : 247,	(13,456) : 363,	(14,456) : 687,	(15,456) : 90,	(16,456) : 161,	(17,456) : 16,	(18,456) : 581,	(19,456) : 245,	(20,456) : 529,	(21,456) : 237,	(22,456) : 556,	(23,456) : 575,	(24,456) : 19,	(25,456) : 919,	(26,456) : 301,	(27,456) : 357,	(28,456) : 528,	(29,456) : 118,	(30,456) : 531,	(31,456) : 553,	(32,456) : 454,	(33,456) : 463,	(34,456) : 540,	(35,456) : 661,	(36,456) : 754,	(37,456) : 700,	(38,456) : 578,	(39,456) : 579,	(40,456) : 945,
(1,457) : 197,	(2,457) : 186,	(3,457) : 463,	(4,457) : 566,	(5,457) : 160,	(6,457) : 135,	(7,457) : 185,	(8,457) : 712,	(9,457) : 213,	(10,457) : 188,	(11,457) : 187,	(12,457) : 583,	(13,457) : 208,	(14,457) : 208,	(15,457) : 415,	(16,457) : 644,	(17,457) : 516,	(18,457) : 143,	(19,457) : 525,	(20,457) : 191,	(21,457) : 265,	(22,457) : 74,	(23,457) : 139,	(24,457) : 519,	(25,457) : 593,	(26,457) : 255,	(27,457) : 210,	(28,457) : 189,	(29,457) : 404,	(30,457) : 195,	(31,457) : 146,	(32,457) : 150,	(33,457) : 270,	(34,457) : 109,	(35,457) : 297,	(36,457) : 270,	(37,457) : 276,	(38,457) : 171,	(39,457) : 183,	(40,457) : 610,
(1,458) : 527,	(2,458) : 565,	(3,458) : 563,	(4,458) : 130,	(5,458) : 577,	(6,458) : 529,	(7,458) : 520,	(8,458) : 849,	(9,458) : 637,	(10,458) : 521,	(11,458) : 566,	(12,458) : 331,	(13,458) : 359,	(14,458) : 633,	(15,458) : 112,	(16,458) : 243,	(17,458) : 97,	(18,458) : 558,	(19,458) : 320,	(20,458) : 521,	(21,458) : 211,	(22,458) : 523,	(23,458) : 553,	(24,458) : 97,	(25,458) : 836,	(26,458) : 302,	(27,458) : 353,	(28,458) : 520,	(29,458) : 56,	(30,458) : 524,	(31,458) : 535,	(32,458) : 441,	(33,458) : 473,	(34,458) : 516,	(35,458) : 658,	(36,458) : 699,	(37,458) : 634,	(38,458) : 561,	(39,458) : 564,	(40,458) : 864,
(1,459) : 116,	(2,459) : 79,	(3,459) : 573,	(4,459) : 639,	(5,459) : 41,	(6,459) : 41,	(7,459) : 108,	(8,459) : 805,	(9,459) : 242,	(10,459) : 110,	(11,459) : 80,	(12,459) : 610,	(13,459) : 227,	(14,459) : 237,	(15,459) : 483,	(16,459) : 702,	(17,459) : 589,	(18,459) : 25,	(19,459) : 547,	(20,459) : 114,	(21,459) : 338,	(22,459) : 51,	(23,459) : 24,	(24,459) : 591,	(25,459) : 668,	(26,459) : 289,	(27,459) : 231,	(28,459) : 112,	(29,459) : 491,	(30,459) : 117,	(31,459) : 49,	(32,459) : 133,	(33,459) : 226,	(34,459) : 34,	(35,459) : 183,	(36,459) : 283,	(37,459) : 338,	(38,459) : 62,	(39,459) : 75,	(40,459) : 681,
(1,460) : 16,	(2,460) : 54,	(3,460) : 643,	(4,460) : 593,	(5,460) : 97,	(6,460) : 63,	(7,460) : 5,	(8,460) : 891,	(9,460) : 345,	(10,460) : 8,	(11,460) : 55,	(12,460) : 529,	(13,460) : 164,	(14,460) : 340,	(15,460) : 437,	(16,460) : 642,	(17,460) : 543,	(18,460) : 90,	(19,460) : 465,	(20,460) : 11,	(21,460) : 307,	(22,460) : 141,	(23,460) : 86,	(24,460) : 546,	(25,460) : 763,	(26,460) : 228,	(27,460) : 170,	(28,460) : 9,	(29,460) : 463,	(30,460) : 15,	(31,460) : 55,	(32,460) : 77,	(33,460) : 124,	(34,460) : 83,	(35,460) : 144,	(36,460) : 387,	(37,460) : 436,	(38,460) : 62,	(39,460) : 56,	(40,460) : 778,
(1,461) : 115,	(2,461) : 168,	(3,461) : 655,	(4,461) : 490,	(5,461) : 207,	(6,461) : 163,	(7,461) : 113,	(8,461) : 922,	(9,461) : 423,	(10,461) : 113,	(11,461) : 169,	(12,461) : 415,	(13,461) : 72,	(14,461) : 418,	(15,461) : 337,	(16,461) : 532,	(17,461) : 442,	(18,461) : 195,	(19,461) : 352,	(20,461) : 111,	(21,461) : 225,	(22,461) : 220,	(23,461) : 190,	(24,461) : 445,	(25,461) : 814,	(26,461) : 126,	(27,461) : 78,	(28,461) : 111,	(29,461) : 377,	(30,461) : 112,	(31,461) : 160,	(32,461) : 75,	(33,461) : 59,	(34,461) : 170,	(35,461) : 235,	(36,461) : 473,	(37,461) : 498,	(38,461) : 174,	(39,461) : 170,	(40,461) : 832,
(1,462) : 6,	(2,462) : 56,	(3,462) : 653,	(4,462) : 596,	(5,462) : 102,	(6,462) : 72,	(7,462) : 6,	(8,462) : 901,	(9,462) : 354,	(10,462) : 3,	(11,462) : 56,	(12,462) : 527,	(13,462) : 167,	(14,462) : 349,	(15,462) : 440,	(16,462) : 643,	(17,462) : 547,	(18,462) : 96,	(19,462) : 463,	(20,462) : 2,	(21,462) : 312,	(22,462) : 151,	(23,462) : 93,	(24,462) : 549,	(25,462) : 773,	(26,462) : 231,	(27,462) : 173,	(28,462) : 3,	(29,462) : 468,	(30,462) : 5,	(31,462) : 63,	(32,462) : 83,	(33,462) : 118,	(34,462) : 93,	(35,462) : 137,	(36,462) : 395,	(37,462) : 446,	(38,462) : 66,	(39,462) : 58,	(40,462) : 788,
(1,463) : 158,	(2,463) : 213,	(3,463) : 746,	(4,463) : 517,	(5,463) : 262,	(6,463) : 227,	(7,463) : 163,	(8,463) : 1014,	(9,463) : 503,	(10,463) : 161,	(11,463) : 213,	(12,463) : 393,	(13,463) : 149,	(14,463) : 498,	(15,463) : 373,	(16,463) : 538,	(17,463) : 472,	(18,463) : 255,	(19,463) : 329,	(20,463) : 158,	(21,463) : 290,	(22,463) : 297,	(23,463) : 252,	(24,463) : 474,	(25,463) : 904,	(26,463) : 180,	(27,463) : 153,	(28,463) : 159,	(29,463) : 429,	(30,463) : 156,	(31,463) : 221,	(32,463) : 161,	(33,463) : 43,	(34,463) : 241,	(35,463) : 237,	(36,463) : 550,	(37,463) : 584,	(38,463) : 225,	(39,463) : 215,	(40,463) : 921,
(1,464) : 296,	(2,464) : 263,	(3,464) : 413,	(4,464) : 678,	(5,464) : 217,	(6,464) : 221,	(7,464) : 286,	(8,464) : 625,	(9,464) : 81,	(10,464) : 288,	(11,464) : 264,	(12,464) : 715,	(13,464) : 341,	(14,464) : 76,	(15,464) : 533,	(16,464) : 764,	(17,464) : 629,	(18,464) : 208,	(19,464) : 657,	(20,464) : 292,	(21,464) : 384,	(22,464) : 140,	(23,464) : 208,	(24,464) : 631,	(25,464) : 484,	(26,464) : 388,	(27,464) : 343,	(28,464) : 290,	(29,464) : 508,	(30,464) : 296,	(31,464) : 231,	(32,464) : 274,	(33,464) : 391,	(34,464) : 199,	(35,464) : 358,	(36,464) : 141,	(37,464) : 155,	(38,464) : 246,	(39,464) : 260,	(40,464) : 497,
(1,465) : 461,	(2,465) : 426,	(3,465) : 325,	(4,465) : 778,	(5,465) : 377,	(6,465) : 386,	(7,465) : 452,	(8,465) : 480,	(9,465) : 117,	(10,465) : 454,	(11,465) : 427,	(12,465) : 853,	(13,465) : 496,	(14,465) : 120,	(15,465) : 648,	(16,465) : 876,	(17,465) : 733,	(18,465) : 370,	(19,465) : 799,	(20,465) : 458,	(21,465) : 508,	(22,465) : 306,	(23,465) : 371,	(24,465) : 735,	(25,465) : 322,	(26,465) : 534,	(27,465) : 497,	(28,465) : 456,	(29,465) : 604,	(30,465) : 461,	(31,465) : 396,	(32,465) : 437,	(33,465) : 556,	(34,465) : 364,	(35,465) : 513,	(36,465) : 127,	(37,465) : 18,	(38,465) : 409,	(39,465) : 422,	(40,465) : 333,
(1,466) : 584,	(2,466) : 631,	(3,466) : 721,	(4,466) : 46,	(5,466) : 655,	(6,466) : 606,	(7,466) : 580,	(8,466) : 1006,	(9,466) : 761,	(10,466) : 581,	(11,466) : 632,	(12,466) : 213,	(13,466) : 415,	(14,466) : 757,	(15,466) : 151,	(16,466) : 87,	(17,466) : 61,	(18,466) : 638,	(19,466) : 230,	(20,466) : 579,	(21,466) : 301,	(22,466) : 618,	(23,466) : 632,	(24,466) : 61,	(25,466) : 991,	(26,466) : 351,	(27,466) : 409,	(28,466) : 579,	(29,466) : 192,	(30,466) : 581,	(31,466) : 609,	(32,466) : 508,	(33,466) : 503,	(34,466) : 598,	(35,466) : 707,	(36,466) : 823,	(37,466) : 773,	(38,466) : 632,	(39,466) : 632,	(40,466) : 1018,
(1,467) : 463,	(2,467) : 495,	(3,467) : 474,	(4,467) : 231,	(5,467) : 500,	(6,467) : 454,	(7,467) : 455,	(8,467) : 763,	(9,467) : 538,	(10,467) : 457,	(11,467) : 496,	(12,467) : 391,	(13,467) : 306,	(14,467) : 534,	(15,467) : 145,	(16,467) : 339,	(17,467) : 191,	(18,467) : 481,	(19,467) : 365,	(20,467) : 457,	(21,467) : 152,	(22,467) : 438,	(23,467) : 476,	(24,467) : 192,	(25,467) : 739,	(26,467) : 261,	(27,467) : 300,	(28,467) : 456,	(29,467) : 62,	(30,467) : 460,	(31,467) : 461,	(32,467) : 375,	(33,467) : 430,	(34,467) : 438,	(35,467) : 595,	(36,467) : 600,	(37,467) : 532,	(38,467) : 489,	(39,467) : 494,	(40,467) : 766,
(1,468) : 375,	(2,468) : 428,	(3,468) : 736,	(4,468) : 278,	(5,468) : 464,	(6,468) : 417,	(7,468) : 374,	(8,468) : 1025,	(9,468) : 637,	(10,468) : 374,	(11,468) : 429,	(12,468) : 160,	(13,468) : 226,	(14,468) : 633,	(15,468) : 170,	(16,468) : 284,	(17,468) : 240,	(18,468) : 450,	(19,468) : 100,	(20,468) : 372,	(21,468) : 210,	(22,468) : 455,	(23,468) : 445,	(24,468) : 243,	(25,468) : 962,	(26,468) : 170,	(27,468) : 222,	(28,468) : 372,	(29,468) : 257,	(30,468) : 372,	(31,468) : 416,	(32,468) : 318,	(33,468) : 275,	(34,468) : 417,	(35,468) : 485,	(36,468) : 695,	(37,468) : 683,	(38,468) : 434,	(39,468) : 430,	(40,468) : 985,
(1,469) : 177,	(2,469) : 147,	(3,469) : 507,	(4,469) : 634,	(5,469) : 105,	(6,469) : 102,	(7,469) : 168,	(8,469) : 737,	(9,469) : 184,	(10,469) : 170,	(11,469) : 148,	(12,469) : 633,	(13,469) : 248,	(14,469) : 179,	(15,469) : 481,	(16,469) : 707,	(17,469) : 584,	(18,469) : 93,	(19,469) : 572,	(20,469) : 174,	(21,469) : 331,	(22,469) : 22,	(23,469) : 92,	(24,469) : 586,	(25,469) : 602,	(26,469) : 304,	(27,469) : 252,	(28,469) : 172,	(29,469) : 476,	(30,469) : 177,	(31,469) : 113,	(32,469) : 167,	(33,469) : 277,	(34,469) : 80,	(35,469) : 249,	(36,469) : 232,	(37,469) : 273,	(38,469) : 129,	(39,469) : 143,	(40,469) : 615,
(1,470) : 311,	(2,470) : 286,	(3,470) : 370,	(4,470) : 638,	(5,470) : 245,	(6,470) : 239,	(7,470) : 300,	(8,470) : 598,	(9,470) : 118,	(10,470) : 303,	(11,470) : 287,	(12,470) : 691,	(13,470) : 329,	(14,470) : 114,	(15,470) : 497,	(16,470) : 728,	(17,470) : 590,	(18,470) : 233,	(19,470) : 635,	(20,470) : 306,	(21,470) : 351,	(22,470) : 159,	(23,470) : 232,	(24,470) : 592,	(25,470) : 470,	(26,470) : 368,	(27,470) : 330,	(28,470) : 304,	(29,470) : 466,	(30,470) : 310,	(31,470) : 250,	(32,470) : 274,	(33,470) : 395,	(34,470) : 215,	(35,470) : 389,	(36,470) : 180,	(37,470) : 152,	(38,470) : 269,	(39,470) : 283,	(40,470) : 486,
(1,471) : 606,	(2,471) : 573,	(3,471) : 275,	(4,471) : 861,	(5,471) : 524,	(6,471) : 531,	(7,471) : 597,	(8,471) : 347,	(9,471) : 262,	(10,471) : 599,	(11,471) : 574,	(12,471) : 966,	(13,471) : 627,	(14,471) : 266,	(15,471) : 745,	(16,471) : 967,	(17,471) : 819,	(18,471) : 517,	(19,471) : 916,	(20,471) : 603,	(21,471) : 616,	(22,471) : 451,	(23,471) : 518,	(24,471) : 820,	(25,471) : 175,	(26,471) : 659,	(27,471) : 627,	(28,471) : 601,	(29,471) : 688,	(30,471) : 606,	(31,471) : 542,	(32,471) : 577,	(33,471) : 697,	(34,471) : 509,	(35,471) : 660,	(36,471) : 255,	(37,471) : 156,	(38,471) : 555,	(39,471) : 569,	(40,471) : 186,
(1,472) : 252,	(2,472) : 223,	(3,472) : 435,	(4,472) : 641,	(5,472) : 181,	(6,472) : 178,	(7,472) : 242,	(8,472) : 661,	(9,472) : 127,	(10,472) : 244,	(11,472) : 224,	(12,472) : 669,	(13,472) : 294,	(14,472) : 122,	(15,472) : 494,	(16,472) : 724,	(17,472) : 592,	(18,472) : 169,	(19,472) : 611,	(20,472) : 248,	(21,472) : 344,	(22,472) : 97,	(23,472) : 168,	(24,472) : 594,	(25,472) : 527,	(26,472) : 341,	(27,472) : 295,	(28,472) : 245,	(29,472) : 474,	(30,472) : 251,	(31,472) : 189,	(32,472) : 226,	(33,472) : 343,	(34,472) : 154,	(35,472) : 324,	(36,472) : 185,	(37,472) : 200,	(38,472) : 206,	(39,472) : 220,	(40,472) : 541,
(1,473) : 553,	(2,473) : 593,	(3,473) : 609,	(4,473) : 83,	(5,473) : 609,	(6,473) : 561,	(7,473) : 546,	(8,473) : 893,	(9,473) : 680,	(10,473) : 548,	(11,473) : 594,	(12,473) : 302,	(13,473) : 383,	(14,473) : 676,	(15,473) : 116,	(16,473) : 198,	(17,473) : 57,	(18,473) : 590,	(19,473) : 300,	(20,473) : 547,	(21,473) : 242,	(22,473) : 559,	(23,473) : 585,	(24,473) : 56,	(25,473) : 883,	(26,473) : 323,	(27,473) : 376,	(28,473) : 546,	(29,473) : 96,	(30,473) : 550,	(31,473) : 565,	(32,473) : 469,	(33,473) : 491,	(34,473) : 549,	(35,473) : 682,	(36,473) : 742,	(37,473) : 680,	(38,473) : 591,	(39,473) : 593,	(40,473) : 910,
(1,474) : 194,	(2,474) : 250,	(3,474) : 735,	(4,474) : 466,	(5,474) : 295,	(6,474) : 255,	(7,474) : 196,	(8,474) : 1009,	(9,474) : 522,	(10,474) : 195,	(11,474) : 250,	(12,474) : 343,	(13,474) : 130,	(14,474) : 517,	(15,474) : 325,	(16,474) : 486,	(17,474) : 421,	(18,474) : 286,	(19,474) : 279,	(20,474) : 192,	(21,474) : 254,	(22,474) : 318,	(23,474) : 282,	(24,474) : 424,	(25,474) : 909,	(26,474) : 144,	(27,474) : 133,	(28,474) : 193,	(29,474) : 386,	(30,474) : 192,	(31,474) : 251,	(32,474) : 174,	(33,474) : 79,	(34,474) : 266,	(35,474) : 286,	(36,474) : 571,	(37,474) : 595,	(38,474) : 260,	(39,474) : 252,	(40,474) : 927,
(1,475) : 239,	(2,475) : 246,	(3,475) : 435,	(4,475) : 484,	(5,475) : 234,	(6,475) : 197,	(7,475) : 228,	(8,475) : 703,	(9,475) : 277,	(10,475) : 230,	(11,475) : 247,	(12,475) : 524,	(13,475) : 179,	(14,475) : 273,	(15,475) : 336,	(16,475) : 567,	(17,475) : 435,	(18,475) : 215,	(19,475) : 469,	(20,475) : 232,	(21,475) : 187,	(22,475) : 158,	(23,475) : 210,	(24,475) : 437,	(25,475) : 607,	(26,475) : 206,	(27,475) : 178,	(28,475) : 230,	(29,475) : 318,	(30,475) : 237,	(31,475) : 207,	(32,475) : 164,	(33,475) : 277,	(34,475) : 174,	(35,475) : 358,	(36,475) : 338,	(37,475) : 313,	(38,475) : 235,	(39,475) : 244,	(40,475) : 627,
(1,476) : 123,	(2,476) : 124,	(3,476) : 536,	(4,476) : 556,	(5,476) : 117,	(6,476) : 75,	(7,476) : 112,	(8,476) : 788,	(9,476) : 271,	(10,476) : 115,	(11,476) : 125,	(12,476) : 541,	(13,476) : 156,	(14,476) : 266,	(15,476) : 400,	(16,476) : 622,	(17,476) : 506,	(18,476) : 98,	(19,476) : 480,	(20,476) : 117,	(21,476) : 254,	(22,476) : 74,	(23,476) : 93,	(24,476) : 508,	(25,476) : 668,	(26,476) : 213,	(27,476) : 159,	(28,476) : 115,	(29,476) : 406,	(30,476) : 122,	(31,476) : 85,	(32,476) : 78,	(33,476) : 196,	(34,476) : 55,	(35,476) : 236,	(36,476) : 323,	(37,476) : 346,	(38,476) : 113,	(39,476) : 122,	(40,476) : 684,
(1,477) : 494,	(2,477) : 543,	(3,477) : 714,	(4,477) : 133,	(5,477) : 571,	(6,477) : 522,	(7,477) : 491,	(8,477) : 1004,	(9,477) : 701,	(10,477) : 491,	(11,477) : 544,	(12,477) : 154,	(13,477) : 328,	(14,477) : 697,	(15,477) : 101,	(16,477) : 152,	(17,477) : 104,	(18,477) : 555,	(19,477) : 148,	(20,477) : 490,	(21,477) : 237,	(22,477) : 543,	(23,477) : 549,	(24,477) : 106,	(25,477) : 969,	(26,477) : 264,	(27,477) : 323,	(28,477) : 489,	(29,477) : 182,	(30,477) : 491,	(31,477) : 524,	(32,477) : 423,	(33,477) : 407,	(34,477) : 517,	(35,477) : 614,	(36,477) : 762,	(37,477) : 725,	(38,477) : 546,	(39,477) : 544,	(40,477) : 995,
(1,478) : 213,	(2,478) : 192,	(3,478) : 456,	(4,478) : 603,	(5,478) : 157,	(6,478) : 143,	(7,478) : 202,	(8,478) : 694,	(9,478) : 175,	(10,478) : 205,	(11,478) : 193,	(12,478) : 622,	(13,478) : 245,	(14,478) : 170,	(15,478) : 453,	(16,478) : 682,	(17,478) : 553,	(18,478) : 142,	(19,478) : 563,	(20,478) : 208,	(21,478) : 302,	(22,478) : 66,	(23,478) : 140,	(24,478) : 555,	(25,478) : 568,	(26,478) : 293,	(27,478) : 247,	(28,478) : 206,	(29,478) : 439,	(30,478) : 212,	(31,478) : 154,	(32,478) : 179,	(33,478) : 298,	(34,478) : 118,	(35,478) : 300,	(36,478) : 232,	(37,478) : 245,	(38,478) : 176,	(39,478) : 189,	(40,478) : 583,
(1,479) : 222,	(2,479) : 204,	(3,479) : 443,	(4,479) : 594,	(5,479) : 170,	(6,479) : 154,	(7,479) : 211,	(8,479) : 685,	(9,479) : 177,	(10,479) : 214,	(11,479) : 205,	(12,479) : 618,	(13,479) : 244,	(14,479) : 172,	(15,479) : 444,	(16,479) : 674,	(17,479) : 544,	(18,479) : 155,	(19,479) : 560,	(20,479) : 217,	(21,479) : 294,	(22,479) : 80,	(23,479) : 152,	(24,479) : 546,	(25,479) : 561,	(26,479) : 291,	(27,479) : 246,	(28,479) : 215,	(29,479) : 429,	(30,479) : 221,	(31,479) : 165,	(32,479) : 183,	(33,479) : 303,	(34,479) : 128,	(35,479) : 312,	(36,479) : 236,	(37,479) : 241,	(38,479) : 188,	(39,479) : 201,	(40,479) : 577,
(1,480) : 232,	(2,480) : 279,	(3,480) : 611,	(4,480) : 367,	(5,480) : 306,	(6,480) : 257,	(7,480) : 228,	(8,480) : 893,	(9,480) : 467,	(10,480) : 229,	(11,480) : 280,	(12,480) : 330,	(13,480) : 63,	(14,480) : 463,	(15,480) : 213,	(16,480) : 417,	(17,480) : 318,	(18,480) : 290,	(19,480) : 270,	(20,480) : 227,	(21,480) : 113,	(22,480) : 286,	(23,480) : 285,	(24,480) : 321,	(25,480) : 811,	(26,480) : 2,	(27,480) : 57,	(28,480) : 227,	(29,480) : 255,	(30,480) : 229,	(31,480) : 259,	(32,480) : 158,	(33,480) : 169,	(34,480) : 254,	(35,480) : 359,	(36,480) : 525,	(37,480) : 518,	(38,480) : 281,	(39,480) : 280,	(40,480) : 832,
(1,481) : 387,	(2,481) : 346,	(3,481) : 409,	(4,481) : 776,	(5,481) : 294,	(6,481) : 311,	(7,481) : 378,	(8,481) : 575,	(9,481) : 29,	(10,481) : 380,	(11,481) : 346,	(12,481) : 821,	(13,481) : 448,	(14,481) : 34,	(15,481) : 635,	(16,481) : 866,	(17,481) : 728,	(18,481) : 291,	(19,481) : 764,	(20,481) : 384,	(21,481) : 488,	(22,481) : 235,	(23,481) : 292,	(24,481) : 730,	(25,481) : 412,	(26,481) : 495,	(27,481) : 450,	(28,481) : 382,	(29,481) : 604,	(30,481) : 387,	(31,481) : 320,	(32,481) : 377,	(33,481) : 490,	(34,481) : 292,	(35,481) : 425,	(36,481) : 47,	(37,481) : 96,	(38,481) : 329,	(39,481) : 342,	(40,481) : 421,
(1,482) : 160,	(2,482) : 146,	(3,482) : 502,	(4,482) : 583,	(5,482) : 121,	(6,482) : 94,	(7,482) : 149,	(8,482) : 747,	(9,482) : 223,	(10,482) : 151,	(11,482) : 147,	(12,482) : 582,	(13,482) : 200,	(14,482) : 218,	(15,482) : 429,	(16,482) : 655,	(17,482) : 533,	(18,482) : 103,	(19,482) : 522,	(20,482) : 155,	(21,482) : 279,	(22,482) : 41,	(23,482) : 99,	(24,482) : 535,	(25,482) : 623,	(26,482) : 253,	(27,482) : 202,	(28,482) : 152,	(29,482) : 426,	(30,482) : 159,	(31,482) : 106,	(32,482) : 126,	(33,482) : 243,	(34,482) : 69,	(35,482) : 257,	(36,482) : 276,	(37,482) : 299,	(38,482) : 131,	(39,482) : 143,	(40,482) : 638,
(1,483) : 331,	(2,483) : 306,	(3,483) : 354,	(4,483) : 646,	(5,483) : 264,	(6,483) : 259,	(7,483) : 320,	(8,483) : 578,	(9,483) : 114,	(10,483) : 323,	(11,483) : 307,	(12,483) : 705,	(13,483) : 346,	(14,483) : 111,	(15,483) : 508,	(16,483) : 739,	(17,483) : 599,	(18,483) : 252,	(19,483) : 650,	(20,483) : 326,	(21,483) : 363,	(22,483) : 178,	(23,483) : 251,	(24,483) : 601,	(25,483) : 450,	(26,483) : 384,	(27,483) : 347,	(28,483) : 324,	(29,483) : 474,	(30,483) : 330,	(31,483) : 270,	(32,483) : 294,	(33,483) : 414,	(34,483) : 234,	(35,483) : 407,	(36,483) : 175,	(37,483) : 134,	(38,483) : 289,	(39,483) : 302,	(40,483) : 466,
(1,484) : 319,	(2,484) : 301,	(3,484) : 347,	(4,484) : 602,	(5,484) : 265,	(6,484) : 251,	(7,484) : 308,	(8,484) : 589,	(9,484) : 158,	(10,484) : 310,	(11,484) : 302,	(12,484) : 665,	(13,484) : 314,	(14,484) : 154,	(15,484) : 464,	(16,484) : 694,	(17,484) : 555,	(18,484) : 251,	(19,484) : 611,	(20,484) : 314,	(21,484) : 321,	(22,484) : 175,	(23,484) : 249,	(24,484) : 556,	(25,484) : 473,	(26,484) : 348,	(27,484) : 314,	(28,484) : 311,	(29,484) : 429,	(30,484) : 318,	(31,484) : 263,	(32,484) : 271,	(33,484) : 392,	(34,484) : 226,	(35,484) : 408,	(36,484) : 219,	(37,484) : 171,	(38,484) : 285,	(39,484) : 298,	(40,484) : 492,
(1,485) : 70,	(2,485) : 117,	(3,485) : 720,	(4,485) : 594,	(5,485) : 169,	(6,485) : 146,	(7,485) : 80,	(8,485) : 973,	(9,485) : 428,	(10,485) : 77,	(11,485) : 117,	(12,485) : 493,	(13,485) : 178,	(14,485) : 423,	(15,485) : 443,	(16,485) : 628,	(17,485) : 546,	(18,485) : 168,	(19,485) : 429,	(20,485) : 74,	(21,485) : 332,	(22,485) : 226,	(23,485) : 165,	(24,485) : 549,	(25,485) : 848,	(26,485) : 233,	(27,485) : 184,	(28,485) : 76,	(29,485) : 484,	(30,485) : 70,	(31,485) : 137,	(32,485) : 128,	(33,485) : 73,	(34,485) : 167,	(35,485) : 137,	(36,485) : 468,	(37,485) : 520,	(38,485) : 132,	(39,485) : 120,	(40,485) : 862,
(1,486) : 889,	(2,486) : 869,	(3,486) : 252,	(4,486) : 927,	(5,486) : 827,	(6,486) : 820,	(7,486) : 878,	(8,486) : 57,	(9,486) : 592,	(10,486) : 880,	(11,486) : 870,	(12,486) : 1108,	(13,486) : 852,	(14,486) : 595,	(15,486) : 862,	(16,486) : 1044,	(17,486) : 900,	(18,486) : 816,	(19,486) : 1073,	(20,486) : 884,	(21,486) : 779,	(22,486) : 741,	(23,486) : 815,	(24,486) : 900,	(25,486) : 230,	(26,486) : 861,	(27,486) : 850,	(28,486) : 881,	(29,486) : 780,	(30,486) : 888,	(31,486) : 832,	(32,486) : 834,	(33,486) : 953,	(34,486) : 795,	(35,486) : 971,	(36,486) : 601,	(37,486) : 479,	(38,486) : 852,	(39,486) : 865,	(40,486) : 251,
(1,487) : 230,	(2,487) : 282,	(3,487) : 674,	(4,487) : 387,	(5,487) : 318,	(6,487) : 272,	(7,487) : 228,	(8,487) : 955,	(9,487) : 509,	(10,487) : 228,	(11,487) : 283,	(12,487) : 303,	(13,487) : 89,	(14,487) : 504,	(15,487) : 241,	(16,487) : 420,	(17,487) : 341,	(18,487) : 305,	(19,487) : 240,	(20,487) : 226,	(21,487) : 172,	(22,487) : 316,	(23,487) : 300,	(24,487) : 343,	(25,487) : 868,	(26,487) : 65,	(27,487) : 88,	(28,487) : 226,	(29,487) : 299,	(30,487) : 227,	(31,487) : 271,	(32,487) : 174,	(33,487) : 138,	(34,487) : 274,	(35,487) : 344,	(36,487) : 564,	(37,487) : 568,	(38,487) : 288,	(39,487) : 284,	(40,487) : 889,
(1,488) : 441,	(2,488) : 495,	(3,488) : 792,	(4,488) : 263,	(5,488) : 532,	(6,488) : 486,	(7,488) : 440,	(8,488) : 1082,	(9,488) : 708,	(10,488) : 440,	(11,488) : 495,	(12,488) : 89,	(13,488) : 296,	(14,488) : 703,	(15,488) : 196,	(16,488) : 241,	(17,488) : 236,	(18,488) : 519,	(19,488) : 31,	(20,488) : 438,	(21,488) : 272,	(22,488) : 525,	(23,488) : 514,	(24,488) : 238,	(25,488) : 1026,	(26,488) : 241,	(27,488) : 292,	(28,488) : 438,	(29,488) : 287,	(30,488) : 438,	(31,488) : 485,	(32,488) : 387,	(33,488) : 335,	(34,488) : 487,	(35,488) : 545,	(36,488) : 766,	(37,488) : 752,	(38,488) : 501,	(39,488) : 497,	(40,488) : 1049,
(1,489) : 137,	(2,489) : 148,	(3,489) : 528,	(4,489) : 526,	(5,489) : 146,	(6,489) : 102,	(7,489) : 126,	(8,489) : 786,	(9,489) : 287,	(10,489) : 128,	(11,489) : 149,	(12,489) : 516,	(13,489) : 133,	(14,489) : 283,	(15,489) : 371,	(16,489) : 594,	(17,489) : 476,	(18,489) : 127,	(19,489) : 456,	(20,489) : 131,	(21,489) : 224,	(22,489) : 100,	(23,489) : 122,	(24,489) : 478,	(25,489) : 672,	(26,489) : 188,	(27,489) : 136,	(28,489) : 128,	(29,489) : 377,	(30,489) : 135,	(31,489) : 110,	(32,489) : 71,	(33,489) : 192,	(34,489) : 84,	(35,489) : 258,	(36,489) : 342,	(37,489) : 356,	(38,489) : 138,	(39,489) : 146,	(40,489) : 690,
(1,490) : 321,	(2,490) : 284,	(3,490) : 417,	(4,490) : 714,	(5,490) : 235,	(6,490) : 245,	(7,490) : 312,	(8,490) : 614,	(9,490) : 44,	(10,490) : 314,	(11,490) : 285,	(12,490) : 752,	(13,490) : 377,	(14,490) : 39,	(15,490) : 570,	(16,490) : 801,	(17,490) : 666,	(18,490) : 228,	(19,490) : 694,	(20,490) : 318,	(21,490) : 422,	(22,490) : 167,	(23,490) : 229,	(24,490) : 668,	(25,490) : 464,	(26,490) : 424,	(27,490) : 379,	(28,490) : 316,	(29,490) : 544,	(30,490) : 321,	(31,490) : 255,	(32,490) : 306,	(33,490) : 421,	(34,490) : 225,	(35,490) : 372,	(36,490) : 104,	(37,490) : 134,	(38,490) : 267,	(39,490) : 280,	(40,490) : 475,
(1,491) : 248,	(2,491) : 214,	(3,491) : 455,	(4,491) : 666,	(5,491) : 168,	(6,491) : 173,	(7,491) : 239,	(8,491) : 673,	(9,491) : 115,	(10,491) : 241,	(11,491) : 215,	(12,491) : 686,	(13,491) : 307,	(14,491) : 110,	(15,491) : 517,	(16,491) : 746,	(17,491) : 616,	(18,491) : 159,	(19,491) : 627,	(20,491) : 245,	(21,491) : 366,	(22,491) : 93,	(23,491) : 159,	(24,491) : 618,	(25,491) : 532,	(26,491) : 357,	(27,491) : 309,	(28,491) : 243,	(29,491) : 500,	(30,491) : 248,	(31,491) : 183,	(32,491) : 233,	(33,491) : 346,	(34,491) : 151,	(35,491) : 310,	(36,491) : 169,	(37,491) : 203,	(38,491) : 197,	(39,491) : 211,	(40,491) : 545,
(1,492) : 549,	(2,492) : 597,	(3,492) : 719,	(4,492) : 78,	(5,492) : 623,	(6,492) : 574,	(7,492) : 545,	(8,492) : 1006,	(9,492) : 738,	(10,492) : 546,	(11,492) : 598,	(12,492) : 184,	(13,492) : 381,	(14,492) : 734,	(15,492) : 126,	(16,492) : 105,	(17,492) : 66,	(18,492) : 606,	(19,492) : 195,	(20,492) : 545,	(21,492) : 275,	(22,492) : 589,	(23,492) : 600,	(24,492) : 67,	(25,492) : 983,	(26,492) : 317,	(27,492) : 375,	(28,492) : 544,	(29,492) : 184,	(30,492) : 546,	(31,492) : 576,	(32,492) : 475,	(33,492) : 466,	(34,492) : 567,	(35,492) : 671,	(36,492) : 800,	(37,492) : 755,	(38,492) : 599,	(39,492) : 598,	(40,492) : 1010,
(1,493) : 604,	(2,493) : 655,	(3,493) : 813,	(4,493) : 142,	(5,493) : 685,	(6,493) : 637,	(7,493) : 601,	(8,493) : 1100,	(9,493) : 820,	(10,493) : 602,	(11,493) : 656,	(12,493) : 140,	(13,493) : 442,	(14,493) : 816,	(15,493) : 213,	(16,493) : 58,	(17,493) : 155,	(18,493) : 670,	(19,493) : 180,	(20,493) : 600,	(21,493) : 356,	(22,493) : 661,	(23,493) : 664,	(24,493) : 156,	(25,493) : 1077,	(26,493) : 378,	(27,493) : 437,	(28,493) : 600,	(29,493) : 278,	(30,493) : 601,	(31,493) : 638,	(32,493) : 537,	(33,493) : 509,	(34,493) : 633,	(35,493) : 719,	(36,493) : 881,	(37,493) : 842,	(38,493) : 658,	(39,493) : 656,	(40,493) : 1103,
(1,494) : 1065,	(2,494) : 1029,	(3,494) : 559,	(4,494) : 1248,	(5,494) : 979,	(6,494) : 989,	(7,494) : 1055,	(8,494) : 306,	(9,494) : 712,	(10,494) : 1057,	(11,494) : 1030,	(12,494) : 1396,	(13,494) : 1081,	(14,494) : 717,	(15,494) : 1159,	(16,494) : 1363,	(17,494) : 1215,	(18,494) : 973,	(19,494) : 1352,	(20,494) : 1061,	(21,494) : 1049,	(22,494) : 909,	(23,494) : 974,	(24,494) : 1215,	(25,494) : 297,	(26,494) : 1107,	(27,494) : 1081,	(28,494) : 1059,	(29,494) : 1087,	(30,494) : 1065,	(31,494) : 999,	(32,494) : 1036,	(33,494) : 1156,	(34,494) : 968,	(35,494) : 1109,	(36,494) : 688,	(37,494) : 615,	(38,494) : 1011,	(39,494) : 1025,	(40,494) : 276,
(1,495) : 459,	(2,495) : 495,	(3,495) : 521,	(4,495) : 194,	(5,495) : 505,	(6,495) : 458,	(7,495) : 452,	(8,495) : 810,	(9,495) : 565,	(10,495) : 453,	(11,495) : 496,	(12,495) : 345,	(13,495) : 295,	(14,495) : 561,	(15,495) : 99,	(16,495) : 297,	(17,495) : 150,	(18,495) : 486,	(19,495) : 320,	(20,495) : 453,	(21,495) : 143,	(22,495) : 450,	(23,495) : 481,	(24,495) : 151,	(25,495) : 783,	(26,495) : 243,	(27,495) : 289,	(28,495) : 452,	(29,495) : 19,	(30,495) : 456,	(31,495) : 464,	(32,495) : 373,	(33,495) : 414,	(34,495) : 444,	(35,495) : 591,	(36,495) : 627,	(37,495) : 566,	(38,495) : 491,	(39,495) : 494,	(40,495) : 810,
(1,496) : 171,	(2,496) : 169,	(3,496) : 488,	(4,496) : 545,	(5,496) : 153,	(6,496) : 118,	(7,496) : 160,	(8,496) : 742,	(9,496) : 246,	(10,496) : 162,	(11,496) : 170,	(12,496) : 552,	(13,496) : 174,	(14,496) : 241,	(15,496) : 391,	(16,496) : 619,	(17,496) : 495,	(18,496) : 134,	(19,496) : 493,	(20,496) : 165,	(21,496) : 242,	(22,496) : 80,	(23,496) : 129,	(24,496) : 497,	(25,496) : 627,	(26,496) : 223,	(27,496) : 176,	(28,496) : 163,	(29,496) : 387,	(30,496) : 170,	(31,496) : 129,	(32,496) : 116,	(33,496) : 237,	(34,496) : 94,	(35,496) : 282,	(36,496) : 303,	(37,496) : 310,	(38,496) : 156,	(39,496) : 167,	(40,496) : 644,
(1,497) : 25,	(2,497) : 42,	(3,497) : 672,	(4,497) : 625,	(5,497) : 94,	(6,497) : 79,	(7,497) : 33,	(8,497) : 915,	(9,497) : 356,	(10,497) : 31,	(11,497) : 41,	(12,497) : 550,	(13,497) : 196,	(14,497) : 352,	(15,497) : 469,	(16,497) : 670,	(17,497) : 576,	(18,497) : 94,	(19,497) : 486,	(20,497) : 31,	(21,497) : 342,	(22,497) : 160,	(23,497) : 92,	(24,497) : 578,	(25,497) : 782,	(26,497) : 259,	(27,497) : 202,	(28,497) : 32,	(29,497) : 498,	(30,497) : 28,	(31,497) : 68,	(32,497) : 113,	(33,497) : 134,	(34,497) : 103,	(35,497) : 108,	(36,497) : 394,	(37,497) : 453,	(38,497) : 57,	(39,497) : 45,	(40,497) : 795,
(1,498) : 527,	(2,498) : 570,	(3,498) : 634,	(4,498) : 78,	(5,498) : 589,	(6,498) : 541,	(7,498) : 522,	(8,498) : 921,	(9,498) : 678,	(10,498) : 523,	(11,498) : 571,	(12,498) : 259,	(13,498) : 356,	(14,498) : 674,	(15,498) : 83,	(16,498) : 177,	(17,498) : 30,	(18,498) : 571,	(19,498) : 255,	(20,498) : 522,	(21,498) : 226,	(22,498) : 545,	(23,498) : 566,	(24,498) : 31,	(25,498) : 902,	(26,498) : 295,	(27,498) : 350,	(28,498) : 521,	(29,498) : 102,	(30,498) : 524,	(31,498) : 544,	(32,498) : 446,	(33,498) : 459,	(34,498) : 530,	(35,498) : 655,	(36,498) : 740,	(37,498) : 685,	(38,498) : 569,	(39,498) : 571,	(40,498) : 929,
(1,499) : 803,	(2,499) : 774,	(3,499) : 273,	(4,499) : 957,	(5,499) : 728,	(6,499) : 730,	(7,499) : 793,	(8,499) : 148,	(9,499) : 471,	(10,499) : 796,	(11,499) : 775,	(12,499) : 1104,	(13,499) : 800,	(14,499) : 475,	(15,499) : 866,	(16,499) : 1071,	(17,499) : 923,	(18,499) : 719,	(19,499) : 1061,	(20,499) : 799,	(21,499) : 758,	(22,499) : 649,	(23,499) : 719,	(24,499) : 924,	(25,499) : 53,	(26,499) : 821,	(27,499) : 799,	(28,499) : 797,	(29,499) : 795,	(30,499) : 803,	(31,499) : 741,	(32,499) : 763,	(33,499) : 884,	(34,499) : 706,	(35,499) : 867,	(36,499) : 466,	(37,499) : 361,	(38,499) : 757,	(39,499) : 771,	(40,499) : 75,
(1,500) : 265,	(2,500) : 249,	(3,500) : 398,	(4,500) : 584,	(5,500) : 216,	(6,500) : 198,	(7,500) : 253,	(8,500) : 643,	(9,500) : 172,	(10,500) : 256,	(11,500) : 250,	(12,500) : 628,	(13,500) : 266,	(14,500) : 168,	(15,500) : 440,	(16,500) : 671,	(17,500) : 536,	(18,500) : 200,	(19,500) : 572,	(20,500) : 259,	(21,500) : 292,	(22,500) : 125,	(23,500) : 198,	(24,500) : 538,	(25,500) : 526,	(26,500) : 305,	(27,500) : 267,	(28,500) : 257,	(29,500) : 415,	(30,500) : 264,	(31,500) : 210,	(32,500) : 218,	(33,500) : 338,	(34,500) : 173,	(35,500) : 358,	(36,500) : 234,	(37,500) : 214,	(38,500) : 233,	(39,500) : 246,	(40,500) : 543,
(1,501) : 313,	(2,501) : 353,	(3,501) : 557,	(4,501) : 295,	(5,501) : 371,	(6,501) : 322,	(7,501) : 306,	(8,501) : 845,	(9,501) : 484,	(10,501) : 307,	(11,501) : 354,	(12,501) : 323,	(13,501) : 144,	(14,501) : 479,	(15,501) : 139,	(16,501) : 364,	(17,501) : 245,	(18,501) : 353,	(19,501) : 274,	(20,501) : 307,	(21,501) : 30,	(22,501) : 330,	(23,501) : 347,	(24,501) : 247,	(25,501) : 782,	(26,501) : 92,	(27,501) : 138,	(28,501) : 306,	(29,501) : 163,	(30,501) : 310,	(31,501) : 326,	(32,501) : 229,	(33,501) : 263,	(34,501) : 312,	(35,501) : 443,	(36,501) : 545,	(37,501) : 515,	(38,501) : 351,	(39,501) : 353,	(40,501) : 806,
(1,502) : 33,	(2,502) : 45,	(3,502) : 629,	(4,502) : 599,	(5,502) : 80,	(6,502) : 44,	(7,502) : 23,	(8,502) : 874,	(9,502) : 326,	(10,502) : 25,	(11,502) : 46,	(12,502) : 542,	(13,502) : 171,	(14,502) : 321,	(15,502) : 443,	(16,502) : 651,	(17,502) : 549,	(18,502) : 72,	(19,502) : 478,	(20,502) : 29,	(21,502) : 309,	(22,502) : 123,	(23,502) : 68,	(24,502) : 552,	(25,502) : 745,	(26,502) : 235,	(27,502) : 177,	(28,502) : 27,	(29,502) : 465,	(30,502) : 33,	(31,502) : 37,	(32,502) : 79,	(33,502) : 142,	(34,502) : 64,	(35,502) : 148,	(36,502) : 368,	(37,502) : 417,	(38,502) : 48,	(39,502) : 46,	(40,502) : 759,
(1,503) : 8,	(2,503) : 58,	(3,503) : 652,	(4,503) : 594,	(5,503) : 103,	(6,503) : 72,	(7,503) : 5,	(8,503) : 901,	(9,503) : 354,	(10,503) : 3,	(11,503) : 58,	(12,503) : 525,	(13,503) : 165,	(14,503) : 349,	(15,503) : 438,	(16,503) : 641,	(17,503) : 544,	(18,503) : 97,	(19,503) : 461,	(20,503) : 1,	(21,503) : 310,	(22,503) : 151,	(23,503) : 94,	(24,503) : 547,	(25,503) : 773,	(26,503) : 228,	(27,503) : 171,	(28,503) : 1,	(29,503) : 466,	(30,503) : 6,	(31,503) : 64,	(32,503) : 81,	(33,503) : 117,	(34,503) : 93,	(35,503) : 139,	(36,503) : 396,	(37,503) : 445,	(38,503) : 67,	(39,503) : 60,	(40,503) : 787,
(1,504) : 496,	(2,504) : 549,	(3,504) : 790,	(4,504) : 204,	(5,504) : 583,	(6,504) : 535,	(7,504) : 494,	(8,504) : 1080,	(9,504) : 741,	(10,504) : 495,	(11,504) : 549,	(12,504) : 70,	(13,504) : 341,	(14,504) : 737,	(15,504) : 177,	(16,504) : 173,	(17,504) : 186,	(18,504) : 568,	(19,504) : 69,	(20,504) : 492,	(21,504) : 287,	(22,504) : 568,	(23,504) : 563,	(24,504) : 188,	(25,504) : 1036,	(26,504) : 281,	(27,504) : 337,	(28,504) : 492,	(29,504) : 264,	(30,504) : 493,	(31,504) : 535,	(32,504) : 436,	(33,504) : 395,	(34,504) : 534,	(35,504) : 606,	(36,504) : 801,	(37,504) : 777,	(38,504) : 554,	(39,504) : 550,	(40,504) : 1061,
(1,505) : 646,	(2,505) : 696,	(3,505) : 805,	(4,505) : 116,	(5,505) : 723,	(6,505) : 674,	(7,505) : 643,	(8,505) : 1089,	(9,505) : 843,	(10,505) : 643,	(11,505) : 696,	(12,505) : 199,	(13,505) : 480,	(14,505) : 838,	(15,505) : 230,	(16,505) : 2,	(17,505) : 147,	(18,505) : 707,	(19,505) : 238,	(20,505) : 642,	(21,505) : 380,	(22,505) : 693,	(23,505) : 701,	(24,505) : 146,	(25,505) : 1077,	(26,505) : 416,	(27,505) : 475,	(28,505) : 641,	(29,505) : 278,	(30,505) : 643,	(31,505) : 676,	(32,505) : 575,	(33,505) : 556,	(34,505) : 669,	(35,505) : 764,	(36,505) : 904,	(37,505) : 857,	(38,505) : 698,	(39,505) : 697,	(40,505) : 1104,
(1,506) : 37,	(2,506) : 71,	(3,506) : 696,	(4,506) : 619,	(5,506) : 124,	(6,506) : 107,	(7,506) : 48,	(8,506) : 942,	(9,506) : 386,	(10,506) : 45,	(11,506) : 71,	(12,506) : 533,	(13,506) : 193,	(14,506) : 382,	(15,506) : 465,	(16,506) : 660,	(17,506) : 571,	(18,506) : 124,	(19,506) : 469,	(20,506) : 43,	(21,506) : 344,	(22,506) : 188,	(23,506) : 122,	(24,506) : 573,	(25,506) : 811,	(26,506) : 254,	(27,506) : 199,	(28,506) : 45,	(29,506) : 499,	(30,506) : 39,	(31,506) : 96,	(32,506) : 120,	(33,506) : 113,	(34,506) : 131,	(35,506) : 107,	(36,506) : 424,	(37,506) : 482,	(38,506) : 87,	(39,506) : 74,	(40,506) : 824,
(1,507) : 351,	(2,507) : 329,	(3,507) : 325,	(4,507) : 633,	(5,507) : 289,	(6,507) : 281,	(7,507) : 340,	(8,507) : 556,	(9,507) : 137,	(10,507) : 343,	(11,507) : 330,	(12,507) : 703,	(13,507) : 353,	(14,507) : 135,	(15,507) : 498,	(16,507) : 728,	(17,507) : 587,	(18,507) : 277,	(19,507) : 650,	(20,507) : 346,	(21,507) : 357,	(22,507) : 202,	(23,507) : 275,	(24,507) : 588,	(25,507) : 436,	(26,507) : 387,	(27,507) : 354,	(28,507) : 344,	(29,507) : 459,	(30,507) : 350,	(31,507) : 292,	(32,507) : 308,	(33,507) : 429,	(34,507) : 256,	(35,507) : 433,	(36,507) : 196,	(37,507) : 133,	(38,507) : 312,	(39,507) : 326,	(40,507) : 453,
(1,508) : 294,	(2,508) : 277,	(3,508) : 370,	(4,508) : 590,	(5,508) : 243,	(6,508) : 227,	(7,508) : 283,	(8,508) : 614,	(9,508) : 165,	(10,508) : 285,	(11,508) : 279,	(12,508) : 645,	(13,508) : 290,	(14,508) : 161,	(15,508) : 449,	(16,508) : 680,	(17,508) : 542,	(18,508) : 228,	(19,508) : 591,	(20,508) : 289,	(21,508) : 304,	(22,508) : 152,	(23,508) : 226,	(24,508) : 544,	(25,508) : 499,	(26,508) : 326,	(27,508) : 290,	(28,508) : 286,	(29,508) : 419,	(30,508) : 293,	(31,508) : 239,	(32,508) : 246,	(33,508) : 366,	(34,508) : 202,	(35,508) : 386,	(36,508) : 227,	(37,508) : 192,	(38,508) : 261,	(39,508) : 274,	(40,508) : 517,
(1,509) : 11,	(2,509) : 61,	(3,509) : 670,	(4,509) : 602,	(5,509) : 111,	(6,509) : 86,	(7,509) : 22,	(8,509) : 918,	(9,509) : 368,	(10,509) : 19,	(11,509) : 61,	(12,509) : 526,	(13,509) : 174,	(14,509) : 363,	(15,509) : 447,	(16,509) : 647,	(17,509) : 553,	(18,509) : 108,	(19,509) : 462,	(20,509) : 16,	(21,509) : 322,	(22,509) : 166,	(23,509) : 105,	(24,509) : 556,	(25,509) : 789,	(26,509) : 237,	(27,509) : 180,	(28,509) : 19,	(29,509) : 478,	(30,509) : 12,	(31,509) : 76,	(32,509) : 95,	(33,509) : 112,	(34,509) : 108,	(35,509) : 127,	(36,509) : 408,	(37,509) : 461,	(38,509) : 74,	(39,509) : 64,	(40,509) : 803,
(1,510) : 565,	(2,510) : 531,	(3,510) : 281,	(4,510) : 834,	(5,510) : 482,	(6,510) : 490,	(7,510) : 555,	(8,510) : 383,	(9,510) : 220,	(10,510) : 557,	(11,510) : 532,	(12,510) : 932,	(13,510) : 588,	(14,510) : 224,	(15,510) : 714,	(16,510) : 938,	(17,510) : 792,	(18,510) : 475,	(19,510) : 881,	(20,510) : 561,	(21,510) : 583,	(22,510) : 409,	(23,510) : 476,	(24,510) : 793,	(25,510) : 217,	(26,510) : 622,	(27,510) : 589,	(28,510) : 559,	(29,510) : 660,	(30,510) : 565,	(31,510) : 500,	(32,510) : 536,	(33,510) : 656,	(34,510) : 467,	(35,510) : 618,	(36,510) : 216,	(37,510) : 115,	(38,510) : 513,	(39,510) : 527,	(40,510) : 228,
(1,511) : 250,	(2,511) : 287,	(3,511) : 543,	(4,511) : 363,	(5,511) : 303,	(6,511) : 254,	(7,511) : 242,	(8,511) : 826,	(9,511) : 425,	(10,511) : 244,	(11,511) : 288,	(12,511) : 372,	(13,511) : 90,	(14,511) : 421,	(15,511) : 207,	(16,511) : 430,	(17,511) : 312,	(18,511) : 285,	(19,511) : 317,	(20,511) : 244,	(21,511) : 67,	(22,511) : 263,	(23,511) : 279,	(24,511) : 315,	(25,511) : 748,	(26,511) : 67,	(27,511) : 84,	(28,511) : 243,	(29,511) : 223,	(30,511) : 247,	(31,511) : 259,	(32,511) : 164,	(33,511) : 217,	(34,511) : 244,	(35,511) : 382,	(36,511) : 485,	(37,511) : 465,	(38,511) : 285,	(39,511) : 287,	(40,511) : 770,
(1,512) : 47,	(2,512) : 15,	(3,512) : 660,	(4,512) : 643,	(5,512) : 67,	(6,512) : 63,	(7,512) : 49,	(8,512) : 898,	(9,512) : 332,	(10,512) : 48,	(11,512) : 14,	(12,512) : 575,	(13,512) : 214,	(14,512) : 328,	(15,512) : 486,	(16,512) : 691,	(17,512) : 593,	(18,512) : 70,	(19,512) : 512,	(20,512) : 50,	(21,512) : 354,	(22,512) : 141,	(23,512) : 70,	(24,512) : 596,	(25,512) : 761,	(26,512) : 278,	(27,512) : 220,	(28,512) : 50,	(29,512) : 511,	(30,512) : 50,	(31,512) : 52,	(32,512) : 124,	(33,512) : 162,	(34,512) : 89,	(35,512) : 104,	(36,512) : 368,	(37,512) : 431,	(38,512) : 32,	(39,512) : 18,	(40,512) : 773,
(1,513) : 57,	(2,513) : 86,	(3,513) : 716,	(4,513) : 630,	(5,513) : 138,	(6,513) : 126,	(7,513) : 68,	(8,513) : 961,	(9,513) : 403,	(10,513) : 66,	(11,513) : 85,	(12,513) : 536,	(13,513) : 207,	(14,513) : 398,	(15,513) : 477,	(16,513) : 668,	(17,513) : 582,	(18,513) : 140,	(19,513) : 472,	(20,513) : 63,	(21,513) : 359,	(22,513) : 207,	(23,513) : 139,	(24,513) : 584,	(25,513) : 829,	(26,513) : 266,	(27,513) : 213,	(28,513) : 65,	(29,513) : 514,	(30,513) : 59,	(31,513) : 115,	(32,513) : 139,	(33,513) : 115,	(34,513) : 150,	(35,513) : 96,	(36,513) : 439,	(37,513) : 500,	(38,513) : 102,	(39,513) : 89,	(40,513) : 842,
(1,514) : 619,	(2,514) : 668,	(3,514) : 795,	(4,514) : 113,	(5,514) : 697,	(6,514) : 648,	(7,514) : 615,	(8,514) : 1080,	(9,514) : 820,	(10,514) : 616,	(11,514) : 669,	(12,514) : 175,	(13,514) : 453,	(14,514) : 816,	(15,514) : 209,	(16,514) : 28,	(17,514) : 135,	(18,514) : 680,	(19,514) : 211,	(20,514) : 614,	(21,514) : 357,	(22,514) : 668,	(23,514) : 675,	(24,514) : 135,	(25,514) : 1064,	(26,514) : 389,	(27,514) : 448,	(28,514) : 614,	(29,514) : 263,	(30,514) : 616,	(31,514) : 649,	(32,514) : 548,	(33,514) : 527,	(34,514) : 642,	(35,514) : 736,	(36,514) : 882,	(37,514) : 838,	(38,514) : 671,	(39,514) : 669,	(40,514) : 1090,
(1,515) : 562,	(2,515) : 610,	(3,515) : 730,	(4,515) : 77,	(5,515) : 636,	(6,515) : 587,	(7,515) : 558,	(8,515) : 1017,	(9,515) : 752,	(10,515) : 559,	(11,515) : 611,	(12,515) : 182,	(13,515) : 394,	(14,515) : 748,	(15,515) : 140,	(16,515) : 91,	(17,515) : 74,	(18,515) : 619,	(19,515) : 198,	(20,515) : 557,	(21,515) : 289,	(22,515) : 603,	(23,515) : 613,	(24,515) : 75,	(25,515) : 996,	(26,515) : 330,	(27,515) : 388,	(28,515) : 557,	(29,515) : 196,	(30,515) : 559,	(31,515) : 589,	(32,515) : 488,	(33,515) : 477,	(34,515) : 580,	(35,515) : 683,	(36,515) : 814,	(37,515) : 768,	(38,515) : 611,	(39,515) : 611,	(40,515) : 1023,
(1,516) : 76,	(2,516) : 65,	(3,516) : 587,	(4,516) : 597,	(5,516) : 68,	(6,516) : 19,	(7,516) : 66,	(8,516) : 832,	(9,516) : 286,	(10,516) : 68,	(11,516) : 67,	(12,516) : 559,	(13,516) : 177,	(14,516) : 281,	(15,516) : 440,	(16,516) : 656,	(17,516) : 547,	(18,516) : 51,	(19,516) : 496,	(20,516) : 72,	(21,516) : 299,	(22,516) : 81,	(23,516) : 45,	(24,516) : 549,	(25,516) : 703,	(26,516) : 241,	(27,516) : 182,	(28,516) : 69,	(29,516) : 455,	(30,516) : 75,	(31,516) : 26,	(32,516) : 83,	(33,516) : 176,	(34,516) : 23,	(35,516) : 178,	(36,516) : 331,	(37,516) : 375,	(38,516) : 55,	(39,516) : 63,	(40,516) : 717,
(1,517) : 696,	(2,517) : 669,	(3,517) : 190,	(4,517) : 855,	(5,517) : 624,	(6,517) : 624,	(7,517) : 685,	(8,517) : 220,	(9,517) : 376,	(10,517) : 688,	(11,517) : 670,	(12,517) : 993,	(13,517) : 688,	(14,517) : 379,	(15,517) : 758,	(16,517) : 967,	(17,517) : 819,	(18,517) : 614,	(19,517) : 949,	(20,517) : 691,	(21,517) : 647,	(22,517) : 543,	(23,517) : 614,	(24,517) : 819,	(25,517) : 112,	(26,517) : 709,	(27,517) : 687,	(28,517) : 689,	(29,517) : 689,	(30,517) : 695,	(31,517) : 635,	(32,517) : 653,	(33,517) : 774,	(34,517) : 599,	(35,517) : 765,	(36,517) : 380,	(37,517) : 262,	(38,517) : 652,	(39,517) : 666,	(40,517) : 140,
(1,518) : 673,	(2,518) : 663,	(3,518) : 34,	(4,518) : 675,	(5,518) : 630,	(6,518) : 612,	(7,518) : 662,	(8,518) : 300,	(9,518) : 445,	(10,518) : 664,	(11,518) : 665,	(12,518) : 850,	(13,518) : 611,	(14,518) : 445,	(15,518) : 605,	(16,518) : 791,	(17,518) : 646,	(18,518) : 615,	(19,518) : 816,	(20,518) : 667,	(21,518) : 525,	(22,518) : 540,	(23,518) : 613,	(24,518) : 646,	(25,518) : 323,	(26,518) : 612,	(27,518) : 608,	(28,518) : 665,	(29,518) : 524,	(30,518) : 671,	(31,518) : 624,	(32,518) : 607,	(33,518) : 721,	(34,518) : 587,	(35,518) : 773,	(36,518) : 478,	(37,518) : 346,	(38,518) : 648,	(39,518) : 660,	(40,518) : 353,
(1,519) : 556,	(2,519) : 608,	(3,519) : 796,	(4,519) : 157,	(5,519) : 639,	(6,519) : 591,	(7,519) : 554,	(8,519) : 1085,	(9,519) : 782,	(10,519) : 554,	(11,519) : 608,	(12,519) : 103,	(13,519) : 396,	(14,519) : 778,	(15,519) : 186,	(16,519) : 104,	(17,519) : 154,	(18,519) : 624,	(19,519) : 133,	(20,519) : 552,	(21,519) : 320,	(22,519) : 618,	(23,519) : 619,	(24,519) : 155,	(25,519) : 1054,	(26,519) : 333,	(27,519) : 391,	(28,519) : 552,	(29,519) : 261,	(30,519) : 553,	(31,519) : 592,	(32,519) : 491,	(33,519) : 459,	(34,519) : 588,	(35,519) : 669,	(36,519) : 843,	(37,519) : 809,	(38,519) : 612,	(39,519) : 609,	(40,519) : 1080,
(1,520) : 505,	(2,520) : 551,	(3,520) : 665,	(4,520) : 96,	(5,520) : 573,	(6,520) : 524,	(7,520) : 500,	(8,520) : 953,	(9,520) : 680,	(10,520) : 501,	(11,520) : 551,	(12,520) : 214,	(13,520) : 334,	(14,520) : 676,	(15,520) : 69,	(16,520) : 163,	(17,520) : 52,	(18,520) : 556,	(19,520) : 207,	(20,520) : 500,	(21,520) : 219,	(22,520) : 536,	(23,520) : 550,	(24,520) : 55,	(25,520) : 926,	(26,520) : 271,	(27,520) : 329,	(28,520) : 499,	(29,520) : 129,	(30,520) : 502,	(31,520) : 527,	(32,520) : 427,	(33,520) : 429,	(34,520) : 516,	(35,520) : 630,	(36,520) : 742,	(37,520) : 696,	(38,520) : 551,	(39,520) : 551,	(40,520) : 953,
(1,521) : 205,	(2,521) : 260,	(3,521) : 775,	(4,521) : 496,	(5,521) : 309,	(6,521) : 273,	(7,521) : 210,	(8,521) : 1047,	(9,521) : 547,	(10,521) : 209,	(11,521) : 260,	(12,521) : 354,	(13,521) : 170,	(14,521) : 542,	(15,521) : 360,	(16,521) : 509,	(17,521) : 453,	(18,521) : 302,	(19,521) : 291,	(20,521) : 205,	(21,521) : 296,	(22,521) : 342,	(23,521) : 298,	(24,521) : 456,	(25,521) : 942,	(26,521) : 185,	(27,521) : 173,	(28,521) : 206,	(29,521) : 424,	(30,521) : 203,	(31,521) : 267,	(32,521) : 202,	(33,521) : 89,	(34,521) : 287,	(35,521) : 279,	(36,521) : 595,	(37,521) : 625,	(38,521) : 272,	(39,521) : 263,	(40,521) : 960,
(1,522) : 118,	(2,522) : 173,	(3,522) : 728,	(4,522) : 541,	(5,522) : 222,	(6,522) : 190,	(7,522) : 124,	(8,522) : 991,	(9,522) : 469,	(10,522) : 122,	(11,522) : 173,	(12,522) : 430,	(13,522) : 146,	(14,522) : 464,	(15,522) : 393,	(16,522) : 569,	(17,522) : 494,	(18,522) : 217,	(19,522) : 366,	(20,522) : 119,	(21,522) : 297,	(22,522) : 263,	(23,522) : 213,	(24,522) : 497,	(25,522) : 876,	(26,522) : 190,	(27,522) : 151,	(28,522) : 120,	(29,522) : 443,	(30,522) : 117,	(31,522) : 182,	(32,522) : 134,	(33,522) : 19,	(34,522) : 206,	(35,522) : 200,	(36,522) : 513,	(37,522) : 553,	(38,522) : 185,	(39,522) : 175,	(40,522) : 892,
(1,523) : 59,	(2,523) : 109,	(3,523) : 705,	(4,523) : 585,	(5,523) : 160,	(6,523) : 134,	(7,523) : 67,	(8,523) : 958,	(9,523) : 416,	(10,523) : 65,	(11,523) : 109,	(12,523) : 491,	(13,523) : 166,	(14,523) : 411,	(15,523) : 433,	(16,523) : 622,	(17,523) : 537,	(18,523) : 157,	(19,523) : 427,	(20,523) : 61,	(21,523) : 320,	(22,523) : 213,	(23,523) : 155,	(24,523) : 540,	(25,523) : 834,	(26,523) : 223,	(27,523) : 172,	(28,523) : 63,	(29,523) : 473,	(30,523) : 58,	(31,523) : 125,	(32,523) : 112,	(33,523) : 71,	(34,523) : 154,	(35,523) : 142,	(36,523) : 457,	(37,523) : 507,	(38,523) : 123,	(39,523) : 112,	(40,523) : 849,
(1,524) : 792,	(2,524) : 771,	(3,524) : 184,	(4,524) : 875,	(5,524) : 728,	(6,524) : 723,	(7,524) : 781,	(8,524) : 116,	(9,524) : 490,	(10,524) : 784,	(11,524) : 772,	(12,524) : 1040,	(13,524) : 764,	(14,524) : 493,	(15,524) : 797,	(16,524) : 990,	(17,524) : 844,	(18,524) : 717,	(19,524) : 1001,	(20,524) : 787,	(21,524) : 703,	(22,524) : 643,	(23,524) : 716,	(24,524) : 844,	(25,524) : 159,	(26,524) : 778,	(27,524) : 763,	(28,524) : 785,	(29,524) : 720,	(30,524) : 792,	(31,524) : 734,	(32,524) : 741,	(33,524) : 861,	(34,524) : 698,	(35,524) : 871,	(36,524) : 499,	(37,524) : 377,	(38,524) : 754,	(39,524) : 767,	(40,524) : 187,
(1,525) : 323,	(2,525) : 365,	(3,525) : 573,	(4,525) : 280,	(5,525) : 385,	(6,525) : 336,	(7,525) : 317,	(8,525) : 861,	(9,525) : 503,	(10,525) : 318,	(11,525) : 366,	(12,525) : 304,	(13,525) : 153,	(14,525) : 498,	(15,525) : 124,	(16,525) : 347,	(17,525) : 230,	(18,525) : 367,	(19,525) : 256,	(20,525) : 318,	(21,525) : 47,	(22,525) : 347,	(23,525) : 361,	(24,525) : 233,	(25,525) : 800,	(26,525) : 96,	(27,525) : 147,	(28,525) : 317,	(29,525) : 157,	(30,525) : 320,	(31,525) : 339,	(32,525) : 241,	(33,525) : 267,	(34,525) : 327,	(35,525) : 453,	(36,525) : 564,	(37,525) : 534,	(38,525) : 364,	(39,525) : 365,	(40,525) : 824,
(1,526) : 759,	(2,526) : 727,	(3,526) : 295,	(4,526) : 959,	(5,526) : 678,	(6,526) : 684,	(7,526) : 749,	(8,526) : 221,	(9,526) : 416,	(10,526) : 752,	(11,526) : 727,	(12,526) : 1090,	(13,526) : 769,	(14,526) : 420,	(15,526) : 858,	(16,526) : 1071,	(17,526) : 922,	(18,526) : 671,	(19,526) : 1044,	(20,526) : 755,	(21,526) : 741,	(22,526) : 604,	(23,526) : 671,	(24,526) : 923,	(25,526) : 23,	(26,526) : 796,	(27,526) : 769,	(28,526) : 753,	(29,526) : 792,	(30,526) : 759,	(31,526) : 695,	(32,526) : 726,	(33,526) : 846,	(34,526) : 662,	(35,526) : 814,	(36,526) : 405,	(37,526) : 310,	(38,526) : 709,	(39,526) : 723,	(40,526) : 36,
(1,527) : 79,	(2,527) : 44,	(3,527) : 688,	(4,527) : 678,	(5,527) : 80,	(6,527) : 95,	(7,527) : 84,	(8,527) : 920,	(9,527) : 344,	(10,527) : 83,	(11,527) : 43,	(12,527) : 603,	(13,527) : 249,	(14,527) : 340,	(15,527) : 522,	(16,527) : 724,	(17,527) : 629,	(18,527) : 91,	(19,527) : 539,	(20,527) : 84,	(21,527) : 392,	(22,527) : 166,	(23,527) : 93,	(24,527) : 631,	(25,527) : 778,	(26,527) : 313,	(27,527) : 255,	(28,527) : 84,	(29,527) : 548,	(30,527) : 82,	(31,527) : 84,	(32,527) : 161,	(33,527) : 184,	(34,527) : 120,	(35,527) : 69,	(36,527) : 374,	(37,527) : 448,	(38,527) : 58,	(39,527) : 46,	(40,527) : 789,
(1,528) : 710,	(2,528) : 696,	(3,528) : 57,	(4,528) : 743,	(5,528) : 659,	(6,528) : 646,	(7,528) : 699,	(8,528) : 234,	(9,528) : 452,	(10,528) : 702,	(11,528) : 697,	(12,528) : 915,	(13,528) : 661,	(14,528) : 453,	(15,528) : 670,	(16,528) : 859,	(17,528) : 714,	(18,528) : 645,	(19,528) : 879,	(20,528) : 705,	(21,528) : 585,	(22,528) : 570,	(23,528) : 644,	(24,528) : 714,	(25,528) : 261,	(26,528) : 667,	(27,528) : 659,	(28,528) : 703,	(29,528) : 591,	(30,528) : 709,	(31,528) : 657,	(32,528) : 650,	(33,528) : 766,	(34,528) : 620,	(35,528) : 803,	(36,528) : 477,	(37,528) : 345,	(38,528) : 680,	(39,528) : 693,	(40,528) : 292,
(1,529) : 480,	(2,529) : 518,	(3,529) : 546,	(4,529) : 163,	(5,529) : 530,	(6,529) : 482,	(7,529) : 473,	(8,529) : 834,	(9,529) : 595,	(10,529) : 474,	(11,529) : 519,	(12,529) : 326,	(13,529) : 313,	(14,529) : 591,	(15,529) : 85,	(16,529) : 267,	(17,529) : 120,	(18,529) : 511,	(19,529) : 306,	(20,529) : 474,	(21,529) : 164,	(22,529) : 477,	(23,529) : 506,	(24,529) : 121,	(25,529) : 812,	(26,529) : 258,	(27,529) : 307,	(28,529) : 473,	(29,529) : 12,	(30,529) : 477,	(31,529) : 488,	(32,529) : 394,	(33,529) : 429,	(34,529) : 469,	(35,529) : 611,	(36,529) : 657,	(37,529) : 597,	(38,529) : 514,	(39,529) : 517,	(40,529) : 839,
(1,530) : 442,	(2,530) : 479,	(3,530) : 533,	(4,530) : 195,	(5,530) : 491,	(6,530) : 443,	(7,530) : 434,	(8,530) : 823,	(9,530) : 560,	(10,530) : 436,	(11,530) : 480,	(12,530) : 328,	(13,530) : 276,	(14,530) : 556,	(15,530) : 82,	(16,530) : 293,	(17,530) : 149,	(18,530) : 472,	(19,530) : 301,	(20,530) : 436,	(21,530) : 125,	(22,530) : 438,	(23,530) : 467,	(24,530) : 150,	(25,530) : 791,	(26,530) : 222,	(27,530) : 269,	(28,530) : 435,	(29,530) : 31,	(30,530) : 439,	(31,530) : 448,	(32,530) : 356,	(33,530) : 394,	(34,530) : 430,	(35,530) : 573,	(36,530) : 622,	(37,530) : 566,	(38,530) : 475,	(39,530) : 478,	(40,530) : 818,
(1,531) : 224,	(2,531) : 252,	(3,531) : 509,	(4,531) : 411,	(5,531) : 261,	(6,531) : 214,	(7,531) : 215,	(8,531) : 787,	(9,531) : 369,	(10,531) : 217,	(11,531) : 254,	(12,531) : 428,	(13,531) : 98,	(14,531) : 364,	(15,531) : 257,	(16,531) : 484,	(17,531) : 361,	(18,531) : 242,	(19,531) : 372,	(20,531) : 217,	(21,531) : 108,	(22,531) : 211,	(23,531) : 236,	(24,531) : 363,	(25,531) : 701,	(26,531) : 110,	(27,531) : 94,	(28,531) : 216,	(29,531) : 261,	(30,531) : 221,	(31,531) : 220,	(32,531) : 135,	(33,531) : 218,	(34,531) : 199,	(35,531) : 355,	(36,531) : 429,	(37,531) : 411,	(38,531) : 247,	(39,531) : 252,	(40,531) : 722,
(1,532) : 286,	(2,532) : 277,	(3,532) : 373,	(4,532) : 551,	(5,532) : 249,	(6,532) : 226,	(7,532) : 275,	(8,532) : 628,	(9,532) : 204,	(10,532) : 278,	(11,532) : 279,	(12,532) : 609,	(13,532) : 262,	(14,532) : 200,	(15,532) : 411,	(16,532) : 642,	(17,532) : 504,	(18,532) : 232,	(19,532) : 556,	(20,532) : 281,	(21,532) : 266,	(22,532) : 159,	(23,532) : 229,	(24,532) : 506,	(25,532) : 523,	(26,532) : 293,	(27,532) : 262,	(28,532) : 279,	(29,532) : 380,	(30,532) : 285,	(31,532) : 237,	(32,532) : 228,	(33,532) : 348,	(34,532) : 200,	(35,532) : 389,	(36,532) : 266,	(37,532) : 227,	(38,532) : 263,	(39,532) : 275,	(40,532) : 542,
(1,533) : 159,	(2,533) : 214,	(3,533) : 726,	(4,533) : 494,	(5,533) : 261,	(6,533) : 223,	(7,533) : 162,	(8,533) : 996,	(9,533) : 494,	(10,533) : 161,	(11,533) : 215,	(12,533) : 379,	(13,533) : 125,	(14,533) : 490,	(15,533) : 349,	(16,533) : 519,	(17,533) : 448,	(18,533) : 253,	(19,533) : 315,	(20,533) : 158,	(21,533) : 265,	(22,533) : 289,	(23,533) : 249,	(24,533) : 451,	(25,533) : 889,	(26,533) : 155,	(27,533) : 129,	(28,533) : 159,	(29,533) : 404,	(30,533) : 157,	(31,533) : 218,	(32,533) : 148,	(33,533) : 42,	(34,533) : 235,	(35,533) : 250,	(36,533) : 542,	(37,533) : 572,	(38,533) : 225,	(39,533) : 217,	(40,533) : 907,
(1,534) : 335,	(2,534) : 295,	(3,534) : 429,	(4,534) : 742,	(5,534) : 244,	(6,534) : 259,	(7,534) : 327,	(8,534) : 614,	(9,534) : 23,	(10,534) : 329,	(11,534) : 296,	(12,534) : 777,	(13,534) : 400,	(14,534) : 18,	(15,534) : 597,	(16,534) : 828,	(17,534) : 694,	(18,534) : 240,	(19,534) : 719,	(20,534) : 332,	(21,534) : 449,	(22,534) : 184,	(23,534) : 241,	(24,534) : 695,	(25,534) : 458,	(26,534) : 449,	(27,534) : 402,	(28,534) : 331,	(29,534) : 572,	(30,534) : 336,	(31,534) : 269,	(32,534) : 327,	(33,534) : 438,	(34,534) : 241,	(35,534) : 377,	(36,534) : 77,	(37,534) : 132,	(38,534) : 278,	(39,534) : 291,	(40,534) : 468,
(1,535) : 912,	(2,535) : 874,	(3,535) : 459,	(4,535) : 1134,	(5,535) : 824,	(6,535) : 836,	(7,535) : 902,	(8,535) : 275,	(9,535) : 557,	(10,535) : 904,	(11,535) : 875,	(12,535) : 1265,	(13,535) : 936,	(14,535) : 562,	(15,535) : 1034,	(16,535) : 1246,	(17,535) : 1097,	(18,535) : 819,	(19,535) : 1217,	(20,535) : 908,	(21,535) : 915,	(22,535) : 756,	(23,535) : 820,	(24,535) : 1098,	(25,535) : 168,	(26,535) : 966,	(27,535) : 936,	(28,535) : 906,	(29,535) : 967,	(30,535) : 912,	(31,535) : 846,	(32,535) : 886,	(33,535) : 1005,	(34,535) : 815,	(35,535) : 953,	(36,535) : 531,	(37,535) : 463,	(38,535) : 857,	(39,535) : 871,	(40,535) : 140,
(1,536) : 385,	(2,536) : 420,	(3,536) : 512,	(4,536) : 251,	(5,536) : 431,	(6,536) : 384,	(7,536) : 377,	(8,536) : 802,	(9,536) : 505,	(10,536) : 379,	(11,536) : 421,	(12,536) : 348,	(13,536) : 223,	(14,536) : 501,	(15,536) : 113,	(16,536) : 341,	(17,536) : 203,	(18,536) : 412,	(19,536) : 310,	(20,536) : 379,	(21,536) : 69,	(22,536) : 378,	(23,536) : 407,	(24,536) : 204,	(25,536) : 757,	(26,536) : 175,	(27,536) : 216,	(28,536) : 378,	(29,536) : 90,	(30,536) : 382,	(31,536) : 389,	(32,536) : 298,	(33,536) : 345,	(34,536) : 370,	(35,536) : 517,	(36,536) : 567,	(37,536) : 517,	(38,536) : 416,	(39,536) : 420,	(40,536) : 783,
(1,537) : 47,	(2,537) : 95,	(3,537) : 700,	(4,537) : 595,	(5,537) : 147,	(6,537) : 123,	(7,537) : 56,	(8,537) : 951,	(9,537) : 405,	(10,537) : 54,	(11,537) : 95,	(12,537) : 504,	(13,537) : 172,	(14,537) : 400,	(15,537) : 442,	(16,537) : 633,	(17,537) : 547,	(18,537) : 145,	(19,537) : 440,	(20,537) : 50,	(21,537) : 325,	(22,537) : 202,	(23,537) : 142,	(24,537) : 549,	(25,537) : 824,	(26,537) : 231,	(27,537) : 179,	(28,537) : 53,	(29,537) : 479,	(30,537) : 47,	(31,537) : 113,	(32,537) : 111,	(33,537) : 84,	(34,537) : 144,	(35,537) : 131,	(36,537) : 445,	(37,537) : 497,	(38,537) : 110,	(39,537) : 99,	(40,537) : 839,
(1,538) : 171,	(2,538) : 167,	(3,538) : 488,	(4,538) : 551,	(5,538) : 149,	(6,538) : 116,	(7,538) : 160,	(8,538) : 740,	(9,538) : 240,	(10,538) : 163,	(11,538) : 168,	(12,538) : 558,	(13,538) : 180,	(14,538) : 235,	(15,538) : 398,	(16,538) : 625,	(17,538) : 501,	(18,538) : 130,	(19,538) : 499,	(20,538) : 166,	(21,538) : 248,	(22,538) : 74,	(23,538) : 126,	(24,538) : 503,	(25,538) : 624,	(26,538) : 229,	(27,538) : 182,	(28,538) : 163,	(29,538) : 393,	(30,538) : 170,	(31,538) : 127,	(32,538) : 120,	(33,538) : 240,	(34,538) : 91,	(35,538) : 280,	(36,538) : 297,	(37,538) : 306,	(38,538) : 154,	(39,538) : 165,	(40,538) : 640,
(1,539) : 199,	(2,539) : 180,	(3,539) : 467,	(4,539) : 595,	(5,539) : 148,	(6,539) : 130,	(7,539) : 189,	(8,539) : 708,	(9,539) : 188,	(10,539) : 191,	(11,539) : 182,	(12,539) : 610,	(13,539) : 232,	(14,539) : 183,	(15,539) : 444,	(16,539) : 673,	(17,539) : 546,	(18,539) : 132,	(19,539) : 551,	(20,539) : 195,	(21,539) : 294,	(22,539) : 57,	(23,539) : 129,	(24,539) : 548,	(25,539) : 582,	(26,539) : 281,	(27,539) : 234,	(28,539) : 192,	(29,539) : 433,	(30,539) : 199,	(31,539) : 142,	(32,539) : 165,	(33,539) : 284,	(34,539) : 105,	(35,539) : 289,	(36,539) : 244,	(37,539) : 259,	(38,539) : 164,	(39,539) : 177,	(40,539) : 598,
(1,540) : 270,	(2,540) : 241,	(3,540) : 419,	(4,540) : 646,	(5,540) : 198,	(6,540) : 196,	(7,540) : 259,	(8,540) : 643,	(9,540) : 115,	(10,540) : 262,	(11,540) : 242,	(12,540) : 680,	(13,540) : 307,	(14,540) : 110,	(15,540) : 500,	(16,540) : 731,	(17,540) : 598,	(18,540) : 187,	(19,540) : 623,	(20,540) : 266,	(21,540) : 351,	(22,540) : 115,	(23,540) : 186,	(24,540) : 599,	(25,540) : 509,	(26,540) : 353,	(27,540) : 309,	(28,540) : 263,	(29,540) : 478,	(30,540) : 269,	(31,540) : 207,	(32,540) : 243,	(33,540) : 360,	(34,540) : 172,	(35,540) : 342,	(36,540) : 175,	(37,540) : 182,	(38,540) : 224,	(39,540) : 238,	(40,540) : 523,
(1,541) : 382,	(2,541) : 344,	(3,541) : 381,	(4,541) : 746,	(5,541) : 295,	(6,541) : 306,	(7,541) : 372,	(8,541) : 560,	(9,541) : 38,	(10,541) : 375,	(11,541) : 345,	(12,541) : 799,	(13,541) : 430,	(14,541) : 40,	(15,541) : 607,	(16,541) : 838,	(17,541) : 699,	(18,541) : 289,	(19,541) : 743,	(20,541) : 378,	(21,541) : 461,	(22,541) : 227,	(23,541) : 290,	(24,541) : 701,	(25,541) : 405,	(26,541) : 474,	(27,541) : 432,	(28,541) : 376,	(29,541) : 573,	(30,541) : 382,	(31,541) : 316,	(32,541) : 364,	(33,541) : 480,	(34,541) : 285,	(35,541) : 431,	(36,541) : 81,	(37,541) : 77,	(38,541) : 327,	(39,541) : 341,	(40,541) : 415,
(1,542) : 370,	(2,542) : 326,	(3,542) : 437,	(4,542) : 782,	(5,542) : 275,	(6,542) : 294,	(7,542) : 361,	(8,542) : 603,	(9,542) : 28,	(10,542) : 364,	(11,542) : 327,	(12,542) : 818,	(13,542) : 441,	(14,542) : 31,	(15,542) : 638,	(16,542) : 869,	(17,542) : 734,	(18,542) : 272,	(19,542) : 760,	(20,542) : 367,	(21,542) : 490,	(22,542) : 222,	(23,542) : 274,	(24,542) : 736,	(25,542) : 438,	(26,542) : 490,	(27,542) : 443,	(28,542) : 365,	(29,542) : 611,	(30,542) : 370,	(31,542) : 303,	(32,542) : 366,	(33,542) : 476,	(34,542) : 277,	(35,542) : 402,	(36,542) : 36,	(37,542) : 125,	(38,542) : 310,	(39,542) : 323,	(40,542) : 446,
(1,543) : 662,	(2,543) : 711,	(3,543) : 822,	(4,543) : 131,	(5,543) : 740,	(6,543) : 691,	(7,543) : 658,	(8,543) : 1105,	(9,543) : 860,	(10,543) : 659,	(11,543) : 712,	(12,543) : 204,	(13,543) : 496,	(14,543) : 856,	(15,543) : 248,	(16,543) : 17,	(17,543) : 164,	(18,543) : 723,	(19,543) : 247,	(20,543) : 657,	(21,543) : 397,	(22,543) : 710,	(23,543) : 718,	(24,543) : 164,	(25,543) : 1094,	(26,543) : 432,	(27,543) : 491,	(28,543) : 657,	(29,543) : 295,	(30,543) : 659,	(31,543) : 692,	(32,543) : 591,	(33,543) : 570,	(34,543) : 685,	(35,543) : 779,	(36,543) : 922,	(37,543) : 875,	(38,543) : 714,	(39,543) : 712,	(40,543) : 1122,
(1,544) : 301,	(2,544) : 268,	(3,544) : 408,	(4,544) : 679,	(5,544) : 222,	(6,544) : 226,	(7,544) : 291,	(8,544) : 619,	(9,544) : 78,	(10,544) : 294,	(11,544) : 269,	(12,544) : 718,	(13,544) : 345,	(14,544) : 74,	(15,544) : 535,	(16,544) : 766,	(17,544) : 631,	(18,544) : 213,	(19,544) : 660,	(20,544) : 297,	(21,544) : 386,	(22,544) : 145,	(23,544) : 213,	(24,544) : 632,	(25,544) : 478,	(26,544) : 391,	(27,544) : 347,	(28,544) : 295,	(29,544) : 509,	(30,544) : 301,	(31,544) : 236,	(32,544) : 279,	(33,544) : 396,	(34,544) : 204,	(35,544) : 363,	(36,544) : 139,	(37,544) : 149,	(38,544) : 251,	(39,544) : 265,	(40,544) : 492,
(1,545) : 24,	(2,545) : 35,	(3,545) : 645,	(4,545) : 611,	(5,545) : 80,	(6,545) : 54,	(7,545) : 20,	(8,545) : 889,	(9,545) : 335,	(10,545) : 20,	(11,545) : 36,	(12,545) : 548,	(13,545) : 183,	(14,545) : 331,	(15,545) : 455,	(16,545) : 661,	(17,545) : 562,	(18,545) : 76,	(19,545) : 484,	(20,545) : 24,	(21,545) : 323,	(22,545) : 135,	(23,545) : 73,	(24,545) : 564,	(25,545) : 758,	(26,545) : 247,	(27,545) : 189,	(28,545) : 23,	(29,545) : 480,	(30,545) : 25,	(31,545) : 44,	(32,545) : 93,	(33,545) : 140,	(34,545) : 78,	(35,545) : 130,	(36,545) : 375,	(37,545) : 429,	(38,545) : 44,	(39,545) : 37,	(40,545) : 772,
(1,546) : 672,	(2,546) : 639,	(3,546) : 269,	(4,546) : 898,	(5,546) : 591,	(6,546) : 597,	(7,546) : 662,	(8,546) : 286,	(9,546) : 329,	(10,546) : 664,	(11,546) : 640,	(12,546) : 1016,	(13,546) : 687,	(14,546) : 333,	(15,546) : 789,	(16,546) : 1007,	(17,546) : 859,	(18,546) : 584,	(19,546) : 968,	(20,546) : 668,	(21,546) : 667,	(22,546) : 516,	(23,546) : 584,	(24,546) : 860,	(25,546) : 108,	(26,546) : 716,	(27,546) : 686,	(28,546) : 666,	(29,546) : 728,	(30,546) : 672,	(31,546) : 608,	(32,546) : 640,	(33,546) : 760,	(34,546) : 575,	(35,546) : 727,	(36,546) : 320,	(37,546) : 223,	(38,546) : 622,	(39,546) : 636,	(40,546) : 120,
(1,547) : 534,	(2,547) : 504,	(3,547) : 247,	(4,547) : 782,	(5,547) : 457,	(6,547) : 460,	(7,547) : 524,	(8,547) : 390,	(9,547) : 206,	(10,547) : 526,	(11,547) : 505,	(12,547) : 884,	(13,547) : 546,	(14,547) : 209,	(15,547) : 664,	(16,547) : 887,	(17,547) : 740,	(18,547) : 448,	(19,547) : 834,	(20,547) : 530,	(21,547) : 534,	(22,547) : 379,	(23,547) : 448,	(24,547) : 741,	(25,547) : 244,	(26,547) : 577,	(27,547) : 546,	(28,547) : 528,	(29,547) : 609,	(30,547) : 534,	(31,547) : 471,	(32,547) : 500,	(33,547) : 620,	(34,547) : 437,	(35,547) : 597,	(36,547) : 217,	(37,547) : 92,	(38,547) : 486,	(39,547) : 500,	(40,547) : 260,
(1,548) : 94,	(2,548) : 145,	(3,548) : 730,	(4,548) : 574,	(5,548) : 196,	(6,548) : 169,	(7,548) : 102,	(8,548) : 988,	(9,548) : 451,	(10,548) : 100,	(11,548) : 145,	(12,548) : 466,	(13,548) : 168,	(14,548) : 446,	(15,548) : 425,	(16,548) : 605,	(17,548) : 527,	(18,548) : 193,	(19,548) : 402,	(20,548) : 96,	(21,548) : 321,	(22,548) : 247,	(23,548) : 190,	(24,548) : 530,	(25,548) : 866,	(26,548) : 218,	(27,548) : 174,	(28,548) : 98,	(29,548) : 471,	(30,548) : 93,	(31,548) : 161,	(32,548) : 134,	(33,548) : 50,	(34,548) : 189,	(35,548) : 163,	(36,548) : 493,	(37,548) : 541,	(38,548) : 159,	(39,548) : 148,	(40,548) : 882,
(1,549) : 203,	(2,549) : 254,	(3,549) : 642,	(4,549) : 401,	(5,549) : 286,	(6,549) : 239,	(7,549) : 200,	(8,549) : 921,	(9,549) : 471,	(10,549) : 200,	(11,549) : 254,	(12,549) : 337,	(13,549) : 51,	(14,549) : 466,	(15,549) : 249,	(16,549) : 443,	(17,549) : 353,	(18,549) : 272,	(19,549) : 275,	(20,549) : 199,	(21,549) : 156,	(22,549) : 279,	(23,549) : 267,	(24,549) : 356,	(25,549) : 831,	(26,549) : 45,	(27,549) : 49,	(28,549) : 198,	(29,549) : 297,	(30,549) : 200,	(31,549) : 238,	(32,549) : 140,	(33,549) : 128,	(34,549) : 239,	(35,549) : 325,	(36,549) : 526,	(37,549) : 530,	(38,549) : 257,	(39,549) : 255,	(40,549) : 852,
(1,550) : 88,	(2,550) : 140,	(3,550) : 648,	(4,550) : 514,	(5,550) : 179,	(6,550) : 136,	(7,550) : 85,	(8,550) : 911,	(9,550) : 401,	(10,550) : 86,	(11,550) : 141,	(12,550) : 443,	(13,550) : 89,	(14,550) : 396,	(15,550) : 360,	(16,550) : 558,	(17,550) : 466,	(18,550) : 168,	(19,550) : 380,	(20,550) : 84,	(21,550) : 242,	(22,550) : 196,	(23,550) : 163,	(24,550) : 468,	(25,550) : 799,	(26,550) : 149,	(27,550) : 95,	(28,550) : 84,	(29,550) : 396,	(30,550) : 85,	(31,550) : 132,	(32,550) : 56,	(33,550) : 65,	(34,550) : 144,	(35,550) : 212,	(36,550) : 450,	(37,550) : 479,	(38,550) : 146,	(39,550) : 142,	(40,550) : 816,
(1,551) : 293,	(2,551) : 261,	(3,551) : 413,	(4,551) : 674,	(5,551) : 215,	(6,551) : 218,	(7,551) : 283,	(8,551) : 626,	(9,551) : 84,	(10,551) : 286,	(11,551) : 262,	(12,551) : 711,	(13,551) : 338,	(14,551) : 80,	(15,551) : 529,	(16,551) : 760,	(17,551) : 626,	(18,551) : 206,	(19,551) : 653,	(20,551) : 289,	(21,551) : 381,	(22,551) : 137,	(23,551) : 206,	(24,551) : 628,	(25,551) : 486,	(26,551) : 384,	(27,551) : 339,	(28,551) : 287,	(29,551) : 504,	(30,551) : 293,	(31,551) : 229,	(32,551) : 271,	(33,551) : 388,	(34,551) : 196,	(35,551) : 356,	(36,551) : 145,	(37,551) : 157,	(38,551) : 244,	(39,551) : 257,	(40,551) : 499,
(1,552) : 469,	(2,552) : 501,	(3,552) : 487,	(4,552) : 217,	(5,552) : 508,	(6,552) : 462,	(7,552) : 461,	(8,552) : 776,	(9,552) : 550,	(10,552) : 462,	(11,552) : 502,	(12,552) : 380,	(13,552) : 309,	(14,552) : 546,	(15,552) : 135,	(16,552) : 326,	(17,552) : 177,	(18,552) : 489,	(19,552) : 356,	(20,552) : 463,	(21,552) : 155,	(22,552) : 447,	(23,552) : 483,	(24,552) : 178,	(25,552) : 753,	(26,552) : 262,	(27,552) : 303,	(28,552) : 461,	(29,552) : 49,	(30,552) : 466,	(31,552) : 468,	(32,552) : 381,	(33,552) : 432,	(34,552) : 446,	(35,552) : 601,	(36,552) : 612,	(37,552) : 545,	(38,552) : 496,	(39,552) : 501,	(40,552) : 780,
(1,553) : 6,	(2,553) : 56,	(3,553) : 653,	(4,553) : 596,	(5,553) : 102,	(6,553) : 72,	(7,553) : 5,	(8,553) : 901,	(9,553) : 354,	(10,553) : 3,	(11,553) : 57,	(12,553) : 527,	(13,553) : 167,	(14,553) : 349,	(15,553) : 440,	(16,553) : 643,	(17,553) : 546,	(18,553) : 97,	(19,553) : 463,	(20,553) : 1,	(21,553) : 311,	(22,553) : 151,	(23,553) : 93,	(24,553) : 549,	(25,553) : 773,	(26,553) : 230,	(27,553) : 173,	(28,553) : 2,	(29,553) : 468,	(30,553) : 5,	(31,553) : 63,	(32,553) : 83,	(33,553) : 118,	(34,553) : 92,	(35,553) : 138,	(36,553) : 395,	(37,553) : 445,	(38,553) : 66,	(39,553) : 58,	(40,553) : 787,
(1,554) : 596,	(2,554) : 645,	(3,554) : 778,	(4,554) : 105,	(5,554) : 673,	(6,554) : 625,	(7,554) : 592,	(8,554) : 1065,	(9,554) : 798,	(10,554) : 593,	(11,554) : 646,	(12,554) : 165,	(13,554) : 430,	(14,554) : 794,	(15,554) : 187,	(16,554) : 51,	(17,554) : 120,	(18,554) : 657,	(19,554) : 195,	(20,554) : 591,	(21,554) : 334,	(22,554) : 644,	(23,554) : 652,	(24,554) : 120,	(25,554) : 1045,	(26,554) : 366,	(27,554) : 425,	(28,554) : 591,	(29,554) : 245,	(30,554) : 593,	(31,554) : 626,	(32,554) : 525,	(33,554) : 505,	(34,554) : 619,	(35,554) : 714,	(36,554) : 860,	(37,554) : 816,	(38,554) : 648,	(39,554) : 646,	(40,554) : 1072,
(1,555) : 665,	(2,555) : 635,	(3,555) : 234,	(4,555) : 870,	(5,555) : 588,	(6,555) : 591,	(7,555) : 655,	(8,555) : 271,	(9,555) : 331,	(10,555) : 658,	(11,555) : 636,	(12,555) : 994,	(13,555) : 672,	(14,555) : 334,	(15,555) : 765,	(16,555) : 980,	(17,555) : 832,	(18,555) : 580,	(19,555) : 947,	(20,555) : 661,	(21,555) : 645,	(22,555) : 510,	(23,555) : 580,	(24,555) : 833,	(25,555) : 114,	(26,555) : 698,	(27,555) : 671,	(28,555) : 659,	(29,555) : 701,	(30,555) : 665,	(31,555) : 602,	(32,555) : 629,	(33,555) : 750,	(34,555) : 568,	(35,555) : 726,	(36,555) : 329,	(37,555) : 220,	(38,555) : 618,	(39,555) : 631,	(40,555) : 133,
(1,556) : 315,	(2,556) : 352,	(3,556) : 530,	(4,556) : 304,	(5,556) : 365,	(6,556) : 317,	(7,556) : 307,	(8,556) : 818,	(9,556) : 465,	(10,556) : 309,	(11,556) : 353,	(12,556) : 348,	(13,556) : 152,	(14,556) : 460,	(15,556) : 151,	(16,556) : 380,	(17,556) : 254,	(18,556) : 347,	(19,556) : 301,	(20,556) : 309,	(21,556) : 3,	(22,556) : 319,	(23,556) : 341,	(24,556) : 256,	(25,556) : 756,	(26,556) : 108,	(27,556) : 145,	(28,556) : 307,	(29,556) : 159,	(30,556) : 312,	(31,556) : 322,	(32,556) : 228,	(33,556) : 275,	(34,556) : 305,	(35,556) : 446,	(36,556) : 526,	(37,556) : 491,	(38,556) : 348,	(39,556) : 351,	(40,556) : 779,
(1,557) : 347,	(2,557) : 369,	(3,557) : 413,	(4,557) : 365,	(5,557) : 366,	(6,557) : 323,	(7,557) : 338,	(8,557) : 700,	(9,557) : 392,	(10,557) : 340,	(11,557) : 370,	(12,557) : 460,	(13,557) : 221,	(14,557) : 388,	(15,557) : 234,	(16,557) : 461,	(17,557) : 319,	(18,557) : 346,	(19,557) : 416,	(20,557) : 341,	(21,557) : 115,	(22,557) : 295,	(23,557) : 341,	(24,557) : 320,	(25,557) : 642,	(26,557) : 205,	(27,557) : 216,	(28,557) : 339,	(29,557) : 192,	(30,557) : 345,	(31,557) : 331,	(32,557) : 261,	(33,557) : 346,	(34,557) : 304,	(35,557) : 477,	(36,557) : 454,	(37,557) : 397,	(38,557) : 360,	(39,557) : 367,	(40,557) : 666,
(1,558) : 549,	(2,558) : 592,	(3,558) : 653,	(4,558) : 54,	(5,558) : 612,	(6,558) : 563,	(7,558) : 543,	(8,558) : 938,	(9,558) : 702,	(10,558) : 544,	(11,558) : 593,	(12,558) : 254,	(13,558) : 378,	(14,558) : 698,	(15,558) : 105,	(16,558) : 155,	(17,558) : 7,	(18,558) : 594,	(19,558) : 256,	(20,558) : 543,	(21,558) : 249,	(22,558) : 569,	(23,558) : 589,	(24,558) : 8,	(25,558) : 923,	(26,558) : 315,	(27,558) : 372,	(28,558) : 543,	(29,558) : 125,	(30,558) : 546,	(31,558) : 567,	(32,558) : 468,	(33,558) : 478,	(34,558) : 553,	(35,558) : 676,	(36,558) : 764,	(37,558) : 709,	(38,558) : 592,	(39,558) : 593,	(40,558) : 950,
(1,559) : 120,	(2,559) : 76,	(3,559) : 698,	(4,559) : 718,	(5,559) : 87,	(6,559) : 120,	(7,559) : 124,	(8,559) : 921,	(9,559) : 335,	(10,559) : 123,	(11,559) : 75,	(12,559) : 644,	(13,559) : 289,	(14,559) : 331,	(15,559) : 562,	(16,559) : 766,	(17,559) : 669,	(18,559) : 104,	(19,559) : 581,	(20,559) : 124,	(21,559) : 429,	(22,559) : 178,	(23,559) : 108,	(24,559) : 671,	(25,559) : 771,	(26,559) : 353,	(27,559) : 295,	(28,559) : 125,	(29,559) : 586,	(30,559) : 123,	(31,559) : 111,	(32,559) : 199,	(33,559) : 225,	(34,559) : 142,	(35,559) : 64,	(36,559) : 359,	(37,559) : 443,	(38,559) : 83,	(39,559) : 77,	(40,559) : 781,
(1,560) : 288,	(2,560) : 306,	(3,560) : 428,	(4,560) : 412,	(5,560) : 302,	(6,560) : 260,	(7,560) : 278,	(8,560) : 709,	(9,560) : 343,	(10,560) : 280,	(11,560) : 307,	(12,560) : 476,	(13,560) : 180,	(14,560) : 339,	(15,560) : 270,	(16,560) : 501,	(17,560) : 364,	(18,560) : 282,	(19,560) : 426,	(20,560) : 282,	(21,560) : 127,	(22,560) : 232,	(23,560) : 277,	(24,560) : 366,	(25,560) : 633,	(26,560) : 183,	(27,560) : 177,	(28,560) : 280,	(29,560) : 244,	(30,560) : 286,	(31,560) : 269,	(32,560) : 203,	(33,560) : 299,	(34,560) : 240,	(35,560) : 416,	(36,560) : 405,	(37,560) : 363,	(38,560) : 297,	(39,560) : 305,	(40,560) : 656,
(1,561) : 232,	(2,561) : 278,	(3,561) : 611,	(4,561) : 368,	(5,561) : 306,	(6,561) : 257,	(7,561) : 227,	(8,561) : 893,	(9,561) : 467,	(10,561) : 228,	(11,561) : 279,	(12,561) : 330,	(13,561) : 62,	(14,561) : 462,	(15,561) : 214,	(16,561) : 418,	(17,561) : 319,	(18,561) : 289,	(19,561) : 271,	(20,561) : 227,	(21,561) : 113,	(22,561) : 285,	(23,561) : 284,	(24,561) : 322,	(25,561) : 811,	(26,561) : 3,	(27,561) : 57,	(28,561) : 226,	(29,561) : 256,	(30,561) : 229,	(31,561) : 258,	(32,561) : 157,	(33,561) : 169,	(34,561) : 253,	(35,561) : 358,	(36,561) : 525,	(37,561) : 518,	(38,561) : 280,	(39,561) : 279,	(40,561) : 832,
(1,562) : 103,	(2,562) : 108,	(3,562) : 557,	(4,562) : 558,	(5,562) : 110,	(6,562) : 63,	(7,562) : 92,	(8,562) : 809,	(9,562) : 286,	(10,562) : 94,	(11,562) : 110,	(12,562) : 533,	(13,562) : 148,	(14,562) : 281,	(15,562) : 401,	(16,562) : 621,	(17,562) : 507,	(18,562) : 91,	(19,562) : 471,	(20,562) : 97,	(21,562) : 258,	(22,562) : 84,	(23,562) : 85,	(24,562) : 510,	(25,562) : 688,	(26,562) : 208,	(27,562) : 152,	(28,562) : 95,	(29,562) : 412,	(30,562) : 101,	(31,562) : 71,	(32,562) : 62,	(33,562) : 177,	(34,562) : 49,	(35,562) : 220,	(36,562) : 337,	(37,562) : 365,	(38,562) : 99,	(39,562) : 107,	(40,562) : 704,
(1,563) : 319,	(2,563) : 343,	(3,563) : 443,	(4,563) : 362,	(5,563) : 344,	(6,563) : 300,	(7,563) : 310,	(8,563) : 729,	(9,563) : 393,	(10,563) : 312,	(11,563) : 345,	(12,563) : 437,	(13,563) : 187,	(14,563) : 389,	(15,563) : 222,	(16,563) : 452,	(17,563) : 314,	(18,563) : 325,	(19,563) : 391,	(20,563) : 313,	(21,563) : 88,	(22,563) : 279,	(23,563) : 320,	(24,563) : 315,	(25,563) : 665,	(26,563) : 172,	(27,563) : 182,	(28,563) : 311,	(29,563) : 193,	(30,563) : 317,	(31,563) : 308,	(32,563) : 231,	(33,563) : 313,	(34,563) : 282,	(35,563) : 450,	(36,563) : 456,	(37,563) : 408,	(38,563) : 336,	(39,563) : 342,	(40,563) : 689,
(1,564) : 197,	(2,564) : 209,	(3,564) : 478,	(4,564) : 487,	(5,564) : 203,	(6,564) : 162,	(7,564) : 186,	(8,564) : 744,	(9,564) : 290,	(10,564) : 189,	(11,564) : 210,	(12,564) : 505,	(13,564) : 142,	(14,564) : 286,	(15,564) : 334,	(16,564) : 563,	(17,564) : 437,	(18,564) : 184,	(19,564) : 448,	(20,564) : 191,	(21,564) : 184,	(22,564) : 138,	(23,564) : 179,	(24,564) : 439,	(25,564) : 643,	(26,564) : 179,	(27,564) : 142,	(28,564) : 188,	(29,564) : 329,	(30,564) : 195,	(31,564) : 171,	(32,564) : 120,	(33,564) : 234,	(34,564) : 141,	(35,564) : 320,	(36,564) : 350,	(37,564) : 340,	(38,564) : 199,	(39,564) : 207,	(40,564) : 662,
(1,565) : 122,	(2,565) : 66,	(3,565) : 637,	(4,565) : 698,	(5,565) : 35,	(6,565) : 82,	(7,565) : 120,	(8,565) : 858,	(9,565) : 275,	(10,565) : 120,	(11,565) : 66,	(12,565) : 647,	(13,565) : 274,	(14,565) : 271,	(15,565) : 541,	(16,565) : 754,	(17,565) : 648,	(18,565) : 54,	(19,565) : 583,	(20,565) : 123,	(21,565) : 401,	(22,565) : 120,	(23,565) : 59,	(24,565) : 650,	(25,565) : 711,	(26,565) : 338,	(27,565) : 279,	(28,565) : 123,	(29,565) : 556,	(30,565) : 124,	(31,565) : 78,	(32,565) : 179,	(33,565) : 239,	(34,565) : 97,	(35,565) : 125,	(36,565) : 303,	(37,565) : 381,	(38,565) : 58,	(39,565) : 63,	(40,565) : 721,
(1,566) : 323,	(2,566) : 365,	(3,566) : 573,	(4,566) : 280,	(5,566) : 385,	(6,566) : 336,	(7,566) : 317,	(8,566) : 862,	(9,566) : 503,	(10,566) : 318,	(11,566) : 366,	(12,566) : 304,	(13,566) : 153,	(14,566) : 498,	(15,566) : 124,	(16,566) : 347,	(17,566) : 230,	(18,566) : 367,	(19,566) : 256,	(20,566) : 318,	(21,566) : 47,	(22,566) : 347,	(23,566) : 361,	(24,566) : 233,	(25,566) : 801,	(26,566) : 96,	(27,566) : 147,	(28,566) : 317,	(29,566) : 157,	(30,566) : 320,	(31,566) : 339,	(32,566) : 241,	(33,566) : 267,	(34,566) : 327,	(35,566) : 453,	(36,566) : 564,	(37,566) : 534,	(38,566) : 364,	(39,566) : 365,	(40,566) : 824,
(1,567) : 158,	(2,567) : 208,	(3,567) : 782,	(4,567) : 567,	(5,567) : 260,	(6,567) : 233,	(7,567) : 166,	(8,567) : 1044,	(9,567) : 515,	(10,567) : 164,	(11,567) : 207,	(12,567) : 431,	(13,567) : 195,	(14,567) : 510,	(15,567) : 425,	(16,567) : 585,	(17,567) : 522,	(18,567) : 257,	(19,567) : 369,	(20,567) : 160,	(21,567) : 341,	(22,567) : 310,	(23,567) : 254,	(24,567) : 525,	(25,567) : 927,	(26,567) : 231,	(27,567) : 200,	(28,567) : 162,	(29,567) : 481,	(30,567) : 157,	(31,567) : 225,	(32,567) : 188,	(33,567) : 71,	(34,567) : 252,	(35,567) : 207,	(36,567) : 557,	(37,567) : 603,	(38,567) : 222,	(39,567) : 211,	(40,567) : 943,
(1,568) : 747,	(2,568) : 725,	(3,568) : 154,	(4,568) : 844,	(5,568) : 683,	(6,568) : 678,	(7,568) : 736,	(8,568) : 160,	(9,568) : 446,	(10,568) : 739,	(11,568) : 726,	(12,568) : 1002,	(13,568) : 721,	(14,568) : 449,	(15,568) : 761,	(16,568) : 958,	(17,568) : 811,	(18,568) : 672,	(19,568) : 962,	(20,568) : 742,	(21,568) : 663,	(22,568) : 598,	(23,568) : 671,	(24,568) : 812,	(25,568) : 154,	(26,568) : 735,	(27,568) : 719,	(28,568) : 740,	(29,568) : 685,	(30,568) : 746,	(31,568) : 689,	(32,568) : 697,	(33,568) : 816,	(34,568) : 653,	(35,568) : 826,	(36,568) : 458,	(37,568) : 333,	(38,568) : 708,	(39,568) : 722,	(40,568) : 184,
(1,569) : 349,	(2,569) : 399,	(3,569) : 669,	(4,569) : 263,	(5,569) : 429,	(6,569) : 380,	(7,569) : 345,	(8,569) : 958,	(9,569) : 582,	(10,569) : 346,	(11,569) : 399,	(12,569) : 214,	(13,569) : 185,	(14,569) : 578,	(15,569) : 127,	(16,569) : 297,	(17,569) : 218,	(18,569) : 413,	(19,569) : 161,	(20,569) : 344,	(21,569) : 143,	(22,569) : 408,	(23,569) : 408,	(24,569) : 221,	(25,569) : 896,	(26,569) : 122,	(27,569) : 180,	(28,569) : 344,	(29,569) : 204,	(30,569) : 346,	(31,569) : 381,	(32,569) : 280,	(33,569) : 263,	(34,569) : 377,	(35,569) : 468,	(36,569) : 642,	(37,569) : 622,	(38,569) : 402,	(39,569) : 400,	(40,569) : 919,
(1,570) : 174,	(2,570) : 177,	(3,570) : 488,	(4,570) : 527,	(5,570) : 166,	(6,570) : 128,	(7,570) : 163,	(8,570) : 746,	(9,570) : 261,	(10,570) : 165,	(11,570) : 179,	(12,570) : 536,	(13,570) : 160,	(14,570) : 256,	(15,570) : 374,	(16,570) : 601,	(17,570) : 477,	(18,570) : 146,	(19,570) : 477,	(20,570) : 168,	(21,570) : 224,	(22,570) : 97,	(23,570) : 142,	(24,570) : 479,	(25,570) : 635,	(26,570) : 207,	(27,570) : 162,	(28,570) : 166,	(29,570) : 370,	(30,570) : 172,	(31,570) : 137,	(32,570) : 111,	(33,570) : 231,	(34,570) : 105,	(35,570) : 290,	(36,570) : 318,	(37,570) : 321,	(38,570) : 166,	(39,570) : 175,	(40,570) : 652,
(1,571) : 502,	(2,571) : 542,	(3,571) : 577,	(4,571) : 128,	(5,571) : 556,	(6,571) : 508,	(7,571) : 495,	(8,571) : 864,	(9,571) : 628,	(10,571) : 497,	(11,571) : 543,	(12,571) : 304,	(13,571) : 333,	(14,571) : 625,	(15,571) : 79,	(16,571) : 234,	(17,571) : 86,	(18,571) : 538,	(19,571) : 290,	(20,571) : 496,	(21,571) : 189,	(22,571) : 506,	(23,571) : 532,	(24,571) : 87,	(25,571) : 845,	(26,571) : 275,	(27,571) : 327,	(28,571) : 495,	(29,571) : 46,	(30,571) : 499,	(31,571) : 513,	(32,571) : 418,	(33,571) : 444,	(34,571) : 496,	(35,571) : 632,	(36,571) : 691,	(37,571) : 632,	(38,571) : 539,	(39,571) : 541,	(40,571) : 872,
(1,572) : 534,	(2,572) : 500,	(3,572) : 283,	(4,572) : 810,	(5,572) : 452,	(6,572) : 459,	(7,572) : 524,	(8,572) : 407,	(9,572) : 192,	(10,572) : 526,	(11,572) : 501,	(12,572) : 904,	(13,572) : 558,	(14,572) : 195,	(15,572) : 688,	(16,572) : 913,	(17,572) : 767,	(18,572) : 445,	(19,572) : 852,	(20,572) : 530,	(21,572) : 555,	(22,572) : 378,	(23,572) : 445,	(24,572) : 769,	(25,572) : 247,	(26,572) : 592,	(27,572) : 558,	(28,572) : 528,	(29,572) : 637,	(30,572) : 534,	(31,572) : 469,	(32,572) : 505,	(33,572) : 625,	(34,572) : 437,	(35,572) : 589,	(36,572) : 193,	(37,572) : 84,	(38,572) : 483,	(39,572) : 497,	(40,572) : 259,
(1,573) : 241,	(2,573) : 226,	(3,573) : 421,	(4,573) : 582,	(5,573) : 194,	(6,573) : 175,	(7,573) : 230,	(8,573) : 666,	(9,573) : 180,	(10,573) : 233,	(11,573) : 227,	(12,573) : 617,	(13,573) : 249,	(14,573) : 175,	(15,573) : 435,	(16,573) : 666,	(17,573) : 533,	(18,573) : 178,	(19,573) : 560,	(20,573) : 236,	(21,573) : 286,	(22,573) : 103,	(23,573) : 175,	(24,573) : 535,	(25,573) : 547,	(26,573) : 291,	(27,573) : 250,	(28,573) : 234,	(29,573) : 415,	(30,573) : 240,	(31,573) : 187,	(32,573) : 196,	(33,573) : 317,	(34,573) : 150,	(35,573) : 335,	(36,573) : 240,	(37,573) : 232,	(38,573) : 210,	(39,573) : 223,	(40,573) : 564,
(1,574) : 88,	(2,574) : 134,	(3,574) : 624,	(4,574) : 512,	(5,574) : 167,	(6,574) : 121,	(7,574) : 82,	(8,574) : 887,	(9,574) : 379,	(10,574) : 83,	(11,574) : 135,	(12,574) : 454,	(13,574) : 84,	(14,574) : 374,	(15,574) : 357,	(16,574) : 561,	(17,574) : 463,	(18,574) : 154,	(19,574) : 391,	(20,574) : 82,	(21,574) : 231,	(22,574) : 175,	(23,574) : 149,	(24,574) : 466,	(25,574) : 775,	(26,574) : 147,	(27,574) : 90,	(28,574) : 81,	(29,574) : 387,	(30,574) : 84,	(31,574) : 119,	(32,574) : 32,	(33,574) : 90,	(34,574) : 126,	(35,574) : 218,	(36,574) : 429,	(37,574) : 456,	(38,574) : 138,	(39,574) : 135,	(40,574) : 792,
(1,575) : 529,	(2,575) : 497,	(3,575) : 268,	(4,575) : 794,	(5,575) : 449,	(6,575) : 454,	(7,575) : 519,	(8,575) : 403,	(9,575) : 193,	(10,575) : 521,	(11,575) : 498,	(12,575) : 890,	(13,575) : 548,	(14,575) : 196,	(15,575) : 673,	(16,575) : 898,	(17,575) : 752,	(18,575) : 441,	(19,575) : 840,	(20,575) : 525,	(21,575) : 541,	(22,575) : 373,	(23,575) : 442,	(24,575) : 753,	(25,575) : 250,	(26,575) : 581,	(27,575) : 548,	(28,575) : 523,	(29,575) : 621,	(30,575) : 529,	(31,575) : 465,	(32,575) : 498,	(33,575) : 618,	(34,575) : 431,	(35,575) : 587,	(36,575) : 199,	(37,575) : 81,	(38,575) : 479,	(39,575) : 493,	(40,575) : 264,
(1,576) : 519,	(2,576) : 560,	(3,576) : 601,	(4,576) : 102,	(5,576) : 576,	(6,576) : 528,	(7,576) : 513,	(8,576) : 887,	(9,576) : 654,	(10,576) : 514,	(11,576) : 561,	(12,576) : 289,	(13,576) : 349,	(14,576) : 650,	(15,576) : 83,	(16,576) : 208,	(17,576) : 60,	(18,576) : 558,	(19,576) : 280,	(20,576) : 514,	(21,576) : 210,	(22,576) : 528,	(23,576) : 552,	(24,576) : 61,	(25,576) : 870,	(26,576) : 289,	(27,576) : 343,	(28,576) : 512,	(29,576) : 72,	(30,576) : 516,	(31,576) : 532,	(32,576) : 436,	(33,576) : 457,	(34,576) : 516,	(35,576) : 649,	(36,576) : 716,	(37,576) : 658,	(38,576) : 558,	(39,576) : 560,	(40,576) : 897,
(1,577) : 1014,	(2,577) : 979,	(3,577) : 504,	(4,577) : 1193,	(5,577) : 930,	(6,577) : 939,	(7,577) : 1004,	(8,577) : 257,	(9,577) : 664,	(10,577) : 1007,	(11,577) : 980,	(12,577) : 1340,	(13,577) : 1027,	(14,577) : 668,	(15,577) : 1103,	(16,577) : 1307,	(17,577) : 1159,	(18,577) : 924,	(19,577) : 1296,	(20,577) : 1010,	(21,577) : 994,	(22,577) : 859,	(23,577) : 925,	(24,577) : 1160,	(25,577) : 242,	(26,577) : 1053,	(27,577) : 1027,	(28,577) : 1008,	(29,577) : 1032,	(30,577) : 1014,	(31,577) : 949,	(32,577) : 983,	(33,577) : 1104,	(34,577) : 917,	(35,577) : 1062,	(36,577) : 642,	(37,577) : 564,	(38,577) : 962,	(39,577) : 976,	(40,577) : 223,
(1,578) : 195,	(2,578) : 227,	(3,578) : 536,	(4,578) : 427,	(5,578) : 240,	(6,578) : 192,	(7,578) : 186,	(8,578) : 811,	(9,578) : 371,	(10,578) : 188,	(11,578) : 228,	(12,578) : 424,	(13,578) : 70,	(14,578) : 367,	(15,578) : 271,	(16,578) : 493,	(17,578) : 376,	(18,578) : 221,	(19,578) : 366,	(20,578) : 188,	(21,578) : 127,	(22,578) : 200,	(23,578) : 216,	(24,578) : 379,	(25,578) : 719,	(26,578) : 97,	(27,578) : 67,	(28,578) : 187,	(29,578) : 283,	(30,578) : 192,	(31,578) : 196,	(32,578) : 106,	(33,578) : 187,	(34,578) : 180,	(35,578) : 327,	(36,578) : 430,	(37,578) : 421,	(38,578) : 223,	(39,578) : 227,	(40,578) : 740,
(1,579) : 464,	(2,579) : 505,	(3,579) : 590,	(4,579) : 146,	(5,579) : 523,	(6,579) : 474,	(7,579) : 458,	(8,579) : 879,	(9,579) : 611,	(10,579) : 459,	(11,579) : 506,	(12,579) : 276,	(13,579) : 294,	(14,579) : 606,	(15,579) : 36,	(16,579) : 237,	(17,579) : 97,	(18,579) : 504,	(19,579) : 255,	(20,579) : 458,	(21,579) : 158,	(22,579) : 477,	(23,579) : 499,	(24,579) : 99,	(25,579) : 850,	(26,579) : 234,	(27,579) : 287,	(28,579) : 457,	(29,579) : 58,	(30,579) : 461,	(31,579) : 478,	(32,579) : 381,	(33,579) : 402,	(34,579) : 463,	(35,579) : 593,	(36,579) : 673,	(37,579) : 622,	(38,579) : 504,	(39,579) : 505,	(40,579) : 876,
(1,580) : 826,	(2,580) : 879,	(3,580) : 1023,	(4,580) : 333,	(5,580) : 913,	(6,580) : 865,	(7,580) : 825,	(8,580) : 1304,	(9,580) : 1056,	(10,580) : 825,	(11,580) : 880,	(12,580) : 305,	(13,580) : 671,	(14,580) : 1051,	(15,580) : 447,	(16,580) : 221,	(17,580) : 369,	(18,580) : 898,	(19,580) : 367,	(20,580) : 823,	(21,580) : 592,	(22,580) : 894,	(23,580) : 893,	(24,580) : 368,	(25,580) : 1298,	(26,580) : 609,	(27,580) : 666,	(28,580) : 823,	(29,580) : 500,	(30,580) : 823,	(31,580) : 865,	(32,580) : 766,	(33,580) : 723,	(34,580) : 863,	(35,580) : 932,	(36,580) : 1117,	(37,580) : 1076,	(38,580) : 884,	(39,580) : 881,	(40,580) : 1326,
(1,581) : 64,	(2,581) : 94,	(3,581) : 604,	(4,581) : 551,	(5,581) : 119,	(6,581) : 72,	(7,581) : 54,	(8,581) : 859,	(9,581) : 334,	(10,581) : 56,	(11,581) : 95,	(12,581) : 503,	(13,581) : 125,	(14,581) : 329,	(15,581) : 394,	(16,581) : 605,	(17,581) : 501,	(18,581) : 105,	(19,581) : 440,	(20,581) : 57,	(21,581) : 259,	(22,581) : 128,	(23,581) : 100,	(24,581) : 503,	(25,581) : 739,	(26,581) : 189,	(27,581) : 130,	(28,581) : 55,	(29,581) : 416,	(30,581) : 61,	(31,581) : 71,	(32,581) : 30,	(33,581) : 127,	(34,581) : 76,	(35,581) : 194,	(36,581) : 382,	(37,581) : 416,	(38,581) : 93,	(39,581) : 94,	(40,581) : 755,
(1,582) : 692,	(2,582) : 662,	(3,582) : 240,	(4,582) : 889,	(5,582) : 614,	(6,582) : 618,	(7,582) : 682,	(8,582) : 248,	(9,582) : 357,	(10,582) : 684,	(11,582) : 663,	(12,582) : 1017,	(13,582) : 697,	(14,582) : 360,	(15,582) : 786,	(16,582) : 999,	(17,582) : 851,	(18,582) : 606,	(19,582) : 971,	(20,582) : 688,	(21,582) : 668,	(22,582) : 537,	(23,582) : 606,	(24,582) : 852,	(25,582) : 88,	(26,582) : 723,	(27,582) : 697,	(28,582) : 686,	(29,582) : 721,	(30,582) : 692,	(31,582) : 629,	(32,582) : 655,	(33,582) : 776,	(34,582) : 595,	(35,582) : 753,	(36,582) : 353,	(37,582) : 247,	(38,582) : 644,	(39,582) : 658,	(40,582) : 109,
(1,583) : 320,	(2,583) : 303,	(3,583) : 346,	(4,583) : 602,	(5,583) : 266,	(6,583) : 253,	(7,583) : 309,	(8,583) : 587,	(9,583) : 157,	(10,583) : 312,	(11,583) : 304,	(12,583) : 666,	(13,583) : 316,	(14,583) : 154,	(15,583) : 465,	(16,583) : 695,	(17,583) : 555,	(18,583) : 252,	(19,583) : 612,	(20,583) : 315,	(21,583) : 322,	(22,583) : 176,	(23,583) : 250,	(24,583) : 557,	(25,583) : 472,	(26,583) : 349,	(27,583) : 316,	(28,583) : 313,	(29,583) : 429,	(30,583) : 319,	(31,583) : 264,	(32,583) : 273,	(33,583) : 394,	(34,583) : 227,	(35,583) : 410,	(36,583) : 219,	(37,583) : 170,	(38,583) : 286,	(39,583) : 299,	(40,583) : 490,
(1,584) : 93,	(2,584) : 55,	(3,584) : 693,	(4,584) : 692,	(5,584) : 82,	(6,584) : 104,	(7,584) : 98,	(8,584) : 922,	(9,584) : 342,	(10,584) : 97,	(11,584) : 54,	(12,584) : 617,	(13,584) : 263,	(14,584) : 338,	(15,584) : 537,	(16,584) : 739,	(17,584) : 643,	(18,584) : 95,	(19,584) : 553,	(20,584) : 98,	(21,584) : 405,	(22,584) : 171,	(23,584) : 98,	(24,584) : 646,	(25,584) : 777,	(26,584) : 327,	(27,584) : 269,	(28,584) : 99,	(29,584) : 562,	(30,584) : 96,	(31,584) : 94,	(32,584) : 175,	(33,584) : 198,	(34,584) : 128,	(35,584) : 62,	(36,584) : 370,	(37,584) : 447,	(38,584) : 66,	(39,584) : 56,	(40,584) : 788,
(1,585) : 219,	(2,585) : 166,	(3,585) : 580,	(4,585) : 750,	(5,585) : 115,	(6,585) : 155,	(7,585) : 214,	(8,585) : 779,	(9,585) : 184,	(10,585) : 215,	(11,585) : 166,	(12,585) : 728,	(13,585) : 345,	(14,585) : 181,	(15,585) : 595,	(16,585) : 818,	(17,585) : 699,	(18,585) : 123,	(19,585) : 666,	(20,585) : 219,	(21,585) : 446,	(22,585) : 127,	(23,585) : 128,	(24,585) : 702,	(25,585) : 620,	(26,585) : 406,	(27,585) : 349,	(28,585) : 218,	(29,585) : 594,	(30,585) : 220,	(31,585) : 159,	(32,585) : 252,	(33,585) : 335,	(34,585) : 152,	(35,585) : 221,	(36,585) : 202,	(37,585) : 297,	(38,585) : 152,	(39,585) : 163,	(40,585) : 628,
(1,586) : 883,	(2,586) : 854,	(3,586) : 328,	(4,586) : 1019,	(5,586) : 807,	(6,586) : 809,	(7,586) : 872,	(8,586) : 110,	(9,586) : 549,	(10,586) : 875,	(11,586) : 855,	(12,586) : 1174,	(13,586) : 877,	(14,586) : 553,	(15,586) : 934,	(16,586) : 1133,	(17,586) : 986,	(18,586) : 798,	(19,586) : 1132,	(20,586) : 878,	(21,586) : 831,	(22,586) : 728,	(23,586) : 798,	(24,586) : 987,	(25,586) : 117,	(26,586) : 897,	(27,586) : 876,	(28,586) : 876,	(29,586) : 860,	(30,586) : 882,	(31,586) : 820,	(32,586) : 842,	(33,586) : 963,	(34,586) : 786,	(35,586) : 946,	(36,586) : 542,	(37,586) : 439,	(38,586) : 836,	(39,586) : 850,	(40,586) : 121,
(1,587) : 797,	(2,587) : 762,	(3,587) : 344,	(4,587) : 1010,	(5,587) : 713,	(6,587) : 722,	(7,587) : 787,	(8,587) : 234,	(9,587) : 448,	(10,587) : 790,	(11,587) : 763,	(12,587) : 1139,	(13,587) : 814,	(14,587) : 452,	(15,587) : 908,	(16,587) : 1121,	(17,587) : 973,	(18,587) : 707,	(19,587) : 1092,	(20,587) : 793,	(21,587) : 790,	(22,587) : 641,	(23,587) : 708,	(24,587) : 974,	(25,587) : 51,	(26,587) : 842,	(27,587) : 814,	(28,587) : 791,	(29,587) : 843,	(30,587) : 797,	(31,587) : 732,	(32,587) : 767,	(33,587) : 887,	(34,587) : 700,	(35,587) : 847,	(36,587) : 431,	(37,587) : 347,	(38,587) : 745,	(39,587) : 759,	(40,587) : 21,
(1,588) : 647,	(2,588) : 696,	(3,588) : 804,	(4,588) : 115,	(5,588) : 724,	(6,588) : 675,	(7,588) : 643,	(8,588) : 1088,	(9,588) : 842,	(10,588) : 644,	(11,588) : 697,	(12,588) : 200,	(13,588) : 481,	(14,588) : 838,	(15,588) : 230,	(16,588) : 3,	(17,588) : 146,	(18,588) : 707,	(19,588) : 240,	(20,588) : 642,	(21,588) : 380,	(22,588) : 693,	(23,588) : 702,	(24,588) : 146,	(25,588) : 1076,	(26,588) : 417,	(27,588) : 475,	(28,588) : 642,	(29,588) : 277,	(30,588) : 644,	(31,588) : 677,	(32,588) : 576,	(33,588) : 556,	(34,588) : 669,	(35,588) : 765,	(36,588) : 904,	(37,588) : 857,	(38,588) : 698,	(39,588) : 697,	(40,588) : 1104,
(1,589) : 467,	(2,589) : 505,	(3,589) : 556,	(4,589) : 164,	(5,589) : 519,	(6,589) : 471,	(7,589) : 460,	(8,589) : 845,	(9,589) : 592,	(10,589) : 461,	(11,589) : 506,	(12,589) : 312,	(13,589) : 298,	(14,589) : 588,	(15,589) : 69,	(16,589) : 263,	(17,589) : 118,	(18,589) : 500,	(19,589) : 290,	(20,589) : 461,	(21,589) : 152,	(22,589) : 468,	(23,589) : 495,	(24,589) : 119,	(25,589) : 818,	(26,589) : 242,	(27,589) : 292,	(28,589) : 460,	(29,589) : 22,	(30,589) : 464,	(31,589) : 476,	(32,589) : 381,	(33,589) : 413,	(34,589) : 458,	(35,589) : 597,	(36,589) : 654,	(37,589) : 597,	(38,589) : 502,	(39,589) : 505,	(40,589) : 845,
(1,590) : 550,	(2,590) : 594,	(3,590) : 660,	(4,590) : 51,	(5,590) : 614,	(6,590) : 565,	(7,590) : 544,	(8,590) : 946,	(9,590) : 707,	(10,590) : 545,	(11,590) : 595,	(12,590) : 247,	(13,590) : 379,	(14,590) : 703,	(15,590) : 106,	(16,590) : 148,	(17,590) : 1,	(18,590) : 596,	(19,590) : 250,	(20,590) : 544,	(21,590) : 252,	(22,590) : 572,	(23,590) : 591,	(24,590) : 4,	(25,590) : 930,	(26,590) : 316,	(27,590) : 373,	(28,590) : 544,	(29,590) : 131,	(30,590) : 547,	(31,590) : 569,	(32,590) : 470,	(33,590) : 477,	(34,590) : 556,	(35,590) : 676,	(36,590) : 769,	(37,590) : 715,	(38,590) : 593,	(39,590) : 594,	(40,590) : 957,
(1,591) : 760,	(2,591) : 815,	(3,591) : 1033,	(4,591) : 352,	(5,591) : 853,	(6,591) : 807,	(7,591) : 760,	(8,591) : 1320,	(9,591) : 1020,	(10,591) : 760,	(11,591) : 815,	(12,591) : 232,	(13,591) : 616,	(14,591) : 1015,	(15,591) : 429,	(16,591) : 235,	(17,591) : 375,	(18,591) : 840,	(19,591) : 296,	(20,591) : 757,	(21,591) : 562,	(22,591) : 844,	(23,591) : 835,	(24,591) : 375,	(25,591) : 1297,	(26,591) : 558,	(27,591) : 612,	(28,591) : 758,	(29,591) : 498,	(30,591) : 758,	(31,591) : 806,	(32,591) : 708,	(33,591) : 650,	(34,591) : 808,	(35,591) : 857,	(36,591) : 1079,	(37,591) : 1052,	(38,591) : 822,	(39,591) : 817,	(40,591) : 1323,
(1,592) : 564,	(2,592) : 610,	(3,592) : 696,	(4,592) : 44,	(5,592) : 633,	(6,592) : 584,	(7,592) : 559,	(8,592) : 982,	(9,592) : 735,	(10,592) : 560,	(11,592) : 611,	(12,592) : 221,	(13,592) : 394,	(14,592) : 731,	(15,592) : 126,	(16,592) : 114,	(17,592) : 36,	(18,592) : 615,	(19,592) : 231,	(20,592) : 559,	(21,592) : 276,	(22,592) : 594,	(23,592) : 610,	(24,592) : 36,	(25,592) : 965,	(26,592) : 330,	(27,592) : 388,	(28,592) : 558,	(29,592) : 165,	(30,592) : 561,	(31,592) : 587,	(32,592) : 487,	(33,592) : 485,	(34,592) : 575,	(35,592) : 688,	(36,592) : 797,	(37,592) : 746,	(38,592) : 610,	(39,592) : 610,	(40,592) : 992,
(1,593) : 541,	(2,593) : 585,	(3,593) : 641,	(4,593) : 64,	(5,593) : 604,	(6,593) : 555,	(7,593) : 536,	(8,593) : 927,	(9,593) : 691,	(10,593) : 537,	(11,593) : 585,	(12,593) : 261,	(13,593) : 370,	(14,593) : 687,	(15,593) : 97,	(16,593) : 167,	(17,593) : 19,	(18,593) : 586,	(19,593) : 260,	(20,593) : 536,	(21,593) : 240,	(22,593) : 559,	(23,593) : 580,	(24,593) : 19,	(25,593) : 912,	(26,593) : 308,	(27,593) : 364,	(28,593) : 535,	(29,593) : 113,	(30,593) : 538,	(31,593) : 559,	(32,593) : 460,	(33,593) : 472,	(34,593) : 544,	(35,593) : 669,	(36,593) : 753,	(37,593) : 698,	(38,593) : 584,	(39,593) : 585,	(40,593) : 939,
(1,594) : 571,	(2,594) : 612,	(3,594) : 620,	(4,594) : 71,	(5,594) : 628,	(6,594) : 579,	(7,594) : 564,	(8,594) : 903,	(9,594) : 698,	(10,594) : 566,	(11,594) : 613,	(12,594) : 304,	(13,594) : 401,	(14,594) : 694,	(15,594) : 132,	(16,594) : 188,	(17,594) : 56,	(18,594) : 609,	(19,594) : 305,	(20,594) : 565,	(21,594) : 261,	(22,594) : 578,	(23,594) : 604,	(24,594) : 55,	(25,594) : 897,	(26,594) : 340,	(27,594) : 394,	(28,594) : 564,	(29,594) : 115,	(30,594) : 568,	(31,594) : 584,	(32,594) : 488,	(33,594) : 508,	(34,594) : 567,	(35,594) : 700,	(36,594) : 760,	(37,594) : 697,	(38,594) : 610,	(39,594) : 612,	(40,594) : 924,
(1,595) : 96,	(2,595) : 152,	(3,595) : 686,	(4,595) : 525,	(5,595) : 197,	(6,595) : 159,	(7,595) : 98,	(8,595) : 949,	(9,595) : 433,	(10,595) : 97,	(11,595) : 152,	(12,595) : 434,	(13,595) : 112,	(14,595) : 428,	(15,595) : 373,	(16,595) : 561,	(17,595) : 477,	(18,595) : 189,	(19,595) : 370,	(20,595) : 94,	(21,595) : 265,	(22,595) : 227,	(23,595) : 185,	(24,595) : 480,	(25,595) : 835,	(26,595) : 164,	(27,595) : 118,	(28,595) : 95,	(29,595) : 416,	(30,595) : 94,	(31,595) : 153,	(32,595) : 93,	(33,595) : 28,	(34,595) : 172,	(35,595) : 203,	(36,595) : 480,	(37,595) : 514,	(38,595) : 162,	(39,595) : 154,	(40,595) : 852,
(1,596) : 392,	(2,596) : 355,	(3,596) : 376,	(4,596) : 753,	(5,596) : 305,	(6,596) : 316,	(7,596) : 383,	(8,596) : 551,	(9,596) : 45,	(10,596) : 385,	(11,596) : 355,	(12,596) : 808,	(13,596) : 440,	(14,596) : 48,	(15,596) : 615,	(16,596) : 846,	(17,596) : 706,	(18,596) : 299,	(19,596) : 752,	(20,596) : 389,	(21,596) : 470,	(22,596) : 237,	(23,596) : 300,	(24,596) : 708,	(25,596) : 395,	(26,596) : 483,	(27,596) : 441,	(28,596) : 387,	(29,596) : 580,	(30,596) : 392,	(31,596) : 326,	(32,596) : 374,	(33,596) : 490,	(34,596) : 296,	(35,596) : 440,	(36,596) : 80,	(37,596) : 69,	(38,596) : 337,	(39,596) : 351,	(40,596) : 405,
(1,597) : 205,	(2,597) : 251,	(3,597) : 604,	(4,597) : 395,	(5,597) : 277,	(6,597) : 229,	(7,597) : 200,	(8,597) : 883,	(9,597) : 443,	(10,597) : 201,	(11,597) : 252,	(12,597) : 356,	(13,597) : 34,	(14,597) : 438,	(15,597) : 239,	(16,597) : 446,	(17,597) : 345,	(18,597) : 261,	(19,597) : 296,	(20,597) : 200,	(21,597) : 127,	(22,597) : 259,	(23,597) : 256,	(24,597) : 348,	(25,597) : 796,	(26,597) : 30,	(27,597) : 29,	(28,597) : 199,	(29,597) : 276,	(30,597) : 202,	(31,597) : 230,	(32,597) : 129,	(33,597) : 151,	(34,597) : 225,	(35,597) : 333,	(36,597) : 500,	(37,597) : 498,	(38,597) : 252,	(39,597) : 251,	(40,597) : 816,
(1,598) : 3,	(2,598) : 59,	(3,598) : 659,	(4,598) : 597,	(5,598) : 107,	(6,598) : 78,	(7,598) : 11,	(8,598) : 907,	(9,598) : 360,	(10,598) : 9,	(11,598) : 59,	(12,598) : 525,	(13,598) : 168,	(14,598) : 355,	(15,598) : 441,	(16,598) : 643,	(17,598) : 547,	(18,598) : 102,	(19,598) : 461,	(20,598) : 6,	(21,598) : 314,	(22,598) : 157,	(23,598) : 98,	(24,598) : 550,	(25,598) : 779,	(26,598) : 231,	(27,598) : 174,	(28,598) : 8,	(29,598) : 470,	(30,598) : 1,	(31,598) : 69,	(32,598) : 86,	(33,598) : 114,	(34,598) : 99,	(35,598) : 135,	(36,598) : 401,	(37,598) : 452,	(38,598) : 70,	(39,598) : 61,	(40,598) : 794,
(1,599) : 559,	(2,599) : 613,	(3,599) : 879,	(4,599) : 260,	(5,599) : 652,	(6,599) : 605,	(7,599) : 559,	(8,599) : 1169,	(9,599) : 824,	(10,599) : 558,	(11,599) : 614,	(12,599) : 30,	(13,599) : 415,	(14,599) : 819,	(15,599) : 265,	(16,599) : 186,	(17,599) : 255,	(18,599) : 638,	(19,599) : 94,	(20,599) : 556,	(21,599) : 376,	(22,599) : 644,	(23,599) : 633,	(24,599) : 257,	(25,599) : 1126,	(26,599) : 359,	(27,599) : 411,	(28,599) : 556,	(29,599) : 350,	(30,599) : 556,	(31,599) : 604,	(32,599) : 507,	(33,599) : 451,	(34,599) : 606,	(35,599) : 659,	(36,599) : 883,	(37,599) : 863,	(38,599) : 620,	(39,599) : 615,	(40,599) : 1150,
(1,600) : 547,	(2,600) : 594,	(3,600) : 702,	(4,600) : 67,	(5,600) : 618,	(6,600) : 569,	(7,600) : 543,	(8,600) : 989,	(9,600) : 728,	(10,600) : 543,	(11,600) : 595,	(12,600) : 200,	(13,600) : 378,	(14,600) : 724,	(15,600) : 116,	(16,600) : 116,	(17,600) : 49,	(18,600) : 601,	(19,600) : 208,	(20,600) : 542,	(21,600) : 266,	(22,600) : 582,	(23,600) : 595,	(24,600) : 50,	(25,600) : 968,	(26,600) : 314,	(27,600) : 372,	(28,600) : 542,	(29,600) : 168,	(30,600) : 544,	(31,600) : 572,	(32,600) : 471,	(33,600) : 466,	(34,600) : 561,	(35,600) : 670,	(36,600) : 790,	(37,600) : 742,	(38,600) : 595,	(39,600) : 595,	(40,600) : 995,
(1,601) : 558,	(2,601) : 605,	(3,601) : 717,	(4,601) : 67,	(5,601) : 630,	(6,601) : 581,	(7,601) : 554,	(8,601) : 1004,	(9,601) : 743,	(10,601) : 555,	(11,601) : 606,	(12,601) : 193,	(13,601) : 389,	(14,601) : 738,	(15,601) : 131,	(16,601) : 101,	(17,601) : 61,	(18,601) : 613,	(19,601) : 206,	(20,601) : 553,	(21,601) : 280,	(22,601) : 596,	(23,601) : 608,	(24,601) : 61,	(25,601) : 983,	(26,601) : 325,	(27,601) : 383,	(28,601) : 553,	(29,601) : 183,	(30,601) : 555,	(31,601) : 584,	(32,601) : 483,	(33,601) : 475,	(34,601) : 574,	(35,601) : 680,	(36,601) : 804,	(37,601) : 758,	(38,601) : 607,	(39,601) : 606,	(40,601) : 1010,
(1,602) : 309,	(2,602) : 345,	(3,602) : 525,	(4,602) : 312,	(5,602) : 358,	(6,602) : 310,	(7,602) : 301,	(8,602) : 812,	(9,602) : 456,	(10,602) : 303,	(11,602) : 346,	(12,602) : 356,	(13,602) : 147,	(14,602) : 451,	(15,602) : 159,	(16,602) : 389,	(17,602) : 262,	(18,602) : 339,	(19,602) : 308,	(20,602) : 303,	(21,602) : 9,	(22,602) : 310,	(23,602) : 334,	(24,602) : 264,	(25,602) : 749,	(26,602) : 107,	(27,602) : 141,	(28,602) : 301,	(29,602) : 165,	(30,602) : 306,	(31,602) : 315,	(32,602) : 222,	(33,602) : 272,	(34,602) : 297,	(35,602) : 440,	(36,602) : 517,	(37,602) : 483,	(38,602) : 341,	(39,602) : 344,	(40,602) : 772,
(1,603) : 512,	(2,603) : 559,	(3,603) : 692,	(4,603) : 99,	(5,603) : 584,	(6,603) : 535,	(7,603) : 508,	(8,603) : 980,	(9,603) : 700,	(10,603) : 508,	(11,603) : 560,	(12,603) : 188,	(13,603) : 343,	(14,603) : 696,	(15,603) : 89,	(16,603) : 144,	(17,603) : 66,	(18,603) : 567,	(19,603) : 185,	(20,603) : 507,	(21,603) : 237,	(22,603) : 550,	(23,603) : 562,	(24,603) : 68,	(25,603) : 953,	(26,603) : 279,	(27,603) : 337,	(28,603) : 506,	(29,603) : 156,	(30,603) : 509,	(31,603) : 537,	(32,603) : 437,	(33,603) : 431,	(34,603) : 528,	(35,603) : 635,	(36,603) : 762,	(37,603) : 718,	(38,603) : 560,	(39,603) : 560,	(40,603) : 979,
(1,604) : 162,	(2,604) : 209,	(3,604) : 792,	(4,604) : 579,	(5,604) : 262,	(6,604) : 238,	(7,604) : 171,	(8,604) : 1053,	(9,604) : 520,	(10,604) : 168,	(11,604) : 209,	(12,604) : 441,	(13,604) : 207,	(14,604) : 515,	(15,604) : 438,	(16,604) : 596,	(17,604) : 535,	(18,604) : 260,	(19,604) : 379,	(20,604) : 165,	(21,604) : 354,	(22,604) : 315,	(23,604) : 258,	(24,604) : 538,	(25,604) : 934,	(26,604) : 244,	(27,604) : 213,	(28,604) : 167,	(29,604) : 494,	(30,604) : 161,	(31,604) : 229,	(32,604) : 197,	(33,604) : 82,	(34,604) : 257,	(35,604) : 201,	(36,604) : 560,	(37,604) : 609,	(38,604) : 225,	(39,604) : 213,	(40,604) : 950,
(1,605) : 625,	(2,605) : 675,	(3,605) : 801,	(4,605) : 118,	(5,605) : 703,	(6,605) : 654,	(7,605) : 621,	(8,605) : 1087,	(9,605) : 827,	(10,605) : 622,	(11,605) : 675,	(12,605) : 176,	(13,605) : 460,	(14,605) : 823,	(15,605) : 216,	(16,605) : 23,	(17,605) : 141,	(18,605) : 687,	(19,605) : 214,	(20,605) : 620,	(21,605) : 364,	(22,605) : 674,	(23,605) : 681,	(24,605) : 141,	(25,605) : 1070,	(26,605) : 396,	(27,605) : 454,	(28,605) : 620,	(29,605) : 270,	(30,605) : 622,	(31,605) : 656,	(32,605) : 555,	(33,605) : 533,	(34,605) : 649,	(35,605) : 742,	(36,605) : 889,	(37,605) : 845,	(38,605) : 677,	(39,605) : 676,	(40,605) : 1097,
(1,606) : 475,	(2,606) : 520,	(3,606) : 632,	(4,606) : 125,	(5,606) : 541,	(6,606) : 492,	(7,606) : 470,	(8,606) : 921,	(9,606) : 643,	(10,606) : 471,	(11,606) : 520,	(12,606) : 236,	(13,606) : 304,	(14,606) : 639,	(15,606) : 32,	(16,606) : 201,	(17,606) : 75,	(18,606) : 523,	(19,606) : 219,	(20,606) : 470,	(21,606) : 182,	(22,606) : 501,	(23,606) : 517,	(24,606) : 78,	(25,606) : 891,	(26,606) : 242,	(27,606) : 298,	(28,606) : 469,	(29,606) : 99,	(30,606) : 472,	(31,606) : 495,	(32,606) : 396,	(33,606) : 404,	(34,606) : 482,	(35,606) : 602,	(36,606) : 705,	(37,606) : 658,	(38,606) : 519,	(39,606) : 520,	(40,606) : 917,
(1,607) : 172,	(2,607) : 150,	(3,607) : 786,	(4,607) : 764,	(5,607) : 175,	(6,607) : 200,	(7,607) : 180,	(8,607) : 1008,	(9,607) : 418,	(10,607) : 179,	(11,607) : 148,	(12,607) : 662,	(13,607) : 339,	(14,607) : 415,	(15,607) : 611,	(16,607) : 801,	(17,607) : 716,	(18,607) : 190,	(19,607) : 598,	(20,607) : 178,	(21,607) : 488,	(22,607) : 265,	(23,607) : 193,	(24,607) : 718,	(25,607) : 855,	(26,607) : 400,	(27,607) : 345,	(28,607) : 180,	(29,607) : 644,	(30,607) : 175,	(31,607) : 190,	(32,607) : 260,	(33,607) : 246,	(34,607) : 225,	(35,607) : 40,	(36,607) : 437,	(37,607) : 528,	(38,607) : 162,	(39,607) : 152,	(40,607) : 864,
(1,608) : 619,	(2,608) : 674,	(3,608) : 961,	(4,608) : 327,	(5,608) : 715,	(6,608) : 670,	(7,608) : 619,	(8,608) : 1251,	(9,608) : 899,	(10,608) : 619,	(11,608) : 674,	(12,608) : 106,	(13,608) : 485,	(14,608) : 894,	(15,608) : 347,	(16,608) : 234,	(17,608) : 330,	(18,608) : 703,	(19,608) : 161,	(20,608) : 616,	(21,608) : 456,	(22,608) : 714,	(23,608) : 698,	(24,608) : 331,	(25,608) : 1207,	(26,608) : 431,	(27,608) : 481,	(28,608) : 617,	(29,608) : 431,	(30,608) : 616,	(31,608) : 668,	(32,608) : 573,	(33,608) : 506,	(34,608) : 673,	(35,608) : 710,	(36,608) : 957,	(37,608) : 941,	(38,608) : 682,	(39,608) : 676,	(40,608) : 1232,
(1,609) : 307,	(2,609) : 351,	(3,609) : 590,	(4,609) : 294,	(5,609) : 373,	(6,609) : 324,	(7,609) : 301,	(8,609) : 878,	(9,609) : 504,	(10,609) : 302,	(11,609) : 352,	(12,609) : 297,	(13,609) : 136,	(14,609) : 499,	(15,609) : 137,	(16,609) : 353,	(17,609) : 244,	(18,609) : 356,	(19,609) : 245,	(20,609) : 302,	(21,609) : 63,	(22,609) : 340,	(23,609) : 350,	(24,609) : 246,	(25,609) : 813,	(26,609) : 76,	(27,609) : 130,	(28,609) : 301,	(29,609) : 179,	(30,609) : 304,	(31,609) : 327,	(32,609) : 227,	(33,609) : 246,	(34,609) : 316,	(35,609) : 435,	(36,609) : 564,	(37,609) : 539,	(38,609) : 351,	(39,609) : 351,	(40,609) : 836,
(1,610) : 897,	(2,610) : 867,	(3,610) : 357,	(4,610) : 1047,	(5,610) : 819,	(6,610) : 823,	(7,610) : 887,	(8,610) : 137,	(9,610) : 559,	(10,610) : 890,	(11,610) : 868,	(12,610) : 1199,	(13,610) : 897,	(14,610) : 562,	(15,610) : 960,	(16,610) : 1161,	(17,610) : 1014,	(18,610) : 811,	(19,610) : 1156,	(20,610) : 893,	(21,610) : 854,	(22,610) : 742,	(23,610) : 812,	(24,610) : 1014,	(25,610) : 122,	(26,610) : 918,	(27,610) : 896,	(28,610) : 891,	(29,610) : 887,	(30,610) : 897,	(31,610) : 834,	(32,610) : 859,	(33,610) : 980,	(34,610) : 800,	(35,610) : 957,	(36,610) : 548,	(37,610) : 451,	(38,610) : 849,	(39,610) : 863,	(40,610) : 118,
(1,611) : 113,	(2,611) : 151,	(3,611) : 766,	(4,611) : 618,	(5,611) : 204,	(6,611) : 187,	(7,611) : 123,	(8,611) : 1018,	(9,611) : 467,	(10,611) : 120,	(11,611) : 151,	(12,611) : 498,	(13,611) : 214,	(14,611) : 463,	(15,611) : 470,	(16,611) : 644,	(17,611) : 571,	(18,611) : 205,	(19,611) : 435,	(20,611) : 117,	(21,611) : 368,	(22,611) : 268,	(23,611) : 204,	(24,611) : 574,	(25,611) : 891,	(26,611) : 264,	(27,611) : 220,	(28,611) : 119,	(29,611) : 517,	(30,611) : 113,	(31,611) : 177,	(32,611) : 173,	(33,611) : 94,	(34,611) : 210,	(35,611) : 135,	(36,611) : 505,	(37,611) : 562,	(38,611) : 168,	(39,611) : 155,	(40,611) : 905,
(1,612) : 1093,	(2,612) : 1058,	(3,612) : 570,	(4,612) : 1261,	(5,612) : 1008,	(6,612) : 1018,	(7,612) : 1083,	(8,612) : 305,	(9,612) : 742,	(10,612) : 1085,	(11,612) : 1059,	(12,612) : 1414,	(13,612) : 1105,	(14,612) : 746,	(15,612) : 1176,	(16,612) : 1376,	(17,612) : 1229,	(18,612) : 1002,	(19,612) : 1371,	(20,612) : 1089,	(21,612) : 1069,	(22,612) : 937,	(23,612) : 1003,	(24,612) : 1229,	(25,612) : 320,	(26,612) : 1129,	(27,612) : 1104,	(28,612) : 1087,	(29,612) : 1102,	(30,612) : 1093,	(31,612) : 1028,	(32,612) : 1062,	(33,612) : 1182,	(34,612) : 996,	(35,612) : 1140,	(36,612) : 720,	(37,612) : 643,	(38,612) : 1040,	(39,612) : 1054,	(40,612) : 302,
(1,613) : 280,	(2,613) : 246,	(3,613) : 431,	(4,613) : 680,	(5,613) : 199,	(6,613) : 205,	(7,613) : 271,	(8,613) : 643,	(9,613) : 87,	(10,613) : 273,	(11,613) : 247,	(12,613) : 710,	(13,613) : 333,	(14,613) : 82,	(15,613) : 533,	(16,613) : 764,	(17,613) : 631,	(18,613) : 190,	(19,613) : 652,	(20,613) : 277,	(21,613) : 384,	(22,613) : 125,	(23,613) : 191,	(24,613) : 633,	(25,613) : 500,	(26,613) : 382,	(27,613) : 335,	(28,613) : 275,	(29,613) : 512,	(30,613) : 280,	(31,613) : 215,	(32,613) : 263,	(33,613) : 378,	(34,613) : 183,	(35,613) : 340,	(36,613) : 144,	(37,613) : 171,	(38,613) : 229,	(39,613) : 242,	(40,613) : 513,
(1,614) : 495,	(2,614) : 535,	(3,614) : 581,	(4,614) : 129,	(5,614) : 551,	(6,614) : 502,	(7,614) : 489,	(8,614) : 868,	(9,614) : 626,	(10,614) : 490,	(11,614) : 536,	(12,614) : 297,	(13,614) : 326,	(14,614) : 622,	(15,614) : 70,	(16,614) : 232,	(17,614) : 85,	(18,614) : 532,	(19,614) : 282,	(20,614) : 490,	(21,614) : 184,	(22,614) : 501,	(23,614) : 526,	(24,614) : 86,	(25,614) : 847,	(26,614) : 267,	(27,614) : 320,	(28,614) : 488,	(29,614) : 47,	(30,614) : 492,	(31,614) : 507,	(32,614) : 411,	(33,614) : 437,	(34,614) : 490,	(35,614) : 625,	(36,614) : 688,	(37,614) : 631,	(38,614) : 533,	(39,614) : 535,	(40,614) : 874,
(1,615) : 342,	(2,615) : 376,	(3,615) : 507,	(4,615) : 291,	(5,615) : 387,	(6,615) : 339,	(7,615) : 334,	(8,615) : 796,	(9,615) : 468,	(10,615) : 335,	(11,615) : 377,	(12,615) : 361,	(13,615) : 182,	(14,615) : 463,	(15,615) : 145,	(16,615) : 376,	(17,615) : 242,	(18,615) : 368,	(19,615) : 317,	(20,615) : 335,	(21,615) : 30,	(22,615) : 334,	(23,615) : 362,	(24,615) : 244,	(25,615) : 741,	(26,615) : 141,	(27,615) : 176,	(28,615) : 334,	(29,615) : 135,	(30,615) : 339,	(31,615) : 345,	(32,615) : 254,	(33,615) : 307,	(34,615) : 325,	(35,615) : 474,	(36,615) : 530,	(37,615) : 487,	(38,615) : 372,	(39,615) : 375,	(40,615) : 765,
(1,616) : 230,	(2,616) : 256,	(3,616) : 492,	(4,616) : 417,	(5,616) : 261,	(6,616) : 215,	(7,616) : 221,	(8,616) : 770,	(9,616) : 356,	(10,616) : 223,	(11,616) : 257,	(12,616) : 442,	(13,616) : 113,	(14,616) : 352,	(15,616) : 264,	(16,616) : 493,	(17,616) : 367,	(18,616) : 242,	(19,616) : 387,	(20,616) : 224,	(21,616) : 114,	(22,616) : 206,	(23,616) : 236,	(24,616) : 369,	(25,616) : 684,	(26,616) : 126,	(27,616) : 110,	(28,616) : 222,	(29,616) : 262,	(30,616) : 228,	(31,616) : 222,	(32,616) : 143,	(33,616) : 232,	(34,616) : 199,	(35,616) : 361,	(36,616) : 417,	(37,616) : 396,	(38,616) : 250,	(39,616) : 255,	(40,616) : 705,
(1,617) : 123,	(2,617) : 84,	(3,617) : 711,	(4,617) : 723,	(5,617) : 100,	(6,617) : 130,	(7,617) : 128,	(8,617) : 934,	(9,617) : 348,	(10,617) : 127,	(11,617) : 83,	(12,617) : 644,	(13,617) : 294,	(14,617) : 344,	(15,617) : 567,	(16,617) : 769,	(17,617) : 673,	(18,617) : 116,	(19,617) : 580,	(20,617) : 128,	(21,617) : 436,	(22,617) : 190,	(23,617) : 120,	(24,617) : 676,	(25,617) : 784,	(26,617) : 357,	(27,617) : 300,	(28,617) : 129,	(29,617) : 592,	(30,617) : 126,	(31,617) : 121,	(32,617) : 205,	(33,617) : 224,	(34,617) : 153,	(35,617) : 51,	(36,617) : 371,	(37,617) : 456,	(38,617) : 92,	(39,617) : 85,	(40,617) : 794,
(1,618) : 646,	(2,618) : 616,	(3,618) : 228,	(4,618) : 854,	(5,618) : 569,	(6,618) : 572,	(7,618) : 636,	(8,618) : 286,	(9,618) : 313,	(10,618) : 638,	(11,618) : 617,	(12,618) : 976,	(13,618) : 652,	(14,618) : 316,	(15,618) : 747,	(16,618) : 963,	(17,618) : 815,	(18,618) : 560,	(19,618) : 929,	(20,618) : 642,	(21,618) : 626,	(22,618) : 491,	(23,618) : 561,	(24,618) : 816,	(25,618) : 134,	(26,618) : 679,	(27,618) : 652,	(28,618) : 640,	(29,618) : 684,	(30,618) : 646,	(31,618) : 583,	(32,618) : 610,	(33,618) : 730,	(34,618) : 549,	(35,618) : 708,	(36,618) : 313,	(37,618) : 202,	(38,618) : 598,	(39,618) : 612,	(40,618) : 153,
(1,619) : 728,	(2,619) : 697,	(3,619) : 253,	(4,619) : 916,	(5,619) : 650,	(6,619) : 654,	(7,619) : 718,	(8,619) : 220,	(9,619) : 392,	(10,619) : 720,	(11,619) : 698,	(12,619) : 1048,	(13,619) : 732,	(14,619) : 395,	(15,619) : 816,	(16,619) : 1027,	(17,619) : 879,	(18,619) : 642,	(19,619) : 1003,	(20,619) : 724,	(21,619) : 700,	(22,619) : 573,	(23,619) : 642,	(24,619) : 879,	(25,619) : 54,	(26,619) : 757,	(27,619) : 731,	(28,619) : 722,	(29,619) : 749,	(30,619) : 728,	(31,619) : 665,	(32,619) : 691,	(33,619) : 812,	(34,619) : 631,	(35,619) : 788,	(36,619) : 386,	(37,619) : 282,	(38,619) : 680,	(39,619) : 694,	(40,619) : 79,
(1,620) : 133,	(2,620) : 187,	(3,620) : 741,	(4,620) : 540,	(5,620) : 237,	(6,620) : 205,	(7,620) : 139,	(8,620) : 1005,	(9,620) : 484,	(10,620) : 137,	(11,620) : 186,	(12,620) : 422,	(13,620) : 154,	(14,620) : 479,	(15,620) : 394,	(16,620) : 565,	(17,620) : 494,	(18,620) : 232,	(19,620) : 358,	(20,620) : 134,	(21,620) : 303,	(22,620) : 278,	(23,620) : 228,	(24,620) : 497,	(25,620) : 891,	(26,620) : 194,	(27,620) : 160,	(28,620) : 135,	(29,620) : 446,	(30,620) : 131,	(31,620) : 197,	(32,620) : 149,	(33,620) : 30,	(34,620) : 221,	(35,620) : 208,	(36,620) : 528,	(37,620) : 568,	(38,620) : 200,	(39,620) : 189,	(40,620) : 907,
(1,621) : 476,	(2,621) : 521,	(3,621) : 648,	(4,621) : 124,	(5,621) : 544,	(6,621) : 495,	(7,621) : 471,	(8,621) : 938,	(9,621) : 653,	(10,621) : 472,	(11,621) : 522,	(12,621) : 219,	(13,621) : 305,	(14,621) : 648,	(15,621) : 41,	(16,621) : 191,	(17,621) : 77,	(18,621) : 526,	(19,621) : 203,	(20,621) : 471,	(21,621) : 190,	(22,621) : 507,	(23,621) : 521,	(24,621) : 79,	(25,621) : 906,	(26,621) : 242,	(27,621) : 300,	(28,621) : 470,	(29,621) : 115,	(30,621) : 473,	(31,621) : 498,	(32,621) : 398,	(33,621) : 401,	(34,621) : 487,	(35,621) : 602,	(36,621) : 714,	(37,621) : 670,	(38,621) : 522,	(39,621) : 522,	(40,621) : 932,
(1,622) : 629,	(2,622) : 684,	(3,622) : 942,	(4,622) : 294,	(5,622) : 723,	(6,622) : 677,	(7,622) : 629,	(8,622) : 1232,	(9,622) : 897,	(10,622) : 629,	(11,622) : 684,	(12,622) : 103,	(13,622) : 488,	(14,622) : 892,	(15,622) : 328,	(16,622) : 197,	(17,622) : 301,	(18,622) : 710,	(19,622) : 165,	(20,622) : 626,	(21,622) : 447,	(22,622) : 717,	(23,622) : 705,	(24,622) : 302,	(25,622) : 1194,	(26,622) : 432,	(27,622) : 484,	(28,622) : 627,	(29,622) : 409,	(30,622) : 626,	(31,622) : 675,	(32,622) : 579,	(33,622) : 519,	(34,622) : 679,	(35,622) : 726,	(36,622) : 956,	(37,622) : 935,	(38,622) : 691,	(39,622) : 686,	(40,622) : 1219,
(1,623) : 530,	(2,623) : 568,	(3,623) : 571,	(4,623) : 123,	(5,623) : 581,	(6,623) : 534,	(7,623) : 523,	(8,623) : 856,	(9,623) : 643,	(10,623) : 524,	(11,623) : 569,	(12,623) : 325,	(13,623) : 362,	(14,623) : 640,	(15,623) : 110,	(16,623) : 236,	(17,623) : 90,	(18,623) : 563,	(19,623) : 315,	(20,623) : 524,	(21,623) : 215,	(22,623) : 528,	(23,623) : 557,	(24,623) : 90,	(25,623) : 844,	(26,623) : 304,	(27,623) : 355,	(28,623) : 523,	(29,623) : 61,	(30,623) : 527,	(31,623) : 539,	(32,623) : 445,	(33,623) : 475,	(34,623) : 520,	(35,623) : 661,	(36,623) : 706,	(37,623) : 641,	(38,623) : 565,	(39,623) : 568,	(40,623) : 871,
(1,624) : 128,	(2,624) : 174,	(3,624) : 766,	(4,624) : 586,	(5,624) : 227,	(6,624) : 204,	(7,624) : 137,	(8,624) : 1024,	(9,624) : 485,	(10,624) : 134,	(11,624) : 174,	(12,624) : 461,	(13,624) : 194,	(14,624) : 481,	(15,624) : 440,	(16,624) : 610,	(17,624) : 540,	(18,624) : 226,	(19,624) : 398,	(20,624) : 131,	(21,624) : 346,	(22,624) : 282,	(23,624) : 223,	(24,624) : 543,	(25,624) : 903,	(26,624) : 239,	(27,624) : 200,	(28,624) : 133,	(29,624) : 492,	(30,624) : 127,	(31,624) : 194,	(32,624) : 170,	(33,624) : 68,	(34,624) : 224,	(35,624) : 172,	(36,624) : 526,	(37,624) : 576,	(38,624) : 190,	(39,624) : 178,	(40,624) : 918,
(1,625) : 361,	(2,625) : 321,	(3,625) : 410,	(4,625) : 751,	(5,625) : 271,	(6,625) : 285,	(7,625) : 352,	(8,625) : 589,	(9,625) : 7,	(10,625) : 354,	(11,625) : 322,	(12,625) : 794,	(13,625) : 420,	(14,625) : 9,	(15,625) : 609,	(16,625) : 840,	(17,625) : 704,	(18,625) : 266,	(19,625) : 737,	(20,625) : 358,	(21,625) : 462,	(22,625) : 208,	(23,625) : 267,	(24,625) : 705,	(25,625) : 432,	(26,625) : 467,	(27,625) : 422,	(28,625) : 356,	(29,625) : 580,	(30,625) : 361,	(31,625) : 294,	(32,625) : 349,	(33,625) : 463,	(34,625) : 266,	(35,625) : 404,	(36,625) : 66,	(37,625) : 107,	(38,625) : 304,	(39,625) : 318,	(40,625) : 441,
(1,626) : 234,	(2,626) : 259,	(3,626) : 489,	(4,626) : 415,	(5,626) : 264,	(6,626) : 218,	(7,626) : 224,	(8,626) : 767,	(9,626) : 357,	(10,626) : 227,	(11,626) : 260,	(12,626) : 442,	(13,626) : 117,	(14,626) : 352,	(15,626) : 263,	(16,626) : 492,	(17,626) : 365,	(18,626) : 245,	(19,626) : 388,	(20,626) : 228,	(21,626) : 113,	(22,626) : 208,	(23,626) : 239,	(24,626) : 368,	(25,626) : 682,	(26,626) : 128,	(27,626) : 114,	(28,626) : 226,	(29,626) : 260,	(30,626) : 231,	(31,626) : 225,	(32,626) : 146,	(33,626) : 236,	(34,626) : 202,	(35,626) : 364,	(36,626) : 417,	(37,626) : 395,	(38,626) : 253,	(39,626) : 258,	(40,626) : 703,
(1,627) : 135,	(2,627) : 116,	(3,627) : 758,	(4,627) : 728,	(5,627) : 146,	(6,627) : 167,	(7,627) : 144,	(8,627) : 985,	(9,627) : 401,	(10,627) : 142,	(11,627) : 114,	(12,627) : 631,	(13,627) : 302,	(14,627) : 397,	(15,627) : 575,	(16,627) : 767,	(17,627) : 680,	(18,627) : 160,	(19,627) : 567,	(20,627) : 141,	(21,627) : 451,	(22,627) : 236,	(23,627) : 163,	(24,627) : 682,	(25,627) : 837,	(26,627) : 363,	(27,627) : 309,	(28,627) : 143,	(29,627) : 607,	(30,627) : 138,	(31,627) : 156,	(32,627) : 223,	(33,627) : 213,	(34,627) : 192,	(35,627) : 3,	(36,627) : 424,	(37,627) : 508,	(38,627) : 129,	(39,627) : 118,	(40,627) : 847,
(1,628) : 532,	(2,628) : 572,	(3,628) : 589,	(4,628) : 106,	(5,628) : 586,	(6,628) : 538,	(7,628) : 525,	(8,628) : 875,	(9,628) : 655,	(10,628) : 527,	(11,628) : 573,	(12,628) : 308,	(13,628) : 362,	(14,628) : 651,	(15,628) : 102,	(16,628) : 218,	(17,628) : 71,	(18,628) : 568,	(19,628) : 300,	(20,628) : 526,	(21,628) : 220,	(22,628) : 536,	(23,628) : 562,	(24,628) : 71,	(25,628) : 861,	(26,628) : 304,	(27,628) : 356,	(28,628) : 525,	(29,628) : 72,	(30,628) : 529,	(31,628) : 543,	(32,628) : 448,	(33,628) : 473,	(34,628) : 526,	(35,628) : 662,	(36,628) : 718,	(37,628) : 656,	(38,628) : 569,	(39,628) : 572,	(40,628) : 889,
(1,629) : 546,	(2,629) : 587,	(3,629) : 619,	(4,629) : 76,	(5,629) : 604,	(6,629) : 556,	(7,629) : 540,	(8,629) : 904,	(9,629) : 682,	(10,629) : 541,	(11,629) : 588,	(12,629) : 287,	(13,629) : 375,	(14,629) : 678,	(15,629) : 105,	(16,629) : 187,	(17,629) : 42,	(18,629) : 586,	(19,629) : 284,	(20,629) : 540,	(21,629) : 238,	(22,629) : 557,	(23,629) : 580,	(24,629) : 42,	(25,629) : 892,	(26,629) : 315,	(27,629) : 369,	(28,629) : 539,	(29,629) : 99,	(30,629) : 543,	(31,629) : 560,	(32,629) : 463,	(33,629) : 481,	(34,629) : 544,	(35,629) : 675,	(36,629) : 744,	(37,629) : 684,	(38,629) : 586,	(39,629) : 587,	(40,629) : 919,
(1,630) : 674,	(2,630) : 727,	(3,630) : 927,	(4,630) : 251,	(5,630) : 763,	(6,630) : 715,	(7,630) : 672,	(8,630) : 1214,	(9,630) : 918,	(10,630) : 672,	(11,630) : 728,	(12,630) : 150,	(13,630) : 522,	(14,630) : 913,	(15,630) : 322,	(16,630) : 139,	(17,630) : 270,	(18,630) : 748,	(19,630) : 213,	(20,630) : 670,	(21,630) : 457,	(22,630) : 748,	(23,630) : 743,	(24,630) : 270,	(25,630) : 1189,	(26,630) : 461,	(27,630) : 517,	(28,630) : 670,	(29,630) : 392,	(30,630) : 671,	(31,630) : 715,	(32,630) : 616,	(33,630) : 569,	(34,630) : 714,	(35,630) : 778,	(36,630) : 978,	(37,630) : 947,	(38,630) : 733,	(39,630) : 729,	(40,630) : 1215,
(1,631) : 428,	(2,631) : 390,	(3,631) : 367,	(4,631) : 781,	(5,631) : 340,	(6,631) : 352,	(7,631) : 419,	(8,631) : 525,	(9,631) : 74,	(10,631) : 421,	(11,631) : 391,	(12,631) : 842,	(13,631) : 476,	(14,631) : 78,	(15,631) : 645,	(16,631) : 875,	(17,631) : 735,	(18,631) : 334,	(19,631) : 786,	(20,631) : 425,	(21,631) : 501,	(22,631) : 274,	(23,631) : 336,	(24,631) : 736,	(25,631) : 363,	(26,631) : 519,	(27,631) : 477,	(28,631) : 423,	(29,631) : 607,	(30,631) : 428,	(31,631) : 362,	(32,631) : 411,	(33,631) : 527,	(34,631) : 332,	(35,631) : 473,	(36,631) : 82,	(37,631) : 50,	(38,631) : 373,	(39,631) : 386,	(40,631) : 372,
(1,632) : 576,	(2,632) : 621,	(3,632) : 683,	(4,632) : 24,	(5,632) : 642,	(6,632) : 593,	(7,632) : 571,	(8,632) : 968,	(9,632) : 736,	(10,632) : 572,	(11,632) : 622,	(12,632) : 245,	(13,632) : 405,	(14,632) : 732,	(15,632) : 134,	(16,632) : 123,	(17,632) : 29,	(18,632) : 624,	(19,632) : 255,	(20,632) : 571,	(21,632) : 281,	(22,632) : 601,	(23,632) : 619,	(24,632) : 27,	(25,632) : 956,	(26,632) : 342,	(27,632) : 399,	(28,632) : 570,	(29,632) : 159,	(30,632) : 573,	(31,632) : 596,	(32,632) : 497,	(33,632) : 501,	(34,632) : 584,	(35,632) : 702,	(36,632) : 798,	(37,632) : 743,	(38,632) : 621,	(39,632) : 621,	(40,632) : 983,
(1,633) : 108,	(2,633) : 52,	(3,633) : 640,	(4,633) : 687,	(5,633) : 30,	(6,633) : 73,	(7,633) : 106,	(8,633) : 865,	(9,633) : 284,	(10,633) : 107,	(11,633) : 52,	(12,633) : 634,	(13,633) : 262,	(14,633) : 280,	(15,633) : 531,	(16,633) : 742,	(17,633) : 637,	(18,633) : 49,	(19,633) : 570,	(20,633) : 110,	(21,633) : 391,	(22,633) : 120,	(23,633) : 53,	(24,633) : 640,	(25,633) : 719,	(26,633) : 326,	(27,633) : 268,	(28,633) : 109,	(29,633) : 547,	(30,633) : 110,	(31,633) : 68,	(32,633) : 168,	(33,633) : 225,	(34,633) : 91,	(35,633) : 118,	(36,633) : 314,	(37,633) : 389,	(38,633) : 45,	(39,633) : 50,	(40,633) : 730,
(1,634) : 310,	(2,634) : 298,	(3,634) : 350,	(4,634) : 569,	(5,634) : 266,	(6,634) : 247,	(7,634) : 298,	(8,634) : 602,	(9,634) : 189,	(10,634) : 301,	(11,634) : 299,	(12,634) : 634,	(13,634) : 290,	(14,634) : 185,	(15,634) : 431,	(16,634) : 662,	(17,634) : 522,	(18,634) : 250,	(19,634) : 581,	(20,634) : 304,	(21,634) : 289,	(22,634) : 175,	(23,634) : 248,	(24,634) : 524,	(25,634) : 495,	(26,634) : 320,	(27,634) : 290,	(28,634) : 302,	(29,634) : 396,	(30,634) : 308,	(31,634) : 258,	(32,634) : 255,	(33,634) : 375,	(34,634) : 221,	(35,634) : 407,	(36,634) : 251,	(37,634) : 202,	(38,634) : 282,	(39,634) : 295,	(40,634) : 515,
(1,635) : 105,	(2,635) : 51,	(3,635) : 655,	(4,635) : 692,	(5,635) : 44,	(6,635) : 82,	(7,635) : 105,	(8,635) : 880,	(9,635) : 298,	(10,635) : 105,	(11,635) : 50,	(12,635) : 633,	(13,635) : 266,	(14,635) : 294,	(15,635) : 536,	(16,635) : 745,	(17,635) : 643,	(18,635) : 62,	(19,635) : 570,	(20,635) : 108,	(21,635) : 398,	(22,635) : 134,	(23,635) : 66,	(24,635) : 645,	(25,635) : 734,	(26,635) : 330,	(27,635) : 271,	(28,635) : 108,	(29,635) : 554,	(30,635) : 108,	(31,635) : 75,	(32,635) : 172,	(33,635) : 221,	(34,635) : 102,	(35,635) : 103,	(36,635) : 327,	(37,635) : 404,	(38,635) : 48,	(39,635) : 49,	(40,635) : 744,
(1,636) : 238,	(2,636) : 224,	(3,636) : 423,	(4,636) : 573,	(5,636) : 194,	(6,636) : 173,	(7,636) : 227,	(8,636) : 671,	(9,636) : 189,	(10,636) : 229,	(11,636) : 226,	(12,636) : 607,	(13,636) : 241,	(14,636) : 185,	(15,636) : 426,	(16,636) : 656,	(17,636) : 524,	(18,636) : 178,	(19,636) : 550,	(20,636) : 232,	(21,636) : 276,	(22,636) : 104,	(23,636) : 175,	(24,636) : 526,	(25,636) : 554,	(26,636) : 282,	(27,636) : 242,	(28,636) : 230,	(29,636) : 406,	(30,636) : 237,	(31,636) : 185,	(32,636) : 190,	(33,636) : 310,	(34,636) : 148,	(35,636) : 335,	(36,636) : 250,	(37,636) : 240,	(38,636) : 209,	(39,636) : 221,	(40,636) : 571,
(1,637) : 57,	(2,637) : 2,	(3,637) : 645,	(4,637) : 644,	(5,637) : 51,	(6,637) : 50,	(7,637) : 55,	(8,637) : 882,	(9,637) : 316,	(10,637) : 56,	(11,637) : 3,	(12,637) : 584,	(13,637) : 216,	(14,637) : 311,	(15,637) : 488,	(16,637) : 696,	(17,637) : 594,	(18,637) : 54,	(19,637) : 520,	(20,637) : 58,	(21,637) : 353,	(22,637) : 126,	(23,637) : 53,	(24,637) : 597,	(25,637) : 745,	(26,637) : 281,	(27,637) : 222,	(28,637) : 58,	(29,637) : 509,	(30,637) : 59,	(31,637) : 39,	(32,637) : 124,	(33,637) : 174,	(34,637) : 75,	(35,637) : 114,	(36,637) : 352,	(37,637) : 415,	(38,637) : 16,	(39,637) : 2,	(40,637) : 757,
(1,638) : 733,	(2,638) : 715,	(3,638) : 105,	(4,638) : 796,	(5,638) : 675,	(6,638) : 666,	(7,638) : 722,	(8,638) : 188,	(9,638) : 452,	(10,638) : 725,	(11,638) : 716,	(12,638) : 962,	(13,638) : 695,	(14,638) : 454,	(15,638) : 719,	(16,638) : 912,	(17,638) : 765,	(18,638) : 663,	(19,638) : 924,	(20,638) : 728,	(21,638) : 627,	(22,638) : 588,	(23,638) : 661,	(24,638) : 765,	(25,638) : 209,	(26,638) : 705,	(27,638) : 693,	(28,638) : 726,	(29,638) : 641,	(30,638) : 732,	(31,638) : 677,	(32,638) : 677,	(33,638) : 796,	(34,638) : 641,	(35,638) : 819,	(36,638) : 470,	(37,638) : 341,	(38,638) : 698,	(39,638) : 712,	(40,638) : 239,
(1,639) : 429,	(2,639) : 393,	(3,639) : 348,	(4,639) : 766,	(5,639) : 344,	(6,639) : 353,	(7,639) : 420,	(8,639) : 513,	(9,639) : 83,	(10,639) : 422,	(11,639) : 394,	(12,639) : 832,	(13,639) : 470,	(14,639) : 86,	(15,639) : 632,	(16,639) : 862,	(17,639) : 720,	(18,639) : 337,	(19,639) : 777,	(20,639) : 426,	(21,639) : 490,	(22,639) : 274,	(23,639) : 338,	(24,639) : 722,	(25,639) : 356,	(26,639) : 510,	(27,639) : 471,	(28,639) : 423,	(29,639) : 592,	(30,639) : 429,	(31,639) : 363,	(32,639) : 408,	(33,639) : 525,	(34,639) : 332,	(35,639) : 479,	(36,639) : 101,	(37,639) : 33,	(38,639) : 376,	(39,639) : 389,	(40,639) : 367,
(1,640) : 48,	(2,640) : 17,	(3,640) : 635,	(4,640) : 628,	(5,640) : 56,	(6,640) : 39,	(7,640) : 44,	(8,640) : 875,	(9,640) : 314,	(10,640) : 45,	(11,640) : 18,	(12,640) : 571,	(13,640) : 201,	(14,640) : 310,	(15,640) : 472,	(16,640) : 681,	(17,640) : 578,	(18,640) : 52,	(19,640) : 507,	(20,640) : 48,	(21,640) : 336,	(22,640) : 118,	(23,640) : 50,	(24,640) : 581,	(25,640) : 740,	(26,640) : 265,	(27,640) : 207,	(28,640) : 47,	(29,640) : 493,	(30,640) : 50,	(31,640) : 27,	(32,640) : 108,	(33,640) : 165,	(34,640) : 64,	(35,640) : 128,	(36,640) : 353,	(37,640) : 411,	(38,640) : 19,	(39,640) : 17,	(40,640) : 753,
(1,641) : 473,	(2,641) : 439,	(3,641) : 303,	(4,641) : 770,	(5,641) : 391,	(6,641) : 397,	(7,641) : 463,	(8,641) : 460,	(9,641) : 135,	(10,641) : 465,	(11,641) : 440,	(12,641) : 852,	(13,641) : 500,	(14,641) : 138,	(15,641) : 642,	(16,641) : 870,	(17,641) : 726,	(18,641) : 384,	(19,641) : 799,	(20,641) : 469,	(21,641) : 505,	(22,641) : 317,	(23,641) : 384,	(24,641) : 727,	(25,641) : 307,	(26,641) : 536,	(27,641) : 500,	(28,641) : 467,	(29,641) : 596,	(30,641) : 472,	(31,641) : 408,	(32,641) : 445,	(33,641) : 564,	(34,641) : 375,	(35,641) : 529,	(36,641) : 149,	(37,641) : 23,	(38,641) : 422,	(39,641) : 436,	(40,641) : 320,
(1,642) : 65,	(2,642) : 74,	(3,642) : 594,	(4,642) : 577,	(5,642) : 89,	(6,642) : 40,	(7,642) : 54,	(8,642) : 843,	(9,642) : 306,	(10,642) : 57,	(11,642) : 75,	(12,642) : 535,	(13,642) : 155,	(14,642) : 301,	(15,642) : 420,	(16,642) : 634,	(17,642) : 527,	(18,642) : 73,	(19,642) : 472,	(20,642) : 60,	(21,642) : 282,	(22,642) : 100,	(23,642) : 68,	(24,642) : 530,	(25,642) : 718,	(26,642) : 218,	(27,642) : 160,	(28,642) : 58,	(29,642) : 438,	(30,642) : 64,	(31,642) : 42,	(32,642) : 60,	(33,642) : 155,	(34,642) : 45,	(35,642) : 183,	(36,642) : 352,	(37,642) : 392,	(38,642) : 68,	(39,642) : 73,	(40,642) : 733,
(1,643) : 294,	(2,643) : 312,	(3,643) : 425,	(4,643) : 409,	(5,643) : 307,	(6,643) : 266,	(7,643) : 283,	(8,643) : 707,	(9,643) : 346,	(10,643) : 286,	(11,643) : 313,	(12,643) : 475,	(13,643) : 184,	(14,643) : 342,	(15,643) : 268,	(16,643) : 498,	(17,643) : 361,	(18,643) : 288,	(19,643) : 426,	(20,643) : 287,	(21,643) : 126,	(22,643) : 237,	(23,643) : 283,	(24,643) : 363,	(25,643) : 632,	(26,643) : 186,	(27,643) : 181,	(28,643) : 285,	(29,643) : 241,	(30,643) : 291,	(31,643) : 274,	(32,643) : 209,	(33,643) : 304,	(34,643) : 245,	(35,643) : 421,	(36,643) : 408,	(37,643) : 364,	(38,643) : 303,	(39,643) : 310,	(40,643) : 655,
(1,644) : 600,	(2,644) : 645,	(3,644) : 693,	(4,644) : 2,	(5,644) : 666,	(6,644) : 617,	(7,644) : 595,	(8,644) : 976,	(9,644) : 756,	(10,644) : 596,	(11,644) : 646,	(12,644) : 257,	(13,644) : 429,	(14,644) : 752,	(15,644) : 157,	(16,644) : 115,	(17,644) : 51,	(18,644) : 648,	(19,644) : 272,	(20,644) : 595,	(21,644) : 304,	(22,644) : 624,	(23,644) : 642,	(24,644) : 49,	(25,644) : 968,	(26,644) : 366,	(27,644) : 423,	(28,644) : 594,	(29,644) : 176,	(30,644) : 597,	(31,644) : 620,	(32,644) : 521,	(33,644) : 525,	(34,644) : 607,	(35,644) : 726,	(36,644) : 818,	(37,644) : 761,	(38,644) : 645,	(39,644) : 645,	(40,644) : 996,
(1,645) : 516,	(2,645) : 555,	(3,645) : 581,	(4,645) : 119,	(5,645) : 570,	(6,645) : 522,	(7,645) : 509,	(8,645) : 867,	(9,645) : 640,	(10,645) : 510,	(11,645) : 556,	(12,645) : 307,	(13,645) : 346,	(14,645) : 636,	(15,645) : 90,	(16,645) : 228,	(17,645) : 79,	(18,645) : 551,	(19,645) : 296,	(20,645) : 510,	(21,645) : 203,	(22,645) : 519,	(23,645) : 546,	(24,645) : 80,	(25,645) : 851,	(26,645) : 288,	(27,645) : 340,	(28,645) : 509,	(29,645) : 56,	(30,645) : 513,	(31,645) : 526,	(32,645) : 431,	(33,645) : 458,	(34,645) : 509,	(35,645) : 646,	(36,645) : 702,	(37,645) : 641,	(38,645) : 553,	(39,645) : 555,	(40,645) : 878,
(1,646) : 227,	(2,646) : 275,	(3,646) : 619,	(4,646) : 373,	(5,646) : 303,	(6,646) : 255,	(7,646) : 223,	(8,646) : 900,	(9,646) : 470,	(10,646) : 224,	(11,646) : 276,	(12,646) : 329,	(13,646) : 60,	(14,646) : 465,	(15,646) : 219,	(16,646) : 421,	(17,646) : 324,	(18,646) : 288,	(19,646) : 269,	(20,646) : 222,	(21,646) : 122,	(22,646) : 286,	(23,646) : 282,	(24,646) : 327,	(25,646) : 817,	(26,646) : 11,	(27,646) : 55,	(28,646) : 222,	(29,646) : 263,	(30,646) : 224,	(31,646) : 256,	(32,646) : 155,	(33,646) : 161,	(34,646) : 252,	(35,646) : 353,	(36,646) : 527,	(37,646) : 522,	(38,646) : 277,	(39,646) : 276,	(40,646) : 838,
(1,647) : 310,	(2,647) : 299,	(3,647) : 350,	(4,647) : 561,	(5,647) : 269,	(6,647) : 248,	(7,647) : 298,	(8,647) : 604,	(9,647) : 196,	(10,647) : 301,	(11,647) : 300,	(12,647) : 628,	(13,647) : 286,	(14,647) : 193,	(15,647) : 424,	(16,647) : 654,	(17,647) : 515,	(18,647) : 252,	(19,647) : 575,	(20,647) : 304,	(21,647) : 282,	(22,647) : 178,	(23,647) : 250,	(24,647) : 516,	(25,647) : 499,	(26,647) : 315,	(27,647) : 285,	(28,647) : 302,	(29,647) : 389,	(30,647) : 308,	(31,647) : 259,	(32,647) : 253,	(33,647) : 373,	(34,647) : 222,	(35,647) : 409,	(36,647) : 258,	(37,647) : 208,	(38,647) : 284,	(39,647) : 296,	(40,647) : 519,
(1,648) : 613,	(2,648) : 661,	(3,648) : 771,	(4,648) : 87,	(5,648) : 688,	(6,648) : 639,	(7,648) : 609,	(8,648) : 1056,	(9,648) : 804,	(10,648) : 609,	(11,648) : 662,	(12,648) : 192,	(13,648) : 445,	(14,648) : 800,	(15,648) : 192,	(16,648) : 39,	(17,648) : 111,	(18,648) : 671,	(19,648) : 222,	(20,648) : 608,	(21,648) : 342,	(22,648) : 656,	(23,648) : 666,	(24,648) : 111,	(25,648) : 1041,	(26,648) : 381,	(27,648) : 440,	(28,648) : 607,	(29,648) : 241,	(30,648) : 610,	(31,648) : 641,	(32,648) : 540,	(33,648) : 525,	(34,648) : 633,	(35,648) : 732,	(36,648) : 866,	(37,648) : 819,	(38,648) : 663,	(39,648) : 662,	(40,648) : 1068,
(1,649) : 806,	(2,649) : 776,	(3,649) : 287,	(4,649) : 969,	(5,649) : 729,	(6,649) : 733,	(7,649) : 796,	(8,649) : 158,	(9,649) : 471,	(10,649) : 799,	(11,649) : 777,	(12,649) : 1113,	(13,649) : 806,	(14,649) : 474,	(15,649) : 877,	(16,649) : 1083,	(17,649) : 935,	(18,649) : 721,	(19,649) : 1070,	(20,649) : 802,	(21,649) : 767,	(22,649) : 652,	(23,649) : 721,	(24,649) : 935,	(25,649) : 42,	(26,649) : 829,	(27,649) : 805,	(28,649) : 800,	(29,649) : 806,	(30,649) : 806,	(31,649) : 743,	(32,649) : 768,	(33,649) : 889,	(34,649) : 709,	(35,649) : 867,	(36,649) : 463,	(37,649) : 361,	(38,649) : 759,	(39,649) : 773,	(40,649) : 61,
(1,650) : 904,	(2,650) : 869,	(3,650) : 424,	(4,650) : 1104,	(5,650) : 819,	(6,650) : 829,	(7,650) : 894,	(8,650) : 231,	(9,650) : 553,	(10,650) : 897,	(11,650) : 869,	(12,650) : 1242,	(13,650) : 921,	(14,650) : 557,	(15,650) : 1008,	(16,650) : 1217,	(17,650) : 1069,	(18,650) : 813,	(19,650) : 1196,	(20,650) : 900,	(21,650) : 893,	(22,650) : 748,	(23,650) : 814,	(24,650) : 1069,	(25,650) : 140,	(26,650) : 948,	(27,650) : 920,	(28,650) : 898,	(29,650) : 940,	(30,650) : 904,	(31,650) : 839,	(32,650) : 874,	(33,650) : 995,	(34,650) : 807,	(35,650) : 951,	(36,650) : 532,	(37,650) : 454,	(38,650) : 851,	(39,650) : 865,	(40,650) : 116,
(1,651) : 320,	(2,651) : 342,	(3,651) : 433,	(4,651) : 371,	(5,651) : 341,	(6,651) : 298,	(7,651) : 310,	(8,651) : 718,	(9,651) : 384,	(10,651) : 312,	(11,651) : 344,	(12,651) : 449,	(13,651) : 193,	(14,651) : 380,	(15,651) : 233,	(16,651) : 463,	(17,651) : 323,	(18,651) : 322,	(19,651) : 403,	(20,651) : 314,	(21,651) : 100,	(22,651) : 274,	(23,651) : 317,	(24,651) : 325,	(25,651) : 654,	(26,651) : 181,	(27,651) : 188,	(28,651) : 312,	(29,651) : 202,	(30,651) : 317,	(31,651) : 306,	(32,651) : 233,	(33,651) : 318,	(34,651) : 279,	(35,651) : 450,	(36,651) : 446,	(37,651) : 397,	(38,651) : 334,	(39,651) : 341,	(40,651) : 677,
(1,652) : 1126,	(2,652) : 1088,	(3,652) : 640,	(4,652) : 1329,	(5,652) : 1037,	(6,652) : 1051,	(7,652) : 1117,	(8,652) : 388,	(9,652) : 771,	(10,652) : 1119,	(11,652) : 1089,	(12,652) : 1472,	(13,652) : 1150,	(14,652) : 775,	(15,652) : 1237,	(16,652) : 1442,	(17,652) : 1295,	(18,652) : 1033,	(19,652) : 1427,	(20,652) : 1123,	(21,652) : 1124,	(22,652) : 971,	(23,652) : 1034,	(24,652) : 1295,	(25,652) : 370,	(26,652) : 1179,	(27,652) : 1150,	(28,652) : 1121,	(29,652) : 1166,	(30,652) : 1127,	(31,652) : 1061,	(32,652) : 1101,	(33,652) : 1221,	(34,652) : 1030,	(35,652) : 1164,	(36,652) : 742,	(37,652) : 678,	(38,652) : 1071,	(39,652) : 1085,	(40,652) : 347,
(1,653) : 106,	(2,653) : 51,	(3,653) : 654,	(4,653) : 692,	(5,653) : 43,	(6,653) : 81,	(7,653) : 106,	(8,653) : 879,	(9,653) : 297,	(10,653) : 106,	(11,653) : 50,	(12,653) : 634,	(13,653) : 266,	(14,653) : 293,	(15,653) : 536,	(16,653) : 746,	(17,653) : 642,	(18,653) : 61,	(19,653) : 570,	(20,653) : 108,	(21,653) : 398,	(22,653) : 134,	(23,653) : 65,	(24,653) : 645,	(25,653) : 733,	(26,653) : 330,	(27,653) : 271,	(28,653) : 108,	(29,653) : 554,	(30,653) : 108,	(31,653) : 75,	(32,653) : 172,	(33,653) : 221,	(34,653) : 101,	(35,653) : 104,	(36,653) : 326,	(37,653) : 403,	(38,653) : 48,	(39,653) : 49,	(40,653) : 743,
(1,654) : 948,	(2,654) : 915,	(3,654) : 423,	(4,654) : 1113,	(5,654) : 866,	(6,654) : 873,	(7,654) : 937,	(8,654) : 189,	(9,654) : 602,	(10,654) : 940,	(11,654) : 916,	(12,654) : 1262,	(13,654) : 954,	(14,654) : 606,	(15,654) : 1024,	(16,654) : 1227,	(17,654) : 1079,	(18,654) : 859,	(19,654) : 1219,	(20,654) : 944,	(21,654) : 916,	(22,654) : 792,	(23,654) : 860,	(24,654) : 1080,	(25,654) : 170,	(26,654) : 978,	(27,654) : 953,	(28,654) : 941,	(29,654) : 952,	(30,654) : 947,	(31,654) : 883,	(32,654) : 913,	(33,654) : 1034,	(34,654) : 850,	(35,654) : 1001,	(36,654) : 586,	(37,654) : 498,	(38,654) : 897,	(39,654) : 911,	(40,654) : 156,
(1,655) : 660,	(2,655) : 710,	(3,655) : 828,	(4,655) : 139,	(5,655) : 739,	(6,655) : 690,	(7,655) : 657,	(8,655) : 1112,	(9,655) : 862,	(10,655) : 657,	(11,655) : 711,	(12,655) : 195,	(13,655) : 495,	(14,655) : 858,	(15,655) : 250,	(16,655) : 22,	(17,655) : 170,	(18,655) : 722,	(19,655) : 240,	(20,655) : 655,	(21,655) : 399,	(22,655) : 710,	(23,655) : 717,	(24,655) : 169,	(25,655) : 1100,	(26,655) : 431,	(27,655) : 490,	(28,655) : 655,	(29,655) : 301,	(30,655) : 657,	(31,655) : 691,	(32,655) : 590,	(33,655) : 567,	(34,655) : 685,	(35,655) : 776,	(36,655) : 924,	(37,655) : 878,	(38,655) : 713,	(39,655) : 711,	(40,655) : 1127,
(1,656) : 591,	(2,656) : 633,	(3,656) : 645,	(4,656) : 48,	(5,656) : 650,	(6,656) : 602,	(7,656) : 585,	(8,656) : 927,	(9,656) : 725,	(10,656) : 586,	(11,656) : 634,	(12,656) : 296,	(13,656) : 420,	(14,656) : 721,	(15,656) : 148,	(16,656) : 165,	(17,656) : 54,	(18,656) : 632,	(19,656) : 303,	(20,656) : 586,	(21,656) : 284,	(22,656) : 602,	(23,656) : 626,	(24,656) : 51,	(25,656) : 923,	(26,656) : 359,	(27,656) : 414,	(28,656) : 585,	(29,656) : 141,	(30,656) : 588,	(31,656) : 606,	(32,656) : 509,	(33,656) : 524,	(34,656) : 590,	(35,656) : 720,	(36,656) : 787,	(37,656) : 725,	(38,656) : 631,	(39,656) : 633,	(40,656) : 951,
(1,657) : 212,	(2,657) : 229,	(3,657) : 475,	(4,657) : 462,	(5,657) : 226,	(6,657) : 183,	(7,657) : 202,	(8,657) : 746,	(9,657) : 311,	(10,657) : 204,	(11,657) : 230,	(12,657) : 486,	(13,657) : 135,	(14,657) : 306,	(15,657) : 310,	(16,657) : 539,	(17,657) : 412,	(18,657) : 207,	(19,657) : 430,	(20,657) : 206,	(21,657) : 160,	(22,657) : 163,	(23,657) : 202,	(24,657) : 414,	(25,657) : 651,	(26,657) : 164,	(27,657) : 134,	(28,657) : 204,	(29,657) : 304,	(30,657) : 210,	(31,657) : 192,	(32,657) : 130,	(33,657) : 238,	(34,657) : 164,	(35,657) : 338,	(36,657) : 371,	(37,657) : 354,	(38,657) : 220,	(39,657) : 228,	(40,657) : 671,
(1,658) : 200,	(2,658) : 188,	(3,658) : 461,	(4,658) : 567,	(5,658) : 162,	(6,658) : 137,	(7,658) : 188,	(8,658) : 709,	(9,658) : 210,	(10,658) : 191,	(11,658) : 190,	(12,658) : 586,	(13,658) : 211,	(14,658) : 206,	(15,658) : 416,	(16,658) : 645,	(17,658) : 517,	(18,658) : 145,	(19,658) : 527,	(20,658) : 194,	(21,658) : 266,	(22,658) : 76,	(23,658) : 141,	(24,658) : 520,	(25,658) : 590,	(26,658) : 257,	(27,658) : 213,	(28,658) : 192,	(29,658) : 405,	(30,658) : 198,	(31,658) : 148,	(32,658) : 153,	(33,658) : 273,	(34,658) : 111,	(35,658) : 300,	(36,658) : 268,	(37,658) : 273,	(38,658) : 174,	(39,658) : 186,	(40,658) : 607,
(1,659) : 237,	(2,659) : 224,	(3,659) : 423,	(4,659) : 572,	(5,659) : 194,	(6,659) : 173,	(7,659) : 226,	(8,659) : 671,	(9,659) : 190,	(10,659) : 229,	(11,659) : 225,	(12,659) : 606,	(13,659) : 240,	(14,659) : 185,	(15,659) : 425,	(16,659) : 656,	(17,659) : 523,	(18,659) : 178,	(19,659) : 550,	(20,659) : 232,	(21,659) : 276,	(22,659) : 104,	(23,659) : 175,	(24,659) : 525,	(25,659) : 554,	(26,659) : 281,	(27,659) : 241,	(28,659) : 230,	(29,659) : 405,	(30,659) : 236,	(31,659) : 185,	(32,659) : 189,	(33,659) : 310,	(34,659) : 147,	(35,659) : 334,	(36,659) : 251,	(37,659) : 241,	(38,659) : 209,	(39,659) : 221,	(40,659) : 572,
(1,660) : 576,	(2,660) : 620,	(3,660) : 677,	(4,660) : 24,	(5,660) : 641,	(6,660) : 592,	(7,660) : 571,	(8,660) : 962,	(9,660) : 733,	(10,660) : 572,	(11,660) : 621,	(12,660) : 251,	(13,660) : 405,	(14,660) : 728,	(15,660) : 133,	(16,660) : 129,	(17,660) : 26,	(18,660) : 623,	(19,660) : 260,	(20,660) : 571,	(21,660) : 279,	(22,660) : 599,	(23,660) : 618,	(24,660) : 24,	(25,660) : 950,	(26,660) : 342,	(27,660) : 399,	(28,660) : 570,	(29,660) : 154,	(30,660) : 573,	(31,660) : 596,	(32,660) : 497,	(33,660) : 502,	(34,660) : 583,	(35,660) : 702,	(36,660) : 795,	(37,660) : 739,	(38,660) : 620,	(39,660) : 621,	(40,660) : 977,
(1,661) : 231,	(2,661) : 278,	(3,661) : 612,	(4,661) : 369,	(5,661) : 305,	(6,661) : 256,	(7,661) : 226,	(8,661) : 894,	(9,661) : 467,	(10,661) : 227,	(11,661) : 279,	(12,661) : 330,	(13,661) : 62,	(14,661) : 463,	(15,661) : 214,	(16,661) : 419,	(17,661) : 320,	(18,661) : 289,	(19,661) : 271,	(20,661) : 226,	(21,661) : 115,	(22,661) : 285,	(23,661) : 284,	(24,661) : 323,	(25,661) : 812,	(26,661) : 4,	(27,661) : 56,	(28,661) : 225,	(29,661) : 257,	(30,661) : 228,	(31,661) : 258,	(32,661) : 157,	(33,661) : 168,	(34,661) : 253,	(35,661) : 357,	(36,661) : 525,	(37,661) : 518,	(38,661) : 279,	(39,661) : 279,	(40,661) : 833,
(1,662) : 230,	(2,662) : 282,	(3,662) : 675,	(4,662) : 387,	(5,662) : 318,	(6,662) : 272,	(7,662) : 228,	(8,662) : 956,	(9,662) : 509,	(10,662) : 228,	(11,662) : 283,	(12,662) : 303,	(13,662) : 90,	(14,662) : 505,	(15,662) : 241,	(16,662) : 420,	(17,662) : 341,	(18,662) : 305,	(19,662) : 240,	(20,662) : 226,	(21,662) : 172,	(22,662) : 316,	(23,662) : 300,	(24,662) : 344,	(25,662) : 869,	(26,662) : 66,	(27,662) : 88,	(28,662) : 226,	(29,662) : 299,	(30,662) : 227,	(31,662) : 271,	(32,662) : 174,	(33,662) : 138,	(34,662) : 274,	(35,662) : 344,	(36,662) : 565,	(37,662) : 569,	(38,662) : 288,	(39,662) : 284,	(40,662) : 889,
(1,663) : 307,	(2,663) : 343,	(3,663) : 529,	(4,663) : 312,	(5,663) : 357,	(6,663) : 309,	(7,663) : 299,	(8,663) : 816,	(9,663) : 457,	(10,663) : 301,	(11,663) : 344,	(12,663) : 353,	(13,663) : 145,	(14,663) : 453,	(15,663) : 159,	(16,663) : 388,	(17,663) : 262,	(18,663) : 339,	(19,663) : 305,	(20,663) : 301,	(21,663) : 10,	(22,663) : 311,	(23,663) : 333,	(24,663) : 264,	(25,663) : 752,	(26,663) : 104,	(27,663) : 138,	(28,663) : 299,	(29,663) : 167,	(30,663) : 304,	(31,663) : 314,	(32,663) : 220,	(33,663) : 269,	(34,663) : 297,	(35,663) : 438,	(36,663) : 519,	(37,663) : 485,	(38,663) : 340,	(39,663) : 343,	(40,663) : 775,
(1,664) : 129,	(2,664) : 181,	(3,664) : 749,	(4,664) : 556,	(5,664) : 232,	(6,664) : 203,	(7,664) : 136,	(8,664) : 1011,	(9,664) : 483,	(10,664) : 134,	(11,664) : 180,	(12,664) : 435,	(13,664) : 168,	(14,664) : 479,	(15,664) : 410,	(16,664) : 580,	(17,664) : 510,	(18,664) : 228,	(19,664) : 372,	(20,664) : 130,	(21,664) : 317,	(22,664) : 278,	(23,664) : 225,	(24,664) : 513,	(25,664) : 894,	(26,664) : 209,	(27,664) : 173,	(28,664) : 132,	(29,664) : 462,	(30,664) : 128,	(31,664) : 195,	(32,664) : 155,	(33,664) : 41,	(34,664) : 220,	(35,664) : 195,	(36,664) : 527,	(37,664) : 570,	(38,664) : 195,	(39,664) : 184,	(40,664) : 910,
(1,665) : 303,	(2,665) : 287,	(3,665) : 360,	(4,665) : 585,	(5,665) : 254,	(6,665) : 237,	(7,665) : 291,	(8,665) : 606,	(9,665) : 171,	(10,665) : 294,	(11,665) : 289,	(12,665) : 644,	(13,665) : 294,	(14,665) : 168,	(15,665) : 446,	(16,665) : 676,	(17,665) : 538,	(18,665) : 238,	(19,665) : 591,	(20,665) : 297,	(21,665) : 301,	(22,665) : 163,	(23,665) : 236,	(24,665) : 539,	(25,665) : 493,	(26,665) : 327,	(27,665) : 294,	(28,665) : 295,	(29,665) : 413,	(30,665) : 301,	(31,665) : 248,	(32,665) : 252,	(33,665) : 373,	(34,665) : 211,	(35,665) : 396,	(36,665) : 233,	(37,665) : 192,	(38,665) : 271,	(39,665) : 284,	(40,665) : 512,
(1,666) : 759,	(2,666) : 726,	(3,666) : 295,	(4,666) : 959,	(5,666) : 678,	(6,666) : 684,	(7,666) : 749,	(8,666) : 221,	(9,666) : 416,	(10,666) : 751,	(11,666) : 727,	(12,666) : 1090,	(13,666) : 769,	(14,666) : 420,	(15,666) : 858,	(16,666) : 1070,	(17,666) : 922,	(18,666) : 671,	(19,666) : 1044,	(20,666) : 755,	(21,666) : 741,	(22,666) : 604,	(23,666) : 671,	(24,666) : 923,	(25,666) : 23,	(26,666) : 796,	(27,666) : 769,	(28,666) : 753,	(29,666) : 792,	(30,666) : 759,	(31,666) : 695,	(32,666) : 725,	(33,666) : 846,	(34,666) : 662,	(35,666) : 814,	(36,666) : 404,	(37,666) : 310,	(38,666) : 709,	(39,666) : 723,	(40,666) : 36,
(1,667) : 15,	(2,667) : 62,	(3,667) : 647,	(4,667) : 588,	(5,667) : 105,	(6,667) : 71,	(7,667) : 7,	(8,667) : 896,	(9,667) : 353,	(10,667) : 8,	(11,667) : 62,	(12,667) : 521,	(13,667) : 159,	(14,667) : 348,	(15,667) : 432,	(16,667) : 635,	(17,667) : 538,	(18,667) : 98,	(19,667) : 457,	(20,667) : 8,	(21,667) : 303,	(22,667) : 148,	(23,667) : 94,	(24,667) : 541,	(25,667) : 770,	(26,667) : 222,	(27,667) : 165,	(28,667) : 6,	(29,667) : 459,	(30,667) : 12,	(31,667) : 64,	(32,667) : 74,	(33,667) : 115,	(34,667) : 90,	(35,667) : 146,	(36,667) : 395,	(37,667) : 443,	(38,667) : 70,	(39,667) : 64,	(40,667) : 784,
(1,668) : 518,	(2,668) : 558,	(3,668) : 596,	(4,668) : 106,	(5,668) : 574,	(6,668) : 526,	(7,668) : 511,	(8,668) : 883,	(9,668) : 650,	(10,668) : 512,	(11,668) : 559,	(12,668) : 292,	(13,668) : 347,	(14,668) : 646,	(15,668) : 83,	(16,668) : 213,	(17,668) : 64,	(18,668) : 556,	(19,668) : 283,	(20,668) : 512,	(21,668) : 207,	(22,668) : 525,	(23,668) : 550,	(24,668) : 65,	(25,668) : 866,	(26,668) : 288,	(27,668) : 341,	(28,668) : 511,	(29,668) : 68,	(30,668) : 514,	(31,668) : 530,	(32,668) : 434,	(33,668) : 456,	(34,668) : 514,	(35,668) : 647,	(36,668) : 712,	(37,668) : 654,	(38,668) : 556,	(39,668) : 558,	(40,668) : 893,
(1,669) : 294,	(2,669) : 267,	(3,669) : 392,	(4,669) : 645,	(5,669) : 225,	(6,669) : 221,	(7,669) : 283,	(8,669) : 617,	(9,669) : 110,	(10,669) : 286,	(11,669) : 268,	(12,669) : 689,	(13,669) : 322,	(14,669) : 106,	(15,669) : 502,	(16,669) : 733,	(17,669) : 597,	(18,669) : 213,	(19,669) : 633,	(20,669) : 289,	(21,669) : 354,	(22,669) : 140,	(23,669) : 212,	(24,669) : 599,	(25,669) : 485,	(26,669) : 364,	(27,669) : 323,	(28,669) : 287,	(29,669) : 474,	(30,669) : 293,	(31,669) : 232,	(32,669) : 262,	(33,669) : 381,	(34,669) : 197,	(35,669) : 368,	(36,669) : 172,	(37,669) : 162,	(38,669) : 250,	(39,669) : 264,	(40,669) : 500,
(1,670) : 664,	(2,670) : 715,	(3,670) : 859,	(4,670) : 173,	(5,670) : 746,	(6,670) : 697,	(7,670) : 661,	(8,670) : 1144,	(9,670) : 880,	(10,670) : 661,	(11,670) : 716,	(12,670) : 176,	(13,670) : 502,	(14,670) : 875,	(15,670) : 270,	(16,670) : 56,	(17,670) : 199,	(18,670) : 730,	(19,670) : 227,	(20,670) : 660,	(21,670) : 416,	(22,670) : 722,	(23,670) : 725,	(24,670) : 199,	(25,670) : 1128,	(26,670) : 439,	(27,670) : 497,	(28,670) : 659,	(29,670) : 328,	(30,670) : 661,	(31,670) : 698,	(32,670) : 598,	(33,670) : 567,	(34,670) : 694,	(35,670) : 777,	(36,670) : 941,	(37,670) : 900,	(38,670) : 719,	(39,670) : 716,	(40,670) : 1155,
(1,671) : 236,	(2,671) : 227,	(3,671) : 423,	(4,671) : 554,	(5,671) : 201,	(6,671) : 176,	(7,671) : 225,	(8,671) : 676,	(9,671) : 207,	(10,671) : 227,	(11,671) : 228,	(12,671) : 590,	(13,671) : 227,	(14,671) : 203,	(15,671) : 407,	(16,671) : 638,	(17,671) : 505,	(18,671) : 183,	(19,671) : 534,	(20,671) : 230,	(21,671) : 258,	(22,671) : 112,	(23,671) : 180,	(24,671) : 507,	(25,671) : 564,	(26,671) : 266,	(27,671) : 227,	(28,671) : 228,	(29,671) : 387,	(30,671) : 235,	(31,671) : 187,	(32,671) : 182,	(33,671) : 302,	(34,671) : 150,	(35,671) : 338,	(36,671) : 268,	(37,671) : 254,	(38,671) : 212,	(39,671) : 224,	(40,671) : 582,
(1,672) : 639,	(2,672) : 688,	(3,672) : 787,	(4,672) : 97,	(5,672) : 715,	(6,672) : 666,	(7,672) : 636,	(8,672) : 1071,	(9,672) : 829,	(10,672) : 636,	(11,672) : 689,	(12,672) : 207,	(13,672) : 472,	(14,672) : 825,	(15,672) : 218,	(16,672) : 21,	(17,672) : 130,	(18,672) : 698,	(19,672) : 243,	(20,672) : 635,	(21,672) : 367,	(22,672) : 682,	(23,672) : 693,	(24,672) : 129,	(25,672) : 1060,	(26,672) : 408,	(27,672) : 467,	(28,672) : 634,	(29,672) : 261,	(30,672) : 636,	(31,672) : 668,	(32,672) : 567,	(33,672) : 551,	(34,672) : 659,	(35,672) : 759,	(36,672) : 891,	(37,672) : 842,	(38,672) : 690,	(39,672) : 689,	(40,672) : 1087,
(1,673) : 734,	(2,673) : 789,	(3,673) : 1033,	(4,673) : 360,	(5,673) : 829,	(6,673) : 783,	(7,673) : 734,	(8,673) : 1321,	(9,673) : 1003,	(10,673) : 734,	(11,673) : 790,	(12,673) : 210,	(13,673) : 595,	(14,673) : 998,	(15,673) : 423,	(16,673) : 246,	(17,673) : 378,	(18,673) : 816,	(19,673) : 272,	(20,673) : 732,	(21,673) : 549,	(22,673) : 824,	(23,673) : 811,	(24,673) : 378,	(25,673) : 1291,	(26,673) : 538,	(27,673) : 591,	(28,673) : 732,	(29,673) : 497,	(30,673) : 732,	(31,673) : 782,	(32,673) : 685,	(33,673) : 623,	(34,673) : 785,	(35,673) : 828,	(36,673) : 1062,	(37,673) : 1039,	(38,673) : 797,	(39,673) : 791,	(40,673) : 1317,
(1,674) : 551,	(2,674) : 605,	(3,674) : 865,	(4,674) : 249,	(5,674) : 642,	(6,674) : 596,	(7,674) : 550,	(8,674) : 1155,	(9,674) : 812,	(10,674) : 550,	(11,674) : 605,	(12,674) : 24,	(13,674) : 405,	(14,674) : 808,	(15,674) : 251,	(16,674) : 179,	(17,674) : 243,	(18,674) : 629,	(19,674) : 87,	(20,674) : 547,	(21,674) : 363,	(22,674) : 634,	(23,674) : 624,	(24,674) : 244,	(25,674) : 1112,	(26,674) : 348,	(27,674) : 401,	(28,674) : 548,	(29,674) : 336,	(30,674) : 548,	(31,674) : 595,	(32,674) : 497,	(33,674) : 443,	(34,674) : 597,	(35,674) : 652,	(36,674) : 871,	(37,674) : 851,	(38,674) : 611,	(39,674) : 606,	(40,674) : 1137,
(1,675) : 434,	(2,675) : 469,	(3,675) : 510,	(4,675) : 217,	(5,675) : 479,	(6,675) : 431,	(7,675) : 426,	(8,675) : 800,	(9,675) : 539,	(10,675) : 428,	(11,675) : 470,	(12,675) : 350,	(13,675) : 271,	(14,675) : 535,	(15,675) : 104,	(16,675) : 317,	(17,675) : 171,	(18,675) : 460,	(19,675) : 321,	(20,675) : 428,	(21,675) : 118,	(22,675) : 423,	(23,675) : 454,	(24,675) : 173,	(25,675) : 767,	(26,675) : 221,	(27,675) : 265,	(28,675) : 426,	(29,675) : 45,	(30,675) : 431,	(31,675) : 437,	(32,675) : 347,	(33,675) : 392,	(34,675) : 417,	(35,675) : 566,	(36,675) : 602,	(37,675) : 544,	(38,675) : 464,	(39,675) : 468,	(40,675) : 794,
(1,676) : 718,	(2,676) : 701,	(3,676) : 88,	(4,676) : 779,	(5,676) : 661,	(6,676) : 651,	(7,676) : 707,	(8,676) : 205,	(9,676) : 441,	(10,676) : 709,	(11,676) : 702,	(12,676) : 945,	(13,676) : 678,	(14,676) : 443,	(15,676) : 701,	(16,676) : 895,	(17,676) : 748,	(18,676) : 648,	(19,676) : 907,	(20,676) : 713,	(21,676) : 610,	(22,676) : 573,	(23,676) : 647,	(24,676) : 749,	(25,676) : 219,	(26,676) : 688,	(27,676) : 676,	(28,676) : 710,	(29,676) : 624,	(30,676) : 717,	(31,676) : 663,	(32,676) : 661,	(33,676) : 780,	(34,676) : 626,	(35,676) : 805,	(36,676) : 462,	(37,676) : 331,	(38,676) : 684,	(39,676) : 697,	(40,676) : 249,
(1,677) : 1086,	(2,677) : 1050,	(3,677) : 579,	(4,677) : 1269,	(5,677) : 1000,	(6,677) : 1011,	(7,677) : 1076,	(8,677) : 323,	(9,677) : 733,	(10,677) : 1079,	(11,677) : 1051,	(12,677) : 1418,	(13,677) : 1103,	(14,677) : 738,	(15,677) : 1181,	(16,677) : 1384,	(17,677) : 1236,	(18,677) : 994,	(19,677) : 1374,	(20,677) : 1082,	(21,677) : 1071,	(22,677) : 931,	(23,677) : 995,	(24,677) : 1237,	(25,677) : 319,	(26,677) : 1129,	(27,677) : 1102,	(28,677) : 1080,	(29,677) : 1109,	(30,677) : 1086,	(31,677) : 1021,	(32,677) : 1057,	(33,677) : 1177,	(34,677) : 989,	(35,677) : 1130,	(36,677) : 709,	(37,677) : 636,	(38,677) : 1033,	(39,677) : 1046,	(40,677) : 298,
(1,678) : 314,	(2,678) : 351,	(3,678) : 532,	(4,678) : 303,	(5,678) : 365,	(6,678) : 317,	(7,678) : 306,	(8,678) : 820,	(9,678) : 466,	(10,678) : 308,	(11,678) : 352,	(12,678) : 346,	(13,678) : 150,	(14,678) : 461,	(15,678) : 150,	(16,678) : 379,	(17,678) : 253,	(18,678) : 347,	(19,678) : 299,	(20,678) : 308,	(21,678) : 5,	(22,678) : 319,	(23,678) : 341,	(24,678) : 256,	(25,678) : 758,	(26,678) : 106,	(27,678) : 144,	(28,678) : 307,	(29,678) : 159,	(30,678) : 311,	(31,678) : 322,	(32,678) : 228,	(33,678) : 274,	(34,678) : 305,	(35,678) : 445,	(36,678) : 527,	(37,678) : 493,	(38,678) : 348,	(39,678) : 351,	(40,678) : 781,
(1,679) : 702,	(2,679) : 694,	(3,679) : 60,	(4,679) : 685,	(5,679) : 661,	(6,679) : 643,	(7,679) : 691,	(8,679) : 289,	(9,679) : 475,	(10,679) : 694,	(11,679) : 695,	(12,679) : 867,	(13,679) : 637,	(14,679) : 476,	(15,679) : 621,	(16,679) : 802,	(17,679) : 658,	(18,679) : 646,	(19,679) : 835,	(20,679) : 697,	(21,679) : 547,	(22,679) : 570,	(23,679) : 643,	(24,679) : 658,	(25,679) : 331,	(26,679) : 637,	(27,679) : 634,	(28,679) : 694,	(29,679) : 539,	(30,679) : 701,	(31,679) : 654,	(32,679) : 636,	(33,679) : 749,	(34,679) : 617,	(35,679) : 803,	(36,679) : 508,	(37,679) : 376,	(38,679) : 678,	(39,679) : 691,	(40,679) : 361,
(1,680) : 326,	(2,680) : 305,	(3,680) : 346,	(4,680) : 620,	(5,680) : 266,	(6,680) : 256,	(7,680) : 315,	(8,680) : 581,	(9,680) : 140,	(10,680) : 317,	(11,680) : 306,	(12,680) : 682,	(13,680) : 329,	(14,680) : 137,	(15,680) : 482,	(16,680) : 713,	(17,680) : 573,	(18,680) : 253,	(19,680) : 628,	(20,680) : 321,	(21,680) : 339,	(22,680) : 178,	(23,680) : 251,	(24,680) : 575,	(25,680) : 461,	(26,680) : 364,	(27,680) : 329,	(28,680) : 318,	(29,680) : 447,	(30,680) : 325,	(31,680) : 267,	(32,680) : 282,	(33,680) : 403,	(34,680) : 231,	(35,680) : 410,	(36,680) : 202,	(37,680) : 154,	(38,680) : 288,	(39,680) : 302,	(40,680) : 479,
(1,681) : 222,	(2,681) : 254,	(3,681) : 525,	(4,681) : 402,	(5,681) : 266,	(6,681) : 218,	(7,681) : 213,	(8,681) : 804,	(9,681) : 385,	(10,681) : 215,	(11,681) : 255,	(12,681) : 411,	(13,681) : 83,	(14,681) : 380,	(15,681) : 247,	(16,681) : 472,	(17,681) : 352,	(18,681) : 247,	(19,681) : 355,	(20,681) : 215,	(21,681) : 101,	(22,681) : 222,	(23,681) : 242,	(24,681) : 354,	(25,681) : 719,	(26,681) : 92,	(27,681) : 79,	(28,681) : 214,	(29,681) : 256,	(30,681) : 219,	(31,681) : 223,	(32,681) : 133,	(33,681) : 207,	(34,681) : 205,	(35,681) : 354,	(36,681) : 444,	(37,681) : 428,	(38,681) : 250,	(39,681) : 254,	(40,681) : 740,
(1,682) : 174,	(2,682) : 190,	(3,682) : 504,	(4,682) : 488,	(5,682) : 190,	(6,682) : 145,	(7,682) : 163,	(8,682) : 770,	(9,682) : 303,	(10,682) : 166,	(11,682) : 191,	(12,682) : 493,	(13,682) : 121,	(14,682) : 299,	(15,682) : 333,	(16,682) : 559,	(17,682) : 438,	(18,682) : 170,	(19,682) : 434,	(20,682) : 167,	(21,682) : 185,	(22,682) : 135,	(23,682) : 165,	(24,682) : 440,	(25,682) : 667,	(26,682) : 164,	(27,682) : 122,	(28,682) : 165,	(29,682) : 335,	(30,682) : 171,	(31,682) : 153,	(32,682) : 94,	(33,682) : 208,	(34,682) : 127,	(35,682) : 299,	(36,682) : 361,	(37,682) : 359,	(38,682) : 182,	(39,682) : 189,	(40,682) : 685,
(1,683) : 240,	(2,683) : 285,	(3,683) : 604,	(4,683) : 360,	(5,683) : 311,	(6,683) : 262,	(7,683) : 234,	(8,683) : 886,	(9,683) : 467,	(10,683) : 235,	(11,683) : 286,	(12,683) : 329,	(13,683) : 69,	(14,683) : 462,	(15,683) : 205,	(16,683) : 412,	(17,683) : 311,	(18,683) : 295,	(19,683) : 271,	(20,683) : 234,	(21,683) : 103,	(22,683) : 288,	(23,683) : 289,	(24,683) : 314,	(25,683) : 806,	(26,683) : 9,	(27,683) : 63,	(28,683) : 234,	(29,683) : 245,	(30,683) : 237,	(31,683) : 264,	(32,683) : 163,	(33,683) : 179,	(34,683) : 257,	(35,683) : 367,	(36,683) : 525,	(37,683) : 515,	(38,683) : 286,	(39,683) : 286,	(40,683) : 827,
(1,684) : 404,	(2,684) : 456,	(3,684) : 737,	(4,684) : 248,	(5,684) : 491,	(6,684) : 443,	(7,684) : 402,	(8,684) : 1026,	(9,684) : 656,	(10,684) : 402,	(11,684) : 457,	(12,684) : 139,	(13,684) : 250,	(14,684) : 651,	(15,684) : 152,	(16,684) : 252,	(17,684) : 212,	(18,684) : 476,	(19,684) : 87,	(20,684) : 400,	(21,684) : 216,	(22,684) : 478,	(23,684) : 471,	(24,684) : 215,	(25,684) : 970,	(26,684) : 191,	(27,684) : 246,	(28,684) : 400,	(29,684) : 241,	(30,684) : 401,	(31,684) : 443,	(32,684) : 344,	(33,684) : 306,	(34,684) : 442,	(35,684) : 516,	(36,684) : 715,	(37,684) : 697,	(38,684) : 461,	(39,684) : 458,	(40,684) : 993,
(1,685) : 428,	(2,685) : 394,	(3,685) : 332,	(4,685) : 751,	(5,685) : 345,	(6,685) : 353,	(7,685) : 418,	(8,685) : 505,	(9,685) : 92,	(10,685) : 420,	(11,685) : 395,	(12,685) : 821,	(13,685) : 461,	(14,685) : 94,	(15,685) : 618,	(16,685) : 847,	(17,685) : 705,	(18,685) : 338,	(19,685) : 766,	(20,685) : 424,	(21,685) : 476,	(22,685) : 272,	(23,685) : 339,	(24,685) : 707,	(25,685) : 353,	(26,685) : 501,	(27,685) : 462,	(28,685) : 422,	(29,685) : 577,	(30,685) : 428,	(31,685) : 363,	(32,685) : 403,	(33,685) : 521,	(34,685) : 331,	(35,685) : 483,	(36,685) : 118,	(37,685) : 23,	(38,685) : 376,	(39,685) : 390,	(40,685) : 365,
(1,686) : 640,	(2,686) : 689,	(3,686) : 796,	(4,686) : 106,	(5,686) : 716,	(6,686) : 667,	(7,686) : 636,	(8,686) : 1080,	(9,686) : 834,	(10,686) : 637,	(11,686) : 690,	(12,686) : 200,	(13,686) : 473,	(14,686) : 830,	(15,686) : 222,	(16,686) : 11,	(17,686) : 138,	(18,686) : 700,	(19,686) : 237,	(20,686) : 636,	(21,686) : 371,	(22,686) : 685,	(23,686) : 694,	(24,686) : 137,	(25,686) : 1068,	(26,686) : 409,	(27,686) : 468,	(28,686) : 635,	(29,686) : 269,	(30,686) : 637,	(31,686) : 669,	(32,686) : 568,	(33,686) : 550,	(34,686) : 661,	(35,686) : 759,	(36,686) : 896,	(37,686) : 848,	(38,686) : 691,	(39,686) : 690,	(40,686) : 1095,
(1,687) : 572,	(2,687) : 618,	(3,687) : 708,	(4,687) : 44,	(5,687) : 642,	(6,687) : 593,	(7,687) : 567,	(8,687) : 993,	(9,687) : 746,	(10,687) : 568,	(11,687) : 619,	(12,687) : 216,	(13,687) : 402,	(14,687) : 742,	(15,687) : 137,	(16,687) : 102,	(17,687) : 48,	(18,687) : 625,	(19,687) : 229,	(20,687) : 567,	(21,687) : 287,	(22,687) : 605,	(23,687) : 619,	(24,687) : 48,	(25,687) : 977,	(26,687) : 338,	(27,687) : 396,	(28,687) : 566,	(29,687) : 177,	(30,687) : 569,	(31,687) : 596,	(32,687) : 495,	(33,687) : 492,	(34,687) : 585,	(35,687) : 696,	(36,687) : 808,	(37,687) : 758,	(38,687) : 619,	(39,687) : 619,	(40,687) : 1004,
(1,688) : 539,	(2,688) : 587,	(3,688) : 714,	(4,688) : 85,	(5,688) : 612,	(6,688) : 563,	(7,688) : 535,	(8,688) : 1002,	(9,688) : 729,	(10,688) : 536,	(11,688) : 588,	(12,688) : 181,	(13,688) : 370,	(14,688) : 725,	(15,688) : 118,	(16,688) : 114,	(17,688) : 68,	(18,688) : 596,	(19,688) : 188,	(20,688) : 534,	(21,688) : 266,	(22,688) : 579,	(23,688) : 590,	(24,688) : 69,	(25,688) : 978,	(26,688) : 306,	(27,688) : 365,	(28,688) : 534,	(29,688) : 179,	(30,688) : 536,	(31,688) : 566,	(32,688) : 465,	(33,688) : 455,	(34,688) : 557,	(35,688) : 661,	(36,688) : 791,	(37,688) : 747,	(38,688) : 588,	(39,688) : 587,	(40,688) : 1004,
(1,689) : 516,	(2,689) : 553,	(3,689) : 551,	(4,689) : 144,	(5,689) : 565,	(6,689) : 518,	(7,689) : 509,	(8,689) : 837,	(9,689) : 623,	(10,689) : 511,	(11,689) : 554,	(12,689) : 337,	(13,689) : 349,	(14,689) : 619,	(15,689) : 110,	(16,689) : 256,	(17,689) : 109,	(18,689) : 546,	(19,689) : 324,	(20,689) : 510,	(21,689) : 200,	(22,689) : 510,	(23,689) : 541,	(24,689) : 109,	(25,689) : 823,	(26,689) : 294,	(27,689) : 343,	(28,689) : 509,	(29,689) : 44,	(30,689) : 513,	(31,689) : 523,	(32,689) : 430,	(33,689) : 465,	(34,689) : 504,	(35,689) : 648,	(36,689) : 685,	(37,689) : 620,	(38,689) : 550,	(39,689) : 553,	(40,689) : 850,
(1,690) : 491,	(2,690) : 464,	(3,690) : 239,	(4,690) : 731,	(5,690) : 419,	(6,690) : 418,	(7,690) : 481,	(8,690) : 421,	(9,690) : 185,	(10,690) : 483,	(11,690) : 465,	(12,690) : 831,	(13,690) : 496,	(14,690) : 186,	(15,690) : 611,	(16,690) : 835,	(17,690) : 689,	(18,690) : 409,	(19,690) : 781,	(20,690) : 487,	(21,690) : 481,	(22,690) : 338,	(23,690) : 409,	(24,690) : 690,	(25,690) : 291,	(26,690) : 525,	(27,690) : 495,	(28,690) : 484,	(29,690) : 558,	(30,690) : 491,	(31,690) : 429,	(32,690) : 452,	(33,690) : 573,	(34,690) : 394,	(35,690) : 561,	(36,690) : 210,	(37,690) : 78,	(38,690) : 447,	(39,690) : 461,	(40,690) : 309,
(1,691) : 705,	(2,691) : 694,	(3,691) : 49,	(4,691) : 705,	(5,691) : 660,	(6,691) : 643,	(7,691) : 693,	(8,691) : 269,	(9,691) : 467,	(10,691) : 696,	(11,691) : 695,	(12,691) : 884,	(13,691) : 645,	(14,691) : 467,	(15,691) : 638,	(16,691) : 822,	(17,691) : 677,	(18,691) : 645,	(19,691) : 850,	(20,691) : 699,	(21,691) : 560,	(22,691) : 569,	(23,691) : 643,	(24,691) : 677,	(25,691) : 307,	(26,691) : 647,	(27,691) : 642,	(28,691) : 697,	(29,691) : 556,	(30,691) : 703,	(31,691) : 655,	(32,691) : 640,	(33,691) : 754,	(34,691) : 617,	(35,691) : 803,	(36,691) : 497,	(37,691) : 364,	(38,691) : 678,	(39,691) : 691,	(40,691) : 337,
(1,692) : 382,	(2,692) : 415,	(3,692) : 491,	(4,692) : 268,	(5,692) : 423,	(6,692) : 377,	(7,692) : 374,	(8,692) : 781,	(9,692) : 487,	(10,692) : 376,	(11,692) : 416,	(12,692) : 368,	(13,692) : 224,	(14,692) : 483,	(15,692) : 134,	(16,692) : 361,	(17,692) : 220,	(18,692) : 404,	(19,692) : 331,	(20,692) : 376,	(21,692) : 71,	(22,692) : 367,	(23,692) : 399,	(24,692) : 222,	(25,692) : 736,	(26,692) : 181,	(27,692) : 218,	(28,692) : 374,	(29,692) : 101,	(30,692) : 379,	(31,692) : 383,	(32,692) : 294,	(33,692) : 348,	(34,692) : 362,	(35,692) : 514,	(36,692) : 550,	(37,692) : 497,	(38,692) : 410,	(39,692) : 414,	(40,692) : 761,
(1,693) : 529,	(2,693) : 573,	(3,693) : 651,	(4,693) : 72,	(5,693) : 593,	(6,693) : 544,	(7,693) : 523,	(8,693) : 938,	(9,693) : 688,	(10,693) : 524,	(11,693) : 574,	(12,693) : 242,	(13,693) : 358,	(14,693) : 684,	(15,693) : 85,	(16,693) : 162,	(17,693) : 22,	(18,693) : 575,	(19,693) : 239,	(20,693) : 523,	(21,693) : 232,	(22,693) : 552,	(23,693) : 570,	(24,693) : 25,	(25,693) : 918,	(26,693) : 295,	(27,693) : 352,	(28,693) : 522,	(29,693) : 118,	(30,693) : 526,	(31,693) : 548,	(32,693) : 449,	(33,693) : 456,	(34,693) : 535,	(35,693) : 655,	(36,693) : 750,	(37,693) : 698,	(38,693) : 572,	(39,693) : 573,	(40,693) : 945,
(1,694) : 215,	(2,694) : 215,	(3,694) : 446,	(4,694) : 524,	(5,694) : 197,	(6,694) : 164,	(7,694) : 204,	(8,694) : 706,	(9,694) : 243,	(10,694) : 207,	(11,694) : 216,	(12,694) : 553,	(13,694) : 189,	(14,694) : 239,	(15,694) : 375,	(16,694) : 605,	(17,694) : 475,	(18,694) : 178,	(19,694) : 496,	(20,694) : 209,	(21,694) : 225,	(22,694) : 116,	(23,694) : 174,	(24,694) : 477,	(25,694) : 598,	(26,694) : 228,	(27,694) : 190,	(28,694) : 207,	(29,694) : 361,	(30,694) : 214,	(31,694) : 174,	(32,694) : 152,	(33,694) : 271,	(34,694) : 140,	(35,694) : 327,	(36,694) : 303,	(37,694) : 291,	(38,694) : 202,	(39,694) : 212,	(40,694) : 617,
(1,695) : 504,	(2,695) : 556,	(3,695) : 796,	(4,695) : 204,	(5,695) : 591,	(6,695) : 543,	(7,695) : 502,	(8,695) : 1086,	(9,695) : 749,	(10,695) : 502,	(11,695) : 557,	(12,695) : 65,	(13,695) : 349,	(14,695) : 744,	(15,695) : 182,	(16,695) : 168,	(17,695) : 187,	(18,695) : 576,	(19,695) : 71,	(20,695) : 500,	(21,695) : 295,	(22,695) : 576,	(23,695) : 571,	(24,695) : 189,	(25,695) : 1043,	(26,695) : 289,	(27,695) : 345,	(28,695) : 500,	(29,695) : 269,	(30,695) : 501,	(31,695) : 543,	(32,695) : 443,	(33,695) : 403,	(34,695) : 542,	(35,695) : 613,	(36,695) : 809,	(37,695) : 784,	(38,695) : 561,	(39,695) : 558,	(40,695) : 1067,
(1,696) : 162,	(2,696) : 209,	(3,696) : 792,	(4,696) : 580,	(5,696) : 261,	(6,696) : 237,	(7,696) : 170,	(8,696) : 1053,	(9,696) : 519,	(10,696) : 168,	(11,696) : 208,	(12,696) : 442,	(13,696) : 208,	(14,696) : 514,	(15,696) : 438,	(16,696) : 597,	(17,696) : 536,	(18,696) : 260,	(19,696) : 380,	(20,696) : 164,	(21,696) : 354,	(22,696) : 315,	(23,696) : 257,	(24,696) : 538,	(25,696) : 934,	(26,696) : 245,	(27,696) : 213,	(28,696) : 166,	(29,696) : 495,	(30,696) : 161,	(31,696) : 228,	(32,696) : 197,	(33,696) : 83,	(34,696) : 257,	(35,696) : 200,	(36,696) : 560,	(37,696) : 609,	(38,696) : 224,	(39,696) : 212,	(40,696) : 949,
(1,697) : 578,	(2,697) : 624,	(3,697) : 710,	(4,697) : 40,	(5,697) : 647,	(6,697) : 598,	(7,697) : 573,	(8,697) : 995,	(9,697) : 751,	(10,697) : 574,	(11,697) : 625,	(12,697) : 218,	(13,697) : 408,	(14,697) : 747,	(15,697) : 142,	(16,697) : 98,	(17,697) : 50,	(18,697) : 630,	(19,697) : 233,	(20,697) : 573,	(21,697) : 292,	(22,697) : 610,	(23,697) : 624,	(24,697) : 50,	(25,697) : 980,	(26,697) : 344,	(27,697) : 402,	(28,697) : 572,	(29,697) : 181,	(30,697) : 575,	(31,697) : 601,	(32,697) : 501,	(33,697) : 498,	(34,697) : 590,	(35,697) : 701,	(36,697) : 813,	(37,697) : 762,	(38,697) : 625,	(39,697) : 625,	(40,697) : 1007,
(1,698) : 674,	(2,698) : 643,	(3,698) : 244,	(4,698) : 882,	(5,698) : 595,	(6,698) : 600,	(7,698) : 664,	(8,698) : 269,	(9,698) : 337,	(10,698) : 666,	(11,698) : 644,	(12,698) : 1006,	(13,698) : 682,	(14,698) : 341,	(15,698) : 776,	(16,698) : 992,	(17,698) : 844,	(18,698) : 587,	(19,698) : 959,	(20,698) : 670,	(21,698) : 656,	(22,698) : 519,	(23,698) : 588,	(24,698) : 844,	(25,698) : 104,	(26,698) : 709,	(27,698) : 682,	(28,698) : 668,	(29,698) : 713,	(30,698) : 674,	(31,698) : 611,	(32,698) : 639,	(33,698) : 759,	(34,698) : 577,	(35,698) : 733,	(36,698) : 333,	(37,698) : 227,	(38,698) : 626,	(39,698) : 639,	(40,698) : 123,
(1,699) : 654,	(2,699) : 621,	(3,699) : 268,	(4,699) : 887,	(5,699) : 572,	(6,699) : 579,	(7,699) : 644,	(8,699) : 302,	(9,699) : 311,	(10,699) : 646,	(11,699) : 622,	(12,699) : 1002,	(13,699) : 670,	(14,699) : 314,	(15,699) : 776,	(16,699) : 995,	(17,699) : 847,	(18,699) : 565,	(19,699) : 953,	(20,699) : 650,	(21,699) : 652,	(22,699) : 498,	(23,699) : 566,	(24,699) : 848,	(25,699) : 126,	(26,699) : 700,	(27,699) : 670,	(28,699) : 648,	(29,699) : 716,	(30,699) : 654,	(31,699) : 590,	(32,699) : 623,	(33,699) : 743,	(34,699) : 557,	(35,699) : 709,	(36,699) : 302,	(37,699) : 204,	(38,699) : 604,	(39,699) : 617,	(40,699) : 138,
(1,700) : 529,	(2,700) : 565,	(3,700) : 544,	(4,700) : 148,	(5,700) : 576,	(6,700) : 529,	(7,700) : 521,	(8,700) : 828,	(9,700) : 627,	(10,700) : 523,	(11,700) : 566,	(12,700) : 351,	(13,700) : 363,	(14,700) : 623,	(15,700) : 127,	(16,700) : 263,	(17,700) : 118,	(18,700) : 557,	(19,700) : 339,	(20,700) : 523,	(21,700) : 212,	(22,700) : 519,	(23,700) : 551,	(24,700) : 118,	(25,700) : 818,	(26,700) : 308,	(27,700) : 357,	(28,700) : 522,	(29,700) : 56,	(30,700) : 526,	(31,700) : 534,	(32,700) : 442,	(33,700) : 479,	(34,700) : 514,	(35,700) : 660,	(36,700) : 689,	(37,700) : 621,	(38,700) : 561,	(39,700) : 565,	(40,700) : 846,
(1,701) : 160,	(2,701) : 172,	(3,701) : 509,	(4,701) : 509,	(5,701) : 169,	(6,701) : 125,	(7,701) : 149,	(8,701) : 771,	(9,701) : 289,	(10,701) : 151,	(11,701) : 173,	(12,701) : 509,	(13,701) : 132,	(14,701) : 284,	(15,701) : 354,	(16,701) : 579,	(17,701) : 459,	(18,701) : 149,	(19,701) : 450,	(20,701) : 153,	(21,701) : 206,	(22,701) : 114,	(23,701) : 144,	(24,701) : 461,	(25,701) : 662,	(26,701) : 180,	(27,701) : 133,	(28,701) : 151,	(29,701) : 356,	(30,701) : 158,	(31,701) : 134,	(32,701) : 87,	(33,701) : 206,	(34,701) : 107,	(35,701) : 282,	(36,701) : 346,	(37,701) : 350,	(38,701) : 162,	(39,701) : 170,	(40,701) : 680,
(1,702) : 317,	(2,702) : 370,	(3,702) : 704,	(4,702) : 314,	(5,702) : 406,	(6,702) : 358,	(7,702) : 316,	(8,702) : 991,	(9,702) : 583,	(10,702) : 316,	(11,702) : 371,	(12,702) : 217,	(13,702) : 168,	(14,702) : 578,	(15,702) : 184,	(16,702) : 335,	(17,702) : 272,	(18,702) : 392,	(19,702) : 156,	(20,702) : 314,	(21,702) : 178,	(22,702) : 397,	(23,702) : 386,	(24,702) : 274,	(25,702) : 919,	(26,702) : 115,	(27,702) : 164,	(28,702) : 314,	(29,702) : 260,	(30,702) : 315,	(31,702) : 358,	(32,702) : 259,	(33,702) : 220,	(34,702) : 359,	(35,702) : 430,	(36,702) : 640,	(37,702) : 633,	(38,702) : 375,	(39,702) : 372,	(40,702) : 941,
(1,703) : 685,	(2,703) : 670,	(3,703) : 41,	(4,703) : 732,	(5,703) : 633,	(6,703) : 620,	(7,703) : 674,	(8,703) : 251,	(9,703) : 426,	(10,703) : 676,	(11,703) : 671,	(12,703) : 898,	(13,703) : 638,	(14,703) : 427,	(15,703) : 654,	(16,703) : 847,	(17,703) : 701,	(18,703) : 620,	(19,703) : 861,	(20,703) : 679,	(21,703) : 565,	(22,703) : 544,	(23,703) : 618,	(24,703) : 701,	(25,703) : 258,	(26,703) : 646,	(27,703) : 636,	(28,703) : 677,	(29,703) : 576,	(30,703) : 684,	(31,703) : 632,	(32,703) : 625,	(33,703) : 742,	(34,703) : 595,	(35,703) : 777,	(36,703) : 452,	(37,703) : 320,	(38,703) : 654,	(39,703) : 667,	(40,703) : 289,
(1,704) : 529,	(2,704) : 572,	(3,704) : 642,	(4,704) : 74,	(5,704) : 592,	(6,704) : 543,	(7,704) : 523,	(8,704) : 929,	(9,704) : 683,	(10,704) : 524,	(11,704) : 573,	(12,704) : 252,	(13,704) : 358,	(14,704) : 679,	(15,704) : 85,	(16,704) : 169,	(17,704) : 24,	(18,704) : 574,	(19,704) : 248,	(20,704) : 524,	(21,704) : 230,	(22,704) : 549,	(23,704) : 569,	(24,704) : 26,	(25,704) : 910,	(26,704) : 296,	(27,704) : 352,	(28,704) : 523,	(29,704) : 110,	(30,704) : 526,	(31,704) : 547,	(32,704) : 448,	(33,704) : 459,	(34,704) : 533,	(35,704) : 656,	(36,704) : 746,	(37,704) : 692,	(38,704) : 572,	(39,704) : 573,	(40,704) : 937,
(1,705) : 103,	(2,705) : 72,	(3,705) : 711,	(4,705) : 703,	(5,705) : 100,	(6,705) : 122,	(7,705) : 109,	(8,705) : 939,	(9,705) : 358,	(10,705) : 108,	(11,705) : 70,	(12,705) : 621,	(13,705) : 274,	(14,705) : 354,	(15,705) : 547,	(16,705) : 747,	(17,705) : 654,	(18,705) : 114,	(19,705) : 557,	(20,705) : 109,	(21,705) : 418,	(22,705) : 189,	(23,705) : 116,	(24,705) : 656,	(25,705) : 794,	(26,705) : 337,	(27,705) : 280,	(28,705) : 110,	(29,705) : 575,	(30,705) : 106,	(31,705) : 111,	(32,705) : 188,	(33,705) : 201,	(34,705) : 146,	(35,705) : 44,	(36,705) : 385,	(37,705) : 464,	(38,705) : 84,	(39,705) : 74,	(40,705) : 804,
(1,706) : 1184,	(2,706) : 1157,	(3,706) : 576,	(4,706) : 1249,	(5,706) : 1111,	(6,706) : 1112,	(7,706) : 1174,	(8,706) : 288,	(9,706) : 855,	(10,706) : 1176,	(11,706) : 1158,	(12,706) : 1433,	(13,706) : 1166,	(14,706) : 858,	(15,706) : 1188,	(16,706) : 1366,	(17,706) : 1224,	(18,706) : 1102,	(19,706) : 1398,	(20,706) : 1180,	(21,706) : 1103,	(22,706) : 1031,	(23,706) : 1102,	(24,706) : 1224,	(25,706) : 419,	(26,706) : 1180,	(27,706) : 1164,	(28,706) : 1177,	(29,706) : 1106,	(30,706) : 1184,	(31,706) : 1123,	(32,706) : 1139,	(33,706) : 1259,	(34,706) : 1088,	(35,706) : 1251,	(36,706) : 846,	(37,706) : 744,	(38,706) : 1140,	(39,706) : 1154,	(40,706) : 416,
(1,707) : 156,	(2,707) : 185,	(3,707) : 544,	(4,707) : 469,	(5,707) : 197,	(6,707) : 148,	(7,707) : 146,	(8,707) : 813,	(9,707) : 344,	(10,707) : 148,	(11,707) : 186,	(12,707) : 455,	(13,707) : 77,	(14,707) : 339,	(15,707) : 313,	(16,707) : 533,	(17,707) : 419,	(18,707) : 178,	(19,707) : 396,	(20,707) : 149,	(21,707) : 170,	(22,707) : 161,	(23,707) : 173,	(24,707) : 421,	(25,707) : 711,	(26,707) : 126,	(27,707) : 78,	(28,707) : 148,	(29,707) : 326,	(30,707) : 153,	(31,707) : 153,	(32,707) : 68,	(33,707) : 168,	(34,707) : 137,	(35,707) : 287,	(36,707) : 400,	(37,707) : 404,	(38,707) : 180,	(39,707) : 184,	(40,707) : 730,
(1,708) : 724,	(2,708) : 690,	(3,708) : 309,	(4,708) : 953,	(5,708) : 640,	(6,708) : 649,	(7,708) : 714,	(8,708) : 269,	(9,708) : 376,	(10,708) : 717,	(11,708) : 690,	(12,708) : 1074,	(13,708) : 743,	(14,708) : 380,	(15,708) : 846,	(16,708) : 1063,	(17,708) : 915,	(18,708) : 634,	(19,708) : 1026,	(20,708) : 720,	(21,708) : 724,	(22,708) : 568,	(23,708) : 635,	(24,708) : 915,	(25,708) : 71,	(26,708) : 773,	(27,708) : 743,	(28,708) : 718,	(29,708) : 784,	(30,708) : 724,	(31,708) : 659,	(32,708) : 695,	(33,708) : 815,	(34,708) : 627,	(35,708) : 774,	(36,708) : 360,	(37,708) : 274,	(38,708) : 672,	(39,708) : 686,	(40,708) : 71,
(1,709) : 334,	(2,709) : 299,	(3,709) : 397,	(4,709) : 710,	(5,709) : 251,	(6,709) : 259,	(7,709) : 325,	(8,709) : 595,	(9,709) : 45,	(10,709) : 327,	(11,709) : 300,	(12,709) : 754,	(13,709) : 382,	(14,709) : 42,	(15,709) : 567,	(16,709) : 798,	(17,709) : 662,	(18,709) : 243,	(19,709) : 697,	(20,709) : 331,	(21,709) : 420,	(22,709) : 179,	(23,709) : 244,	(24,709) : 664,	(25,709) : 448,	(26,709) : 428,	(27,709) : 384,	(28,709) : 329,	(29,709) : 538,	(30,709) : 334,	(31,709) : 269,	(32,709) : 315,	(33,709) : 431,	(34,709) : 238,	(35,709) : 389,	(36,709) : 108,	(37,709) : 118,	(38,709) : 282,	(39,709) : 295,	(40,709) : 460,
(1,710) : 328,	(2,710) : 366,	(3,710) : 537,	(4,710) : 288,	(5,710) : 380,	(6,710) : 332,	(7,710) : 320,	(8,710) : 826,	(9,710) : 480,	(10,710) : 322,	(11,710) : 367,	(12,710) : 336,	(13,710) : 162,	(14,710) : 476,	(15,710) : 135,	(16,710) : 364,	(17,710) : 238,	(18,710) : 362,	(19,710) : 290,	(20,710) : 322,	(21,710) : 16,	(22,710) : 335,	(23,710) : 356,	(24,710) : 240,	(25,710) : 767,	(26,710) : 114,	(27,710) : 156,	(28,710) : 321,	(29,710) : 145,	(30,710) : 325,	(31,710) : 337,	(32,710) : 242,	(33,710) : 284,	(34,710) : 320,	(35,710) : 459,	(36,710) : 541,	(37,710) : 505,	(38,710) : 363,	(39,710) : 365,	(40,710) : 791,
(1,711) : 528,	(2,711) : 583,	(3,711) : 854,	(4,711) : 254,	(5,711) : 620,	(6,711) : 574,	(7,711) : 528,	(8,711) : 1145,	(9,711) : 793,	(10,711) : 528,	(11,711) : 583,	(12,711) : 5,	(13,711) : 384,	(14,711) : 789,	(15,711) : 242,	(16,711) : 193,	(17,711) : 243,	(18,711) : 607,	(19,711) : 64,	(20,711) : 525,	(21,711) : 347,	(22,711) : 613,	(23,711) : 602,	(24,711) : 245,	(25,711) : 1098,	(26,711) : 328,	(27,711) : 380,	(28,711) : 525,	(29,711) : 329,	(30,711) : 526,	(31,711) : 573,	(32,711) : 476,	(33,711) : 421,	(34,711) : 575,	(35,711) : 630,	(36,711) : 852,	(37,711) : 833,	(38,711) : 589,	(39,711) : 584,	(40,711) : 1123,
(1,712) : 109,	(2,712) : 104,	(3,712) : 551,	(4,712) : 574,	(5,712) : 97,	(6,712) : 55,	(7,712) : 98,	(8,712) : 799,	(9,712) : 269,	(10,712) : 101,	(11,712) : 106,	(12,712) : 552,	(13,712) : 167,	(14,712) : 264,	(15,712) : 418,	(16,712) : 638,	(17,712) : 524,	(18,712) : 77,	(19,712) : 490,	(20,712) : 104,	(21,712) : 273,	(22,712) : 65,	(23,712) : 72,	(24,712) : 526,	(25,712) : 675,	(26,712) : 227,	(27,712) : 171,	(28,712) : 101,	(29,712) : 426,	(30,712) : 108,	(31,712) : 64,	(32,712) : 81,	(33,712) : 193,	(34,712) : 35,	(35,712) : 217,	(36,712) : 318,	(37,712) : 350,	(38,712) : 93,	(39,712) : 102,	(40,712) : 690,
(1,713) : 661,	(2,713) : 716,	(3,713) : 975,	(4,713) : 319,	(5,713) : 756,	(6,713) : 710,	(7,713) : 661,	(8,713) : 1264,	(9,713) : 932,	(10,713) : 661,	(11,713) : 717,	(12,713) : 137,	(13,713) : 522,	(14,713) : 927,	(15,713) : 361,	(16,713) : 214,	(17,713) : 329,	(18,713) : 743,	(19,713) : 199,	(20,713) : 659,	(21,713) : 481,	(22,713) : 751,	(23,713) : 738,	(24,713) : 330,	(25,713) : 1228,	(26,713) : 466,	(27,713) : 518,	(28,713) : 659,	(29,713) : 440,	(30,713) : 659,	(31,713) : 708,	(32,713) : 612,	(33,713) : 550,	(34,713) : 712,	(35,713) : 756,	(36,713) : 991,	(37,713) : 970,	(38,713) : 724,	(39,713) : 718,	(40,713) : 1253,
(1,714) : 586,	(2,714) : 629,	(3,714) : 650,	(4,714) : 42,	(5,714) : 646,	(6,714) : 598,	(7,714) : 580,	(8,714) : 932,	(9,714) : 724,	(10,714) : 582,	(11,714) : 630,	(12,714) : 288,	(13,714) : 415,	(14,714) : 720,	(15,714) : 143,	(16,714) : 159,	(17,714) : 46,	(18,714) : 628,	(19,714) : 295,	(20,714) : 581,	(21,714) : 281,	(22,714) : 600,	(23,714) : 623,	(24,714) : 43,	(25,714) : 926,	(26,714) : 354,	(27,714) : 409,	(28,714) : 580,	(29,714) : 141,	(30,714) : 583,	(31,714) : 602,	(32,714) : 505,	(33,714) : 518,	(34,714) : 587,	(35,714) : 714,	(36,714) : 786,	(37,714) : 725,	(38,714) : 627,	(39,714) : 629,	(40,714) : 954,
(1,715) : 543,	(2,715) : 587,	(3,715) : 658,	(4,715) : 57,	(5,715) : 608,	(6,715) : 559,	(7,715) : 538,	(8,715) : 944,	(9,715) : 701,	(10,715) : 539,	(11,715) : 588,	(12,715) : 245,	(13,715) : 372,	(14,715) : 697,	(15,715) : 100,	(16,715) : 152,	(17,715) : 7,	(18,715) : 590,	(19,715) : 246,	(20,715) : 538,	(21,715) : 246,	(22,715) : 566,	(23,715) : 584,	(24,715) : 10,	(25,715) : 927,	(26,715) : 310,	(27,715) : 366,	(28,715) : 537,	(29,715) : 127,	(30,715) : 540,	(31,715) : 562,	(32,715) : 464,	(33,715) : 471,	(34,715) : 549,	(35,715) : 670,	(36,715) : 763,	(37,715) : 710,	(38,715) : 587,	(39,715) : 588,	(40,715) : 954,
(1,716) : 377,	(2,716) : 355,	(3,716) : 300,	(4,716) : 638,	(5,716) : 316,	(6,716) : 307,	(7,716) : 366,	(8,716) : 530,	(9,716) : 149,	(10,716) : 368,	(11,716) : 356,	(12,716) : 716,	(13,716) : 374,	(14,716) : 147,	(15,716) : 507,	(16,716) : 735,	(17,716) : 592,	(18,716) : 303,	(19,716) : 664,	(20,716) : 372,	(21,716) : 369,	(22,716) : 228,	(23,716) : 302,	(24,716) : 594,	(25,716) : 413,	(26,716) : 405,	(27,716) : 373,	(28,716) : 369,	(29,716) : 464,	(30,716) : 376,	(31,716) : 318,	(32,716) : 332,	(33,716) : 453,	(34,716) : 282,	(35,716) : 459,	(36,716) : 204,	(37,716) : 123,	(38,716) : 339,	(39,716) : 352,	(40,716) : 431,
(1,717) : 348,	(2,717) : 369,	(3,717) : 410,	(4,717) : 367,	(5,717) : 366,	(6,717) : 324,	(7,717) : 338,	(8,717) : 697,	(9,717) : 390,	(10,717) : 341,	(11,717) : 370,	(12,717) : 463,	(13,717) : 223,	(14,717) : 386,	(15,717) : 237,	(16,717) : 464,	(17,717) : 321,	(18,717) : 346,	(19,717) : 419,	(20,717) : 342,	(21,717) : 118,	(22,717) : 295,	(23,717) : 341,	(24,717) : 322,	(25,717) : 639,	(26,717) : 208,	(27,717) : 218,	(28,717) : 340,	(29,717) : 194,	(30,717) : 346,	(31,717) : 332,	(32,717) : 262,	(33,717) : 348,	(34,717) : 304,	(35,717) : 477,	(36,717) : 452,	(37,717) : 395,	(38,717) : 360,	(39,717) : 368,	(40,717) : 663,
(1,718) : 653,	(2,718) : 702,	(3,718) : 812,	(4,718) : 122,	(5,718) : 730,	(6,718) : 681,	(7,718) : 649,	(8,718) : 1096,	(9,718) : 850,	(10,718) : 650,	(11,718) : 703,	(12,718) : 201,	(13,718) : 487,	(14,718) : 846,	(15,718) : 238,	(16,718) : 7,	(17,718) : 154,	(18,718) : 714,	(19,718) : 242,	(20,718) : 648,	(21,718) : 387,	(22,718) : 700,	(23,718) : 708,	(24,718) : 154,	(25,718) : 1084,	(26,718) : 423,	(27,718) : 482,	(28,718) : 648,	(29,718) : 285,	(30,718) : 650,	(31,718) : 683,	(32,718) : 582,	(33,718) : 562,	(34,718) : 676,	(35,718) : 771,	(36,718) : 912,	(37,718) : 865,	(38,718) : 705,	(39,718) : 703,	(40,718) : 1112,
(1,719) : 1118,	(2,719) : 1082,	(3,719) : 609,	(4,719) : 1300,	(5,719) : 1031,	(6,719) : 1043,	(7,719) : 1109,	(8,719) : 348,	(9,719) : 765,	(10,719) : 1111,	(11,719) : 1083,	(12,719) : 1449,	(13,719) : 1135,	(14,719) : 769,	(15,719) : 1212,	(16,719) : 1414,	(17,719) : 1267,	(18,719) : 1026,	(19,719) : 1405,	(20,719) : 1115,	(21,719) : 1103,	(22,719) : 963,	(23,719) : 1027,	(24,719) : 1267,	(25,719) : 351,	(26,719) : 1161,	(27,719) : 1135,	(28,719) : 1112,	(29,719) : 1140,	(30,719) : 1118,	(31,719) : 1053,	(32,719) : 1089,	(33,719) : 1210,	(34,719) : 1021,	(35,719) : 1161,	(36,719) : 740,	(37,719) : 668,	(38,719) : 1065,	(39,719) : 1078,	(40,719) : 330,
(1,720) : 91,	(2,720) : 42,	(3,720) : 670,	(4,720) : 686,	(5,720) : 58,	(6,720) : 85,	(7,720) : 93,	(8,720) : 898,	(9,720) : 319,	(10,720) : 93,	(11,720) : 41,	(12,720) : 619,	(13,720) : 257,	(14,720) : 315,	(15,720) : 529,	(16,720) : 736,	(17,720) : 636,	(18,720) : 73,	(19,720) : 555,	(20,720) : 95,	(21,720) : 395,	(22,720) : 148,	(23,720) : 76,	(24,720) : 639,	(25,720) : 754,	(26,720) : 321,	(27,720) : 263,	(28,720) : 95,	(29,720) : 551,	(30,720) : 94,	(31,720) : 76,	(32,720) : 166,	(33,720) : 203,	(34,720) : 108,	(35,720) : 85,	(36,720) : 349,	(37,720) : 424,	(38,720) : 47,	(39,720) : 42,	(40,720) : 765,
(1,721) : 654,	(2,721) : 709,	(3,721) : 958,	(4,721) : 301,	(5,721) : 748,	(6,721) : 702,	(7,721) : 654,	(8,721) : 1247,	(9,721) : 920,	(10,721) : 654,	(11,721) : 710,	(12,721) : 127,	(13,721) : 512,	(14,721) : 916,	(15,721) : 345,	(16,721) : 197,	(17,721) : 311,	(18,721) : 735,	(19,721) : 190,	(20,721) : 652,	(21,721) : 468,	(22,721) : 741,	(23,721) : 730,	(24,721) : 312,	(25,721) : 1213,	(26,721) : 456,	(27,721) : 509,	(28,721) : 652,	(29,721) : 423,	(30,721) : 652,	(31,721) : 700,	(32,721) : 604,	(33,721) : 544,	(34,721) : 703,	(35,721) : 751,	(36,721) : 979,	(37,721) : 957,	(38,721) : 716,	(39,721) : 711,	(40,721) : 1238,
(1,722) : 430,	(2,722) : 478,	(3,722) : 668,	(4,722) : 178,	(5,722) : 504,	(6,722) : 455,	(7,722) : 426,	(8,722) : 959,	(9,722) : 634,	(10,722) : 426,	(11,722) : 479,	(12,722) : 190,	(13,722) : 262,	(14,722) : 630,	(15,722) : 63,	(16,722) : 219,	(17,722) : 134,	(18,722) : 488,	(19,722) : 159,	(20,722) : 425,	(21,722) : 172,	(22,722) : 475,	(23,722) : 482,	(24,722) : 137,	(25,722) : 915,	(26,722) : 197,	(27,722) : 256,	(28,722) : 425,	(29,722) : 154,	(30,722) : 427,	(31,722) : 457,	(32,722) : 356,	(33,722) : 348,	(34,722) : 449,	(35,722) : 552,	(36,722) : 695,	(37,722) : 662,	(38,722) : 479,	(39,722) : 479,	(40,722) : 940,
(1,723) : 310,	(2,723) : 286,	(3,723) : 370,	(4,723) : 636,	(5,723) : 245,	(6,723) : 238,	(7,723) : 299,	(8,723) : 598,	(9,723) : 120,	(10,723) : 302,	(11,723) : 287,	(12,723) : 689,	(13,723) : 327,	(14,723) : 116,	(15,723) : 495,	(16,723) : 726,	(17,723) : 588,	(18,723) : 233,	(19,723) : 633,	(20,723) : 305,	(21,723) : 349,	(22,723) : 158,	(23,723) : 231,	(24,723) : 590,	(25,723) : 471,	(26,723) : 366,	(27,723) : 328,	(28,723) : 303,	(29,723) : 464,	(30,723) : 309,	(31,723) : 250,	(32,723) : 273,	(33,723) : 393,	(34,723) : 214,	(35,723) : 389,	(36,723) : 182,	(37,723) : 153,	(38,723) : 269,	(39,723) : 282,	(40,723) : 487,
(1,724) : 54,	(2,724) : 77,	(3,724) : 607,	(4,724) : 568,	(5,724) : 102,	(6,724) : 55,	(7,724) : 43,	(8,724) : 858,	(9,724) : 325,	(10,724) : 45,	(11,724) : 78,	(12,724) : 520,	(13,724) : 143,	(14,724) : 320,	(15,724) : 412,	(16,724) : 623,	(17,724) : 519,	(18,724) : 88,	(19,724) : 457,	(20,724) : 48,	(21,724) : 276,	(22,724) : 118,	(23,724) : 83,	(24,724) : 521,	(25,724) : 735,	(26,724) : 207,	(27,724) : 148,	(28,724) : 45,	(29,724) : 433,	(30,724) : 52,	(31,724) : 54,	(32,724) : 48,	(33,724) : 136,	(34,724) : 63,	(35,724) : 180,	(36,724) : 371,	(37,724) : 410,	(38,724) : 76,	(39,724) : 77,	(40,724) : 750,
(1,725) : 305,	(2,725) : 272,	(3,725) : 407,	(4,725) : 684,	(5,725) : 225,	(6,725) : 230,	(7,725) : 296,	(8,725) : 617,	(9,725) : 72,	(10,725) : 298,	(11,725) : 273,	(12,725) : 724,	(13,725) : 351,	(14,725) : 68,	(15,725) : 540,	(16,725) : 771,	(17,725) : 636,	(18,725) : 217,	(19,725) : 666,	(20,725) : 302,	(21,725) : 392,	(22,725) : 150,	(23,725) : 217,	(24,725) : 638,	(25,725) : 474,	(26,725) : 397,	(27,725) : 353,	(28,725) : 300,	(29,725) : 514,	(30,725) : 305,	(31,725) : 241,	(32,725) : 284,	(33,725) : 401,	(34,725) : 208,	(35,725) : 366,	(36,725) : 134,	(37,725) : 145,	(38,725) : 255,	(39,725) : 269,	(40,725) : 487,
(1,726) : 157,	(2,726) : 202,	(3,726) : 794,	(4,726) : 591,	(5,726) : 255,	(6,726) : 233,	(7,726) : 166,	(8,726) : 1053,	(9,726) : 514,	(10,726) : 163,	(11,726) : 202,	(12,726) : 454,	(13,726) : 214,	(14,726) : 510,	(15,726) : 449,	(16,726) : 609,	(17,726) : 547,	(18,726) : 254,	(19,726) : 392,	(20,726) : 160,	(21,726) : 362,	(22,726) : 311,	(23,726) : 252,	(24,726) : 549,	(25,726) : 932,	(26,726) : 253,	(27,726) : 219,	(28,726) : 162,	(29,726) : 504,	(30,726) : 156,	(31,726) : 223,	(32,726) : 198,	(33,726) : 88,	(34,726) : 253,	(35,726) : 189,	(36,726) : 554,	(37,726) : 606,	(38,726) : 218,	(39,726) : 206,	(40,726) : 947,
(1,727) : 141,	(2,727) : 184,	(3,727) : 600,	(4,727) : 460,	(5,727) : 211,	(6,727) : 162,	(7,727) : 135,	(8,727) : 870,	(9,727) : 394,	(10,727) : 136,	(11,727) : 185,	(12,727) : 416,	(13,727) : 33,	(14,727) : 389,	(15,727) : 304,	(16,727) : 513,	(17,727) : 411,	(18,727) : 195,	(19,727) : 355,	(20,727) : 135,	(21,727) : 178,	(22,727) : 199,	(23,727) : 190,	(24,727) : 413,	(25,727) : 769,	(26,727) : 97,	(27,727) : 38,	(28,727) : 134,	(29,727) : 333,	(30,727) : 138,	(31,727) : 163,	(32,727) : 63,	(33,727) : 117,	(34,727) : 160,	(35,727) : 271,	(36,727) : 448,	(37,727) : 459,	(38,727) : 185,	(39,727) : 185,	(40,727) : 788,
(1,728) : 357,	(2,728) : 374,	(3,728) : 389,	(4,728) : 382,	(5,728) : 368,	(6,728) : 328,	(7,728) : 346,	(8,728) : 676,	(9,728) : 378,	(10,728) : 349,	(11,728) : 375,	(12,728) : 483,	(13,728) : 238,	(14,728) : 374,	(15,728) : 255,	(16,728) : 480,	(17,728) : 337,	(18,728) : 349,	(19,728) : 439,	(20,728) : 350,	(21,728) : 139,	(22,728) : 293,	(23,728) : 344,	(24,728) : 338,	(25,728) : 619,	(26,728) : 227,	(27,728) : 234,	(28,728) : 348,	(29,728) : 208,	(30,728) : 354,	(31,728) : 336,	(32,728) : 271,	(33,728) : 362,	(34,728) : 307,	(35,728) : 484,	(36,728) : 440,	(37,728) : 378,	(38,728) : 365,	(39,728) : 373,	(40,728) : 643,
(1,729) : 661,	(2,729) : 629,	(3,729) : 249,	(4,729) : 878,	(5,729) : 581,	(6,729) : 586,	(7,729) : 650,	(8,729) : 284,	(9,729) : 322,	(10,729) : 653,	(11,729) : 630,	(12,729) : 998,	(13,729) : 671,	(14,729) : 326,	(15,729) : 770,	(16,729) : 987,	(17,729) : 839,	(18,729) : 573,	(19,729) : 950,	(20,729) : 657,	(21,729) : 648,	(22,729) : 505,	(23,729) : 574,	(24,729) : 839,	(25,729) : 118,	(26,729) : 699,	(27,729) : 671,	(28,729) : 654,	(29,729) : 707,	(30,729) : 660,	(31,729) : 597,	(32,729) : 626,	(33,729) : 747,	(34,729) : 563,	(35,729) : 719,	(36,729) : 317,	(37,729) : 213,	(38,729) : 612,	(39,729) : 625,	(40,729) : 134,
(1,730) : 214,	(2,730) : 248,	(3,730) : 539,	(4,730) : 403,	(5,730) : 263,	(6,730) : 214,	(7,730) : 205,	(8,730) : 817,	(9,730) : 391,	(10,730) : 207,	(11,730) : 249,	(12,730) : 404,	(13,730) : 70,	(14,730) : 387,	(15,730) : 247,	(16,730) : 470,	(17,730) : 353,	(18,730) : 244,	(19,730) : 347,	(20,730) : 207,	(21,730) : 105,	(22,730) : 223,	(23,730) : 239,	(24,730) : 356,	(25,730) : 731,	(26,730) : 81,	(27,730) : 66,	(28,730) : 206,	(29,730) : 262,	(30,730) : 211,	(31,730) : 219,	(32,730) : 126,	(33,730) : 195,	(34,730) : 203,	(35,730) : 346,	(36,730) : 450,	(37,730) : 438,	(38,730) : 245,	(39,730) : 248,	(40,730) : 751,
(1,731) : 339,	(2,731) : 322,	(3,731) : 326,	(4,731) : 601,	(5,731) : 286,	(6,731) : 272,	(7,731) : 328,	(8,731) : 569,	(9,731) : 165,	(10,731) : 331,	(11,731) : 323,	(12,731) : 673,	(13,731) : 329,	(14,731) : 162,	(15,731) : 467,	(16,731) : 696,	(17,731) : 555,	(18,731) : 272,	(19,731) : 620,	(20,731) : 334,	(21,731) : 326,	(22,731) : 196,	(23,731) : 270,	(24,731) : 557,	(25,731) : 457,	(26,731) : 360,	(27,731) : 329,	(28,731) : 332,	(29,731) : 428,	(30,731) : 338,	(31,731) : 284,	(32,731) : 290,	(33,731) : 410,	(34,731) : 247,	(35,731) : 430,	(36,731) : 225,	(37,731) : 164,	(38,731) : 306,	(39,731) : 319,	(40,731) : 476,
(1,732) : 164,	(2,732) : 148,	(3,732) : 789,	(4,732) : 752,	(5,732) : 177,	(6,732) : 199,	(7,732) : 173,	(8,732) : 1014,	(9,732) : 427,	(10,732) : 171,	(11,732) : 146,	(12,732) : 646,	(13,732) : 329,	(14,732) : 423,	(15,732) : 599,	(16,732) : 787,	(17,732) : 704,	(18,732) : 192,	(19,732) : 583,	(20,732) : 171,	(21,732) : 479,	(22,732) : 267,	(23,732) : 194,	(24,732) : 707,	(25,732) : 864,	(26,732) : 388,	(27,732) : 335,	(28,732) : 172,	(29,732) : 635,	(30,732) : 167,	(31,732) : 188,	(32,732) : 253,	(33,732) : 232,	(34,732) : 224,	(35,732) : 35,	(36,732) : 448,	(37,732) : 536,	(38,732) : 161,	(39,732) : 150,	(40,732) : 873,
(1,733) : 285,	(2,733) : 279,	(3,733) : 375,	(4,733) : 535,	(5,733) : 254,	(6,733) : 228,	(7,733) : 273,	(8,733) : 635,	(9,733) : 220,	(10,733) : 276,	(11,733) : 280,	(12,733) : 594,	(13,733) : 251,	(14,733) : 216,	(15,733) : 394,	(16,733) : 625,	(17,733) : 487,	(18,733) : 236,	(19,733) : 541,	(20,733) : 279,	(21,733) : 250,	(22,733) : 165,	(23,733) : 233,	(24,733) : 489,	(25,733) : 534,	(26,733) : 280,	(27,733) : 251,	(28,733) : 277,	(29,733) : 364,	(30,733) : 283,	(31,733) : 239,	(32,733) : 223,	(33,733) : 341,	(34,733) : 202,	(35,733) : 391,	(36,733) : 283,	(37,733) : 242,	(38,733) : 265,	(39,733) : 276,	(40,733) : 554,
(1,734) : 247,	(2,734) : 269,	(3,734) : 473,	(4,734) : 416,	(5,734) : 271,	(6,734) : 226,	(7,734) : 237,	(8,734) : 751,	(9,734) : 350,	(10,734) : 239,	(11,734) : 271,	(12,734) : 452,	(13,734) : 133,	(14,734) : 346,	(15,734) : 265,	(16,734) : 495,	(17,734) : 366,	(18,734) : 252,	(19,734) : 399,	(20,734) : 240,	(21,734) : 115,	(22,734) : 211,	(23,734) : 246,	(24,734) : 368,	(25,734) : 668,	(26,734) : 143,	(27,734) : 130,	(28,734) : 238,	(29,734) : 256,	(30,734) : 244,	(31,734) : 234,	(32,734) : 160,	(33,734) : 252,	(34,734) : 209,	(35,734) : 376,	(36,734) : 411,	(37,734) : 384,	(38,734) : 262,	(39,734) : 268,	(40,734) : 689,
(1,735) : 154,	(2,735) : 208,	(3,735) : 751,	(4,735) : 527,	(5,735) : 257,	(6,735) : 224,	(7,735) : 159,	(8,735) : 1018,	(9,735) : 502,	(10,735) : 158,	(11,735) : 208,	(12,735) : 402,	(13,735) : 156,	(14,735) : 497,	(15,735) : 384,	(16,735) : 549,	(17,735) : 482,	(18,735) : 252,	(19,735) : 339,	(20,735) : 154,	(21,735) : 300,	(22,735) : 296,	(23,735) : 248,	(24,735) : 485,	(25,735) : 906,	(26,735) : 190,	(27,735) : 161,	(28,735) : 156,	(29,735) : 440,	(30,735) : 152,	(31,735) : 217,	(32,735) : 163,	(33,735) : 42,	(34,735) : 240,	(35,735) : 228,	(36,735) : 548,	(37,735) : 585,	(38,735) : 220,	(39,735) : 211,	(40,735) : 923,
(1,736) : 734,	(2,736) : 716,	(3,736) : 106,	(4,736) : 797,	(5,736) : 676,	(6,736) : 666,	(7,736) : 723,	(8,736) : 187,	(9,736) : 452,	(10,736) : 725,	(11,736) : 717,	(12,736) : 963,	(13,736) : 696,	(14,736) : 454,	(15,736) : 720,	(16,736) : 912,	(17,736) : 766,	(18,736) : 663,	(19,736) : 925,	(20,736) : 728,	(21,736) : 628,	(22,736) : 588,	(23,736) : 662,	(24,736) : 766,	(25,736) : 208,	(26,736) : 706,	(27,736) : 694,	(28,736) : 726,	(29,736) : 642,	(30,736) : 733,	(31,736) : 678,	(32,736) : 678,	(33,736) : 796,	(34,736) : 641,	(35,736) : 819,	(36,736) : 471,	(37,736) : 341,	(38,736) : 699,	(39,736) : 712,	(40,736) : 238,
(1,737) : 113,	(2,737) : 164,	(3,737) : 742,	(4,737) : 567,	(5,737) : 216,	(6,737) : 188,	(7,737) : 121,	(8,737) : 1002,	(9,737) : 469,	(10,737) : 119,	(11,737) : 164,	(12,737) : 451,	(13,737) : 169,	(14,737) : 465,	(15,737) : 419,	(16,737) : 594,	(17,737) : 520,	(18,737) : 212,	(19,737) : 388,	(20,737) : 115,	(21,737) : 321,	(22,737) : 264,	(23,737) : 209,	(24,737) : 523,	(25,737) : 883,	(26,737) : 215,	(27,737) : 175,	(28,737) : 117,	(29,737) : 468,	(30,737) : 112,	(31,737) : 179,	(32,737) : 146,	(33,737) : 44,	(34,737) : 206,	(35,737) : 178,	(36,737) : 512,	(37,737) : 558,	(38,737) : 178,	(39,737) : 167,	(40,737) : 898,
(1,738) : 531,	(2,738) : 586,	(3,738) : 854,	(4,738) : 251,	(5,738) : 623,	(6,738) : 577,	(7,738) : 531,	(8,738) : 1145,	(9,738) : 795,	(10,738) : 531,	(11,738) : 586,	(12,738) : 8,	(13,738) : 386,	(14,738) : 791,	(15,738) : 242,	(16,738) : 189,	(17,738) : 241,	(18,738) : 610,	(19,738) : 68,	(20,738) : 528,	(21,738) : 348,	(22,738) : 615,	(23,738) : 605,	(24,738) : 243,	(25,738) : 1099,	(26,738) : 330,	(27,738) : 383,	(28,738) : 528,	(29,738) : 329,	(30,738) : 529,	(31,738) : 576,	(32,738) : 478,	(33,738) : 424,	(34,738) : 578,	(35,738) : 633,	(36,738) : 854,	(37,738) : 835,	(38,738) : 592,	(39,738) : 587,	(40,738) : 1124,
(1,739) : 417,	(2,739) : 376,	(3,739) : 390,	(4,739) : 789,	(5,739) : 325,	(6,739) : 341,	(7,739) : 408,	(8,739) : 547,	(9,739) : 59,	(10,739) : 410,	(11,739) : 377,	(12,739) : 843,	(13,739) : 472,	(14,739) : 64,	(15,739) : 651,	(16,739) : 881,	(17,739) : 742,	(18,739) : 321,	(19,739) : 786,	(20,739) : 414,	(21,739) : 505,	(22,739) : 264,	(23,739) : 323,	(24,739) : 744,	(25,739) : 381,	(26,739) : 517,	(27,739) : 474,	(28,739) : 412,	(29,739) : 616,	(30,739) : 417,	(31,739) : 350,	(32,739) : 404,	(33,739) : 518,	(34,739) : 321,	(35,739) : 456,	(36,739) : 59,	(37,739) : 74,	(38,739) : 359,	(39,739) : 373,	(40,739) : 390,
(1,740) : 173,	(2,740) : 212,	(3,740) : 822,	(4,740) : 627,	(5,740) : 264,	(6,740) : 248,	(7,740) : 183,	(8,740) : 1077,	(9,740) : 529,	(10,740) : 181,	(11,740) : 211,	(12,740) : 483,	(13,740) : 250,	(14,740) : 524,	(15,740) : 485,	(16,740) : 642,	(17,740) : 583,	(18,740) : 266,	(19,740) : 422,	(20,740) : 177,	(21,740) : 399,	(22,740) : 329,	(23,740) : 265,	(24,740) : 585,	(25,740) : 952,	(26,740) : 290,	(27,740) : 255,	(28,740) : 180,	(29,740) : 541,	(30,740) : 174,	(31,740) : 238,	(32,740) : 226,	(33,740) : 123,	(34,740) : 271,	(35,740) : 178,	(36,740) : 565,	(37,740) : 623,	(38,740) : 229,	(39,740) : 216,	(40,740) : 966,
(1,741) : 541,	(2,741) : 596,	(3,741) : 889,	(4,741) : 288,	(5,741) : 636,	(6,741) : 591,	(7,741) : 541,	(8,741) : 1179,	(9,741) : 818,	(10,741) : 541,	(11,741) : 597,	(12,741) : 32,	(13,741) : 404,	(14,741) : 813,	(15,741) : 278,	(16,741) : 219,	(17,741) : 280,	(18,741) : 623,	(19,741) : 80,	(20,741) : 539,	(21,741) : 377,	(22,741) : 633,	(23,741) : 618,	(24,741) : 281,	(25,741) : 1130,	(26,741) : 350,	(27,741) : 400,	(28,741) : 539,	(29,741) : 366,	(30,741) : 539,	(31,741) : 589,	(32,741) : 493,	(33,741) : 431,	(34,741) : 593,	(35,741) : 637,	(36,741) : 876,	(37,741) : 861,	(38,741) : 604,	(39,741) : 598,	(40,741) : 1154,
(1,742) : 447,	(2,742) : 495,	(3,742) : 674,	(4,742) : 162,	(5,742) : 521,	(6,742) : 472,	(7,742) : 443,	(8,742) : 964,	(9,742) : 648,	(10,742) : 443,	(11,742) : 495,	(12,742) : 185,	(13,742) : 278,	(14,742) : 644,	(15,742) : 62,	(16,742) : 203,	(17,742) : 119,	(18,742) : 504,	(19,742) : 160,	(20,742) : 442,	(21,742) : 185,	(22,742) : 491,	(23,742) : 499,	(24,742) : 122,	(25,742) : 923,	(26,742) : 214,	(27,742) : 273,	(28,742) : 441,	(29,742) : 152,	(30,742) : 444,	(31,742) : 474,	(32,742) : 373,	(33,742) : 365,	(34,742) : 466,	(35,742) : 569,	(36,742) : 709,	(37,742) : 674,	(38,742) : 496,	(39,742) : 495,	(40,742) : 948,
(1,743) : 558,	(2,743) : 604,	(3,743) : 689,	(4,743) : 46,	(5,743) : 626,	(6,743) : 577,	(7,743) : 553,	(8,743) : 975,	(9,743) : 728,	(10,743) : 554,	(11,743) : 605,	(12,743) : 224,	(13,743) : 388,	(14,743) : 724,	(15,743) : 119,	(16,743) : 121,	(17,743) : 30,	(18,743) : 609,	(19,743) : 231,	(20,743) : 553,	(21,743) : 269,	(22,743) : 588,	(23,743) : 603,	(24,743) : 30,	(25,743) : 958,	(26,743) : 324,	(27,743) : 382,	(28,743) : 552,	(29,743) : 158,	(30,743) : 555,	(31,743) : 580,	(32,743) : 481,	(33,743) : 481,	(34,743) : 569,	(35,743) : 683,	(36,743) : 790,	(37,743) : 739,	(38,743) : 604,	(39,743) : 604,	(40,743) : 985,
(1,744) : 92,	(2,744) : 68,	(3,744) : 712,	(4,744) : 691,	(5,744) : 103,	(6,744) : 119,	(7,744) : 100,	(8,744) : 943,	(9,744) : 365,	(10,744) : 98,	(11,744) : 66,	(12,744) : 608,	(13,744) : 263,	(14,744) : 361,	(15,744) : 536,	(16,744) : 735,	(17,744) : 642,	(18,744) : 115,	(19,744) : 544,	(20,744) : 98,	(21,744) : 409,	(22,744) : 190,	(23,744) : 117,	(24,744) : 645,	(25,744) : 799,	(26,744) : 326,	(27,744) : 269,	(28,744) : 99,	(29,744) : 565,	(30,744) : 95,	(31,744) : 108,	(32,744) : 179,	(33,744) : 187,	(34,744) : 144,	(35,744) : 45,	(36,744) : 393,	(37,744) : 470,	(38,744) : 82,	(39,744) : 70,	(40,744) : 810,
(1,745) : 233,	(2,745) : 260,	(3,745) : 495,	(4,745) : 411,	(5,745) : 266,	(6,745) : 219,	(7,745) : 224,	(8,745) : 773,	(9,745) : 363,	(10,745) : 226,	(11,745) : 261,	(12,745) : 436,	(13,745) : 112,	(14,745) : 358,	(15,745) : 258,	(16,745) : 487,	(17,745) : 361,	(18,745) : 247,	(19,745) : 381,	(20,745) : 227,	(21,745) : 108,	(22,745) : 212,	(23,745) : 241,	(24,745) : 363,	(25,745) : 689,	(26,745) : 122,	(27,745) : 108,	(28,745) : 225,	(29,745) : 257,	(30,745) : 231,	(31,745) : 226,	(32,745) : 145,	(33,745) : 232,	(34,745) : 204,	(35,745) : 364,	(36,745) : 423,	(37,745) : 401,	(38,745) : 254,	(39,745) : 259,	(40,745) : 710,
(1,746) : 567,	(2,746) : 611,	(3,746) : 671,	(4,746) : 34,	(5,746) : 632,	(6,746) : 583,	(7,746) : 561,	(8,746) : 956,	(9,746) : 723,	(10,746) : 562,	(11,746) : 612,	(12,746) : 250,	(13,746) : 396,	(14,746) : 719,	(15,746) : 123,	(16,746) : 136,	(17,746) : 17,	(18,746) : 614,	(19,746) : 256,	(20,746) : 561,	(21,746) : 270,	(22,746) : 589,	(23,746) : 608,	(24,746) : 14,	(25,746) : 943,	(26,746) : 333,	(27,746) : 390,	(28,746) : 561,	(29,746) : 146,	(30,746) : 564,	(31,746) : 586,	(32,746) : 487,	(33,746) : 493,	(34,746) : 573,	(35,746) : 693,	(36,746) : 785,	(37,746) : 730,	(38,746) : 611,	(39,746) : 611,	(40,746) : 970,
(1,747) : 913,	(2,747) : 880,	(3,747) : 402,	(4,747) : 1089,	(5,747) : 831,	(6,747) : 838,	(7,747) : 903,	(8,747) : 190,	(9,747) : 567,	(10,747) : 905,	(11,747) : 881,	(12,747) : 1234,	(13,747) : 922,	(14,747) : 571,	(15,747) : 997,	(16,747) : 1203,	(17,747) : 1055,	(18,747) : 824,	(19,747) : 1190,	(20,747) : 909,	(21,747) : 887,	(22,747) : 757,	(23,747) : 825,	(24,747) : 1055,	(25,747) : 137,	(26,747) : 946,	(27,747) : 921,	(28,747) : 907,	(29,747) : 926,	(30,747) : 913,	(31,747) : 849,	(32,747) : 879,	(33,747) : 1000,	(34,747) : 816,	(35,747) : 965,	(36,747) : 550,	(37,747) : 463,	(38,747) : 862,	(39,747) : 876,	(40,747) : 121,
(1,748) : 130,	(2,748) : 176,	(3,748) : 618,	(4,748) : 470,	(5,748) : 207,	(6,748) : 159,	(7,748) : 125,	(8,748) : 887,	(9,748) : 402,	(10,748) : 126,	(11,748) : 177,	(12,748) : 416,	(13,748) : 42,	(14,748) : 397,	(15,748) : 315,	(16,748) : 519,	(17,748) : 421,	(18,748) : 193,	(19,748) : 353,	(20,748) : 124,	(21,748) : 193,	(22,748) : 204,	(23,748) : 187,	(24,748) : 423,	(25,748) : 783,	(26,748) : 105,	(27,748) : 48,	(28,748) : 124,	(29,748) : 348,	(30,748) : 127,	(31,748) : 159,	(32,748) : 61,	(33,748) : 98,	(34,748) : 160,	(35,748) : 258,	(36,748) : 455,	(37,748) : 471,	(38,748) : 179,	(39,748) : 177,	(40,748) : 802,
(1,749) : 481,	(2,749) : 446,	(3,749) : 313,	(4,749) : 787,	(5,749) : 397,	(6,749) : 405,	(7,749) : 471,	(8,749) : 461,	(9,749) : 137,	(10,749) : 473,	(11,749) : 447,	(12,749) : 867,	(13,749) : 512,	(14,749) : 140,	(15,749) : 658,	(16,749) : 886,	(17,749) : 742,	(18,749) : 390,	(19,749) : 814,	(20,749) : 477,	(21,749) : 520,	(22,749) : 325,	(23,749) : 391,	(24,749) : 744,	(25,749) : 302,	(26,749) : 550,	(27,749) : 513,	(28,749) : 475,	(29,749) : 613,	(30,749) : 481,	(31,749) : 416,	(32,749) : 456,	(33,749) : 574,	(34,749) : 384,	(35,749) : 533,	(36,749) : 143,	(37,749) : 32,	(38,749) : 429,	(39,749) : 442,	(40,749) : 313,
(1,750) : 690,	(2,750) : 744,	(3,750) : 960,	(4,750) : 286,	(5,750) : 781,	(6,750) : 734,	(7,750) : 689,	(8,750) : 1248,	(9,750) : 943,	(10,750) : 689,	(11,750) : 744,	(12,750) : 162,	(13,750) : 542,	(14,750) : 939,	(15,750) : 353,	(16,750) : 173,	(17,750) : 304,	(18,750) : 767,	(19,750) : 226,	(20,750) : 687,	(21,750) : 485,	(22,750) : 770,	(23,750) : 762,	(24,750) : 304,	(25,750) : 1221,	(26,750) : 483,	(27,750) : 538,	(28,750) : 687,	(29,750) : 425,	(30,750) : 687,	(31,750) : 733,	(32,750) : 635,	(33,750) : 582,	(34,750) : 734,	(35,750) : 791,	(36,750) : 1003,	(37,750) : 975,	(38,750) : 750,	(39,750) : 746,	(40,750) : 1247,
(1,751) : 269,	(2,751) : 294,	(3,751) : 472,	(4,751) : 389,	(5,751) : 298,	(6,751) : 252,	(7,751) : 260,	(8,751) : 754,	(9,751) : 373,	(10,751) : 262,	(11,751) : 296,	(12,751) : 435,	(13,751) : 142,	(14,751) : 368,	(15,751) : 240,	(16,751) : 471,	(17,751) : 340,	(18,751) : 278,	(19,751) : 383,	(20,751) : 263,	(21,751) : 91,	(22,751) : 238,	(23,751) : 273,	(24,751) : 342,	(25,751) : 677,	(26,751) : 139,	(27,751) : 138,	(28,751) : 261,	(29,751) : 229,	(30,751) : 266,	(31,751) : 260,	(32,751) : 181,	(33,751) : 265,	(34,751) : 236,	(35,751) : 400,	(36,751) : 434,	(37,751) : 401,	(38,751) : 288,	(39,751) : 293,	(40,751) : 700,
(1,752) : 720,	(2,752) : 775,	(3,752) : 1015,	(4,752) : 343,	(5,752) : 815,	(6,752) : 769,	(7,752) : 720,	(8,752) : 1303,	(9,752) : 987,	(10,752) : 720,	(11,752) : 776,	(12,752) : 194,	(13,752) : 579,	(14,752) : 982,	(15,752) : 405,	(16,752) : 230,	(17,752) : 361,	(18,752) : 802,	(19,752) : 257,	(20,752) : 718,	(21,752) : 532,	(22,752) : 808,	(23,752) : 796,	(24,752) : 361,	(25,752) : 1274,	(26,752) : 523,	(27,752) : 576,	(28,752) : 718,	(29,752) : 479,	(30,752) : 718,	(31,752) : 767,	(32,752) : 671,	(33,752) : 610,	(34,752) : 770,	(35,752) : 815,	(36,752) : 1046,	(37,752) : 1022,	(38,752) : 782,	(39,752) : 777,	(40,752) : 1299,
(1,753) : 301,	(2,753) : 265,	(3,753) : 425,	(4,753) : 699,	(5,753) : 217,	(6,753) : 225,	(7,753) : 292,	(8,753) : 628,	(9,753) : 63,	(10,753) : 294,	(11,753) : 266,	(12,753) : 733,	(13,753) : 357,	(14,753) : 59,	(15,753) : 554,	(16,753) : 784,	(17,753) : 651,	(18,753) : 209,	(19,753) : 675,	(20,753) : 298,	(21,753) : 405,	(22,753) : 147,	(23,753) : 210,	(24,753) : 652,	(25,753) : 482,	(26,753) : 405,	(27,753) : 359,	(28,753) : 296,	(29,753) : 530,	(30,753) : 301,	(31,753) : 235,	(32,753) : 286,	(33,753) : 400,	(34,753) : 205,	(35,753) : 355,	(36,753) : 122,	(37,753) : 152,	(38,753) : 248,	(39,753) : 262,	(40,753) : 494,
(1,754) : 638,	(2,754) : 689,	(3,754) : 844,	(4,754) : 164,	(5,754) : 721,	(6,754) : 672,	(7,754) : 635,	(8,754) : 1131,	(9,754) : 857,	(10,754) : 636,	(11,754) : 690,	(12,754) : 154,	(13,754) : 477,	(14,754) : 852,	(15,754) : 249,	(16,754) : 55,	(17,754) : 185,	(18,754) : 705,	(19,754) : 203,	(20,754) : 634,	(21,754) : 393,	(22,754) : 697,	(23,754) : 700,	(24,754) : 185,	(25,754) : 1111,	(26,754) : 414,	(27,754) : 472,	(28,754) : 634,	(29,754) : 311,	(30,754) : 635,	(31,754) : 673,	(32,754) : 572,	(33,754) : 541,	(34,754) : 668,	(35,754) : 751,	(36,754) : 918,	(37,754) : 879,	(38,754) : 693,	(39,754) : 691,	(40,754) : 1137,
(1,755) : 482,	(2,755) : 523,	(3,755) : 599,	(4,755) : 128,	(5,755) : 541,	(6,755) : 492,	(7,755) : 476,	(8,755) : 887,	(9,755) : 628,	(10,755) : 477,	(11,755) : 524,	(12,755) : 273,	(13,755) : 311,	(14,755) : 624,	(15,755) : 46,	(16,755) : 222,	(17,755) : 79,	(18,755) : 523,	(19,755) : 256,	(20,755) : 476,	(21,755) : 176,	(22,755) : 495,	(23,755) : 517,	(24,755) : 81,	(25,755) : 861,	(26,755) : 251,	(27,755) : 305,	(28,755) : 475,	(29,755) : 63,	(30,755) : 479,	(31,755) : 496,	(32,755) : 399,	(33,755) : 419,	(34,755) : 481,	(35,755) : 611,	(36,755) : 690,	(37,755) : 637,	(38,755) : 522,	(39,755) : 523,	(40,755) : 888,
(1,756) : 101,	(2,756) : 74,	(3,756) : 717,	(4,756) : 700,	(5,756) : 106,	(6,756) : 125,	(7,756) : 108,	(8,756) : 946,	(9,756) : 366,	(10,756) : 107,	(11,756) : 73,	(12,756) : 616,	(13,756) : 272,	(14,756) : 361,	(15,756) : 545,	(16,756) : 744,	(17,756) : 651,	(18,756) : 119,	(19,756) : 552,	(20,756) : 107,	(21,756) : 418,	(22,756) : 195,	(23,756) : 121,	(24,756) : 654,	(25,756) : 801,	(26,756) : 335,	(27,756) : 278,	(28,756) : 108,	(29,756) : 574,	(30,756) : 104,	(31,756) : 114,	(32,756) : 188,	(33,756) : 196,	(34,756) : 150,	(35,756) : 39,	(36,756) : 392,	(37,756) : 471,	(38,756) : 87,	(39,756) : 76,	(40,756) : 812,
(1,757) : 568,	(2,757) : 536,	(3,757) : 248,	(4,757) : 812,	(5,757) : 489,	(6,757) : 493,	(7,757) : 558,	(8,757) : 364,	(9,757) : 233,	(10,757) : 560,	(11,757) : 537,	(12,757) : 918,	(13,757) : 582,	(14,757) : 236,	(15,757) : 696,	(16,757) : 918,	(17,757) : 771,	(18,757) : 481,	(19,757) : 868,	(20,757) : 564,	(21,757) : 568,	(22,757) : 412,	(23,757) : 481,	(24,757) : 772,	(25,757) : 210,	(26,757) : 612,	(27,757) : 582,	(28,757) : 562,	(29,757) : 640,	(30,757) : 567,	(31,757) : 504,	(32,757) : 534,	(33,757) : 655,	(34,757) : 470,	(35,757) : 628,	(36,757) : 237,	(37,757) : 121,	(38,757) : 519,	(39,757) : 533,	(40,757) : 226,
(1,758) : 770,	(2,758) : 733,	(3,758) : 366,	(4,758) : 1014,	(5,758) : 683,	(6,758) : 694,	(7,758) : 760,	(8,758) : 284,	(9,758) : 416,	(10,758) : 763,	(11,758) : 734,	(12,758) : 1132,	(13,758) : 796,	(14,758) : 421,	(15,758) : 906,	(16,758) : 1123,	(17,758) : 975,	(18,758) : 678,	(19,758) : 1083,	(20,758) : 767,	(21,758) : 782,	(22,758) : 615,	(23,758) : 679,	(24,758) : 976,	(25,758) : 90,	(26,758) : 828,	(27,758) : 797,	(28,758) : 764,	(29,758) : 844,	(30,758) : 770,	(31,758) : 704,	(32,758) : 745,	(33,758) : 864,	(34,758) : 673,	(35,758) : 813,	(36,758) : 394,	(37,758) : 321,	(38,758) : 716,	(39,758) : 729,	(40,758) : 68,
(1,759) : 880,	(2,759) : 845,	(3,759) : 393,	(4,759) : 1074,	(5,759) : 796,	(6,759) : 805,	(7,759) : 870,	(8,759) : 211,	(9,759) : 531,	(10,759) : 872,	(11,759) : 846,	(12,759) : 1212,	(13,759) : 893,	(14,759) : 535,	(15,759) : 978,	(16,759) : 1186,	(17,759) : 1038,	(18,759) : 790,	(19,759) : 1166,	(20,759) : 876,	(21,759) : 864,	(22,759) : 724,	(23,759) : 790,	(24,759) : 1039,	(25,759) : 110,	(26,759) : 920,	(27,759) : 893,	(28,759) : 874,	(29,759) : 909,	(30,759) : 880,	(31,759) : 815,	(32,759) : 848,	(33,759) : 969,	(34,759) : 782,	(35,759) : 929,	(36,759) : 512,	(37,759) : 429,	(38,759) : 828,	(39,759) : 842,	(40,759) : 89,
(1,760) : 604,	(2,760) : 652,	(3,760) : 749,	(4,760) : 65,	(5,760) : 677,	(6,760) : 628,	(7,760) : 600,	(8,760) : 1034,	(9,760) : 788,	(10,760) : 600,	(11,760) : 652,	(12,760) : 204,	(13,760) : 435,	(14,760) : 784,	(15,760) : 176,	(16,760) : 59,	(17,760) : 90,	(18,760) : 660,	(19,760) : 229,	(20,760) : 599,	(21,760) : 327,	(22,760) : 643,	(23,760) : 654,	(24,760) : 89,	(25,760) : 1020,	(26,760) : 371,	(27,760) : 430,	(28,760) : 598,	(29,760) : 220,	(30,760) : 601,	(31,760) : 630,	(32,760) : 530,	(33,760) : 519,	(34,760) : 621,	(35,760) : 725,	(36,760) : 850,	(37,760) : 801,	(38,760) : 653,	(39,760) : 652,	(40,760) : 1047,
(1,761) : 332,	(2,761) : 383,	(3,761) : 684,	(4,761) : 287,	(5,761) : 416,	(6,761) : 368,	(7,761) : 329,	(8,761) : 971,	(9,761) : 580,	(10,761) : 330,	(11,761) : 384,	(12,761) : 215,	(13,761) : 173,	(14,761) : 576,	(15,761) : 153,	(16,761) : 315,	(17,761) : 244,	(18,761) : 401,	(19,761) : 158,	(20,761) : 328,	(21,761) : 156,	(22,761) : 401,	(23,761) : 395,	(24,761) : 246,	(25,761) : 905,	(26,761) : 114,	(27,761) : 169,	(28,761) : 328,	(29,761) : 229,	(30,761) : 329,	(31,761) : 368,	(32,761) : 268,	(33,761) : 241,	(34,761) : 366,	(35,761) : 449,	(36,761) : 639,	(37,761) : 625,	(38,761) : 387,	(39,761) : 384,	(40,761) : 928,
(1,762) : 584,	(2,762) : 639,	(3,762) : 925,	(4,762) : 302,	(5,762) : 680,	(6,762) : 634,	(7,762) : 585,	(8,762) : 1215,	(9,762) : 861,	(10,762) : 584,	(11,762) : 640,	(12,762) : 68,	(13,762) : 448,	(14,762) : 856,	(15,762) : 311,	(16,762) : 218,	(17,762) : 301,	(18,762) : 667,	(19,762) : 124,	(20,762) : 582,	(21,762) : 418,	(22,762) : 677,	(23,762) : 662,	(24,762) : 302,	(25,762) : 1170,	(26,762) : 394,	(27,762) : 444,	(28,762) : 582,	(29,762) : 397,	(30,762) : 582,	(31,762) : 632,	(32,762) : 537,	(33,762) : 473,	(34,762) : 637,	(35,762) : 678,	(36,762) : 919,	(37,762) : 903,	(38,762) : 647,	(39,762) : 641,	(40,762) : 1194,
(1,763) : 118,	(2,763) : 66,	(3,763) : 668,	(4,763) : 709,	(5,763) : 60,	(6,763) : 99,	(7,763) : 119,	(8,763) : 890,	(9,763) : 305,	(10,763) : 119,	(11,763) : 65,	(12,763) : 647,	(13,763) : 282,	(14,763) : 301,	(15,763) : 553,	(16,763) : 761,	(17,763) : 659,	(18,763) : 78,	(19,763) : 583,	(20,763) : 121,	(21,763) : 415,	(22,763) : 149,	(23,763) : 83,	(24,763) : 662,	(25,763) : 741,	(26,763) : 346,	(27,763) : 287,	(28,763) : 121,	(29,763) : 572,	(30,763) : 121,	(31,763) : 93,	(32,763) : 189,	(33,763) : 231,	(34,763) : 119,	(35,763) : 94,	(36,763) : 330,	(37,763) : 412,	(38,763) : 66,	(39,763) : 65,	(40,763) : 751,
(1,764) : 563,	(2,764) : 606,	(3,764) : 646,	(4,764) : 49,	(5,764) : 624,	(6,764) : 576,	(7,764) : 557,	(8,764) : 930,	(9,764) : 707,	(10,764) : 558,	(11,764) : 607,	(12,764) : 272,	(13,764) : 392,	(14,764) : 703,	(15,764) : 119,	(16,764) : 161,	(17,764) : 25,	(18,764) : 606,	(19,764) : 275,	(20,764) : 558,	(21,764) : 260,	(22,764) : 579,	(23,764) : 601,	(24,764) : 23,	(25,764) : 919,	(26,764) : 330,	(27,764) : 386,	(28,764) : 557,	(29,764) : 126,	(30,764) : 560,	(31,764) : 580,	(32,764) : 482,	(33,764) : 494,	(34,764) : 565,	(35,764) : 691,	(36,764) : 770,	(37,764) : 711,	(38,764) : 605,	(39,764) : 606,	(40,764) : 947,
(1,765) : 263,	(2,765) : 282,	(3,765) : 448,	(4,765) : 422,	(5,765) : 279,	(6,765) : 236,	(7,765) : 252,	(8,765) : 727,	(9,765) : 338,	(10,765) : 255,	(11,765) : 283,	(12,765) : 470,	(13,765) : 158,	(14,765) : 333,	(15,765) : 275,	(16,765) : 506,	(17,765) : 373,	(18,765) : 259,	(19,765) : 418,	(20,765) : 256,	(21,765) : 127,	(22,765) : 212,	(23,765) : 254,	(24,765) : 375,	(25,765) : 645,	(26,765) : 167,	(27,765) : 155,	(28,765) : 254,	(29,765) : 258,	(30,765) : 260,	(31,765) : 244,	(32,765) : 178,	(33,765) : 275,	(34,765) : 217,	(35,765) : 390,	(36,765) : 399,	(37,765) : 366,	(38,765) : 273,	(39,765) : 280,	(40,765) : 667,
(1,766) : 812,	(2,766) : 779,	(3,766) : 331,	(4,766) : 1005,	(5,766) : 730,	(6,766) : 737,	(7,766) : 802,	(8,766) : 202,	(9,766) : 467,	(10,766) : 804,	(11,766) : 779,	(12,766) : 1141,	(13,766) : 822,	(14,766) : 470,	(15,766) : 907,	(16,766) : 1117,	(17,766) : 969,	(18,766) : 723,	(19,766) : 1095,	(20,766) : 808,	(21,766) : 792,	(22,766) : 656,	(23,766) : 723,	(24,766) : 970,	(25,766) : 39,	(26,766) : 848,	(27,766) : 822,	(28,766) : 805,	(29,766) : 839,	(30,766) : 811,	(31,766) : 747,	(32,766) : 778,	(33,766) : 899,	(34,766) : 714,	(35,766) : 865,	(36,766) : 452,	(37,766) : 362,	(38,766) : 761,	(39,766) : 775,	(40,766) : 21,
(1,767) : 1155,	(2,767) : 1118,	(3,767) : 652,	(4,767) : 1342,	(5,767) : 1067,	(6,767) : 1079,	(7,767) : 1146,	(8,767) : 389,	(9,767) : 801,	(10,767) : 1148,	(11,767) : 1119,	(12,767) : 1491,	(13,767) : 1175,	(14,767) : 805,	(15,767) : 1254,	(16,767) : 1457,	(17,767) : 1309,	(18,767) : 1063,	(19,767) : 1447,	(20,767) : 1152,	(21,767) : 1144,	(22,767) : 1000,	(23,767) : 1064,	(24,767) : 1310,	(25,767) : 391,	(26,767) : 1202,	(27,767) : 1174,	(28,767) : 1150,	(29,767) : 1182,	(30,767) : 1155,	(31,767) : 1090,	(32,767) : 1128,	(33,767) : 1248,	(34,767) : 1058,	(35,767) : 1196,	(36,767) : 774,	(37,767) : 706,	(38,767) : 1101,	(39,767) : 1114,	(40,767) : 370,
(1,768) : 616,	(2,768) : 669,	(3,768) : 896,	(4,768) : 242,	(5,768) : 706,	(6,768) : 659,	(7,768) : 615,	(8,768) : 1185,	(9,768) : 868,	(10,768) : 615,	(11,768) : 670,	(12,768) : 91,	(13,768) : 466,	(14,768) : 863,	(15,768) : 284,	(16,768) : 146,	(17,768) : 250,	(18,768) : 692,	(19,768) : 154,	(20,768) : 612,	(21,768) : 411,	(22,768) : 694,	(23,768) : 687,	(24,768) : 250,	(25,768) : 1152,	(26,768) : 407,	(27,768) : 462,	(28,768) : 612,	(29,768) : 361,	(30,768) : 613,	(31,768) : 658,	(32,768) : 559,	(33,768) : 510,	(34,768) : 659,	(35,768) : 719,	(36,768) : 927,	(37,768) : 901,	(38,768) : 675,	(39,768) : 671,	(40,768) : 1178,
(1,769) : 334,	(2,769) : 297,	(3,769) : 408,	(4,769) : 720,	(5,769) : 248,	(6,769) : 258,	(7,769) : 325,	(8,769) : 601,	(9,769) : 35,	(10,769) : 327,	(11,769) : 298,	(12,769) : 761,	(13,769) : 388,	(14,769) : 31,	(15,769) : 577,	(16,769) : 808,	(17,769) : 672,	(18,769) : 241,	(19,769) : 704,	(20,769) : 331,	(21,769) : 429,	(22,769) : 180,	(23,769) : 242,	(24,769) : 674,	(25,769) : 451,	(26,769) : 434,	(27,769) : 389,	(28,769) : 329,	(29,769) : 549,	(30,769) : 334,	(31,769) : 268,	(32,769) : 318,	(33,769) : 433,	(34,769) : 238,	(35,769) : 385,	(36,769) : 97,	(37,769) : 121,	(38,769) : 280,	(39,769) : 293,	(40,769) : 462,
(1,770) : 14,	(2,770) : 49,	(3,770) : 668,	(4,770) : 613,	(5,770) : 100,	(6,770) : 78,	(7,770) : 23,	(8,770) : 913,	(9,770) : 359,	(10,770) : 21,	(11,770) : 49,	(12,770) : 539,	(13,770) : 184,	(14,770) : 354,	(15,770) : 458,	(16,770) : 659,	(17,770) : 564,	(18,770) : 98,	(19,770) : 475,	(20,770) : 20,	(21,770) : 330,	(22,770) : 159,	(23,770) : 95,	(24,770) : 567,	(25,770) : 782,	(26,770) : 247,	(27,770) : 191,	(28,770) : 22,	(29,770) : 487,	(30,770) : 17,	(31,770) : 68,	(32,770) : 102,	(33,770) : 125,	(34,770) : 102,	(35,770) : 118,	(36,770) : 398,	(37,770) : 453,	(38,770) : 62,	(39,770) : 52,	(40,770) : 796,
(1,771) : 828,	(2,771) : 798,	(3,771) : 300,	(4,771) : 985,	(5,771) : 751,	(6,771) : 755,	(7,771) : 818,	(8,771) : 145,	(9,771) : 493,	(10,771) : 821,	(11,771) : 799,	(12,771) : 1132,	(13,771) : 827,	(14,771) : 496,	(15,771) : 895,	(16,771) : 1099,	(17,771) : 951,	(18,771) : 743,	(19,771) : 1089,	(20,771) : 824,	(21,771) : 787,	(22,771) : 674,	(23,771) : 743,	(24,771) : 952,	(25,771) : 60,	(26,771) : 849,	(27,771) : 826,	(28,771) : 822,	(29,771) : 823,	(30,771) : 828,	(31,771) : 765,	(32,771) : 790,	(33,771) : 910,	(34,771) : 731,	(35,771) : 889,	(36,771) : 485,	(37,771) : 383,	(38,771) : 781,	(39,771) : 795,	(40,771) : 72,
(1,772) : 230,	(2,772) : 257,	(3,772) : 497,	(4,772) : 414,	(5,772) : 263,	(6,772) : 216,	(7,772) : 221,	(8,772) : 774,	(9,772) : 361,	(10,772) : 223,	(11,772) : 258,	(12,772) : 437,	(13,772) : 109,	(14,772) : 357,	(15,772) : 260,	(16,772) : 489,	(17,772) : 364,	(18,772) : 243,	(19,772) : 382,	(20,772) : 224,	(21,772) : 111,	(22,772) : 209,	(23,772) : 238,	(24,772) : 366,	(25,772) : 689,	(26,772) : 121,	(27,772) : 106,	(28,772) : 222,	(29,772) : 260,	(30,772) : 227,	(31,772) : 223,	(32,772) : 142,	(33,772) : 229,	(34,772) : 201,	(35,772) : 361,	(36,772) : 421,	(37,772) : 401,	(38,772) : 251,	(39,772) : 256,	(40,772) : 710,
(1,773) : 1090,	(2,773) : 1055,	(3,773) : 572,	(4,773) : 1262,	(5,773) : 1005,	(6,773) : 1015,	(7,773) : 1080,	(8,773) : 309,	(9,773) : 739,	(10,773) : 1083,	(11,773) : 1056,	(12,773) : 1414,	(13,773) : 1103,	(14,773) : 743,	(15,773) : 1176,	(16,773) : 1377,	(17,773) : 1230,	(18,773) : 999,	(19,773) : 1371,	(20,773) : 1086,	(21,773) : 1068,	(22,773) : 935,	(23,773) : 1000,	(24,773) : 1230,	(25,773) : 318,	(26,773) : 1128,	(27,773) : 1103,	(28,773) : 1084,	(29,773) : 1103,	(30,773) : 1090,	(31,773) : 1025,	(32,773) : 1060,	(33,773) : 1180,	(34,773) : 993,	(35,773) : 1136,	(36,773) : 716,	(37,773) : 640,	(38,773) : 1038,	(39,773) : 1051,	(40,773) : 300,
(1,774) : 445,	(2,774) : 411,	(3,774) : 327,	(4,774) : 764,	(5,774) : 362,	(6,774) : 370,	(7,774) : 436,	(8,774) : 491,	(9,774) : 105,	(10,774) : 438,	(11,774) : 412,	(12,774) : 837,	(13,774) : 479,	(14,774) : 108,	(15,774) : 633,	(16,774) : 862,	(17,774) : 719,	(18,774) : 355,	(19,774) : 783,	(20,774) : 442,	(21,774) : 492,	(22,774) : 290,	(23,774) : 356,	(24,774) : 721,	(25,774) : 336,	(26,774) : 518,	(27,774) : 480,	(28,774) : 440,	(29,774) : 590,	(30,774) : 445,	(31,774) : 380,	(32,774) : 421,	(33,774) : 539,	(34,774) : 348,	(35,774) : 499,	(36,774) : 122,	(37,774) : 11,	(38,774) : 393,	(39,774) : 407,	(40,774) : 348,
(1,775) : 528,	(2,775) : 581,	(3,775) : 835,	(4,775) : 229,	(5,775) : 618,	(6,775) : 571,	(7,775) : 526,	(8,775) : 1125,	(9,775) : 783,	(10,775) : 527,	(11,775) : 582,	(12,775) : 30,	(13,775) : 378,	(14,775) : 779,	(15,775) : 221,	(16,775) : 173,	(17,775) : 219,	(18,775) : 604,	(19,775) : 70,	(20,775) : 524,	(21,775) : 332,	(22,775) : 607,	(23,775) : 598,	(24,775) : 220,	(25,775) : 1081,	(26,775) : 320,	(27,775) : 374,	(28,775) : 524,	(29,775) : 307,	(30,775) : 525,	(31,775) : 570,	(32,775) : 471,	(33,775) : 423,	(34,775) : 571,	(35,775) : 632,	(36,775) : 843,	(37,775) : 821,	(38,775) : 587,	(39,775) : 583,	(40,775) : 1106,
(1,776) : 382,	(2,776) : 429,	(3,776) : 646,	(4,776) : 222,	(5,776) : 455,	(6,776) : 406,	(7,776) : 377,	(8,776) : 936,	(9,776) : 590,	(10,776) : 378,	(11,776) : 430,	(12,776) : 218,	(13,776) : 213,	(14,776) : 585,	(15,776) : 80,	(16,776) : 268,	(17,776) : 175,	(18,776) : 439,	(19,776) : 176,	(20,776) : 377,	(21,776) : 132,	(22,776) : 427,	(23,776) : 433,	(24,776) : 178,	(25,776) : 884,	(26,776) : 149,	(27,776) : 207,	(28,776) : 376,	(29,776) : 160,	(30,776) : 379,	(31,776) : 408,	(32,776) : 307,	(33,776) : 303,	(34,776) : 400,	(35,776) : 505,	(36,776) : 650,	(37,776) : 622,	(38,776) : 431,	(39,776) : 430,	(40,776) : 908,
(1,777) : 705,	(2,777) : 757,	(3,777) : 896,	(4,777) : 206,	(5,777) : 788,	(6,777) : 740,	(7,777) : 703,	(8,777) : 1180,	(9,777) : 923,	(10,777) : 703,	(11,777) : 758,	(12,777) : 205,	(13,777) : 545,	(14,777) : 919,	(15,777) : 313,	(16,777) : 90,	(17,777) : 238,	(18,777) : 773,	(19,777) : 261,	(20,777) : 701,	(21,777) : 459,	(22,777) : 765,	(23,777) : 767,	(24,777) : 237,	(25,777) : 1168,	(26,777) : 482,	(27,777) : 540,	(28,777) : 701,	(29,777) : 368,	(30,777) : 702,	(31,777) : 741,	(32,777) : 640,	(33,777) : 607,	(34,777) : 736,	(35,777) : 817,	(36,777) : 985,	(37,777) : 943,	(38,777) : 761,	(39,777) : 758,	(40,777) : 1195,
(1,778) : 681,	(2,778) : 650,	(3,778) : 241,	(4,778) : 884,	(5,778) : 603,	(6,778) : 607,	(7,778) : 670,	(8,778) : 261,	(9,778) : 345,	(10,778) : 673,	(11,778) : 651,	(12,778) : 1009,	(13,778) : 687,	(14,778) : 348,	(15,778) : 779,	(16,778) : 994,	(17,778) : 846,	(18,778) : 594,	(19,778) : 963,	(20,778) : 677,	(21,778) : 660,	(22,778) : 526,	(23,778) : 595,	(24,778) : 846,	(25,778) : 98,	(26,778) : 714,	(27,778) : 687,	(28,778) : 674,	(29,778) : 715,	(30,778) : 680,	(31,778) : 617,	(32,778) : 645,	(33,778) : 766,	(34,778) : 583,	(35,778) : 741,	(36,778) : 341,	(37,778) : 235,	(38,778) : 633,	(39,778) : 646,	(40,778) : 118,
(1,779) : 766,	(2,779) : 744,	(3,779) : 168,	(4,779) : 859,	(5,779) : 702,	(6,779) : 697,	(7,779) : 756,	(8,779) : 141,	(9,779) : 464,	(10,779) : 758,	(11,779) : 745,	(12,779) : 1020,	(13,779) : 740,	(14,779) : 467,	(15,779) : 778,	(16,779) : 974,	(17,779) : 827,	(18,779) : 691,	(19,779) : 980,	(20,779) : 761,	(21,779) : 681,	(22,779) : 617,	(23,779) : 690,	(24,779) : 827,	(25,779) : 152,	(26,779) : 754,	(27,779) : 739,	(28,779) : 759,	(29,779) : 702,	(30,779) : 766,	(31,779) : 708,	(32,779) : 716,	(33,779) : 836,	(34,779) : 672,	(35,779) : 845,	(36,779) : 474,	(37,779) : 350,	(38,779) : 727,	(39,779) : 741,	(40,779) : 181,
(1,780) : 1072,	(2,780) : 1038,	(3,780) : 538,	(4,780) : 1229,	(5,780) : 989,	(6,780) : 997,	(7,780) : 1062,	(8,780) : 270,	(9,780) : 724,	(10,780) : 1064,	(11,780) : 1039,	(12,780) : 1385,	(13,780) : 1080,	(14,780) : 728,	(15,780) : 1145,	(16,780) : 1344,	(17,780) : 1197,	(18,780) : 982,	(19,780) : 1342,	(20,780) : 1068,	(21,780) : 1040,	(22,780) : 916,	(23,780) : 983,	(24,780) : 1198,	(25,780) : 295,	(26,780) : 1103,	(27,780) : 1079,	(28,780) : 1066,	(29,780) : 1071,	(30,780) : 1072,	(31,780) : 1007,	(32,780) : 1038,	(33,780) : 1159,	(34,780) : 975,	(35,780) : 1122,	(36,780) : 704,	(37,780) : 622,	(38,780) : 1021,	(39,780) : 1034,	(40,780) : 280,
(1,781) : 585,	(2,781) : 628,	(3,781) : 665,	(4,781) : 26,	(5,781) : 647,	(6,781) : 599,	(7,781) : 579,	(8,781) : 948,	(9,781) : 731,	(10,781) : 580,	(11,781) : 629,	(12,781) : 271,	(13,781) : 414,	(14,781) : 728,	(15,781) : 140,	(16,781) : 143,	(17,781) : 36,	(18,781) : 629,	(19,781) : 280,	(20,781) : 579,	(21,781) : 283,	(22,781) : 603,	(23,781) : 624,	(24,781) : 33,	(25,781) : 940,	(26,781) : 351,	(27,781) : 408,	(28,781) : 579,	(29,781) : 149,	(30,781) : 582,	(31,781) : 602,	(32,781) : 504,	(33,781) : 514,	(34,781) : 588,	(35,781) : 712,	(36,781) : 794,	(37,781) : 735,	(38,781) : 627,	(39,781) : 628,	(40,781) : 968,
(1,782) : 539,	(2,782) : 588,	(3,782) : 733,	(4,782) : 103,	(5,782) : 615,	(6,782) : 566,	(7,782) : 535,	(8,782) : 1022,	(9,782) : 739,	(10,782) : 536,	(11,782) : 589,	(12,782) : 159,	(13,782) : 372,	(14,782) : 735,	(15,782) : 130,	(16,782) : 108,	(17,782) : 89,	(18,782) : 599,	(19,782) : 170,	(20,782) : 534,	(21,782) : 275,	(22,782) : 585,	(23,782) : 593,	(24,782) : 91,	(25,782) : 995,	(26,782) : 308,	(27,782) : 367,	(28,782) : 534,	(29,782) : 198,	(30,782) : 536,	(31,782) : 568,	(32,782) : 467,	(33,782) : 452,	(34,782) : 560,	(35,782) : 659,	(36,782) : 801,	(37,782) : 760,	(38,782) : 590,	(39,782) : 589,	(40,782) : 1021,
(1,783) : 769,	(2,783) : 748,	(3,783) : 157,	(4,783) : 848,	(5,783) : 706,	(6,783) : 700,	(7,783) : 758,	(8,783) : 142,	(9,783) : 472,	(10,783) : 760,	(11,783) : 749,	(12,783) : 1012,	(13,783) : 738,	(14,783) : 475,	(15,783) : 769,	(16,783) : 963,	(17,783) : 817,	(18,783) : 694,	(19,783) : 973,	(20,783) : 763,	(21,783) : 675,	(22,783) : 620,	(23,783) : 694,	(24,783) : 817,	(25,783) : 171,	(26,783) : 751,	(27,783) : 736,	(28,783) : 761,	(29,783) : 692,	(30,783) : 768,	(31,783) : 711,	(32,783) : 716,	(33,783) : 836,	(34,783) : 675,	(35,783) : 849,	(36,783) : 484,	(37,783) : 359,	(38,783) : 731,	(39,783) : 744,	(40,783) : 200,
(1,784) : 592,	(2,784) : 634,	(3,784) : 653,	(4,784) : 39,	(5,784) : 652,	(6,784) : 603,	(7,784) : 586,	(8,784) : 936,	(9,784) : 729,	(10,784) : 587,	(11,784) : 635,	(12,784) : 289,	(13,784) : 421,	(14,784) : 726,	(15,784) : 148,	(16,784) : 156,	(17,784) : 49,	(18,784) : 634,	(19,784) : 297,	(20,784) : 586,	(21,784) : 286,	(22,784) : 605,	(23,784) : 628,	(24,784) : 46,	(25,784) : 930,	(26,784) : 359,	(27,784) : 415,	(28,784) : 585,	(29,784) : 146,	(30,784) : 588,	(31,784) : 607,	(32,784) : 510,	(33,784) : 523,	(34,784) : 592,	(35,784) : 720,	(36,784) : 792,	(37,784) : 730,	(38,784) : 633,	(39,784) : 634,	(40,784) : 958,
(1,785) : 893,	(2,785) : 872,	(3,785) : 262,	(4,785) : 940,	(5,785) : 830,	(6,785) : 824,	(7,785) : 882,	(8,785) : 40,	(9,785) : 592,	(10,785) : 885,	(11,785) : 874,	(12,785) : 1119,	(13,785) : 860,	(14,785) : 595,	(15,785) : 874,	(16,785) : 1057,	(17,785) : 913,	(18,785) : 819,	(19,785) : 1084,	(20,785) : 888,	(21,785) : 789,	(22,785) : 745,	(23,785) : 818,	(24,785) : 913,	(25,785) : 220,	(26,785) : 869,	(27,785) : 857,	(28,785) : 886,	(29,785) : 793,	(30,785) : 892,	(31,785) : 836,	(32,785) : 840,	(33,785) : 959,	(34,785) : 799,	(35,785) : 973,	(36,785) : 600,	(37,785) : 479,	(38,785) : 856,	(39,785) : 869,	(40,785) : 240,
(1,786) : 188,	(2,786) : 240,	(3,786) : 662,	(4,786) : 423,	(5,786) : 276,	(6,786) : 230,	(7,786) : 186,	(8,786) : 939,	(9,786) : 474,	(10,786) : 186,	(11,786) : 241,	(12,786) : 345,	(13,786) : 60,	(14,786) : 470,	(15,786) : 273,	(16,786) : 460,	(17,786) : 376,	(18,786) : 263,	(19,786) : 281,	(20,786) : 184,	(21,786) : 183,	(22,786) : 278,	(23,786) : 258,	(24,786) : 379,	(25,786) : 845,	(26,786) : 72,	(27,786) : 61,	(28,786) : 184,	(29,786) : 323,	(30,786) : 185,	(31,786) : 229,	(32,786) : 134,	(33,786) : 103,	(34,786) : 234,	(35,786) : 305,	(36,786) : 528,	(37,786) : 539,	(38,786) : 246,	(39,786) : 242,	(40,786) : 864,
(1,787) : 643,	(2,787) : 692,	(3,787) : 793,	(4,787) : 103,	(5,787) : 719,	(6,787) : 670,	(7,787) : 640,	(8,787) : 1077,	(9,787) : 835,	(10,787) : 640,	(11,787) : 693,	(12,787) : 206,	(13,787) : 476,	(14,787) : 831,	(15,787) : 223,	(16,787) : 15,	(17,787) : 136,	(18,787) : 703,	(19,787) : 243,	(20,787) : 639,	(21,787) : 373,	(22,787) : 687,	(23,787) : 697,	(24,787) : 136,	(25,787) : 1066,	(26,787) : 412,	(27,787) : 471,	(28,787) : 638,	(29,787) : 268,	(30,787) : 640,	(31,787) : 672,	(32,787) : 571,	(33,787) : 554,	(34,787) : 664,	(35,787) : 763,	(36,787) : 897,	(37,787) : 848,	(38,787) : 694,	(39,787) : 693,	(40,787) : 1093,
(1,788) : 711,	(2,788) : 767,	(3,788) : 1033,	(4,788) : 370,	(5,788) : 808,	(6,788) : 763,	(7,788) : 712,	(8,788) : 1322,	(9,788) : 988,	(10,788) : 712,	(11,788) : 767,	(12,788) : 193,	(13,788) : 576,	(14,788) : 984,	(15,788) : 420,	(16,788) : 260,	(17,788) : 384,	(18,788) : 795,	(19,788) : 252,	(20,788) : 709,	(21,788) : 540,	(22,788) : 806,	(23,788) : 790,	(24,788) : 384,	(25,788) : 1286,	(26,788) : 522,	(27,788) : 573,	(28,788) : 709,	(29,788) : 498,	(30,788) : 709,	(31,788) : 760,	(32,788) : 666,	(33,788) : 598,	(34,788) : 766,	(35,788) : 802,	(36,788) : 1047,	(37,788) : 1028,	(38,788) : 775,	(39,788) : 769,	(40,788) : 1312,
(1,789) : 1089,	(2,789) : 1052,	(3,789) : 587,	(4,789) : 1277,	(5,789) : 1002,	(6,789) : 1013,	(7,789) : 1079,	(8,789) : 332,	(9,789) : 736,	(10,789) : 1082,	(11,789) : 1053,	(12,789) : 1424,	(13,789) : 1107,	(14,789) : 740,	(15,789) : 1187,	(16,789) : 1391,	(17,789) : 1243,	(18,789) : 997,	(19,789) : 1379,	(20,789) : 1085,	(21,789) : 1077,	(22,789) : 933,	(23,789) : 998,	(24,789) : 1244,	(25,789) : 324,	(26,789) : 1134,	(27,789) : 1107,	(28,789) : 1083,	(29,789) : 1116,	(30,789) : 1089,	(31,789) : 1023,	(32,789) : 1061,	(33,789) : 1181,	(34,789) : 992,	(35,789) : 1132,	(36,789) : 710,	(37,789) : 639,	(38,789) : 1035,	(39,789) : 1049,	(40,789) : 303,
(1,790) : 913,	(2,790) : 887,	(3,790) : 316,	(4,790) : 1005,	(5,790) : 842,	(6,790) : 841,	(7,790) : 902,	(8,790) : 44,	(9,790) : 592,	(10,790) : 905,	(11,790) : 888,	(12,790) : 1173,	(13,790) : 894,	(14,790) : 595,	(15,790) : 930,	(16,790) : 1121,	(17,790) : 975,	(18,790) : 833,	(19,790) : 1134,	(20,790) : 908,	(21,790) : 836,	(22,790) : 760,	(23,790) : 832,	(24,790) : 975,	(25,790) : 176,	(26,790) : 909,	(27,790) : 892,	(28,790) : 906,	(29,790) : 852,	(30,790) : 912,	(31,790) : 852,	(32,790) : 866,	(33,790) : 987,	(34,790) : 817,	(35,790) : 983,	(36,790) : 590,	(37,790) : 479,	(38,790) : 870,	(39,790) : 884,	(40,790) : 187,
(1,791) : 664,	(2,791) : 714,	(3,791) : 834,	(4,791) : 144,	(5,791) : 743,	(6,791) : 694,	(7,791) : 660,	(8,791) : 1118,	(9,791) : 867,	(10,791) : 661,	(11,791) : 714,	(12,791) : 196,	(13,791) : 499,	(14,791) : 863,	(15,791) : 256,	(16,791) : 27,	(17,791) : 176,	(18,791) : 727,	(19,791) : 241,	(20,791) : 659,	(21,791) : 404,	(22,791) : 715,	(23,791) : 721,	(24,791) : 175,	(25,791) : 1106,	(26,791) : 435,	(27,791) : 494,	(28,791) : 659,	(29,791) : 306,	(30,791) : 661,	(31,791) : 695,	(32,791) : 594,	(33,791) : 570,	(34,791) : 689,	(35,791) : 780,	(36,791) : 929,	(37,791) : 884,	(38,791) : 717,	(39,791) : 715,	(40,791) : 1133,
(1,792) : 713,	(2,792) : 680,	(3,792) : 278,	(4,792) : 927,	(5,792) : 632,	(6,792) : 638,	(7,792) : 703,	(8,792) : 254,	(9,792) : 370,	(10,792) : 706,	(11,792) : 681,	(12,792) : 1051,	(13,792) : 726,	(14,792) : 374,	(15,792) : 822,	(16,792) : 1037,	(17,792) : 888,	(18,792) : 625,	(19,792) : 1004,	(20,792) : 709,	(21,792) : 702,	(22,792) : 558,	(23,792) : 625,	(24,792) : 889,	(25,792) : 67,	(26,792) : 753,	(27,792) : 725,	(28,792) : 707,	(29,792) : 758,	(30,792) : 713,	(31,792) : 649,	(32,792) : 680,	(33,792) : 801,	(34,792) : 616,	(35,792) : 768,	(36,792) : 360,	(37,792) : 264,	(38,792) : 663,	(39,792) : 677,	(40,792) : 80,
(1,793) : 270,	(2,793) : 239,	(3,793) : 429,	(4,793) : 662,	(5,793) : 194,	(6,793) : 196,	(7,793) : 261,	(8,793) : 647,	(9,793) : 103,	(10,793) : 263,	(11,793) : 240,	(12,793) : 693,	(13,793) : 318,	(14,793) : 98,	(15,793) : 516,	(16,793) : 747,	(17,793) : 614,	(18,793) : 184,	(19,793) : 635,	(20,793) : 267,	(21,793) : 366,	(22,793) : 115,	(23,793) : 184,	(24,793) : 616,	(25,793) : 508,	(26,793) : 365,	(27,793) : 319,	(28,793) : 265,	(29,793) : 494,	(30,793) : 270,	(31,793) : 206,	(32,793) : 249,	(33,793) : 365,	(34,793) : 173,	(35,793) : 336,	(36,793) : 161,	(37,793) : 180,	(38,793) : 222,	(39,793) : 235,	(40,793) : 522,
(1,794) : 625,	(2,794) : 673,	(3,794) : 769,	(4,794) : 81,	(5,794) : 699,	(6,794) : 650,	(7,794) : 621,	(8,794) : 1053,	(9,794) : 811,	(10,794) : 621,	(11,794) : 674,	(12,794) : 207,	(13,794) : 457,	(14,794) : 807,	(15,794) : 200,	(16,794) : 37,	(17,794) : 112,	(18,794) : 682,	(19,794) : 238,	(20,794) : 620,	(21,794) : 350,	(22,794) : 665,	(23,794) : 676,	(24,794) : 111,	(25,794) : 1042,	(26,794) : 393,	(27,794) : 451,	(28,794) : 619,	(29,794) : 243,	(30,794) : 622,	(31,794) : 652,	(32,794) : 551,	(33,794) : 538,	(34,794) : 643,	(35,794) : 745,	(36,794) : 873,	(37,794) : 824,	(38,794) : 675,	(39,794) : 674,	(40,794) : 1069,
(1,795) : 653,	(2,795) : 708,	(3,795) : 964,	(4,795) : 309,	(5,795) : 748,	(6,795) : 702,	(7,795) : 653,	(8,795) : 1253,	(9,795) : 922,	(10,795) : 653,	(11,795) : 709,	(12,795) : 128,	(13,795) : 513,	(14,795) : 917,	(15,795) : 351,	(16,795) : 206,	(17,795) : 319,	(18,795) : 735,	(19,795) : 190,	(20,795) : 651,	(21,795) : 471,	(22,795) : 742,	(23,795) : 730,	(24,795) : 319,	(25,795) : 1217,	(26,795) : 457,	(27,795) : 509,	(28,795) : 651,	(29,795) : 430,	(30,795) : 651,	(31,795) : 700,	(32,795) : 604,	(33,795) : 543,	(34,795) : 704,	(35,795) : 749,	(36,795) : 981,	(37,795) : 960,	(38,795) : 715,	(39,795) : 710,	(40,795) : 1242,
(1,796) : 549,	(2,796) : 593,	(3,796) : 660,	(4,796) : 51,	(5,796) : 614,	(6,796) : 565,	(7,796) : 544,	(8,796) : 946,	(9,796) : 706,	(10,796) : 545,	(11,796) : 594,	(12,796) : 247,	(13,796) : 378,	(14,796) : 702,	(15,796) : 105,	(16,796) : 149,	(17,796) : 2,	(18,796) : 596,	(19,796) : 249,	(20,796) : 544,	(21,796) : 252,	(22,796) : 571,	(23,796) : 590,	(24,796) : 4,	(25,796) : 930,	(26,796) : 315,	(27,796) : 372,	(28,796) : 543,	(29,796) : 131,	(30,796) : 546,	(31,796) : 568,	(32,796) : 469,	(33,796) : 476,	(34,796) : 555,	(35,796) : 676,	(36,796) : 768,	(37,796) : 714,	(38,796) : 593,	(39,796) : 594,	(40,796) : 957,
(1,797) : 475,	(2,797) : 530,	(3,797) : 845,	(4,797) : 291,	(5,797) : 570,	(6,797) : 525,	(7,797) : 475,	(8,797) : 1134,	(9,797) : 755,	(10,797) : 475,	(11,797) : 531,	(12,797) : 59,	(13,797) : 339,	(14,797) : 750,	(15,797) : 245,	(16,797) : 247,	(17,797) : 271,	(18,797) : 558,	(19,797) : 23,	(20,797) : 473,	(21,797) : 324,	(22,797) : 569,	(23,797) : 553,	(24,797) : 273,	(25,797) : 1078,	(26,797) : 287,	(27,797) : 336,	(28,797) : 473,	(29,797) : 335,	(30,797) : 473,	(31,797) : 523,	(32,797) : 428,	(33,797) : 365,	(34,797) : 528,	(35,797) : 572,	(36,797) : 813,	(37,797) : 802,	(38,797) : 538,	(39,797) : 532,	(40,797) : 1101,
(1,798) : 604,	(2,798) : 650,	(3,798) : 716,	(4,798) : 27,	(5,798) : 672,	(6,798) : 623,	(7,798) : 599,	(8,798) : 1000,	(9,798) : 771,	(10,798) : 600,	(11,798) : 651,	(12,798) : 237,	(13,798) : 434,	(14,798) : 767,	(15,798) : 165,	(16,798) : 91,	(17,798) : 64,	(18,798) : 655,	(19,798) : 256,	(20,798) : 599,	(21,798) : 315,	(22,798) : 633,	(23,798) : 649,	(24,798) : 63,	(25,798) : 990,	(26,798) : 370,	(27,798) : 428,	(28,798) : 598,	(29,798) : 195,	(30,798) : 601,	(31,798) : 626,	(32,798) : 527,	(33,798) : 525,	(34,798) : 615,	(35,798) : 728,	(36,798) : 833,	(37,798) : 779,	(38,798) : 650,	(39,798) : 650,	(40,798) : 1018,
(1,799) : 487,	(2,799) : 462,	(3,799) : 227,	(4,799) : 714,	(5,799) : 419,	(6,799) : 416,	(7,799) : 477,	(8,799) : 422,	(9,799) : 192,	(10,799) : 479,	(11,799) : 463,	(12,799) : 817,	(13,799) : 486,	(14,799) : 194,	(15,799) : 595,	(16,799) : 819,	(17,799) : 672,	(18,799) : 408,	(19,799) : 767,	(20,799) : 483,	(21,799) : 467,	(22,799) : 335,	(23,799) : 407,	(24,799) : 673,	(25,799) : 299,	(26,799) : 514,	(27,799) : 485,	(28,799) : 480,	(29,799) : 541,	(30,799) : 487,	(31,799) : 427,	(32,799) : 445,	(33,799) : 566,	(34,799) : 391,	(35,799) : 561,	(36,799) : 222,	(37,799) : 91,	(38,799) : 445,	(39,799) : 459,	(40,799) : 319,
(1,800) : 338,	(2,800) : 302,	(3,800) : 400,	(4,800) : 717,	(5,800) : 253,	(6,800) : 262,	(7,800) : 329,	(8,800) : 595,	(9,800) : 38,	(10,800) : 331,	(11,800) : 303,	(12,800) : 760,	(13,800) : 388,	(14,800) : 35,	(15,800) : 574,	(16,800) : 805,	(17,800) : 669,	(18,800) : 246,	(19,800) : 703,	(20,800) : 335,	(21,800) : 427,	(22,800) : 183,	(23,800) : 247,	(24,800) : 671,	(25,800) : 445,	(26,800) : 434,	(27,800) : 390,	(28,800) : 333,	(29,800) : 545,	(30,800) : 338,	(31,800) : 272,	(32,800) : 320,	(33,800) : 436,	(34,800) : 241,	(35,800) : 391,	(36,800) : 101,	(37,800) : 116,	(38,800) : 284,	(39,800) : 298,	(40,800) : 457,
(1,801) : 300,	(2,801) : 316,	(3,801) : 412,	(4,801) : 417,	(5,801) : 309,	(6,801) : 269,	(7,801) : 290,	(8,801) : 693,	(9,801) : 338,	(10,801) : 292,	(11,801) : 317,	(12,801) : 488,	(13,801) : 196,	(14,801) : 334,	(15,801) : 278,	(16,801) : 508,	(17,801) : 370,	(18,801) : 290,	(19,801) : 439,	(20,801) : 294,	(21,801) : 139,	(22,801) : 236,	(23,801) : 285,	(24,801) : 371,	(25,801) : 619,	(26,801) : 199,	(27,801) : 193,	(28,801) : 292,	(29,801) : 247,	(30,801) : 298,	(31,801) : 278,	(32,801) : 217,	(33,801) : 315,	(34,801) : 248,	(35,801) : 426,	(36,801) : 400,	(37,801) : 353,	(38,801) : 307,	(39,801) : 315,	(40,801) : 642,
(1,802) : 517,	(2,802) : 558,	(3,802) : 606,	(4,802) : 100,	(5,802) : 575,	(6,802) : 526,	(7,802) : 510,	(8,802) : 893,	(9,802) : 655,	(10,802) : 512,	(11,802) : 559,	(12,802) : 282,	(13,802) : 346,	(14,802) : 651,	(15,802) : 78,	(16,802) : 204,	(17,802) : 56,	(18,802) : 556,	(19,802) : 273,	(20,802) : 511,	(21,802) : 209,	(22,802) : 527,	(23,802) : 551,	(24,802) : 57,	(25,802) : 875,	(26,802) : 286,	(27,802) : 340,	(28,802) : 510,	(29,802) : 75,	(30,802) : 513,	(31,802) : 531,	(32,802) : 434,	(33,802) : 453,	(34,802) : 515,	(35,802) : 646,	(36,802) : 717,	(37,802) : 660,	(38,802) : 556,	(39,802) : 558,	(40,802) : 902,
(1,803) : 410,	(2,803) : 464,	(3,803) : 771,	(4,803) : 272,	(5,803) : 501,	(6,803) : 455,	(7,803) : 409,	(8,803) : 1060,	(9,803) : 678,	(10,803) : 409,	(11,803) : 465,	(12,803) : 120,	(13,803) : 265,	(14,803) : 673,	(15,803) : 187,	(16,803) : 262,	(17,803) : 240,	(18,803) : 488,	(19,803) : 60,	(20,803) : 407,	(21,803) : 247,	(22,803) : 494,	(23,803) : 483,	(24,803) : 243,	(25,803) : 1000,	(26,803) : 210,	(27,803) : 261,	(28,803) : 407,	(29,803) : 277,	(30,803) : 408,	(31,803) : 454,	(32,803) : 356,	(33,803) : 306,	(34,803) : 456,	(35,803) : 516,	(36,803) : 736,	(37,803) : 723,	(38,803) : 470,	(39,803) : 466,	(40,803) : 1024,
(1,804) : 361,	(2,804) : 321,	(3,804) : 418,	(4,804) : 758,	(5,804) : 270,	(6,804) : 286,	(7,804) : 353,	(8,804) : 594,	(9,804) : 4,	(10,804) : 355,	(11,804) : 322,	(12,804) : 799,	(13,804) : 424,	(14,804) : 9,	(15,804) : 616,	(16,804) : 847,	(17,804) : 711,	(18,804) : 266,	(19,804) : 741,	(20,804) : 359,	(21,804) : 468,	(22,804) : 210,	(23,804) : 267,	(24,804) : 712,	(25,804) : 434,	(26,804) : 472,	(27,804) : 426,	(28,804) : 357,	(29,804) : 587,	(30,804) : 362,	(31,804) : 295,	(32,804) : 352,	(33,804) : 464,	(34,804) : 267,	(35,804) : 402,	(36,804) : 59,	(37,804) : 112,	(38,804) : 304,	(39,804) : 317,	(40,804) : 443,
(1,805) : 498,	(2,805) : 540,	(3,805) : 607,	(4,805) : 112,	(5,805) : 558,	(6,805) : 509,	(7,805) : 492,	(8,805) : 895,	(9,805) : 644,	(10,805) : 494,	(11,805) : 541,	(12,805) : 271,	(13,805) : 328,	(14,805) : 640,	(15,805) : 59,	(16,805) : 208,	(17,805) : 63,	(18,805) : 540,	(19,805) : 258,	(20,805) : 493,	(21,805) : 193,	(22,805) : 512,	(23,805) : 534,	(24,805) : 65,	(25,805) : 873,	(26,805) : 267,	(27,805) : 322,	(28,805) : 492,	(29,805) : 72,	(30,805) : 495,	(31,805) : 513,	(32,805) : 416,	(33,805) : 434,	(34,805) : 498,	(35,805) : 627,	(36,805) : 706,	(37,805) : 652,	(38,805) : 539,	(39,805) : 540,	(40,805) : 899,
(1,806) : 590,	(2,806) : 641,	(3,806) : 789,	(4,806) : 122,	(5,806) : 670,	(6,806) : 621,	(7,806) : 587,	(8,806) : 1076,	(9,806) : 800,	(10,806) : 588,	(11,806) : 641,	(12,806) : 148,	(13,806) : 426,	(14,806) : 796,	(15,806) : 191,	(16,806) : 59,	(17,806) : 132,	(18,806) : 654,	(19,806) : 181,	(20,806) : 586,	(21,806) : 336,	(22,806) : 643,	(23,806) : 648,	(24,806) : 133,	(25,806) : 1054,	(26,806) : 363,	(27,806) : 421,	(28,806) : 586,	(29,806) : 254,	(30,806) : 587,	(31,806) : 622,	(32,806) : 522,	(33,806) : 498,	(34,806) : 616,	(35,806) : 707,	(36,806) : 861,	(37,806) : 821,	(38,806) : 643,	(39,806) : 642,	(40,806) : 1080,
(1,807) : 203,	(2,807) : 215,	(3,807) : 472,	(4,807) : 485,	(5,807) : 209,	(6,807) : 168,	(7,807) : 193,	(8,807) : 739,	(9,807) : 289,	(10,807) : 195,	(11,807) : 216,	(12,807) : 506,	(13,807) : 147,	(14,807) : 285,	(15,807) : 333,	(16,807) : 562,	(17,807) : 435,	(18,807) : 189,	(19,807) : 450,	(20,807) : 197,	(21,807) : 183,	(22,807) : 142,	(23,807) : 184,	(24,807) : 437,	(25,807) : 639,	(26,807) : 182,	(27,807) : 146,	(28,807) : 195,	(29,807) : 326,	(30,807) : 201,	(31,807) : 176,	(32,807) : 127,	(33,807) : 240,	(34,807) : 147,	(35,807) : 326,	(36,807) : 349,	(37,807) : 337,	(38,807) : 205,	(39,807) : 213,	(40,807) : 658,
(1,808) : 161,	(2,808) : 201,	(3,808) : 577,	(4,808) : 444,	(5,808) : 222,	(6,808) : 173,	(7,808) : 154,	(8,808) : 850,	(9,808) : 387,	(10,808) : 156,	(11,808) : 202,	(12,808) : 415,	(13,808) : 33,	(14,808) : 382,	(15,808) : 288,	(16,808) : 502,	(17,808) : 394,	(18,808) : 205,	(19,808) : 354,	(20,808) : 155,	(21,808) : 155,	(22,808) : 200,	(23,808) : 200,	(24,808) : 397,	(25,808) : 753,	(26,808) : 88,	(27,808) : 35,	(28,808) : 154,	(29,808) : 312,	(30,808) : 158,	(31,808) : 176,	(32,808) : 77,	(33,808) : 142,	(34,808) : 167,	(35,808) : 293,	(36,808) : 443,	(37,808) : 447,	(38,808) : 200,	(39,808) : 201,	(40,808) : 772,
(1,809) : 29,	(2,809) : 80,	(3,809) : 647,	(4,809) : 571,	(5,809) : 122,	(6,809) : 84,	(7,809) : 25,	(8,809) : 900,	(9,809) : 363,	(10,809) : 25,	(11,809) : 80,	(12,809) : 503,	(13,809) : 142,	(14,809) : 358,	(15,809) : 415,	(16,809) : 618,	(17,809) : 521,	(18,809) : 113,	(19,809) : 439,	(20,809) : 24,	(21,809) : 288,	(22,809) : 158,	(23,809) : 109,	(24,809) : 524,	(25,809) : 776,	(26,809) : 205,	(27,809) : 148,	(28,809) : 23,	(29,809) : 445,	(30,809) : 26,	(31,809) : 78,	(32,809) : 62,	(33,809) : 101,	(34,809) : 100,	(35,809) : 160,	(36,809) : 407,	(37,809) : 451,	(38,809) : 87,	(39,809) : 82,	(40,809) : 791,
(1,810) : 105,	(2,810) : 72,	(3,810) : 710,	(4,810) : 705,	(5,810) : 99,	(6,810) : 122,	(7,810) : 111,	(8,810) : 938,	(9,810) : 356,	(10,810) : 110,	(11,810) : 71,	(12,810) : 624,	(13,810) : 276,	(14,810) : 352,	(15,810) : 549,	(16,810) : 749,	(17,810) : 656,	(18,810) : 113,	(19,810) : 560,	(20,810) : 111,	(21,810) : 420,	(22,810) : 189,	(23,810) : 116,	(24,810) : 658,	(25,810) : 792,	(26,810) : 339,	(27,810) : 282,	(28,810) : 112,	(29,810) : 577,	(30,810) : 108,	(31,810) : 112,	(32,810) : 190,	(33,810) : 204,	(34,810) : 146,	(35,810) : 45,	(36,810) : 382,	(37,810) : 462,	(38,810) : 84,	(39,810) : 74,	(40,810) : 802,
(1,811) : 411,	(2,811) : 374,	(3,811) : 365,	(4,811) : 763,	(5,811) : 324,	(6,811) : 336,	(7,811) : 402,	(8,811) : 534,	(9,811) : 62,	(10,811) : 404,	(11,811) : 375,	(12,811) : 823,	(13,811) : 457,	(14,811) : 66,	(15,811) : 627,	(16,811) : 857,	(17,811) : 717,	(18,811) : 319,	(19,811) : 767,	(20,811) : 408,	(21,811) : 483,	(22,811) : 257,	(23,811) : 320,	(24,811) : 718,	(25,811) : 376,	(26,811) : 500,	(27,811) : 458,	(28,811) : 406,	(29,811) : 590,	(30,811) : 411,	(31,811) : 345,	(32,811) : 393,	(33,811) : 509,	(34,811) : 315,	(35,811) : 459,	(36,811) : 85,	(37,811) : 52,	(38,811) : 357,	(39,811) : 371,	(40,811) : 386,
(1,812) : 131,	(2,812) : 170,	(3,812) : 584,	(4,812) : 475,	(5,812) : 192,	(6,812) : 143,	(7,812) : 124,	(8,812) : 852,	(9,812) : 370,	(10,812) : 125,	(11,812) : 171,	(12,812) : 439,	(13,812) : 54,	(14,812) : 365,	(15,812) : 319,	(16,812) : 532,	(17,812) : 425,	(18,812) : 175,	(19,812) : 378,	(20,812) : 125,	(21,812) : 186,	(22,812) : 175,	(23,812) : 170,	(24,812) : 428,	(25,812) : 748,	(26,812) : 116,	(27,812) : 59,	(28,812) : 124,	(29,812) : 342,	(30,812) : 128,	(31,812) : 145,	(32,812) : 45,	(33,812) : 129,	(34,812) : 139,	(35,812) : 263,	(36,812) : 424,	(37,812) : 436,	(38,812) : 169,	(39,812) : 170,	(40,812) : 766,
(1,813) : 547,	(2,813) : 594,	(3,813) : 710,	(4,813) : 73,	(5,813) : 619,	(6,813) : 570,	(7,813) : 542,	(8,813) : 997,	(9,813) : 732,	(10,813) : 543,	(11,813) : 595,	(12,813) : 192,	(13,813) : 378,	(14,813) : 727,	(15,813) : 119,	(16,813) : 112,	(17,813) : 58,	(18,813) : 602,	(19,813) : 200,	(20,813) : 542,	(21,813) : 269,	(22,813) : 584,	(23,813) : 596,	(24,813) : 59,	(25,813) : 975,	(26,813) : 313,	(27,813) : 372,	(28,813) : 541,	(29,813) : 175,	(30,813) : 544,	(31,813) : 572,	(32,813) : 472,	(33,813) : 464,	(34,813) : 562,	(35,813) : 669,	(36,813) : 793,	(37,813) : 747,	(38,813) : 595,	(39,813) : 595,	(40,813) : 1002,
(1,814) : 694,	(2,814) : 686,	(3,814) : 61,	(4,814) : 672,	(5,814) : 653,	(6,814) : 635,	(7,814) : 682,	(8,814) : 302,	(9,814) : 472,	(10,814) : 685,	(11,814) : 687,	(12,814) : 855,	(13,814) : 626,	(14,814) : 473,	(15,814) : 609,	(16,814) : 789,	(17,814) : 645,	(18,814) : 638,	(19,814) : 822,	(20,814) : 688,	(21,814) : 535,	(22,814) : 563,	(23,814) : 636,	(24,814) : 645,	(25,814) : 340,	(26,814) : 626,	(27,814) : 623,	(28,814) : 686,	(29,814) : 526,	(30,814) : 692,	(31,814) : 646,	(32,814) : 626,	(33,814) : 739,	(34,814) : 609,	(35,814) : 796,	(36,814) : 506,	(37,814) : 374,	(38,814) : 670,	(39,814) : 683,	(40,814) : 371,
(1,815) : 498,	(2,815) : 540,	(3,815) : 607,	(4,815) : 112,	(5,815) : 558,	(6,815) : 509,	(7,815) : 492,	(8,815) : 895,	(9,815) : 644,	(10,815) : 494,	(11,815) : 541,	(12,815) : 271,	(13,815) : 328,	(14,815) : 640,	(15,815) : 59,	(16,815) : 208,	(17,815) : 63,	(18,815) : 540,	(19,815) : 258,	(20,815) : 493,	(21,815) : 193,	(22,815) : 512,	(23,815) : 534,	(24,815) : 65,	(25,815) : 873,	(26,815) : 267,	(27,815) : 322,	(28,815) : 492,	(29,815) : 72,	(30,815) : 495,	(31,815) : 513,	(32,815) : 416,	(33,815) : 434,	(34,815) : 498,	(35,815) : 627,	(36,815) : 706,	(37,815) : 652,	(38,815) : 539,	(39,815) : 540,	(40,815) : 899,
(1,816) : 259,	(2,816) : 231,	(3,816) : 425,	(4,816) : 638,	(5,816) : 189,	(6,816) : 185,	(7,816) : 249,	(8,816) : 652,	(9,816) : 126,	(10,816) : 251,	(11,816) : 233,	(12,816) : 670,	(13,816) : 296,	(14,816) : 121,	(15,816) : 491,	(16,816) : 722,	(17,816) : 589,	(18,816) : 178,	(19,816) : 612,	(20,816) : 255,	(21,816) : 342,	(22,816) : 104,	(23,816) : 177,	(24,816) : 591,	(25,816) : 520,	(26,816) : 342,	(27,816) : 298,	(28,816) : 252,	(29,816) : 470,	(30,816) : 258,	(31,816) : 196,	(32,816) : 231,	(33,816) : 349,	(34,816) : 162,	(35,816) : 333,	(36,816) : 185,	(37,816) : 194,	(38,816) : 214,	(39,816) : 228,	(40,816) : 534,
(1,817) : 521,	(2,817) : 575,	(3,817) : 838,	(4,817) : 239,	(5,817) : 612,	(6,817) : 565,	(7,817) : 520,	(8,817) : 1128,	(9,817) : 781,	(10,817) : 520,	(11,817) : 576,	(12,817) : 22,	(13,817) : 374,	(14,817) : 776,	(15,817) : 225,	(16,817) : 184,	(17,817) : 227,	(18,817) : 598,	(19,817) : 61,	(20,817) : 518,	(21,817) : 332,	(22,817) : 602,	(23,817) : 593,	(24,817) : 229,	(25,817) : 1083,	(26,817) : 316,	(27,817) : 370,	(28,817) : 518,	(29,817) : 312,	(30,817) : 518,	(31,817) : 564,	(32,817) : 466,	(33,817) : 415,	(34,817) : 566,	(35,817) : 625,	(36,817) : 840,	(37,817) : 819,	(38,817) : 581,	(39,817) : 577,	(40,817) : 1107,
(1,818) : 874,	(2,818) : 841,	(3,818) : 372,	(4,818) : 1055,	(5,818) : 792,	(6,818) : 799,	(7,818) : 864,	(8,818) : 186,	(9,818) : 529,	(10,818) : 867,	(11,818) : 842,	(12,818) : 1197,	(13,818) : 883,	(14,818) : 533,	(15,818) : 962,	(16,818) : 1168,	(17,818) : 1020,	(18,818) : 785,	(19,818) : 1152,	(20,818) : 870,	(21,818) : 850,	(22,818) : 719,	(23,818) : 786,	(24,818) : 1021,	(25,818) : 98,	(26,818) : 908,	(27,818) : 882,	(28,818) : 868,	(29,818) : 892,	(30,818) : 874,	(31,818) : 810,	(32,818) : 840,	(33,818) : 961,	(34,818) : 777,	(35,818) : 927,	(36,818) : 513,	(37,818) : 425,	(38,818) : 824,	(39,818) : 837,	(40,818) : 82,
(1,819) : 831,	(2,819) : 822,	(3,819) : 177,	(4,819) : 772,	(5,819) : 788,	(6,819) : 771,	(7,819) : 820,	(8,819) : 234,	(9,819) : 592,	(10,819) : 822,	(11,819) : 823,	(12,819) : 972,	(13,819) : 762,	(14,819) : 593,	(15,819) : 726,	(16,819) : 890,	(17,819) : 750,	(18,819) : 773,	(19,819) : 944,	(20,819) : 825,	(21,819) : 665,	(22,819) : 698,	(23,819) : 771,	(24,819) : 750,	(25,819) : 354,	(26,819) : 759,	(27,819) : 759,	(28,819) : 823,	(29,819) : 639,	(30,819) : 829,	(31,819) : 782,	(32,819) : 764,	(33,819) : 876,	(34,819) : 745,	(35,819) : 931,	(36,819) : 619,	(37,819) : 486,	(38,819) : 806,	(39,819) : 819,	(40,819) : 382,
(1,820) : 12,	(2,820) : 45,	(3,820) : 662,	(4,820) : 612,	(5,820) : 95,	(6,820) : 73,	(7,820) : 19,	(8,820) : 907,	(9,820) : 353,	(10,820) : 17,	(11,820) : 45,	(12,820) : 540,	(13,820) : 183,	(14,820) : 349,	(15,820) : 456,	(16,820) : 658,	(17,820) : 563,	(18,820) : 93,	(19,820) : 477,	(20,820) : 17,	(21,820) : 328,	(22,820) : 153,	(23,820) : 90,	(24,820) : 565,	(25,820) : 776,	(26,820) : 246,	(27,820) : 189,	(28,820) : 19,	(29,820) : 484,	(30,820) : 15,	(31,820) : 62,	(32,820) : 99,	(33,820) : 128,	(34,820) : 96,	(35,820) : 122,	(36,820) : 393,	(37,820) : 448,	(38,820) : 58,	(39,820) : 48,	(40,820) : 790,
(1,821) : 270,	(2,821) : 291,	(3,821) : 452,	(4,821) : 409,	(5,821) : 289,	(6,821) : 246,	(7,821) : 260,	(8,821) : 732,	(9,821) : 350,	(10,821) : 262,	(11,821) : 292,	(12,821) : 459,	(13,821) : 156,	(14,821) : 346,	(15,821) : 262,	(16,821) : 493,	(17,821) : 360,	(18,821) : 270,	(19,821) : 408,	(20,821) : 263,	(21,821) : 114,	(22,821) : 224,	(23,821) : 265,	(24,821) : 362,	(25,821) : 653,	(26,821) : 160,	(27,821) : 153,	(28,821) : 261,	(29,821) : 245,	(30,821) : 267,	(31,821) : 254,	(32,821) : 183,	(33,821) : 276,	(34,821) : 227,	(35,821) : 398,	(36,821) : 412,	(37,821) : 377,	(38,821) : 282,	(39,821) : 289,	(40,821) : 676,
(1,822) : 673,	(2,822) : 728,	(3,822) : 972,	(4,822) : 308,	(5,822) : 767,	(6,822) : 721,	(7,822) : 673,	(8,822) : 1260,	(9,822) : 938,	(10,822) : 673,	(11,822) : 728,	(12,822) : 146,	(13,822) : 531,	(14,822) : 933,	(15,822) : 360,	(16,822) : 201,	(17,822) : 321,	(18,822) : 754,	(19,822) : 209,	(20,822) : 670,	(21,822) : 484,	(22,822) : 760,	(23,822) : 748,	(24,822) : 322,	(25,822) : 1228,	(26,822) : 474,	(27,822) : 527,	(28,822) : 671,	(29,822) : 436,	(30,822) : 670,	(31,822) : 719,	(32,822) : 622,	(33,822) : 563,	(34,822) : 722,	(35,822) : 770,	(36,822) : 997,	(37,822) : 974,	(38,822) : 735,	(39,822) : 730,	(40,822) : 1253,
(1,823) : 645,	(2,823) : 695,	(3,823) : 806,	(4,823) : 117,	(5,823) : 723,	(6,823) : 674,	(7,823) : 642,	(8,823) : 1090,	(9,823) : 843,	(10,823) : 643,	(11,823) : 696,	(12,823) : 197,	(13,823) : 479,	(14,823) : 838,	(15,823) : 231,	(16,823) : 1,	(17,823) : 148,	(18,823) : 706,	(19,823) : 237,	(20,823) : 641,	(21,823) : 380,	(22,823) : 692,	(23,823) : 701,	(24,823) : 147,	(25,823) : 1078,	(26,823) : 415,	(27,823) : 474,	(28,823) : 641,	(29,823) : 279,	(30,823) : 642,	(31,823) : 676,	(32,823) : 575,	(33,823) : 555,	(34,823) : 668,	(35,823) : 763,	(36,823) : 904,	(37,823) : 858,	(38,823) : 697,	(39,823) : 696,	(40,823) : 1105,
(1,824) : 156,	(2,824) : 149,	(3,824) : 503,	(4,824) : 564,	(5,824) : 131,	(6,824) : 98,	(7,824) : 145,	(8,824) : 752,	(9,824) : 239,	(10,824) : 148,	(11,824) : 150,	(12,824) : 564,	(13,824) : 182,	(14,824) : 234,	(15,824) : 410,	(16,824) : 636,	(17,824) : 514,	(18,824) : 112,	(19,824) : 504,	(20,824) : 151,	(21,824) : 261,	(22,824) : 59,	(23,824) : 107,	(24,824) : 516,	(25,824) : 632,	(26,824) : 235,	(27,824) : 184,	(28,824) : 149,	(29,824) : 408,	(30,824) : 155,	(31,824) : 109,	(32,824) : 113,	(33,824) : 232,	(34,824) : 73,	(35,824) : 262,	(36,824) : 294,	(37,824) : 311,	(38,824) : 136,	(39,824) : 147,	(40,824) : 648,
(1,825) : 852,	(2,825) : 824,	(3,825) : 301,	(4,825) : 991,	(5,825) : 777,	(6,825) : 779,	(7,825) : 842,	(8,825) : 114,	(9,825) : 521,	(10,825) : 844,	(11,825) : 825,	(12,825) : 1144,	(13,825) : 846,	(14,825) : 524,	(15,825) : 905,	(16,825) : 1105,	(17,825) : 958,	(18,825) : 769,	(19,825) : 1102,	(20,825) : 848,	(21,825) : 800,	(22,825) : 698,	(23,825) : 769,	(24,825) : 958,	(25,825) : 93,	(26,825) : 866,	(27,825) : 845,	(28,825) : 846,	(29,825) : 831,	(30,825) : 852,	(31,825) : 790,	(32,825) : 811,	(33,825) : 932,	(34,825) : 756,	(35,825) : 916,	(36,825) : 515,	(37,825) : 410,	(38,825) : 806,	(39,825) : 820,	(40,825) : 104,
(1,826) : 666,	(2,826) : 719,	(3,826) : 894,	(4,826) : 213,	(5,826) : 752,	(6,826) : 704,	(7,826) : 664,	(8,826) : 1180,	(9,826) : 898,	(10,826) : 664,	(11,826) : 719,	(12,826) : 156,	(13,826) : 509,	(14,826) : 893,	(15,826) : 295,	(16,826) : 99,	(17,826) : 234,	(18,826) : 737,	(19,826) : 214,	(20,826) : 662,	(21,826) : 435,	(22,826) : 733,	(23,826) : 732,	(24,826) : 234,	(25,826) : 1159,	(26,826) : 447,	(27,826) : 505,	(28,826) : 662,	(29,826) : 359,	(30,826) : 663,	(31,826) : 704,	(32,826) : 604,	(33,826) : 564,	(34,826) : 702,	(35,826) : 775,	(36,826) : 959,	(37,826) : 923,	(38,826) : 723,	(39,826) : 720,	(40,826) : 1185,
(1,827) : 561,	(2,827) : 613,	(3,827) : 817,	(4,827) : 178,	(5,827) : 647,	(6,827) : 599,	(7,827) : 559,	(8,827) : 1106,	(9,827) : 796,	(10,827) : 559,	(11,827) : 614,	(12,827) : 85,	(13,827) : 404,	(14,827) : 791,	(15,827) : 205,	(16,827) : 115,	(17,827) : 177,	(18,827) : 632,	(19,827) : 124,	(20,827) : 557,	(21,827) : 335,	(22,827) : 628,	(23,827) : 626,	(24,827) : 178,	(25,827) : 1073,	(26,827) : 342,	(27,827) : 399,	(28,827) : 557,	(29,827) : 282,	(30,827) : 558,	(31,827) : 599,	(32,827) : 499,	(33,827) : 462,	(34,827) : 596,	(35,827) : 672,	(36,827) : 856,	(37,827) : 825,	(38,827) : 618,	(39,827) : 615,	(40,827) : 1099,
(1,828) : 288,	(2,828) : 327,	(3,828) : 550,	(4,828) : 321,	(5,828) : 344,	(6,828) : 296,	(7,828) : 281,	(8,828) : 836,	(9,828) : 460,	(10,828) : 283,	(11,828) : 328,	(12,828) : 342,	(13,828) : 122,	(14,828) : 455,	(15,828) : 165,	(16,828) : 390,	(17,828) : 271,	(18,828) : 326,	(19,828) : 291,	(20,828) : 282,	(21,828) : 34,	(22,828) : 304,	(23,828) : 321,	(24,828) : 274,	(25,828) : 767,	(26,828) : 78,	(27,828) : 116,	(28,828) : 281,	(29,828) : 185,	(30,828) : 285,	(31,828) : 300,	(32,828) : 203,	(33,828) : 245,	(34,828) : 285,	(35,828) : 419,	(36,828) : 521,	(37,828) : 494,	(38,828) : 325,	(39,828) : 327,	(40,828) : 790,
(1,829) : 700,	(2,829) : 751,	(3,829) : 885,	(4,829) : 195,	(5,829) : 782,	(6,829) : 733,	(7,829) : 697,	(8,829) : 1169,	(9,829) : 914,	(10,829) : 697,	(11,829) : 752,	(12,829) : 205,	(13,829) : 538,	(14,829) : 910,	(15,829) : 304,	(16,829) : 78,	(17,829) : 227,	(18,829) : 766,	(19,829) : 260,	(20,829) : 696,	(21,829) : 450,	(22,829) : 757,	(23,829) : 761,	(24,829) : 226,	(25,829) : 1157,	(26,829) : 475,	(27,829) : 533,	(28,829) : 695,	(29,829) : 357,	(30,829) : 697,	(31,829) : 734,	(32,829) : 634,	(33,829) : 602,	(34,829) : 730,	(35,829) : 812,	(36,829) : 976,	(37,829) : 933,	(38,829) : 755,	(39,829) : 752,	(40,829) : 1184,
(1,830) : 203,	(2,830) : 257,	(3,830) : 693,	(4,830) : 424,	(5,830) : 297,	(6,830) : 252,	(7,830) : 202,	(8,830) : 970,	(9,830) : 503,	(10,830) : 202,	(11,830) : 257,	(12,830) : 326,	(13,830) : 91,	(14,830) : 498,	(15,830) : 278,	(16,830) : 453,	(17,830) : 378,	(18,830) : 285,	(19,830) : 262,	(20,830) : 200,	(21,830) : 203,	(22,830) : 304,	(23,830) : 280,	(24,830) : 381,	(25,830) : 876,	(26,830) : 93,	(27,830) : 92,	(28,830) : 200,	(29,830) : 336,	(30,830) : 200,	(31,830) : 250,	(32,830) : 159,	(33,830) : 103,	(34,830) : 258,	(35,830) : 312,	(36,830) : 556,	(37,830) : 569,	(38,830) : 264,	(39,830) : 259,	(40,830) : 895,
(1,831) : 997,	(2,831) : 963,	(3,831) : 488,	(4,831) : 1177,	(5,831) : 913,	(6,831) : 922,	(7,831) : 988,	(8,831) : 246,	(9,831) : 647,	(10,831) : 990,	(11,831) : 963,	(12,831) : 1324,	(13,831) : 1010,	(14,831) : 652,	(15,831) : 1087,	(16,831) : 1291,	(17,831) : 1143,	(18,831) : 907,	(19,831) : 1280,	(20,831) : 994,	(21,831) : 977,	(22,831) : 842,	(23,831) : 908,	(24,831) : 1144,	(25,831) : 225,	(26,831) : 1036,	(27,831) : 1010,	(28,831) : 991,	(29,831) : 1015,	(30,831) : 997,	(31,831) : 933,	(32,831) : 966,	(33,831) : 1087,	(34,831) : 900,	(35,831) : 1045,	(36,831) : 626,	(37,831) : 547,	(38,831) : 945,	(39,831) : 959,	(40,831) : 206,
(1,832) : 279,	(2,832) : 268,	(3,832) : 380,	(4,832) : 564,	(5,832) : 238,	(6,832) : 217,	(7,832) : 268,	(8,832) : 632,	(9,832) : 191,	(10,832) : 271,	(11,832) : 269,	(12,832) : 617,	(13,832) : 264,	(14,832) : 187,	(15,832) : 422,	(16,832) : 653,	(17,832) : 516,	(18,832) : 221,	(19,832) : 562,	(20,832) : 274,	(21,832) : 276,	(22,832) : 147,	(23,832) : 219,	(24,832) : 518,	(25,832) : 522,	(26,832) : 298,	(27,832) : 264,	(28,832) : 271,	(29,832) : 393,	(30,832) : 278,	(31,832) : 228,	(32,832) : 225,	(33,832) : 346,	(34,832) : 191,	(35,832) : 378,	(36,832) : 253,	(37,832) : 220,	(38,832) : 253,	(39,832) : 265,	(40,832) : 541,
(1,833) : 114,	(2,833) : 65,	(3,833) : 681,	(4,833) : 709,	(5,833) : 71,	(6,833) : 105,	(7,833) : 116,	(8,833) : 905,	(9,833) : 320,	(10,833) : 116,	(11,833) : 64,	(12,833) : 641,	(13,833) : 281,	(14,833) : 316,	(15,833) : 553,	(16,833) : 759,	(17,833) : 659,	(18,833) : 88,	(19,833) : 577,	(20,833) : 118,	(21,833) : 418,	(22,833) : 161,	(23,833) : 92,	(24,833) : 662,	(25,833) : 756,	(26,833) : 345,	(27,833) : 287,	(28,833) : 118,	(29,833) : 574,	(30,833) : 116,	(31,833) : 97,	(32,833) : 189,	(33,833) : 224,	(34,833) : 127,	(35,833) : 79,	(36,833) : 346,	(37,833) : 428,	(38,833) : 69,	(39,833) : 65,	(40,833) : 766,
(1,834) : 544,	(2,834) : 590,	(3,834) : 692,	(4,834) : 64,	(5,834) : 614,	(6,834) : 565,	(7,834) : 539,	(8,834) : 979,	(9,834) : 720,	(10,834) : 540,	(11,834) : 591,	(12,834) : 209,	(13,834) : 374,	(14,834) : 716,	(15,834) : 109,	(16,834) : 124,	(17,834) : 39,	(18,834) : 596,	(19,834) : 215,	(20,834) : 539,	(21,834) : 260,	(22,834) : 577,	(23,834) : 591,	(24,834) : 41,	(25,834) : 958,	(26,834) : 310,	(27,834) : 368,	(28,834) : 538,	(29,834) : 158,	(30,834) : 541,	(31,834) : 568,	(32,834) : 467,	(33,834) : 465,	(34,834) : 557,	(35,834) : 668,	(36,834) : 782,	(37,834) : 734,	(38,834) : 591,	(39,834) : 591,	(40,834) : 985,
(1,835) : 1058,	(2,835) : 1023,	(3,835) : 546,	(4,835) : 1236,	(5,835) : 973,	(6,835) : 983,	(7,835) : 1048,	(8,835) : 290,	(9,835) : 707,	(10,835) : 1051,	(11,835) : 1024,	(12,835) : 1385,	(13,835) : 1072,	(14,835) : 711,	(15,835) : 1148,	(16,835) : 1350,	(17,835) : 1203,	(18,835) : 967,	(19,835) : 1341,	(20,835) : 1054,	(21,835) : 1039,	(22,835) : 903,	(23,835) : 968,	(24,835) : 1203,	(25,835) : 287,	(26,835) : 1098,	(27,835) : 1072,	(28,835) : 1052,	(29,835) : 1075,	(30,835) : 1058,	(31,835) : 993,	(32,835) : 1028,	(33,835) : 1148,	(34,835) : 961,	(35,835) : 1104,	(36,835) : 684,	(37,835) : 608,	(38,835) : 1005,	(39,835) : 1019,	(40,835) : 268,
(1,836) : 1024,	(2,836) : 985,	(3,836) : 571,	(4,836) : 1251,	(5,836) : 934,	(6,836) : 948,	(7,836) : 1015,	(8,836) : 351,	(9,836) : 668,	(10,836) : 1017,	(11,836) : 986,	(12,836) : 1384,	(13,836) : 1054,	(14,836) : 672,	(15,836) : 1153,	(16,836) : 1364,	(17,836) : 1215,	(18,836) : 930,	(19,836) : 1337,	(20,836) : 1021,	(21,836) : 1035,	(22,836) : 870,	(23,836) : 931,	(24,836) : 1216,	(25,836) : 285,	(26,836) : 1085,	(27,836) : 1054,	(28,836) : 1019,	(29,836) : 1086,	(30,836) : 1024,	(31,836) : 958,	(32,836) : 1002,	(33,836) : 1120,	(34,836) : 928,	(35,836) : 1059,	(36,836) : 637,	(37,836) : 578,	(38,836) : 968,	(39,836) : 981,	(40,836) : 259,
(1,837) : 912,	(2,837) : 877,	(3,837) : 422,	(4,837) : 1105,	(5,837) : 827,	(6,837) : 837,	(7,837) : 902,	(8,837) : 221,	(9,837) : 562,	(10,837) : 904,	(11,837) : 878,	(12,837) : 1245,	(13,837) : 926,	(14,837) : 566,	(15,837) : 1010,	(16,837) : 1218,	(17,837) : 1070,	(18,837) : 821,	(19,837) : 1199,	(20,837) : 908,	(21,837) : 897,	(22,837) : 756,	(23,837) : 822,	(24,837) : 1070,	(25,837) : 143,	(26,837) : 953,	(27,837) : 926,	(28,837) : 906,	(29,837) : 941,	(30,837) : 912,	(31,837) : 847,	(32,837) : 881,	(33,837) : 1001,	(34,837) : 814,	(35,837) : 960,	(36,837) : 542,	(37,837) : 462,	(38,837) : 860,	(39,837) : 873,	(40,837) : 122,
(1,838) : 587,	(2,838) : 640,	(3,838) : 851,	(4,838) : 203,	(5,838) : 674,	(6,838) : 627,	(7,838) : 585,	(8,838) : 1140,	(9,838) : 828,	(10,838) : 585,	(11,838) : 640,	(12,838) : 82,	(13,838) : 432,	(14,838) : 824,	(15,838) : 240,	(16,838) : 120,	(17,838) : 206,	(18,838) : 660,	(19,838) : 135,	(20,838) : 583,	(21,838) : 369,	(22,838) : 658,	(23,838) : 654,	(24,838) : 207,	(25,838) : 1108,	(26,838) : 372,	(27,838) : 428,	(28,838) : 583,	(29,838) : 316,	(30,838) : 584,	(31,838) : 626,	(32,838) : 527,	(33,838) : 484,	(34,838) : 625,	(35,838) : 694,	(36,838) : 889,	(37,838) : 859,	(38,838) : 645,	(39,838) : 641,	(40,838) : 1133,
(1,839) : 600,	(2,839) : 645,	(3,839) : 691,	(4,839) : 1,	(5,839) : 666,	(6,839) : 617,	(7,839) : 595,	(8,839) : 973,	(9,839) : 755,	(10,839) : 596,	(11,839) : 646,	(12,839) : 260,	(13,839) : 430,	(14,839) : 751,	(15,839) : 157,	(16,839) : 118,	(17,839) : 51,	(18,839) : 648,	(19,839) : 274,	(20,839) : 595,	(21,839) : 304,	(22,839) : 623,	(23,839) : 642,	(24,839) : 48,	(25,839) : 966,	(26,839) : 366,	(27,839) : 424,	(28,839) : 594,	(29,839) : 174,	(30,839) : 597,	(31,839) : 620,	(32,839) : 521,	(33,839) : 526,	(34,839) : 607,	(35,839) : 726,	(36,839) : 817,	(37,839) : 760,	(38,839) : 645,	(39,839) : 645,	(40,839) : 994,
(1,840) : 646,	(2,840) : 700,	(3,840) : 907,	(4,840) : 239,	(5,840) : 735,	(6,840) : 688,	(7,840) : 645,	(8,840) : 1195,	(9,840) : 892,	(10,840) : 645,	(11,840) : 700,	(12,840) : 124,	(13,840) : 494,	(14,840) : 887,	(15,840) : 299,	(16,840) : 132,	(17,840) : 253,	(18,840) : 721,	(19,840) : 186,	(20,840) : 643,	(21,840) : 432,	(22,840) : 721,	(23,840) : 716,	(24,840) : 253,	(25,840) : 1167,	(26,840) : 434,	(27,840) : 490,	(28,840) : 643,	(29,840) : 371,	(30,840) : 643,	(31,840) : 687,	(32,840) : 588,	(33,840) : 541,	(34,840) : 687,	(35,840) : 751,	(36,840) : 952,	(37,840) : 922,	(38,840) : 705,	(39,840) : 701,	(40,840) : 1193,
(1,841) : 720,	(2,841) : 681,	(3,841) : 389,	(4,841) : 1005,	(5,841) : 629,	(6,841) : 645,	(7,841) : 711,	(8,841) : 350,	(9,841) : 363,	(10,841) : 714,	(11,841) : 681,	(12,841) : 1108,	(13,841) : 760,	(14,841) : 368,	(15,841) : 889,	(16,841) : 1111,	(17,841) : 964,	(18,841) : 626,	(19,841) : 1056,	(20,841) : 717,	(21,841) : 758,	(22,841) : 567,	(23,841) : 627,	(24,841) : 965,	(25,841) : 151,	(26,841) : 796,	(27,841) : 760,	(28,841) : 715,	(29,841) : 833,	(30,841) : 721,	(31,841) : 654,	(32,841) : 702,	(33,841) : 819,	(34,841) : 625,	(35,841) : 756,	(36,841) : 334,	(37,841) : 279,	(38,841) : 663,	(39,841) : 677,	(40,841) : 136,
(1,842) : 580,	(2,842) : 629,	(3,842) : 772,	(4,842) : 108,	(5,842) : 658,	(6,842) : 609,	(7,842) : 576,	(8,842) : 1059,	(9,842) : 785,	(10,842) : 577,	(11,842) : 630,	(12,842) : 155,	(13,842) : 414,	(14,842) : 780,	(15,842) : 175,	(16,842) : 67,	(17,842) : 116,	(18,842) : 642,	(19,842) : 182,	(20,842) : 575,	(21,842) : 321,	(22,842) : 630,	(23,842) : 636,	(24,842) : 116,	(25,842) : 1036,	(26,842) : 351,	(27,842) : 409,	(28,842) : 575,	(29,842) : 237,	(30,842) : 577,	(31,842) : 611,	(32,842) : 510,	(33,842) : 489,	(34,842) : 604,	(35,842) : 698,	(36,842) : 846,	(37,842) : 804,	(38,842) : 632,	(39,842) : 630,	(40,842) : 1063,
(1,843) : 400,	(2,843) : 353,	(3,843) : 468,	(4,843) : 832,	(5,843) : 300,	(6,843) : 326,	(7,843) : 392,	(8,843) : 614,	(9,843) : 78,	(10,843) : 394,	(11,843) : 353,	(12,843) : 864,	(13,843) : 483,	(14,843) : 81,	(15,843) : 687,	(16,843) : 918,	(17,843) : 783,	(18,843) : 301,	(19,843) : 805,	(20,843) : 398,	(21,843) : 538,	(22,843) : 259,	(23,843) : 304,	(24,843) : 785,	(25,843) : 439,	(26,843) : 535,	(27,843) : 486,	(28,843) : 396,	(29,843) : 661,	(30,843) : 401,	(31,843) : 334,	(32,843) : 404,	(33,843) : 510,	(34,843) : 311,	(35,843) : 417,	(36,843) : 20,	(37,843) : 152,	(38,843) : 337,	(39,843) : 349,	(40,843) : 443,
(1,844) : 893,	(2,844) : 858,	(3,844) : 417,	(4,844) : 1096,	(5,844) : 808,	(6,844) : 818,	(7,844) : 883,	(8,844) : 232,	(9,844) : 542,	(10,844) : 886,	(11,844) : 858,	(12,844) : 1232,	(13,844) : 910,	(14,844) : 546,	(15,844) : 999,	(16,844) : 1209,	(17,844) : 1060,	(18,844) : 802,	(19,844) : 1186,	(20,844) : 889,	(21,844) : 884,	(22,844) : 737,	(23,844) : 803,	(24,844) : 1061,	(25,844) : 131,	(26,844) : 938,	(27,844) : 910,	(28,844) : 887,	(29,844) : 931,	(30,844) : 893,	(31,844) : 828,	(32,844) : 864,	(33,844) : 984,	(34,844) : 796,	(35,844) : 940,	(36,844) : 521,	(37,844) : 443,	(38,844) : 840,	(39,844) : 854,	(40,844) : 106,
(1,845) : 578,	(2,845) : 622,	(3,845) : 673,	(4,845) : 24,	(5,845) : 642,	(6,845) : 593,	(7,845) : 572,	(8,845) : 957,	(9,845) : 731,	(10,845) : 573,	(11,845) : 623,	(12,845) : 256,	(13,845) : 407,	(14,845) : 727,	(15,845) : 134,	(16,845) : 133,	(17,845) : 27,	(18,845) : 624,	(19,845) : 265,	(20,845) : 572,	(21,845) : 280,	(22,845) : 599,	(23,845) : 618,	(24,845) : 25,	(25,845) : 947,	(26,845) : 344,	(27,845) : 401,	(28,845) : 571,	(29,845) : 152,	(30,845) : 574,	(31,845) : 597,	(32,845) : 498,	(33,845) : 504,	(34,845) : 583,	(35,845) : 704,	(36,845) : 794,	(37,845) : 737,	(38,845) : 621,	(39,845) : 622,	(40,845) : 974,
(1,846) : 595,	(2,846) : 644,	(3,846) : 778,	(4,846) : 105,	(5,846) : 672,	(6,846) : 623,	(7,846) : 591,	(8,846) : 1064,	(9,846) : 797,	(10,846) : 592,	(11,846) : 645,	(12,846) : 164,	(13,846) : 429,	(14,846) : 792,	(15,846) : 186,	(16,846) : 52,	(17,846) : 119,	(18,846) : 656,	(19,846) : 195,	(20,846) : 590,	(21,846) : 333,	(22,846) : 643,	(23,846) : 650,	(24,846) : 119,	(25,846) : 1044,	(26,846) : 365,	(27,846) : 424,	(28,846) : 590,	(29,846) : 244,	(30,846) : 592,	(31,846) : 625,	(32,846) : 524,	(33,846) : 504,	(34,846) : 618,	(35,846) : 713,	(36,846) : 858,	(37,846) : 815,	(38,846) : 647,	(39,846) : 645,	(40,846) : 1071,
(1,847) : 691,	(2,847) : 660,	(3,847) : 245,	(4,847) : 892,	(5,847) : 613,	(6,847) : 617,	(7,847) : 681,	(8,847) : 252,	(9,847) : 355,	(10,847) : 683,	(11,847) : 661,	(12,847) : 1018,	(13,847) : 697,	(14,847) : 358,	(15,847) : 788,	(16,847) : 1002,	(17,847) : 854,	(18,847) : 605,	(19,847) : 972,	(20,847) : 687,	(21,847) : 669,	(22,847) : 536,	(23,847) : 605,	(24,847) : 854,	(25,847) : 88,	(26,847) : 724,	(27,847) : 697,	(28,847) : 685,	(29,847) : 723,	(30,847) : 691,	(31,847) : 628,	(32,847) : 655,	(33,847) : 776,	(34,847) : 594,	(35,847) : 751,	(36,847) : 350,	(37,847) : 245,	(38,847) : 643,	(39,847) : 657,	(40,847) : 108,
(1,848) : 874,	(2,848) : 835,	(3,848) : 466,	(4,848) : 1127,	(5,848) : 783,	(6,848) : 798,	(7,848) : 865,	(8,848) : 317,	(9,848) : 517,	(10,848) : 867,	(11,848) : 835,	(12,848) : 1246,	(13,848) : 908,	(14,848) : 522,	(15,848) : 1020,	(16,848) : 1237,	(17,848) : 1089,	(18,848) : 780,	(19,848) : 1197,	(20,848) : 871,	(21,848) : 897,	(22,848) : 720,	(23,848) : 781,	(24,848) : 1090,	(25,848) : 172,	(26,848) : 941,	(27,848) : 908,	(28,848) : 869,	(29,848) : 958,	(30,848) : 874,	(31,848) : 808,	(32,848) : 853,	(33,848) : 971,	(34,848) : 778,	(35,848) : 909,	(36,848) : 487,	(37,848) : 430,	(38,848) : 818,	(39,848) : 831,	(40,848) : 142,
(1,849) : 35,	(2,849) : 65,	(3,849) : 625,	(4,849) : 579,	(5,849) : 99,	(6,849) : 57,	(7,849) : 24,	(8,849) : 875,	(9,849) : 335,	(10,849) : 27,	(11,849) : 66,	(12,849) : 522,	(13,849) : 151,	(14,849) : 331,	(15,849) : 423,	(16,849) : 631,	(17,849) : 530,	(18,849) : 88,	(19,849) : 459,	(20,849) : 29,	(21,849) : 290,	(22,849) : 130,	(23,849) : 84,	(24,849) : 532,	(25,849) : 750,	(26,849) : 215,	(27,849) : 157,	(28,849) : 27,	(29,849) : 447,	(30,849) : 33,	(31,849) : 53,	(32,849) : 60,	(33,849) : 128,	(34,849) : 72,	(35,849) : 162,	(36,849) : 380,	(37,849) : 423,	(38,849) : 68,	(39,849) : 66,	(40,849) : 765,
(1,850) : 538,	(2,850) : 581,	(3,850) : 651,	(4,850) : 64,	(5,850) : 601,	(6,850) : 553,	(7,850) : 532,	(8,850) : 937,	(9,850) : 694,	(10,850) : 533,	(11,850) : 582,	(12,850) : 249,	(13,850) : 367,	(14,850) : 690,	(15,850) : 93,	(16,850) : 159,	(17,850) : 14,	(18,850) : 584,	(19,850) : 248,	(20,850) : 532,	(21,850) : 239,	(22,850) : 559,	(23,850) : 578,	(24,850) : 16,	(25,850) : 919,	(26,850) : 304,	(27,850) : 361,	(28,850) : 531,	(29,850) : 120,	(30,850) : 534,	(31,850) : 556,	(32,850) : 457,	(33,850) : 466,	(34,850) : 543,	(35,850) : 664,	(36,850) : 756,	(37,850) : 702,	(38,850) : 581,	(39,850) : 582,	(40,850) : 946,
(1,851) : 504,	(2,851) : 558,	(3,851) : 814,	(4,851) : 225,	(5,851) : 594,	(6,851) : 546,	(7,851) : 503,	(8,851) : 1104,	(9,851) : 758,	(10,851) : 503,	(11,851) : 558,	(12,851) : 45,	(13,851) : 354,	(14,851) : 754,	(15,851) : 201,	(16,851) : 182,	(17,851) : 209,	(18,851) : 579,	(19,851) : 56,	(20,851) : 501,	(21,851) : 308,	(22,851) : 582,	(23,851) : 574,	(24,851) : 211,	(25,851) : 1058,	(26,851) : 295,	(27,851) : 350,	(28,851) : 501,	(29,851) : 289,	(30,851) : 501,	(31,851) : 546,	(32,851) : 447,	(33,851) : 400,	(34,851) : 546,	(35,851) : 611,	(36,851) : 818,	(37,851) : 796,	(38,851) : 563,	(39,851) : 559,	(40,851) : 1083,
(1,852) : 970,	(2,852) : 937,	(3,852) : 449,	(4,852) : 1139,	(5,852) : 888,	(6,852) : 895,	(7,852) : 960,	(8,852) : 209,	(9,852) : 623,	(10,852) : 963,	(11,852) : 937,	(12,852) : 1288,	(13,852) : 978,	(14,852) : 627,	(15,852) : 1050,	(16,852) : 1253,	(17,852) : 1105,	(18,852) : 881,	(19,852) : 1244,	(20,852) : 966,	(21,852) : 942,	(22,852) : 814,	(23,852) : 882,	(24,852) : 1106,	(25,852) : 194,	(26,852) : 1003,	(27,852) : 978,	(28,852) : 964,	(29,852) : 978,	(30,852) : 970,	(31,852) : 906,	(32,852) : 937,	(33,852) : 1057,	(34,852) : 873,	(35,852) : 1022,	(36,852) : 605,	(37,852) : 520,	(38,852) : 919,	(39,852) : 933,	(40,852) : 178,
(1,853) : 728,	(2,853) : 721,	(3,853) : 94,	(4,853) : 684,	(5,853) : 689,	(6,853) : 670,	(7,853) : 717,	(8,853) : 294,	(9,853) : 509,	(10,853) : 720,	(11,853) : 722,	(12,853) : 874,	(13,853) : 657,	(14,853) : 509,	(15,853) : 628,	(16,853) : 801,	(17,853) : 659,	(18,853) : 674,	(19,853) : 843,	(20,853) : 722,	(21,853) : 561,	(22,853) : 599,	(23,853) : 671,	(24,853) : 659,	(25,853) : 355,	(26,853) : 654,	(27,853) : 654,	(28,853) : 720,	(29,853) : 543,	(30,853) : 727,	(31,853) : 681,	(32,853) : 659,	(33,853) : 771,	(34,853) : 644,	(35,853) : 831,	(36,853) : 542,	(37,853) : 410,	(38,853) : 706,	(39,853) : 718,	(40,853) : 385,
(1,854) : 825,	(2,854) : 797,	(3,854) : 275,	(4,854) : 964,	(5,854) : 751,	(6,854) : 752,	(7,854) : 815,	(8,854) : 121,	(9,854) : 496,	(10,854) : 817,	(11,854) : 798,	(12,854) : 1115,	(13,854) : 817,	(14,854) : 499,	(15,854) : 876,	(16,854) : 1078,	(17,854) : 930,	(18,854) : 742,	(19,854) : 1073,	(20,854) : 821,	(21,854) : 772,	(22,854) : 671,	(23,854) : 742,	(24,854) : 931,	(25,854) : 79,	(26,854) : 837,	(27,854) : 816,	(28,854) : 818,	(29,854) : 803,	(30,854) : 825,	(31,854) : 763,	(32,854) : 783,	(33,854) : 904,	(34,854) : 728,	(35,854) : 890,	(36,854) : 492,	(37,854) : 385,	(38,854) : 780,	(39,854) : 794,	(40,854) : 98,
(1,855) : 663,	(2,855) : 649,	(3,855) : 16,	(4,855) : 706,	(5,855) : 613,	(6,855) : 599,	(7,855) : 652,	(8,855) : 278,	(9,855) : 413,	(10,855) : 654,	(11,855) : 651,	(12,855) : 871,	(13,855) : 613,	(14,855) : 415,	(15,855) : 627,	(16,855) : 821,	(17,855) : 674,	(18,855) : 599,	(19,855) : 834,	(20,855) : 657,	(21,855) : 538,	(22,855) : 524,	(23,855) : 597,	(24,855) : 674,	(25,855) : 278,	(26,855) : 619,	(27,855) : 610,	(28,855) : 655,	(29,855) : 549,	(30,855) : 661,	(31,855) : 611,	(32,855) : 602,	(33,855) : 718,	(34,855) : 573,	(35,855) : 757,	(36,855) : 443,	(37,855) : 310,	(38,855) : 633,	(39,855) : 646,	(40,855) : 309,
(1,856) : 694,	(2,856) : 681,	(3,856) : 37,	(4,856) : 722,	(5,856) : 645,	(6,856) : 631,	(7,856) : 683,	(8,856) : 255,	(9,856) : 443,	(10,856) : 685,	(11,856) : 682,	(12,856) : 893,	(13,856) : 642,	(14,856) : 444,	(15,856) : 649,	(16,856) : 838,	(17,856) : 692,	(18,856) : 631,	(19,856) : 857,	(20,856) : 688,	(21,856) : 564,	(22,856) : 555,	(23,856) : 629,	(24,856) : 692,	(25,856) : 277,	(26,856) : 647,	(27,856) : 639,	(28,856) : 686,	(29,856) : 569,	(30,856) : 693,	(31,856) : 642,	(32,856) : 632,	(33,856) : 748,	(34,856) : 605,	(35,856) : 788,	(36,856) : 471,	(37,856) : 338,	(38,856) : 665,	(39,856) : 678,	(40,856) : 307,
(1,857) : 357,	(2,857) : 318,	(3,857) : 414,	(4,857) : 752,	(5,857) : 267,	(6,857) : 282,	(7,857) : 349,	(8,857) : 594,	(9,857) : 4,	(10,857) : 351,	(11,857) : 318,	(12,857) : 793,	(13,857) : 418,	(14,857) : 5,	(15,857) : 609,	(16,857) : 840,	(17,857) : 704,	(18,857) : 262,	(19,857) : 735,	(20,857) : 355,	(21,857) : 461,	(22,857) : 205,	(23,857) : 264,	(24,857) : 705,	(25,857) : 436,	(26,857) : 465,	(27,857) : 420,	(28,857) : 353,	(29,857) : 580,	(30,857) : 358,	(31,857) : 291,	(32,857) : 347,	(33,857) : 460,	(34,857) : 263,	(35,857) : 400,	(36,857) : 66,	(37,857) : 111,	(38,857) : 300,	(39,857) : 314,	(40,857) : 445,
(1,858) : 272,	(2,858) : 324,	(3,858) : 690,	(4,858) : 353,	(5,858) : 360,	(6,858) : 314,	(7,858) : 270,	(8,858) : 974,	(9,858) : 546,	(10,858) : 270,	(11,858) : 325,	(12,858) : 261,	(13,858) : 127,	(14,858) : 541,	(15,858) : 213,	(16,858) : 380,	(17,858) : 308,	(18,858) : 347,	(19,858) : 198,	(20,858) : 268,	(21,858) : 172,	(22,858) : 356,	(23,858) : 342,	(24,858) : 311,	(25,858) : 895,	(26,858) : 84,	(27,858) : 124,	(28,858) : 268,	(29,858) : 280,	(30,858) : 269,	(31,858) : 313,	(32,858) : 215,	(33,858) : 176,	(34,858) : 315,	(35,858) : 384,	(36,858) : 602,	(37,858) : 601,	(38,858) : 330,	(39,858) : 326,	(40,858) : 916,
(1,859) : 269,	(2,859) : 291,	(3,859) : 456,	(4,859) : 405,	(5,859) : 291,	(6,859) : 247,	(7,859) : 259,	(8,859) : 737,	(9,859) : 355,	(10,859) : 261,	(11,859) : 292,	(12,859) : 454,	(13,859) : 153,	(14,859) : 351,	(15,859) : 257,	(16,859) : 488,	(17,859) : 355,	(18,859) : 271,	(19,859) : 402,	(20,859) : 263,	(21,859) : 109,	(22,859) : 227,	(23,859) : 266,	(24,859) : 357,	(25,859) : 659,	(26,859) : 155,	(27,859) : 149,	(28,859) : 261,	(29,859) : 241,	(30,859) : 266,	(31,859) : 255,	(32,859) : 182,	(33,859) : 273,	(34,859) : 229,	(35,859) : 398,	(36,859) : 417,	(37,859) : 382,	(38,859) : 283,	(39,859) : 290,	(40,859) : 681,
(1,860) : 694,	(2,860) : 667,	(3,860) : 193,	(4,860) : 856,	(5,860) : 621,	(6,860) : 621,	(7,860) : 683,	(8,860) : 224,	(9,860) : 372,	(10,860) : 686,	(11,860) : 668,	(12,860) : 994,	(13,860) : 686,	(14,860) : 375,	(15,860) : 759,	(16,860) : 968,	(17,860) : 820,	(18,860) : 612,	(19,860) : 950,	(20,860) : 689,	(21,860) : 647,	(22,860) : 540,	(23,860) : 611,	(24,860) : 821,	(25,860) : 110,	(26,860) : 708,	(27,860) : 686,	(28,860) : 687,	(29,860) : 690,	(30,860) : 693,	(31,860) : 632,	(32,860) : 651,	(33,860) : 772,	(34,860) : 597,	(35,860) : 762,	(36,860) : 376,	(37,860) : 259,	(38,860) : 649,	(39,860) : 663,	(40,860) : 138,
(1,861) : 960,	(2,861) : 926,	(3,861) : 443,	(4,861) : 1132,	(5,861) : 877,	(6,861) : 885,	(7,861) : 950,	(8,861) : 208,	(9,861) : 613,	(10,861) : 952,	(11,861) : 927,	(12,861) : 1280,	(13,861) : 969,	(14,861) : 617,	(15,861) : 1042,	(16,861) : 1246,	(17,861) : 1098,	(18,861) : 871,	(19,861) : 1236,	(20,861) : 956,	(21,861) : 933,	(22,861) : 804,	(23,861) : 871,	(24,861) : 1099,	(25,861) : 184,	(26,861) : 993,	(27,861) : 968,	(28,861) : 954,	(29,861) : 971,	(30,861) : 960,	(31,861) : 895,	(32,861) : 927,	(33,861) : 1047,	(34,861) : 863,	(35,861) : 1011,	(36,861) : 595,	(37,861) : 510,	(38,861) : 909,	(39,861) : 923,	(40,861) : 168,
(1,862) : 675,	(2,862) : 643,	(3,862) : 262,	(4,862) : 895,	(5,862) : 594,	(6,862) : 600,	(7,862) : 665,	(8,862) : 279,	(9,862) : 334,	(10,862) : 667,	(11,862) : 643,	(12,862) : 1015,	(13,862) : 687,	(14,862) : 337,	(15,862) : 787,	(16,862) : 1004,	(17,862) : 856,	(18,862) : 587,	(19,862) : 967,	(20,862) : 671,	(21,862) : 665,	(22,862) : 519,	(23,862) : 587,	(24,862) : 857,	(25,862) : 104,	(26,862) : 716,	(27,862) : 687,	(28,862) : 669,	(29,862) : 725,	(30,862) : 675,	(31,862) : 611,	(32,862) : 642,	(33,862) : 762,	(34,862) : 578,	(35,862) : 731,	(36,862) : 326,	(37,862) : 226,	(38,862) : 625,	(39,862) : 639,	(40,862) : 118,
(1,863) : 68,	(2,863) : 123,	(3,863) : 689,	(4,863) : 555,	(5,863) : 171,	(6,863) : 137,	(7,863) : 72,	(8,863) : 947,	(9,863) : 417,	(10,863) : 71,	(11,863) : 123,	(12,863) : 466,	(13,863) : 136,	(14,863) : 412,	(15,863) : 403,	(16,863) : 593,	(17,863) : 507,	(18,863) : 165,	(19,863) : 402,	(20,863) : 67,	(21,863) : 290,	(22,863) : 211,	(23,863) : 161,	(24,863) : 510,	(25,863) : 827,	(26,863) : 193,	(27,863) : 142,	(28,863) : 69,	(29,863) : 443,	(30,863) : 66,	(31,863) : 130,	(32,863) : 93,	(33,863) : 50,	(34,863) : 154,	(35,863) : 171,	(36,863) : 461,	(37,863) : 503,	(38,863) : 135,	(39,863) : 126,	(40,863) : 843,
(1,864) : 724,	(2,864) : 709,	(3,864) : 74,	(4,864) : 760,	(5,864) : 672,	(6,864) : 660,	(7,864) : 713,	(8,864) : 217,	(9,864) : 460,	(10,864) : 716,	(11,864) : 711,	(12,864) : 932,	(13,864) : 677,	(14,864) : 462,	(15,864) : 688,	(16,864) : 876,	(17,864) : 730,	(18,864) : 658,	(19,864) : 896,	(20,864) : 719,	(21,864) : 602,	(22,864) : 583,	(23,864) : 657,	(24,864) : 731,	(25,864) : 250,	(26,864) : 684,	(27,864) : 675,	(28,864) : 717,	(29,864) : 608,	(30,864) : 723,	(31,864) : 671,	(32,864) : 665,	(33,864) : 782,	(34,864) : 634,	(35,864) : 816,	(36,864) : 484,	(37,864) : 352,	(38,864) : 693,	(39,864) : 706,	(40,864) : 281,
(1,865) : 235,	(2,865) : 262,	(3,865) : 493,	(4,865) : 410,	(5,865) : 267,	(6,865) : 221,	(7,865) : 226,	(8,865) : 772,	(9,865) : 363,	(10,865) : 228,	(11,865) : 263,	(12,865) : 436,	(13,865) : 113,	(14,865) : 358,	(15,865) : 257,	(16,865) : 486,	(17,865) : 360,	(18,865) : 248,	(19,865) : 382,	(20,865) : 229,	(21,865) : 107,	(22,865) : 213,	(23,865) : 243,	(24,865) : 362,	(25,865) : 688,	(26,865) : 123,	(27,865) : 110,	(28,865) : 227,	(29,865) : 255,	(30,865) : 233,	(31,865) : 228,	(32,865) : 147,	(33,865) : 234,	(34,865) : 205,	(35,865) : 366,	(36,865) : 423,	(37,865) : 401,	(38,865) : 256,	(39,865) : 261,	(40,865) : 709,
(1,866) : 532,	(2,866) : 576,	(3,866) : 652,	(4,866) : 68,	(5,866) : 597,	(6,866) : 548,	(7,866) : 527,	(8,866) : 939,	(9,866) : 691,	(10,866) : 528,	(11,866) : 577,	(12,866) : 243,	(13,866) : 361,	(14,866) : 687,	(15,866) : 89,	(16,866) : 160,	(17,866) : 18,	(18,866) : 579,	(19,866) : 242,	(20,866) : 527,	(21,866) : 236,	(22,866) : 555,	(23,866) : 573,	(24,866) : 21,	(25,866) : 920,	(26,866) : 299,	(27,866) : 355,	(28,866) : 526,	(29,866) : 120,	(30,866) : 529,	(31,866) : 551,	(32,866) : 453,	(33,866) : 460,	(34,866) : 538,	(35,866) : 659,	(36,866) : 753,	(37,866) : 701,	(38,866) : 576,	(39,866) : 577,	(40,866) : 947,
(1,867) : 676,	(2,867) : 728,	(3,867) : 882,	(4,867) : 195,	(5,867) : 760,	(6,867) : 711,	(7,867) : 673,	(8,867) : 1167,	(9,867) : 898,	(10,867) : 674,	(11,867) : 728,	(12,867) : 176,	(13,867) : 516,	(14,867) : 894,	(15,867) : 290,	(16,867) : 79,	(17,867) : 222,	(18,867) : 744,	(19,867) : 231,	(20,867) : 672,	(21,867) : 434,	(22,867) : 737,	(23,867) : 739,	(24,867) : 221,	(25,867) : 1150,	(26,867) : 454,	(27,867) : 511,	(28,867) : 672,	(29,867) : 350,	(30,867) : 673,	(31,867) : 712,	(32,867) : 611,	(33,867) : 577,	(34,867) : 708,	(35,867) : 787,	(36,867) : 959,	(37,867) : 920,	(38,867) : 732,	(39,867) : 729,	(40,867) : 1177,
(1,868) : 524,	(2,868) : 499,	(3,868) : 204,	(4,868) : 734,	(5,868) : 455,	(6,868) : 452,	(7,868) : 513,	(8,868) : 386,	(9,868) : 223,	(10,868) : 516,	(11,868) : 500,	(12,868) : 845,	(13,868) : 520,	(14,868) : 225,	(15,868) : 620,	(16,868) : 840,	(17,868) : 693,	(18,868) : 444,	(19,868) : 796,	(20,868) : 519,	(21,868) : 495,	(22,868) : 371,	(23,868) : 444,	(24,868) : 694,	(25,868) : 265,	(26,868) : 546,	(27,868) : 519,	(28,868) : 517,	(29,868) : 562,	(30,868) : 523,	(31,868) : 463,	(32,868) : 481,	(33,868) : 602,	(34,868) : 428,	(35,868) : 597,	(36,868) : 247,	(37,868) : 114,	(38,868) : 481,	(39,868) : 495,	(40,868) : 286,
(1,869) : 697,	(2,869) : 670,	(3,869) : 188,	(4,869) : 854,	(5,869) : 625,	(6,869) : 624,	(7,869) : 686,	(8,869) : 219,	(9,869) : 377,	(10,869) : 689,	(11,869) : 671,	(12,869) : 993,	(13,869) : 688,	(14,869) : 380,	(15,869) : 757,	(16,869) : 966,	(17,869) : 818,	(18,869) : 615,	(19,869) : 949,	(20,869) : 692,	(21,869) : 647,	(22,869) : 544,	(23,869) : 615,	(24,869) : 818,	(25,869) : 113,	(26,869) : 709,	(27,869) : 687,	(28,869) : 690,	(29,869) : 688,	(30,869) : 696,	(31,869) : 635,	(32,869) : 653,	(33,869) : 774,	(34,869) : 600,	(35,869) : 766,	(36,869) : 382,	(37,869) : 264,	(38,869) : 653,	(39,869) : 667,	(40,869) : 141,
(1,870) : 693,	(2,870) : 666,	(3,870) : 188,	(4,870) : 852,	(5,870) : 621,	(6,870) : 621,	(7,870) : 682,	(8,870) : 223,	(9,870) : 373,	(10,870) : 685,	(11,870) : 667,	(12,870) : 991,	(13,870) : 685,	(14,870) : 376,	(15,870) : 755,	(16,870) : 964,	(17,870) : 816,	(18,870) : 611,	(19,870) : 947,	(20,870) : 688,	(21,870) : 644,	(22,870) : 540,	(23,870) : 611,	(24,870) : 817,	(25,870) : 114,	(26,870) : 706,	(27,870) : 684,	(28,870) : 686,	(29,870) : 687,	(30,870) : 692,	(31,870) : 632,	(32,870) : 650,	(33,870) : 771,	(34,870) : 596,	(35,870) : 762,	(36,870) : 378,	(37,870) : 260,	(38,870) : 649,	(39,870) : 663,	(40,870) : 142,
(1,871) : 319,	(2,871) : 294,	(3,871) : 364,	(4,871) : 642,	(5,871) : 252,	(6,871) : 247,	(7,871) : 308,	(8,871) : 590,	(9,871) : 115,	(10,871) : 311,	(11,871) : 295,	(12,871) : 697,	(13,871) : 336,	(14,871) : 111,	(15,871) : 502,	(16,871) : 733,	(17,871) : 595,	(18,871) : 240,	(19,871) : 642,	(20,871) : 314,	(21,871) : 357,	(22,871) : 167,	(23,871) : 240,	(24,871) : 597,	(25,871) : 462,	(26,871) : 375,	(27,871) : 337,	(28,871) : 312,	(29,871) : 470,	(30,871) : 318,	(31,871) : 258,	(32,871) : 283,	(33,871) : 403,	(34,871) : 223,	(35,871) : 396,	(36,871) : 177,	(37,871) : 144,	(38,871) : 277,	(39,871) : 291,	(40,871) : 478,
(1,872) : 633,	(2,872) : 601,	(3,872) : 250,	(4,872) : 862,	(5,872) : 554,	(6,872) : 559,	(7,872) : 623,	(8,872) : 309,	(9,872) : 294,	(10,872) : 626,	(11,872) : 602,	(12,872) : 977,	(13,872) : 646,	(14,872) : 298,	(15,872) : 751,	(16,872) : 969,	(17,872) : 822,	(18,872) : 546,	(19,872) : 928,	(20,872) : 629,	(21,872) : 627,	(22,872) : 478,	(23,872) : 546,	(24,872) : 823,	(25,872) : 145,	(26,872) : 675,	(27,872) : 646,	(28,872) : 627,	(29,872) : 690,	(30,872) : 633,	(31,872) : 570,	(32,872) : 600,	(33,872) : 721,	(34,872) : 536,	(35,872) : 691,	(36,872) : 291,	(37,872) : 185,	(38,872) : 584,	(39,872) : 598,	(40,872) : 160,
(1,873) : 591,	(2,873) : 638,	(3,873) : 725,	(4,873) : 45,	(5,873) : 662,	(6,873) : 613,	(7,873) : 586,	(8,873) : 1010,	(9,873) : 767,	(10,873) : 587,	(11,873) : 639,	(12,873) : 215,	(13,873) : 421,	(14,873) : 763,	(15,873) : 157,	(16,873) : 82,	(17,873) : 66,	(18,873) : 645,	(19,873) : 234,	(20,873) : 586,	(21,873) : 308,	(22,873) : 625,	(23,873) : 639,	(24,873) : 65,	(25,873) : 996,	(26,873) : 357,	(27,873) : 416,	(28,873) : 585,	(29,873) : 197,	(30,873) : 588,	(31,873) : 615,	(32,873) : 515,	(33,873) : 509,	(34,873) : 605,	(35,873) : 714,	(36,873) : 829,	(37,873) : 779,	(38,873) : 639,	(39,873) : 638,	(40,873) : 1023,
(1,874) : 962,	(2,874) : 923,	(3,874) : 519,	(4,874) : 1195,	(5,874) : 872,	(6,874) : 886,	(7,874) : 953,	(8,874) : 321,	(9,874) : 606,	(10,874) : 955,	(11,874) : 924,	(12,874) : 1324,	(13,874) : 992,	(14,874) : 610,	(15,874) : 1094,	(16,874) : 1307,	(17,874) : 1158,	(18,874) : 868,	(19,874) : 1276,	(20,874) : 959,	(21,874) : 974,	(22,874) : 807,	(23,874) : 869,	(24,874) : 1159,	(25,874) : 229,	(26,874) : 1023,	(27,874) : 992,	(28,874) : 957,	(29,874) : 1028,	(30,874) : 962,	(31,874) : 896,	(32,874) : 940,	(33,874) : 1058,	(34,874) : 866,	(35,874) : 998,	(36,874) : 576,	(37,874) : 516,	(38,874) : 906,	(39,874) : 920,	(40,874) : 201,
(1,875) : 965,	(2,875) : 934,	(3,875) : 417,	(4,875) : 1108,	(5,875) : 886,	(6,875) : 891,	(7,875) : 955,	(8,875) : 162,	(9,875) : 624,	(10,875) : 957,	(11,875) : 935,	(12,875) : 1264,	(13,875) : 965,	(14,875) : 628,	(15,875) : 1025,	(16,875) : 1223,	(17,875) : 1076,	(18,875) : 878,	(19,875) : 1223,	(20,875) : 961,	(21,875) : 921,	(22,875) : 810,	(23,875) : 879,	(24,875) : 1076,	(25,875) : 188,	(26,875) : 986,	(27,875) : 964,	(28,875) : 959,	(29,875) : 950,	(30,875) : 965,	(31,875) : 902,	(32,875) : 927,	(33,875) : 1048,	(34,875) : 868,	(35,875) : 1023,	(36,875) : 611,	(37,875) : 518,	(38,875) : 917,	(39,875) : 930,	(40,875) : 179,
(1,876) : 341,	(2,876) : 299,	(3,876) : 445,	(4,876) : 762,	(5,876) : 247,	(6,876) : 266,	(7,876) : 333,	(8,876) : 622,	(9,876) : 27,	(10,876) : 335,	(11,876) : 299,	(12,876) : 793,	(13,876) : 414,	(14,876) : 26,	(15,876) : 616,	(16,876) : 847,	(17,876) : 713,	(18,876) : 244,	(19,876) : 734,	(20,876) : 339,	(21,876) : 467,	(22,876) : 194,	(23,876) : 246,	(24,876) : 715,	(25,876) : 461,	(26,876) : 464,	(27,876) : 416,	(28,876) : 337,	(29,876) : 592,	(30,876) : 342,	(31,876) : 275,	(32,876) : 338,	(33,876) : 447,	(34,876) : 249,	(35,876) : 376,	(36,876) : 61,	(37,876) : 141,	(38,876) : 282,	(39,876) : 295,	(40,876) : 470,
(1,877) : 349,	(2,877) : 399,	(3,877) : 678,	(4,877) : 267,	(5,877) : 430,	(6,877) : 382,	(7,877) : 346,	(8,877) : 967,	(9,877) : 588,	(10,877) : 346,	(11,877) : 400,	(12,877) : 208,	(13,877) : 187,	(14,877) : 583,	(15,877) : 134,	(16,877) : 298,	(17,877) : 223,	(18,877) : 415,	(19,877) : 154,	(20,877) : 345,	(21,877) : 152,	(22,877) : 412,	(23,877) : 410,	(24,877) : 226,	(25,877) : 904,	(26,877) : 125,	(27,877) : 182,	(28,877) : 344,	(29,877) : 212,	(30,877) : 346,	(31,877) : 383,	(32,877) : 282,	(33,877) : 260,	(34,877) : 379,	(35,877) : 467,	(36,877) : 647,	(37,877) : 629,	(38,877) : 403,	(39,877) : 400,	(40,877) : 927,
(1,878) : 884,	(2,878) : 849,	(3,878) : 411,	(4,878) : 1089,	(5,878) : 799,	(6,878) : 809,	(7,878) : 874,	(8,878) : 232,	(9,878) : 533,	(10,878) : 877,	(11,878) : 849,	(12,878) : 1224,	(13,878) : 901,	(14,878) : 537,	(15,878) : 991,	(16,878) : 1201,	(17,878) : 1053,	(18,878) : 793,	(19,878) : 1178,	(20,878) : 880,	(21,878) : 875,	(22,878) : 728,	(23,878) : 794,	(24,878) : 1054,	(25,878) : 123,	(26,878) : 929,	(27,878) : 901,	(28,878) : 878,	(29,878) : 924,	(30,878) : 884,	(31,878) : 819,	(32,878) : 855,	(33,878) : 975,	(34,878) : 787,	(35,878) : 931,	(36,878) : 512,	(37,878) : 434,	(38,878) : 831,	(39,878) : 845,	(40,878) : 98,
(1,879) : 883,	(2,879) : 854,	(3,879) : 329,	(4,879) : 1020,	(5,879) : 807,	(6,879) : 809,	(7,879) : 872,	(8,879) : 112,	(9,879) : 549,	(10,879) : 875,	(11,879) : 855,	(12,879) : 1174,	(13,879) : 877,	(14,879) : 553,	(15,879) : 935,	(16,879) : 1134,	(17,879) : 987,	(18,879) : 798,	(19,879) : 1133,	(20,879) : 878,	(21,879) : 831,	(22,879) : 728,	(23,879) : 798,	(24,879) : 988,	(25,879) : 116,	(26,879) : 897,	(27,879) : 876,	(28,879) : 876,	(29,879) : 861,	(30,879) : 882,	(31,879) : 820,	(32,879) : 842,	(33,879) : 963,	(34,879) : 786,	(35,879) : 946,	(36,879) : 542,	(37,879) : 439,	(38,879) : 836,	(39,879) : 850,	(40,879) : 120,
(1,880) : 717,	(2,880) : 771,	(3,880) : 969,	(4,880) : 287,	(5,880) : 807,	(6,880) : 760,	(7,880) : 716,	(8,880) : 1256,	(9,880) : 964,	(10,880) : 716,	(11,880) : 772,	(12,880) : 191,	(13,880) : 567,	(14,880) : 960,	(15,880) : 368,	(16,880) : 171,	(17,880) : 310,	(18,880) : 793,	(19,880) : 254,	(20,880) : 714,	(21,880) : 504,	(22,880) : 794,	(23,880) : 788,	(24,880) : 310,	(25,880) : 1234,	(26,880) : 507,	(27,880) : 563,	(28,880) : 714,	(29,880) : 435,	(30,880) : 714,	(31,880) : 759,	(32,880) : 661,	(33,880) : 611,	(34,880) : 760,	(35,880) : 820,	(36,880) : 1025,	(37,880) : 993,	(38,880) : 777,	(39,880) : 773,	(40,880) : 1260,
(1,881) : 415,	(2,881) : 449,	(3,881) : 503,	(4,881) : 235,	(5,881) : 459,	(6,881) : 412,	(7,881) : 407,	(8,881) : 793,	(9,881) : 521,	(10,881) : 409,	(11,881) : 450,	(12,881) : 356,	(13,881) : 253,	(14,881) : 517,	(15,881) : 113,	(16,881) : 332,	(17,881) : 188,	(18,881) : 440,	(19,881) : 323,	(20,881) : 409,	(21,881) : 99,	(22,881) : 402,	(23,881) : 434,	(24,881) : 190,	(25,881) : 756,	(26,881) : 206,	(27,881) : 247,	(28,881) : 407,	(29,881) : 65,	(30,881) : 412,	(31,881) : 418,	(32,881) : 328,	(33,881) : 376,	(34,881) : 397,	(35,881) : 547,	(36,881) : 583,	(37,881) : 527,	(38,881) : 445,	(39,881) : 449,	(40,881) : 782,
(1,882) : 546,	(2,882) : 591,	(3,882) : 660,	(4,882) : 54,	(5,882) : 611,	(6,882) : 562,	(7,882) : 541,	(8,882) : 947,	(9,882) : 705,	(10,882) : 542,	(11,882) : 592,	(12,882) : 244,	(13,882) : 375,	(14,882) : 701,	(15,882) : 103,	(16,882) : 149,	(17,882) : 5,	(18,882) : 593,	(19,882) : 246,	(20,882) : 541,	(21,882) : 250,	(22,882) : 569,	(23,882) : 588,	(24,882) : 8,	(25,882) : 930,	(26,882) : 313,	(27,882) : 369,	(28,882) : 540,	(29,882) : 131,	(30,882) : 543,	(31,882) : 566,	(32,882) : 467,	(33,882) : 473,	(34,882) : 553,	(35,882) : 673,	(36,882) : 767,	(37,882) : 713,	(38,882) : 590,	(39,882) : 591,	(40,882) : 957,
(1,883) : 77,	(2,883) : 65,	(3,883) : 587,	(4,883) : 598,	(5,883) : 67,	(6,883) : 19,	(7,883) : 66,	(8,883) : 831,	(9,883) : 285,	(10,883) : 69,	(11,883) : 67,	(12,883) : 560,	(13,883) : 179,	(14,883) : 280,	(15,883) : 441,	(16,883) : 657,	(17,883) : 548,	(18,883) : 50,	(19,883) : 497,	(20,883) : 73,	(21,883) : 300,	(22,883) : 80,	(23,883) : 44,	(24,883) : 550,	(25,883) : 702,	(26,883) : 242,	(27,883) : 183,	(28,883) : 70,	(29,883) : 455,	(30,883) : 76,	(31,883) : 26,	(32,883) : 84,	(33,883) : 177,	(34,883) : 22,	(35,883) : 178,	(36,883) : 329,	(37,883) : 374,	(38,883) : 55,	(39,883) : 63,	(40,883) : 716,
(1,884) : 721,	(2,884) : 776,	(3,884) : 1032,	(4,884) : 364,	(5,884) : 816,	(6,884) : 771,	(7,884) : 721,	(8,884) : 1320,	(9,884) : 994,	(10,884) : 721,	(11,884) : 776,	(12,884) : 199,	(13,884) : 583,	(14,884) : 989,	(15,884) : 420,	(16,884) : 253,	(17,884) : 380,	(18,884) : 804,	(19,884) : 260,	(20,884) : 718,	(21,884) : 543,	(22,884) : 813,	(23,884) : 799,	(24,884) : 380,	(25,884) : 1287,	(26,884) : 528,	(27,884) : 580,	(28,884) : 718,	(29,884) : 496,	(30,884) : 718,	(31,884) : 769,	(32,884) : 673,	(33,884) : 608,	(34,884) : 773,	(35,884) : 812,	(36,884) : 1053,	(37,884) : 1032,	(38,884) : 784,	(39,884) : 778,	(40,884) : 1313,
(1,885) : 621,	(2,885) : 669,	(3,885) : 754,	(4,885) : 65,	(5,885) : 693,	(6,885) : 644,	(7,885) : 617,	(8,885) : 1038,	(9,885) : 801,	(10,885) : 617,	(11,885) : 669,	(12,885) : 217,	(13,885) : 452,	(14,885) : 797,	(15,885) : 191,	(16,885) : 52,	(17,885) : 98,	(18,885) : 676,	(19,885) : 245,	(20,885) : 616,	(21,885) : 341,	(22,885) : 658,	(23,885) : 671,	(24,885) : 97,	(25,885) : 1027,	(26,885) : 388,	(27,885) : 446,	(28,885) : 616,	(29,885) : 230,	(30,885) : 618,	(31,885) : 647,	(32,885) : 546,	(33,885) : 537,	(34,885) : 637,	(35,885) : 743,	(36,885) : 863,	(37,885) : 812,	(38,885) : 670,	(39,885) : 669,	(40,885) : 1055,
(1,886) : 775,	(2,886) : 755,	(3,886) : 154,	(4,886) : 844,	(5,886) : 714,	(6,886) : 707,	(7,886) : 764,	(8,886) : 139,	(9,886) : 483,	(10,886) : 767,	(11,886) : 757,	(12,886) : 1011,	(13,886) : 742,	(14,886) : 485,	(15,886) : 768,	(16,886) : 960,	(17,886) : 814,	(18,886) : 702,	(19,886) : 974,	(20,886) : 770,	(21,886) : 676,	(22,886) : 628,	(23,886) : 701,	(24,886) : 814,	(25,886) : 184,	(26,886) : 753,	(27,886) : 740,	(28,886) : 768,	(29,886) : 690,	(30,886) : 774,	(31,886) : 718,	(32,886) : 722,	(33,886) : 841,	(34,886) : 682,	(35,886) : 858,	(36,886) : 496,	(37,886) : 370,	(38,886) : 739,	(39,886) : 752,	(40,886) : 213,
(1,887) : 566,	(2,887) : 605,	(3,887) : 596,	(4,887) : 95,	(5,887) : 619,	(6,887) : 571,	(7,887) : 559,	(8,887) : 879,	(9,887) : 682,	(10,887) : 560,	(11,887) : 606,	(12,887) : 324,	(13,887) : 396,	(14,887) : 678,	(15,887) : 134,	(16,887) : 212,	(17,887) : 77,	(18,887) : 600,	(19,887) : 322,	(20,887) : 560,	(21,887) : 252,	(22,887) : 567,	(23,887) : 595,	(24,887) : 76,	(25,887) : 874,	(26,887) : 338,	(27,887) : 390,	(28,887) : 559,	(29,887) : 100,	(30,887) : 563,	(31,887) : 576,	(32,887) : 481,	(33,887) : 507,	(34,887) : 558,	(35,887) : 696,	(36,887) : 744,	(37,887) : 678,	(38,887) : 602,	(39,887) : 605,	(40,887) : 901,
(1,888) : 830,	(2,888) : 793,	(3,888) : 402,	(4,888) : 1065,	(5,888) : 742,	(6,888) : 754,	(7,888) : 821,	(8,888) : 272,	(9,888) : 476,	(10,888) : 823,	(11,888) : 794,	(12,888) : 1188,	(13,888) : 856,	(14,888) : 480,	(15,888) : 960,	(16,888) : 1175,	(17,888) : 1027,	(18,888) : 737,	(19,888) : 1140,	(20,888) : 827,	(21,888) : 839,	(22,888) : 675,	(23,888) : 739,	(24,888) : 1028,	(25,888) : 109,	(26,888) : 887,	(27,888) : 856,	(28,888) : 825,	(29,888) : 896,	(30,888) : 830,	(31,888) : 764,	(32,888) : 805,	(33,888) : 924,	(34,888) : 733,	(35,888) : 872,	(36,888) : 452,	(37,888) : 381,	(38,888) : 776,	(39,888) : 789,	(40,888) : 78,
(1,889) : 126,	(2,889) : 180,	(3,889) : 663,	(4,889) : 483,	(5,889) : 219,	(6,889) : 176,	(7,889) : 125,	(8,889) : 932,	(9,889) : 436,	(10,889) : 125,	(11,889) : 180,	(12,889) : 403,	(13,889) : 72,	(14,889) : 431,	(15,889) : 331,	(16,889) : 522,	(17,889) : 435,	(18,889) : 208,	(19,889) : 339,	(20,889) : 123,	(21,889) : 224,	(22,889) : 233,	(23,889) : 203,	(24,889) : 438,	(25,889) : 826,	(26,889) : 121,	(27,889) : 77,	(28,889) : 123,	(29,889) : 373,	(30,889) : 123,	(31,889) : 173,	(32,889) : 88,	(33,889) : 55,	(34,889) : 183,	(35,889) : 243,	(36,889) : 487,	(37,889) : 510,	(38,889) : 187,	(39,889) : 182,	(40,889) : 844,
(1,890) : 523,	(2,890) : 498,	(3,890) : 204,	(4,890) : 734,	(5,890) : 455,	(6,890) : 452,	(7,890) : 513,	(8,890) : 386,	(9,890) : 223,	(10,890) : 515,	(11,890) : 499,	(12,890) : 844,	(13,890) : 519,	(14,890) : 224,	(15,890) : 620,	(16,890) : 840,	(17,890) : 693,	(18,890) : 444,	(19,890) : 796,	(20,890) : 519,	(21,890) : 495,	(22,890) : 371,	(23,890) : 443,	(24,890) : 694,	(25,890) : 266,	(26,890) : 545,	(27,890) : 519,	(28,890) : 517,	(29,890) : 561,	(30,890) : 523,	(31,890) : 463,	(32,890) : 481,	(33,890) : 602,	(34,890) : 427,	(35,890) : 597,	(36,890) : 247,	(37,890) : 114,	(38,890) : 481,	(39,890) : 495,	(40,890) : 286,
(1,891) : 607,	(2,891) : 573,	(3,891) : 275,	(4,891) : 861,	(5,891) : 524,	(6,891) : 532,	(7,891) : 597,	(8,891) : 347,	(9,891) : 262,	(10,891) : 599,	(11,891) : 574,	(12,891) : 966,	(13,891) : 627,	(14,891) : 266,	(15,891) : 745,	(16,891) : 967,	(17,891) : 819,	(18,891) : 517,	(19,891) : 916,	(20,891) : 603,	(21,891) : 616,	(22,891) : 451,	(23,891) : 518,	(24,891) : 821,	(25,891) : 175,	(26,891) : 659,	(27,891) : 627,	(28,891) : 601,	(29,891) : 688,	(30,891) : 606,	(31,891) : 542,	(32,891) : 577,	(33,891) : 697,	(34,891) : 509,	(35,891) : 660,	(36,891) : 255,	(37,891) : 156,	(38,891) : 555,	(39,891) : 569,	(40,891) : 186,
(1,892) : 352,	(2,892) : 402,	(3,892) : 678,	(4,892) : 264,	(5,892) : 433,	(6,892) : 385,	(7,892) : 348,	(8,892) : 967,	(9,892) : 590,	(10,892) : 349,	(11,892) : 403,	(12,892) : 206,	(13,892) : 190,	(14,892) : 585,	(15,892) : 132,	(16,892) : 295,	(17,892) : 220,	(18,892) : 418,	(19,892) : 152,	(20,892) : 347,	(21,892) : 153,	(22,892) : 414,	(23,892) : 412,	(24,892) : 223,	(25,892) : 905,	(26,892) : 128,	(27,892) : 185,	(28,892) : 347,	(29,892) : 211,	(30,892) : 349,	(31,892) : 385,	(32,892) : 285,	(33,892) : 263,	(34,892) : 382,	(35,892) : 470,	(36,892) : 649,	(37,892) : 631,	(38,892) : 405,	(39,892) : 403,	(40,892) : 928,
(1,893) : 753,	(2,893) : 808,	(3,893) : 1040,	(4,893) : 363,	(5,893) : 847,	(6,893) : 801,	(7,893) : 753,	(8,893) : 1328,	(9,893) : 1018,	(10,893) : 753,	(11,893) : 808,	(12,893) : 227,	(13,893) : 612,	(14,893) : 1013,	(15,893) : 433,	(16,893) : 247,	(17,893) : 383,	(18,893) : 834,	(19,893) : 289,	(20,893) : 750,	(21,893) : 562,	(22,893) : 841,	(23,893) : 829,	(24,893) : 384,	(25,893) : 1301,	(26,893) : 555,	(27,893) : 608,	(28,893) : 751,	(29,893) : 505,	(30,893) : 750,	(31,893) : 800,	(32,893) : 703,	(33,893) : 642,	(34,893) : 803,	(35,893) : 848,	(36,893) : 1078,	(37,893) : 1052,	(38,893) : 815,	(39,893) : 810,	(40,893) : 1327,
(1,894) : 319,	(2,894) : 336,	(3,894) : 410,	(4,894) : 398,	(5,894) : 331,	(6,894) : 290,	(7,894) : 308,	(8,894) : 694,	(9,894) : 358,	(10,894) : 311,	(11,894) : 338,	(12,894) : 477,	(13,894) : 205,	(14,894) : 354,	(15,894) : 261,	(16,894) : 490,	(17,894) : 350,	(18,894) : 312,	(19,894) : 430,	(20,894) : 312,	(21,894) : 128,	(22,894) : 259,	(23,894) : 307,	(24,894) : 352,	(25,894) : 626,	(26,894) : 200,	(27,894) : 201,	(28,894) : 310,	(29,894) : 226,	(30,894) : 316,	(31,894) : 299,	(32,894) : 233,	(33,894) : 327,	(34,894) : 270,	(35,894) : 446,	(36,894) : 420,	(37,894) : 369,	(38,894) : 327,	(39,894) : 335,	(40,894) : 650,
(1,895) : 1003,	(2,895) : 966,	(3,895) : 520,	(4,895) : 1206,	(5,895) : 916,	(6,895) : 928,	(7,895) : 994,	(8,895) : 291,	(9,895) : 650,	(10,895) : 996,	(11,895) : 967,	(12,895) : 1346,	(13,895) : 1024,	(14,895) : 654,	(15,895) : 1112,	(16,895) : 1319,	(17,895) : 1171,	(18,895) : 911,	(19,895) : 1300,	(20,895) : 1000,	(21,895) : 997,	(22,895) : 848,	(23,895) : 912,	(24,895) : 1171,	(25,895) : 244,	(26,895) : 1052,	(27,895) : 1024,	(28,895) : 998,	(29,895) : 1042,	(30,895) : 1003,	(31,895) : 938,	(32,895) : 976,	(33,895) : 1096,	(34,895) : 906,	(35,895) : 1045,	(36,895) : 624,	(37,895) : 554,	(38,895) : 949,	(39,895) : 963,	(40,895) : 221,
(1,896) : 579,	(2,896) : 634,	(3,896) : 941,	(4,896) : 330,	(5,896) : 676,	(6,896) : 632,	(7,896) : 580,	(8,896) : 1232,	(9,896) : 865,	(10,896) : 579,	(11,896) : 635,	(12,896) : 84,	(13,896) : 448,	(14,896) : 860,	(15,896) : 330,	(16,896) : 248,	(17,896) : 326,	(18,896) : 664,	(19,896) : 128,	(20,896) : 577,	(21,896) : 429,	(22,896) : 677,	(23,896) : 659,	(24,896) : 327,	(25,896) : 1182,	(26,896) : 397,	(27,896) : 445,	(28,896) : 577,	(29,896) : 417,	(30,896) : 576,	(31,896) : 629,	(32,896) : 536,	(33,896) : 466,	(34,896) : 635,	(35,896) : 669,	(36,896) : 922,	(37,896) : 911,	(38,896) : 643,	(39,896) : 636,	(40,896) : 1206,
(1,897) : 451,	(2,897) : 502,	(3,897) : 736,	(4,897) : 198,	(5,897) : 534,	(6,897) : 486,	(7,897) : 448,	(8,897) : 1026,	(9,897) : 685,	(10,897) : 449,	(11,897) : 503,	(12,897) : 122,	(13,897) : 291,	(14,897) : 681,	(15,897) : 129,	(16,897) : 200,	(17,897) : 166,	(18,897) : 519,	(19,897) : 94,	(20,897) : 447,	(21,897) : 230,	(22,897) : 515,	(23,897) : 514,	(24,897) : 169,	(25,897) : 980,	(26,897) : 229,	(27,897) : 286,	(28,897) : 447,	(29,897) : 219,	(30,897) : 448,	(31,897) : 486,	(32,897) : 386,	(33,897) : 356,	(34,897) : 483,	(35,897) : 566,	(36,897) : 745,	(37,897) : 720,	(38,897) : 506,	(39,897) : 504,	(40,897) : 1004,
(1,898) : 694,	(2,898) : 749,	(3,898) : 1005,	(4,898) : 341,	(5,898) : 789,	(6,898) : 744,	(7,898) : 695,	(8,898) : 1294,	(9,898) : 966,	(10,898) : 694,	(11,898) : 750,	(12,898) : 171,	(13,898) : 556,	(14,898) : 961,	(15,898) : 392,	(16,898) : 232,	(17,898) : 355,	(18,898) : 777,	(19,898) : 232,	(20,898) : 692,	(21,898) : 514,	(22,898) : 785,	(23,898) : 772,	(24,898) : 355,	(25,898) : 1260,	(26,898) : 500,	(27,898) : 552,	(28,898) : 692,	(29,898) : 470,	(30,898) : 692,	(31,898) : 742,	(32,898) : 646,	(33,898) : 583,	(34,898) : 746,	(35,898) : 788,	(36,898) : 1025,	(37,898) : 1004,	(38,898) : 757,	(39,898) : 751,	(40,898) : 1285,
(1,899) : 694,	(2,899) : 664,	(3,899) : 221,	(4,899) : 876,	(5,899) : 618,	(6,899) : 620,	(7,899) : 683,	(8,899) : 236,	(9,899) : 364,	(10,899) : 686,	(11,899) : 665,	(12,899) : 1008,	(13,899) : 694,	(14,899) : 367,	(15,899) : 776,	(16,899) : 988,	(17,899) : 839,	(18,899) : 609,	(19,899) : 963,	(20,899) : 689,	(21,899) : 660,	(22,899) : 539,	(23,899) : 609,	(24,899) : 840,	(25,899) : 93,	(26,899) : 718,	(27,899) : 693,	(28,899) : 687,	(29,899) : 709,	(30,899) : 693,	(31,899) : 631,	(32,899) : 655,	(33,899) : 776,	(34,899) : 597,	(35,899) : 757,	(36,899) : 363,	(37,899) : 252,	(38,899) : 647,	(39,899) : 661,	(40,899) : 118,
(1,900) : 147,	(2,900) : 116,	(3,900) : 748,	(4,900) : 745,	(5,900) : 137,	(6,900) : 165,	(7,900) : 154,	(8,900) : 970,	(9,900) : 381,	(10,900) : 152,	(11,900) : 115,	(12,900) : 655,	(13,900) : 317,	(14,900) : 378,	(15,900) : 590,	(16,900) : 786,	(17,900) : 696,	(18,900) : 153,	(19,900) : 591,	(20,900) : 152,	(21,900) : 463,	(22,900) : 228,	(23,900) : 156,	(24,900) : 699,	(25,900) : 818,	(26,900) : 379,	(27,900) : 323,	(28,900) : 154,	(29,900) : 619,	(30,900) : 150,	(31,900) : 155,	(32,900) : 233,	(33,900) : 235,	(34,900) : 189,	(35,900) : 29,	(36,900) : 402,	(37,900) : 491,	(38,900) : 127,	(39,900) : 118,	(40,900) : 828,
(1,901) : 555,	(2,901) : 600,	(3,901) : 671,	(4,901) : 45,	(5,901) : 621,	(6,901) : 572,	(7,901) : 550,	(8,901) : 957,	(9,901) : 716,	(10,901) : 551,	(11,901) : 601,	(12,901) : 240,	(13,901) : 384,	(14,901) : 712,	(15,901) : 112,	(16,901) : 137,	(17,901) : 11,	(18,901) : 603,	(19,901) : 245,	(20,901) : 550,	(21,901) : 260,	(22,901) : 580,	(23,901) : 597,	(24,901) : 11,	(25,901) : 941,	(26,901) : 321,	(27,901) : 378,	(28,901) : 549,	(29,901) : 142,	(30,901) : 552,	(31,901) : 575,	(32,901) : 476,	(33,901) : 481,	(34,901) : 562,	(35,901) : 681,	(36,901) : 778,	(37,901) : 725,	(38,901) : 599,	(39,901) : 600,	(40,901) : 968,
(1,902) : 936,	(2,902) : 902,	(3,902) : 433,	(4,902) : 1119,	(5,902) : 852,	(6,902) : 861,	(7,902) : 926,	(8,902) : 215,	(9,902) : 587,	(10,902) : 928,	(11,902) : 902,	(12,902) : 1263,	(13,902) : 948,	(14,902) : 591,	(15,902) : 1027,	(16,902) : 1233,	(17,902) : 1085,	(18,902) : 846,	(19,902) : 1218,	(20,902) : 932,	(21,902) : 916,	(22,902) : 780,	(23,902) : 847,	(24,902) : 1086,	(25,902) : 163,	(26,902) : 974,	(27,902) : 947,	(28,902) : 930,	(29,902) : 957,	(30,902) : 936,	(31,902) : 871,	(32,902) : 904,	(33,902) : 1025,	(34,902) : 839,	(35,902) : 985,	(36,902) : 568,	(37,902) : 486,	(38,902) : 884,	(39,902) : 898,	(40,902) : 144,
(1,903) : 911,	(2,903) : 878,	(3,903) : 387,	(4,903) : 1075,	(5,903) : 830,	(6,903) : 836,	(7,903) : 901,	(8,903) : 169,	(9,903) : 567,	(10,903) : 903,	(11,903) : 879,	(12,903) : 1223,	(13,903) : 915,	(14,903) : 571,	(15,903) : 986,	(16,903) : 1189,	(17,903) : 1042,	(18,903) : 823,	(19,903) : 1180,	(20,903) : 907,	(21,903) : 878,	(22,903) : 755,	(23,903) : 823,	(24,903) : 1042,	(25,903) : 133,	(26,903) : 939,	(27,903) : 915,	(28,903) : 904,	(29,903) : 914,	(30,903) : 910,	(31,903) : 847,	(32,903) : 875,	(33,903) : 996,	(34,903) : 813,	(35,903) : 966,	(36,903) : 553,	(37,903) : 462,	(38,903) : 861,	(39,903) : 875,	(40,903) : 121,
(1,904) : 637,	(2,904) : 603,	(3,904) : 270,	(4,904) : 878,	(5,904) : 555,	(6,904) : 562,	(7,904) : 627,	(8,904) : 319,	(9,904) : 293,	(10,904) : 629,	(11,904) : 604,	(12,904) : 989,	(13,904) : 654,	(14,904) : 296,	(15,904) : 765,	(16,904) : 985,	(17,904) : 837,	(18,904) : 548,	(19,904) : 940,	(20,904) : 633,	(21,904) : 639,	(22,904) : 481,	(23,904) : 548,	(24,904) : 838,	(25,904) : 144,	(26,904) : 685,	(27,904) : 654,	(28,904) : 631,	(29,904) : 706,	(30,904) : 637,	(31,904) : 572,	(32,904) : 606,	(33,904) : 726,	(34,904) : 539,	(35,904) : 691,	(36,904) : 285,	(37,904) : 187,	(38,904) : 586,	(39,904) : 600,	(40,904) : 156,
(1,905) : 506,	(2,905) : 471,	(3,905) : 298,	(4,905) : 797,	(5,905) : 423,	(6,905) : 430,	(7,905) : 496,	(8,905) : 435,	(9,905) : 163,	(10,905) : 498,	(11,905) : 472,	(12,905) : 884,	(13,905) : 533,	(14,905) : 166,	(15,905) : 672,	(16,905) : 899,	(17,905) : 754,	(18,905) : 416,	(19,905) : 831,	(20,905) : 502,	(21,905) : 536,	(22,905) : 350,	(23,905) : 417,	(24,905) : 755,	(25,905) : 276,	(26,905) : 569,	(27,905) : 534,	(28,905) : 500,	(29,905) : 623,	(30,905) : 506,	(31,905) : 441,	(32,905) : 479,	(33,905) : 598,	(34,905) : 408,	(35,905) : 559,	(36,905) : 166,	(37,905) : 56,	(38,905) : 454,	(39,905) : 468,	(40,905) : 287,
(1,906) : 788,	(2,906) : 771,	(3,906) : 148,	(4,906) : 831,	(5,906) : 731,	(6,906) : 722,	(7,906) : 777,	(8,906) : 143,	(9,906) : 506,	(10,906) : 780,	(11,906) : 772,	(12,906) : 1006,	(13,906) : 748,	(14,906) : 508,	(15,906) : 761,	(16,906) : 948,	(17,906) : 803,	(18,906) : 718,	(19,906) : 970,	(20,906) : 783,	(21,906) : 675,	(22,906) : 643,	(23,906) : 717,	(24,906) : 803,	(25,906) : 218,	(26,906) : 756,	(27,906) : 745,	(28,906) : 781,	(29,906) : 681,	(30,906) : 787,	(31,906) : 733,	(32,906) : 732,	(33,906) : 850,	(34,906) : 696,	(35,906) : 875,	(36,906) : 522,	(37,906) : 394,	(38,906) : 754,	(39,906) : 767,	(40,906) : 246,
(1,907) : 355,	(2,907) : 315,	(3,907) : 418,	(4,907) : 752,	(5,907) : 264,	(6,907) : 280,	(7,907) : 347,	(8,907) : 597,	(9,907) : 3,	(10,907) : 349,	(11,907) : 316,	(12,907) : 793,	(13,907) : 417,	(14,907) : 2,	(15,907) : 610,	(16,907) : 841,	(17,907) : 705,	(18,907) : 260,	(19,907) : 735,	(20,907) : 353,	(21,907) : 462,	(22,907) : 204,	(23,907) : 262,	(24,907) : 706,	(25,907) : 439,	(26,907) : 465,	(27,907) : 419,	(28,907) : 351,	(29,907) : 581,	(30,907) : 356,	(31,907) : 289,	(32,907) : 345,	(33,907) : 458,	(34,907) : 261,	(35,907) : 397,	(36,907) : 65,	(37,907) : 115,	(38,907) : 298,	(39,907) : 312,	(40,907) : 448,
(1,908) : 349,	(2,908) : 307,	(3,908) : 437,	(4,908) : 763,	(5,908) : 256,	(6,908) : 274,	(7,908) : 341,	(8,908) : 613,	(9,908) : 19,	(10,908) : 343,	(11,908) : 308,	(12,908) : 797,	(13,908) : 419,	(14,908) : 19,	(15,908) : 618,	(16,908) : 849,	(17,908) : 715,	(18,908) : 253,	(19,908) : 739,	(20,908) : 347,	(21,908) : 470,	(22,908) : 200,	(23,908) : 254,	(24,908) : 717,	(25,908) : 452,	(26,908) : 469,	(27,908) : 422,	(28,908) : 345,	(29,908) : 593,	(30,908) : 350,	(31,908) : 283,	(32,908) : 344,	(33,908) : 455,	(34,908) : 256,	(35,908) : 385,	(36,908) : 57,	(37,908) : 132,	(38,908) : 290,	(39,908) : 304,	(40,908) : 461,
(1,909) : 1150,	(2,909) : 1114,	(3,909) : 625,	(4,909) : 1316,	(5,909) : 1064,	(6,909) : 1074,	(7,909) : 1140,	(8,909) : 355,	(9,909) : 798,	(10,909) : 1142,	(11,909) : 1115,	(12,909) : 1471,	(13,909) : 1163,	(14,909) : 803,	(15,909) : 1233,	(16,909) : 1432,	(17,909) : 1285,	(18,909) : 1059,	(19,909) : 1429,	(20,909) : 1146,	(21,909) : 1126,	(22,909) : 994,	(23,909) : 1060,	(24,909) : 1285,	(25,909) : 378,	(26,909) : 1187,	(27,909) : 1162,	(28,909) : 1144,	(29,909) : 1159,	(30,909) : 1150,	(31,909) : 1085,	(32,909) : 1119,	(33,909) : 1239,	(34,909) : 1053,	(35,909) : 1195,	(36,909) : 775,	(37,909) : 700,	(38,909) : 1097,	(39,909) : 1111,	(40,909) : 359,
(1,910) : 7,	(2,910) : 62,	(3,910) : 658,	(4,910) : 593,	(5,910) : 109,	(6,910) : 79,	(7,910) : 12,	(8,910) : 907,	(9,910) : 361,	(10,910) : 10,	(11,910) : 62,	(12,910) : 522,	(13,910) : 165,	(14,910) : 356,	(15,910) : 438,	(16,910) : 639,	(17,910) : 544,	(18,910) : 104,	(19,910) : 458,	(20,910) : 6,	(21,910) : 311,	(22,910) : 158,	(23,910) : 100,	(24,910) : 547,	(25,910) : 780,	(26,910) : 228,	(27,910) : 171,	(28,910) : 8,	(29,910) : 467,	(30,910) : 4,	(31,910) : 70,	(32,910) : 83,	(33,910) : 112,	(34,910) : 99,	(35,910) : 138,	(36,910) : 402,	(37,910) : 452,	(38,910) : 72,	(39,910) : 64,	(40,910) : 794,
(1,911) : 349,	(2,911) : 399,	(3,911) : 679,	(4,911) : 267,	(5,911) : 430,	(6,911) : 382,	(7,911) : 346,	(8,911) : 967,	(9,911) : 588,	(10,911) : 346,	(11,911) : 400,	(12,911) : 207,	(13,911) : 187,	(14,911) : 583,	(15,911) : 134,	(16,911) : 298,	(17,911) : 223,	(18,911) : 415,	(19,911) : 153,	(20,911) : 345,	(21,911) : 152,	(22,911) : 412,	(23,911) : 410,	(24,911) : 226,	(25,911) : 905,	(26,911) : 126,	(27,911) : 182,	(28,911) : 344,	(29,911) : 213,	(30,911) : 346,	(31,911) : 383,	(32,911) : 282,	(33,911) : 260,	(34,911) : 380,	(35,911) : 467,	(36,911) : 647,	(37,911) : 629,	(38,911) : 403,	(39,911) : 401,	(40,911) : 928,
(1,912) : 798,	(2,912) : 778,	(3,912) : 174,	(4,912) : 862,	(5,912) : 737,	(6,912) : 730,	(7,912) : 787,	(8,912) : 117,	(9,912) : 504,	(10,912) : 790,	(11,912) : 779,	(12,912) : 1032,	(13,912) : 765,	(14,912) : 506,	(15,912) : 788,	(16,912) : 978,	(17,912) : 833,	(18,912) : 725,	(19,912) : 995,	(20,912) : 793,	(21,912) : 698,	(22,912) : 651,	(23,912) : 724,	(24,912) : 833,	(25,912) : 186,	(26,912) : 776,	(27,912) : 763,	(28,912) : 791,	(29,912) : 709,	(30,912) : 797,	(31,912) : 741,	(32,912) : 745,	(33,912) : 864,	(34,912) : 705,	(35,912) : 880,	(36,912) : 516,	(37,912) : 391,	(38,912) : 761,	(39,912) : 775,	(40,912) : 213,
(1,913) : 343,	(2,913) : 300,	(3,913) : 454,	(4,913) : 771,	(5,913) : 248,	(6,913) : 268,	(7,913) : 336,	(8,913) : 628,	(9,913) : 36,	(10,913) : 338,	(11,913) : 300,	(12,913) : 800,	(13,913) : 420,	(14,913) : 35,	(15,913) : 625,	(16,913) : 856,	(17,913) : 722,	(18,913) : 246,	(19,913) : 741,	(20,913) : 341,	(21,913) : 476,	(22,913) : 198,	(23,913) : 248,	(24,913) : 724,	(25,913) : 465,	(26,913) : 471,	(27,913) : 423,	(28,913) : 339,	(29,913) : 601,	(30,913) : 344,	(31,913) : 277,	(32,913) : 343,	(33,913) : 451,	(34,913) : 252,	(35,913) : 374,	(36,913) : 56,	(37,913) : 147,	(38,913) : 283,	(39,913) : 296,	(40,913) : 473,
(1,914) : 491,	(2,914) : 533,	(3,914) : 603,	(4,914) : 119,	(5,914) : 550,	(6,914) : 502,	(7,914) : 485,	(8,914) : 891,	(9,914) : 636,	(10,914) : 486,	(11,914) : 534,	(12,914) : 272,	(13,914) : 321,	(14,914) : 632,	(15,914) : 53,	(16,914) : 214,	(17,914) : 71,	(18,914) : 532,	(19,914) : 258,	(20,914) : 486,	(21,914) : 185,	(22,914) : 505,	(23,914) : 527,	(24,914) : 72,	(25,914) : 867,	(26,914) : 260,	(27,914) : 314,	(28,914) : 485,	(29,914) : 68,	(30,914) : 488,	(31,914) : 506,	(32,914) : 409,	(33,914) : 427,	(34,914) : 491,	(35,914) : 620,	(36,914) : 699,	(37,914) : 645,	(38,914) : 531,	(39,914) : 533,	(40,914) : 894,
(1,915) : 1059,	(2,915) : 1022,	(3,915) : 573,	(4,915) : 1260,	(5,915) : 971,	(6,915) : 983,	(7,915) : 1050,	(8,915) : 331,	(9,915) : 705,	(10,915) : 1052,	(11,915) : 1022,	(12,915) : 1402,	(13,915) : 1081,	(14,915) : 709,	(15,915) : 1168,	(16,915) : 1374,	(17,915) : 1226,	(18,915) : 966,	(19,915) : 1357,	(20,915) : 1056,	(21,915) : 1054,	(22,915) : 904,	(23,915) : 968,	(24,915) : 1227,	(25,915) : 301,	(26,915) : 1109,	(27,915) : 1081,	(28,915) : 1054,	(29,915) : 1098,	(30,915) : 1059,	(31,915) : 993,	(32,915) : 1033,	(33,915) : 1152,	(34,915) : 962,	(35,915) : 1099,	(36,915) : 678,	(37,915) : 610,	(38,915) : 1004,	(39,915) : 1018,	(40,915) : 278,
(1,916) : 873,	(2,916) : 837,	(3,916) : 415,	(4,916) : 1089,	(5,916) : 787,	(6,916) : 798,	(7,916) : 863,	(8,916) : 249,	(9,916) : 521,	(10,916) : 866,	(11,916) : 838,	(12,916) : 1220,	(13,916) : 894,	(14,916) : 525,	(15,916) : 989,	(16,916) : 1201,	(17,916) : 1053,	(18,916) : 781,	(19,916) : 1174,	(20,916) : 869,	(21,916) : 871,	(22,916) : 718,	(23,916) : 782,	(24,916) : 1053,	(25,916) : 123,	(26,916) : 923,	(27,916) : 894,	(28,916) : 867,	(29,916) : 923,	(30,916) : 873,	(31,916) : 808,	(32,916) : 846,	(33,916) : 965,	(34,916) : 776,	(35,916) : 918,	(36,916) : 498,	(37,916) : 423,	(38,916) : 820,	(39,916) : 833,	(40,916) : 95,
(1,917) : 954,	(2,917) : 926,	(3,917) : 376,	(4,917) : 1066,	(5,917) : 880,	(6,917) : 881,	(7,917) : 944,	(8,917) : 104,	(9,917) : 623,	(10,917) : 946,	(11,917) : 927,	(12,917) : 1231,	(13,917) : 944,	(14,917) : 627,	(15,917) : 989,	(16,917) : 1182,	(17,917) : 1036,	(18,917) : 871,	(19,917) : 1191,	(20,917) : 950,	(21,917) : 891,	(22,917) : 800,	(23,917) : 871,	(24,917) : 1036,	(25,917) : 190,	(26,917) : 961,	(27,917) : 942,	(28,917) : 948,	(29,917) : 912,	(30,917) : 954,	(31,917) : 892,	(32,917) : 912,	(33,917) : 1033,	(34,917) : 858,	(35,917) : 1019,	(36,917) : 616,	(37,917) : 513,	(38,917) : 909,	(39,917) : 923,	(40,917) : 191,
(1,918) : 62,	(2,918) : 94,	(3,918) : 606,	(4,918) : 551,	(5,918) : 120,	(6,918) : 72,	(7,918) : 52,	(8,918) : 861,	(9,918) : 336,	(10,918) : 54,	(11,918) : 95,	(12,918) : 502,	(13,918) : 125,	(14,918) : 331,	(15,918) : 395,	(16,918) : 605,	(17,918) : 501,	(18,918) : 105,	(19,918) : 439,	(20,918) : 56,	(21,918) : 260,	(22,918) : 130,	(23,918) : 100,	(24,918) : 504,	(25,918) : 741,	(26,918) : 189,	(27,918) : 130,	(28,918) : 54,	(29,918) : 417,	(30,918) : 60,	(31,918) : 72,	(32,918) : 31,	(33,918) : 126,	(34,918) : 77,	(35,918) : 192,	(36,918) : 383,	(37,918) : 418,	(38,918) : 93,	(39,918) : 94,	(40,918) : 757,
(1,919) : 628,	(2,919) : 594,	(3,919) : 274,	(4,919) : 874,	(5,919) : 545,	(6,919) : 553,	(7,919) : 618,	(8,919) : 329,	(9,919) : 283,	(10,919) : 620,	(11,919) : 595,	(12,919) : 984,	(13,919) : 647,	(14,919) : 287,	(15,919) : 761,	(16,919) : 981,	(17,919) : 834,	(18,919) : 538,	(19,919) : 934,	(20,919) : 624,	(21,919) : 634,	(22,919) : 472,	(23,919) : 539,	(24,919) : 835,	(25,919) : 154,	(26,919) : 678,	(27,919) : 647,	(28,919) : 622,	(29,919) : 702,	(30,919) : 628,	(31,919) : 563,	(32,919) : 598,	(33,919) : 718,	(34,919) : 530,	(35,919) : 681,	(36,919) : 274,	(37,919) : 178,	(38,919) : 577,	(39,919) : 590,	(40,919) : 165,
(1,920) : 937,	(2,920) : 907,	(3,920) : 381,	(4,920) : 1072,	(5,920) : 859,	(6,920) : 863,	(7,920) : 926,	(8,920) : 131,	(9,920) : 600,	(10,920) : 929,	(11,920) : 908,	(12,920) : 1230,	(13,920) : 933,	(14,920) : 603,	(15,920) : 990,	(16,920) : 1188,	(17,920) : 1041,	(18,920) : 851,	(19,920) : 1188,	(20,920) : 932,	(21,920) : 887,	(22,920) : 782,	(23,920) : 851,	(24,920) : 1041,	(25,920) : 163,	(26,920) : 953,	(27,920) : 932,	(28,920) : 930,	(29,920) : 915,	(30,920) : 936,	(31,920) : 874,	(32,920) : 897,	(33,920) : 1018,	(34,920) : 840,	(35,920) : 997,	(36,920) : 589,	(37,920) : 491,	(38,920) : 889,	(39,920) : 903,	(40,920) : 159,
(1,921) : 15,	(2,921) : 45,	(3,921) : 647,	(4,921) : 603,	(5,921) : 90,	(6,921) : 61,	(7,921) : 10,	(8,921) : 893,	(9,921) : 342,	(10,921) : 11,	(11,921) : 45,	(12,921) : 538,	(13,921) : 174,	(14,921) : 338,	(15,921) : 447,	(16,921) : 652,	(17,921) : 554,	(18,921) : 84,	(19,921) : 474,	(20,921) : 14,	(21,921) : 316,	(22,921) : 140,	(23,921) : 81,	(24,921) : 556,	(25,921) : 763,	(26,921) : 238,	(27,921) : 180,	(28,921) : 13,	(29,921) : 473,	(30,921) : 16,	(31,921) : 51,	(32,921) : 86,	(33,921) : 131,	(34,921) : 82,	(35,921) : 135,	(36,921) : 383,	(37,921) : 435,	(38,921) : 54,	(39,921) : 47,	(40,921) : 777,
(1,922) : 464,	(2,922) : 495,	(3,922) : 475,	(4,922) : 230,	(5,922) : 501,	(6,922) : 455,	(7,922) : 456,	(8,922) : 763,	(9,922) : 538,	(10,922) : 458,	(11,922) : 497,	(12,922) : 391,	(13,922) : 307,	(14,922) : 535,	(15,922) : 145,	(16,922) : 339,	(17,922) : 190,	(18,922) : 482,	(19,922) : 365,	(20,922) : 458,	(21,922) : 153,	(22,922) : 439,	(23,922) : 476,	(24,922) : 191,	(25,922) : 740,	(26,922) : 261,	(27,922) : 301,	(28,922) : 456,	(29,922) : 61,	(30,922) : 461,	(31,922) : 462,	(32,922) : 376,	(33,922) : 431,	(34,922) : 439,	(35,922) : 596,	(36,922) : 601,	(37,922) : 533,	(38,922) : 490,	(39,922) : 495,	(40,922) : 767,
(1,923) : 803,	(2,923) : 789,	(3,923) : 147,	(4,923) : 808,	(5,923) : 752,	(6,923) : 739,	(7,923) : 792,	(8,923) : 170,	(9,923) : 537,	(10,923) : 795,	(11,923) : 790,	(12,923) : 992,	(13,923) : 752,	(14,923) : 539,	(15,923) : 746,	(16,923) : 925,	(17,923) : 782,	(18,923) : 738,	(19,923) : 959,	(20,923) : 798,	(21,923) : 670,	(22,923) : 663,	(23,923) : 736,	(24,923) : 781,	(25,923) : 270,	(26,923) : 756,	(27,923) : 749,	(28,923) : 796,	(29,923) : 663,	(30,923) : 802,	(31,923) : 750,	(32,923) : 743,	(33,923) : 859,	(34,923) : 713,	(35,923) : 895,	(36,923) : 558,	(37,923) : 427,	(38,923) : 773,	(39,923) : 786,	(40,923) : 298,
(1,924) : 746,	(2,924) : 726,	(3,924) : 133,	(4,924) : 824,	(5,924) : 685,	(6,924) : 677,	(7,924) : 735,	(8,924) : 167,	(9,924) : 454,	(10,924) : 737,	(11,924) : 727,	(12,924) : 987,	(13,924) : 713,	(14,924) : 457,	(15,924) : 744,	(16,924) : 939,	(17,924) : 792,	(18,924) : 673,	(19,924) : 948,	(20,924) : 741,	(21,924) : 650,	(22,924) : 598,	(23,924) : 672,	(24,924) : 793,	(25,924) : 181,	(26,924) : 726,	(27,924) : 712,	(28,924) : 738,	(29,924) : 667,	(30,924) : 745,	(31,924) : 689,	(32,924) : 692,	(33,924) : 812,	(34,924) : 652,	(35,924) : 828,	(36,924) : 469,	(37,924) : 342,	(38,924) : 709,	(39,924) : 722,	(40,924) : 211,
(1,925) : 244,	(2,925) : 290,	(3,925) : 603,	(4,925) : 355,	(5,925) : 316,	(6,925) : 267,	(7,925) : 239,	(8,925) : 886,	(9,925) : 470,	(10,925) : 240,	(11,925) : 291,	(12,925) : 326,	(13,925) : 74,	(14,925) : 465,	(15,925) : 200,	(16,925) : 408,	(17,925) : 306,	(18,925) : 299,	(19,925) : 268,	(20,925) : 239,	(21,925) : 99,	(22,925) : 292,	(23,925) : 294,	(24,925) : 309,	(25,925) : 807,	(26,925) : 12,	(27,925) : 68,	(28,925) : 238,	(29,925) : 241,	(30,925) : 241,	(31,925) : 268,	(32,925) : 168,	(33,925) : 184,	(34,925) : 262,	(35,925) : 372,	(36,925) : 528,	(37,925) : 517,	(38,925) : 291,	(39,925) : 291,	(40,925) : 828,
(1,926) : 916,	(2,926) : 894,	(3,926) : 290,	(4,926) : 969,	(5,926) : 852,	(6,926) : 847,	(7,926) : 905,	(8,926) : 23,	(9,926) : 610,	(10,926) : 908,	(11,926) : 895,	(12,926) : 1147,	(13,926) : 885,	(14,926) : 613,	(15,926) : 902,	(16,926) : 1085,	(17,926) : 941,	(18,926) : 841,	(19,926) : 1112,	(20,926) : 911,	(21,926) : 817,	(22,926) : 767,	(23,926) : 840,	(24,926) : 941,	(25,926) : 222,	(26,926) : 896,	(27,926) : 883,	(28,926) : 909,	(29,926) : 821,	(30,926) : 915,	(31,926) : 858,	(32,926) : 864,	(33,926) : 984,	(34,926) : 822,	(35,926) : 994,	(36,926) : 615,	(37,926) : 496,	(38,926) : 877,	(39,926) : 891,	(40,926) : 239,
(1,927) : 869,	(2,927) : 855,	(3,927) : 211,	(4,927) : 851,	(5,927) : 817,	(6,927) : 805,	(7,927) : 857,	(8,927) : 154,	(9,927) : 601,	(10,927) : 860,	(11,927) : 856,	(12,927) : 1044,	(13,927) : 814,	(14,927) : 603,	(15,927) : 798,	(16,927) : 968,	(17,927) : 827,	(18,927) : 804,	(19,927) : 1013,	(20,927) : 863,	(21,927) : 727,	(22,927) : 728,	(23,927) : 802,	(24,927) : 827,	(25,927) : 301,	(26,927) : 816,	(27,927) : 811,	(28,927) : 861,	(29,927) : 713,	(30,927) : 867,	(31,927) : 816,	(32,927) : 807,	(33,927) : 923,	(34,927) : 779,	(35,927) : 961,	(36,927) : 620,	(37,927) : 491,	(38,927) : 838,	(39,927) : 852,	(40,927) : 326,
(1,928) : 948,	(2,928) : 923,	(3,928) : 339,	(4,928) : 1023,	(5,928) : 879,	(6,928) : 877,	(7,928) : 937,	(8,928) : 49,	(9,928) : 630,	(10,928) : 940,	(11,928) : 924,	(12,928) : 1197,	(13,928) : 925,	(14,928) : 633,	(15,928) : 953,	(16,928) : 1140,	(17,928) : 995,	(18,928) : 869,	(19,928) : 1160,	(20,928) : 943,	(21,928) : 863,	(22,928) : 796,	(23,928) : 868,	(24,928) : 995,	(25,928) : 215,	(26,928) : 939,	(27,928) : 924,	(28,928) : 941,	(29,928) : 873,	(30,928) : 947,	(31,928) : 888,	(32,928) : 900,	(33,928) : 1020,	(34,928) : 852,	(35,928) : 1020,	(36,928) : 629,	(37,928) : 517,	(38,928) : 906,	(39,928) : 920,	(40,928) : 225,
(1,929) : 1048,	(2,929) : 1012,	(3,929) : 547,	(4,929) : 1235,	(5,929) : 962,	(6,929) : 973,	(7,929) : 1038,	(8,929) : 298,	(9,929) : 696,	(10,929) : 1041,	(11,929) : 1013,	(12,929) : 1381,	(13,929) : 1065,	(14,929) : 700,	(15,929) : 1145,	(16,929) : 1350,	(17,929) : 1202,	(18,929) : 956,	(19,929) : 1337,	(20,929) : 1044,	(21,929) : 1034,	(22,929) : 893,	(23,929) : 957,	(24,929) : 1202,	(25,929) : 281,	(26,929) : 1092,	(27,929) : 1065,	(28,929) : 1042,	(29,929) : 1074,	(30,929) : 1048,	(31,929) : 983,	(32,929) : 1019,	(33,929) : 1139,	(34,929) : 951,	(35,929) : 1092,	(36,929) : 671,	(37,929) : 598,	(38,929) : 995,	(39,929) : 1008,	(40,929) : 260,
(1,930) : 563,	(2,930) : 609,	(3,930) : 690,	(4,930) : 41,	(5,930) : 631,	(6,930) : 582,	(7,930) : 558,	(8,930) : 976,	(9,930) : 732,	(10,930) : 559,	(11,930) : 610,	(12,930) : 226,	(13,930) : 393,	(14,930) : 727,	(15,930) : 124,	(16,930) : 119,	(17,930) : 31,	(18,930) : 614,	(19,930) : 235,	(20,930) : 558,	(21,930) : 274,	(22,930) : 592,	(23,930) : 608,	(24,930) : 31,	(25,930) : 960,	(26,930) : 329,	(27,930) : 387,	(28,930) : 557,	(29,930) : 161,	(30,930) : 560,	(31,930) : 585,	(32,930) : 485,	(33,930) : 486,	(34,930) : 573,	(35,930) : 688,	(36,930) : 794,	(37,930) : 742,	(38,930) : 609,	(39,930) : 609,	(40,930) : 987,
(1,931) : 131,	(2,931) : 169,	(3,931) : 584,	(4,931) : 476,	(5,931) : 192,	(6,931) : 143,	(7,931) : 123,	(8,931) : 852,	(9,931) : 370,	(10,931) : 125,	(11,931) : 170,	(12,931) : 439,	(13,931) : 55,	(14,931) : 365,	(15,931) : 319,	(16,931) : 532,	(17,931) : 426,	(18,931) : 175,	(19,931) : 378,	(20,931) : 125,	(21,931) : 186,	(22,931) : 175,	(23,931) : 170,	(24,931) : 428,	(25,931) : 748,	(26,931) : 116,	(27,931) : 59,	(28,931) : 123,	(29,931) : 343,	(30,931) : 128,	(31,931) : 145,	(32,931) : 45,	(33,931) : 128,	(34,931) : 139,	(35,931) : 263,	(36,931) : 424,	(37,931) : 436,	(38,931) : 168,	(39,931) : 169,	(40,931) : 766,
(1,932) : 231,	(2,932) : 277,	(3,932) : 604,	(4,932) : 369,	(5,932) : 303,	(6,932) : 254,	(7,932) : 226,	(8,932) : 886,	(9,932) : 461,	(10,932) : 227,	(11,932) : 278,	(12,932) : 336,	(13,932) : 61,	(14,932) : 456,	(15,932) : 213,	(16,932) : 421,	(17,932) : 319,	(18,932) : 287,	(19,932) : 277,	(20,932) : 226,	(21,932) : 108,	(22,932) : 281,	(23,932) : 281,	(24,932) : 322,	(25,932) : 803,	(26,932) : 7,	(27,932) : 55,	(28,932) : 225,	(29,932) : 253,	(30,932) : 228,	(31,932) : 256,	(32,932) : 155,	(33,932) : 172,	(34,932) : 249,	(35,932) : 359,	(36,932) : 519,	(37,932) : 511,	(38,932) : 278,	(39,932) : 278,	(40,932) : 825,
(1,933) : 623,	(2,933) : 671,	(3,933) : 769,	(4,933) : 80,	(5,933) : 697,	(6,933) : 648,	(7,933) : 619,	(8,933) : 1053,	(9,933) : 810,	(10,933) : 620,	(11,933) : 672,	(12,933) : 205,	(13,933) : 455,	(14,933) : 806,	(15,933) : 198,	(16,933) : 38,	(17,933) : 111,	(18,933) : 680,	(19,933) : 236,	(20,933) : 618,	(21,933) : 348,	(22,933) : 664,	(23,933) : 675,	(24,933) : 110,	(25,933) : 1041,	(26,933) : 391,	(27,933) : 450,	(28,933) : 618,	(29,933) : 242,	(30,933) : 620,	(31,933) : 650,	(32,933) : 550,	(33,933) : 536,	(34,933) : 641,	(35,933) : 744,	(36,933) : 872,	(37,933) : 823,	(38,933) : 673,	(39,933) : 672,	(40,933) : 1068,
(1,934) : 810,	(2,934) : 786,	(3,934) : 223,	(4,934) : 914,	(5,934) : 742,	(6,934) : 739,	(7,934) : 799,	(8,934) : 100,	(9,934) : 496,	(10,934) : 802,	(11,934) : 787,	(12,934) : 1074,	(13,934) : 790,	(14,934) : 499,	(15,934) : 833,	(16,934) : 1029,	(17,934) : 883,	(18,934) : 732,	(19,934) : 1035,	(20,934) : 805,	(21,934) : 735,	(22,934) : 659,	(23,934) : 731,	(24,934) : 883,	(25,934) : 126,	(26,934) : 806,	(27,934) : 789,	(28,934) : 803,	(29,934) : 757,	(30,934) : 810,	(31,934) : 750,	(32,934) : 763,	(33,934) : 883,	(34,934) : 715,	(35,934) : 884,	(36,934) : 500,	(37,934) : 383,	(38,934) : 769,	(39,934) : 782,	(40,934) : 151,
(1,935) : 213,	(2,935) : 248,	(3,935) : 539,	(4,935) : 404,	(5,935) : 262,	(6,935) : 214,	(7,935) : 205,	(8,935) : 817,	(9,935) : 391,	(10,935) : 207,	(11,935) : 249,	(12,935) : 404,	(13,935) : 69,	(14,935) : 387,	(15,935) : 248,	(16,935) : 470,	(17,935) : 353,	(18,935) : 244,	(19,935) : 347,	(20,935) : 207,	(21,935) : 105,	(22,935) : 223,	(23,935) : 238,	(24,935) : 356,	(25,935) : 731,	(26,935) : 81,	(27,935) : 65,	(28,935) : 205,	(29,935) : 262,	(30,935) : 210,	(31,935) : 218,	(32,935) : 125,	(33,935) : 194,	(34,935) : 203,	(35,935) : 345,	(36,935) : 450,	(37,935) : 438,	(38,935) : 245,	(39,935) : 248,	(40,935) : 752,
(1,936) : 1058,	(2,936) : 1017,	(3,936) : 620,	(4,936) : 1299,	(5,936) : 966,	(6,936) : 982,	(7,936) : 1049,	(8,936) : 401,	(9,936) : 701,	(10,936) : 1051,	(11,936) : 1018,	(12,936) : 1428,	(13,936) : 1093,	(14,936) : 705,	(15,936) : 1199,	(16,936) : 1411,	(17,936) : 1263,	(18,936) : 963,	(19,936) : 1380,	(20,936) : 1055,	(21,936) : 1078,	(22,936) : 904,	(23,936) : 964,	(24,936) : 1263,	(25,936) : 333,	(26,936) : 1126,	(27,936) : 1094,	(28,936) : 1053,	(29,936) : 1133,	(30,936) : 1058,	(31,936) : 992,	(32,936) : 1039,	(33,936) : 1157,	(34,936) : 962,	(35,936) : 1088,	(36,936) : 667,	(37,936) : 615,	(38,936) : 1001,	(39,936) : 1014,	(40,936) : 305,
(1,937) : 561,	(2,937) : 603,	(3,937) : 633,	(4,937) : 60,	(5,937) : 620,	(6,937) : 572,	(7,937) : 555,	(8,937) : 918,	(9,937) : 699,	(10,937) : 556,	(11,937) : 604,	(12,937) : 283,	(13,937) : 390,	(14,937) : 695,	(15,937) : 118,	(16,937) : 173,	(17,937) : 35,	(18,937) : 602,	(19,937) : 285,	(20,937) : 555,	(21,937) : 255,	(22,937) : 573,	(23,937) : 597,	(24,937) : 34,	(25,937) : 907,	(26,937) : 329,	(27,937) : 384,	(28,937) : 554,	(29,937) : 116,	(30,937) : 558,	(31,937) : 576,	(32,937) : 479,	(33,937) : 494,	(34,937) : 561,	(35,937) : 689,	(36,937) : 761,	(37,937) : 701,	(38,937) : 601,	(39,937) : 603,	(40,937) : 935,
(1,938) : 537,	(2,938) : 581,	(3,938) : 648,	(4,938) : 65,	(5,938) : 601,	(6,938) : 552,	(7,938) : 532,	(8,938) : 934,	(9,938) : 692,	(10,938) : 533,	(11,938) : 582,	(12,938) : 251,	(13,938) : 366,	(14,938) : 688,	(15,938) : 93,	(16,938) : 162,	(17,938) : 15,	(18,938) : 583,	(19,938) : 250,	(20,938) : 532,	(21,938) : 238,	(22,938) : 558,	(23,938) : 577,	(24,938) : 17,	(25,938) : 917,	(26,938) : 304,	(27,938) : 360,	(28,938) : 531,	(29,938) : 117,	(30,938) : 534,	(31,938) : 555,	(32,938) : 457,	(33,938) : 466,	(34,938) : 542,	(35,938) : 664,	(36,938) : 754,	(37,938) : 700,	(38,938) : 580,	(39,938) : 581,	(40,938) : 944,
(1,939) : 505,	(2,939) : 545,	(3,939) : 590,	(4,939) : 118,	(5,939) : 561,	(6,939) : 513,	(7,939) : 498,	(8,939) : 877,	(9,939) : 638,	(10,939) : 500,	(11,939) : 546,	(12,939) : 293,	(13,939) : 335,	(14,939) : 634,	(15,939) : 74,	(16,939) : 222,	(17,939) : 74,	(18,939) : 543,	(19,939) : 280,	(20,939) : 499,	(21,939) : 194,	(22,939) : 512,	(23,939) : 537,	(24,939) : 75,	(25,939) : 857,	(26,939) : 276,	(27,939) : 329,	(28,939) : 498,	(29,939) : 58,	(30,939) : 502,	(31,939) : 517,	(32,939) : 421,	(33,939) : 445,	(34,939) : 501,	(35,939) : 635,	(36,939) : 700,	(37,939) : 642,	(38,939) : 543,	(39,939) : 545,	(40,939) : 884,
(1,940) : 455,	(2,940) : 418,	(3,940) : 348,	(4,940) : 791,	(5,940) : 368,	(6,940) : 379,	(7,940) : 446,	(8,940) : 497,	(9,940) : 103,	(10,940) : 448,	(11,940) : 419,	(12,940) : 860,	(13,940) : 498,	(14,940) : 107,	(15,940) : 658,	(16,940) : 888,	(17,940) : 746,	(18,940) : 362,	(19,940) : 805,	(20,940) : 452,	(21,940) : 517,	(22,940) : 300,	(23,940) : 363,	(24,940) : 747,	(25,940) : 334,	(26,940) : 539,	(27,940) : 499,	(28,940) : 450,	(29,940) : 617,	(30,940) : 455,	(31,940) : 389,	(32,940) : 436,	(33,940) : 553,	(34,940) : 359,	(35,940) : 502,	(36,940) : 105,	(37,940) : 37,	(38,940) : 401,	(39,940) : 414,	(40,940) : 343,
(1,941) : 95,	(2,941) : 124,	(3,941) : 581,	(4,941) : 524,	(5,941) : 142,	(6,941) : 93,	(7,941) : 85,	(8,941) : 840,	(9,941) : 332,	(10,941) : 87,	(11,941) : 125,	(12,941) : 487,	(13,941) : 104,	(14,941) : 327,	(15,941) : 367,	(16,941) : 582,	(17,941) : 474,	(18,941) : 125,	(19,941) : 425,	(20,941) : 89,	(21,941) : 230,	(22,941) : 131,	(23,941) : 120,	(24,941) : 476,	(25,941) : 726,	(26,941) : 166,	(27,941) : 108,	(28,941) : 87,	(29,941) : 386,	(30,941) : 92,	(31,941) : 96,	(32,941) : 17,	(33,941) : 138,	(34,941) : 89,	(35,941) : 225,	(36,941) : 383,	(37,941) : 407,	(38,941) : 121,	(39,941) : 123,	(40,941) : 743,
(1,942) : 269,	(2,942) : 286,	(3,942) : 439,	(4,942) : 425,	(5,942) : 282,	(6,942) : 240,	(7,942) : 258,	(8,942) : 717,	(9,942) : 333,	(10,942) : 261,	(11,942) : 287,	(12,942) : 478,	(13,942) : 167,	(14,942) : 328,	(15,942) : 280,	(16,942) : 511,	(17,942) : 376,	(18,942) : 262,	(19,942) : 426,	(20,942) : 262,	(21,942) : 133,	(22,942) : 213,	(23,942) : 257,	(24,942) : 378,	(25,942) : 636,	(26,942) : 176,	(27,942) : 164,	(28,942) : 260,	(29,942) : 259,	(30,942) : 266,	(31,942) : 248,	(32,942) : 184,	(33,942) : 283,	(34,942) : 220,	(35,942) : 396,	(36,942) : 394,	(37,942) : 359,	(38,942) : 277,	(39,942) : 285,	(40,942) : 658,
(1,943) : 614,	(2,943) : 663,	(3,943) : 777,	(4,943) : 93,	(5,943) : 690,	(6,943) : 641,	(7,943) : 611,	(8,943) : 1062,	(9,943) : 809,	(10,943) : 611,	(11,943) : 664,	(12,943) : 188,	(13,943) : 448,	(14,943) : 804,	(15,943) : 197,	(16,943) : 35,	(17,943) : 117,	(18,943) : 674,	(19,943) : 220,	(20,943) : 610,	(21,943) : 346,	(22,943) : 659,	(23,943) : 668,	(24,943) : 117,	(25,943) : 1047,	(26,943) : 383,	(27,943) : 442,	(28,943) : 609,	(29,943) : 246,	(30,943) : 611,	(31,943) : 643,	(32,943) : 542,	(33,943) : 526,	(34,943) : 635,	(35,943) : 734,	(36,943) : 870,	(37,943) : 824,	(38,943) : 665,	(39,943) : 664,	(40,943) : 1074,
(1,944) : 462,	(2,944) : 496,	(3,944) : 505,	(4,944) : 205,	(5,944) : 505,	(6,944) : 458,	(7,944) : 454,	(8,944) : 794,	(9,944) : 556,	(10,944) : 456,	(11,944) : 497,	(12,944) : 361,	(13,944) : 300,	(14,944) : 553,	(15,944) : 115,	(16,944) : 311,	(17,944) : 163,	(18,944) : 486,	(19,944) : 336,	(20,944) : 456,	(21,944) : 146,	(22,944) : 447,	(23,944) : 480,	(24,944) : 164,	(25,944) : 769,	(26,944) : 250,	(27,944) : 294,	(28,944) : 455,	(29,944) : 32,	(30,944) : 459,	(31,944) : 464,	(32,944) : 375,	(33,944) : 421,	(34,944) : 443,	(35,944) : 594,	(36,944) : 619,	(37,944) : 556,	(38,944) : 492,	(39,944) : 496,	(40,944) : 795,
(1,945) : 492,	(2,945) : 541,	(3,945) : 701,	(4,945) : 126,	(5,945) : 567,	(6,945) : 518,	(7,945) : 488,	(8,945) : 990,	(9,945) : 692,	(10,945) : 489,	(11,945) : 541,	(12,945) : 168,	(13,945) : 324,	(14,945) : 688,	(15,945) : 89,	(16,945) : 157,	(17,945) : 93,	(18,945) : 551,	(19,945) : 160,	(20,945) : 488,	(21,945) : 228,	(22,945) : 537,	(23,945) : 545,	(24,945) : 96,	(25,945) : 957,	(26,945) : 260,	(27,945) : 319,	(28,945) : 487,	(29,945) : 168,	(30,945) : 489,	(31,945) : 520,	(32,945) : 419,	(33,945) : 408,	(34,945) : 512,	(35,945) : 613,	(36,945) : 754,	(37,945) : 715,	(38,945) : 542,	(39,945) : 541,	(40,945) : 983,
(1,946) : 362,	(2,946) : 408,	(3,946) : 626,	(4,946) : 239,	(5,946) : 433,	(6,946) : 384,	(7,946) : 357,	(8,946) : 915,	(9,946) : 564,	(10,946) : 358,	(11,946) : 409,	(12,946) : 243,	(13,946) : 192,	(14,946) : 560,	(15,946) : 88,	(16,946) : 292,	(17,946) : 191,	(18,946) : 416,	(19,946) : 197,	(20,946) : 357,	(21,946) : 107,	(22,946) : 402,	(23,946) : 410,	(24,946) : 193,	(25,946) : 860,	(26,946) : 128,	(27,946) : 186,	(28,946) : 356,	(29,946) : 156,	(30,946) : 359,	(31,946) : 386,	(32,946) : 285,	(33,946) : 288,	(34,946) : 377,	(35,946) : 487,	(36,946) : 625,	(37,946) : 596,	(38,946) : 409,	(39,946) : 409,	(40,946) : 884,
(1,947) : 882,	(2,947) : 847,	(3,947) : 409,	(4,947) : 1087,	(5,947) : 797,	(6,947) : 807,	(7,947) : 873,	(8,947) : 231,	(9,947) : 531,	(10,947) : 875,	(11,947) : 848,	(12,947) : 1222,	(13,947) : 900,	(14,947) : 536,	(15,947) : 989,	(16,947) : 1199,	(17,947) : 1051,	(18,947) : 791,	(19,947) : 1176,	(20,947) : 879,	(21,947) : 873,	(22,947) : 727,	(23,947) : 792,	(24,947) : 1052,	(25,947) : 121,	(26,947) : 927,	(27,947) : 899,	(28,947) : 876,	(29,947) : 921,	(30,947) : 882,	(31,947) : 817,	(32,947) : 853,	(33,947) : 973,	(34,947) : 785,	(35,947) : 929,	(36,947) : 511,	(37,947) : 432,	(38,947) : 830,	(39,947) : 843,	(40,947) : 96,
(1,948) : 669,	(2,948) : 719,	(3,948) : 832,	(4,948) : 142,	(5,948) : 748,	(6,948) : 699,	(7,948) : 666,	(8,948) : 1116,	(9,948) : 870,	(10,948) : 666,	(11,948) : 720,	(12,948) : 204,	(13,948) : 504,	(14,948) : 866,	(15,948) : 258,	(16,948) : 27,	(17,948) : 175,	(18,948) : 731,	(19,948) : 250,	(20,948) : 665,	(21,948) : 407,	(22,948) : 719,	(23,948) : 726,	(24,948) : 174,	(25,948) : 1105,	(26,948) : 440,	(27,948) : 499,	(28,948) : 664,	(29,948) : 306,	(30,948) : 666,	(31,948) : 700,	(32,948) : 599,	(33,948) : 577,	(34,948) : 694,	(35,948) : 786,	(36,948) : 932,	(37,948) : 885,	(38,948) : 722,	(39,948) : 720,	(40,948) : 1132,
(1,949) : 600,	(2,949) : 645,	(3,949) : 691,	(4,949) : 1,	(5,949) : 666,	(6,949) : 617,	(7,949) : 595,	(8,949) : 974,	(9,949) : 756,	(10,949) : 596,	(11,949) : 646,	(12,949) : 259,	(13,949) : 430,	(14,949) : 752,	(15,949) : 157,	(16,949) : 117,	(17,949) : 51,	(18,949) : 648,	(19,949) : 273,	(20,949) : 595,	(21,949) : 304,	(22,949) : 623,	(23,949) : 642,	(24,949) : 49,	(25,949) : 967,	(26,949) : 366,	(27,949) : 424,	(28,949) : 594,	(29,949) : 175,	(30,949) : 597,	(31,949) : 620,	(32,949) : 521,	(33,949) : 526,	(34,949) : 607,	(35,949) : 726,	(36,949) : 818,	(37,949) : 761,	(38,949) : 645,	(39,949) : 645,	(40,949) : 994,
(1,950) : 911,	(2,950) : 879,	(3,950) : 387,	(4,950) : 1075,	(5,950) : 830,	(6,950) : 836,	(7,950) : 901,	(8,950) : 169,	(9,950) : 567,	(10,950) : 903,	(11,950) : 879,	(12,950) : 1223,	(13,950) : 916,	(14,950) : 571,	(15,950) : 986,	(16,950) : 1189,	(17,950) : 1042,	(18,950) : 823,	(19,950) : 1180,	(20,950) : 907,	(21,950) : 878,	(22,950) : 755,	(23,950) : 823,	(24,950) : 1042,	(25,950) : 133,	(26,950) : 939,	(27,950) : 915,	(28,950) : 904,	(29,950) : 914,	(30,950) : 910,	(31,950) : 847,	(32,950) : 875,	(33,950) : 996,	(34,950) : 813,	(35,950) : 966,	(36,950) : 553,	(37,950) : 462,	(38,950) : 861,	(39,950) : 875,	(40,950) : 121,
(1,951) : 361,	(2,951) : 321,	(3,951) : 418,	(4,951) : 758,	(5,951) : 270,	(6,951) : 286,	(7,951) : 353,	(8,951) : 594,	(9,951) : 4,	(10,951) : 355,	(11,951) : 322,	(12,951) : 799,	(13,951) : 424,	(14,951) : 9,	(15,951) : 616,	(16,951) : 847,	(17,951) : 711,	(18,951) : 266,	(19,951) : 741,	(20,951) : 359,	(21,951) : 468,	(22,951) : 210,	(23,951) : 267,	(24,951) : 712,	(25,951) : 434,	(26,951) : 472,	(27,951) : 426,	(28,951) : 357,	(29,951) : 587,	(30,951) : 362,	(31,951) : 295,	(32,951) : 352,	(33,951) : 464,	(34,951) : 267,	(35,951) : 402,	(36,951) : 59,	(37,951) : 112,	(38,951) : 304,	(39,951) : 317,	(40,951) : 443,
(1,952) : 1148,	(2,952) : 1108,	(3,952) : 691,	(4,952) : 1376,	(5,952) : 1056,	(6,952) : 1072,	(7,952) : 1139,	(8,952) : 448,	(9,952) : 791,	(10,952) : 1141,	(11,952) : 1108,	(12,952) : 1511,	(13,952) : 1181,	(14,952) : 795,	(15,952) : 1279,	(16,952) : 1488,	(17,952) : 1340,	(18,952) : 1053,	(19,952) : 1464,	(20,952) : 1145,	(21,952) : 1162,	(22,952) : 994,	(23,952) : 1055,	(24,952) : 1341,	(25,952) : 411,	(26,952) : 1212,	(27,952) : 1181,	(28,952) : 1143,	(29,952) : 1211,	(30,952) : 1148,	(31,952) : 1082,	(32,952) : 1128,	(33,952) : 1246,	(34,952) : 1052,	(35,952) : 1179,	(36,952) : 757,	(37,952) : 704,	(38,952) : 1091,	(39,952) : 1104,	(40,952) : 385,
(1,953) : 563,	(2,953) : 611,	(3,953) : 735,	(4,953) : 80,	(5,953) : 638,	(6,953) : 589,	(7,953) : 559,	(8,953) : 1022,	(9,953) : 755,	(10,953) : 560,	(11,953) : 612,	(12,953) : 178,	(13,953) : 395,	(14,953) : 751,	(15,953) : 143,	(16,953) : 88,	(17,953) : 79,	(18,953) : 621,	(19,953) : 195,	(20,953) : 558,	(21,953) : 292,	(22,953) : 605,	(23,953) : 615,	(24,953) : 80,	(25,953) : 1000,	(26,953) : 331,	(27,953) : 390,	(28,953) : 558,	(29,953) : 200,	(30,953) : 560,	(31,953) : 591,	(32,953) : 490,	(33,953) : 477,	(34,953) : 582,	(35,953) : 684,	(36,953) : 817,	(37,953) : 772,	(38,953) : 613,	(39,953) : 612,	(40,953) : 1027,
(1,954) : 45,	(2,954) : 92,	(3,954) : 700,	(4,954) : 599,	(5,954) : 144,	(6,954) : 121,	(7,954) : 55,	(8,954) : 950,	(9,954) : 402,	(10,954) : 53,	(11,954) : 92,	(12,954) : 509,	(13,954) : 176,	(14,954) : 398,	(15,954) : 446,	(16,954) : 638,	(17,954) : 551,	(18,954) : 142,	(19,954) : 445,	(20,954) : 49,	(21,954) : 329,	(22,954) : 201,	(23,954) : 140,	(24,954) : 554,	(25,954) : 823,	(26,954) : 235,	(27,954) : 182,	(28,954) : 51,	(29,954) : 483,	(30,954) : 45,	(31,954) : 111,	(32,954) : 113,	(33,954) : 89,	(34,954) : 142,	(35,954) : 127,	(36,954) : 442,	(37,954) : 495,	(38,954) : 107,	(39,954) : 95,	(40,954) : 837,
(1,955) : 440,	(2,955) : 494,	(3,955) : 785,	(4,955) : 255,	(5,955) : 531,	(6,955) : 484,	(7,955) : 439,	(8,955) : 1074,	(9,955) : 704,	(10,955) : 439,	(11,955) : 494,	(12,955) : 92,	(13,955) : 293,	(14,955) : 699,	(15,955) : 188,	(16,955) : 236,	(17,955) : 228,	(18,955) : 517,	(19,955) : 37,	(20,955) : 437,	(21,955) : 265,	(22,955) : 522,	(23,955) : 512,	(24,955) : 230,	(25,955) : 1019,	(26,955) : 237,	(27,955) : 289,	(28,955) : 437,	(29,955) : 279,	(30,955) : 437,	(31,955) : 483,	(32,955) : 385,	(33,955) : 336,	(34,955) : 485,	(35,955) : 546,	(36,955) : 762,	(37,955) : 747,	(38,955) : 500,	(39,955) : 495,	(40,955) : 1043,
(1,956) : 207,	(2,956) : 250,	(3,956) : 589,	(4,956) : 394,	(5,956) : 274,	(6,956) : 225,	(7,956) : 201,	(8,956) : 868,	(9,956) : 432,	(10,956) : 202,	(11,956) : 251,	(12,956) : 366,	(13,956) : 37,	(14,956) : 427,	(15,956) : 238,	(16,956) : 449,	(17,956) : 344,	(18,956) : 258,	(19,956) : 306,	(20,956) : 201,	(21,956) : 117,	(22,956) : 251,	(23,956) : 252,	(24,956) : 347,	(25,956) : 781,	(26,956) : 36,	(27,956) : 31,	(28,956) : 200,	(29,956) : 270,	(30,956) : 204,	(31,956) : 227,	(32,956) : 127,	(33,956) : 161,	(34,956) : 220,	(35,956) : 336,	(36,956) : 489,	(37,956) : 484,	(38,956) : 251,	(39,956) : 251,	(40,956) : 801,
(1,957) : 505,	(2,957) : 545,	(3,957) : 589,	(4,957) : 118,	(5,957) : 561,	(6,957) : 513,	(7,957) : 498,	(8,957) : 877,	(9,957) : 638,	(10,957) : 500,	(11,957) : 546,	(12,957) : 293,	(13,957) : 335,	(14,957) : 634,	(15,957) : 74,	(16,957) : 222,	(17,957) : 74,	(18,957) : 543,	(19,957) : 280,	(20,957) : 499,	(21,957) : 194,	(22,957) : 512,	(23,957) : 537,	(24,957) : 75,	(25,957) : 857,	(26,957) : 276,	(27,957) : 329,	(28,957) : 498,	(29,957) : 58,	(30,957) : 502,	(31,957) : 517,	(32,957) : 421,	(33,957) : 445,	(34,957) : 501,	(35,957) : 635,	(36,957) : 700,	(37,957) : 642,	(38,957) : 543,	(39,957) : 545,	(40,957) : 884,
(1,958) : 574,	(2,958) : 620,	(3,958) : 704,	(4,958) : 39,	(5,958) : 643,	(6,958) : 594,	(7,958) : 569,	(8,958) : 989,	(9,958) : 745,	(10,958) : 570,	(11,958) : 621,	(12,958) : 221,	(13,958) : 404,	(14,958) : 741,	(15,958) : 137,	(16,958) : 104,	(17,958) : 44,	(18,958) : 626,	(19,958) : 234,	(20,958) : 569,	(21,958) : 287,	(22,958) : 605,	(23,958) : 620,	(24,958) : 44,	(25,958) : 974,	(26,958) : 340,	(27,958) : 398,	(28,958) : 568,	(29,958) : 175,	(30,958) : 571,	(31,958) : 597,	(32,958) : 497,	(33,958) : 495,	(34,958) : 586,	(35,958) : 698,	(36,958) : 808,	(37,958) : 756,	(38,958) : 621,	(39,958) : 621,	(40,958) : 1001,
(1,959) : 557,	(2,959) : 603,	(3,959) : 693,	(4,959) : 50,	(5,959) : 626,	(6,959) : 577,	(7,959) : 552,	(8,959) : 980,	(9,959) : 729,	(10,959) : 553,	(11,959) : 603,	(12,959) : 217,	(13,959) : 386,	(14,959) : 725,	(15,959) : 120,	(16,959) : 118,	(17,959) : 35,	(18,959) : 608,	(19,959) : 226,	(20,959) : 552,	(21,959) : 270,	(22,959) : 588,	(23,959) : 603,	(24,959) : 35,	(25,959) : 962,	(26,959) : 323,	(27,959) : 381,	(28,959) : 551,	(29,959) : 162,	(30,959) : 554,	(31,959) : 579,	(32,959) : 480,	(33,959) : 478,	(34,959) : 568,	(35,959) : 681,	(36,959) : 791,	(37,959) : 741,	(38,959) : 603,	(39,959) : 603,	(40,959) : 989,
(1,960) : 131,	(2,960) : 171,	(3,960) : 591,	(4,960) : 473,	(5,960) : 195,	(6,960) : 147,	(7,960) : 124,	(8,960) : 859,	(9,960) : 377,	(10,960) : 125,	(11,960) : 172,	(12,960) : 434,	(13,960) : 49,	(14,960) : 372,	(15,960) : 317,	(16,960) : 528,	(17,960) : 423,	(18,960) : 179,	(19,960) : 372,	(20,960) : 125,	(21,960) : 186,	(22,960) : 182,	(23,960) : 174,	(24,960) : 426,	(25,960) : 755,	(26,960) : 112,	(27,960) : 54,	(28,960) : 124,	(29,960) : 342,	(30,960) : 128,	(31,960) : 148,	(32,960) : 48,	(33,960) : 122,	(34,960) : 144,	(35,960) : 262,	(36,960) : 431,	(37,960) : 444,	(38,960) : 171,	(39,960) : 171,	(40,960) : 774,
(1,961) : 683,	(2,961) : 736,	(3,961) : 933,	(4,961) : 255,	(5,961) : 772,	(6,961) : 724,	(7,961) : 681,	(8,961) : 1220,	(9,961) : 926,	(10,961) : 681,	(11,961) : 737,	(12,961) : 159,	(13,961) : 530,	(14,961) : 922,	(15,961) : 329,	(16,961) : 141,	(17,961) : 275,	(18,961) : 757,	(19,961) : 222,	(20,961) : 679,	(21,961) : 465,	(22,961) : 757,	(23,961) : 752,	(24,961) : 275,	(25,961) : 1196,	(26,961) : 470,	(27,961) : 526,	(28,961) : 679,	(29,961) : 398,	(30,961) : 680,	(31,961) : 724,	(32,961) : 624,	(33,961) : 577,	(34,961) : 723,	(35,961) : 787,	(36,961) : 987,	(37,961) : 955,	(38,961) : 742,	(39,961) : 738,	(40,961) : 1222,
(1,962) : 884,	(2,962) : 849,	(3,962) : 411,	(4,962) : 1089,	(5,962) : 799,	(6,962) : 809,	(7,962) : 874,	(8,962) : 232,	(9,962) : 533,	(10,962) : 877,	(11,962) : 849,	(12,962) : 1224,	(13,962) : 901,	(14,962) : 537,	(15,962) : 991,	(16,962) : 1201,	(17,962) : 1053,	(18,962) : 793,	(19,962) : 1178,	(20,962) : 880,	(21,962) : 875,	(22,962) : 728,	(23,962) : 794,	(24,962) : 1054,	(25,962) : 123,	(26,962) : 929,	(27,962) : 901,	(28,962) : 878,	(29,962) : 924,	(30,962) : 884,	(31,962) : 819,	(32,962) : 855,	(33,962) : 975,	(34,962) : 787,	(35,962) : 931,	(36,962) : 512,	(37,962) : 434,	(38,962) : 831,	(39,962) : 845,	(40,962) : 98,
(1,963) : 441,	(2,963) : 495,	(3,963) : 792,	(4,963) : 263,	(5,963) : 532,	(6,963) : 486,	(7,963) : 440,	(8,963) : 1082,	(9,963) : 708,	(10,963) : 440,	(11,963) : 495,	(12,963) : 89,	(13,963) : 296,	(14,963) : 703,	(15,963) : 196,	(16,963) : 241,	(17,963) : 236,	(18,963) : 519,	(19,963) : 31,	(20,963) : 438,	(21,963) : 272,	(22,963) : 525,	(23,963) : 514,	(24,963) : 238,	(25,963) : 1026,	(26,963) : 241,	(27,963) : 292,	(28,963) : 438,	(29,963) : 287,	(30,963) : 438,	(31,963) : 485,	(32,963) : 387,	(33,963) : 335,	(34,963) : 487,	(35,963) : 545,	(36,963) : 766,	(37,963) : 752,	(38,963) : 501,	(39,963) : 497,	(40,963) : 1049,
(1,964) : 254,	(2,964) : 232,	(3,964) : 417,	(4,964) : 610,	(5,964) : 195,	(6,964) : 184,	(7,964) : 243,	(8,964) : 653,	(9,964) : 151,	(10,964) : 246,	(11,964) : 233,	(12,964) : 645,	(13,964) : 275,	(14,964) : 146,	(15,964) : 464,	(16,964) : 695,	(17,964) : 561,	(18,964) : 181,	(19,964) : 588,	(20,964) : 249,	(21,964) : 315,	(22,964) : 105,	(23,964) : 179,	(24,964) : 563,	(25,964) : 528,	(26,964) : 319,	(27,964) : 277,	(28,964) : 247,	(29,964) : 442,	(30,964) : 253,	(31,964) : 195,	(32,964) : 217,	(33,964) : 337,	(34,964) : 159,	(35,964) : 338,	(36,964) : 211,	(37,964) : 208,	(38,964) : 216,	(39,964) : 229,	(40,964) : 544,
(1,965) : 814,	(2,965) : 870,	(3,965) : 1099,	(4,965) : 416,	(5,965) : 909,	(6,965) : 864,	(7,965) : 815,	(8,965) : 1386,	(9,965) : 1082,	(10,965) : 814,	(11,965) : 870,	(12,965) : 290,	(13,965) : 675,	(14,965) : 1077,	(15,965) : 495,	(16,965) : 299,	(17,965) : 440,	(18,965) : 897,	(19,965) : 352,	(20,965) : 812,	(21,965) : 626,	(22,965) : 904,	(23,965) : 891,	(24,965) : 440,	(25,965) : 1362,	(26,965) : 618,	(27,965) : 671,	(28,965) : 812,	(29,965) : 564,	(30,965) : 812,	(31,965) : 862,	(32,965) : 766,	(33,965) : 703,	(34,965) : 866,	(35,965) : 907,	(36,965) : 1141,	(37,965) : 1116,	(38,965) : 877,	(39,965) : 872,	(40,965) : 1388,
(1,966) : 674,	(2,966) : 727,	(3,966) : 927,	(4,966) : 250,	(5,966) : 763,	(6,966) : 715,	(7,966) : 672,	(8,966) : 1214,	(9,966) : 918,	(10,966) : 672,	(11,966) : 728,	(12,966) : 150,	(13,966) : 522,	(14,966) : 913,	(15,966) : 322,	(16,966) : 138,	(17,966) : 269,	(18,966) : 748,	(19,966) : 213,	(20,966) : 670,	(21,966) : 457,	(22,966) : 748,	(23,966) : 743,	(24,966) : 269,	(25,966) : 1189,	(26,966) : 461,	(27,966) : 517,	(28,966) : 670,	(29,966) : 391,	(30,966) : 671,	(31,966) : 715,	(32,966) : 616,	(33,966) : 569,	(34,966) : 714,	(35,966) : 778,	(36,966) : 978,	(37,966) : 947,	(38,966) : 733,	(39,966) : 729,	(40,966) : 1215,
(1,967) : 71,	(2,967) : 95,	(3,967) : 594,	(4,967) : 552,	(5,967) : 116,	(6,967) : 67,	(7,967) : 60,	(8,967) : 848,	(9,967) : 324,	(10,967) : 62,	(11,967) : 96,	(12,967) : 509,	(13,967) : 128,	(14,967) : 319,	(15,967) : 395,	(16,967) : 608,	(17,967) : 502,	(18,967) : 100,	(19,967) : 446,	(20,967) : 64,	(21,967) : 258,	(22,967) : 119,	(23,967) : 95,	(24,967) : 504,	(25,967) : 728,	(26,967) : 192,	(27,967) : 133,	(28,967) : 62,	(29,967) : 415,	(30,967) : 68,	(31,967) : 68,	(32,967) : 33,	(33,967) : 138,	(34,967) : 68,	(35,967) : 198,	(36,967) : 372,	(37,967) : 405,	(38,967) : 92,	(39,967) : 95,	(40,967) : 744,
(1,968) : 472,	(2,968) : 524,	(3,968) : 761,	(4,968) : 198,	(5,968) : 557,	(6,968) : 509,	(7,968) : 470,	(8,968) : 1051,	(9,968) : 711,	(10,968) : 470,	(11,968) : 525,	(12,968) : 98,	(13,968) : 314,	(14,968) : 707,	(15,968) : 150,	(16,968) : 185,	(17,968) : 173,	(18,968) : 542,	(19,968) : 79,	(20,968) : 468,	(21,968) : 257,	(22,968) : 540,	(23,968) : 537,	(24,968) : 175,	(25,968) : 1006,	(26,968) : 253,	(27,968) : 310,	(28,968) : 468,	(29,968) : 239,	(30,968) : 469,	(31,968) : 509,	(32,968) : 409,	(33,968) : 374,	(34,968) : 507,	(35,968) : 584,	(36,968) : 771,	(37,968) : 746,	(38,968) : 528,	(39,968) : 525,	(40,968) : 1030,
(1,969) : 512,	(2,969) : 557,	(3,969) : 664,	(4,969) : 89,	(5,969) : 580,	(6,969) : 531,	(7,969) : 507,	(8,969) : 952,	(9,969) : 684,	(10,969) : 508,	(11,969) : 558,	(12,969) : 219,	(13,969) : 341,	(14,969) : 680,	(15,969) : 73,	(16,969) : 160,	(17,969) : 44,	(18,969) : 562,	(19,969) : 213,	(20,969) : 507,	(21,969) : 224,	(22,969) : 541,	(23,969) : 557,	(24,969) : 47,	(25,969) : 927,	(26,969) : 278,	(27,969) : 336,	(28,969) : 506,	(29,969) : 128,	(30,969) : 509,	(31,969) : 534,	(32,969) : 434,	(33,969) : 436,	(34,969) : 522,	(35,969) : 637,	(36,969) : 746,	(37,969) : 699,	(38,969) : 558,	(39,969) : 558,	(40,969) : 953,
(1,970) : 258,	(2,970) : 302,	(3,970) : 585,	(4,970) : 343,	(5,970) : 324,	(6,970) : 275,	(7,970) : 252,	(8,970) : 869,	(9,970) : 465,	(10,970) : 254,	(11,970) : 302,	(12,970) : 330,	(13,970) : 88,	(14,970) : 461,	(15,970) : 186,	(16,970) : 401,	(17,970) : 293,	(18,970) : 307,	(19,970) : 274,	(20,970) : 253,	(21,970) : 77,	(22,970) : 294,	(23,970) : 301,	(24,970) : 296,	(25,970) : 793,	(26,970) : 34,	(27,970) : 81,	(28,970) : 252,	(29,970) : 221,	(30,970) : 255,	(31,970) : 278,	(32,970) : 178,	(33,970) : 204,	(34,970) : 268,	(35,970) : 387,	(36,970) : 525,	(37,970) : 509,	(38,970) : 301,	(39,970) : 302,	(40,970) : 815,
(1,971) : 693,	(2,971) : 666,	(3,971) : 189,	(4,971) : 852,	(5,971) : 621,	(6,971) : 620,	(7,971) : 682,	(8,971) : 223,	(9,971) : 373,	(10,971) : 685,	(11,971) : 667,	(12,971) : 991,	(13,971) : 684,	(14,971) : 376,	(15,971) : 755,	(16,971) : 964,	(17,971) : 816,	(18,971) : 611,	(19,971) : 947,	(20,971) : 688,	(21,971) : 644,	(22,971) : 540,	(23,971) : 611,	(24,971) : 817,	(25,971) : 114,	(26,971) : 706,	(27,971) : 683,	(28,971) : 686,	(29,971) : 687,	(30,971) : 692,	(31,971) : 631,	(32,971) : 650,	(33,971) : 771,	(34,971) : 596,	(35,971) : 762,	(36,971) : 377,	(37,971) : 259,	(38,971) : 649,	(39,971) : 663,	(40,971) : 142,
(1,972) : 663,	(2,972) : 718,	(3,972) : 1022,	(4,972) : 383,	(5,972) : 761,	(6,972) : 717,	(7,972) : 664,	(8,972) : 1312,	(9,972) : 953,	(10,972) : 664,	(11,972) : 719,	(12,972) : 165,	(13,972) : 536,	(14,972) : 948,	(15,972) : 407,	(16,972) : 283,	(17,972) : 388,	(18,972) : 750,	(19,972) : 216,	(20,972) : 661,	(21,972) : 515,	(22,972) : 765,	(23,972) : 745,	(24,972) : 389,	(25,972) : 1267,	(26,972) : 485,	(27,972) : 533,	(28,972) : 661,	(29,972) : 491,	(30,972) : 660,	(31,972) : 714,	(32,972) : 622,	(33,972) : 548,	(34,972) : 722,	(35,972) : 748,	(36,972) : 1010,	(37,972) : 998,	(38,972) : 727,	(39,972) : 721,	(40,972) : 1291,
(1,973) : 811,	(2,973) : 866,	(3,973) : 1108,	(4,973) : 427,	(5,973) : 906,	(6,973) : 861,	(7,973) : 811,	(8,973) : 1395,	(9,973) : 1083,	(10,973) : 811,	(11,973) : 866,	(12,973) : 289,	(13,973) : 674,	(14,973) : 1078,	(15,973) : 501,	(16,973) : 311,	(17,973) : 450,	(18,973) : 894,	(19,973) : 350,	(20,973) : 808,	(21,973) : 629,	(22,973) : 903,	(23,973) : 889,	(24,973) : 450,	(25,973) : 1369,	(26,973) : 618,	(27,973) : 670,	(28,973) : 809,	(29,973) : 572,	(30,973) : 808,	(31,973) : 859,	(32,973) : 764,	(33,973) : 698,	(34,973) : 864,	(35,973) : 901,	(36,973) : 1142,	(37,973) : 1119,	(38,973) : 874,	(39,973) : 868,	(40,973) : 1395,
(1,974) : 661,	(2,974) : 716,	(3,974) : 975,	(4,974) : 319,	(5,974) : 756,	(6,974) : 710,	(7,974) : 661,	(8,974) : 1264,	(9,974) : 932,	(10,974) : 661,	(11,974) : 717,	(12,974) : 137,	(13,974) : 522,	(14,974) : 927,	(15,974) : 361,	(16,974) : 214,	(17,974) : 329,	(18,974) : 743,	(19,974) : 199,	(20,974) : 659,	(21,974) : 481,	(22,974) : 751,	(23,974) : 738,	(24,974) : 330,	(25,974) : 1228,	(26,974) : 466,	(27,974) : 518,	(28,974) : 659,	(29,974) : 440,	(30,974) : 659,	(31,974) : 708,	(32,974) : 612,	(33,974) : 550,	(34,974) : 712,	(35,974) : 756,	(36,974) : 991,	(37,974) : 970,	(38,974) : 724,	(39,974) : 718,	(40,974) : 1253,
(1,975) : 556,	(2,975) : 603,	(3,975) : 700,	(4,975) : 55,	(5,975) : 626,	(6,975) : 577,	(7,975) : 552,	(8,975) : 987,	(9,975) : 733,	(10,975) : 553,	(11,975) : 604,	(12,975) : 210,	(13,975) : 386,	(14,975) : 728,	(15,975) : 122,	(16,975) : 113,	(17,975) : 43,	(18,975) : 609,	(19,975) : 219,	(20,975) : 551,	(21,975) : 272,	(22,975) : 589,	(23,975) : 603,	(24,975) : 43,	(25,975) : 968,	(26,975) : 323,	(27,975) : 381,	(28,975) : 551,	(29,975) : 168,	(30,975) : 553,	(31,975) : 580,	(32,975) : 480,	(33,975) : 476,	(34,975) : 569,	(35,975) : 680,	(36,975) : 795,	(37,975) : 746,	(38,975) : 603,	(39,975) : 603,	(40,975) : 995,
(1,976) : 668,	(2,976) : 636,	(3,976) : 256,	(4,976) : 887,	(5,976) : 588,	(6,976) : 594,	(7,976) : 658,	(8,976) : 282,	(9,976) : 328,	(10,976) : 660,	(11,976) : 637,	(12,976) : 1007,	(13,976) : 680,	(14,976) : 332,	(15,976) : 779,	(16,976) : 996,	(17,976) : 848,	(18,976) : 580,	(19,976) : 959,	(20,976) : 664,	(21,976) : 657,	(22,976) : 513,	(23,976) : 581,	(24,976) : 849,	(25,976) : 110,	(26,976) : 708,	(27,976) : 679,	(28,976) : 662,	(29,976) : 717,	(30,976) : 668,	(31,976) : 604,	(32,976) : 635,	(33,976) : 755,	(34,976) : 571,	(35,976) : 725,	(36,976) : 322,	(37,976) : 220,	(38,976) : 619,	(39,976) : 632,	(40,976) : 125,
(1,977) : 230,	(2,977) : 276,	(3,977) : 608,	(4,977) : 370,	(5,977) : 303,	(6,977) : 254,	(7,977) : 225,	(8,977) : 890,	(9,977) : 463,	(10,977) : 226,	(11,977) : 277,	(12,977) : 334,	(13,977) : 60,	(14,977) : 459,	(15,977) : 215,	(16,977) : 420,	(17,977) : 321,	(18,977) : 287,	(19,977) : 274,	(20,977) : 225,	(21,977) : 112,	(22,977) : 282,	(23,977) : 281,	(24,977) : 323,	(25,977) : 807,	(26,977) : 4,	(27,977) : 54,	(28,977) : 224,	(29,977) : 255,	(30,977) : 227,	(31,977) : 256,	(32,977) : 155,	(33,977) : 169,	(34,977) : 250,	(35,977) : 357,	(36,977) : 521,	(37,977) : 514,	(38,977) : 278,	(39,977) : 277,	(40,977) : 828,
(1,978) : 356,	(2,978) : 316,	(3,978) : 414,	(4,978) : 750,	(5,978) : 266,	(6,978) : 280,	(7,978) : 347,	(8,978) : 594,	(9,978) : 6,	(10,978) : 350,	(11,978) : 317,	(12,978) : 791,	(13,978) : 416,	(14,978) : 5,	(15,978) : 607,	(16,978) : 838,	(17,978) : 702,	(18,978) : 261,	(19,978) : 733,	(20,978) : 353,	(21,978) : 459,	(22,978) : 204,	(23,978) : 263,	(24,978) : 704,	(25,978) : 436,	(26,978) : 464,	(27,978) : 418,	(28,978) : 351,	(29,978) : 578,	(30,978) : 357,	(31,978) : 290,	(32,978) : 345,	(33,978) : 458,	(34,978) : 261,	(35,978) : 399,	(36,978) : 68,	(37,978) : 111,	(38,978) : 299,	(39,978) : 313,	(40,978) : 446,
(1,979) : 551,	(2,979) : 595,	(3,979) : 663,	(4,979) : 49,	(5,979) : 616,	(6,979) : 567,	(7,979) : 545,	(8,979) : 949,	(9,979) : 709,	(10,979) : 546,	(11,979) : 596,	(12,979) : 245,	(13,979) : 380,	(14,979) : 705,	(15,979) : 107,	(16,979) : 145,	(17,979) : 3,	(18,979) : 598,	(19,979) : 248,	(20,979) : 545,	(21,979) : 254,	(22,979) : 574,	(23,979) : 592,	(24,979) : 5,	(25,979) : 933,	(26,979) : 317,	(27,979) : 374,	(28,979) : 545,	(29,979) : 134,	(30,979) : 548,	(31,979) : 570,	(32,979) : 471,	(33,979) : 478,	(34,979) : 557,	(35,979) : 677,	(36,979) : 771,	(37,979) : 717,	(38,979) : 595,	(39,979) : 595,	(40,979) : 960,
(1,980) : 230,	(2,980) : 276,	(3,980) : 608,	(4,980) : 370,	(5,980) : 303,	(6,980) : 254,	(7,980) : 225,	(8,980) : 890,	(9,980) : 463,	(10,980) : 226,	(11,980) : 277,	(12,980) : 334,	(13,980) : 60,	(14,980) : 459,	(15,980) : 215,	(16,980) : 420,	(17,980) : 321,	(18,980) : 287,	(19,980) : 274,	(20,980) : 225,	(21,980) : 112,	(22,980) : 282,	(23,980) : 281,	(24,980) : 323,	(25,980) : 807,	(26,980) : 4,	(27,980) : 54,	(28,980) : 224,	(29,980) : 255,	(30,980) : 227,	(31,980) : 256,	(32,980) : 155,	(33,980) : 169,	(34,980) : 250,	(35,980) : 357,	(36,980) : 521,	(37,980) : 514,	(38,980) : 278,	(39,980) : 277,	(40,980) : 828,
(1,981) : 563,	(2,981) : 611,	(3,981) : 735,	(4,981) : 80,	(5,981) : 638,	(6,981) : 589,	(7,981) : 559,	(8,981) : 1022,	(9,981) : 755,	(10,981) : 560,	(11,981) : 612,	(12,981) : 178,	(13,981) : 395,	(14,981) : 751,	(15,981) : 143,	(16,981) : 88,	(17,981) : 79,	(18,981) : 621,	(19,981) : 195,	(20,981) : 558,	(21,981) : 292,	(22,981) : 605,	(23,981) : 615,	(24,981) : 80,	(25,981) : 1000,	(26,981) : 331,	(27,981) : 390,	(28,981) : 558,	(29,981) : 200,	(30,981) : 560,	(31,981) : 591,	(32,981) : 490,	(33,981) : 477,	(34,981) : 582,	(35,981) : 684,	(36,981) : 817,	(37,981) : 772,	(38,981) : 613,	(39,981) : 612,	(40,981) : 1027,
(1,982) : 833,	(2,982) : 824,	(3,982) : 180,	(4,982) : 773,	(5,982) : 790,	(6,982) : 773,	(7,982) : 822,	(8,982) : 235,	(9,982) : 595,	(10,982) : 825,	(11,982) : 825,	(12,982) : 974,	(13,982) : 765,	(14,982) : 596,	(15,982) : 728,	(16,982) : 891,	(17,982) : 752,	(18,982) : 776,	(19,982) : 946,	(20,982) : 827,	(21,982) : 667,	(22,982) : 700,	(23,982) : 773,	(24,982) : 751,	(25,982) : 356,	(26,982) : 761,	(27,982) : 761,	(28,982) : 825,	(29,982) : 641,	(30,982) : 832,	(31,982) : 784,	(32,982) : 766,	(33,982) : 878,	(34,982) : 747,	(35,982) : 933,	(36,982) : 621,	(37,982) : 489,	(38,982) : 808,	(39,982) : 821,	(40,982) : 384,
(1,983) : 132,	(2,983) : 112,	(3,983) : 755,	(4,983) : 726,	(5,983) : 143,	(6,983) : 163,	(7,983) : 140,	(8,983) : 982,	(9,983) : 398,	(10,983) : 138,	(11,983) : 111,	(12,983) : 629,	(13,983) : 300,	(14,983) : 394,	(15,983) : 572,	(16,983) : 764,	(17,983) : 677,	(18,983) : 157,	(19,983) : 565,	(20,983) : 138,	(21,983) : 448,	(22,983) : 233,	(23,983) : 160,	(24,983) : 680,	(25,983) : 835,	(26,983) : 360,	(27,983) : 306,	(28,983) : 140,	(29,983) : 604,	(30,983) : 135,	(31,983) : 153,	(32,983) : 220,	(33,983) : 210,	(34,983) : 189,	(35,983) : 0,	(36,983) : 422,	(37,983) : 506,	(38,983) : 126,	(39,983) : 115,	(40,983) : 845,
(1,984) : 204,	(2,984) : 254,	(3,984) : 642,	(4,984) : 400,	(5,984) : 287,	(6,984) : 239,	(7,984) : 201,	(8,984) : 921,	(9,984) : 471,	(10,984) : 201,	(11,984) : 255,	(12,984) : 337,	(13,984) : 51,	(14,984) : 466,	(15,984) : 249,	(16,984) : 442,	(17,984) : 352,	(18,984) : 272,	(19,984) : 275,	(20,984) : 199,	(21,984) : 155,	(22,984) : 279,	(23,984) : 267,	(24,984) : 355,	(25,984) : 831,	(26,984) : 44,	(27,984) : 49,	(28,984) : 199,	(29,984) : 296,	(30,984) : 201,	(31,984) : 239,	(32,984) : 140,	(33,984) : 129,	(34,984) : 240,	(35,984) : 326,	(36,984) : 526,	(37,984) : 530,	(38,984) : 258,	(39,984) : 255,	(40,984) : 851,
(1,985) : 463,	(2,985) : 497,	(3,985) : 505,	(4,985) : 204,	(5,985) : 506,	(6,985) : 459,	(7,985) : 455,	(8,985) : 794,	(9,985) : 557,	(10,985) : 456,	(11,985) : 498,	(12,985) : 361,	(13,985) : 300,	(14,985) : 553,	(15,985) : 115,	(16,985) : 310,	(17,985) : 162,	(18,985) : 486,	(19,985) : 336,	(20,985) : 457,	(21,985) : 147,	(22,985) : 447,	(23,985) : 481,	(24,985) : 163,	(25,985) : 769,	(26,985) : 251,	(27,985) : 294,	(28,985) : 455,	(29,985) : 31,	(30,985) : 460,	(31,985) : 465,	(32,985) : 375,	(33,985) : 421,	(34,985) : 444,	(35,985) : 595,	(36,985) : 619,	(37,985) : 556,	(38,985) : 492,	(39,985) : 496,	(40,985) : 796,
(1,986) : 563,	(2,986) : 611,	(3,986) : 735,	(4,986) : 80,	(5,986) : 638,	(6,986) : 589,	(7,986) : 559,	(8,986) : 1022,	(9,986) : 755,	(10,986) : 560,	(11,986) : 612,	(12,986) : 178,	(13,986) : 395,	(14,986) : 751,	(15,986) : 143,	(16,986) : 88,	(17,986) : 79,	(18,986) : 621,	(19,986) : 195,	(20,986) : 558,	(21,986) : 292,	(22,986) : 605,	(23,986) : 615,	(24,986) : 80,	(25,986) : 1000,	(26,986) : 331,	(27,986) : 390,	(28,986) : 558,	(29,986) : 200,	(30,986) : 560,	(31,986) : 591,	(32,986) : 490,	(33,986) : 477,	(34,986) : 582,	(35,986) : 684,	(36,986) : 817,	(37,986) : 772,	(38,986) : 613,	(39,986) : 612,	(40,986) : 1027,
(1,987) : 217,	(2,987) : 188,	(3,987) : 467,	(4,987) : 636,	(5,987) : 146,	(6,987) : 142,	(7,987) : 207,	(8,987) : 696,	(9,987) : 152,	(10,987) : 209,	(11,987) : 189,	(12,987) : 650,	(13,987) : 270,	(14,987) : 147,	(15,987) : 485,	(16,987) : 714,	(17,987) : 586,	(18,987) : 134,	(19,987) : 591,	(20,987) : 213,	(21,987) : 335,	(22,987) : 61,	(23,987) : 133,	(24,987) : 588,	(25,987) : 561,	(26,987) : 321,	(27,987) : 272,	(28,987) : 211,	(29,987) : 472,	(30,987) : 217,	(31,987) : 153,	(32,987) : 196,	(33,987) : 311,	(34,987) : 119,	(35,987) : 290,	(36,987) : 205,	(37,987) : 234,	(38,987) : 171,	(39,987) : 184,	(40,987) : 576,
(1,988) : 30,	(2,988) : 85,	(3,988) : 657,	(4,988) : 571,	(5,988) : 129,	(6,988) : 93,	(7,988) : 30,	(8,988) : 910,	(9,988) : 373,	(10,988) : 30,	(11,988) : 85,	(12,988) : 499,	(13,988) : 143,	(14,988) : 368,	(15,988) : 416,	(16,988) : 616,	(17,988) : 522,	(18,988) : 121,	(19,988) : 435,	(20,988) : 27,	(21,988) : 292,	(22,988) : 168,	(23,988) : 118,	(24,988) : 525,	(25,988) : 787,	(26,988) : 205,	(27,988) : 149,	(28,988) : 27,	(29,988) : 447,	(30,988) : 27,	(31,988) : 86,	(32,988) : 69,	(33,988) : 92,	(34,988) : 110,	(35,988) : 157,	(36,988) : 417,	(37,988) : 461,	(38,988) : 94,	(39,988) : 87,	(40,988) : 802,
(1,989) : 568,	(2,989) : 537,	(3,989) : 247,	(4,989) : 812,	(5,989) : 489,	(6,989) : 494,	(7,989) : 558,	(8,989) : 363,	(9,989) : 234,	(10,989) : 560,	(11,989) : 538,	(12,989) : 917,	(13,989) : 582,	(14,989) : 237,	(15,989) : 696,	(16,989) : 917,	(17,989) : 770,	(18,989) : 481,	(19,989) : 868,	(20,989) : 564,	(21,989) : 568,	(22,989) : 413,	(23,989) : 482,	(24,989) : 772,	(25,989) : 210,	(26,989) : 612,	(27,989) : 582,	(28,989) : 562,	(29,989) : 639,	(30,989) : 568,	(31,989) : 504,	(32,989) : 535,	(33,989) : 655,	(34,989) : 471,	(35,989) : 628,	(36,989) : 238,	(37,989) : 122,	(38,989) : 519,	(39,989) : 533,	(40,989) : 225,
(1,990) : 277,	(2,990) : 303,	(3,990) : 472,	(4,990) : 380,	(5,990) : 307,	(6,990) : 262,	(7,990) : 268,	(8,990) : 755,	(9,990) : 381,	(10,990) : 270,	(11,990) : 304,	(12,990) : 429,	(13,990) : 146,	(14,990) : 377,	(15,990) : 232,	(16,990) : 463,	(17,990) : 331,	(18,990) : 288,	(19,990) : 378,	(20,990) : 271,	(21,990) : 83,	(22,990) : 248,	(23,990) : 282,	(24,990) : 333,	(25,990) : 681,	(26,990) : 138,	(27,990) : 141,	(28,990) : 269,	(29,990) : 219,	(30,990) : 274,	(31,990) : 269,	(32,990) : 189,	(33,990) : 271,	(34,990) : 245,	(35,990) : 408,	(36,990) : 443,	(37,990) : 408,	(38,990) : 297,	(39,990) : 302,	(40,990) : 703,
(1,991) : 247,	(2,991) : 213,	(3,991) : 456,	(4,991) : 667,	(5,991) : 167,	(6,991) : 172,	(7,991) : 238,	(8,991) : 674,	(9,991) : 116,	(10,991) : 240,	(11,991) : 214,	(12,991) : 686,	(13,991) : 307,	(14,991) : 111,	(15,991) : 517,	(16,991) : 747,	(17,991) : 617,	(18,991) : 158,	(19,991) : 627,	(20,991) : 244,	(21,991) : 367,	(22,991) : 92,	(23,991) : 158,	(24,991) : 619,	(25,991) : 533,	(26,991) : 357,	(27,991) : 309,	(28,991) : 242,	(29,991) : 501,	(30,991) : 247,	(31,991) : 182,	(32,991) : 232,	(33,991) : 346,	(34,991) : 150,	(35,991) : 308,	(36,991) : 169,	(37,991) : 204,	(38,991) : 196,	(39,991) : 210,	(40,991) : 546,
(1,992) : 496,	(2,992) : 535,	(3,992) : 579,	(4,992) : 130,	(5,992) : 551,	(6,992) : 502,	(7,992) : 489,	(8,992) : 867,	(9,992) : 625,	(10,992) : 490,	(11,992) : 536,	(12,992) : 299,	(13,992) : 326,	(14,992) : 621,	(15,992) : 72,	(16,992) : 234,	(17,992) : 86,	(18,992) : 532,	(19,992) : 284,	(20,992) : 490,	(21,992) : 184,	(22,992) : 501,	(23,992) : 526,	(24,992) : 87,	(25,992) : 846,	(26,992) : 268,	(27,992) : 320,	(28,992) : 489,	(29,992) : 45,	(30,992) : 493,	(31,992) : 507,	(32,992) : 411,	(33,992) : 437,	(34,992) : 490,	(35,992) : 626,	(36,992) : 688,	(37,992) : 630,	(38,992) : 533,	(39,992) : 535,	(40,992) : 872,
(1,993) : 271,	(2,993) : 324,	(3,993) : 693,	(4,993) : 355,	(5,993) : 361,	(6,993) : 314,	(7,993) : 270,	(8,993) : 976,	(9,993) : 547,	(10,993) : 270,	(11,993) : 325,	(12,993) : 261,	(13,993) : 127,	(14,993) : 542,	(15,993) : 215,	(16,993) : 381,	(17,993) : 310,	(18,993) : 347,	(19,993) : 198,	(20,993) : 268,	(21,993) : 174,	(22,993) : 356,	(23,993) : 342,	(24,993) : 313,	(25,993) : 896,	(26,993) : 86,	(27,993) : 125,	(28,993) : 268,	(29,993) : 282,	(30,993) : 268,	(31,993) : 313,	(32,993) : 216,	(33,993) : 175,	(34,993) : 316,	(35,993) : 384,	(36,993) : 603,	(37,993) : 602,	(38,993) : 330,	(39,993) : 326,	(40,993) : 917,
(1,994) : 360,	(2,994) : 315,	(3,994) : 456,	(4,994) : 788,	(5,994) : 263,	(6,994) : 285,	(7,994) : 352,	(8,994) : 622,	(9,994) : 42,	(10,994) : 354,	(11,994) : 316,	(12,994) : 818,	(13,994) : 438,	(14,994) : 43,	(15,994) : 643,	(16,994) : 873,	(17,994) : 739,	(18,994) : 262,	(19,994) : 759,	(20,994) : 358,	(21,994) : 493,	(22,994) : 215,	(23,994) : 264,	(24,994) : 741,	(25,994) : 456,	(26,994) : 489,	(27,994) : 441,	(28,994) : 356,	(29,994) : 618,	(30,994) : 361,	(31,994) : 293,	(32,994) : 360,	(33,994) : 468,	(34,994) : 269,	(35,994) : 387,	(36,994) : 39,	(37,994) : 145,	(38,994) : 299,	(39,994) : 312,	(40,994) : 462,
(1,995) : 384,	(2,995) : 339,	(3,995) : 449,	(4,995) : 804,	(5,995) : 287,	(6,995) : 309,	(7,995) : 376,	(8,995) : 606,	(9,995) : 50,	(10,995) : 378,	(11,995) : 340,	(12,995) : 839,	(13,995) : 460,	(14,995) : 54,	(15,995) : 660,	(16,995) : 891,	(17,995) : 756,	(18,995) : 286,	(19,995) : 781,	(20,995) : 382,	(21,995) : 512,	(22,995) : 239,	(23,995) : 288,	(24,995) : 758,	(25,995) : 436,	(26,995) : 511,	(27,995) : 463,	(28,995) : 380,	(29,995) : 634,	(30,995) : 385,	(31,995) : 317,	(32,995) : 384,	(33,995) : 492,	(34,995) : 293,	(35,995) : 409,	(36,995) : 15,	(37,995) : 134,	(38,995) : 322,	(39,995) : 335,	(40,995) : 442,
(1,996) : 641,	(2,996) : 690,	(3,996) : 789,	(4,996) : 98,	(5,996) : 717,	(6,996) : 668,	(7,996) : 638,	(8,996) : 1072,	(9,996) : 831,	(10,996) : 638,	(11,996) : 691,	(12,996) : 208,	(13,996) : 474,	(14,996) : 827,	(15,996) : 220,	(16,996) : 19,	(17,996) : 132,	(18,996) : 700,	(19,996) : 244,	(20,996) : 637,	(21,996) : 370,	(22,996) : 684,	(23,996) : 695,	(24,996) : 131,	(25,996) : 1062,	(26,996) : 410,	(27,996) : 469,	(28,996) : 636,	(29,996) : 263,	(30,996) : 638,	(31,996) : 670,	(32,996) : 569,	(33,996) : 553,	(34,996) : 661,	(35,996) : 761,	(36,996) : 893,	(37,996) : 844,	(38,996) : 692,	(39,996) : 691,	(40,996) : 1089,
(1,997) : 600,	(2,997) : 644,	(3,997) : 691,	(4,997) : 0,	(5,997) : 665,	(6,997) : 616,	(7,997) : 594,	(8,997) : 973,	(9,997) : 755,	(10,997) : 595,	(11,997) : 645,	(12,997) : 259,	(13,997) : 429,	(14,997) : 751,	(15,997) : 156,	(16,997) : 118,	(17,997) : 50,	(18,997) : 647,	(19,997) : 273,	(20,997) : 594,	(21,997) : 303,	(22,997) : 622,	(23,997) : 641,	(24,997) : 48,	(25,997) : 966,	(26,997) : 366,	(27,997) : 423,	(28,997) : 594,	(29,997) : 174,	(30,997) : 597,	(31,997) : 619,	(32,997) : 520,	(33,997) : 525,	(34,997) : 606,	(35,997) : 726,	(36,997) : 817,	(37,997) : 760,	(38,997) : 644,	(39,997) : 645,	(40,997) : 994,
(1,998) : 631,	(2,998) : 605,	(3,998) : 173,	(4,998) : 804,	(5,998) : 560,	(6,998) : 559,	(7,998) : 620,	(8,998) : 281,	(9,998) : 317,	(10,998) : 623,	(11,998) : 606,	(12,998) : 935,	(13,998) : 623,	(14,998) : 320,	(15,998) : 702,	(16,998) : 915,	(17,998) : 767,	(18,998) : 550,	(19,998) : 889,	(20,998) : 626,	(21,998) : 587,	(22,998) : 478,	(23,998) : 550,	(24,998) : 767,	(25,998) : 167,	(26,998) : 646,	(27,998) : 622,	(28,998) : 624,	(29,998) : 636,	(30,998) : 630,	(31,998) : 570,	(32,998) : 588,	(33,998) : 709,	(34,998) : 535,	(35,998) : 702,	(36,998) : 328,	(37,998) : 203,	(38,998) : 588,	(39,998) : 602,	(40,998) : 192,
(1,999) : 813,	(2,999) : 868,	(3,999) : 1097,	(4,999) : 414,	(5,999) : 908,	(6,999) : 862,	(7,999) : 813,	(8,999) : 1384,	(9,999) : 1080,	(10,999) : 813,	(11,999) : 868,	(12,999) : 288,	(13,999) : 673,	(14,999) : 1075,	(15,999) : 493,	(16,999) : 297,	(17,999) : 438,	(18,999) : 895,	(19,999) : 350,	(20,999) : 810,	(21,999) : 624,	(22,999) : 902,	(23,999) : 890,	(24,999) : 438,	(25,999) : 1361,	(26,999) : 616,	(27,999) : 669,	(28,999) : 811,	(29,999) : 562,	(30,999) : 810,	(31,999) : 860,	(32,999) : 764,	(33,999) : 701,	(34,999) : 864,	(35,999) : 905,	(36,999) : 1139,	(37,999) : 1114,	(38,999) : 875,	(39,999) : 870,	(40,999) : 1387,
(1,1000) : 96,	(2,1000) : 152,	(3,1000) : 686,	(4,1000) : 525,	(5,1000) : 197,	(6,1000) : 159,	(7,1000) : 98,	(8,1000) : 949,	(9,1000) : 433,	(10,1000) : 97,	(11,1000) : 152,	(12,1000) : 434,	(13,1000) : 112,	(14,1000) : 428,	(15,1000) : 373,	(16,1000) : 561,	(17,1000) : 477,	(18,1000) : 189,	(19,1000) : 370,	(20,1000) : 94,	(21,1000) : 265,	(22,1000) : 227,	(23,1000) : 185,	(24,1000) : 480,	(25,1000) : 835,	(26,1000) : 164,	(27,1000) : 118,	(28,1000) : 95,	(29,1000) : 416,	(30,1000) : 94,	(31,1000) : 153,	(32,1000) : 93,	(33,1000) : 28,	(34,1000) : 172,	(35,1000) : 203,	(36,1000) : 480,	(37,1000) : 514,	(38,1000) : 162,	(39,1000) : 154,	(40,1000) : 852,
(1,1001) : 308,	(2,1001) : 344,	(3,1001) : 529,	(4,1001) : 311,	(5,1001) : 358,	(6,1001) : 310,	(7,1001) : 300,	(8,1001) : 816,	(9,1001) : 459,	(10,1001) : 302,	(11,1001) : 345,	(12,1001) : 352,	(13,1001) : 145,	(14,1001) : 454,	(15,1001) : 157,	(16,1001) : 387,	(17,1001) : 261,	(18,1001) : 340,	(19,1001) : 304,	(20,1001) : 302,	(21,1001) : 9,	(22,1001) : 312,	(23,1001) : 334,	(24,1001) : 263,	(25,1001) : 753,	(26,1001) : 104,	(27,1001) : 139,	(28,1001) : 300,	(29,1001) : 166,	(30,1001) : 305,	(31,1001) : 315,	(32,1001) : 221,	(33,1001) : 269,	(34,1001) : 298,	(35,1001) : 439,	(36,1001) : 520,	(37,1001) : 486,	(38,1001) : 341,	(39,1001) : 344,	(40,1001) : 776,
(1,1002) : 680,	(2,1002) : 650,	(3,1002) : 241,	(4,1002) : 884,	(5,1002) : 602,	(6,1002) : 606,	(7,1002) : 670,	(8,1002) : 261,	(9,1002) : 344,	(10,1002) : 673,	(11,1002) : 651,	(12,1002) : 1009,	(13,1002) : 687,	(14,1002) : 348,	(15,1002) : 779,	(16,1002) : 994,	(17,1002) : 845,	(18,1002) : 594,	(19,1002) : 962,	(20,1002) : 676,	(21,1002) : 660,	(22,1002) : 525,	(23,1002) : 595,	(24,1002) : 846,	(25,1002) : 99,	(26,1002) : 713,	(27,1002) : 687,	(28,1002) : 674,	(29,1002) : 715,	(30,1002) : 680,	(31,1002) : 617,	(32,1002) : 645,	(33,1002) : 765,	(34,1002) : 583,	(35,1002) : 741,	(36,1002) : 341,	(37,1002) : 234,	(38,1002) : 632,	(39,1002) : 646,	(40,1002) : 118,
(1,1003) : 681,	(2,1003) : 656,	(3,1003) : 165,	(4,1003) : 829,	(5,1003) : 612,	(6,1003) : 609,	(7,1003) : 670,	(8,1003) : 229,	(9,1003) : 369,	(10,1003) : 673,	(11,1003) : 657,	(12,1003) : 969,	(13,1003) : 668,	(14,1003) : 371,	(15,1003) : 733,	(16,1003) : 941,	(17,1003) : 793,	(18,1003) : 601,	(19,1003) : 926,	(20,1003) : 676,	(21,1003) : 624,	(22,1003) : 529,	(23,1003) : 601,	(24,1003) : 793,	(25,1003) : 138,	(26,1003) : 688,	(27,1003) : 667,	(28,1003) : 674,	(29,1003) : 664,	(30,1003) : 680,	(31,1003) : 621,	(32,1003) : 636,	(33,1003) : 756,	(34,1003) : 585,	(35,1003) : 754,	(36,1003) : 378,	(37,1003) : 255,	(38,1003) : 639,	(39,1003) : 652,	(40,1003) : 166,
(1,1004) : 413,	(2,1004) : 458,	(3,1004) : 623,	(4,1004) : 187,	(5,1004) : 480,	(6,1004) : 431,	(7,1004) : 407,	(8,1004) : 913,	(9,1004) : 597,	(10,1004) : 408,	(11,1004) : 459,	(12,1004) : 235,	(13,1004) : 242,	(14,1004) : 592,	(15,1004) : 36,	(16,1004) : 249,	(17,1004) : 138,	(18,1004) : 463,	(19,1004) : 202,	(20,1004) : 407,	(21,1004) : 132,	(22,1004) : 444,	(23,1004) : 457,	(24,1004) : 141,	(25,1004) : 870,	(26,1004) : 179,	(27,1004) : 236,	(28,1004) : 407,	(29,1004) : 118,	(30,1004) : 410,	(31,1004) : 434,	(32,1004) : 334,	(33,1004) : 341,	(34,1004) : 423,	(35,1004) : 539,	(36,1004) : 658,	(37,1004) : 620,	(38,1004) : 458,	(39,1004) : 458,	(40,1004) : 895,
(1,1005) : 691,	(2,1005) : 660,	(3,1005) : 246,	(4,1005) : 892,	(5,1005) : 612,	(6,1005) : 616,	(7,1005) : 680,	(8,1005) : 254,	(9,1005) : 354,	(10,1005) : 683,	(11,1005) : 661,	(12,1005) : 1019,	(13,1005) : 697,	(14,1005) : 357,	(15,1005) : 788,	(16,1005) : 1003,	(17,1005) : 854,	(18,1005) : 604,	(19,1005) : 972,	(20,1005) : 686,	(21,1005) : 670,	(22,1005) : 535,	(23,1005) : 604,	(24,1005) : 855,	(25,1005) : 88,	(26,1005) : 724,	(27,1005) : 697,	(28,1005) : 684,	(29,1005) : 724,	(30,1005) : 690,	(31,1005) : 627,	(32,1005) : 655,	(33,1005) : 776,	(34,1005) : 593,	(35,1005) : 750,	(36,1005) : 349,	(37,1005) : 244,	(38,1005) : 642,	(39,1005) : 656,	(40,1005) : 108,
(1,1006) : 363,	(2,1006) : 409,	(3,1006) : 627,	(4,1006) : 238,	(5,1006) : 434,	(6,1006) : 385,	(7,1006) : 358,	(8,1006) : 917,	(9,1006) : 566,	(10,1006) : 359,	(11,1006) : 410,	(12,1006) : 241,	(13,1006) : 193,	(14,1006) : 561,	(15,1006) : 88,	(16,1006) : 290,	(17,1006) : 190,	(18,1006) : 417,	(19,1006) : 196,	(20,1006) : 358,	(21,1006) : 109,	(22,1006) : 404,	(23,1006) : 412,	(24,1006) : 192,	(25,1006) : 862,	(26,1006) : 129,	(27,1006) : 187,	(28,1006) : 357,	(29,1006) : 157,	(30,1006) : 360,	(31,1006) : 387,	(32,1006) : 287,	(33,1006) : 289,	(34,1006) : 378,	(35,1006) : 488,	(36,1006) : 626,	(37,1006) : 598,	(38,1006) : 410,	(39,1006) : 410,	(40,1006) : 886,
(1,1007) : 242,	(2,1007) : 289,	(3,1007) : 612,	(4,1007) : 358,	(5,1007) : 316,	(6,1007) : 267,	(7,1007) : 238,	(8,1007) : 895,	(9,1007) : 475,	(10,1007) : 239,	(11,1007) : 290,	(12,1007) : 321,	(13,1007) : 73,	(14,1007) : 470,	(15,1007) : 203,	(16,1007) : 407,	(17,1007) : 309,	(18,1007) : 300,	(19,1007) : 263,	(20,1007) : 237,	(21,1007) : 108,	(22,1007) : 295,	(23,1007) : 294,	(24,1007) : 311,	(25,1007) : 815,	(26,1007) : 9,	(27,1007) : 67,	(28,1007) : 237,	(29,1007) : 247,	(30,1007) : 239,	(31,1007) : 269,	(32,1007) : 168,	(33,1007) : 178,	(34,1007) : 263,	(35,1007) : 369,	(36,1007) : 533,	(37,1007) : 524,	(38,1007) : 291,	(39,1007) : 290,	(40,1007) : 836,
(1,1008) : 533,	(2,1008) : 577,	(3,1008) : 652,	(4,1008) : 68,	(5,1008) : 597,	(6,1008) : 548,	(7,1008) : 527,	(8,1008) : 938,	(9,1008) : 691,	(10,1008) : 528,	(11,1008) : 578,	(12,1008) : 244,	(13,1008) : 362,	(14,1008) : 687,	(15,1008) : 89,	(16,1008) : 160,	(17,1008) : 18,	(18,1008) : 579,	(19,1008) : 243,	(20,1008) : 528,	(21,1008) : 236,	(22,1008) : 555,	(23,1008) : 574,	(24,1008) : 20,	(25,1008) : 920,	(26,1008) : 299,	(27,1008) : 356,	(28,1008) : 527,	(29,1008) : 119,	(30,1008) : 530,	(31,1008) : 552,	(32,1008) : 453,	(33,1008) : 461,	(34,1008) : 539,	(35,1008) : 660,	(36,1008) : 753,	(37,1008) : 701,	(38,1008) : 576,	(39,1008) : 577,	(40,1008) : 946,
(1,1009) : 411,	(2,1009) : 374,	(3,1009) : 365,	(4,1009) : 763,	(5,1009) : 324,	(6,1009) : 336,	(7,1009) : 402,	(8,1009) : 534,	(9,1009) : 62,	(10,1009) : 404,	(11,1009) : 375,	(12,1009) : 823,	(13,1009) : 457,	(14,1009) : 66,	(15,1009) : 627,	(16,1009) : 857,	(17,1009) : 717,	(18,1009) : 319,	(19,1009) : 767,	(20,1009) : 408,	(21,1009) : 483,	(22,1009) : 257,	(23,1009) : 320,	(24,1009) : 718,	(25,1009) : 376,	(26,1009) : 500,	(27,1009) : 458,	(28,1009) : 406,	(29,1009) : 590,	(30,1009) : 411,	(31,1009) : 345,	(32,1009) : 393,	(33,1009) : 509,	(34,1009) : 315,	(35,1009) : 459,	(36,1009) : 85,	(37,1009) : 52,	(38,1009) : 357,	(39,1009) : 371,	(40,1009) : 386,
(1,1010) : 652,	(2,1010) : 701,	(3,1010) : 801,	(4,1010) : 111,	(5,1010) : 728,	(6,1010) : 679,	(7,1010) : 648,	(8,1010) : 1085,	(9,1010) : 844,	(10,1010) : 649,	(11,1010) : 702,	(12,1010) : 209,	(13,1010) : 485,	(14,1010) : 840,	(15,1010) : 232,	(16,1010) : 12,	(17,1010) : 145,	(18,1010) : 712,	(19,1010) : 248,	(20,1010) : 648,	(21,1010) : 382,	(22,1010) : 697,	(23,1010) : 706,	(24,1010) : 144,	(25,1010) : 1075,	(26,1010) : 421,	(27,1010) : 480,	(28,1010) : 647,	(29,1010) : 277,	(30,1010) : 649,	(31,1010) : 681,	(32,1010) : 580,	(33,1010) : 563,	(34,1010) : 673,	(35,1010) : 771,	(36,1010) : 906,	(37,1010) : 858,	(38,1010) : 703,	(39,1010) : 702,	(40,1010) : 1102,
(1,1011) : 430,	(2,1011) : 477,	(3,1011) : 655,	(4,1011) : 174,	(5,1011) : 502,	(6,1011) : 453,	(7,1011) : 425,	(8,1011) : 945,	(9,1011) : 626,	(10,1011) : 426,	(11,1011) : 478,	(12,1011) : 204,	(13,1011) : 260,	(14,1011) : 622,	(15,1011) : 49,	(16,1011) : 223,	(17,1011) : 128,	(18,1011) : 485,	(19,1011) : 174,	(20,1011) : 425,	(21,1011) : 163,	(22,1011) : 470,	(23,1011) : 479,	(24,1011) : 131,	(25,1011) : 902,	(26,1011) : 196,	(27,1011) : 255,	(28,1011) : 424,	(29,1011) : 139,	(30,1011) : 427,	(31,1011) : 455,	(32,1011) : 355,	(33,1011) : 351,	(34,1011) : 446,	(35,1011) : 554,	(36,1011) : 688,	(37,1011) : 652,	(38,1011) : 478,	(39,1011) : 477,	(40,1011) : 927,
(1,1012) : 891,	(2,1012) : 856,	(3,1012) : 416,	(4,1012) : 1095,	(5,1012) : 806,	(6,1012) : 816,	(7,1012) : 882,	(8,1012) : 232,	(9,1012) : 540,	(10,1012) : 884,	(11,1012) : 857,	(12,1012) : 1231,	(13,1012) : 909,	(14,1012) : 545,	(15,1012) : 998,	(16,1012) : 1207,	(17,1012) : 1059,	(18,1012) : 800,	(19,1012) : 1185,	(20,1012) : 888,	(21,1012) : 882,	(22,1012) : 736,	(23,1012) : 801,	(24,1012) : 1060,	(25,1012) : 129,	(26,1012) : 936,	(27,1012) : 908,	(28,1012) : 886,	(29,1012) : 930,	(30,1012) : 891,	(31,1012) : 826,	(32,1012) : 862,	(33,1012) : 982,	(34,1012) : 794,	(35,1012) : 938,	(36,1012) : 519,	(37,1012) : 441,	(38,1012) : 839,	(39,1012) : 852,	(40,1012) : 105,
(1,1013) : 405,	(2,1013) : 438,	(3,1013) : 490,	(4,1013) : 234,	(5,1013) : 447,	(6,1013) : 401,	(7,1013) : 397,	(8,1013) : 774,	(9,1013) : 507,	(10,1013) : 398,	(11,1013) : 439,	(12,1013) : 352,	(13,1013) : 247,	(14,1013) : 503,	(15,1013) : 114,	(16,1013) : 329,	(17,1013) : 188,	(18,1013) : 428,	(19,1013) : 319,	(20,1013) : 399,	(21,1013) : 96,	(22,1013) : 391,	(23,1013) : 423,	(24,1013) : 190,	(25,1013) : 738,	(26,1013) : 201,	(27,1013) : 241,	(28,1013) : 397,	(29,1013) : 67,	(30,1013) : 402,	(31,1013) : 407,	(32,1013) : 319,	(33,1013) : 367,	(34,1013) : 386,	(35,1013) : 534,	(36,1013) : 568,	(37,1013) : 513,	(38,1013) : 433,	(39,1013) : 437,	(40,1013) : 763,
(1,1014) : 573,	(2,1014) : 619,	(3,1014) : 706,	(4,1014) : 45,	(5,1014) : 642,	(6,1014) : 594,	(7,1014) : 568,	(8,1014) : 986,	(9,1014) : 746,	(10,1014) : 569,	(11,1014) : 619,	(12,1014) : 209,	(13,1014) : 407,	(14,1014) : 742,	(15,1014) : 148,	(16,1014) : 86,	(17,1014) : 60,	(18,1014) : 625,	(19,1014) : 226,	(20,1014) : 568,	(21,1014) : 295,	(22,1014) : 606,	(23,1014) : 620,	(24,1014) : 59,	(25,1014) : 971,	(26,1014) : 344,	(27,1014) : 401,	(28,1014) : 567,	(29,1014) : 188,	(30,1014) : 570,	(31,1014) : 597,	(32,1014) : 498,	(33,1014) : 493,	(34,1014) : 586,	(35,1014) : 693,	(36,1014) : 807,	(37,1014) : 757,	(38,1014) : 619,	(39,1014) : 619,	(40,1014) : 998,
(1,1015) : 623,	(2,1015) : 669,	(3,1015) : 735,	(4,1015) : 58,	(5,1015) : 692,	(6,1015) : 644,	(7,1015) : 618,	(8,1015) : 1012,	(9,1015) : 792,	(10,1015) : 619,	(11,1015) : 669,	(12,1015) : 233,	(13,1015) : 457,	(14,1015) : 788,	(15,1015) : 196,	(16,1015) : 62,	(17,1015) : 99,	(18,1015) : 675,	(19,1015) : 260,	(20,1015) : 618,	(21,1015) : 343,	(22,1015) : 655,	(23,1015) : 670,	(24,1015) : 98,	(25,1015) : 1005,	(26,1015) : 394,	(27,1015) : 451,	(28,1015) : 617,	(29,1015) : 227,	(30,1015) : 620,	(31,1015) : 647,	(32,1015) : 548,	(33,1015) : 542,	(34,1015) : 636,	(35,1015) : 743,	(36,1015) : 853,	(37,1015) : 800,	(38,1015) : 669,	(39,1015) : 669,	(40,1015) : 1032,
(1,1016) : 599,	(2,1016) : 645,	(3,1016) : 721,	(4,1016) : 46,	(5,1016) : 668,	(6,1016) : 620,	(7,1016) : 595,	(8,1016) : 999,	(9,1016) : 770,	(10,1016) : 596,	(11,1016) : 646,	(12,1016) : 221,	(13,1016) : 433,	(14,1016) : 766,	(15,1016) : 173,	(16,1016) : 70,	(17,1016) : 79,	(18,1016) : 652,	(19,1016) : 243,	(20,1016) : 594,	(21,1016) : 321,	(22,1016) : 632,	(23,1016) : 646,	(24,1016) : 78,	(25,1016) : 989,	(26,1016) : 370,	(27,1016) : 427,	(28,1016) : 594,	(29,1016) : 208,	(30,1016) : 596,	(31,1016) : 623,	(32,1016) : 525,	(33,1016) : 519,	(34,1016) : 613,	(35,1016) : 720,	(36,1016) : 831,	(37,1016) : 780,	(38,1016) : 646,	(39,1016) : 646,	(40,1016) : 1016,

    }
    return warehouses, customers, customer_demands, distance



## In this function, we are testing the input paramaters that is correct or not

In [10]:

def test_input(warehouses, customers, customer_demands, distance, distance_band, wh_status):
    for c in customers:
        for w in warehouses:
            if(distance[w,c]>=0):
                pass
            else:
                file.write(f'\nDistance between warehouse {w} and customer {c} is not available or invalid')
    
    wh_counter = 0
    for w in wh_status:
        if wh_status[w][1] == 1:
            if wh_status[w][2]==1:
                wh_counter = wh_counter+1
            else:
                print("In wh_status, if the first value is 1, the second value can't be 0.  The problem is WH: ", w)
                file.write(f'\nwh_status for warehouse {w} cannot have a 1 and 0 for status')
    if wh_counter <= number_of_whs:
        pass
    else: 
        print("Your fixed number of warehouses in wh_status is larger than the number of warehouses.  You need to fix and re-run")
        file.write(f'\nYour fixed number of warehouses in wh_status is larger than the number of warehouses.  You need to fix and re-run')                   
        


    for c in customers.keys():
        if(customer_demands[c]>=0):
            pass
        else:
            file.write(f'\nDemand for Customer {c} is not available')
        
    
    if ((distance_band[0] < distance_band[1]) and (distance_band[1]  < distance_band[2]) and (distance_band[2]  < distance_band[3]) ):
        pass
    else:
        file.write(f'\nDistance bands are not in ascending order. You must fix and re-run')
        
    if isinstance(number_of_whs, int)==False:
        file.write(f'\nnumber_of_whs = {number_of_whs} is not integer.  You must fix and re-run')
              
    
        

# Create Map for Potantial Warehouse and Customers

In [11]:
# In this function, we are visualizaing the input data-- the data before the optimization has run
def input_visual(warehouses, customers,supplier):
    warehouse_list = []
    for w in warehouses.keys():
        wh = {
                    'text':'Warehouse-'+warehouses[w][1],
                    'State':warehouses[w][2],
                    'ZipCode':warehouses[w][3],
                    'lat':warehouses[w][4],
                    'long':warehouses[w][5],
                    'cnt':10000000,
                    'size' : 20,
                    'color' : 'rgba(0, 100, 0)'
                    }
        warehouse_list.append(wh)    
    # map location for customer
    customer_list =[] 
    for c in customers.keys():
        cust = {
            'text':'Customer-'+customers[c][0],
            'State':customers[c][2],
            'ZipCode':customers[c][3],
            'lat':customers[c][4],
            'long':customers[c][5] ,
            'cnt':customer_demands[c],
            'size' : 3,
            'color' : 'rgb(255, 0, 0)'
        }
        customer_list.append(cust)     
    supplier_list = []
    for s in supplier.keys():
        supp = {
            'text' : 'Supplier-'+supplier[s][0],
            'lat' : supplier[s][2],
            'long' : supplier[s][3],
            'cnt':1,
            'size' : 20,
            'color' : 'rgb(150,200,100)'
        }
        supplier_list.append(supp)

    df = pd.DataFrame.from_records(warehouse_list)
    df['shape'] =  "triangle-down"
    df_cust = pd.DataFrame.from_records(customer_list)
    df_cust['shape'] =  "circle"
    df = df.append(df_cust, ignore_index = True) 
    df_supp = pd.DataFrame.from_records(supplier_list)
    df_supp['shape'] =  "triangle-up"
    df = df.append(df_supp, ignore_index = True) 
    
    locations = [ dict(
            type = 'scattergeo',
            locationmode = 'USA-states',
            lon = df['long'],
            lat = df['lat'],
            hoverinfo = 'text',
            text = df['text'],
            mode = 'markers',
            marker = dict( 
                size=df['size'], 
                color=df['color'],
                symbol = df['shape'],
                line = dict(
                    width=3,
                    color='rgba(68, 68, 68, 0)'
                ),

            ))]

    layout = dict(
            title = 'location of potential warehouse and customers',
            showlegend = False, 
            geo = dict(
                scope='asia',
                projection=dict( type='miller' ),
                showland = True,
                landcolor = 'rgb(243, 243, 243)',
                countrycolor = 'rgb(204, 204, 204)',
            ),
        )
    py.offline.plot({ "data":locations, "layout":layout}, filename = scenario_name+'_input.html')  

    

# Generate map for Optimal Location of Warehouse and its connectivity

In [12]:
# In this function, we are visualizaing the output data-- the data after the optimization has run
def output_visual(warehouses, customers, df_cu, wh_loc, scenario_name,supplier,df_su ):   
    
    warehouse_list = []
    for w in wh_loc:
            wh = {
                        'text':'Warehouse-'+warehouses[w][1],
                        'State':warehouses[w][2],
                        'ZipCode':warehouses[w][3],
                        'lat':warehouses[w][4],
                        'long':warehouses[w][5],
                        'cnt':10000000,
                        'size' : 30,
                        'color' : 'rgba(0, 100, 0)'
                        }
            warehouse_list.append(wh)    
            
    customer_list =[] 
    for c in customers.keys():
        cust = {
            'text':'Customer-'+customers[c][0],
            'State':customers[c][2],
            'ZipCode':customers[c][3],
            'lat':customers[c][4],
            'long':customers[c][5] ,
            'cnt':customer_demands[c],
            'size' : 3,
            'color' : 'rgb(255, 0, 0)'
        }
        customer_list.append(cust)
        
    supplier_list = []
    for s in supplier.keys():
        supp = {
            'text' : 'Supplier-'+supplier[s][0],
            'lat' : supplier[s][2],
            'long' : supplier[s][3],
            'cnt':1000,
            'size' : 30,
            'color' : 'rgb(100,200,50)'
        }
        supplier_list.append(supp)
        
    df = pd.DataFrame.from_records(warehouse_list)
    df['shape'] =  "triangle-down"
    df_cust = pd.DataFrame.from_records(customer_list)
    df_cust['shape'] =  "circle"
    df = df.append(df_cust, ignore_index = True) 
    df_supp = pd.DataFrame.from_records(supplier_list)
    df_supp['shape'] =  "triangle-up"
    df = df.append(df_supp, ignore_index = True) 

    paths = []        
    for i in range( len( df_cu) ):
            paths.append(
                dict(
                    type = 'scattergeo',
                    locationmode = 'USA-states',
                    lon = [ df_cu['Warehouse Longitude'][i], df_cu['Customers Longitude'][i] ],
                    lat = [ df_cu['Warehouse Latitude'][i], df_cu['Customers Latitude'][i] ],
                    mode = 'lines',
                    line = dict(
                        width = 1,
                        color = 'red',
                    ),
                    opacity = 0.8,
                )
            )  
    
    for j in range( len( df_su) ):
            paths.append(
                dict(
                    type = 'scattergeo',
                    locationmode = 'USA-states',
                    lon = [ df_su['Warehouse Longitude'][j], df_su['Supplier Longitude'][j] ],
                    lat = [ df_su['Warehouse Latitude'][j], df_su['Supplier Latitude'][j] ],
                    mode = 'lines',
                    line = dict(
                        width = 1,
                        color = 'blue',
                    ),
                    opacity = 0.8,
                )
            )  
    

    locations = [ dict(
            type = 'scattergeo',
            locationmode = 'USA-states',
            lon = df['long'],
            lat = df['lat'],
            hoverinfo = 'text',
            text = df['text'],
            mode = 'markers',
            marker = dict( 
                size=df['size'], 
                color=df['color'],
                symbol = df['shape'],
                line = dict(
                    width=3,
                    color='rgba(68, 68, 68, 0)'
                ),
            ))]

    layout = dict(
            title = 'Optimal location of warehouses',
            showlegend = False, 
            geo = dict(
                scope='asia',
                projection=dict( type='miller' ),
                showland = True,
                landcolor = 'rgb(243, 243, 243)',
                countrycolor = 'rgb(204, 204, 204)',
            ),
        )

    py.offline.plot({"data":locations+paths, "layout":layout},filename=scenario_name+'_output.html')         

## This is the block of code that runs all the functions (including the optimization)

In [13]:
#getting input data
warehouses, customers, customer_demands, distance = get_data()

#print("loaded input data")
distance_band = [distance_band_1, distance_band_2, distance_band_3, distance_band_4]

#opening text file in write mode
file = open(scenario_name+'details about our analysis' + '.txt',"w")
file.write('loaded input data')
print("loaded input data")  #Also print to Jupyter

#testing input data
file.write('\nTesting Input Data')
test_input(warehouses, customers, customer_demands, distance, distance_band,wh_status)

#RunCode = ()  #We need some function in here that runs the whole thing.
file.write('\nBuilding the model')
df_assign_vars, list_facility_vars, df_supply_var = optimal_location(number_of_whs, warehouses, customers, customer_demands, distance, distance_band, scenario_name,supplier,dist_supply,demand_supp)

#printing the maps
if input_map == True:
    input_visual(warehouses, customers,supplier)

if output_map == True:
    output_visual(warehouses, customers, df_assign_vars, list_facility_vars, scenario_name,supplier,df_supply_var)

#closing the file
file.close()

loaded input data


c:\users\ssshi\appdata\local\programs\python\python38-32\lib\site-packages\pulp\apis\coin_api.py:71: UserWarning: Parameter fracGap is being depreciated for gapRel
  warnings.warn("Parameter fracGap is being depreciated for gapRel")
c:\users\ssshi\appdata\local\programs\python\python38-32\lib\site-packages\pulp\apis\coin_api.py:77: UserWarning: Parameter maxSeconds is being depreciated for timeLimit
  warnings.warn("Parameter maxSeconds is being depreciated for timeLimit")


Optimization Status Optimal
Weighted Average Distance: 112.83 km
Run Time in seconds 44.2
